In [2]:
# Import packages
from __future__ import print_function
import cachetools.func
import re
import pickle
# from dash import html, dash_table, dcc, Output, Input, Dash, State, callback

# from umap import UMAP
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
import plotly.express as px
import plotly.graph_objs as go

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import time
import numpy as np
import pandas as pd
import seaborn as sns


%matplotlib inline

In [3]:

# import vegi_esc_api.logger as logger
# from vegi_esc_api.sustained import SustainedAPI
# from vegi_esc_api.word_vec_model import getModel

ModuleNotFoundError: No module named 'vegi_esc_api'

In [4]:
from typing import Any
from gensim import models
import argparse
import os
# import gensim.models.keyedvectors as word2vec


def getModel(args: argparse.Namespace | None = None):
    model: Any
    models_path = "./models"
    # model_abs_dir = os.path.abspath(models_path)
    if os.path.exists(models_path) is False:
        os.mkdir(path=models_path)
    model_name = "word2vec-google-news-300"
    # model_name = "glove-twitter-25"
    # model_path = f"{models_path}/GoogleNews-vectors-negative300"
    model_path = f"{models_path}/{model_name}"
    model_abs_path = os.path.abspath(model_path)
    if args and args.model:
        display("Loading model from args path...")
        mp = args.model
        if mp.endswith(".bin"):
            binary = True if args.binary else mp.endswith(".bin") if mp else False
            binary_mode = "BINARY_MODE" if binary else "NON_BINARY_MODE"
            display(f'Running "{mp}" in {binary_mode}')
            model = models.KeyedVectors.load_word2vec_format(mp, binary=binary)
            model.save(model_path)
        else:
            model = models.KeyedVectors.load(mp)
    elif os.path.exists(f"{model_path}"):
        display(f'Loading model from saved location: "{model_abs_path}"')
        model = models.KeyedVectors.load(model_path)
    elif os.path.exists(f"{model_path}.bin"):
        mp = f"{model_path}.bin"
        display(f'Loading model from saved location: "{mp}"')
        binary = mp.endswith(".bin") if model_path else False
        binary_mode = "BINARY_MODE" if binary else "NON_BINARY_MODE"
        display(f'Running "{mp}" in {binary_mode}')
        model = models.KeyedVectors.load_word2vec_format(mp, binary=binary)
        model.save(model_path)
    else:
        if os.path.exists(models_path) is False:
            os.mkdir(path=models_path)
        display(f"Loading model from gensim downloader as doesn't exist in {models_path}...")
        import gensim.downloader as api

        model = api.load(model_name)
        # ~ https://stackoverflow.com/a/59912447
        # model.wv.save_word2vec_format(f"{model_path}.bin", binary=True)
        display(f"Saving model downloaded from gensim downloader to {model_path}...")
        model.save(model_path)

    # ~ https://stackoverflow.com/a/43067907
    # model_save_path = "./models/GoogleNews-vectors-negative300.model"
    # if not os.path.exists(model_save_path):
    #     model.save()
    return model


In [5]:
model = getModel()

"Loading model from gensim downloader as doesn't exist in ./models..."

'Saving model downloaded from gensim downloader to ./models/word2vec-google-news-300...'

In [11]:
def append_list(sim_words, words):
    list_of_words = []

    for i in range(len(sim_words)):
        sim_words_list = list(sim_words[i])
        sim_words_list.append(words)
        sim_words_tuple = tuple(sim_words_list)
        list_of_words.append(sim_words_tuple)

    return list_of_words

In [40]:
# Import and download stopwords from NLTK.
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.
stop_words = stopwords.words('english')

def preprocess(sentence):
    return [w for w in sentence.lower().split() if w not in stop_words]

sentence_cat = preprocess("Baby Foods & Drinks")
sentence_product = preprocess("Raw hot kimchi sauce")

[nltk_data] Downloading package stopwords to /Users/joey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
import gensim.downloader as api
from gensim.models import fasttext
import numpy as np
# ~ https://fasttext.cc/docs/en/crawl-vectors.html
import fasttext
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')
display(ft.get_dimension())
display(ft.get_word_vector('hello').shape(100,))
display(ft.get_nearest_neighbors('hello'))
ft.save_model('models/cc.en.100.bin')

ValueError: cc.en.300.bin cannot be opened for loading!

In [ ]:
# wv = fasttext.load_facebook_vectors("transtotag/cc.da.300.bin")
# For .bin use: load_fasttext_format() (this typically contains full model with parameters, ngrams, etc).

# For .vec use: load_word2vec_format (this contains ONLY word-vectors -> no ngrams + you can't update an model).

# Note:: If you are facing issues with the memory or you are not able to load .bin models, then check the pyfasttext model for the same.

# Credits : Ivan Menshikh (Gensim Maintainer)
# Update 04/2020

# load_fasttext_format() is now deprecated, the updated way is to load the models is with gensim.models.fasttext.load_facebook_model() or gensim.models.fasttext.load_facebook_vectors() for binaries and vecs respectively.



In [46]:
distance = model.wmdistance(sentence_cat, sentence_product)
display(distance)
# ~ https://stackoverflow.com/a/75762491
sentence_vec_cat = model.get_sentence_vector(sentence_cat)

1.2607337983527989

AttributeError: 'KeyedVectors' object has no attribute 'get_sentence_vector'

In [44]:
def get_word_vectors(
    words=None,
    sample=10,
):
    if words is None:
        if sample > 0:
            words = np.random.choice(list(model.key_to_index.keys()), sample)
        else:
            words = [word for word in model.vocab]

    vocab = model.key_to_index.keys()
    if all((phrase in vocab for phrase in words)):
        word_vectors = np.array([model[w] for w in words])
        return word_vectors
    else:
        with open('tfidf_vectorizer.pk', 'rb') as fin:
            tfidf_vectorizer = pickle.load(fin)
        tfidf_weighted_phrase_vects = [
            (
                (1 / tfidf_vectorizer.transform([phrase])[0].T.todense())
                * np.sum([w for w in re.split(r'\s', phrase)])
            )
            for phrase in words]
        phrase_vectors = tfidf_weighted_phrase_vects
        return phrase_vectors

In [7]:
def display_scatterplot_2D(
    model,
    user_input=None,
    words=None,
    labels=None,
    color_map=None,
    annotation="On",
    dim_red="PCA",
    perplexity=0,
    learning_rate=0,
    iteration=0,
    topn=0,
    sample=10,
):
    '''
    Use the `model` to calculate word vector embeddings for each word in the `words` parameter which contains all words that we are comparing to the supplied `user_input`:list[str] which is a list of phrases that we want to visualise vs the embedded `words`.
    '''
    if user_input is None:
        user_input = ""

    word_vectors = get_word_vectors(words=words, sample=sample)

    if dim_red == "PCA":
        two_dim = PCA(random_state=0).fit_transform(word_vectors)[:, :2]
    else:
        two_dim = TSNE(
            random_state=0,
            perplexity=perplexity,
            learning_rate=learning_rate,
            n_iter=iteration,
        ).fit_transform(word_vectors)[:, :2]

    data = []
    count = 0
    for i in range(len(user_input)):
        trace = go.Scatter(
            x=two_dim[count : count + topn, 0],
            y=two_dim[count : count + topn, 1],
            text=words[count : count + topn] if annotation == "On" else "",
            name=user_input[i],
            textposition="top center",
            textfont_size=16,
            mode="markers+text",
            marker={"size": 15, "opacity": 0.8, "color": 2},
        )

        data.append(trace)
        count = count + topn

    trace_input = go.Scatter(
        x=two_dim[count:, 0],
        y=two_dim[count:, 1],
        text=words[count:],
        name="input words",
        textposition="top center",
        textfont_size=12,
        mode="markers+text",
        marker={"size": 25, "opacity": 1, "color": "black"},
    )

    data.append(trace_input)

    # Configure the layout.
    layout = go.Layout(
        margin={"l": 0, "r": 0, "b": 0, "t": 0},
        showlegend=True,
        hoverlabel=dict(bgcolor="white", font_size=20, font_family="Courier New"),
        legend=dict(
            x=1, y=0.5, font=dict(family="Courier New", size=25, color="black")
        ),
        font=dict(family=" Courier New ", size=14),
        autosize=False,
        width=1000,
        height=1000,
    )

    plot_figure = go.Figure(data=data, layout=layout)
    return plot_figure

In [16]:
sustained_categories_json = {
    "categories": [
        {
            "id": "baby-foods-and-drinks",
            "name": "Baby Foods & Drinks",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=baby-foods-and-drinks",
                "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks"
            }
        },
        {
            "id": "bacon-and-sausages",
            "name": "Bacon & Sausages",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=bacon-and-sausages",
                "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages"
            }
        },
        {
            "id": "beef",
            "name": "Beef",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=beef",
                "category": "https://api.sustained.com/choice/v1/categories/beef"
            }
        },
        {
            "id": "beer-and-cider",
            "name": "Beer & Cider",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=beer-and-cider",
                "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider"
            }
        },
        {
            "id": "biscuits",
            "name": "Biscuits",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=biscuits",
                "category": "https://api.sustained.com/choice/v1/categories/biscuits"
            }
        },
        {
            "id": "bread",
            "name": "Bread",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=bread",
                "category": "https://api.sustained.com/choice/v1/categories/bread"
            }
        },
        {
            "id": "breakfast-foods",
            "name": "Breakfast Foods",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=breakfast-foods",
                "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods"
            }
        },
        {
            "id": "butter-and-margarine",
            "name": "Butter & Margarine",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=butter-and-margarine",
                "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine"
            }
        },
        {
            "id": "cakes",
            "name": "Cakes",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=cakes",
                "category": "https://api.sustained.com/choice/v1/categories/cakes"
            }
        },
        {
            "id": "carbonated-drinks",
            "name": "Carbonated Drinks",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=carbonated-drinks",
                "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks"
            }
        },
        {
            "id": "cereals",
            "name": "Cereals",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=cereals",
                "category": "https://api.sustained.com/choice/v1/categories/cereals"
            }
        },
        {
            "id": "champagne-and-rose",
            "name": "Champagne & Rose",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=champagne-and-rose",
                "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose"
            }
        },
        {
            "id": "cheese",
            "name": "Cheese",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=cheese",
                "category": "https://api.sustained.com/choice/v1/categories/cheese"
            }
        },
        {
            "id": "chicken-and-turkey",
            "name": "Chicken & Turkey",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=chicken-and-turkey",
                "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey"
            }
        },
        {
            "id": "chocolate-and-sweets",
            "name": "Chocolate & Sweets",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=chocolate-and-sweets",
                "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets"
            }
        },
        {
            "id": "cooking-sauces",
            "name": "Cooking Sauces",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=cooking-sauces",
                "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces"
            }
        },
        {
            "id": "crisps-and-snacks",
            "name": "Crisps & Snacks",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=crisps-and-snacks",
                "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks"
            }
        },
        {
            "id": "deli",
            "name": "Deli",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=deli",
                "category": "https://api.sustained.com/choice/v1/categories/deli"
            }
        },
        {
            "id": "dried-fruits-and-nuts",
            "name": "Dried Fruits & Nuts",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=dried-fruits-and-nuts",
                "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts"
            }
        },
        {
            "id": "eggs",
            "name": "Eggs",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=eggs",
                "category": "https://api.sustained.com/choice/v1/categories/eggs"
            }
        },
        {
            "id": "fish-and-seafood",
            "name": "Fish & Seafood",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=fish-and-seafood",
                "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood"
            }
        },
        {
            "id": "frozen-foods",
            "name": "Frozen Foods",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=frozen-foods",
                "category": "https://api.sustained.com/choice/v1/categories/frozen-foods"
            }
        },
        {
            "id": "fruit-juice-and-drinks",
            "name": "Fruit Juice & Drinks",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=fruit-juice-and-drinks",
                "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks"
            }
        },
        {
            "id": "fruits",
            "name": "Fruits",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=fruits",
                "category": "https://api.sustained.com/choice/v1/categories/fruits"
            }
        },
        {
            "id": "game",
            "name": "Game",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=game",
                "category": "https://api.sustained.com/choice/v1/categories/game"
            }
        },
        {
            "id": "herbs",
            "name": "Herbs",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=herbs",
                "category": "https://api.sustained.com/choice/v1/categories/herbs"
            }
        },
        {
            "id": "home-baking-and-sugar",
            "name": "Home Baking & Sugar",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=home-baking-and-sugar",
                "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar"
            }
        },
        {
            "id": "hot-beverages",
            "name": "Hot Beverages",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=hot-beverages",
                "category": "https://api.sustained.com/choice/v1/categories/hot-beverages"
            }
        },
        {
            "id": "jams-and-spread",
            "name": "Jams & Spread",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=jams-and-spread",
                "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread"
            }
        },
        {
            "id": "lamb",
            "name": "Lamb",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=lamb",
                "category": "https://api.sustained.com/choice/v1/categories/lamb"
            }
        },
        {
            "id": "meat-free-alternatives",
            "name": "Meat Free Alternatives",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=meat-free-alternatives",
                "category": "https://api.sustained.com/choice/v1/categories/meat-free-alternatives"
            }
        },
        {
            "id": "milk-and-cream",
            "name": "Milk & Cream",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=milk-and-cream",
                "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream"
            }
        },
        {
            "id": "organic-produce",
            "name": "Organic Produce",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=organic-produce",
                "category": "https://api.sustained.com/choice/v1/categories/organic-produce"
            }
        },
        {
            "id": "other-alcoholic-beverages",
            "name": "Other Alcoholic Beverages",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=other-alcoholic-beverages",
                "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages"
            }
        },
        {
            "id": "other-fresh-meat-and-fish",
            "name": "Other Fresh Meat & Fish",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=other-fresh-meat-and-fish",
                "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish"
            }
        },
        {
            "id": "other-non-alcoholic-drinks",
            "name": "Other Non-Alcoholic Drinks",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=other-non-alcoholic-drinks",
                "category": "https://api.sustained.com/choice/v1/categories/other-non-alcoholic-drinks"
            }
        },
        {
            "id": "pork",
            "name": "Pork",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=pork",
                "category": "https://api.sustained.com/choice/v1/categories/pork"
            }
        },
        {
            "id": "ready-made-foods",
            "name": "Ready Made Foods",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=ready-made-foods",
                "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods"
            }
        },
        {
            "id": "rice-and-pasta",
            "name": "Rice & Pasta",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=rice-and-pasta",
                "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta"
            }
        },
        {
            "id": "soups",
            "name": "Soups",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=soups",
                "category": "https://api.sustained.com/choice/v1/categories/soups"
            }
        },
        {
            "id": "spirit-and-liqueurs",
            "name": "Spirit & Liqueurs",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=spirit-and-liqueurs",
                "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs"
            }
        },
        {
            "id": "table-sauces-and-condiments",
            "name": "Table Sauces & Condiments",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=table-sauces-and-condiments",
                "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments"
            }
        },
        {
            "id": "tins-comma-pickles-and-chutney",
            "name": "Tins, Pickles & Chutney",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=tins-comma-pickles-and-chutney",
                "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney"
            }
        },
        {
            "id": "veal-and-venison",
            "name": "Veal & Venison",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=veal-and-venison",
                "category": "https://api.sustained.com/choice/v1/categories/veal-and-venison"
            }
        },
        {
            "id": "vegetables",
            "name": "Vegetables",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=vegetables",
                "category": "https://api.sustained.com/choice/v1/categories/vegetables"
            }
        },
        {
            "id": "water",
            "name": "Water",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=water",
                "category": "https://api.sustained.com/choice/v1/categories/water"
            }
        },
        {
            "id": "wine",
            "name": "Wine",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=wine",
                "category": "https://api.sustained.com/choice/v1/categories/wine"
            }
        },
        {
            "id": "world-foods",
            "name": "World Foods",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=world-foods",
                "category": "https://api.sustained.com/choice/v1/categories/world-foods"
            }
        },
        {
            "id": "yoghurts",
            "name": "Yoghurts",
            "links": {
                "products": "https://api.sustained.com/choice/v1/products?category=yoghurts",
                "category": "https://api.sustained.com/choice/v1/categories/yoghurts"
            }
        }
    ],
    "page": 1,
    "page_size": 50,
    "next_page_token": "",
    "total_results": 49,
    "links": {
        "self": "https://api.sustained.com/choice/v1/categories",
        "next": "",
        "first": "https://api.sustained.com/choice/v1/categories"
    },
    "products": [
        [
            {
                "id": "productid.0029b87e6bbdcf398860dd24371ff1fd",
                "name": "Organic Veggie & Salmon Pie Stage 2 from 7+ Months",
                "gtin": "5057753912260",
                "image": "https://static.euw1.sustained.app/products/images/5057753912260_0.jpg",
                "pack": "130g",
                "grade": "D",
                "points": {
                    "pef": 25.442759736644405
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0029b87e6bbdcf398860dd24371ff1fd",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0029b87e6bbdcf398860dd24371ff1fd/impacts"
                }
            },
            {
                "id": "productid.00363e5801122b2899e5564c32c49bf2",
                "name": "Sweet Potato Mash & Chicken from 10m Onwards",
                "gtin": "3041094546224",
                "image": "https://static.euw1.sustained.app/products/images/3041094546224_0.jpg",
                "pack": "230g",
                "grade": "C",
                "points": {
                    "pef": 16.988369037820192
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00363e5801122b2899e5564c32c49bf2",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00363e5801122b2899e5564c32c49bf2/impacts"
                }
            },
            {
                "id": "productid.02610e023d22c982e2d82a088d0f8659",
                "name": "Mini Coconut Rolls, Infant Snack, 12 Months+, Multipack",
                "gtin": "5060040257890",
                "image": "https://static.euw1.sustained.app/products/images/5060040257890_0.jpg",
                "pack": "6.8g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02610e023d22c982e2d82a088d0f8659",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02610e023d22c982e2d82a088d0f8659/impacts"
                }
            },
            {
                "id": "productid.03a5701d451f076c541f81b599b50090",
                "name": "Medi+ Lactose-Free Infant Formula 0 - 12 Months",
                "gtin": "5056000503060",
                "image": "https://static.euw1.sustained.app/products/images/5056000503060_0.jpg",
                "pack": "400g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03a5701d451f076c541f81b599b50090",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03a5701d451f076c541f81b599b50090/impacts"
                }
            },
            {
                "id": "productid.06b14ef9ce081698922136e61174fb93",
                "name": "Apple Crisps, Kids Snack, 12+ Months",
                "gtin": "5060040250778",
                "image": "https://static.euw1.sustained.app/products/images/5060040250778_0.jpg",
                "pack": "12g",
                "grade": "B",
                "points": {
                    "pef": 11.808422527904124
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06b14ef9ce081698922136e61174fb93",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06b14ef9ce081698922136e61174fb93/impacts"
                }
            },
            {
                "id": "productid.07630aa18c0769d796dd379e1c99a449",
                "name": "Organic Bangers and Mash Baby Pouch 7+ Months",
                "gtin": "5060107335103",
                "image": "https://static.euw1.sustained.app/products/images/5060107335103_0.jpg",
                "pack": "130g",
                "grade": "G",
                "points": {
                    "pef": 101.82416209284428
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07630aa18c0769d796dd379e1c99a449",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07630aa18c0769d796dd379e1c99a449/impacts"
                }
            },
            {
                "id": "productid.0a0c3db5ccf1818d912ac926071c54ba",
                "name": "Little Angels Organic Pasta with Chicken & Mushrooms 10+ Months",
                "gtin": "5057172300754",
                "image": "https://static.euw1.sustained.app/products/images/5057172300754_0.jpg",
                "pack": "190g",
                "grade": "C",
                "points": {
                    "pef": 18.013750410197517
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a0c3db5ccf1818d912ac926071c54ba",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a0c3db5ccf1818d912ac926071c54ba/impacts"
                }
            },
            {
                "id": "productid.0a99141d9a84ecf2f0e04221a643560e",
                "name": "Organic Stage 2 +7 months Mashed Cottage Pie",
                "gtin": "5391527190585",
                "image": "https://static.euw1.sustained.app/products/images/5391527190585_0.jpg",
                "pack": "120g",
                "grade": "E",
                "points": {
                    "pef": 37.96992063554568
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a99141d9a84ecf2f0e04221a643560e",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a99141d9a84ecf2f0e04221a643560e/impacts"
                }
            },
            {
                "id": "productid.0b493af60fe98f6253e40e09ac0d4edd",
                "name": "Organic Seven Grain Cereal 7 Months+",
                "gtin": "3041091645968",
                "image": "https://static.euw1.sustained.app/products/images/3041091645968_0.jpg",
                "pack": "180g",
                "grade": "A",
                "points": {
                    "pef": 10.428070202056544
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b493af60fe98f6253e40e09ac0d4edd",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b493af60fe98f6253e40e09ac0d4edd/impacts"
                }
            },
            {
                "id": "productid.0bc311fc2d8cce8a7043590fec1b7fb6",
                "name": "Sweetcorn + Carrot Melty Sticks from 7 Months",
                "gtin": "5060503502932",
                "image": "https://static.euw1.sustained.app/products/images/5060503502932_0.jpg",
                "pack": "6g",
                "grade": "C",
                "points": {
                    "pef": 19.57293971378376
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bc311fc2d8cce8a7043590fec1b7fb6",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bc311fc2d8cce8a7043590fec1b7fb6/impacts"
                }
            },
            {
                "id": "productid.0c9ef4a8ad171c0b65a4dd378d945a35",
                "name": "Organic Tutti Fruity Chicken Curry +8 Months Textured",
                "gtin": "5391527190943",
                "image": "https://static.euw1.sustained.app/products/images/5391527190943_0.jpg",
                "pack": "170g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c9ef4a8ad171c0b65a4dd378d945a35",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c9ef4a8ad171c0b65a4dd378d945a35/impacts"
                }
            },
            {
                "id": "productid.0d190ca190af0e84b0eaed8876f44616",
                "name": "Sweet Potato, Chicken & Veggies Pouch 7m+",
                "gtin": "5000157079909",
                "image": "https://static.euw1.sustained.app/products/images/5000157079909_0.jpg",
                "pack": "130g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d190ca190af0e84b0eaed8876f44616",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d190ca190af0e84b0eaed8876f44616/impacts"
                }
            },
            {
                "id": "productid.0e7395de6067fab067d1114083e3b434",
                "name": "Mild Chilli Con Carne with a Pinch of Paprika 7+ Months",
                "gtin": "5060107339675",
                "image": "https://static.euw1.sustained.app/products/images/5060107339675_0.jpg",
                "pack": "130g",
                "grade": "F",
                "points": {
                    "pef": 68.19840198826515
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e7395de6067fab067d1114083e3b434",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e7395de6067fab067d1114083e3b434/impacts"
                }
            },
            {
                "id": "productid.105f995e1dd488bb90bce184372337bf",
                "name": "Organic Greek-Style Yoghurt with Prunes & Oats Around 6 Months",
                "gtin": "5060454500667",
                "image": "https://static.euw1.sustained.app/products/images/5060454500667_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 65.32060430502602
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.105f995e1dd488bb90bce184372337bf",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.105f995e1dd488bb90bce184372337bf/impacts"
                }
            },
            {
                "id": "productid.1205b00ff6a59da14221de2612a9ccce",
                "name": "Organic Squares Gentle Teething Wafers Mango Textured 10+ Months",
                "gtin": "5060452490922",
                "image": "https://static.euw1.sustained.app/products/images/5060452490922_0.jpg",
                "pack": "20g",
                "grade": "E",
                "points": {
                    "pef": 41.925335091700006
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1205b00ff6a59da14221de2612a9ccce",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1205b00ff6a59da14221de2612a9ccce/impacts"
                }
            },
            {
                "id": "productid.12819f9c07091afc6fb0b847ecb30655",
                "name": "Apple, Apricot, Banana & Orange",
                "gtin": "5000157144119",
                "image": "https://static.euw1.sustained.app/products/images/5000157144119_0.jpg",
                "pack": "120g",
                "grade": "A",
                "points": {
                    "pef": 7.639452226837631
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12819f9c07091afc6fb0b847ecb30655",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12819f9c07091afc6fb0b847ecb30655/impacts"
                }
            },
            {
                "id": "productid.158c5f29ad045cd5bdddcb13cf74c784",
                "name": "Thai Chicken Curry Pouch 10m+",
                "gtin": "5000157079794",
                "image": "https://static.euw1.sustained.app/products/images/5000157079794_0.jpg",
                "pack": "180g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.158c5f29ad045cd5bdddcb13cf74c784",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.158c5f29ad045cd5bdddcb13cf74c784/impacts"
                }
            },
            {
                "id": "productid.15ea5f9bcbd2e32829d88d0d93f3cd9b",
                "name": "Banana Wholegrain Porridge Baby Cereal",
                "gtin": "3041091431790",
                "image": "https://static.euw1.sustained.app/products/images/3041091431790_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 38.573070279198454
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.15ea5f9bcbd2e32829d88d0d93f3cd9b",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.15ea5f9bcbd2e32829d88d0d93f3cd9b/impacts"
                }
            },
            {
                "id": "productid.164e2de39d299b7f801873c159314d38",
                "name": "Kids Marvellous Mandarin & Apple Oaty Bars",
                "gtin": "5024121497216",
                "image": "https://static.euw1.sustained.app/products/images/5024121497216_0.jpg",
                "pack": "23g",
                "grade": "D",
                "points": {
                    "pef": 26.650274214022556
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.164e2de39d299b7f801873c159314d38",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.164e2de39d299b7f801873c159314d38/impacts"
                }
            },
            {
                "id": "productid.1a02b578fc37847149bb60749c6c2bc4",
                "name": "Little Ones Organic Cottage Pie 10+ Months",
                "gtin": "0000000332118",
                "image": "https://static.euw1.sustained.app/products/images/0000000332118_0.jpg",
                "pack": "190g",
                "grade": "E",
                "points": {
                    "pef": 38.197927783635706
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1a02b578fc37847149bb60749c6c2bc4",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1a02b578fc37847149bb60749c6c2bc4/impacts"
                }
            },
            {
                "id": "productid.1b5d076d223e799af8b68d27ce63ff36",
                "name": "Let's Cook Animal Shapes Pasta 10+ Months",
                "gtin": "5000157146113",
                "image": "https://static.euw1.sustained.app/products/images/5000157146113_0.jpg",
                "pack": "340g",
                "grade": "A",
                "points": {
                    "pef": 10.713198898827812
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b5d076d223e799af8b68d27ce63ff36",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b5d076d223e799af8b68d27ce63ff36/impacts"
                }
            },
            {
                "id": "productid.1b93cdfd20085996847ccbbe308c9053",
                "name": "Chocolate Biscotti 7m+",
                "gtin": "5000157065759",
                "image": "https://static.euw1.sustained.app/products/images/5000157065759_0.jpg",
                "pack": "60g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b93cdfd20085996847ccbbe308c9053",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b93cdfd20085996847ccbbe308c9053/impacts"
                }
            },
            {
                "id": "productid.1ba421245eedaaefeb145982012c544b",
                "name": "Organic Mangoes, Pears and Papaya Baby Pouch 4+ Months",
                "gtin": "5060107334786",
                "image": "https://static.euw1.sustained.app/products/images/5060107334786_0.jpg",
                "pack": "120g",
                "grade": "B",
                "points": {
                    "pef": 11.971671427113165
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1ba421245eedaaefeb145982012c544b",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1ba421245eedaaefeb145982012c544b/impacts"
                }
            },
            {
                "id": "productid.1bc030b745bf686dafee55cf31264ec5",
                "name": "Organic Veggie Moussaka Baby Pouch 7+ Months",
                "gtin": "5060107336506",
                "image": "https://static.euw1.sustained.app/products/images/5060107336506_0.jpg",
                "pack": "130g",
                "grade": "F",
                "points": {
                    "pef": 68.36242004384562
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1bc030b745bf686dafee55cf31264ec5",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1bc030b745bf686dafee55cf31264ec5/impacts"
                }
            },
            {
                "id": "productid.1c4483efd88d7e30a2f1d98e1036eb96",
                "name": "Fish & Coconut Curry 1-3 Years",
                "gtin": "5060398490222",
                "image": "https://static.euw1.sustained.app/products/images/5060398490222_0.jpg",
                "pack": "230g",
                "grade": "D",
                "points": {
                    "pef": 23.578141619009944
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c4483efd88d7e30a2f1d98e1036eb96",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c4483efd88d7e30a2f1d98e1036eb96/impacts"
                }
            },
            {
                "id": "productid.1c4fa544151fdc5096a1b450d49b8bff",
                "name": "7+ Months By Nature Cottage Pie",
                "gtin": "5000157140487",
                "image": "https://static.euw1.sustained.app/products/images/5000157140487_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c4fa544151fdc5096a1b450d49b8bff",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c4fa544151fdc5096a1b450d49b8bff/impacts"
                }
            },
            {
                "id": "productid.1d417316ff28c35afbdf53d42eea6af3",
                "name": "Mini Organic Cheese Crackers Toddler Snack Multipack",
                "gtin": "5024121626104",
                "image": "https://static.euw1.sustained.app/products/images/5024121626104_0.jpg",
                "pack": "20g",
                "grade": "D",
                "points": {
                    "pef": 25.214834171493443
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1d417316ff28c35afbdf53d42eea6af3",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1d417316ff28c35afbdf53d42eea6af3/impacts"
                }
            },
            {
                "id": "productid.204533b5653e6cf3c18cdddbd1f29329",
                "name": "2 Follow On Baby Milk Formula",
                "gtin": "5051594002715",
                "image": "https://static.euw1.sustained.app/products/images/5051594002715_0.jpg",
                "pack": "1l",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.204533b5653e6cf3c18cdddbd1f29329",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.204533b5653e6cf3c18cdddbd1f29329/impacts"
                }
            },
            {
                "id": "productid.206c28d1f7ab80e321a7886a50436b1b",
                "name": "Chicken & Sweet Potato Curry 10+ Months Stage 3",
                "gtin": "5060398490178",
                "image": "https://static.euw1.sustained.app/products/images/5060398490178_0.jpg",
                "pack": "190g",
                "grade": "C",
                "points": {
                    "pef": 19.69754820844496
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.206c28d1f7ab80e321a7886a50436b1b",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.206c28d1f7ab80e321a7886a50436b1b/impacts"
                }
            },
            {
                "id": "productid.2203406b07802a75ac5be1b44bfc7716",
                "name": "Creamy Porridge Baby Cereal 6+ Months",
                "gtin": "4062300013957",
                "image": "https://static.euw1.sustained.app/products/images/4062300013957_0.jpg",
                "pack": "160g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2203406b07802a75ac5be1b44bfc7716",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2203406b07802a75ac5be1b44bfc7716/impacts"
                }
            },
            {
                "id": "productid.22bcd9660530b59b1a3f3d3f4e04542b",
                "name": "Organic Red & Go Smoothie Multipack 6 Months+",
                "gtin": "5060452490533",
                "image": "https://static.euw1.sustained.app/products/images/5060452490533_0.jpg",
                "pack": "90g",
                "grade": "C",
                "points": {
                    "pef": 20.1107745795256
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.22bcd9660530b59b1a3f3d3f4e04542b",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.22bcd9660530b59b1a3f3d3f4e04542b/impacts"
                }
            },
            {
                "id": "productid.2362b79846a183ecdbc83c928ed7f66a",
                "name": "10+ Months By Nature Veggie Singapore Noodles",
                "gtin": "5000157079817",
                "image": "https://static.euw1.sustained.app/products/images/5000157079817_0.jpg",
                "pack": "180g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2362b79846a183ecdbc83c928ed7f66a",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2362b79846a183ecdbc83c928ed7f66a/impacts"
                }
            },
            {
                "id": "productid.271b1c9443c0b2da761f67d377eebc36",
                "name": "Organic Blushing Berries, Pear & Banana",
                "gtin": "5060452490281",
                "image": "https://static.euw1.sustained.app/products/images/5060452490281_0.jpg",
                "pack": "100g",
                "grade": "D",
                "points": {
                    "pef": 22.206555363866126
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.271b1c9443c0b2da761f67d377eebc36",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.271b1c9443c0b2da761f67d377eebc36/impacts"
                }
            },
            {
                "id": "productid.271d2b274e0bbf3ca2cdb028874fb3ad",
                "name": "Beefy Stew with Spuds from 10 Months",
                "gtin": "5060107333970",
                "image": "https://static.euw1.sustained.app/products/images/5060107333970_0.jpg",
                "pack": "190g",
                "grade": "F",
                "points": {
                    "pef": 67.56881530465056
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.271d2b274e0bbf3ca2cdb028874fb3ad",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.271d2b274e0bbf3ca2cdb028874fb3ad/impacts"
                }
            },
            {
                "id": "productid.28632cdbd0bd76730f8970ed5394be2d",
                "name": "Scrumptious Lasagne 12+ Months",
                "gtin": "4062300024519",
                "image": "https://static.euw1.sustained.app/products/images/4062300024519_0.jpg",
                "pack": "230g",
                "grade": "E",
                "points": {
                    "pef": 35.06269945745182
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.28632cdbd0bd76730f8970ed5394be2d",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.28632cdbd0bd76730f8970ed5394be2d/impacts"
                }
            },
            {
                "id": "productid.293b0b48865afd7cc6b93526998266ba",
                "name": "Spaghetti Bolognese Baby Food Jar 6+ Months",
                "gtin": "4062300161481",
                "image": "https://static.euw1.sustained.app/products/images/4062300161481_0.jpg",
                "pack": "125g",
                "grade": "E",
                "points": {
                    "pef": 31.652305769588658
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.293b0b48865afd7cc6b93526998266ba",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.293b0b48865afd7cc6b93526998266ba/impacts"
                }
            },
            {
                "id": "productid.2a781971c1f2f1463a36a3b525b21123",
                "name": "Organic Mango, Apple and Coconut Milk Stage 1 from Around 6 Months",
                "gtin": "5060077680692",
                "image": "https://static.euw1.sustained.app/products/images/5060077680692_0.jpg",
                "pack": "100g",
                "grade": "B",
                "points": {
                    "pef": 11.333109684074593
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a781971c1f2f1463a36a3b525b21123",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a781971c1f2f1463a36a3b525b21123/impacts"
                }
            },
            {
                "id": "productid.2bc866a3af9d0e27a611792ded6d3784",
                "name": "Organics Organic Pineapple & Banana Super Puffs Age 6+ Months",
                "gtin": "5060095650776",
                "image": "https://static.euw1.sustained.app/products/images/5060095650776_0.jpg",
                "pack": "20g",
                "grade": "D",
                "points": {
                    "pef": 26.758481496450376
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2bc866a3af9d0e27a611792ded6d3784",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2bc866a3af9d0e27a611792ded6d3784/impacts"
                }
            },
            {
                "id": "productid.2eb342c7f82c69fd97cdd4af84955336",
                "name": "Strawberry + Apple Oaty Biccies 12+ Months",
                "gtin": "5060503504554",
                "image": "https://static.euw1.sustained.app/products/images/5060503504554_0.jpg",
                "pack": "20g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2eb342c7f82c69fd97cdd4af84955336",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2eb342c7f82c69fd97cdd4af84955336/impacts"
                }
            },
            {
                "id": "productid.2fae156e1dfd4fab4f5c06bea8a4e03b",
                "name": "Wheat Based Fortified Baby Cereal with Honey & Wheat, Just Add Water, 12 Months+,",
                "gtin": "7616100581535",
                "image": "https://static.euw1.sustained.app/products/images/7616100581535_0.jpg",
                "pack": "1kg",
                "grade": "F",
                "points": {
                    "pef": 71.61575191336787
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2fae156e1dfd4fab4f5c06bea8a4e03b",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2fae156e1dfd4fab4f5c06bea8a4e03b/impacts"
                }
            },
            {
                "id": "productid.2fe4bfa0885de2ebe0c8cc3badcada05",
                "name": "Kids Apple Soft Biscotti 7+ Months",
                "gtin": "5391500921571",
                "image": "https://static.euw1.sustained.app/products/images/5391500921571_0.jpg",
                "pack": "20g",
                "grade": "E",
                "points": {
                    "pef": 37.33883821344808
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2fe4bfa0885de2ebe0c8cc3badcada05",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2fe4bfa0885de2ebe0c8cc3badcada05/impacts"
                }
            },
            {
                "id": "productid.31375b13a4acc3173292fc3644d2dd2a",
                "name": "Smoothie Melts, Blackcurrant, Apple & Banana 12months+",
                "gtin": "5060040252024",
                "image": "https://static.euw1.sustained.app/products/images/5060040252024_0.jpg",
                "pack": "6g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.31375b13a4acc3173292fc3644d2dd2a",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.31375b13a4acc3173292fc3644d2dd2a/impacts"
                }
            },
            {
                "id": "productid.336c788d285503e3b9243dc124c474cb",
                "name": "Apple Blackcurrant & Banana Fruit Puree Pots",
                "gtin": "3041091265555",
                "image": "https://static.euw1.sustained.app/products/images/3041091265555_0.jpg",
                "pack": "100g",
                "grade": "A",
                "points": {
                    "pef": 8.613879274472591
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.336c788d285503e3b9243dc124c474cb",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.336c788d285503e3b9243dc124c474cb/impacts"
                }
            },
            {
                "id": "productid.386d92f13e98aa16313c79b8a05a51ba",
                "name": "HYGIENE AQUAINT Sanitising Water",
                "gtin": "5060080353408",
                "image": "https://static.euw1.sustained.app/products/images/5060080353408_0.jpg",
                "pack": "50ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.386d92f13e98aa16313c79b8a05a51ba",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.386d92f13e98aa16313c79b8a05a51ba/impacts"
                }
            },
            {
                "id": "productid.397a51335d318ac7914a76bb61f35440",
                "name": "Organic Macaroni Cheese Baby Pouch 6+ Months",
                "gtin": "5060107336629",
                "image": "https://static.euw1.sustained.app/products/images/5060107336629_0.jpg",
                "pack": "120g",
                "grade": "F",
                "points": {
                    "pef": 83.60853578947658
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.397a51335d318ac7914a76bb61f35440",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.397a51335d318ac7914a76bb61f35440/impacts"
                }
            },
            {
                "id": "productid.3b17b7ff11232d128e2e3c074b72bfe4",
                "name": "Organic Chicken and Noodles Baby Pouch 7+ Months",
                "gtin": "5060107333871",
                "image": "https://static.euw1.sustained.app/products/images/5060107333871_0.jpg",
                "pack": "130g",
                "grade": "E",
                "points": {
                    "pef": 51.76534175510773
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3b17b7ff11232d128e2e3c074b72bfe4",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3b17b7ff11232d128e2e3c074b72bfe4/impacts"
                }
            },
            {
                "id": "productid.3c4e9844f4be2786e0318ecdac898f7b",
                "name": "Organic Avocado Breakfast with Yoghurt, Spinach & Oats Around 6 Months",
                "gtin": "5060454500629",
                "image": "https://static.euw1.sustained.app/products/images/5060454500629_0.jpg",
                "pack": "100g",
                "grade": "C",
                "points": {
                    "pef": 18.709371697743812
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3c4e9844f4be2786e0318ecdac898f7b",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3c4e9844f4be2786e0318ecdac898f7b/impacts"
                }
            },
            {
                "id": "productid.3e0c81e5ef289cfbea849eb052445d94",
                "name": "Lamb Hot Pot Pouch 7m+",
                "gtin": "5000157079855",
                "image": "https://static.euw1.sustained.app/products/images/5000157079855_0.jpg",
                "pack": "130g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3e0c81e5ef289cfbea849eb052445d94",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3e0c81e5ef289cfbea849eb052445d94/impacts"
                }
            },
            {
                "id": "productid.45cdacce66be05fe043b381e31128ed1",
                "name": "Blueberry Porridge 7m+",
                "gtin": "5000157077028",
                "image": "https://static.euw1.sustained.app/products/images/5000157077028_0.jpg",
                "pack": "240g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.45cdacce66be05fe043b381e31128ed1",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.45cdacce66be05fe043b381e31128ed1/impacts"
                }
            },
            {
                "id": "productid.45e4732366819efc3bdb299c096d4728",
                "name": "4 Growing Up Baby Milk Powder from 2 Years Onwards",
                "gtin": "4062300270732",
                "image": "https://static.euw1.sustained.app/products/images/4062300270732_0.jpg",
                "pack": "600g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.45e4732366819efc3bdb299c096d4728",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.45e4732366819efc3bdb299c096d4728/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.00e0cef98fcdf810c12379eb4bc1ae42",
                "name": "6 Traditional Unsmoked Back Rashers",
                "gtin": "5390171004088",
                "image": "https://static.euw1.sustained.app/products/images/5390171004088_0.jpg",
                "pack": "190g",
                "grade": "G",
                "points": {
                    "pef": 127.59533272704213
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00e0cef98fcdf810c12379eb4bc1ae42",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00e0cef98fcdf810c12379eb4bc1ae42/impacts"
                }
            },
            {
                "id": "productid.012d1196db0a9d02f3bb0fae6ea67b61",
                "name": "Traditional Steak Slice",
                "gtin": "5060238331692",
                "image": "https://static.euw1.sustained.app/products/images/5060238331692_0.jpg",
                "pack": "1.1kg",
                "grade": "G",
                "points": {
                    "pef": 193.60638534125954
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.012d1196db0a9d02f3bb0fae6ea67b61",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.012d1196db0a9d02f3bb0fae6ea67b61/impacts"
                }
            },
            {
                "id": "productid.018f582dc3018be777cfe8b84d44d37a",
                "name": "Unsmoked 2 Bacon Loin Steaks",
                "gtin": "0000001832617",
                "image": "https://static.euw1.sustained.app/products/images/0000001832617_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 132.19673159991638
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.018f582dc3018be777cfe8b84d44d37a",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.018f582dc3018be777cfe8b84d44d37a/impacts"
                }
            },
            {
                "id": "productid.01ec2eff9a5cf0ee164c5f7810e9b72b",
                "name": "Food German Bratwurst",
                "gtin": "0000029196876",
                "image": "https://static.euw1.sustained.app/products/images/0000029196876_0.jpg",
                "pack": "360g",
                "grade": "G",
                "points": {
                    "pef": 119.34791739993811
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01ec2eff9a5cf0ee164c5f7810e9b72b",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01ec2eff9a5cf0ee164c5f7810e9b72b/impacts"
                }
            },
            {
                "id": "productid.02f54a10124baf06f5bfc4e348aceafb",
                "name": "Butcher's Selection 12 Smoked Back Bacon Rashers",
                "gtin": "5054070504550",
                "image": "https://static.euw1.sustained.app/products/images/5054070504550_0.jpg",
                "pack": "345g",
                "grade": "G",
                "points": {
                    "pef": 134.07525824923528
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02f54a10124baf06f5bfc4e348aceafb",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02f54a10124baf06f5bfc4e348aceafb/impacts"
                }
            },
            {
                "id": "productid.03600e64ed8f1a0684ac4edd3651f4aa",
                "name": "Dry Zywiecka Sausage",
                "gtin": "5906453734453",
                "image": "https://static.euw1.sustained.app/products/images/5906453734453_0.jpg",
                "pack": "0.250kg",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03600e64ed8f1a0684ac4edd3651f4aa",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03600e64ed8f1a0684ac4edd3651f4aa/impacts"
                }
            },
            {
                "id": "productid.03aa4fec11c013d1d1efb669cc4eef36",
                "name": "Smoked Back Bacon",
                "gtin": "5010251716211",
                "image": "https://static.euw1.sustained.app/products/images/5010251716211_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 132.45296154865304
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03aa4fec11c013d1d1efb669cc4eef36",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03aa4fec11c013d1d1efb669cc4eef36/impacts"
                }
            },
            {
                "id": "productid.06312b5114d7847f1012cd4ed0f71c7f",
                "name": "Tesco Fuller Flavoured Bacon Company Maple Cured Back Bacon 8 Rashers",
                "gtin": "5022977011570",
                "image": "https://static.euw1.sustained.app/products/images/5022977011570_0.jpg",
                "pack": "280g",
                "grade": "G",
                "points": {
                    "pef": 138.06267598155813
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06312b5114d7847f1012cd4ed0f71c7f",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06312b5114d7847f1012cd4ed0f71c7f/impacts"
                }
            },
            {
                "id": "productid.07be4f4345c94f456eeeaa631a8eccef",
                "name": "Cumberland Thick Sliced Sausage 5 Slices",
                "gtin": "5010525118277",
                "image": "https://static.euw1.sustained.app/products/images/5010525118277_0.jpg",
                "pack": "150g",
                "grade": "G",
                "points": {
                    "pef": 112.43613887014276
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07be4f4345c94f456eeeaa631a8eccef",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07be4f4345c94f456eeeaa631a8eccef/impacts"
                }
            },
            {
                "id": "productid.0952d82849f30b7ebd5022804a9960de",
                "name": "Maple Cure Back Bacon",
                "gtin": "5017326136660",
                "image": "https://static.euw1.sustained.app/products/images/5017326136660_0.jpg",
                "pack": "300g",
                "grade": "G",
                "points": {
                    "pef": 128.02894898494387
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0952d82849f30b7ebd5022804a9960de",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0952d82849f30b7ebd5022804a9960de/impacts"
                }
            },
            {
                "id": "productid.0a92b8abc036635d60e3ca6186fc4d35",
                "name": "Finest British Unsmoked Dry Cured Back Bacon 8 Rashers",
                "gtin": "5052910096562",
                "image": "https://static.euw1.sustained.app/products/images/5052910096562_0.jpg",
                "pack": "240g",
                "grade": "G",
                "points": {
                    "pef": 95.23089826646051
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a92b8abc036635d60e3ca6186fc4d35",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a92b8abc036635d60e3ca6186fc4d35/impacts"
                }
            },
            {
                "id": "productid.0ba38414d7bf85dc40606ac6eba9373c",
                "name": "Vegetarian Jumbo Hotdogs",
                "gtin": "7290112969132",
                "image": "https://static.euw1.sustained.app/products/images/7290112969132_0.jpg",
                "pack": "400g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ba38414d7bf85dc40606ac6eba9373c",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ba38414d7bf85dc40606ac6eba9373c/impacts"
                }
            },
            {
                "id": "productid.0be45299ef0bb4155491560af39f175c",
                "name": "8 Thick Cut Unsmoked Back Bacon Rashers",
                "gtin": "5060234454593",
                "image": "https://static.euw1.sustained.app/products/images/5060234454593_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 132.95136839973136
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0be45299ef0bb4155491560af39f175c",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0be45299ef0bb4155491560af39f175c/impacts"
                }
            },
            {
                "id": "productid.0cf6d70afa3e611020e06912a1eabcb9",
                "name": "Extra Special Pigs in Blankets Festive Feast",
                "gtin": "5057172237104",
                "image": "https://static.euw1.sustained.app/products/images/5057172237104_0.jpg",
                "pack": "694g",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0cf6d70afa3e611020e06912a1eabcb9",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0cf6d70afa3e611020e06912a1eabcb9/impacts"
                }
            },
            {
                "id": "productid.0e1635b8914c71a94f5828f1f5038a12",
                "name": "The Best 12 British Spicy Lamb Merguez Chipolatas",
                "gtin": "5010251882176",
                "image": "https://static.euw1.sustained.app/products/images/5010251882176_0.jpg",
                "pack": "375g",
                "grade": "G",
                "points": {
                    "pef": 407.54108668814115
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e1635b8914c71a94f5828f1f5038a12",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e1635b8914c71a94f5828f1f5038a12/impacts"
                }
            },
            {
                "id": "productid.0e5ef1222f249900d813f2e1de859594",
                "name": "Butcher's Selection 4 Cheese and Jalape\u00f1o Pork Hot Dogs",
                "gtin": "5054070655610",
                "image": "https://static.euw1.sustained.app/products/images/5054070655610_0.jpg",
                "pack": "400g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e5ef1222f249900d813f2e1de859594",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e5ef1222f249900d813f2e1de859594/impacts"
                }
            },
            {
                "id": "productid.0e703d240d86fd4950cc2f2784e2e8c6",
                "name": "12 Plant-Based Chipolata Sausages",
                "gtin": "5060626405059",
                "image": "https://static.euw1.sustained.app/products/images/5060626405059_0.jpg",
                "pack": "288g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e703d240d86fd4950cc2f2784e2e8c6",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e703d240d86fd4950cc2f2784e2e8c6/impacts"
                }
            },
            {
                "id": "productid.0f0934bb09d9792a4b309a2c34e96ab5",
                "name": "The Scottish Butcher Classic Breakfast Beef Lorne Sausage 4 Slices",
                "gtin": "5060004212774",
                "image": "https://static.euw1.sustained.app/products/images/5060004212774_0.jpg",
                "pack": "270g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f0934bb09d9792a4b309a2c34e96ab5",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f0934bb09d9792a4b309a2c34e96ab5/impacts"
                }
            },
            {
                "id": "productid.0f501b2b1135814c6a71bb1dd4b8e493",
                "name": "Italian Mortadella 5 Slices",
                "gtin": "5000169582275",
                "image": "https://static.euw1.sustained.app/products/images/5000169582275_0.jpg",
                "pack": "75g",
                "grade": "G",
                "points": {
                    "pef": 144.58423969347473
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f501b2b1135814c6a71bb1dd4b8e493",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f501b2b1135814c6a71bb1dd4b8e493/impacts"
                }
            },
            {
                "id": "productid.102042c589be106437aeeac7eab45e39",
                "name": "Extra Special 4 New York Style Hot Dog Sausages",
                "gtin": "5054781855026",
                "image": "https://static.euw1.sustained.app/products/images/5054781855026_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 126.97979760310086
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.102042c589be106437aeeac7eab45e39",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.102042c589be106437aeeac7eab45e39/impacts"
                }
            },
            {
                "id": "productid.103550720c1975e1f15fd0a891774919",
                "name": "Smoky Hot Dogs",
                "gtin": "5060055251432",
                "image": "https://static.euw1.sustained.app/products/images/5060055251432_0.jpg",
                "pack": "1.2kg",
                "grade": "G",
                "points": {
                    "pef": 126.4399828826582
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.103550720c1975e1f15fd0a891774919",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.103550720c1975e1f15fd0a891774919/impacts"
                }
            },
            {
                "id": "productid.103db7f910a96785075d763d33d5aca9",
                "name": "6 Chicken Sausages",
                "gtin": "5054775603329",
                "image": "https://static.euw1.sustained.app/products/images/5054775603329_0.jpg",
                "pack": "400g",
                "grade": "F",
                "points": {
                    "pef": 53.46459740610108
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.103db7f910a96785075d763d33d5aca9",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.103db7f910a96785075d763d33d5aca9/impacts"
                }
            },
            {
                "id": "productid.104def12688a1f46464e70ee7b353a58",
                "name": "Butcher's Selection 4 New York Pork Hot Dogs",
                "gtin": "5054070655603",
                "image": "https://static.euw1.sustained.app/products/images/5054070655603_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 126.03495417555642
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.104def12688a1f46464e70ee7b353a58",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.104def12688a1f46464e70ee7b353a58/impacts"
                }
            },
            {
                "id": "productid.112fdfef55241956698bb5ba1735e74a",
                "name": "Kabanosy Classic",
                "gtin": "5907785150119",
                "image": "https://static.euw1.sustained.app/products/images/5907785150119_0.jpg",
                "pack": "175g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.112fdfef55241956698bb5ba1735e74a",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.112fdfef55241956698bb5ba1735e74a/impacts"
                }
            },
            {
                "id": "productid.11f621b5a0a1a758b2c641e79638797f",
                "name": "Butcher's Selection 8 Thick Cut Smoked Back Bacon Rashers",
                "gtin": "5057172706297",
                "image": "https://static.euw1.sustained.app/products/images/5057172706297_0.jpg",
                "pack": "350g",
                "grade": "G",
                "points": {
                    "pef": 133.3213410957882
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11f621b5a0a1a758b2c641e79638797f",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11f621b5a0a1a758b2c641e79638797f/impacts"
                }
            },
            {
                "id": "productid.122435767c64bf55b7906b2d97b39002",
                "name": "Naked Veg and Proud 6 Delicious Red Pepper & Smoky Squash Sausages",
                "gtin": "5060055252248",
                "image": "https://static.euw1.sustained.app/products/images/5060055252248_0.jpg",
                "pack": "270g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.122435767c64bf55b7906b2d97b39002",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.122435767c64bf55b7906b2d97b39002/impacts"
                }
            },
            {
                "id": "productid.1560f81a38f8aeeda7e9c17f1069b255",
                "name": "German Frankfurters",
                "gtin": "5010251733256",
                "image": "https://static.euw1.sustained.app/products/images/5010251733256_0.jpg",
                "pack": "350g",
                "grade": "G",
                "points": {
                    "pef": 118.58238523719618
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1560f81a38f8aeeda7e9c17f1069b255",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1560f81a38f8aeeda7e9c17f1069b255/impacts"
                }
            },
            {
                "id": "productid.15f722ced3e072b7e8367bf0a42c734f",
                "name": "Noel Chorizo with Truffle",
                "gtin": "29065691",
                "image": "https://static.euw1.sustained.app/products/images/29065691_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.15f722ced3e072b7e8367bf0a42c734f",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.15f722ced3e072b7e8367bf0a42c734f/impacts"
                }
            },
            {
                "id": "productid.164048e623579ecce8dbbe4e413b92c1",
                "name": "8 Unsmoked Back Bacon Rashers",
                "gtin": "5010251603924",
                "image": "https://static.euw1.sustained.app/products/images/5010251603924_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 132.9987880358399
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.164048e623579ecce8dbbe4e413b92c1",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.164048e623579ecce8dbbe4e413b92c1/impacts"
                }
            },
            {
                "id": "productid.16990b05ee040eeaef2b7e56a3bd02a5",
                "name": "12 Aberdeen Angus Beef Chipolatas with Black Pepper",
                "gtin": "5000169246795",
                "image": "https://static.euw1.sustained.app/products/images/5000169246795_0.jpg",
                "pack": "340g",
                "grade": "G",
                "points": {
                    "pef": 224.09953181010957
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16990b05ee040eeaef2b7e56a3bd02a5",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16990b05ee040eeaef2b7e56a3bd02a5/impacts"
                }
            },
            {
                "id": "productid.1792e47a09d26202678fcf9e1f75cdd8",
                "name": "Farm Stores Smoked Back Bacon",
                "gtin": "5054781006619",
                "image": "https://static.euw1.sustained.app/products/images/5054781006619_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 131.8131037961962
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1792e47a09d26202678fcf9e1f75cdd8",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1792e47a09d26202678fcf9e1f75cdd8/impacts"
                }
            },
            {
                "id": "productid.1821da0c2dbfdceefb9965640d954ec1",
                "name": "Kabanos Sausage",
                "gtin": "5907785151895",
                "image": "https://static.euw1.sustained.app/products/images/5907785151895_0.jpg",
                "pack": "170g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1821da0c2dbfdceefb9965640d954ec1",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1821da0c2dbfdceefb9965640d954ec1/impacts"
                }
            },
            {
                "id": "productid.184f1905640173d8e7d58af1d3f05000",
                "name": "Little Bit Lighter Steak Sausages",
                "gtin": "5032992983127",
                "image": "https://static.euw1.sustained.app/products/images/5032992983127_0.jpg",
                "pack": "350g",
                "grade": "G",
                "points": {
                    "pef": 189.3231413714228
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.184f1905640173d8e7d58af1d3f05000",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.184f1905640173d8e7d58af1d3f05000/impacts"
                }
            },
            {
                "id": "productid.19cf56446145fb96fb6cf506cc6f04f3",
                "name": "8 Flavourful Pork Sausages",
                "gtin": "20335045",
                "image": "https://static.euw1.sustained.app/products/images/20335045_0.jpg",
                "pack": "454g",
                "grade": "G",
                "points": {
                    "pef": 114.47787128002341
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.19cf56446145fb96fb6cf506cc6f04f3",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.19cf56446145fb96fb6cf506cc6f04f3/impacts"
                }
            },
            {
                "id": "productid.1bf2794cd914f0ec41791bb5013df1a9",
                "name": "Farm Stores Usmoked Middle Bacon",
                "gtin": "5052449451931",
                "image": "https://static.euw1.sustained.app/products/images/5052449451931_0.jpg",
                "pack": "350g",
                "grade": "G",
                "points": {
                    "pef": 129.05310721896214
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1bf2794cd914f0ec41791bb5013df1a9",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1bf2794cd914f0ec41791bb5013df1a9/impacts"
                }
            },
            {
                "id": "productid.1efe339760cbd0dc1b7a2a41c84b1f8b",
                "name": "Traditional Irish Rindless Bacon",
                "gtin": "5391519200636",
                "image": "https://static.euw1.sustained.app/products/images/5391519200636_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 129.54911374669385
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1efe339760cbd0dc1b7a2a41c84b1f8b",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1efe339760cbd0dc1b7a2a41c84b1f8b/impacts"
                }
            },
            {
                "id": "productid.20f2818d1e618bd4b47d1302751e5a50",
                "name": "Spanish Spicy Chorizo Ring",
                "gtin": "01738339",
                "image": "https://static.euw1.sustained.app/products/images/01738339_0.jpg",
                "pack": "225g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.20f2818d1e618bd4b47d1302751e5a50",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.20f2818d1e618bd4b47d1302751e5a50/impacts"
                }
            },
            {
                "id": "productid.218ffb65b26f16b909301126660b542e",
                "name": "6 Smoked Back Bacon Extra Thick Cut Rashers",
                "gtin": "5010525058993",
                "image": "https://static.euw1.sustained.app/products/images/5010525058993_0.jpg",
                "pack": "270g",
                "grade": "G",
                "points": {
                    "pef": 127.62437940130658
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.218ffb65b26f16b909301126660b542e",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.218ffb65b26f16b909301126660b542e/impacts"
                }
            },
            {
                "id": "productid.21bed32a6710b0fc317aefb8bf5a0ce4",
                "name": "Chorizo Bites Punchy Paprika",
                "gtin": "0634114047752",
                "image": "https://static.euw1.sustained.app/products/images/0634114047752_0.jpg",
                "pack": "50g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.21bed32a6710b0fc317aefb8bf5a0ce4",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.21bed32a6710b0fc317aefb8bf5a0ce4/impacts"
                }
            },
            {
                "id": "productid.223a770c0f53fd165df90e4264027a92",
                "name": "12 Irish Pork Sausages",
                "gtin": "5099120004246",
                "image": "https://static.euw1.sustained.app/products/images/5099120004246_0.jpg",
                "pack": "340g",
                "grade": "F",
                "points": {
                    "pef": 93.04348489959413
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.223a770c0f53fd165df90e4264027a92",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.223a770c0f53fd165df90e4264027a92/impacts"
                }
            },
            {
                "id": "productid.23231bb89283db43ed2f26c0924141b0",
                "name": "Salame Toscano",
                "gtin": "8019063002380",
                "image": "https://static.euw1.sustained.app/products/images/8019063002380_0.jpg",
                "pack": "0.100kg",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.23231bb89283db43ed2f26c0924141b0",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.23231bb89283db43ed2f26c0924141b0/impacts"
                }
            },
            {
                "id": "productid.255d38db7d0df29255a85c1e067e2139",
                "name": "Hot & Spicy Pepperoni Snack",
                "gtin": "5998001705930",
                "image": "https://static.euw1.sustained.app/products/images/5998001705930_0.jpg",
                "pack": "0.090kg",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.255d38db7d0df29255a85c1e067e2139",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.255d38db7d0df29255a85c1e067e2139/impacts"
                }
            },
            {
                "id": "productid.26cdb7565d53b0844659d27a231ca7e4",
                "name": "Mini Rosette Saucisson",
                "gtin": "5010292948480",
                "image": "https://static.euw1.sustained.app/products/images/5010292948480_0.jpg",
                "pack": "300g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.26cdb7565d53b0844659d27a231ca7e4",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.26cdb7565d53b0844659d27a231ca7e4/impacts"
                }
            },
            {
                "id": "productid.2756802ba22dda318df7f9143bb521c4",
                "name": "Chicken Sausage",
                "gtin": "0216120000000",
                "image": "https://static.euw1.sustained.app/products/images/0216120000000_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 83.55303379841666
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2756802ba22dda318df7f9143bb521c4",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2756802ba22dda318df7f9143bb521c4/impacts"
                }
            },
            {
                "id": "productid.27bdd951a7c3256c4121b689bb83f8b9",
                "name": "Bacon Boiling Joint",
                "gtin": "5099490000121",
                "image": "https://static.euw1.sustained.app/products/images/5099490000121_0.jpg",
                "pack": "750g",
                "grade": "G",
                "points": {
                    "pef": 129.01705120387305
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.27bdd951a7c3256c4121b689bb83f8b9",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.27bdd951a7c3256c4121b689bb83f8b9/impacts"
                }
            },
            {
                "id": "productid.27c0efca9cf4f9a4a89e259d9725b108",
                "name": "Butcher's Selection 12 Smoked Back Bacon Rashers",
                "gtin": "5057172706273",
                "image": "https://static.euw1.sustained.app/products/images/5057172706273_0.jpg",
                "pack": "350g",
                "grade": "G",
                "points": {
                    "pef": 133.3213410957882
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.27c0efca9cf4f9a4a89e259d9725b108",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.27c0efca9cf4f9a4a89e259d9725b108/impacts"
                }
            },
            {
                "id": "productid.288a0113a81dfae364ad90427468d456",
                "name": "Premium Bacon Stamps",
                "gtin": "5099530006250",
                "image": "https://static.euw1.sustained.app/products/images/5099530006250_0.jpg",
                "pack": "1kg",
                "grade": "G",
                "points": {
                    "pef": 139.00037800048625
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.288a0113a81dfae364ad90427468d456",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.288a0113a81dfae364ad90427468d456/impacts"
                }
            },
            {
                "id": "productid.2925d30945fc98a7d7fdbbc731227b62",
                "name": "The Best 6 Thick British Pork & Sweet Chilli Sausages",
                "gtin": "5010251797630",
                "image": "https://static.euw1.sustained.app/products/images/5010251797630_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 119.12990506351414
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2925d30945fc98a7d7fdbbc731227b62",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2925d30945fc98a7d7fdbbc731227b62/impacts"
                }
            },
            {
                "id": "productid.29469154358ee4f918fa44d81931b180",
                "name": "Smoked Streaky Bacon 16 Rashers",
                "gtin": "5010525059266",
                "image": "https://static.euw1.sustained.app/products/images/5010525059266_0.jpg",
                "pack": "225g",
                "grade": "G",
                "points": {
                    "pef": 130.44511594327486
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.29469154358ee4f918fa44d81931b180",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.29469154358ee4f918fa44d81931b180/impacts"
                }
            },
            {
                "id": "productid.2a2a34fbd294b43e7929676a4a31a4c0",
                "name": "Berlinki Cheese",
                "gtin": "5900567001517",
                "image": "https://static.euw1.sustained.app/products/images/5900567001517_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 104.83768382481128
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a2a34fbd294b43e7929676a4a31a4c0",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a2a34fbd294b43e7929676a4a31a4c0/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.015486d26e6d08505ea48e79fdae4c0d",
                "name": "Diced Beef",
                "gtin": "5054402922007",
                "image": "https://static.euw1.sustained.app/products/images/5054402922007_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.015486d26e6d08505ea48e79fdae4c0d",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.015486d26e6d08505ea48e79fdae4c0d/impacts"
                }
            },
            {
                "id": "productid.053c626d0044fe59547868df54639aaf",
                "name": "Our Family Butcher 2 Standard Beef Fillets",
                "gtin": "2331665000000",
                "image": "https://static.euw1.sustained.app/products/images/2331665000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.053c626d0044fe59547868df54639aaf",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.053c626d0044fe59547868df54639aaf/impacts"
                }
            },
            {
                "id": "productid.05683f790fbc20fc573175b561a2a530",
                "name": "Extra Special British Wagyu Fillet Steak",
                "gtin": "05057172472604",
                "image": "https://static.euw1.sustained.app/products/images/05057172472604_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05683f790fbc20fc573175b561a2a530",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05683f790fbc20fc573175b561a2a530/impacts"
                }
            },
            {
                "id": "productid.059d7b6de8892c914e43639e8748695a",
                "name": "North Highland Beef Fillet Steak",
                "gtin": "0000000426787",
                "image": "https://static.euw1.sustained.app/products/images/0000000426787_0.jpg",
                "pack": "170g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.059d7b6de8892c914e43639e8748695a",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.059d7b6de8892c914e43639e8748695a/impacts"
                }
            },
            {
                "id": "productid.06507e8c82aea13a2f8d24e863c9266f",
                "name": "Irish Slow Roasting Beef Joint",
                "gtin": "0000010023334",
                "image": "https://static.euw1.sustained.app/products/images/0000010023334_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06507e8c82aea13a2f8d24e863c9266f",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06507e8c82aea13a2f8d24e863c9266f/impacts"
                }
            },
            {
                "id": "productid.06604580ad578b5fdff0451d57e3c7f8",
                "name": "10% Fat Northern Irish Beef Mince",
                "gtin": "00122139",
                "image": "https://static.euw1.sustained.app/products/images/00122139_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06604580ad578b5fdff0451d57e3c7f8",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06604580ad578b5fdff0451d57e3c7f8/impacts"
                }
            },
            {
                "id": "productid.066a8f35145933433782cb6db8260421",
                "name": "Food Select Farms British Rose Veal Escalopes",
                "gtin": "0000000773867",
                "image": "https://static.euw1.sustained.app/products/images/0000000773867_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.066a8f35145933433782cb6db8260421",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.066a8f35145933433782cb6db8260421/impacts"
                }
            },
            {
                "id": "productid.07a6b52ea3794e4b40a9732f9f22db41",
                "name": "Essential Waitrose & Partners British Beef Topside",
                "gtin": "0283980000004",
                "image": "https://static.euw1.sustained.app/products/images/0283980000004_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07a6b52ea3794e4b40a9732f9f22db41",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07a6b52ea3794e4b40a9732f9f22db41/impacts"
                }
            },
            {
                "id": "productid.0815b7b6bd88c69568820602a3200c46",
                "name": "SO Organic Northern Irish Beef Fillet Steak",
                "gtin": "0225270000004",
                "image": "https://static.euw1.sustained.app/products/images/0225270000004_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0815b7b6bd88c69568820602a3200c46",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0815b7b6bd88c69568820602a3200c46/impacts"
                }
            },
            {
                "id": "productid.08a6e042a086c29ca4d46607bf74cbf2",
                "name": "North Highland Scotch Beef 5% Fat Mince",
                "gtin": "0000000426732",
                "image": "https://static.euw1.sustained.app/products/images/0000000426732_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08a6e042a086c29ca4d46607bf74cbf2",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08a6e042a086c29ca4d46607bf74cbf2/impacts"
                }
            },
            {
                "id": "productid.08dac344693996381f3c3467d2de3531",
                "name": "Waitrose Aberdeen Angus Top Rump Joint",
                "gtin": "0265450000004",
                "image": "https://static.euw1.sustained.app/products/images/0265450000004_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08dac344693996381f3c3467d2de3531",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08dac344693996381f3c3467d2de3531/impacts"
                }
            },
            {
                "id": "productid.09235c94a888ac727fe08a7c31f5b9a3",
                "name": "British Extra Lean Beef Casserole Steak",
                "gtin": "0000000328920",
                "image": "https://static.euw1.sustained.app/products/images/0000000328920_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09235c94a888ac727fe08a7c31f5b9a3",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09235c94a888ac727fe08a7c31f5b9a3/impacts"
                }
            },
            {
                "id": "productid.09735fd1cd67c3edae45ead70bb743c3",
                "name": "North Highland Beef Rump Steak",
                "gtin": "0000000426817",
                "image": "https://static.euw1.sustained.app/products/images/0000000426817_0.jpg",
                "pack": "225g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09735fd1cd67c3edae45ead70bb743c3",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09735fd1cd67c3edae45ead70bb743c3/impacts"
                }
            },
            {
                "id": "productid.0a33d8f7a6c99d1b60ef3a881fca1674",
                "name": "Our Family Butcher Ribeye Standard Cut 1 Pack",
                "gtin": "2331669000000",
                "image": "https://static.euw1.sustained.app/products/images/2331669000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a33d8f7a6c99d1b60ef3a881fca1674",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a33d8f7a6c99d1b60ef3a881fca1674/impacts"
                }
            },
            {
                "id": "productid.0b3dd619f4180babc10b7ea6ac894e7c",
                "name": "Northern Irish Extra Lean Diced Beef",
                "gtin": "0000000414746",
                "image": "https://static.euw1.sustained.app/products/images/0000000414746_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b3dd619f4180babc10b7ea6ac894e7c",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b3dd619f4180babc10b7ea6ac894e7c/impacts"
                }
            },
            {
                "id": "productid.0f6dcec1bd0c30a2b8a50738bad226cb",
                "name": "Tender Large Beef Roasting Joint",
                "gtin": "0217710000002",
                "image": "https://static.euw1.sustained.app/products/images/0217710000002_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 221.5993954224299
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f6dcec1bd0c30a2b8a50738bad226cb",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f6dcec1bd0c30a2b8a50738bad226cb/impacts"
                }
            },
            {
                "id": "productid.0fc8d045abd377d7690433ecc618ba16",
                "name": "Irish Beef Sirloin Steak with Mixed Pepper Seasoning",
                "gtin": "5054775951048",
                "image": "https://static.euw1.sustained.app/products/images/5054775951048_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fc8d045abd377d7690433ecc618ba16",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fc8d045abd377d7690433ecc618ba16/impacts"
                }
            },
            {
                "id": "productid.1187ab6f483bd80d24510275a3e47709",
                "name": "Extra Special British Aberdeen Angus Sirloin Steak",
                "gtin": "5057172878611",
                "image": "https://static.euw1.sustained.app/products/images/5057172878611_0.jpg",
                "pack": "320g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1187ab6f483bd80d24510275a3e47709",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1187ab6f483bd80d24510275a3e47709/impacts"
                }
            },
            {
                "id": "productid.11e2da63b416f1ba093c66080a01c777",
                "name": "Fajita Style Beef Stir Fry Strips",
                "gtin": "5059697761041",
                "image": "https://static.euw1.sustained.app/products/images/5059697761041_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11e2da63b416f1ba093c66080a01c777",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11e2da63b416f1ba093c66080a01c777/impacts"
                }
            },
            {
                "id": "productid.137e907d99a341fd1bd06ba182a8d852",
                "name": "British Topside of Beef",
                "gtin": "0000000038140",
                "image": "https://static.euw1.sustained.app/products/images/0000000038140_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.137e907d99a341fd1bd06ba182a8d852",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.137e907d99a341fd1bd06ba182a8d852/impacts"
                }
            },
            {
                "id": "productid.13eac86cf7805bf4c7e8f697da7a81b2",
                "name": "British Extra Lean Beef Casserole Steak",
                "gtin": "00328920",
                "image": "https://static.euw1.sustained.app/products/images/00328920_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.13eac86cf7805bf4c7e8f697da7a81b2",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.13eac86cf7805bf4c7e8f697da7a81b2/impacts"
                }
            },
            {
                "id": "productid.14868a46711c1feab250e688887e9294",
                "name": "British Beef Rump Steak",
                "gtin": "5057753747824",
                "image": "https://static.euw1.sustained.app/products/images/5057753747824_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14868a46711c1feab250e688887e9294",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14868a46711c1feab250e688887e9294/impacts"
                }
            },
            {
                "id": "productid.14d42e47f73f65396de6871d03160a2c",
                "name": "Taste the Difference British Beef Thin Cut Sirloin Steak",
                "gtin": "0000000327336",
                "image": "https://static.euw1.sustained.app/products/images/0000000327336_0.jpg",
                "pack": "155g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14d42e47f73f65396de6871d03160a2c",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14d42e47f73f65396de6871d03160a2c/impacts"
                }
            },
            {
                "id": "productid.15357fad5f833c4e99443a6bd6748148",
                "name": "SO Organic British Beef Rump Steak",
                "gtin": "0260420000008",
                "image": "https://static.euw1.sustained.app/products/images/0260420000008_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.15357fad5f833c4e99443a6bd6748148",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.15357fad5f833c4e99443a6bd6748148/impacts"
                }
            },
            {
                "id": "productid.1556fe27d7ce4cef8a19c60f5df91b32",
                "name": "Selected Cuts Corned Beef 4 Slices",
                "gtin": "5052003441354",
                "image": "https://static.euw1.sustained.app/products/images/5052003441354_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1556fe27d7ce4cef8a19c60f5df91b32",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1556fe27d7ce4cef8a19c60f5df91b32/impacts"
                }
            },
            {
                "id": "productid.1836b3ecc8c0d93f59d5ae9c90420421",
                "name": "British Beef Steak Medallions",
                "gtin": "5057545807613",
                "image": "https://static.euw1.sustained.app/products/images/5057545807613_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1836b3ecc8c0d93f59d5ae9c90420421",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1836b3ecc8c0d93f59d5ae9c90420421/impacts"
                }
            },
            {
                "id": "productid.1a1e07617b08868d790760f23462e260",
                "name": "Organic Diced Beef",
                "gtin": "5099536801866",
                "image": "https://static.euw1.sustained.app/products/images/5099536801866_0.jpg",
                "pack": "380g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1a1e07617b08868d790760f23462e260",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1a1e07617b08868d790760f23462e260/impacts"
                }
            },
            {
                "id": "productid.1a573597a7118cdb256785f6da258315",
                "name": "Irish Beef Round Steak",
                "gtin": "5057967692828",
                "image": "https://static.euw1.sustained.app/products/images/5057967692828_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1a573597a7118cdb256785f6da258315",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1a573597a7118cdb256785f6da258315/impacts"
                }
            },
            {
                "id": "productid.1c28b7614d15201651e3e1f3d1af7b06",
                "name": "The Butcher's On Market Street British Beef Rib Eye Steak",
                "gtin": "5010251987079",
                "image": "https://static.euw1.sustained.app/products/images/5010251987079_0.jpg",
                "pack": "0.200kg",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c28b7614d15201651e3e1f3d1af7b06",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c28b7614d15201651e3e1f3d1af7b06/impacts"
                }
            },
            {
                "id": "productid.1e0e3988c001f6bb827fa8754b15dbf7",
                "name": "Essential Waitrose & Partners British Beef Rump Steak",
                "gtin": "5000169131411",
                "image": "https://static.euw1.sustained.app/products/images/5000169131411_0.jpg",
                "pack": "230g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e0e3988c001f6bb827fa8754b15dbf7",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e0e3988c001f6bb827fa8754b15dbf7/impacts"
                }
            },
            {
                "id": "productid.1e2ca377db44b68c07a1d0b47dacd72b",
                "name": "Finest Certified Irish Angus Beef Chateaubriand",
                "gtin": "0207900000000",
                "image": "https://static.euw1.sustained.app/products/images/0207900000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e2ca377db44b68c07a1d0b47dacd72b",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e2ca377db44b68c07a1d0b47dacd72b/impacts"
                }
            },
            {
                "id": "productid.1e9dcb38e0b2989abb5127c9b93444fd",
                "name": "5% Fat Beef Mince",
                "gtin": "00288187",
                "image": "https://static.euw1.sustained.app/products/images/00288187_0.jpg",
                "pack": "750g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e9dcb38e0b2989abb5127c9b93444fd",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e9dcb38e0b2989abb5127c9b93444fd/impacts"
                }
            },
            {
                "id": "productid.20306faca649d61c8fcdf1dbef5e5b52",
                "name": "Extra Special British Wagyu Rump Steak",
                "gtin": "05057172472574",
                "image": "https://static.euw1.sustained.app/products/images/05057172472574_0.jpg",
                "pack": "225g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.20306faca649d61c8fcdf1dbef5e5b52",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.20306faca649d61c8fcdf1dbef5e5b52/impacts"
                }
            },
            {
                "id": "productid.21b85d9f988a8ce56d9bd1c60743a010",
                "name": "Silverside Beef",
                "gtin": "5030450718618",
                "image": "https://static.euw1.sustained.app/products/images/5030450718618_0.jpg",
                "pack": "2kg",
                "grade": "G",
                "points": {
                    "pef": 225.93074704394266
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.21b85d9f988a8ce56d9bd1c60743a010",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.21b85d9f988a8ce56d9bd1c60743a010/impacts"
                }
            },
            {
                "id": "productid.22de0cd26b3956ae5938ca20c6416446",
                "name": "Butcher's Selection Beef Reduced Fat Mince",
                "gtin": "5052449754452",
                "image": "https://static.euw1.sustained.app/products/images/5052449754452_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.22de0cd26b3956ae5938ca20c6416446",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.22de0cd26b3956ae5938ca20c6416446/impacts"
                }
            },
            {
                "id": "productid.23f74ae5e51374df3428bf3e17f5f4d1",
                "name": "Irish Fillet Steak",
                "gtin": "4088600467030",
                "image": "https://static.euw1.sustained.app/products/images/4088600467030_0.jpg",
                "pack": "220g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.23f74ae5e51374df3428bf3e17f5f4d1",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.23f74ae5e51374df3428bf3e17f5f4d1/impacts"
                }
            },
            {
                "id": "productid.2488a85e932afba57fb6ed5a3830df60",
                "name": "Northern Irish Beef Rump Steak",
                "gtin": "0000000199421",
                "image": "https://static.euw1.sustained.app/products/images/0000000199421_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2488a85e932afba57fb6ed5a3830df60",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2488a85e932afba57fb6ed5a3830df60/impacts"
                }
            },
            {
                "id": "productid.26fffcf56dbb171556aa691239ec533d",
                "name": "British Diced Beef Shin",
                "gtin": "0000000295031",
                "image": "https://static.euw1.sustained.app/products/images/0000000295031_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.26fffcf56dbb171556aa691239ec533d",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.26fffcf56dbb171556aa691239ec533d/impacts"
                }
            },
            {
                "id": "productid.27a27132c86b96b204f9a88369875ca3",
                "name": "The Butcher's On Market Street Lean British Beef Medallion Steaks",
                "gtin": "5010251797647",
                "image": "https://static.euw1.sustained.app/products/images/5010251797647_0.jpg",
                "pack": "0.340kg",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.27a27132c86b96b204f9a88369875ca3",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.27a27132c86b96b204f9a88369875ca3/impacts"
                }
            },
            {
                "id": "productid.289c3a15edf29d55060598b50427f7e4",
                "name": "Taste the Difference British Beef Ultimate Rump Steak",
                "gtin": "0211860000004",
                "image": "https://static.euw1.sustained.app/products/images/0211860000004_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.289c3a15edf29d55060598b50427f7e4",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.289c3a15edf29d55060598b50427f7e4/impacts"
                }
            },
            {
                "id": "productid.28a2b7d263230ba387045a1919a7332d",
                "name": "Our Family Butcher British Beef Roasting Joint Large",
                "gtin": "2334710000000",
                "image": "https://static.euw1.sustained.app/products/images/2334710000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.28a2b7d263230ba387045a1919a7332d",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.28a2b7d263230ba387045a1919a7332d/impacts"
                }
            },
            {
                "id": "productid.2974ee5d773f6b18d624170b3505db5a",
                "name": "Butcher's Selection 2 Soy & Black Pepper Beef Steaks",
                "gtin": "5057172580415",
                "image": "https://static.euw1.sustained.app/products/images/5057172580415_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 268.9907409225208
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2974ee5d773f6b18d624170b3505db5a",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2974ee5d773f6b18d624170b3505db5a/impacts"
                }
            },
            {
                "id": "productid.29fd2a73b7e4ae401b5e2eba478b9614",
                "name": "Succulent Beef Fillet Steak",
                "gtin": "0290960000008",
                "image": "https://static.euw1.sustained.app/products/images/0290960000008_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.29fd2a73b7e4ae401b5e2eba478b9614",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.29fd2a73b7e4ae401b5e2eba478b9614/impacts"
                }
            },
            {
                "id": "productid.2a84cc08ff344e1346c0be436e9efc57",
                "name": "Finest West Cork Dry Aged Angus Ribeye Steak",
                "gtin": "5057753467517",
                "image": "https://static.euw1.sustained.app/products/images/5057753467517_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a84cc08ff344e1346c0be436e9efc57",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a84cc08ff344e1346c0be436e9efc57/impacts"
                }
            },
            {
                "id": "productid.2aa8c7d5fe5b0cab8ae38c7abb8e7716",
                "name": "Extra Lean Steak Pieces",
                "gtin": "5060393476115",
                "image": "https://static.euw1.sustained.app/products/images/5060393476115_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2aa8c7d5fe5b0cab8ae38c7abb8e7716",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2aa8c7d5fe5b0cab8ae38c7abb8e7716/impacts"
                }
            },
            {
                "id": "productid.2d3427131e1223eec5d1823cf3bebcf9",
                "name": "Finest Beef Fillet",
                "gtin": "0231590000000",
                "image": "https://static.euw1.sustained.app/products/images/0231590000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2d3427131e1223eec5d1823cf3bebcf9",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2d3427131e1223eec5d1823cf3bebcf9/impacts"
                }
            },
            {
                "id": "productid.313cb9ac958a232c5241b855411a765e",
                "name": "Succulent Beef Sirloin Steak",
                "gtin": "5052449753714",
                "image": "https://static.euw1.sustained.app/products/images/5052449753714_0.jpg",
                "pack": "227g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.313cb9ac958a232c5241b855411a765e",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.313cb9ac958a232c5241b855411a765e/impacts"
                }
            },
            {
                "id": "productid.3566ca2ca9b5a6a93729d740281f4aa5",
                "name": "Irish Beef Ribeye Steaks",
                "gtin": "5057753882839",
                "image": "https://static.euw1.sustained.app/products/images/5057753882839_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3566ca2ca9b5a6a93729d740281f4aa5",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3566ca2ca9b5a6a93729d740281f4aa5/impacts"
                }
            },
            {
                "id": "productid.3845915405784231f9e66f71993e6751",
                "name": "British Diced Beef",
                "gtin": "5057545807538",
                "image": "https://static.euw1.sustained.app/products/images/5057545807538_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3845915405784231f9e66f71993e6751",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3845915405784231f9e66f71993e6751/impacts"
                }
            },
            {
                "id": "productid.393733956592816945ed03aa1b7baa45",
                "name": "Organic British Beef Rump Steak",
                "gtin": "0247370000000",
                "image": "https://static.euw1.sustained.app/products/images/0247370000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.393733956592816945ed03aa1b7baa45",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.393733956592816945ed03aa1b7baa45/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.007ca6dfa8cd6fb18926a7a1f5d7c8ba",
                "name": "Thatchers Green Goblin Somerset Dry Cider",
                "gtin": "5020628002359",
                "image": "https://static.euw1.sustained.app/products/images/5020628002359_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 13.50865670029716
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.007ca6dfa8cd6fb18926a7a1f5d7c8ba",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.007ca6dfa8cd6fb18926a7a1f5d7c8ba/impacts"
                }
            },
            {
                "id": "productid.00c3068ba5ce79d840ef321de5d59f33",
                "name": "Magners Original",
                "gtin": "5391516871563",
                "image": "https://static.euw1.sustained.app/products/images/5391516871563_0.jpg",
                "pack": "568ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00c3068ba5ce79d840ef321de5d59f33",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00c3068ba5ce79d840ef321de5d59f33/impacts"
                }
            },
            {
                "id": "productid.00d7dcdfa8efd002a193d64ff62d8195",
                "name": "Asda Biere De Luxe Premium French Lager",
                "gtin": "5054781212126",
                "image": "https://static.euw1.sustained.app/products/images/5054781212126_0.jpg",
                "pack": "25cl",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00d7dcdfa8efd002a193d64ff62d8195",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00d7dcdfa8efd002a193d64ff62d8195/impacts"
                }
            },
            {
                "id": "productid.020b68e2a03c98140369e91af348588d",
                "name": "Kopparberg Premium Cider with Strawberry & Lime Alcohol Free",
                "gtin": "7393714518989",
                "image": "https://static.euw1.sustained.app/products/images/7393714518989_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.020b68e2a03c98140369e91af348588d",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.020b68e2a03c98140369e91af348588d/impacts"
                }
            },
            {
                "id": "productid.026713bc8fa11605f205b1a5594068d9",
                "name": "Pilsner Urquell The Original Pilsner",
                "gtin": "8594404010847",
                "image": "https://static.euw1.sustained.app/products/images/8594404010847_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.026713bc8fa11605f205b1a5594068d9",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.026713bc8fa11605f205b1a5594068d9/impacts"
                }
            },
            {
                "id": "productid.02930bb157e509e4f83d2deba7aa1428",
                "name": "Purity Brewing Co. Bunny Hop Unfiltered Pale Ale",
                "gtin": "5060140900375",
                "image": "https://static.euw1.sustained.app/products/images/5060140900375_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02930bb157e509e4f83d2deba7aa1428",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02930bb157e509e4f83d2deba7aa1428/impacts"
                }
            },
            {
                "id": "productid.029e98823dff0e95c925af73b54163f3",
                "name": "Camden Town Brewery Beer Imperial Pilsner",
                "gtin": "5060228273490",
                "image": "https://static.euw1.sustained.app/products/images/5060228273490_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.029e98823dff0e95c925af73b54163f3",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.029e98823dff0e95c925af73b54163f3/impacts"
                }
            },
            {
                "id": "productid.02c45d9217d535865e155e3c0a012b96",
                "name": "Stella Artois Belgium Premium Lager",
                "gtin": "5014379012859",
                "image": "https://static.euw1.sustained.app/products/images/5014379012859_0.jpg",
                "pack": "284ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02c45d9217d535865e155e3c0a012b96",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02c45d9217d535865e155e3c0a012b96/impacts"
                }
            },
            {
                "id": "productid.02ef715a0022377e5ba77caa124f4eb6",
                "name": "Super Bock Premium Imported Beer",
                "gtin": "5601164114401",
                "image": "https://static.euw1.sustained.app/products/images/5601164114401_0.jpg",
                "pack": "66cl",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02ef715a0022377e5ba77caa124f4eb6",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02ef715a0022377e5ba77caa124f4eb6/impacts"
                }
            },
            {
                "id": "productid.0365c90e475e669f541979286af3e3f4",
                "name": "Corona Lager Beer Bottle",
                "gtin": "5014379014457",
                "image": "https://static.euw1.sustained.app/products/images/5014379014457_0.jpg",
                "pack": "620ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0365c90e475e669f541979286af3e3f4",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0365c90e475e669f541979286af3e3f4/impacts"
                }
            },
            {
                "id": "productid.03cc356707ae8dd7c26825bcba8cef9e",
                "name": "Peroni Nastro Azzurro",
                "gtin": "5023837000697",
                "image": "https://static.euw1.sustained.app/products/images/5023837000697_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03cc356707ae8dd7c26825bcba8cef9e",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03cc356707ae8dd7c26825bcba8cef9e/impacts"
                }
            },
            {
                "id": "productid.04a1d6036030d3076399663bf76b4eaa",
                "name": "Lerwick Brewery Lerwick IPA",
                "gtin": "5060347790168",
                "image": "https://static.euw1.sustained.app/products/images/5060347790168_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04a1d6036030d3076399663bf76b4eaa",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04a1d6036030d3076399663bf76b4eaa/impacts"
                }
            },
            {
                "id": "productid.04b3579759271a479cc5e1d09811053a",
                "name": "Roadworks Brewing Co. Pale Ale",
                "gtin": "5391528373376",
                "image": "https://static.euw1.sustained.app/products/images/5391528373376_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04b3579759271a479cc5e1d09811053a",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04b3579759271a479cc5e1d09811053a/impacts"
                }
            },
            {
                "id": "productid.04f85903a3b266cf21ad6bc21b534d0b",
                "name": "Cloudwater West Coast IPA",
                "gtin": "5056025460553",
                "image": "https://static.euw1.sustained.app/products/images/5056025460553_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04f85903a3b266cf21ad6bc21b534d0b",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04f85903a3b266cf21ad6bc21b534d0b/impacts"
                }
            },
            {
                "id": "productid.061ee00c294adcff4ad3f39c67531b15",
                "name": "Stewart Brewing Hollyrood Pale Ale",
                "gtin": "5060143110047",
                "image": "https://static.euw1.sustained.app/products/images/5060143110047_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.061ee00c294adcff4ad3f39c67531b15",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.061ee00c294adcff4ad3f39c67531b15/impacts"
                }
            },
            {
                "id": "productid.0635358d650a841d5581f4771917efc9",
                "name": "Dartmoor Brewery Dragon's Breath",
                "gtin": "5032970000693",
                "image": "https://static.euw1.sustained.app/products/images/5032970000693_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0635358d650a841d5581f4771917efc9",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0635358d650a841d5581f4771917efc9/impacts"
                }
            },
            {
                "id": "productid.06660682f6db24bd833a140be54ebaa6",
                "name": "Guinness Draught Stout Beer",
                "gtin": "5000213101025",
                "image": "https://static.euw1.sustained.app/products/images/5000213101025_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.114109174210071
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06660682f6db24bd833a140be54ebaa6",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06660682f6db24bd833a140be54ebaa6/impacts"
                }
            },
            {
                "id": "productid.06c47a559473d7ecba157f2ba74f3f07",
                "name": "Porterhouse Brew Co. Renegade",
                "gtin": "5391518560748",
                "image": "https://static.euw1.sustained.app/products/images/5391518560748_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.114109174210071
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06c47a559473d7ecba157f2ba74f3f07",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06c47a559473d7ecba157f2ba74f3f07/impacts"
                }
            },
            {
                "id": "productid.06c9c5ab8e76d9988a15d65ba2181e90",
                "name": "Blackthorn Reserve",
                "gtin": "5391520442575",
                "image": "https://static.euw1.sustained.app/products/images/5391520442575_0.jpg",
                "pack": "3l",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06c9c5ab8e76d9988a15d65ba2181e90",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06c9c5ab8e76d9988a15d65ba2181e90/impacts"
                }
            },
            {
                "id": "productid.06f7c7d391a926c55dd28e44a37c00f5",
                "name": "Wychwood Brewery Hobgoblin IPA",
                "gtin": "5011348022758",
                "image": "https://static.euw1.sustained.app/products/images/5011348022758_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06f7c7d391a926c55dd28e44a37c00f5",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06f7c7d391a926c55dd28e44a37c00f5/impacts"
                }
            },
            {
                "id": "productid.07075381e9557c5972f11d35026f6b3e",
                "name": "Camden Show Off Juicy Lager",
                "gtin": "5060228271496",
                "image": "https://static.euw1.sustained.app/products/images/5060228271496_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07075381e9557c5972f11d35026f6b3e",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07075381e9557c5972f11d35026f6b3e/impacts"
                }
            },
            {
                "id": "productid.0786d854539be1991f46799fdd8ce3d4",
                "name": "Krombacher Pils",
                "gtin": "4008287058628",
                "image": "https://static.euw1.sustained.app/products/images/4008287058628_0.jpg",
                "pack": "660ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0786d854539be1991f46799fdd8ce3d4",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0786d854539be1991f46799fdd8ce3d4/impacts"
                }
            },
            {
                "id": "productid.07bc2b6e0c41b39a4befddaa814ea151",
                "name": "Brooklyn Brewery Brooklyn Pilsner Crisp Lager",
                "gtin": "5016325851475",
                "image": "https://static.euw1.sustained.app/products/images/5016325851475_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07bc2b6e0c41b39a4befddaa814ea151",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07bc2b6e0c41b39a4befddaa814ea151/impacts"
                }
            },
            {
                "id": "productid.07efaa4b4ca19b2a9b77fc102ac6002b",
                "name": "Loch Ness Brewery Light Ness Session Pale Ale",
                "gtin": "5034866026039",
                "image": "https://static.euw1.sustained.app/products/images/5034866026039_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07efaa4b4ca19b2a9b77fc102ac6002b",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07efaa4b4ca19b2a9b77fc102ac6002b/impacts"
                }
            },
            {
                "id": "productid.0852efada9c1317f70980fe14420db08",
                "name": "Franciscan Well Rebel Red Ale",
                "gtin": "5010038467275",
                "image": "https://static.euw1.sustained.app/products/images/5010038467275_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0852efada9c1317f70980fe14420db08",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0852efada9c1317f70980fe14420db08/impacts"
                }
            },
            {
                "id": "productid.086e7a1c64bd6caafacc21b94c58ccef",
                "name": "Urban Fox Cunningly Distinctive American Red IPA",
                "gtin": "5021984109157",
                "image": "https://static.euw1.sustained.app/products/images/5021984109157_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.086e7a1c64bd6caafacc21b94c58ccef",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.086e7a1c64bd6caafacc21b94c58ccef/impacts"
                }
            },
            {
                "id": "productid.08a16f70d16c65e626b362e660fa10f0",
                "name": "Peroni Nastro Azzurro Lager Beer Bottle",
                "gtin": "8008440112286",
                "image": "https://static.euw1.sustained.app/products/images/8008440112286_0.jpg",
                "pack": "250ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08a16f70d16c65e626b362e660fa10f0",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08a16f70d16c65e626b362e660fa10f0/impacts"
                }
            },
            {
                "id": "productid.092ffe619af5d97396cbd7e042342623",
                "name": "Fyne Ales Workbench IPA",
                "gtin": "5060041571117",
                "image": "https://static.euw1.sustained.app/products/images/5060041571117_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.092ffe619af5d97396cbd7e042342623",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.092ffe619af5d97396cbd7e042342623/impacts"
                }
            },
            {
                "id": "productid.0ab7031fbb4dfd3b92a667230c9f0101",
                "name": "Hop House 13 Lager Beer",
                "gtin": "5000213023662",
                "image": "https://static.euw1.sustained.app/products/images/5000213023662_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ab7031fbb4dfd3b92a667230c9f0101",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ab7031fbb4dfd3b92a667230c9f0101/impacts"
                }
            },
            {
                "id": "productid.0aeca0853f1daa62e224808df2603d1c",
                "name": "Session Ale",
                "gtin": "00417266",
                "image": "https://static.euw1.sustained.app/products/images/00417266_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0aeca0853f1daa62e224808df2603d1c",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0aeca0853f1daa62e224808df2603d1c/impacts"
                }
            },
            {
                "id": "productid.0af86014f4ba489ce2232019ee9ce268",
                "name": "Stella Artois Cidre",
                "gtin": "5014379001587",
                "image": "https://static.euw1.sustained.app/products/images/5014379001587_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0af86014f4ba489ce2232019ee9ce268",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0af86014f4ba489ce2232019ee9ce268/impacts"
                }
            },
            {
                "id": "productid.0af9fcd70f295802d68516ae69276cb2",
                "name": "Dutch Gold",
                "gtin": "5390347300754",
                "image": "https://static.euw1.sustained.app/products/images/5390347300754_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0af9fcd70f295802d68516ae69276cb2",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0af9fcd70f295802d68516ae69276cb2/impacts"
                }
            },
            {
                "id": "productid.0b1474ff47be42beb85a5e393966eac6",
                "name": "Goose Island 312 Urban Wheat Ale Craft Beer Bottles",
                "gtin": "0736920111839",
                "image": "https://static.euw1.sustained.app/products/images/0736920111839_0.jpg",
                "pack": "355ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b1474ff47be42beb85a5e393966eac6",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b1474ff47be42beb85a5e393966eac6/impacts"
                }
            },
            {
                "id": "productid.0b9a882852ba1f7bb375032f75fa4ed2",
                "name": "Old Mout Cider Pineapple & Raspberry",
                "gtin": "5411718912809",
                "image": "https://static.euw1.sustained.app/products/images/5411718912809_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b9a882852ba1f7bb375032f75fa4ed2",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b9a882852ba1f7bb375032f75fa4ed2/impacts"
                }
            },
            {
                "id": "productid.0bef39b16c63447dacbc4d246739374b",
                "name": "Morrisons Lager Continental",
                "gtin": "5010251157212",
                "image": "https://static.euw1.sustained.app/products/images/5010251157212_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bef39b16c63447dacbc4d246739374b",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bef39b16c63447dacbc4d246739374b/impacts"
                }
            },
            {
                "id": "productid.0c05bc2aa018bb92254b8e1f50b3fea8",
                "name": "Open Gate Brewery Pure Brew Non-Alcoholic Lager",
                "gtin": "5000213021347",
                "image": "https://static.euw1.sustained.app/products/images/5000213021347_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.114109174210057
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c05bc2aa018bb92254b8e1f50b3fea8",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c05bc2aa018bb92254b8e1f50b3fea8/impacts"
                }
            },
            {
                "id": "productid.0c76ac5208035ba1c702b24b11e1e84b",
                "name": "The Vermont Sessions New England IPA Brewdog vs Northern Monk",
                "gtin": "5056025436879",
                "image": "https://static.euw1.sustained.app/products/images/5056025436879_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c76ac5208035ba1c702b24b11e1e84b",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c76ac5208035ba1c702b24b11e1e84b/impacts"
                }
            },
            {
                "id": "productid.0ce8d36c96d27a0174e5e1faf0e571c4",
                "name": "BrewDog Hazy Jane New England IPA",
                "gtin": "5056025434516",
                "image": "https://static.euw1.sustained.app/products/images/5056025434516_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ce8d36c96d27a0174e5e1faf0e571c4",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ce8d36c96d27a0174e5e1faf0e571c4/impacts"
                }
            },
            {
                "id": "productid.0d4a0c290d0a53f72dafca1fc326575d",
                "name": "Heineken",
                "gtin": "5035766064404",
                "image": "https://static.euw1.sustained.app/products/images/5035766064404_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d4a0c290d0a53f72dafca1fc326575d",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d4a0c290d0a53f72dafca1fc326575d/impacts"
                }
            },
            {
                "id": "productid.0e0438c02fe9b7b709e174d751b37ca1",
                "name": "Stowford Press Apple Cider PMP",
                "gtin": "5014201701562",
                "image": "https://static.euw1.sustained.app/products/images/5014201701562_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e0438c02fe9b7b709e174d751b37ca1",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e0438c02fe9b7b709e174d751b37ca1/impacts"
                }
            },
            {
                "id": "productid.0ed22eb4af85bc4206f38047336d2630",
                "name": "Thatchers Gold Cider",
                "gtin": "5020628001178",
                "image": "https://static.euw1.sustained.app/products/images/5020628001178_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ed22eb4af85bc4206f38047336d2630",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ed22eb4af85bc4206f38047336d2630/impacts"
                }
            },
            {
                "id": "productid.10030e53f32408faa7623f5f603802ac",
                "name": "Peroni Nastro Azzurro",
                "gtin": "8008440322029",
                "image": "https://static.euw1.sustained.app/products/images/8008440322029_0.jpg",
                "pack": "50l",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10030e53f32408faa7623f5f603802ac",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10030e53f32408faa7623f5f603802ac/impacts"
                }
            },
            {
                "id": "productid.1070c4101b801e9a618cc6a056d72f66",
                "name": "Stella Artois Lager Beer Cans",
                "gtin": "5010017109851",
                "image": "https://static.euw1.sustained.app/products/images/5010017109851_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1070c4101b801e9a618cc6a056d72f66",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1070c4101b801e9a618cc6a056d72f66/impacts"
                }
            },
            {
                "id": "productid.10bb3c253b7e1297ef60e2e29c7c659d",
                "name": "Woodpecker Cider",
                "gtin": "5035766040125",
                "image": "https://static.euw1.sustained.app/products/images/5035766040125_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10bb3c253b7e1297ef60e2e29c7c659d",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10bb3c253b7e1297ef60e2e29c7c659d/impacts"
                }
            },
            {
                "id": "productid.116bef912234ed18dcd2cfd7f0d66023",
                "name": "Oakham Inferno Blonde Ale",
                "gtin": "5060044870071",
                "image": "https://static.euw1.sustained.app/products/images/5060044870071_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.116bef912234ed18dcd2cfd7f0d66023",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.116bef912234ed18dcd2cfd7f0d66023/impacts"
                }
            },
            {
                "id": "productid.11bad0dbc56da5abcb3f8f511c654a69",
                "name": "Coors Lager",
                "gtin": "5010038447475",
                "image": "https://static.euw1.sustained.app/products/images/5010038447475_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11bad0dbc56da5abcb3f8f511c654a69",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11bad0dbc56da5abcb3f8f511c654a69/impacts"
                }
            },
            {
                "id": "productid.11c4bc7fd46d353ffd8abdc0c301c1c1",
                "name": "Asda West Coast Pale",
                "gtin": "5060424341474",
                "image": "https://static.euw1.sustained.app/products/images/5060424341474_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11c4bc7fd46d353ffd8abdc0c301c1c1",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11c4bc7fd46d353ffd8abdc0c301c1c1/impacts"
                }
            },
            {
                "id": "productid.120d5c9be9476be7aa6b84ef5a485e53",
                "name": "Triple Secret Des Moines Blonde",
                "gtin": "3347970000270",
                "image": "https://static.euw1.sustained.app/products/images/3347970000270_0.jpg",
                "pack": "75cl",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.120d5c9be9476be7aa6b84ef5a485e53",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.120d5c9be9476be7aa6b84ef5a485e53/impacts"
                }
            },
            {
                "id": "productid.12376d90b1bad4b4af052226c824d855",
                "name": "Bulmers Irish Cider Original",
                "gtin": "5391516870054",
                "image": "https://static.euw1.sustained.app/products/images/5391516870054_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12376d90b1bad4b4af052226c824d855",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12376d90b1bad4b4af052226c824d855/impacts"
                }
            },
            {
                "id": "productid.124fc36853b4f1109f181edf9af4f2c0",
                "name": "Peroni Nastro Azzurro",
                "gtin": "8008440222015",
                "image": "https://static.euw1.sustained.app/products/images/8008440222015_0.jpg",
                "pack": "660ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.124fc36853b4f1109f181edf9af4f2c0",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.124fc36853b4f1109f181edf9af4f2c0/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.003034a2555bf84b93a07c217f7638fc",
                "name": "Collection Italian Luxury Biscuit Collection",
                "gtin": "0000029078691",
                "image": "https://static.euw1.sustained.app/products/images/0000029078691_0.jpg",
                "pack": "420g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.003034a2555bf84b93a07c217f7638fc",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.003034a2555bf84b93a07c217f7638fc/impacts"
                }
            },
            {
                "id": "productid.005df910da5b45dfaeb0efa2890bff5f",
                "name": "Gluten-Free Dark Chocolate Shorts",
                "gtin": "8008698005347",
                "image": "https://static.euw1.sustained.app/products/images/8008698005347_0.jpg",
                "pack": "150g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.005df910da5b45dfaeb0efa2890bff5f",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.005df910da5b45dfaeb0efa2890bff5f/impacts"
                }
            },
            {
                "id": "productid.0133cccd72096c4a2fd6498aefb72f4b",
                "name": "Choc Chip Cookies",
                "gtin": "5051777000033",
                "image": "https://static.euw1.sustained.app/products/images/5051777000033_0.jpg",
                "pack": "150g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0133cccd72096c4a2fd6498aefb72f4b",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0133cccd72096c4a2fd6498aefb72f4b/impacts"
                }
            },
            {
                "id": "productid.013a2c6e1fea53858d5dd8afcfe6791e",
                "name": "Malted Milk",
                "gtin": "5010204765167",
                "image": "https://static.euw1.sustained.app/products/images/5010204765167_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 44.866185785241875
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.013a2c6e1fea53858d5dd8afcfe6791e",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.013a2c6e1fea53858d5dd8afcfe6791e/impacts"
                }
            },
            {
                "id": "productid.022c60c0048255fdf38c3ef23d9be651",
                "name": "Pure Butter Shortbread",
                "gtin": "0039047001138",
                "image": "https://static.euw1.sustained.app/products/images/0039047001138_0.jpg",
                "pack": "160g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.022c60c0048255fdf38c3ef23d9be651",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.022c60c0048255fdf38c3ef23d9be651/impacts"
                }
            },
            {
                "id": "productid.02b0fae19a25799db87b413c06bea102",
                "name": "Deliciously Free From Shortbread",
                "gtin": "0000000227681",
                "image": "https://static.euw1.sustained.app/products/images/0000000227681_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02b0fae19a25799db87b413c06bea102",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02b0fae19a25799db87b413c06bea102/impacts"
                }
            },
            {
                "id": "productid.02bd7ce7ef842ae8f7518068a9ddadb4",
                "name": "Food Multiseed Wholegrain Flatbreads",
                "gtin": "0000000139656",
                "image": "https://static.euw1.sustained.app/products/images/0000000139656_0.jpg",
                "pack": "150g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02bd7ce7ef842ae8f7518068a9ddadb4",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02bd7ce7ef842ae8f7518068a9ddadb4/impacts"
                }
            },
            {
                "id": "productid.02dd5fec55cc86ee4e8eef204e5eeb82",
                "name": "Yogurt Breaks Forest Fruit Biscuit Bars (5x35.5g)",
                "gtin": "5000168181578",
                "image": "https://static.euw1.sustained.app/products/images/5000168181578_0.jpg",
                "pack": "35.5g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02dd5fec55cc86ee4e8eef204e5eeb82",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02dd5fec55cc86ee4e8eef204e5eeb82/impacts"
                }
            },
            {
                "id": "productid.035b13e9769bb33a82d05e04939b3f87",
                "name": "Shortbread Fingers",
                "gtin": "20337148",
                "image": "https://static.euw1.sustained.app/products/images/20337148_0.jpg",
                "pack": "210g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.035b13e9769bb33a82d05e04939b3f87",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.035b13e9769bb33a82d05e04939b3f87/impacts"
                }
            },
            {
                "id": "productid.038e62319518af2c44f772ba0a814a20",
                "name": "Creations 205 Pompadour Fans Catering Pack",
                "gtin": "5029728545001",
                "image": "https://static.euw1.sustained.app/products/images/5029728545001_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.038e62319518af2c44f772ba0a814a20",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.038e62319518af2c44f772ba0a814a20/impacts"
                }
            },
            {
                "id": "productid.03beb985652e1de6bd785dcca154fdb9",
                "name": "The Best All Butter Scottish Shortbread Selection",
                "gtin": "5010525092553",
                "image": "https://static.euw1.sustained.app/products/images/5010525092553_0.jpg",
                "pack": "500g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03beb985652e1de6bd785dcca154fdb9",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03beb985652e1de6bd785dcca154fdb9/impacts"
                }
            },
            {
                "id": "productid.03d7f357f1b955f37985e60bc2e21d49",
                "name": "Pick Up! Choco",
                "gtin": "4017100263101",
                "image": "https://static.euw1.sustained.app/products/images/4017100263101_0.jpg",
                "pack": "28g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03d7f357f1b955f37985e60bc2e21d49",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03d7f357f1b955f37985e60bc2e21d49/impacts"
                }
            },
            {
                "id": "productid.03f00c33b5ff9fdd28959506999c8f9d",
                "name": "Crispy Milk-Hazelnut Wafer",
                "gtin": "0000040144382",
                "image": "https://static.euw1.sustained.app/products/images/0000040144382_0.jpg",
                "pack": "25g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03f00c33b5ff9fdd28959506999c8f9d",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03f00c33b5ff9fdd28959506999c8f9d/impacts"
                }
            },
            {
                "id": "productid.043732e571e493f4d9ad09819ef9c739",
                "name": "Chocolate Soft Baked Bears 8 Pack",
                "gtin": "7622201415600",
                "image": "https://static.euw1.sustained.app/products/images/7622201415600_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.043732e571e493f4d9ad09819ef9c739",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.043732e571e493f4d9ad09819ef9c739/impacts"
                }
            },
            {
                "id": "productid.04ffc95a321f0d885f3e1f7858819ec7",
                "name": "Food Collection Belgian Chocolate Caf\u00e9 Cream Biscuits",
                "gtin": "0000029063635",
                "image": "https://static.euw1.sustained.app/products/images/0000029063635_0.jpg",
                "pack": "430g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04ffc95a321f0d885f3e1f7858819ec7",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04ffc95a321f0d885f3e1f7858819ec7/impacts"
                }
            },
            {
                "id": "productid.05a8bddea324b3a83f6293c984a171a7",
                "name": "Original Sourdough Crispbread",
                "gtin": "5060198820939",
                "image": "https://static.euw1.sustained.app/products/images/5060198820939_0.jpg",
                "pack": "140g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05a8bddea324b3a83f6293c984a171a7",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05a8bddea324b3a83f6293c984a171a7/impacts"
                }
            },
            {
                "id": "productid.05edbb86c0737b748f03cc8d12ea99e1",
                "name": "Delicious Shortbread Assortment",
                "gtin": "5010745055604",
                "image": "https://static.euw1.sustained.app/products/images/5010745055604_0.jpg",
                "pack": "1kg",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05edbb86c0737b748f03cc8d12ea99e1",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05edbb86c0737b748f03cc8d12ea99e1/impacts"
                }
            },
            {
                "id": "productid.0636e1a3e81704d9427cd6ecfc2be618",
                "name": "6 Rice Cakes Yoghurt",
                "gtin": "5060262484951",
                "image": "https://static.euw1.sustained.app/products/images/5060262484951_0.jpg",
                "pack": "100g",
                "grade": "D",
                "points": {
                    "pef": 26.62527639287613
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0636e1a3e81704d9427cd6ecfc2be618",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0636e1a3e81704d9427cd6ecfc2be618/impacts"
                }
            },
            {
                "id": "productid.0639c62060ec0cb6e25e04089d8b1483",
                "name": "All Butter Viennese Raspberry Creams",
                "gtin": "0000000028356",
                "image": "https://static.euw1.sustained.app/products/images/0000000028356_0.jpg",
                "pack": "125g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0639c62060ec0cb6e25e04089d8b1483",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0639c62060ec0cb6e25e04089d8b1483/impacts"
                }
            },
            {
                "id": "productid.066c86c21ef85898935fe6fa36aa72e6",
                "name": "Yoghurt Topped Rice Cakes",
                "gtin": "5013665113072",
                "image": "https://static.euw1.sustained.app/products/images/5013665113072_0.jpg",
                "pack": "100g",
                "grade": "D",
                "points": {
                    "pef": 26.57640402747611
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.066c86c21ef85898935fe6fa36aa72e6",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.066c86c21ef85898935fe6fa36aa72e6/impacts"
                }
            },
            {
                "id": "productid.068b71f8a635bdf1d06b3e4150249bdc",
                "name": "No1 Belgian Dark Dipped Chocolate Shortbread",
                "gtin": "5000169571538",
                "image": "https://static.euw1.sustained.app/products/images/5000169571538_0.jpg",
                "pack": "135g",
                "grade": "F",
                "points": {
                    "pef": 77.35102804145704
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.068b71f8a635bdf1d06b3e4150249bdc",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.068b71f8a635bdf1d06b3e4150249bdc/impacts"
                }
            },
            {
                "id": "productid.06c3db787602bfd2c1f5bf8d3f5c7334",
                "name": "Chocolate Biscuit Fingers Multipack",
                "gtin": "5000159430678",
                "image": "https://static.euw1.sustained.app/products/images/5000159430678_0.jpg",
                "pack": "23g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06c3db787602bfd2c1f5bf8d3f5c7334",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06c3db787602bfd2c1f5bf8d3f5c7334/impacts"
                }
            },
            {
                "id": "productid.06ddca5d114fefe2f1c003c4ede20ef6",
                "name": "Sainsbury's Digestives",
                "gtin": "00448383",
                "image": "https://static.euw1.sustained.app/products/images/00448383_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06ddca5d114fefe2f1c003c4ede20ef6",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06ddca5d114fefe2f1c003c4ede20ef6/impacts"
                }
            },
            {
                "id": "productid.0700f12dd990be966ed6f7274f544a99",
                "name": "Cookies Milk Chocolate",
                "gtin": "7622210809049",
                "image": "https://static.euw1.sustained.app/products/images/7622210809049_0.jpg",
                "pack": "135g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0700f12dd990be966ed6f7274f544a99",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0700f12dd990be966ed6f7274f544a99/impacts"
                }
            },
            {
                "id": "productid.083965ff2e3fc8b185b1056d5aa2c277",
                "name": "Love Life Plain Breadsticks",
                "gtin": "5000169026335",
                "image": "https://static.euw1.sustained.app/products/images/5000169026335_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 11.56958710985014
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.083965ff2e3fc8b185b1056d5aa2c277",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.083965ff2e3fc8b185b1056d5aa2c277/impacts"
                }
            },
            {
                "id": "productid.0918005afa481e73f5c0551997ae56b1",
                "name": "Malted Milk Biscuits",
                "gtin": "5010251630814",
                "image": "https://static.euw1.sustained.app/products/images/5010251630814_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0918005afa481e73f5c0551997ae56b1",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0918005afa481e73f5c0551997ae56b1/impacts"
                }
            },
            {
                "id": "productid.092e69f58e6c0d38b8df6875d96debd5",
                "name": "Club Mint Bars",
                "gtin": "5000168210728",
                "image": "https://static.euw1.sustained.app/products/images/5000168210728_0.jpg",
                "pack": "22g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.092e69f58e6c0d38b8df6875d96debd5",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.092e69f58e6c0d38b8df6875d96debd5/impacts"
                }
            },
            {
                "id": "productid.0a15c36751a83be9ade37b807caa0101",
                "name": "Extra Special Dark Chocolate Ginger Thins",
                "gtin": "5057172728961",
                "image": "https://static.euw1.sustained.app/products/images/5057172728961_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 94.14747184106628
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a15c36751a83be9ade37b807caa0101",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a15c36751a83be9ade37b807caa0101/impacts"
                }
            },
            {
                "id": "productid.0a6fc224ce4746900dd22551b47b42ba",
                "name": "Craft Bakery 2 Cream Cookies",
                "gtin": "5029912014030",
                "image": "https://static.euw1.sustained.app/products/images/5029912014030_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 18.034080732353328
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a6fc224ce4746900dd22551b47b42ba",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a6fc224ce4746900dd22551b47b42ba/impacts"
                }
            },
            {
                "id": "productid.0ad874ffd1017b2914b255f9290547a1",
                "name": "Grissini Sesame Italian Breadsticks",
                "gtin": "8003899181203",
                "image": "https://static.euw1.sustained.app/products/images/8003899181203_0.jpg",
                "pack": "125g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ad874ffd1017b2914b255f9290547a1",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ad874ffd1017b2914b255f9290547a1/impacts"
                }
            },
            {
                "id": "productid.0b0362f230eda6b12cec220c8a68ea4d",
                "name": "Mummy and Me Heart Message Biscuit",
                "gtin": "5054402323286",
                "image": "https://static.euw1.sustained.app/products/images/5054402323286_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 44.866185785241875
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b0362f230eda6b12cec220c8a68ea4d",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b0362f230eda6b12cec220c8a68ea4d/impacts"
                }
            },
            {
                "id": "productid.0b569d59bc9faa84c31022f2c03d7a5b",
                "name": "Butterscotch Crunch",
                "gtin": "0652655001088",
                "image": "https://static.euw1.sustained.app/products/images/0652655001088_0.jpg",
                "pack": "135g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b569d59bc9faa84c31022f2c03d7a5b",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b569d59bc9faa84c31022f2c03d7a5b/impacts"
                }
            },
            {
                "id": "productid.0bc36de53138d9acae5bb9a8af069035",
                "name": "Baked Snack Crackers Cheddar",
                "gtin": "0014100085393",
                "image": "https://static.euw1.sustained.app/products/images/0014100085393_0.jpg",
                "pack": "187g",
                "grade": "D",
                "points": {
                    "pef": 22.516785086530177
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bc36de53138d9acae5bb9a8af069035",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bc36de53138d9acae5bb9a8af069035/impacts"
                }
            },
            {
                "id": "productid.0e0ef2cf154f10fdee8197bf5cad9ef6",
                "name": "Original Cream Crackers",
                "gtin": "5000137415970",
                "image": "https://static.euw1.sustained.app/products/images/5000137415970_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e0ef2cf154f10fdee8197bf5cad9ef6",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e0ef2cf154f10fdee8197bf5cad9ef6/impacts"
                }
            },
            {
                "id": "productid.0e3f93969d0b0e4d032db6f5a346a825",
                "name": "4 Milk Chocolate Chunk Cookies",
                "gtin": "5055552401350",
                "image": "https://static.euw1.sustained.app/products/images/5055552401350_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e3f93969d0b0e4d032db6f5a346a825",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e3f93969d0b0e4d032db6f5a346a825/impacts"
                }
            },
            {
                "id": "productid.0e5b4807ddb1766c966000cf1ccca7ed",
                "name": "Duchy Organic Orange Biscuits Coated in Dark Chocolate",
                "gtin": "5000169149812",
                "image": "https://static.euw1.sustained.app/products/images/5000169149812_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 86.76746947023173
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e5b4807ddb1766c966000cf1ccca7ed",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e5b4807ddb1766c966000cf1ccca7ed/impacts"
                }
            },
            {
                "id": "productid.0e8dae52bff272f8435e96a667b1aef2",
                "name": "Vegan Coconut Crunch Biscuit Share Bag",
                "gtin": "7640155340533",
                "image": "https://static.euw1.sustained.app/products/images/7640155340533_0.jpg",
                "pack": "135g",
                "grade": "E",
                "points": {
                    "pef": 33.04037957649413
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e8dae52bff272f8435e96a667b1aef2",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e8dae52bff272f8435e96a667b1aef2/impacts"
                }
            },
            {
                "id": "productid.0ee96205206c538695b9ff57c360eb81",
                "name": "Free From Cheese Crackers",
                "gtin": "5010251782940",
                "image": "https://static.euw1.sustained.app/products/images/5010251782940_0.jpg",
                "pack": "125g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ee96205206c538695b9ff57c360eb81",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ee96205206c538695b9ff57c360eb81/impacts"
                }
            },
            {
                "id": "productid.0fa1becbddf24763c8971c40c0029dbf",
                "name": "Oats & Grains Peanut & Caramel Bars",
                "gtin": "5056357907597",
                "image": "https://static.euw1.sustained.app/products/images/5056357907597_0.jpg",
                "pack": "22.5g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fa1becbddf24763c8971c40c0029dbf",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fa1becbddf24763c8971c40c0029dbf/impacts"
                }
            },
            {
                "id": "productid.101bba50ce38ab1302d792bf6f687f71",
                "name": "Peanut Cookies",
                "gtin": "8710508913529",
                "image": "https://static.euw1.sustained.app/products/images/8710508913529_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.101bba50ce38ab1302d792bf6f687f71",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.101bba50ce38ab1302d792bf6f687f71/impacts"
                }
            },
            {
                "id": "productid.103bacf7dd8de642b375b3b309c93d51",
                "name": "of Yorkshire Mini Snackers Herb & Onion Crackers",
                "gtin": "5010112004785",
                "image": "https://static.euw1.sustained.app/products/images/5010112004785_0.jpg",
                "pack": "85g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.103bacf7dd8de642b375b3b309c93d51",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.103bacf7dd8de642b375b3b309c93d51/impacts"
                }
            },
            {
                "id": "productid.10bba8814fa61996417b3d759b5c70a4",
                "name": "Easter Bunny Gingerbread Biscuit",
                "gtin": "5010525165844",
                "image": "https://static.euw1.sustained.app/products/images/5010525165844_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10bba8814fa61996417b3d759b5c70a4",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10bba8814fa61996417b3d759b5c70a4/impacts"
                }
            },
            {
                "id": "productid.10bc4be1779a273ad09d30430506a2f0",
                "name": "SMA LITTLE STEPS Organic Tomato Savoury Crackers 100g, Baby Snack 12 Months+, 16 Portions Per Bag",
                "gtin": "8000300395471",
                "image": "https://static.euw1.sustained.app/products/images/8000300395471_0.jpg",
                "pack": "100g",
                "grade": "C",
                "points": {
                    "pef": 18.168430987904163
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10bc4be1779a273ad09d30430506a2f0",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10bc4be1779a273ad09d30430506a2f0/impacts"
                }
            },
            {
                "id": "productid.12e1851469a45dccce46a6772f259e6e",
                "name": "Food Crispy Cheese Crackers",
                "gtin": "0000000130912",
                "image": "https://static.euw1.sustained.app/products/images/0000000130912_0.jpg",
                "pack": "150g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12e1851469a45dccce46a6772f259e6e",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12e1851469a45dccce46a6772f259e6e/impacts"
                }
            },
            {
                "id": "productid.135be51b2a008a210722e0f14edca4d2",
                "name": "Organic Rye Kn\u00e4cke Sesame Crispbread",
                "gtin": "5032722318137",
                "image": "https://static.euw1.sustained.app/products/images/5032722318137_0.jpg",
                "pack": "200g",
                "grade": "B",
                "points": {
                    "pef": 14.056211268632643
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.135be51b2a008a210722e0f14edca4d2",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.135be51b2a008a210722e0f14edca4d2/impacts"
                }
            },
            {
                "id": "productid.136755ac70ebdda47d35213a43ecda50",
                "name": "Original Rye Crispbread",
                "gtin": "5057008761377",
                "image": "https://static.euw1.sustained.app/products/images/5057008761377_0.jpg",
                "pack": "250g",
                "grade": "A",
                "points": {
                    "pef": 10.509467853519002
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.136755ac70ebdda47d35213a43ecda50",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.136755ac70ebdda47d35213a43ecda50/impacts"
                }
            },
            {
                "id": "productid.1385af39bef9fa635a6665be841845b4",
                "name": "Mini Rice Crackers Seaweed Flavour",
                "gtin": "4710144900154",
                "image": "https://static.euw1.sustained.app/products/images/4710144900154_0.jpg",
                "pack": "60g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1385af39bef9fa635a6665be841845b4",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1385af39bef9fa635a6665be841845b4/impacts"
                }
            },
            {
                "id": "productid.14e91296861e0bc1ee71849899721900",
                "name": "Jaffa Cakes 100% Extra Free",
                "gtin": "5011005056362",
                "image": "https://static.euw1.sustained.app/products/images/5011005056362_0.jpg",
                "pack": "294g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14e91296861e0bc1ee71849899721900",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14e91296861e0bc1ee71849899721900/impacts"
                }
            },
            {
                "id": "productid.155c347919fe3771dff57d50cecb1f20",
                "name": "Peri Peri Spicy Oatcakes",
                "gtin": "0612322001216",
                "image": "https://static.euw1.sustained.app/products/images/0612322001216_0.jpg",
                "pack": "200g",
                "grade": "C",
                "points": {
                    "pef": 16.446323728188773
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.155c347919fe3771dff57d50cecb1f20",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.155c347919fe3771dff57d50cecb1f20/impacts"
                }
            },
            {
                "id": "productid.167bd0cee4db61d147347b3827c315f0",
                "name": "Choccy Rocks Moofreesas",
                "gtin": "5060235831454",
                "image": "https://static.euw1.sustained.app/products/images/5060235831454_0.jpg",
                "pack": "35g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.167bd0cee4db61d147347b3827c315f0",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.167bd0cee4db61d147347b3827c315f0/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.0012124ec5e5a585c220476b669fa06f",
                "name": "4 Hot Cross Buns",
                "gtin": "5099023123440",
                "image": "https://static.euw1.sustained.app/products/images/5099023123440_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0012124ec5e5a585c220476b669fa06f",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0012124ec5e5a585c220476b669fa06f/impacts"
                }
            },
            {
                "id": "productid.006dbef4da76143355781a6a92214619",
                "name": "White Family Pan",
                "gtin": "5052320937400",
                "image": "https://static.euw1.sustained.app/products/images/5052320937400_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 11.56958710985014
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.006dbef4da76143355781a6a92214619",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.006dbef4da76143355781a6a92214619/impacts"
                }
            },
            {
                "id": "productid.00e067714b610934b223439f3f274e51",
                "name": "Chosen by You White Tin Loaf",
                "gtin": "0230010000008",
                "image": "https://static.euw1.sustained.app/products/images/0230010000008_0.jpg",
                "pack": "800g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00e067714b610934b223439f3f274e51",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00e067714b610934b223439f3f274e51/impacts"
                }
            },
            {
                "id": "productid.018ee793a1e1a8b2dc2755f7e545a5fa",
                "name": "Taste the Difference 4 Sesame Bagels",
                "gtin": "00434737",
                "image": "https://static.euw1.sustained.app/products/images/00434737_0.jpg",
                "pack": "320g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.018ee793a1e1a8b2dc2755f7e545a5fa",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.018ee793a1e1a8b2dc2755f7e545a5fa/impacts"
                }
            },
            {
                "id": "productid.01ad0a6e9b65e46c508a8b3b5bfe2c4c",
                "name": "6 Large Flour Tortillas",
                "gtin": "8410076470300",
                "image": "https://static.euw1.sustained.app/products/images/8410076470300_0.jpg",
                "pack": "350g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01ad0a6e9b65e46c508a8b3b5bfe2c4c",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01ad0a6e9b65e46c508a8b3b5bfe2c4c/impacts"
                }
            },
            {
                "id": "productid.022015f8a91e14d51269573278607341",
                "name": "Multigrain Batch",
                "gtin": "5057545928912",
                "image": "https://static.euw1.sustained.app/products/images/5057545928912_0.jpg",
                "pack": "800g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.022015f8a91e14d51269573278607341",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.022015f8a91e14d51269573278607341/impacts"
                }
            },
            {
                "id": "productid.025d2048ebc3239b716daaf37dae4e85",
                "name": "Chosen by You Tiger Tin Loaf",
                "gtin": "0250600000003",
                "image": "https://static.euw1.sustained.app/products/images/0250600000003_0.jpg",
                "pack": "800g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.025d2048ebc3239b716daaf37dae4e85",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.025d2048ebc3239b716daaf37dae4e85/impacts"
                }
            },
            {
                "id": "productid.0282fbe0a342cbc415c6b4d092cea00d",
                "name": "Slimsters Squares White & Wholemeal Bread",
                "gtin": "5099268044906",
                "image": "https://static.euw1.sustained.app/products/images/5099268044906_0.jpg",
                "pack": "164g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0282fbe0a342cbc415c6b4d092cea00d",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0282fbe0a342cbc415c6b4d092cea00d/impacts"
                }
            },
            {
                "id": "productid.028f2cb2e4740a95f9254bc582be5855",
                "name": "Food 4 Five Grain Bagels",
                "gtin": "0000029143504",
                "image": "https://static.euw1.sustained.app/products/images/0000029143504_0.jpg",
                "pack": "300g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.028f2cb2e4740a95f9254bc582be5855",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.028f2cb2e4740a95f9254bc582be5855/impacts"
                }
            },
            {
                "id": "productid.029e068bb60dee84f984bafe115f0651",
                "name": "1 Elderflower, Lemon & Lovage Hot Cross Buns",
                "gtin": "5000169543153",
                "image": "https://static.euw1.sustained.app/products/images/5000169543153_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.029e068bb60dee84f984bafe115f0651",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.029e068bb60dee84f984bafe115f0651/impacts"
                }
            },
            {
                "id": "productid.0313aab51fcb99976861195147162459",
                "name": "Roasted Garlic Ciabatta",
                "gtin": "5000169137000",
                "image": "https://static.euw1.sustained.app/products/images/5000169137000_0.jpg",
                "pack": "275g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0313aab51fcb99976861195147162459",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0313aab51fcb99976861195147162459/impacts"
                }
            },
            {
                "id": "productid.031d01a220a08f3c12b4f86e0d5532b9",
                "name": "Bakery White Split Tin Loaf Sliced",
                "gtin": "0000001052220",
                "image": "https://static.euw1.sustained.app/products/images/0000001052220_0.jpg",
                "pack": "800g",
                "grade": "C",
                "points": {
                    "pef": 21.57773710653433
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.031d01a220a08f3c12b4f86e0d5532b9",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.031d01a220a08f3c12b4f86e0d5532b9/impacts"
                }
            },
            {
                "id": "productid.031fb05f59822df6f52ac547c055922f",
                "name": "White Quinoa Vegan Loaf",
                "gtin": "5060509390076",
                "image": "https://static.euw1.sustained.app/products/images/5060509390076_0.jpg",
                "pack": "550g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.031fb05f59822df6f52ac547c055922f",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.031fb05f59822df6f52ac547c055922f/impacts"
                }
            },
            {
                "id": "productid.038b2263d1c1ef6c24d7a1cc884cef0d",
                "name": "Gluten Free 4 Plain Bagels",
                "gtin": "5060195906360",
                "image": "https://static.euw1.sustained.app/products/images/5060195906360_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.038b2263d1c1ef6c24d7a1cc884cef0d",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.038b2263d1c1ef6c24d7a1cc884cef0d/impacts"
                }
            },
            {
                "id": "productid.03db6efef7f47caa73f389557ccb527a",
                "name": "Sticky Cinnamon Swirls",
                "gtin": "0000000336611",
                "image": "https://static.euw1.sustained.app/products/images/0000000336611_0.jpg",
                "pack": "349g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03db6efef7f47caa73f389557ccb527a",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03db6efef7f47caa73f389557ccb527a/impacts"
                }
            },
            {
                "id": "productid.03e53e22c9062fea3dba48166517cfb8",
                "name": "4 Hot Cross Buns",
                "gtin": "5010092705115",
                "image": "https://static.euw1.sustained.app/products/images/5010092705115_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03e53e22c9062fea3dba48166517cfb8",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03e53e22c9062fea3dba48166517cfb8/impacts"
                }
            },
            {
                "id": "productid.04ab4f88f284fce5a36b637dab09eb7e",
                "name": "Sliced White Pan",
                "gtin": "5099061000246",
                "image": "https://static.euw1.sustained.app/products/images/5099061000246_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04ab4f88f284fce5a36b637dab09eb7e",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04ab4f88f284fce5a36b637dab09eb7e/impacts"
                }
            },
            {
                "id": "productid.04ef6768e76e22cbdc1ee705cba949f4",
                "name": "Taste the Difference Summer Edition Soft Milk Rolls",
                "gtin": "0000000462167",
                "image": "https://static.euw1.sustained.app/products/images/0000000462167_0.jpg",
                "pack": "160g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04ef6768e76e22cbdc1ee705cba949f4",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04ef6768e76e22cbdc1ee705cba949f4/impacts"
                }
            },
            {
                "id": "productid.06029e5e4992e79bcc61b225e60ad915",
                "name": "Wholewheat Farmhouse Soda Bread",
                "gtin": "5099867000686",
                "image": "https://static.euw1.sustained.app/products/images/5099867000686_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06029e5e4992e79bcc61b225e60ad915",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06029e5e4992e79bcc61b225e60ad915/impacts"
                }
            },
            {
                "id": "productid.07e40c4a687df78cdc9ca9267c44e342",
                "name": "Bakery Poppy Seed Baguette",
                "gtin": "0000001053159",
                "image": "https://static.euw1.sustained.app/products/images/0000001053159_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 12.856364462263942
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07e40c4a687df78cdc9ca9267c44e342",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07e40c4a687df78cdc9ca9267c44e342/impacts"
                }
            },
            {
                "id": "productid.089e989eee7b637c3893364132e77e04",
                "name": "Essential Waitrose & Partners 9 Garlic Slices",
                "gtin": "5000169149393",
                "image": "https://static.euw1.sustained.app/products/images/5000169149393_0.jpg",
                "pack": "270g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.089e989eee7b637c3893364132e77e04",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.089e989eee7b637c3893364132e77e04/impacts"
                }
            },
            {
                "id": "productid.08b8e58de37b6dc605e5e3f64a72b2e3",
                "name": "6 Plain Wraps",
                "gtin": "5012121005630",
                "image": "https://static.euw1.sustained.app/products/images/5012121005630_0.jpg",
                "pack": "366g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08b8e58de37b6dc605e5e3f64a72b2e3",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08b8e58de37b6dc605e5e3f64a72b2e3/impacts"
                }
            },
            {
                "id": "productid.08bf465e0b22abc9097d065d77b12043",
                "name": "Seed Croutons with Caraway Seeds",
                "gtin": "5000169331385",
                "image": "https://static.euw1.sustained.app/products/images/5000169331385_0.jpg",
                "pack": "100g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08bf465e0b22abc9097d065d77b12043",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08bf465e0b22abc9097d065d77b12043/impacts"
                }
            },
            {
                "id": "productid.08d798a17f49c79e5fd155a1691d4c52",
                "name": "Taste the Difference Autumn Edition 2 Chilli & Coriander Wheat & Corn Flatbreads",
                "gtin": "00471060",
                "image": "https://static.euw1.sustained.app/products/images/00471060_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08d798a17f49c79e5fd155a1691d4c52",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08d798a17f49c79e5fd155a1691d4c52/impacts"
                }
            },
            {
                "id": "productid.08f78d3aac74a51e97e5aee9a66463a7",
                "name": "White Farmhouse Sliced",
                "gtin": "0282760000000",
                "image": "https://static.euw1.sustained.app/products/images/0282760000000_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 11.56958710985014
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08f78d3aac74a51e97e5aee9a66463a7",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08f78d3aac74a51e97e5aee9a66463a7/impacts"
                }
            },
            {
                "id": "productid.08fabac23a50c321778d0e52cc359294",
                "name": "Tiger Bloomer",
                "gtin": "0280880000000",
                "image": "https://static.euw1.sustained.app/products/images/0280880000000_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 11.56958710985014
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08fabac23a50c321778d0e52cc359294",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08fabac23a50c321778d0e52cc359294/impacts"
                }
            },
            {
                "id": "productid.08fb9540a980aa641d5e512f3f2e62bc",
                "name": "4 Skinny Italian Pizza Bases",
                "gtin": "5099077004061",
                "image": "https://static.euw1.sustained.app/products/images/5099077004061_0.jpg",
                "pack": "360g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08fb9540a980aa641d5e512f3f2e62bc",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08fb9540a980aa641d5e512f3f2e62bc/impacts"
                }
            },
            {
                "id": "productid.090d95ea1e7afb3c6aaafcb51832a36f",
                "name": "6 Chocolate Orange Hot Cross Buns",
                "gtin": "5010251778738",
                "image": "https://static.euw1.sustained.app/products/images/5010251778738_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.090d95ea1e7afb3c6aaafcb51832a36f",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.090d95ea1e7afb3c6aaafcb51832a36f/impacts"
                }
            },
            {
                "id": "productid.097c7571702a74fc42859e02f88e6cd3",
                "name": "Food Brewers Malted Barley Cob",
                "gtin": "0000029204908",
                "image": "https://static.euw1.sustained.app/products/images/0000029204908_0.jpg",
                "pack": "450g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.097c7571702a74fc42859e02f88e6cd3",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.097c7571702a74fc42859e02f88e6cd3/impacts"
                }
            },
            {
                "id": "productid.0aa582dbd984e53ba67a9fc5e1e26206",
                "name": "Free From 50/50 Loaf",
                "gtin": "5057172515684",
                "image": "https://static.euw1.sustained.app/products/images/5057172515684_0.jpg",
                "pack": "460g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0aa582dbd984e53ba67a9fc5e1e26206",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0aa582dbd984e53ba67a9fc5e1e26206/impacts"
                }
            },
            {
                "id": "productid.0ae9b23f021110b177fbe63c475120ca",
                "name": "12 Fruit Buns",
                "gtin": "5099823001245",
                "image": "https://static.euw1.sustained.app/products/images/5099823001245_0.jpg",
                "pack": "320g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ae9b23f021110b177fbe63c475120ca",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ae9b23f021110b177fbe63c475120ca/impacts"
                }
            },
            {
                "id": "productid.0b8348144c688cf3abaf015b4ab201e5",
                "name": "20 Cocktail Sausage Rolls",
                "gtin": "5027951007389",
                "image": "https://static.euw1.sustained.app/products/images/5027951007389_0.jpg",
                "pack": "320g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b8348144c688cf3abaf015b4ab201e5",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b8348144c688cf3abaf015b4ab201e5/impacts"
                }
            },
            {
                "id": "productid.0b8f499b8cd2cd429f035665530d5c12",
                "name": "Food Super Seeded Loaf",
                "gtin": "0000000642118",
                "image": "https://static.euw1.sustained.app/products/images/0000000642118_0.jpg",
                "pack": "800g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b8f499b8cd2cd429f035665530d5c12",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b8f499b8cd2cd429f035665530d5c12/impacts"
                }
            },
            {
                "id": "productid.0ba33de48ec0dc3dd3d8875e36399cee",
                "name": "Craft Bakery 4 Floury Buns",
                "gtin": "5029912003294",
                "image": "https://static.euw1.sustained.app/products/images/5029912003294_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 9.62384206102552
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ba33de48ec0dc3dd3d8875e36399cee",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ba33de48ec0dc3dd3d8875e36399cee/impacts"
                }
            },
            {
                "id": "productid.0ba8c8d2d1d1aef85337e90bac98ca8c",
                "name": "Essential Waitrose & Partners 6 White Floured Baps",
                "gtin": "5000169014721",
                "image": "https://static.euw1.sustained.app/products/images/5000169014721_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ba8c8d2d1d1aef85337e90bac98ca8c",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ba8c8d2d1d1aef85337e90bac98ca8c/impacts"
                }
            },
            {
                "id": "productid.0bdde412859351870a6f11c633b43944",
                "name": "Poppy Vienna",
                "gtin": "0289180000000",
                "image": "https://static.euw1.sustained.app/products/images/0289180000000_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 11.56958710985014
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bdde412859351870a6f11c633b43944",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bdde412859351870a6f11c633b43944/impacts"
                }
            },
            {
                "id": "productid.0c1f263a04a212d36557431d171a34e0",
                "name": "The Baker's On Market Street Golden Sunflower & Pumpkin Seed Boule",
                "gtin": "0202511001006",
                "image": "https://static.euw1.sustained.app/products/images/0202511001006_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c1f263a04a212d36557431d171a34e0",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c1f263a04a212d36557431d171a34e0/impacts"
                }
            },
            {
                "id": "productid.0d5ab54ffb66a292dcdea11434890be3",
                "name": "Traditional Brown Soda Sliced Brown Bread",
                "gtin": "5011041001692",
                "image": "https://static.euw1.sustained.app/products/images/5011041001692_0.jpg",
                "pack": "500g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d5ab54ffb66a292dcdea11434890be3",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d5ab54ffb66a292dcdea11434890be3/impacts"
                }
            },
            {
                "id": "productid.0e22dc226b58dc33b24c419ea94b0dd1",
                "name": "Family Bread Long Loaf Malted Grain Thick Sliced Bread",
                "gtin": "5011376000186",
                "image": "https://static.euw1.sustained.app/products/images/5011376000186_0.jpg",
                "pack": "800g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e22dc226b58dc33b24c419ea94b0dd1",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e22dc226b58dc33b24c419ea94b0dd1/impacts"
                }
            },
            {
                "id": "productid.0e54ec7228c8da30607e9e232b13b829",
                "name": "Chosen by You Burger Buns",
                "gtin": "0210890000008",
                "image": "https://static.euw1.sustained.app/products/images/0210890000008_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e54ec7228c8da30607e9e232b13b829",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e54ec7228c8da30607e9e232b13b829/impacts"
                }
            },
            {
                "id": "productid.0ea71a14457c2a78deb859185cf76c68",
                "name": "Karaway Bakery 100% Rye Bread with Grains",
                "gtin": "5060172065707",
                "image": "https://static.euw1.sustained.app/products/images/5060172065707_0.jpg",
                "pack": "600g",
                "grade": "B",
                "points": {
                    "pef": 15.634414176709257
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ea71a14457c2a78deb859185cf76c68",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ea71a14457c2a78deb859185cf76c68/impacts"
                }
            },
            {
                "id": "productid.0f16af38d0bbec3f1458009b87dcea0b",
                "name": "Elior White Sandwich Bread Thick 14 Slice",
                "gtin": "5000227203395",
                "image": "https://static.euw1.sustained.app/products/images/5000227203395_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f16af38d0bbec3f1458009b87dcea0b",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f16af38d0bbec3f1458009b87dcea0b/impacts"
                }
            },
            {
                "id": "productid.0f47a74007badc0c915cc2f17397f2aa",
                "name": "8 Brioche Rolls Milk Chocolate Chip",
                "gtin": "5010251758600",
                "image": "https://static.euw1.sustained.app/products/images/5010251758600_0.jpg",
                "pack": "280g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f47a74007badc0c915cc2f17397f2aa",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f47a74007badc0c915cc2f17397f2aa/impacts"
                }
            },
            {
                "id": "productid.0f66e9d9f0c482dce358e2faab465062",
                "name": "Gluten Free 3 Stonebaked Pitta Breads",
                "gtin": "5060195909002",
                "image": "https://static.euw1.sustained.app/products/images/5060195909002_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f66e9d9f0c482dce358e2faab465062",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f66e9d9f0c482dce358e2faab465062/impacts"
                }
            },
            {
                "id": "productid.0f7a54c75e5535dc2d8937df782b6d99",
                "name": "Gluten Free Spicy Fruit Loaf",
                "gtin": "5060195900498",
                "image": "https://static.euw1.sustained.app/products/images/5060195900498_0.jpg",
                "pack": "400g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f7a54c75e5535dc2d8937df782b6d99",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f7a54c75e5535dc2d8937df782b6d99/impacts"
                }
            },
            {
                "id": "productid.108bd5965011c9ab9758be67d3e0e2c7",
                "name": "4 Bundys Floury Baps",
                "gtin": "5099867002178",
                "image": "https://static.euw1.sustained.app/products/images/5099867002178_0.jpg",
                "pack": "320g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.108bd5965011c9ab9758be67d3e0e2c7",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.108bd5965011c9ab9758be67d3e0e2c7/impacts"
                }
            },
            {
                "id": "productid.112ac1ac828452c38704194a8b21a3d0",
                "name": "Softee Soft White Medium Sliced Bread",
                "gtin": "5013427014272",
                "image": "https://static.euw1.sustained.app/products/images/5013427014272_0.jpg",
                "pack": "800g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.112ac1ac828452c38704194a8b21a3d0",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.112ac1ac828452c38704194a8b21a3d0/impacts"
                }
            },
            {
                "id": "productid.1151db5c5bf08b41b49c41c8ba32e636",
                "name": "4 York Baps",
                "gtin": "5060047682800",
                "image": "https://static.euw1.sustained.app/products/images/5060047682800_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1151db5c5bf08b41b49c41c8ba32e636",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1151db5c5bf08b41b49c41c8ba32e636/impacts"
                }
            },
            {
                "id": "productid.11d85b59dc9e3fe17cb887e5ed00c4e8",
                "name": "Plain Nan",
                "gtin": "5030197123454",
                "image": "https://static.euw1.sustained.app/products/images/5030197123454_0.jpg",
                "pack": "375g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11d85b59dc9e3fe17cb887e5ed00c4e8",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11d85b59dc9e3fe17cb887e5ed00c4e8/impacts"
                }
            },
            {
                "id": "productid.1213e367403dd86253520094d2753d1f",
                "name": "Golden Bake Medium Sliced White Bread",
                "gtin": "5013427010342",
                "image": "https://static.euw1.sustained.app/products/images/5013427010342_0.jpg",
                "pack": "800g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1213e367403dd86253520094d2753d1f",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1213e367403dd86253520094d2753d1f/impacts"
                }
            }
        ]
    ],
    "old_products": [
        [
            {
                "id": "productid.0029b87e6bbdcf398860dd24371ff1fd",
                "name": "Organic Veggie & Salmon Pie Stage 2 from 7+ Months",
                "gtin": "5057753912260",
                "image": "https://static.euw1.sustained.app/products/images/5057753912260_0.jpg",
                "pack": "130g",
                "grade": "D",
                "points": {
                    "pef": 25.442759736644405
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0029b87e6bbdcf398860dd24371ff1fd",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0029b87e6bbdcf398860dd24371ff1fd/impacts"
                }
            },
            {
                "id": "productid.00363e5801122b2899e5564c32c49bf2",
                "name": "Sweet Potato Mash & Chicken from 10m Onwards",
                "gtin": "3041094546224",
                "image": "https://static.euw1.sustained.app/products/images/3041094546224_0.jpg",
                "pack": "230g",
                "grade": "C",
                "points": {
                    "pef": 16.988369037820192
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00363e5801122b2899e5564c32c49bf2",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00363e5801122b2899e5564c32c49bf2/impacts"
                }
            },
            {
                "id": "productid.02610e023d22c982e2d82a088d0f8659",
                "name": "Mini Coconut Rolls, Infant Snack, 12 Months+, Multipack",
                "gtin": "5060040257890",
                "image": "https://static.euw1.sustained.app/products/images/5060040257890_0.jpg",
                "pack": "6.8g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02610e023d22c982e2d82a088d0f8659",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02610e023d22c982e2d82a088d0f8659/impacts"
                }
            },
            {
                "id": "productid.03a5701d451f076c541f81b599b50090",
                "name": "Medi+ Lactose-Free Infant Formula 0 - 12 Months",
                "gtin": "5056000503060",
                "image": "https://static.euw1.sustained.app/products/images/5056000503060_0.jpg",
                "pack": "400g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03a5701d451f076c541f81b599b50090",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03a5701d451f076c541f81b599b50090/impacts"
                }
            },
            {
                "id": "productid.06b14ef9ce081698922136e61174fb93",
                "name": "Apple Crisps, Kids Snack, 12+ Months",
                "gtin": "5060040250778",
                "image": "https://static.euw1.sustained.app/products/images/5060040250778_0.jpg",
                "pack": "12g",
                "grade": "B",
                "points": {
                    "pef": 11.808422527904124
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06b14ef9ce081698922136e61174fb93",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06b14ef9ce081698922136e61174fb93/impacts"
                }
            },
            {
                "id": "productid.07630aa18c0769d796dd379e1c99a449",
                "name": "Organic Bangers and Mash Baby Pouch 7+ Months",
                "gtin": "5060107335103",
                "image": "https://static.euw1.sustained.app/products/images/5060107335103_0.jpg",
                "pack": "130g",
                "grade": "G",
                "points": {
                    "pef": 101.82416209284428
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07630aa18c0769d796dd379e1c99a449",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07630aa18c0769d796dd379e1c99a449/impacts"
                }
            },
            {
                "id": "productid.0a0c3db5ccf1818d912ac926071c54ba",
                "name": "Little Angels Organic Pasta with Chicken & Mushrooms 10+ Months",
                "gtin": "5057172300754",
                "image": "https://static.euw1.sustained.app/products/images/5057172300754_0.jpg",
                "pack": "190g",
                "grade": "C",
                "points": {
                    "pef": 18.013750410197517
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a0c3db5ccf1818d912ac926071c54ba",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a0c3db5ccf1818d912ac926071c54ba/impacts"
                }
            },
            {
                "id": "productid.0a99141d9a84ecf2f0e04221a643560e",
                "name": "Organic Stage 2 +7 months Mashed Cottage Pie",
                "gtin": "5391527190585",
                "image": "https://static.euw1.sustained.app/products/images/5391527190585_0.jpg",
                "pack": "120g",
                "grade": "E",
                "points": {
                    "pef": 37.96992063554568
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a99141d9a84ecf2f0e04221a643560e",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a99141d9a84ecf2f0e04221a643560e/impacts"
                }
            },
            {
                "id": "productid.0b493af60fe98f6253e40e09ac0d4edd",
                "name": "Organic Seven Grain Cereal 7 Months+",
                "gtin": "3041091645968",
                "image": "https://static.euw1.sustained.app/products/images/3041091645968_0.jpg",
                "pack": "180g",
                "grade": "A",
                "points": {
                    "pef": 10.428070202056544
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b493af60fe98f6253e40e09ac0d4edd",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b493af60fe98f6253e40e09ac0d4edd/impacts"
                }
            },
            {
                "id": "productid.0bc311fc2d8cce8a7043590fec1b7fb6",
                "name": "Sweetcorn + Carrot Melty Sticks from 7 Months",
                "gtin": "5060503502932",
                "image": "https://static.euw1.sustained.app/products/images/5060503502932_0.jpg",
                "pack": "6g",
                "grade": "C",
                "points": {
                    "pef": 19.57293971378376
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bc311fc2d8cce8a7043590fec1b7fb6",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bc311fc2d8cce8a7043590fec1b7fb6/impacts"
                }
            },
            {
                "id": "productid.0c9ef4a8ad171c0b65a4dd378d945a35",
                "name": "Organic Tutti Fruity Chicken Curry +8 Months Textured",
                "gtin": "5391527190943",
                "image": "https://static.euw1.sustained.app/products/images/5391527190943_0.jpg",
                "pack": "170g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c9ef4a8ad171c0b65a4dd378d945a35",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c9ef4a8ad171c0b65a4dd378d945a35/impacts"
                }
            },
            {
                "id": "productid.0d190ca190af0e84b0eaed8876f44616",
                "name": "Sweet Potato, Chicken & Veggies Pouch 7m+",
                "gtin": "5000157079909",
                "image": "https://static.euw1.sustained.app/products/images/5000157079909_0.jpg",
                "pack": "130g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d190ca190af0e84b0eaed8876f44616",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d190ca190af0e84b0eaed8876f44616/impacts"
                }
            },
            {
                "id": "productid.0e7395de6067fab067d1114083e3b434",
                "name": "Mild Chilli Con Carne with a Pinch of Paprika 7+ Months",
                "gtin": "5060107339675",
                "image": "https://static.euw1.sustained.app/products/images/5060107339675_0.jpg",
                "pack": "130g",
                "grade": "F",
                "points": {
                    "pef": 68.19840198826515
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e7395de6067fab067d1114083e3b434",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e7395de6067fab067d1114083e3b434/impacts"
                }
            },
            {
                "id": "productid.105f995e1dd488bb90bce184372337bf",
                "name": "Organic Greek-Style Yoghurt with Prunes & Oats Around 6 Months",
                "gtin": "5060454500667",
                "image": "https://static.euw1.sustained.app/products/images/5060454500667_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 65.32060430502602
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.105f995e1dd488bb90bce184372337bf",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.105f995e1dd488bb90bce184372337bf/impacts"
                }
            },
            {
                "id": "productid.1205b00ff6a59da14221de2612a9ccce",
                "name": "Organic Squares Gentle Teething Wafers Mango Textured 10+ Months",
                "gtin": "5060452490922",
                "image": "https://static.euw1.sustained.app/products/images/5060452490922_0.jpg",
                "pack": "20g",
                "grade": "E",
                "points": {
                    "pef": 41.925335091700006
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1205b00ff6a59da14221de2612a9ccce",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1205b00ff6a59da14221de2612a9ccce/impacts"
                }
            },
            {
                "id": "productid.12819f9c07091afc6fb0b847ecb30655",
                "name": "Apple, Apricot, Banana & Orange",
                "gtin": "5000157144119",
                "image": "https://static.euw1.sustained.app/products/images/5000157144119_0.jpg",
                "pack": "120g",
                "grade": "A",
                "points": {
                    "pef": 7.639452226837631
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12819f9c07091afc6fb0b847ecb30655",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12819f9c07091afc6fb0b847ecb30655/impacts"
                }
            },
            {
                "id": "productid.158c5f29ad045cd5bdddcb13cf74c784",
                "name": "Thai Chicken Curry Pouch 10m+",
                "gtin": "5000157079794",
                "image": "https://static.euw1.sustained.app/products/images/5000157079794_0.jpg",
                "pack": "180g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.158c5f29ad045cd5bdddcb13cf74c784",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.158c5f29ad045cd5bdddcb13cf74c784/impacts"
                }
            },
            {
                "id": "productid.15ea5f9bcbd2e32829d88d0d93f3cd9b",
                "name": "Banana Wholegrain Porridge Baby Cereal",
                "gtin": "3041091431790",
                "image": "https://static.euw1.sustained.app/products/images/3041091431790_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 38.573070279198454
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.15ea5f9bcbd2e32829d88d0d93f3cd9b",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.15ea5f9bcbd2e32829d88d0d93f3cd9b/impacts"
                }
            },
            {
                "id": "productid.164e2de39d299b7f801873c159314d38",
                "name": "Kids Marvellous Mandarin & Apple Oaty Bars",
                "gtin": "5024121497216",
                "image": "https://static.euw1.sustained.app/products/images/5024121497216_0.jpg",
                "pack": "23g",
                "grade": "D",
                "points": {
                    "pef": 26.650274214022556
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.164e2de39d299b7f801873c159314d38",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.164e2de39d299b7f801873c159314d38/impacts"
                }
            },
            {
                "id": "productid.1a02b578fc37847149bb60749c6c2bc4",
                "name": "Little Ones Organic Cottage Pie 10+ Months",
                "gtin": "0000000332118",
                "image": "https://static.euw1.sustained.app/products/images/0000000332118_0.jpg",
                "pack": "190g",
                "grade": "E",
                "points": {
                    "pef": 38.197927783635706
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1a02b578fc37847149bb60749c6c2bc4",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1a02b578fc37847149bb60749c6c2bc4/impacts"
                }
            },
            {
                "id": "productid.1b5d076d223e799af8b68d27ce63ff36",
                "name": "Let's Cook Animal Shapes Pasta 10+ Months",
                "gtin": "5000157146113",
                "image": "https://static.euw1.sustained.app/products/images/5000157146113_0.jpg",
                "pack": "340g",
                "grade": "A",
                "points": {
                    "pef": 10.713198898827812
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b5d076d223e799af8b68d27ce63ff36",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b5d076d223e799af8b68d27ce63ff36/impacts"
                }
            },
            {
                "id": "productid.1b93cdfd20085996847ccbbe308c9053",
                "name": "Chocolate Biscotti 7m+",
                "gtin": "5000157065759",
                "image": "https://static.euw1.sustained.app/products/images/5000157065759_0.jpg",
                "pack": "60g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b93cdfd20085996847ccbbe308c9053",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b93cdfd20085996847ccbbe308c9053/impacts"
                }
            },
            {
                "id": "productid.1ba421245eedaaefeb145982012c544b",
                "name": "Organic Mangoes, Pears and Papaya Baby Pouch 4+ Months",
                "gtin": "5060107334786",
                "image": "https://static.euw1.sustained.app/products/images/5060107334786_0.jpg",
                "pack": "120g",
                "grade": "B",
                "points": {
                    "pef": 11.971671427113165
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1ba421245eedaaefeb145982012c544b",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1ba421245eedaaefeb145982012c544b/impacts"
                }
            },
            {
                "id": "productid.1bc030b745bf686dafee55cf31264ec5",
                "name": "Organic Veggie Moussaka Baby Pouch 7+ Months",
                "gtin": "5060107336506",
                "image": "https://static.euw1.sustained.app/products/images/5060107336506_0.jpg",
                "pack": "130g",
                "grade": "F",
                "points": {
                    "pef": 68.36242004384562
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1bc030b745bf686dafee55cf31264ec5",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1bc030b745bf686dafee55cf31264ec5/impacts"
                }
            },
            {
                "id": "productid.1c4483efd88d7e30a2f1d98e1036eb96",
                "name": "Fish & Coconut Curry 1-3 Years",
                "gtin": "5060398490222",
                "image": "https://static.euw1.sustained.app/products/images/5060398490222_0.jpg",
                "pack": "230g",
                "grade": "D",
                "points": {
                    "pef": 23.578141619009944
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c4483efd88d7e30a2f1d98e1036eb96",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c4483efd88d7e30a2f1d98e1036eb96/impacts"
                }
            },
            {
                "id": "productid.1c4fa544151fdc5096a1b450d49b8bff",
                "name": "7+ Months By Nature Cottage Pie",
                "gtin": "5000157140487",
                "image": "https://static.euw1.sustained.app/products/images/5000157140487_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c4fa544151fdc5096a1b450d49b8bff",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c4fa544151fdc5096a1b450d49b8bff/impacts"
                }
            },
            {
                "id": "productid.1d417316ff28c35afbdf53d42eea6af3",
                "name": "Mini Organic Cheese Crackers Toddler Snack Multipack",
                "gtin": "5024121626104",
                "image": "https://static.euw1.sustained.app/products/images/5024121626104_0.jpg",
                "pack": "20g",
                "grade": "D",
                "points": {
                    "pef": 25.214834171493443
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1d417316ff28c35afbdf53d42eea6af3",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1d417316ff28c35afbdf53d42eea6af3/impacts"
                }
            },
            {
                "id": "productid.204533b5653e6cf3c18cdddbd1f29329",
                "name": "2 Follow On Baby Milk Formula",
                "gtin": "5051594002715",
                "image": "https://static.euw1.sustained.app/products/images/5051594002715_0.jpg",
                "pack": "1l",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.204533b5653e6cf3c18cdddbd1f29329",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.204533b5653e6cf3c18cdddbd1f29329/impacts"
                }
            },
            {
                "id": "productid.206c28d1f7ab80e321a7886a50436b1b",
                "name": "Chicken & Sweet Potato Curry 10+ Months Stage 3",
                "gtin": "5060398490178",
                "image": "https://static.euw1.sustained.app/products/images/5060398490178_0.jpg",
                "pack": "190g",
                "grade": "C",
                "points": {
                    "pef": 19.69754820844496
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.206c28d1f7ab80e321a7886a50436b1b",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.206c28d1f7ab80e321a7886a50436b1b/impacts"
                }
            },
            {
                "id": "productid.2203406b07802a75ac5be1b44bfc7716",
                "name": "Creamy Porridge Baby Cereal 6+ Months",
                "gtin": "4062300013957",
                "image": "https://static.euw1.sustained.app/products/images/4062300013957_0.jpg",
                "pack": "160g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2203406b07802a75ac5be1b44bfc7716",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2203406b07802a75ac5be1b44bfc7716/impacts"
                }
            },
            {
                "id": "productid.22bcd9660530b59b1a3f3d3f4e04542b",
                "name": "Organic Red & Go Smoothie Multipack 6 Months+",
                "gtin": "5060452490533",
                "image": "https://static.euw1.sustained.app/products/images/5060452490533_0.jpg",
                "pack": "90g",
                "grade": "C",
                "points": {
                    "pef": 20.1107745795256
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.22bcd9660530b59b1a3f3d3f4e04542b",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.22bcd9660530b59b1a3f3d3f4e04542b/impacts"
                }
            },
            {
                "id": "productid.2362b79846a183ecdbc83c928ed7f66a",
                "name": "10+ Months By Nature Veggie Singapore Noodles",
                "gtin": "5000157079817",
                "image": "https://static.euw1.sustained.app/products/images/5000157079817_0.jpg",
                "pack": "180g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2362b79846a183ecdbc83c928ed7f66a",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2362b79846a183ecdbc83c928ed7f66a/impacts"
                }
            },
            {
                "id": "productid.271b1c9443c0b2da761f67d377eebc36",
                "name": "Organic Blushing Berries, Pear & Banana",
                "gtin": "5060452490281",
                "image": "https://static.euw1.sustained.app/products/images/5060452490281_0.jpg",
                "pack": "100g",
                "grade": "D",
                "points": {
                    "pef": 22.206555363866126
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.271b1c9443c0b2da761f67d377eebc36",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.271b1c9443c0b2da761f67d377eebc36/impacts"
                }
            },
            {
                "id": "productid.271d2b274e0bbf3ca2cdb028874fb3ad",
                "name": "Beefy Stew with Spuds from 10 Months",
                "gtin": "5060107333970",
                "image": "https://static.euw1.sustained.app/products/images/5060107333970_0.jpg",
                "pack": "190g",
                "grade": "F",
                "points": {
                    "pef": 67.56881530465056
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.271d2b274e0bbf3ca2cdb028874fb3ad",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.271d2b274e0bbf3ca2cdb028874fb3ad/impacts"
                }
            },
            {
                "id": "productid.28632cdbd0bd76730f8970ed5394be2d",
                "name": "Scrumptious Lasagne 12+ Months",
                "gtin": "4062300024519",
                "image": "https://static.euw1.sustained.app/products/images/4062300024519_0.jpg",
                "pack": "230g",
                "grade": "E",
                "points": {
                    "pef": 35.06269945745182
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.28632cdbd0bd76730f8970ed5394be2d",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.28632cdbd0bd76730f8970ed5394be2d/impacts"
                }
            },
            {
                "id": "productid.293b0b48865afd7cc6b93526998266ba",
                "name": "Spaghetti Bolognese Baby Food Jar 6+ Months",
                "gtin": "4062300161481",
                "image": "https://static.euw1.sustained.app/products/images/4062300161481_0.jpg",
                "pack": "125g",
                "grade": "E",
                "points": {
                    "pef": 31.652305769588658
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.293b0b48865afd7cc6b93526998266ba",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.293b0b48865afd7cc6b93526998266ba/impacts"
                }
            },
            {
                "id": "productid.2a781971c1f2f1463a36a3b525b21123",
                "name": "Organic Mango, Apple and Coconut Milk Stage 1 from Around 6 Months",
                "gtin": "5060077680692",
                "image": "https://static.euw1.sustained.app/products/images/5060077680692_0.jpg",
                "pack": "100g",
                "grade": "B",
                "points": {
                    "pef": 11.333109684074593
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a781971c1f2f1463a36a3b525b21123",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a781971c1f2f1463a36a3b525b21123/impacts"
                }
            },
            {
                "id": "productid.2bc866a3af9d0e27a611792ded6d3784",
                "name": "Organics Organic Pineapple & Banana Super Puffs Age 6+ Months",
                "gtin": "5060095650776",
                "image": "https://static.euw1.sustained.app/products/images/5060095650776_0.jpg",
                "pack": "20g",
                "grade": "D",
                "points": {
                    "pef": 26.758481496450376
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2bc866a3af9d0e27a611792ded6d3784",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2bc866a3af9d0e27a611792ded6d3784/impacts"
                }
            },
            {
                "id": "productid.2eb342c7f82c69fd97cdd4af84955336",
                "name": "Strawberry + Apple Oaty Biccies 12+ Months",
                "gtin": "5060503504554",
                "image": "https://static.euw1.sustained.app/products/images/5060503504554_0.jpg",
                "pack": "20g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2eb342c7f82c69fd97cdd4af84955336",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2eb342c7f82c69fd97cdd4af84955336/impacts"
                }
            },
            {
                "id": "productid.2fae156e1dfd4fab4f5c06bea8a4e03b",
                "name": "Wheat Based Fortified Baby Cereal with Honey & Wheat, Just Add Water, 12 Months+,",
                "gtin": "7616100581535",
                "image": "https://static.euw1.sustained.app/products/images/7616100581535_0.jpg",
                "pack": "1kg",
                "grade": "F",
                "points": {
                    "pef": 71.61575191336787
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2fae156e1dfd4fab4f5c06bea8a4e03b",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2fae156e1dfd4fab4f5c06bea8a4e03b/impacts"
                }
            },
            {
                "id": "productid.2fe4bfa0885de2ebe0c8cc3badcada05",
                "name": "Kids Apple Soft Biscotti 7+ Months",
                "gtin": "5391500921571",
                "image": "https://static.euw1.sustained.app/products/images/5391500921571_0.jpg",
                "pack": "20g",
                "grade": "E",
                "points": {
                    "pef": 37.33883821344808
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2fe4bfa0885de2ebe0c8cc3badcada05",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2fe4bfa0885de2ebe0c8cc3badcada05/impacts"
                }
            },
            {
                "id": "productid.31375b13a4acc3173292fc3644d2dd2a",
                "name": "Smoothie Melts, Blackcurrant, Apple & Banana 12months+",
                "gtin": "5060040252024",
                "image": "https://static.euw1.sustained.app/products/images/5060040252024_0.jpg",
                "pack": "6g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.31375b13a4acc3173292fc3644d2dd2a",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.31375b13a4acc3173292fc3644d2dd2a/impacts"
                }
            },
            {
                "id": "productid.336c788d285503e3b9243dc124c474cb",
                "name": "Apple Blackcurrant & Banana Fruit Puree Pots",
                "gtin": "3041091265555",
                "image": "https://static.euw1.sustained.app/products/images/3041091265555_0.jpg",
                "pack": "100g",
                "grade": "A",
                "points": {
                    "pef": 8.613879274472591
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.336c788d285503e3b9243dc124c474cb",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.336c788d285503e3b9243dc124c474cb/impacts"
                }
            },
            {
                "id": "productid.386d92f13e98aa16313c79b8a05a51ba",
                "name": "HYGIENE AQUAINT Sanitising Water",
                "gtin": "5060080353408",
                "image": "https://static.euw1.sustained.app/products/images/5060080353408_0.jpg",
                "pack": "50ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.386d92f13e98aa16313c79b8a05a51ba",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.386d92f13e98aa16313c79b8a05a51ba/impacts"
                }
            },
            {
                "id": "productid.397a51335d318ac7914a76bb61f35440",
                "name": "Organic Macaroni Cheese Baby Pouch 6+ Months",
                "gtin": "5060107336629",
                "image": "https://static.euw1.sustained.app/products/images/5060107336629_0.jpg",
                "pack": "120g",
                "grade": "F",
                "points": {
                    "pef": 83.60853578947658
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.397a51335d318ac7914a76bb61f35440",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.397a51335d318ac7914a76bb61f35440/impacts"
                }
            },
            {
                "id": "productid.3b17b7ff11232d128e2e3c074b72bfe4",
                "name": "Organic Chicken and Noodles Baby Pouch 7+ Months",
                "gtin": "5060107333871",
                "image": "https://static.euw1.sustained.app/products/images/5060107333871_0.jpg",
                "pack": "130g",
                "grade": "E",
                "points": {
                    "pef": 51.76534175510773
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3b17b7ff11232d128e2e3c074b72bfe4",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3b17b7ff11232d128e2e3c074b72bfe4/impacts"
                }
            },
            {
                "id": "productid.3c4e9844f4be2786e0318ecdac898f7b",
                "name": "Organic Avocado Breakfast with Yoghurt, Spinach & Oats Around 6 Months",
                "gtin": "5060454500629",
                "image": "https://static.euw1.sustained.app/products/images/5060454500629_0.jpg",
                "pack": "100g",
                "grade": "C",
                "points": {
                    "pef": 18.709371697743812
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3c4e9844f4be2786e0318ecdac898f7b",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3c4e9844f4be2786e0318ecdac898f7b/impacts"
                }
            },
            {
                "id": "productid.3e0c81e5ef289cfbea849eb052445d94",
                "name": "Lamb Hot Pot Pouch 7m+",
                "gtin": "5000157079855",
                "image": "https://static.euw1.sustained.app/products/images/5000157079855_0.jpg",
                "pack": "130g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3e0c81e5ef289cfbea849eb052445d94",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3e0c81e5ef289cfbea849eb052445d94/impacts"
                }
            },
            {
                "id": "productid.45cdacce66be05fe043b381e31128ed1",
                "name": "Blueberry Porridge 7m+",
                "gtin": "5000157077028",
                "image": "https://static.euw1.sustained.app/products/images/5000157077028_0.jpg",
                "pack": "240g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.45cdacce66be05fe043b381e31128ed1",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.45cdacce66be05fe043b381e31128ed1/impacts"
                }
            },
            {
                "id": "productid.45e4732366819efc3bdb299c096d4728",
                "name": "4 Growing Up Baby Milk Powder from 2 Years Onwards",
                "gtin": "4062300270732",
                "image": "https://static.euw1.sustained.app/products/images/4062300270732_0.jpg",
                "pack": "600g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Baby Foods & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.45e4732366819efc3bdb299c096d4728",
                    "category": "https://api.sustained.com/choice/v1/categories/baby-foods-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.45e4732366819efc3bdb299c096d4728/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.00e0cef98fcdf810c12379eb4bc1ae42",
                "name": "6 Traditional Unsmoked Back Rashers",
                "gtin": "5390171004088",
                "image": "https://static.euw1.sustained.app/products/images/5390171004088_0.jpg",
                "pack": "190g",
                "grade": "G",
                "points": {
                    "pef": 127.59533272704213
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00e0cef98fcdf810c12379eb4bc1ae42",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00e0cef98fcdf810c12379eb4bc1ae42/impacts"
                }
            },
            {
                "id": "productid.012d1196db0a9d02f3bb0fae6ea67b61",
                "name": "Traditional Steak Slice",
                "gtin": "5060238331692",
                "image": "https://static.euw1.sustained.app/products/images/5060238331692_0.jpg",
                "pack": "1.1kg",
                "grade": "G",
                "points": {
                    "pef": 193.60638534125954
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.012d1196db0a9d02f3bb0fae6ea67b61",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.012d1196db0a9d02f3bb0fae6ea67b61/impacts"
                }
            },
            {
                "id": "productid.018f582dc3018be777cfe8b84d44d37a",
                "name": "Unsmoked 2 Bacon Loin Steaks",
                "gtin": "0000001832617",
                "image": "https://static.euw1.sustained.app/products/images/0000001832617_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 132.19673159991638
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.018f582dc3018be777cfe8b84d44d37a",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.018f582dc3018be777cfe8b84d44d37a/impacts"
                }
            },
            {
                "id": "productid.01ec2eff9a5cf0ee164c5f7810e9b72b",
                "name": "Food German Bratwurst",
                "gtin": "0000029196876",
                "image": "https://static.euw1.sustained.app/products/images/0000029196876_0.jpg",
                "pack": "360g",
                "grade": "G",
                "points": {
                    "pef": 119.34791739993811
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01ec2eff9a5cf0ee164c5f7810e9b72b",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01ec2eff9a5cf0ee164c5f7810e9b72b/impacts"
                }
            },
            {
                "id": "productid.02f54a10124baf06f5bfc4e348aceafb",
                "name": "Butcher's Selection 12 Smoked Back Bacon Rashers",
                "gtin": "5054070504550",
                "image": "https://static.euw1.sustained.app/products/images/5054070504550_0.jpg",
                "pack": "345g",
                "grade": "G",
                "points": {
                    "pef": 134.07525824923528
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02f54a10124baf06f5bfc4e348aceafb",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02f54a10124baf06f5bfc4e348aceafb/impacts"
                }
            },
            {
                "id": "productid.03600e64ed8f1a0684ac4edd3651f4aa",
                "name": "Dry Zywiecka Sausage",
                "gtin": "5906453734453",
                "image": "https://static.euw1.sustained.app/products/images/5906453734453_0.jpg",
                "pack": "0.250kg",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03600e64ed8f1a0684ac4edd3651f4aa",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03600e64ed8f1a0684ac4edd3651f4aa/impacts"
                }
            },
            {
                "id": "productid.03aa4fec11c013d1d1efb669cc4eef36",
                "name": "Smoked Back Bacon",
                "gtin": "5010251716211",
                "image": "https://static.euw1.sustained.app/products/images/5010251716211_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 132.45296154865304
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03aa4fec11c013d1d1efb669cc4eef36",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03aa4fec11c013d1d1efb669cc4eef36/impacts"
                }
            },
            {
                "id": "productid.06312b5114d7847f1012cd4ed0f71c7f",
                "name": "Tesco Fuller Flavoured Bacon Company Maple Cured Back Bacon 8 Rashers",
                "gtin": "5022977011570",
                "image": "https://static.euw1.sustained.app/products/images/5022977011570_0.jpg",
                "pack": "280g",
                "grade": "G",
                "points": {
                    "pef": 138.06267598155813
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06312b5114d7847f1012cd4ed0f71c7f",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06312b5114d7847f1012cd4ed0f71c7f/impacts"
                }
            },
            {
                "id": "productid.07be4f4345c94f456eeeaa631a8eccef",
                "name": "Cumberland Thick Sliced Sausage 5 Slices",
                "gtin": "5010525118277",
                "image": "https://static.euw1.sustained.app/products/images/5010525118277_0.jpg",
                "pack": "150g",
                "grade": "G",
                "points": {
                    "pef": 112.43613887014276
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07be4f4345c94f456eeeaa631a8eccef",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07be4f4345c94f456eeeaa631a8eccef/impacts"
                }
            },
            {
                "id": "productid.0952d82849f30b7ebd5022804a9960de",
                "name": "Maple Cure Back Bacon",
                "gtin": "5017326136660",
                "image": "https://static.euw1.sustained.app/products/images/5017326136660_0.jpg",
                "pack": "300g",
                "grade": "G",
                "points": {
                    "pef": 128.02894898494387
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0952d82849f30b7ebd5022804a9960de",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0952d82849f30b7ebd5022804a9960de/impacts"
                }
            },
            {
                "id": "productid.0a92b8abc036635d60e3ca6186fc4d35",
                "name": "Finest British Unsmoked Dry Cured Back Bacon 8 Rashers",
                "gtin": "5052910096562",
                "image": "https://static.euw1.sustained.app/products/images/5052910096562_0.jpg",
                "pack": "240g",
                "grade": "G",
                "points": {
                    "pef": 95.23089826646051
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a92b8abc036635d60e3ca6186fc4d35",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a92b8abc036635d60e3ca6186fc4d35/impacts"
                }
            },
            {
                "id": "productid.0ba38414d7bf85dc40606ac6eba9373c",
                "name": "Vegetarian Jumbo Hotdogs",
                "gtin": "7290112969132",
                "image": "https://static.euw1.sustained.app/products/images/7290112969132_0.jpg",
                "pack": "400g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ba38414d7bf85dc40606ac6eba9373c",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ba38414d7bf85dc40606ac6eba9373c/impacts"
                }
            },
            {
                "id": "productid.0be45299ef0bb4155491560af39f175c",
                "name": "8 Thick Cut Unsmoked Back Bacon Rashers",
                "gtin": "5060234454593",
                "image": "https://static.euw1.sustained.app/products/images/5060234454593_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 132.95136839973136
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0be45299ef0bb4155491560af39f175c",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0be45299ef0bb4155491560af39f175c/impacts"
                }
            },
            {
                "id": "productid.0cf6d70afa3e611020e06912a1eabcb9",
                "name": "Extra Special Pigs in Blankets Festive Feast",
                "gtin": "5057172237104",
                "image": "https://static.euw1.sustained.app/products/images/5057172237104_0.jpg",
                "pack": "694g",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0cf6d70afa3e611020e06912a1eabcb9",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0cf6d70afa3e611020e06912a1eabcb9/impacts"
                }
            },
            {
                "id": "productid.0e1635b8914c71a94f5828f1f5038a12",
                "name": "The Best 12 British Spicy Lamb Merguez Chipolatas",
                "gtin": "5010251882176",
                "image": "https://static.euw1.sustained.app/products/images/5010251882176_0.jpg",
                "pack": "375g",
                "grade": "G",
                "points": {
                    "pef": 407.54108668814115
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e1635b8914c71a94f5828f1f5038a12",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e1635b8914c71a94f5828f1f5038a12/impacts"
                }
            },
            {
                "id": "productid.0e5ef1222f249900d813f2e1de859594",
                "name": "Butcher's Selection 4 Cheese and Jalape\u00f1o Pork Hot Dogs",
                "gtin": "5054070655610",
                "image": "https://static.euw1.sustained.app/products/images/5054070655610_0.jpg",
                "pack": "400g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e5ef1222f249900d813f2e1de859594",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e5ef1222f249900d813f2e1de859594/impacts"
                }
            },
            {
                "id": "productid.0e703d240d86fd4950cc2f2784e2e8c6",
                "name": "12 Plant-Based Chipolata Sausages",
                "gtin": "5060626405059",
                "image": "https://static.euw1.sustained.app/products/images/5060626405059_0.jpg",
                "pack": "288g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e703d240d86fd4950cc2f2784e2e8c6",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e703d240d86fd4950cc2f2784e2e8c6/impacts"
                }
            },
            {
                "id": "productid.0f0934bb09d9792a4b309a2c34e96ab5",
                "name": "The Scottish Butcher Classic Breakfast Beef Lorne Sausage 4 Slices",
                "gtin": "5060004212774",
                "image": "https://static.euw1.sustained.app/products/images/5060004212774_0.jpg",
                "pack": "270g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f0934bb09d9792a4b309a2c34e96ab5",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f0934bb09d9792a4b309a2c34e96ab5/impacts"
                }
            },
            {
                "id": "productid.0f501b2b1135814c6a71bb1dd4b8e493",
                "name": "Italian Mortadella 5 Slices",
                "gtin": "5000169582275",
                "image": "https://static.euw1.sustained.app/products/images/5000169582275_0.jpg",
                "pack": "75g",
                "grade": "G",
                "points": {
                    "pef": 144.58423969347473
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f501b2b1135814c6a71bb1dd4b8e493",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f501b2b1135814c6a71bb1dd4b8e493/impacts"
                }
            },
            {
                "id": "productid.102042c589be106437aeeac7eab45e39",
                "name": "Extra Special 4 New York Style Hot Dog Sausages",
                "gtin": "5054781855026",
                "image": "https://static.euw1.sustained.app/products/images/5054781855026_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 126.97979760310086
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.102042c589be106437aeeac7eab45e39",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.102042c589be106437aeeac7eab45e39/impacts"
                }
            },
            {
                "id": "productid.103550720c1975e1f15fd0a891774919",
                "name": "Smoky Hot Dogs",
                "gtin": "5060055251432",
                "image": "https://static.euw1.sustained.app/products/images/5060055251432_0.jpg",
                "pack": "1.2kg",
                "grade": "G",
                "points": {
                    "pef": 126.4399828826582
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.103550720c1975e1f15fd0a891774919",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.103550720c1975e1f15fd0a891774919/impacts"
                }
            },
            {
                "id": "productid.103db7f910a96785075d763d33d5aca9",
                "name": "6 Chicken Sausages",
                "gtin": "5054775603329",
                "image": "https://static.euw1.sustained.app/products/images/5054775603329_0.jpg",
                "pack": "400g",
                "grade": "F",
                "points": {
                    "pef": 53.46459740610108
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.103db7f910a96785075d763d33d5aca9",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.103db7f910a96785075d763d33d5aca9/impacts"
                }
            },
            {
                "id": "productid.104def12688a1f46464e70ee7b353a58",
                "name": "Butcher's Selection 4 New York Pork Hot Dogs",
                "gtin": "5054070655603",
                "image": "https://static.euw1.sustained.app/products/images/5054070655603_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 126.03495417555642
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.104def12688a1f46464e70ee7b353a58",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.104def12688a1f46464e70ee7b353a58/impacts"
                }
            },
            {
                "id": "productid.112fdfef55241956698bb5ba1735e74a",
                "name": "Kabanosy Classic",
                "gtin": "5907785150119",
                "image": "https://static.euw1.sustained.app/products/images/5907785150119_0.jpg",
                "pack": "175g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.112fdfef55241956698bb5ba1735e74a",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.112fdfef55241956698bb5ba1735e74a/impacts"
                }
            },
            {
                "id": "productid.11f621b5a0a1a758b2c641e79638797f",
                "name": "Butcher's Selection 8 Thick Cut Smoked Back Bacon Rashers",
                "gtin": "5057172706297",
                "image": "https://static.euw1.sustained.app/products/images/5057172706297_0.jpg",
                "pack": "350g",
                "grade": "G",
                "points": {
                    "pef": 133.3213410957882
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11f621b5a0a1a758b2c641e79638797f",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11f621b5a0a1a758b2c641e79638797f/impacts"
                }
            },
            {
                "id": "productid.122435767c64bf55b7906b2d97b39002",
                "name": "Naked Veg and Proud 6 Delicious Red Pepper & Smoky Squash Sausages",
                "gtin": "5060055252248",
                "image": "https://static.euw1.sustained.app/products/images/5060055252248_0.jpg",
                "pack": "270g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.122435767c64bf55b7906b2d97b39002",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.122435767c64bf55b7906b2d97b39002/impacts"
                }
            },
            {
                "id": "productid.1560f81a38f8aeeda7e9c17f1069b255",
                "name": "German Frankfurters",
                "gtin": "5010251733256",
                "image": "https://static.euw1.sustained.app/products/images/5010251733256_0.jpg",
                "pack": "350g",
                "grade": "G",
                "points": {
                    "pef": 118.58238523719618
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1560f81a38f8aeeda7e9c17f1069b255",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1560f81a38f8aeeda7e9c17f1069b255/impacts"
                }
            },
            {
                "id": "productid.15f722ced3e072b7e8367bf0a42c734f",
                "name": "Noel Chorizo with Truffle",
                "gtin": "29065691",
                "image": "https://static.euw1.sustained.app/products/images/29065691_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.15f722ced3e072b7e8367bf0a42c734f",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.15f722ced3e072b7e8367bf0a42c734f/impacts"
                }
            },
            {
                "id": "productid.164048e623579ecce8dbbe4e413b92c1",
                "name": "8 Unsmoked Back Bacon Rashers",
                "gtin": "5010251603924",
                "image": "https://static.euw1.sustained.app/products/images/5010251603924_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 132.9987880358399
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.164048e623579ecce8dbbe4e413b92c1",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.164048e623579ecce8dbbe4e413b92c1/impacts"
                }
            },
            {
                "id": "productid.16990b05ee040eeaef2b7e56a3bd02a5",
                "name": "12 Aberdeen Angus Beef Chipolatas with Black Pepper",
                "gtin": "5000169246795",
                "image": "https://static.euw1.sustained.app/products/images/5000169246795_0.jpg",
                "pack": "340g",
                "grade": "G",
                "points": {
                    "pef": 224.09953181010957
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16990b05ee040eeaef2b7e56a3bd02a5",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16990b05ee040eeaef2b7e56a3bd02a5/impacts"
                }
            },
            {
                "id": "productid.1792e47a09d26202678fcf9e1f75cdd8",
                "name": "Farm Stores Smoked Back Bacon",
                "gtin": "5054781006619",
                "image": "https://static.euw1.sustained.app/products/images/5054781006619_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 131.8131037961962
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1792e47a09d26202678fcf9e1f75cdd8",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1792e47a09d26202678fcf9e1f75cdd8/impacts"
                }
            },
            {
                "id": "productid.1821da0c2dbfdceefb9965640d954ec1",
                "name": "Kabanos Sausage",
                "gtin": "5907785151895",
                "image": "https://static.euw1.sustained.app/products/images/5907785151895_0.jpg",
                "pack": "170g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1821da0c2dbfdceefb9965640d954ec1",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1821da0c2dbfdceefb9965640d954ec1/impacts"
                }
            },
            {
                "id": "productid.184f1905640173d8e7d58af1d3f05000",
                "name": "Little Bit Lighter Steak Sausages",
                "gtin": "5032992983127",
                "image": "https://static.euw1.sustained.app/products/images/5032992983127_0.jpg",
                "pack": "350g",
                "grade": "G",
                "points": {
                    "pef": 189.3231413714228
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.184f1905640173d8e7d58af1d3f05000",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.184f1905640173d8e7d58af1d3f05000/impacts"
                }
            },
            {
                "id": "productid.19cf56446145fb96fb6cf506cc6f04f3",
                "name": "8 Flavourful Pork Sausages",
                "gtin": "20335045",
                "image": "https://static.euw1.sustained.app/products/images/20335045_0.jpg",
                "pack": "454g",
                "grade": "G",
                "points": {
                    "pef": 114.47787128002341
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.19cf56446145fb96fb6cf506cc6f04f3",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.19cf56446145fb96fb6cf506cc6f04f3/impacts"
                }
            },
            {
                "id": "productid.1bf2794cd914f0ec41791bb5013df1a9",
                "name": "Farm Stores Usmoked Middle Bacon",
                "gtin": "5052449451931",
                "image": "https://static.euw1.sustained.app/products/images/5052449451931_0.jpg",
                "pack": "350g",
                "grade": "G",
                "points": {
                    "pef": 129.05310721896214
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1bf2794cd914f0ec41791bb5013df1a9",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1bf2794cd914f0ec41791bb5013df1a9/impacts"
                }
            },
            {
                "id": "productid.1efe339760cbd0dc1b7a2a41c84b1f8b",
                "name": "Traditional Irish Rindless Bacon",
                "gtin": "5391519200636",
                "image": "https://static.euw1.sustained.app/products/images/5391519200636_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 129.54911374669385
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1efe339760cbd0dc1b7a2a41c84b1f8b",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1efe339760cbd0dc1b7a2a41c84b1f8b/impacts"
                }
            },
            {
                "id": "productid.20f2818d1e618bd4b47d1302751e5a50",
                "name": "Spanish Spicy Chorizo Ring",
                "gtin": "01738339",
                "image": "https://static.euw1.sustained.app/products/images/01738339_0.jpg",
                "pack": "225g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.20f2818d1e618bd4b47d1302751e5a50",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.20f2818d1e618bd4b47d1302751e5a50/impacts"
                }
            },
            {
                "id": "productid.218ffb65b26f16b909301126660b542e",
                "name": "6 Smoked Back Bacon Extra Thick Cut Rashers",
                "gtin": "5010525058993",
                "image": "https://static.euw1.sustained.app/products/images/5010525058993_0.jpg",
                "pack": "270g",
                "grade": "G",
                "points": {
                    "pef": 127.62437940130658
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.218ffb65b26f16b909301126660b542e",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.218ffb65b26f16b909301126660b542e/impacts"
                }
            },
            {
                "id": "productid.21bed32a6710b0fc317aefb8bf5a0ce4",
                "name": "Chorizo Bites Punchy Paprika",
                "gtin": "0634114047752",
                "image": "https://static.euw1.sustained.app/products/images/0634114047752_0.jpg",
                "pack": "50g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.21bed32a6710b0fc317aefb8bf5a0ce4",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.21bed32a6710b0fc317aefb8bf5a0ce4/impacts"
                }
            },
            {
                "id": "productid.223a770c0f53fd165df90e4264027a92",
                "name": "12 Irish Pork Sausages",
                "gtin": "5099120004246",
                "image": "https://static.euw1.sustained.app/products/images/5099120004246_0.jpg",
                "pack": "340g",
                "grade": "F",
                "points": {
                    "pef": 93.04348489959413
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.223a770c0f53fd165df90e4264027a92",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.223a770c0f53fd165df90e4264027a92/impacts"
                }
            },
            {
                "id": "productid.23231bb89283db43ed2f26c0924141b0",
                "name": "Salame Toscano",
                "gtin": "8019063002380",
                "image": "https://static.euw1.sustained.app/products/images/8019063002380_0.jpg",
                "pack": "0.100kg",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.23231bb89283db43ed2f26c0924141b0",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.23231bb89283db43ed2f26c0924141b0/impacts"
                }
            },
            {
                "id": "productid.255d38db7d0df29255a85c1e067e2139",
                "name": "Hot & Spicy Pepperoni Snack",
                "gtin": "5998001705930",
                "image": "https://static.euw1.sustained.app/products/images/5998001705930_0.jpg",
                "pack": "0.090kg",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.255d38db7d0df29255a85c1e067e2139",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.255d38db7d0df29255a85c1e067e2139/impacts"
                }
            },
            {
                "id": "productid.26cdb7565d53b0844659d27a231ca7e4",
                "name": "Mini Rosette Saucisson",
                "gtin": "5010292948480",
                "image": "https://static.euw1.sustained.app/products/images/5010292948480_0.jpg",
                "pack": "300g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.26cdb7565d53b0844659d27a231ca7e4",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.26cdb7565d53b0844659d27a231ca7e4/impacts"
                }
            },
            {
                "id": "productid.2756802ba22dda318df7f9143bb521c4",
                "name": "Chicken Sausage",
                "gtin": "0216120000000",
                "image": "https://static.euw1.sustained.app/products/images/0216120000000_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 83.55303379841666
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2756802ba22dda318df7f9143bb521c4",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2756802ba22dda318df7f9143bb521c4/impacts"
                }
            },
            {
                "id": "productid.27bdd951a7c3256c4121b689bb83f8b9",
                "name": "Bacon Boiling Joint",
                "gtin": "5099490000121",
                "image": "https://static.euw1.sustained.app/products/images/5099490000121_0.jpg",
                "pack": "750g",
                "grade": "G",
                "points": {
                    "pef": 129.01705120387305
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.27bdd951a7c3256c4121b689bb83f8b9",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.27bdd951a7c3256c4121b689bb83f8b9/impacts"
                }
            },
            {
                "id": "productid.27c0efca9cf4f9a4a89e259d9725b108",
                "name": "Butcher's Selection 12 Smoked Back Bacon Rashers",
                "gtin": "5057172706273",
                "image": "https://static.euw1.sustained.app/products/images/5057172706273_0.jpg",
                "pack": "350g",
                "grade": "G",
                "points": {
                    "pef": 133.3213410957882
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.27c0efca9cf4f9a4a89e259d9725b108",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.27c0efca9cf4f9a4a89e259d9725b108/impacts"
                }
            },
            {
                "id": "productid.288a0113a81dfae364ad90427468d456",
                "name": "Premium Bacon Stamps",
                "gtin": "5099530006250",
                "image": "https://static.euw1.sustained.app/products/images/5099530006250_0.jpg",
                "pack": "1kg",
                "grade": "G",
                "points": {
                    "pef": 139.00037800048625
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.288a0113a81dfae364ad90427468d456",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.288a0113a81dfae364ad90427468d456/impacts"
                }
            },
            {
                "id": "productid.2925d30945fc98a7d7fdbbc731227b62",
                "name": "The Best 6 Thick British Pork & Sweet Chilli Sausages",
                "gtin": "5010251797630",
                "image": "https://static.euw1.sustained.app/products/images/5010251797630_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 119.12990506351414
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2925d30945fc98a7d7fdbbc731227b62",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2925d30945fc98a7d7fdbbc731227b62/impacts"
                }
            },
            {
                "id": "productid.29469154358ee4f918fa44d81931b180",
                "name": "Smoked Streaky Bacon 16 Rashers",
                "gtin": "5010525059266",
                "image": "https://static.euw1.sustained.app/products/images/5010525059266_0.jpg",
                "pack": "225g",
                "grade": "G",
                "points": {
                    "pef": 130.44511594327486
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.29469154358ee4f918fa44d81931b180",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.29469154358ee4f918fa44d81931b180/impacts"
                }
            },
            {
                "id": "productid.2a2a34fbd294b43e7929676a4a31a4c0",
                "name": "Berlinki Cheese",
                "gtin": "5900567001517",
                "image": "https://static.euw1.sustained.app/products/images/5900567001517_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 104.83768382481128
                },
                "info_icons": [],
                "category": "Bacon & Sausages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a2a34fbd294b43e7929676a4a31a4c0",
                    "category": "https://api.sustained.com/choice/v1/categories/bacon-and-sausages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a2a34fbd294b43e7929676a4a31a4c0/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.015486d26e6d08505ea48e79fdae4c0d",
                "name": "Diced Beef",
                "gtin": "5054402922007",
                "image": "https://static.euw1.sustained.app/products/images/5054402922007_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.015486d26e6d08505ea48e79fdae4c0d",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.015486d26e6d08505ea48e79fdae4c0d/impacts"
                }
            },
            {
                "id": "productid.053c626d0044fe59547868df54639aaf",
                "name": "Our Family Butcher 2 Standard Beef Fillets",
                "gtin": "2331665000000",
                "image": "https://static.euw1.sustained.app/products/images/2331665000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.053c626d0044fe59547868df54639aaf",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.053c626d0044fe59547868df54639aaf/impacts"
                }
            },
            {
                "id": "productid.05683f790fbc20fc573175b561a2a530",
                "name": "Extra Special British Wagyu Fillet Steak",
                "gtin": "05057172472604",
                "image": "https://static.euw1.sustained.app/products/images/05057172472604_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05683f790fbc20fc573175b561a2a530",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05683f790fbc20fc573175b561a2a530/impacts"
                }
            },
            {
                "id": "productid.059d7b6de8892c914e43639e8748695a",
                "name": "North Highland Beef Fillet Steak",
                "gtin": "0000000426787",
                "image": "https://static.euw1.sustained.app/products/images/0000000426787_0.jpg",
                "pack": "170g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.059d7b6de8892c914e43639e8748695a",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.059d7b6de8892c914e43639e8748695a/impacts"
                }
            },
            {
                "id": "productid.06507e8c82aea13a2f8d24e863c9266f",
                "name": "Irish Slow Roasting Beef Joint",
                "gtin": "0000010023334",
                "image": "https://static.euw1.sustained.app/products/images/0000010023334_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06507e8c82aea13a2f8d24e863c9266f",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06507e8c82aea13a2f8d24e863c9266f/impacts"
                }
            },
            {
                "id": "productid.06604580ad578b5fdff0451d57e3c7f8",
                "name": "10% Fat Northern Irish Beef Mince",
                "gtin": "00122139",
                "image": "https://static.euw1.sustained.app/products/images/00122139_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06604580ad578b5fdff0451d57e3c7f8",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06604580ad578b5fdff0451d57e3c7f8/impacts"
                }
            },
            {
                "id": "productid.066a8f35145933433782cb6db8260421",
                "name": "Food Select Farms British Rose Veal Escalopes",
                "gtin": "0000000773867",
                "image": "https://static.euw1.sustained.app/products/images/0000000773867_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.066a8f35145933433782cb6db8260421",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.066a8f35145933433782cb6db8260421/impacts"
                }
            },
            {
                "id": "productid.07a6b52ea3794e4b40a9732f9f22db41",
                "name": "Essential Waitrose & Partners British Beef Topside",
                "gtin": "0283980000004",
                "image": "https://static.euw1.sustained.app/products/images/0283980000004_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07a6b52ea3794e4b40a9732f9f22db41",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07a6b52ea3794e4b40a9732f9f22db41/impacts"
                }
            },
            {
                "id": "productid.0815b7b6bd88c69568820602a3200c46",
                "name": "SO Organic Northern Irish Beef Fillet Steak",
                "gtin": "0225270000004",
                "image": "https://static.euw1.sustained.app/products/images/0225270000004_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0815b7b6bd88c69568820602a3200c46",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0815b7b6bd88c69568820602a3200c46/impacts"
                }
            },
            {
                "id": "productid.08a6e042a086c29ca4d46607bf74cbf2",
                "name": "North Highland Scotch Beef 5% Fat Mince",
                "gtin": "0000000426732",
                "image": "https://static.euw1.sustained.app/products/images/0000000426732_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08a6e042a086c29ca4d46607bf74cbf2",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08a6e042a086c29ca4d46607bf74cbf2/impacts"
                }
            },
            {
                "id": "productid.08dac344693996381f3c3467d2de3531",
                "name": "Waitrose Aberdeen Angus Top Rump Joint",
                "gtin": "0265450000004",
                "image": "https://static.euw1.sustained.app/products/images/0265450000004_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08dac344693996381f3c3467d2de3531",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08dac344693996381f3c3467d2de3531/impacts"
                }
            },
            {
                "id": "productid.09235c94a888ac727fe08a7c31f5b9a3",
                "name": "British Extra Lean Beef Casserole Steak",
                "gtin": "0000000328920",
                "image": "https://static.euw1.sustained.app/products/images/0000000328920_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09235c94a888ac727fe08a7c31f5b9a3",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09235c94a888ac727fe08a7c31f5b9a3/impacts"
                }
            },
            {
                "id": "productid.09735fd1cd67c3edae45ead70bb743c3",
                "name": "North Highland Beef Rump Steak",
                "gtin": "0000000426817",
                "image": "https://static.euw1.sustained.app/products/images/0000000426817_0.jpg",
                "pack": "225g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09735fd1cd67c3edae45ead70bb743c3",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09735fd1cd67c3edae45ead70bb743c3/impacts"
                }
            },
            {
                "id": "productid.0a33d8f7a6c99d1b60ef3a881fca1674",
                "name": "Our Family Butcher Ribeye Standard Cut 1 Pack",
                "gtin": "2331669000000",
                "image": "https://static.euw1.sustained.app/products/images/2331669000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a33d8f7a6c99d1b60ef3a881fca1674",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a33d8f7a6c99d1b60ef3a881fca1674/impacts"
                }
            },
            {
                "id": "productid.0b3dd619f4180babc10b7ea6ac894e7c",
                "name": "Northern Irish Extra Lean Diced Beef",
                "gtin": "0000000414746",
                "image": "https://static.euw1.sustained.app/products/images/0000000414746_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b3dd619f4180babc10b7ea6ac894e7c",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b3dd619f4180babc10b7ea6ac894e7c/impacts"
                }
            },
            {
                "id": "productid.0f6dcec1bd0c30a2b8a50738bad226cb",
                "name": "Tender Large Beef Roasting Joint",
                "gtin": "0217710000002",
                "image": "https://static.euw1.sustained.app/products/images/0217710000002_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 221.5993954224299
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f6dcec1bd0c30a2b8a50738bad226cb",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f6dcec1bd0c30a2b8a50738bad226cb/impacts"
                }
            },
            {
                "id": "productid.0fc8d045abd377d7690433ecc618ba16",
                "name": "Irish Beef Sirloin Steak with Mixed Pepper Seasoning",
                "gtin": "5054775951048",
                "image": "https://static.euw1.sustained.app/products/images/5054775951048_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fc8d045abd377d7690433ecc618ba16",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fc8d045abd377d7690433ecc618ba16/impacts"
                }
            },
            {
                "id": "productid.1187ab6f483bd80d24510275a3e47709",
                "name": "Extra Special British Aberdeen Angus Sirloin Steak",
                "gtin": "5057172878611",
                "image": "https://static.euw1.sustained.app/products/images/5057172878611_0.jpg",
                "pack": "320g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1187ab6f483bd80d24510275a3e47709",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1187ab6f483bd80d24510275a3e47709/impacts"
                }
            },
            {
                "id": "productid.11e2da63b416f1ba093c66080a01c777",
                "name": "Fajita Style Beef Stir Fry Strips",
                "gtin": "5059697761041",
                "image": "https://static.euw1.sustained.app/products/images/5059697761041_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11e2da63b416f1ba093c66080a01c777",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11e2da63b416f1ba093c66080a01c777/impacts"
                }
            },
            {
                "id": "productid.137e907d99a341fd1bd06ba182a8d852",
                "name": "British Topside of Beef",
                "gtin": "0000000038140",
                "image": "https://static.euw1.sustained.app/products/images/0000000038140_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.137e907d99a341fd1bd06ba182a8d852",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.137e907d99a341fd1bd06ba182a8d852/impacts"
                }
            },
            {
                "id": "productid.13eac86cf7805bf4c7e8f697da7a81b2",
                "name": "British Extra Lean Beef Casserole Steak",
                "gtin": "00328920",
                "image": "https://static.euw1.sustained.app/products/images/00328920_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.13eac86cf7805bf4c7e8f697da7a81b2",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.13eac86cf7805bf4c7e8f697da7a81b2/impacts"
                }
            },
            {
                "id": "productid.14868a46711c1feab250e688887e9294",
                "name": "British Beef Rump Steak",
                "gtin": "5057753747824",
                "image": "https://static.euw1.sustained.app/products/images/5057753747824_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14868a46711c1feab250e688887e9294",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14868a46711c1feab250e688887e9294/impacts"
                }
            },
            {
                "id": "productid.14d42e47f73f65396de6871d03160a2c",
                "name": "Taste the Difference British Beef Thin Cut Sirloin Steak",
                "gtin": "0000000327336",
                "image": "https://static.euw1.sustained.app/products/images/0000000327336_0.jpg",
                "pack": "155g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14d42e47f73f65396de6871d03160a2c",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14d42e47f73f65396de6871d03160a2c/impacts"
                }
            },
            {
                "id": "productid.15357fad5f833c4e99443a6bd6748148",
                "name": "SO Organic British Beef Rump Steak",
                "gtin": "0260420000008",
                "image": "https://static.euw1.sustained.app/products/images/0260420000008_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.15357fad5f833c4e99443a6bd6748148",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.15357fad5f833c4e99443a6bd6748148/impacts"
                }
            },
            {
                "id": "productid.1556fe27d7ce4cef8a19c60f5df91b32",
                "name": "Selected Cuts Corned Beef 4 Slices",
                "gtin": "5052003441354",
                "image": "https://static.euw1.sustained.app/products/images/5052003441354_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1556fe27d7ce4cef8a19c60f5df91b32",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1556fe27d7ce4cef8a19c60f5df91b32/impacts"
                }
            },
            {
                "id": "productid.1836b3ecc8c0d93f59d5ae9c90420421",
                "name": "British Beef Steak Medallions",
                "gtin": "5057545807613",
                "image": "https://static.euw1.sustained.app/products/images/5057545807613_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1836b3ecc8c0d93f59d5ae9c90420421",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1836b3ecc8c0d93f59d5ae9c90420421/impacts"
                }
            },
            {
                "id": "productid.1a1e07617b08868d790760f23462e260",
                "name": "Organic Diced Beef",
                "gtin": "5099536801866",
                "image": "https://static.euw1.sustained.app/products/images/5099536801866_0.jpg",
                "pack": "380g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1a1e07617b08868d790760f23462e260",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1a1e07617b08868d790760f23462e260/impacts"
                }
            },
            {
                "id": "productid.1a573597a7118cdb256785f6da258315",
                "name": "Irish Beef Round Steak",
                "gtin": "5057967692828",
                "image": "https://static.euw1.sustained.app/products/images/5057967692828_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1a573597a7118cdb256785f6da258315",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1a573597a7118cdb256785f6da258315/impacts"
                }
            },
            {
                "id": "productid.1c28b7614d15201651e3e1f3d1af7b06",
                "name": "The Butcher's On Market Street British Beef Rib Eye Steak",
                "gtin": "5010251987079",
                "image": "https://static.euw1.sustained.app/products/images/5010251987079_0.jpg",
                "pack": "0.200kg",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c28b7614d15201651e3e1f3d1af7b06",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c28b7614d15201651e3e1f3d1af7b06/impacts"
                }
            },
            {
                "id": "productid.1e0e3988c001f6bb827fa8754b15dbf7",
                "name": "Essential Waitrose & Partners British Beef Rump Steak",
                "gtin": "5000169131411",
                "image": "https://static.euw1.sustained.app/products/images/5000169131411_0.jpg",
                "pack": "230g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e0e3988c001f6bb827fa8754b15dbf7",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e0e3988c001f6bb827fa8754b15dbf7/impacts"
                }
            },
            {
                "id": "productid.1e2ca377db44b68c07a1d0b47dacd72b",
                "name": "Finest Certified Irish Angus Beef Chateaubriand",
                "gtin": "0207900000000",
                "image": "https://static.euw1.sustained.app/products/images/0207900000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e2ca377db44b68c07a1d0b47dacd72b",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e2ca377db44b68c07a1d0b47dacd72b/impacts"
                }
            },
            {
                "id": "productid.1e9dcb38e0b2989abb5127c9b93444fd",
                "name": "5% Fat Beef Mince",
                "gtin": "00288187",
                "image": "https://static.euw1.sustained.app/products/images/00288187_0.jpg",
                "pack": "750g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e9dcb38e0b2989abb5127c9b93444fd",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e9dcb38e0b2989abb5127c9b93444fd/impacts"
                }
            },
            {
                "id": "productid.20306faca649d61c8fcdf1dbef5e5b52",
                "name": "Extra Special British Wagyu Rump Steak",
                "gtin": "05057172472574",
                "image": "https://static.euw1.sustained.app/products/images/05057172472574_0.jpg",
                "pack": "225g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.20306faca649d61c8fcdf1dbef5e5b52",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.20306faca649d61c8fcdf1dbef5e5b52/impacts"
                }
            },
            {
                "id": "productid.21b85d9f988a8ce56d9bd1c60743a010",
                "name": "Silverside Beef",
                "gtin": "5030450718618",
                "image": "https://static.euw1.sustained.app/products/images/5030450718618_0.jpg",
                "pack": "2kg",
                "grade": "G",
                "points": {
                    "pef": 225.93074704394266
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.21b85d9f988a8ce56d9bd1c60743a010",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.21b85d9f988a8ce56d9bd1c60743a010/impacts"
                }
            },
            {
                "id": "productid.22de0cd26b3956ae5938ca20c6416446",
                "name": "Butcher's Selection Beef Reduced Fat Mince",
                "gtin": "5052449754452",
                "image": "https://static.euw1.sustained.app/products/images/5052449754452_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.22de0cd26b3956ae5938ca20c6416446",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.22de0cd26b3956ae5938ca20c6416446/impacts"
                }
            },
            {
                "id": "productid.23f74ae5e51374df3428bf3e17f5f4d1",
                "name": "Irish Fillet Steak",
                "gtin": "4088600467030",
                "image": "https://static.euw1.sustained.app/products/images/4088600467030_0.jpg",
                "pack": "220g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.23f74ae5e51374df3428bf3e17f5f4d1",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.23f74ae5e51374df3428bf3e17f5f4d1/impacts"
                }
            },
            {
                "id": "productid.2488a85e932afba57fb6ed5a3830df60",
                "name": "Northern Irish Beef Rump Steak",
                "gtin": "0000000199421",
                "image": "https://static.euw1.sustained.app/products/images/0000000199421_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2488a85e932afba57fb6ed5a3830df60",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2488a85e932afba57fb6ed5a3830df60/impacts"
                }
            },
            {
                "id": "productid.26fffcf56dbb171556aa691239ec533d",
                "name": "British Diced Beef Shin",
                "gtin": "0000000295031",
                "image": "https://static.euw1.sustained.app/products/images/0000000295031_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.26fffcf56dbb171556aa691239ec533d",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.26fffcf56dbb171556aa691239ec533d/impacts"
                }
            },
            {
                "id": "productid.27a27132c86b96b204f9a88369875ca3",
                "name": "The Butcher's On Market Street Lean British Beef Medallion Steaks",
                "gtin": "5010251797647",
                "image": "https://static.euw1.sustained.app/products/images/5010251797647_0.jpg",
                "pack": "0.340kg",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.27a27132c86b96b204f9a88369875ca3",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.27a27132c86b96b204f9a88369875ca3/impacts"
                }
            },
            {
                "id": "productid.289c3a15edf29d55060598b50427f7e4",
                "name": "Taste the Difference British Beef Ultimate Rump Steak",
                "gtin": "0211860000004",
                "image": "https://static.euw1.sustained.app/products/images/0211860000004_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.289c3a15edf29d55060598b50427f7e4",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.289c3a15edf29d55060598b50427f7e4/impacts"
                }
            },
            {
                "id": "productid.28a2b7d263230ba387045a1919a7332d",
                "name": "Our Family Butcher British Beef Roasting Joint Large",
                "gtin": "2334710000000",
                "image": "https://static.euw1.sustained.app/products/images/2334710000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.28a2b7d263230ba387045a1919a7332d",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.28a2b7d263230ba387045a1919a7332d/impacts"
                }
            },
            {
                "id": "productid.2974ee5d773f6b18d624170b3505db5a",
                "name": "Butcher's Selection 2 Soy & Black Pepper Beef Steaks",
                "gtin": "5057172580415",
                "image": "https://static.euw1.sustained.app/products/images/5057172580415_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 268.9907409225208
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2974ee5d773f6b18d624170b3505db5a",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2974ee5d773f6b18d624170b3505db5a/impacts"
                }
            },
            {
                "id": "productid.29fd2a73b7e4ae401b5e2eba478b9614",
                "name": "Succulent Beef Fillet Steak",
                "gtin": "0290960000008",
                "image": "https://static.euw1.sustained.app/products/images/0290960000008_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.29fd2a73b7e4ae401b5e2eba478b9614",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.29fd2a73b7e4ae401b5e2eba478b9614/impacts"
                }
            },
            {
                "id": "productid.2a84cc08ff344e1346c0be436e9efc57",
                "name": "Finest West Cork Dry Aged Angus Ribeye Steak",
                "gtin": "5057753467517",
                "image": "https://static.euw1.sustained.app/products/images/5057753467517_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a84cc08ff344e1346c0be436e9efc57",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a84cc08ff344e1346c0be436e9efc57/impacts"
                }
            },
            {
                "id": "productid.2aa8c7d5fe5b0cab8ae38c7abb8e7716",
                "name": "Extra Lean Steak Pieces",
                "gtin": "5060393476115",
                "image": "https://static.euw1.sustained.app/products/images/5060393476115_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2aa8c7d5fe5b0cab8ae38c7abb8e7716",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2aa8c7d5fe5b0cab8ae38c7abb8e7716/impacts"
                }
            },
            {
                "id": "productid.2d3427131e1223eec5d1823cf3bebcf9",
                "name": "Finest Beef Fillet",
                "gtin": "0231590000000",
                "image": "https://static.euw1.sustained.app/products/images/0231590000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2d3427131e1223eec5d1823cf3bebcf9",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2d3427131e1223eec5d1823cf3bebcf9/impacts"
                }
            },
            {
                "id": "productid.313cb9ac958a232c5241b855411a765e",
                "name": "Succulent Beef Sirloin Steak",
                "gtin": "5052449753714",
                "image": "https://static.euw1.sustained.app/products/images/5052449753714_0.jpg",
                "pack": "227g",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.313cb9ac958a232c5241b855411a765e",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.313cb9ac958a232c5241b855411a765e/impacts"
                }
            },
            {
                "id": "productid.3566ca2ca9b5a6a93729d740281f4aa5",
                "name": "Irish Beef Ribeye Steaks",
                "gtin": "5057753882839",
                "image": "https://static.euw1.sustained.app/products/images/5057753882839_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3566ca2ca9b5a6a93729d740281f4aa5",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3566ca2ca9b5a6a93729d740281f4aa5/impacts"
                }
            },
            {
                "id": "productid.3845915405784231f9e66f71993e6751",
                "name": "British Diced Beef",
                "gtin": "5057545807538",
                "image": "https://static.euw1.sustained.app/products/images/5057545807538_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3845915405784231f9e66f71993e6751",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3845915405784231f9e66f71993e6751/impacts"
                }
            },
            {
                "id": "productid.393733956592816945ed03aa1b7baa45",
                "name": "Organic British Beef Rump Steak",
                "gtin": "0247370000000",
                "image": "https://static.euw1.sustained.app/products/images/0247370000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Beef",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.393733956592816945ed03aa1b7baa45",
                    "category": "https://api.sustained.com/choice/v1/categories/beef",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.393733956592816945ed03aa1b7baa45/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.007ca6dfa8cd6fb18926a7a1f5d7c8ba",
                "name": "Thatchers Green Goblin Somerset Dry Cider",
                "gtin": "5020628002359",
                "image": "https://static.euw1.sustained.app/products/images/5020628002359_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 13.50865670029716
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.007ca6dfa8cd6fb18926a7a1f5d7c8ba",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.007ca6dfa8cd6fb18926a7a1f5d7c8ba/impacts"
                }
            },
            {
                "id": "productid.00c3068ba5ce79d840ef321de5d59f33",
                "name": "Magners Original",
                "gtin": "5391516871563",
                "image": "https://static.euw1.sustained.app/products/images/5391516871563_0.jpg",
                "pack": "568ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00c3068ba5ce79d840ef321de5d59f33",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00c3068ba5ce79d840ef321de5d59f33/impacts"
                }
            },
            {
                "id": "productid.00d7dcdfa8efd002a193d64ff62d8195",
                "name": "Asda Biere De Luxe Premium French Lager",
                "gtin": "5054781212126",
                "image": "https://static.euw1.sustained.app/products/images/5054781212126_0.jpg",
                "pack": "25cl",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00d7dcdfa8efd002a193d64ff62d8195",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00d7dcdfa8efd002a193d64ff62d8195/impacts"
                }
            },
            {
                "id": "productid.020b68e2a03c98140369e91af348588d",
                "name": "Kopparberg Premium Cider with Strawberry & Lime Alcohol Free",
                "gtin": "7393714518989",
                "image": "https://static.euw1.sustained.app/products/images/7393714518989_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.020b68e2a03c98140369e91af348588d",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.020b68e2a03c98140369e91af348588d/impacts"
                }
            },
            {
                "id": "productid.026713bc8fa11605f205b1a5594068d9",
                "name": "Pilsner Urquell The Original Pilsner",
                "gtin": "8594404010847",
                "image": "https://static.euw1.sustained.app/products/images/8594404010847_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.026713bc8fa11605f205b1a5594068d9",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.026713bc8fa11605f205b1a5594068d9/impacts"
                }
            },
            {
                "id": "productid.02930bb157e509e4f83d2deba7aa1428",
                "name": "Purity Brewing Co. Bunny Hop Unfiltered Pale Ale",
                "gtin": "5060140900375",
                "image": "https://static.euw1.sustained.app/products/images/5060140900375_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02930bb157e509e4f83d2deba7aa1428",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02930bb157e509e4f83d2deba7aa1428/impacts"
                }
            },
            {
                "id": "productid.029e98823dff0e95c925af73b54163f3",
                "name": "Camden Town Brewery Beer Imperial Pilsner",
                "gtin": "5060228273490",
                "image": "https://static.euw1.sustained.app/products/images/5060228273490_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.029e98823dff0e95c925af73b54163f3",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.029e98823dff0e95c925af73b54163f3/impacts"
                }
            },
            {
                "id": "productid.02c45d9217d535865e155e3c0a012b96",
                "name": "Stella Artois Belgium Premium Lager",
                "gtin": "5014379012859",
                "image": "https://static.euw1.sustained.app/products/images/5014379012859_0.jpg",
                "pack": "284ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02c45d9217d535865e155e3c0a012b96",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02c45d9217d535865e155e3c0a012b96/impacts"
                }
            },
            {
                "id": "productid.02ef715a0022377e5ba77caa124f4eb6",
                "name": "Super Bock Premium Imported Beer",
                "gtin": "5601164114401",
                "image": "https://static.euw1.sustained.app/products/images/5601164114401_0.jpg",
                "pack": "66cl",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02ef715a0022377e5ba77caa124f4eb6",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02ef715a0022377e5ba77caa124f4eb6/impacts"
                }
            },
            {
                "id": "productid.0365c90e475e669f541979286af3e3f4",
                "name": "Corona Lager Beer Bottle",
                "gtin": "5014379014457",
                "image": "https://static.euw1.sustained.app/products/images/5014379014457_0.jpg",
                "pack": "620ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0365c90e475e669f541979286af3e3f4",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0365c90e475e669f541979286af3e3f4/impacts"
                }
            },
            {
                "id": "productid.03cc356707ae8dd7c26825bcba8cef9e",
                "name": "Peroni Nastro Azzurro",
                "gtin": "5023837000697",
                "image": "https://static.euw1.sustained.app/products/images/5023837000697_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03cc356707ae8dd7c26825bcba8cef9e",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03cc356707ae8dd7c26825bcba8cef9e/impacts"
                }
            },
            {
                "id": "productid.04a1d6036030d3076399663bf76b4eaa",
                "name": "Lerwick Brewery Lerwick IPA",
                "gtin": "5060347790168",
                "image": "https://static.euw1.sustained.app/products/images/5060347790168_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04a1d6036030d3076399663bf76b4eaa",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04a1d6036030d3076399663bf76b4eaa/impacts"
                }
            },
            {
                "id": "productid.04b3579759271a479cc5e1d09811053a",
                "name": "Roadworks Brewing Co. Pale Ale",
                "gtin": "5391528373376",
                "image": "https://static.euw1.sustained.app/products/images/5391528373376_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04b3579759271a479cc5e1d09811053a",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04b3579759271a479cc5e1d09811053a/impacts"
                }
            },
            {
                "id": "productid.04f85903a3b266cf21ad6bc21b534d0b",
                "name": "Cloudwater West Coast IPA",
                "gtin": "5056025460553",
                "image": "https://static.euw1.sustained.app/products/images/5056025460553_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04f85903a3b266cf21ad6bc21b534d0b",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04f85903a3b266cf21ad6bc21b534d0b/impacts"
                }
            },
            {
                "id": "productid.061ee00c294adcff4ad3f39c67531b15",
                "name": "Stewart Brewing Hollyrood Pale Ale",
                "gtin": "5060143110047",
                "image": "https://static.euw1.sustained.app/products/images/5060143110047_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.061ee00c294adcff4ad3f39c67531b15",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.061ee00c294adcff4ad3f39c67531b15/impacts"
                }
            },
            {
                "id": "productid.0635358d650a841d5581f4771917efc9",
                "name": "Dartmoor Brewery Dragon's Breath",
                "gtin": "5032970000693",
                "image": "https://static.euw1.sustained.app/products/images/5032970000693_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0635358d650a841d5581f4771917efc9",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0635358d650a841d5581f4771917efc9/impacts"
                }
            },
            {
                "id": "productid.06660682f6db24bd833a140be54ebaa6",
                "name": "Guinness Draught Stout Beer",
                "gtin": "5000213101025",
                "image": "https://static.euw1.sustained.app/products/images/5000213101025_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.114109174210071
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06660682f6db24bd833a140be54ebaa6",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06660682f6db24bd833a140be54ebaa6/impacts"
                }
            },
            {
                "id": "productid.06c47a559473d7ecba157f2ba74f3f07",
                "name": "Porterhouse Brew Co. Renegade",
                "gtin": "5391518560748",
                "image": "https://static.euw1.sustained.app/products/images/5391518560748_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.114109174210071
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06c47a559473d7ecba157f2ba74f3f07",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06c47a559473d7ecba157f2ba74f3f07/impacts"
                }
            },
            {
                "id": "productid.06c9c5ab8e76d9988a15d65ba2181e90",
                "name": "Blackthorn Reserve",
                "gtin": "5391520442575",
                "image": "https://static.euw1.sustained.app/products/images/5391520442575_0.jpg",
                "pack": "3l",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06c9c5ab8e76d9988a15d65ba2181e90",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06c9c5ab8e76d9988a15d65ba2181e90/impacts"
                }
            },
            {
                "id": "productid.06f7c7d391a926c55dd28e44a37c00f5",
                "name": "Wychwood Brewery Hobgoblin IPA",
                "gtin": "5011348022758",
                "image": "https://static.euw1.sustained.app/products/images/5011348022758_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06f7c7d391a926c55dd28e44a37c00f5",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06f7c7d391a926c55dd28e44a37c00f5/impacts"
                }
            },
            {
                "id": "productid.07075381e9557c5972f11d35026f6b3e",
                "name": "Camden Show Off Juicy Lager",
                "gtin": "5060228271496",
                "image": "https://static.euw1.sustained.app/products/images/5060228271496_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07075381e9557c5972f11d35026f6b3e",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07075381e9557c5972f11d35026f6b3e/impacts"
                }
            },
            {
                "id": "productid.0786d854539be1991f46799fdd8ce3d4",
                "name": "Krombacher Pils",
                "gtin": "4008287058628",
                "image": "https://static.euw1.sustained.app/products/images/4008287058628_0.jpg",
                "pack": "660ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0786d854539be1991f46799fdd8ce3d4",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0786d854539be1991f46799fdd8ce3d4/impacts"
                }
            },
            {
                "id": "productid.07bc2b6e0c41b39a4befddaa814ea151",
                "name": "Brooklyn Brewery Brooklyn Pilsner Crisp Lager",
                "gtin": "5016325851475",
                "image": "https://static.euw1.sustained.app/products/images/5016325851475_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07bc2b6e0c41b39a4befddaa814ea151",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07bc2b6e0c41b39a4befddaa814ea151/impacts"
                }
            },
            {
                "id": "productid.07efaa4b4ca19b2a9b77fc102ac6002b",
                "name": "Loch Ness Brewery Light Ness Session Pale Ale",
                "gtin": "5034866026039",
                "image": "https://static.euw1.sustained.app/products/images/5034866026039_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07efaa4b4ca19b2a9b77fc102ac6002b",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07efaa4b4ca19b2a9b77fc102ac6002b/impacts"
                }
            },
            {
                "id": "productid.0852efada9c1317f70980fe14420db08",
                "name": "Franciscan Well Rebel Red Ale",
                "gtin": "5010038467275",
                "image": "https://static.euw1.sustained.app/products/images/5010038467275_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0852efada9c1317f70980fe14420db08",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0852efada9c1317f70980fe14420db08/impacts"
                }
            },
            {
                "id": "productid.086e7a1c64bd6caafacc21b94c58ccef",
                "name": "Urban Fox Cunningly Distinctive American Red IPA",
                "gtin": "5021984109157",
                "image": "https://static.euw1.sustained.app/products/images/5021984109157_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.086e7a1c64bd6caafacc21b94c58ccef",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.086e7a1c64bd6caafacc21b94c58ccef/impacts"
                }
            },
            {
                "id": "productid.08a16f70d16c65e626b362e660fa10f0",
                "name": "Peroni Nastro Azzurro Lager Beer Bottle",
                "gtin": "8008440112286",
                "image": "https://static.euw1.sustained.app/products/images/8008440112286_0.jpg",
                "pack": "250ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08a16f70d16c65e626b362e660fa10f0",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08a16f70d16c65e626b362e660fa10f0/impacts"
                }
            },
            {
                "id": "productid.092ffe619af5d97396cbd7e042342623",
                "name": "Fyne Ales Workbench IPA",
                "gtin": "5060041571117",
                "image": "https://static.euw1.sustained.app/products/images/5060041571117_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.092ffe619af5d97396cbd7e042342623",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.092ffe619af5d97396cbd7e042342623/impacts"
                }
            },
            {
                "id": "productid.0ab7031fbb4dfd3b92a667230c9f0101",
                "name": "Hop House 13 Lager Beer",
                "gtin": "5000213023662",
                "image": "https://static.euw1.sustained.app/products/images/5000213023662_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ab7031fbb4dfd3b92a667230c9f0101",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ab7031fbb4dfd3b92a667230c9f0101/impacts"
                }
            },
            {
                "id": "productid.0aeca0853f1daa62e224808df2603d1c",
                "name": "Session Ale",
                "gtin": "00417266",
                "image": "https://static.euw1.sustained.app/products/images/00417266_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0aeca0853f1daa62e224808df2603d1c",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0aeca0853f1daa62e224808df2603d1c/impacts"
                }
            },
            {
                "id": "productid.0af86014f4ba489ce2232019ee9ce268",
                "name": "Stella Artois Cidre",
                "gtin": "5014379001587",
                "image": "https://static.euw1.sustained.app/products/images/5014379001587_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0af86014f4ba489ce2232019ee9ce268",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0af86014f4ba489ce2232019ee9ce268/impacts"
                }
            },
            {
                "id": "productid.0af9fcd70f295802d68516ae69276cb2",
                "name": "Dutch Gold",
                "gtin": "5390347300754",
                "image": "https://static.euw1.sustained.app/products/images/5390347300754_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0af9fcd70f295802d68516ae69276cb2",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0af9fcd70f295802d68516ae69276cb2/impacts"
                }
            },
            {
                "id": "productid.0b1474ff47be42beb85a5e393966eac6",
                "name": "Goose Island 312 Urban Wheat Ale Craft Beer Bottles",
                "gtin": "0736920111839",
                "image": "https://static.euw1.sustained.app/products/images/0736920111839_0.jpg",
                "pack": "355ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b1474ff47be42beb85a5e393966eac6",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b1474ff47be42beb85a5e393966eac6/impacts"
                }
            },
            {
                "id": "productid.0b9a882852ba1f7bb375032f75fa4ed2",
                "name": "Old Mout Cider Pineapple & Raspberry",
                "gtin": "5411718912809",
                "image": "https://static.euw1.sustained.app/products/images/5411718912809_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b9a882852ba1f7bb375032f75fa4ed2",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b9a882852ba1f7bb375032f75fa4ed2/impacts"
                }
            },
            {
                "id": "productid.0bef39b16c63447dacbc4d246739374b",
                "name": "Morrisons Lager Continental",
                "gtin": "5010251157212",
                "image": "https://static.euw1.sustained.app/products/images/5010251157212_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bef39b16c63447dacbc4d246739374b",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bef39b16c63447dacbc4d246739374b/impacts"
                }
            },
            {
                "id": "productid.0c05bc2aa018bb92254b8e1f50b3fea8",
                "name": "Open Gate Brewery Pure Brew Non-Alcoholic Lager",
                "gtin": "5000213021347",
                "image": "https://static.euw1.sustained.app/products/images/5000213021347_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.114109174210057
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c05bc2aa018bb92254b8e1f50b3fea8",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c05bc2aa018bb92254b8e1f50b3fea8/impacts"
                }
            },
            {
                "id": "productid.0c76ac5208035ba1c702b24b11e1e84b",
                "name": "The Vermont Sessions New England IPA Brewdog vs Northern Monk",
                "gtin": "5056025436879",
                "image": "https://static.euw1.sustained.app/products/images/5056025436879_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c76ac5208035ba1c702b24b11e1e84b",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c76ac5208035ba1c702b24b11e1e84b/impacts"
                }
            },
            {
                "id": "productid.0ce8d36c96d27a0174e5e1faf0e571c4",
                "name": "BrewDog Hazy Jane New England IPA",
                "gtin": "5056025434516",
                "image": "https://static.euw1.sustained.app/products/images/5056025434516_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ce8d36c96d27a0174e5e1faf0e571c4",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ce8d36c96d27a0174e5e1faf0e571c4/impacts"
                }
            },
            {
                "id": "productid.0d4a0c290d0a53f72dafca1fc326575d",
                "name": "Heineken",
                "gtin": "5035766064404",
                "image": "https://static.euw1.sustained.app/products/images/5035766064404_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d4a0c290d0a53f72dafca1fc326575d",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d4a0c290d0a53f72dafca1fc326575d/impacts"
                }
            },
            {
                "id": "productid.0e0438c02fe9b7b709e174d751b37ca1",
                "name": "Stowford Press Apple Cider PMP",
                "gtin": "5014201701562",
                "image": "https://static.euw1.sustained.app/products/images/5014201701562_0.jpg",
                "pack": "440ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e0438c02fe9b7b709e174d751b37ca1",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e0438c02fe9b7b709e174d751b37ca1/impacts"
                }
            },
            {
                "id": "productid.0ed22eb4af85bc4206f38047336d2630",
                "name": "Thatchers Gold Cider",
                "gtin": "5020628001178",
                "image": "https://static.euw1.sustained.app/products/images/5020628001178_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ed22eb4af85bc4206f38047336d2630",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ed22eb4af85bc4206f38047336d2630/impacts"
                }
            },
            {
                "id": "productid.10030e53f32408faa7623f5f603802ac",
                "name": "Peroni Nastro Azzurro",
                "gtin": "8008440322029",
                "image": "https://static.euw1.sustained.app/products/images/8008440322029_0.jpg",
                "pack": "50l",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10030e53f32408faa7623f5f603802ac",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10030e53f32408faa7623f5f603802ac/impacts"
                }
            },
            {
                "id": "productid.1070c4101b801e9a618cc6a056d72f66",
                "name": "Stella Artois Lager Beer Cans",
                "gtin": "5010017109851",
                "image": "https://static.euw1.sustained.app/products/images/5010017109851_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1070c4101b801e9a618cc6a056d72f66",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1070c4101b801e9a618cc6a056d72f66/impacts"
                }
            },
            {
                "id": "productid.10bb3c253b7e1297ef60e2e29c7c659d",
                "name": "Woodpecker Cider",
                "gtin": "5035766040125",
                "image": "https://static.euw1.sustained.app/products/images/5035766040125_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10bb3c253b7e1297ef60e2e29c7c659d",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10bb3c253b7e1297ef60e2e29c7c659d/impacts"
                }
            },
            {
                "id": "productid.116bef912234ed18dcd2cfd7f0d66023",
                "name": "Oakham Inferno Blonde Ale",
                "gtin": "5060044870071",
                "image": "https://static.euw1.sustained.app/products/images/5060044870071_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.116bef912234ed18dcd2cfd7f0d66023",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.116bef912234ed18dcd2cfd7f0d66023/impacts"
                }
            },
            {
                "id": "productid.11bad0dbc56da5abcb3f8f511c654a69",
                "name": "Coors Lager",
                "gtin": "5010038447475",
                "image": "https://static.euw1.sustained.app/products/images/5010038447475_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11bad0dbc56da5abcb3f8f511c654a69",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11bad0dbc56da5abcb3f8f511c654a69/impacts"
                }
            },
            {
                "id": "productid.11c4bc7fd46d353ffd8abdc0c301c1c1",
                "name": "Asda West Coast Pale",
                "gtin": "5060424341474",
                "image": "https://static.euw1.sustained.app/products/images/5060424341474_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11c4bc7fd46d353ffd8abdc0c301c1c1",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11c4bc7fd46d353ffd8abdc0c301c1c1/impacts"
                }
            },
            {
                "id": "productid.120d5c9be9476be7aa6b84ef5a485e53",
                "name": "Triple Secret Des Moines Blonde",
                "gtin": "3347970000270",
                "image": "https://static.euw1.sustained.app/products/images/3347970000270_0.jpg",
                "pack": "75cl",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.120d5c9be9476be7aa6b84ef5a485e53",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.120d5c9be9476be7aa6b84ef5a485e53/impacts"
                }
            },
            {
                "id": "productid.12376d90b1bad4b4af052226c824d855",
                "name": "Bulmers Irish Cider Original",
                "gtin": "5391516870054",
                "image": "https://static.euw1.sustained.app/products/images/5391516870054_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12376d90b1bad4b4af052226c824d855",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12376d90b1bad4b4af052226c824d855/impacts"
                }
            },
            {
                "id": "productid.124fc36853b4f1109f181edf9af4f2c0",
                "name": "Peroni Nastro Azzurro",
                "gtin": "8008440222015",
                "image": "https://static.euw1.sustained.app/products/images/8008440222015_0.jpg",
                "pack": "660ml",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Beer & Cider",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.124fc36853b4f1109f181edf9af4f2c0",
                    "category": "https://api.sustained.com/choice/v1/categories/beer-and-cider",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.124fc36853b4f1109f181edf9af4f2c0/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.003034a2555bf84b93a07c217f7638fc",
                "name": "Collection Italian Luxury Biscuit Collection",
                "gtin": "0000029078691",
                "image": "https://static.euw1.sustained.app/products/images/0000029078691_0.jpg",
                "pack": "420g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.003034a2555bf84b93a07c217f7638fc",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.003034a2555bf84b93a07c217f7638fc/impacts"
                }
            },
            {
                "id": "productid.005df910da5b45dfaeb0efa2890bff5f",
                "name": "Gluten-Free Dark Chocolate Shorts",
                "gtin": "8008698005347",
                "image": "https://static.euw1.sustained.app/products/images/8008698005347_0.jpg",
                "pack": "150g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.005df910da5b45dfaeb0efa2890bff5f",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.005df910da5b45dfaeb0efa2890bff5f/impacts"
                }
            },
            {
                "id": "productid.0133cccd72096c4a2fd6498aefb72f4b",
                "name": "Choc Chip Cookies",
                "gtin": "5051777000033",
                "image": "https://static.euw1.sustained.app/products/images/5051777000033_0.jpg",
                "pack": "150g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0133cccd72096c4a2fd6498aefb72f4b",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0133cccd72096c4a2fd6498aefb72f4b/impacts"
                }
            },
            {
                "id": "productid.013a2c6e1fea53858d5dd8afcfe6791e",
                "name": "Malted Milk",
                "gtin": "5010204765167",
                "image": "https://static.euw1.sustained.app/products/images/5010204765167_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 44.866185785241875
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.013a2c6e1fea53858d5dd8afcfe6791e",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.013a2c6e1fea53858d5dd8afcfe6791e/impacts"
                }
            },
            {
                "id": "productid.022c60c0048255fdf38c3ef23d9be651",
                "name": "Pure Butter Shortbread",
                "gtin": "0039047001138",
                "image": "https://static.euw1.sustained.app/products/images/0039047001138_0.jpg",
                "pack": "160g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.022c60c0048255fdf38c3ef23d9be651",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.022c60c0048255fdf38c3ef23d9be651/impacts"
                }
            },
            {
                "id": "productid.02b0fae19a25799db87b413c06bea102",
                "name": "Deliciously Free From Shortbread",
                "gtin": "0000000227681",
                "image": "https://static.euw1.sustained.app/products/images/0000000227681_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02b0fae19a25799db87b413c06bea102",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02b0fae19a25799db87b413c06bea102/impacts"
                }
            },
            {
                "id": "productid.02bd7ce7ef842ae8f7518068a9ddadb4",
                "name": "Food Multiseed Wholegrain Flatbreads",
                "gtin": "0000000139656",
                "image": "https://static.euw1.sustained.app/products/images/0000000139656_0.jpg",
                "pack": "150g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02bd7ce7ef842ae8f7518068a9ddadb4",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02bd7ce7ef842ae8f7518068a9ddadb4/impacts"
                }
            },
            {
                "id": "productid.02dd5fec55cc86ee4e8eef204e5eeb82",
                "name": "Yogurt Breaks Forest Fruit Biscuit Bars (5x35.5g)",
                "gtin": "5000168181578",
                "image": "https://static.euw1.sustained.app/products/images/5000168181578_0.jpg",
                "pack": "35.5g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02dd5fec55cc86ee4e8eef204e5eeb82",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02dd5fec55cc86ee4e8eef204e5eeb82/impacts"
                }
            },
            {
                "id": "productid.035b13e9769bb33a82d05e04939b3f87",
                "name": "Shortbread Fingers",
                "gtin": "20337148",
                "image": "https://static.euw1.sustained.app/products/images/20337148_0.jpg",
                "pack": "210g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.035b13e9769bb33a82d05e04939b3f87",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.035b13e9769bb33a82d05e04939b3f87/impacts"
                }
            },
            {
                "id": "productid.038e62319518af2c44f772ba0a814a20",
                "name": "Creations 205 Pompadour Fans Catering Pack",
                "gtin": "5029728545001",
                "image": "https://static.euw1.sustained.app/products/images/5029728545001_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.038e62319518af2c44f772ba0a814a20",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.038e62319518af2c44f772ba0a814a20/impacts"
                }
            },
            {
                "id": "productid.03beb985652e1de6bd785dcca154fdb9",
                "name": "The Best All Butter Scottish Shortbread Selection",
                "gtin": "5010525092553",
                "image": "https://static.euw1.sustained.app/products/images/5010525092553_0.jpg",
                "pack": "500g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03beb985652e1de6bd785dcca154fdb9",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03beb985652e1de6bd785dcca154fdb9/impacts"
                }
            },
            {
                "id": "productid.03d7f357f1b955f37985e60bc2e21d49",
                "name": "Pick Up! Choco",
                "gtin": "4017100263101",
                "image": "https://static.euw1.sustained.app/products/images/4017100263101_0.jpg",
                "pack": "28g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03d7f357f1b955f37985e60bc2e21d49",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03d7f357f1b955f37985e60bc2e21d49/impacts"
                }
            },
            {
                "id": "productid.03f00c33b5ff9fdd28959506999c8f9d",
                "name": "Crispy Milk-Hazelnut Wafer",
                "gtin": "0000040144382",
                "image": "https://static.euw1.sustained.app/products/images/0000040144382_0.jpg",
                "pack": "25g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03f00c33b5ff9fdd28959506999c8f9d",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03f00c33b5ff9fdd28959506999c8f9d/impacts"
                }
            },
            {
                "id": "productid.043732e571e493f4d9ad09819ef9c739",
                "name": "Chocolate Soft Baked Bears 8 Pack",
                "gtin": "7622201415600",
                "image": "https://static.euw1.sustained.app/products/images/7622201415600_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.043732e571e493f4d9ad09819ef9c739",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.043732e571e493f4d9ad09819ef9c739/impacts"
                }
            },
            {
                "id": "productid.04ffc95a321f0d885f3e1f7858819ec7",
                "name": "Food Collection Belgian Chocolate Caf\u00e9 Cream Biscuits",
                "gtin": "0000029063635",
                "image": "https://static.euw1.sustained.app/products/images/0000029063635_0.jpg",
                "pack": "430g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04ffc95a321f0d885f3e1f7858819ec7",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04ffc95a321f0d885f3e1f7858819ec7/impacts"
                }
            },
            {
                "id": "productid.05a8bddea324b3a83f6293c984a171a7",
                "name": "Original Sourdough Crispbread",
                "gtin": "5060198820939",
                "image": "https://static.euw1.sustained.app/products/images/5060198820939_0.jpg",
                "pack": "140g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05a8bddea324b3a83f6293c984a171a7",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05a8bddea324b3a83f6293c984a171a7/impacts"
                }
            },
            {
                "id": "productid.05edbb86c0737b748f03cc8d12ea99e1",
                "name": "Delicious Shortbread Assortment",
                "gtin": "5010745055604",
                "image": "https://static.euw1.sustained.app/products/images/5010745055604_0.jpg",
                "pack": "1kg",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05edbb86c0737b748f03cc8d12ea99e1",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05edbb86c0737b748f03cc8d12ea99e1/impacts"
                }
            },
            {
                "id": "productid.0636e1a3e81704d9427cd6ecfc2be618",
                "name": "6 Rice Cakes Yoghurt",
                "gtin": "5060262484951",
                "image": "https://static.euw1.sustained.app/products/images/5060262484951_0.jpg",
                "pack": "100g",
                "grade": "D",
                "points": {
                    "pef": 26.62527639287613
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0636e1a3e81704d9427cd6ecfc2be618",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0636e1a3e81704d9427cd6ecfc2be618/impacts"
                }
            },
            {
                "id": "productid.0639c62060ec0cb6e25e04089d8b1483",
                "name": "All Butter Viennese Raspberry Creams",
                "gtin": "0000000028356",
                "image": "https://static.euw1.sustained.app/products/images/0000000028356_0.jpg",
                "pack": "125g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0639c62060ec0cb6e25e04089d8b1483",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0639c62060ec0cb6e25e04089d8b1483/impacts"
                }
            },
            {
                "id": "productid.066c86c21ef85898935fe6fa36aa72e6",
                "name": "Yoghurt Topped Rice Cakes",
                "gtin": "5013665113072",
                "image": "https://static.euw1.sustained.app/products/images/5013665113072_0.jpg",
                "pack": "100g",
                "grade": "D",
                "points": {
                    "pef": 26.57640402747611
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.066c86c21ef85898935fe6fa36aa72e6",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.066c86c21ef85898935fe6fa36aa72e6/impacts"
                }
            },
            {
                "id": "productid.068b71f8a635bdf1d06b3e4150249bdc",
                "name": "No1 Belgian Dark Dipped Chocolate Shortbread",
                "gtin": "5000169571538",
                "image": "https://static.euw1.sustained.app/products/images/5000169571538_0.jpg",
                "pack": "135g",
                "grade": "F",
                "points": {
                    "pef": 77.35102804145704
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.068b71f8a635bdf1d06b3e4150249bdc",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.068b71f8a635bdf1d06b3e4150249bdc/impacts"
                }
            },
            {
                "id": "productid.06c3db787602bfd2c1f5bf8d3f5c7334",
                "name": "Chocolate Biscuit Fingers Multipack",
                "gtin": "5000159430678",
                "image": "https://static.euw1.sustained.app/products/images/5000159430678_0.jpg",
                "pack": "23g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06c3db787602bfd2c1f5bf8d3f5c7334",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06c3db787602bfd2c1f5bf8d3f5c7334/impacts"
                }
            },
            {
                "id": "productid.06ddca5d114fefe2f1c003c4ede20ef6",
                "name": "Sainsbury's Digestives",
                "gtin": "00448383",
                "image": "https://static.euw1.sustained.app/products/images/00448383_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06ddca5d114fefe2f1c003c4ede20ef6",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06ddca5d114fefe2f1c003c4ede20ef6/impacts"
                }
            },
            {
                "id": "productid.0700f12dd990be966ed6f7274f544a99",
                "name": "Cookies Milk Chocolate",
                "gtin": "7622210809049",
                "image": "https://static.euw1.sustained.app/products/images/7622210809049_0.jpg",
                "pack": "135g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0700f12dd990be966ed6f7274f544a99",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0700f12dd990be966ed6f7274f544a99/impacts"
                }
            },
            {
                "id": "productid.083965ff2e3fc8b185b1056d5aa2c277",
                "name": "Love Life Plain Breadsticks",
                "gtin": "5000169026335",
                "image": "https://static.euw1.sustained.app/products/images/5000169026335_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 11.56958710985014
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.083965ff2e3fc8b185b1056d5aa2c277",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.083965ff2e3fc8b185b1056d5aa2c277/impacts"
                }
            },
            {
                "id": "productid.0918005afa481e73f5c0551997ae56b1",
                "name": "Malted Milk Biscuits",
                "gtin": "5010251630814",
                "image": "https://static.euw1.sustained.app/products/images/5010251630814_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0918005afa481e73f5c0551997ae56b1",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0918005afa481e73f5c0551997ae56b1/impacts"
                }
            },
            {
                "id": "productid.092e69f58e6c0d38b8df6875d96debd5",
                "name": "Club Mint Bars",
                "gtin": "5000168210728",
                "image": "https://static.euw1.sustained.app/products/images/5000168210728_0.jpg",
                "pack": "22g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.092e69f58e6c0d38b8df6875d96debd5",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.092e69f58e6c0d38b8df6875d96debd5/impacts"
                }
            },
            {
                "id": "productid.0a15c36751a83be9ade37b807caa0101",
                "name": "Extra Special Dark Chocolate Ginger Thins",
                "gtin": "5057172728961",
                "image": "https://static.euw1.sustained.app/products/images/5057172728961_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 94.14747184106628
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a15c36751a83be9ade37b807caa0101",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a15c36751a83be9ade37b807caa0101/impacts"
                }
            },
            {
                "id": "productid.0a6fc224ce4746900dd22551b47b42ba",
                "name": "Craft Bakery 2 Cream Cookies",
                "gtin": "5029912014030",
                "image": "https://static.euw1.sustained.app/products/images/5029912014030_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 18.034080732353328
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a6fc224ce4746900dd22551b47b42ba",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a6fc224ce4746900dd22551b47b42ba/impacts"
                }
            },
            {
                "id": "productid.0ad874ffd1017b2914b255f9290547a1",
                "name": "Grissini Sesame Italian Breadsticks",
                "gtin": "8003899181203",
                "image": "https://static.euw1.sustained.app/products/images/8003899181203_0.jpg",
                "pack": "125g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ad874ffd1017b2914b255f9290547a1",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ad874ffd1017b2914b255f9290547a1/impacts"
                }
            },
            {
                "id": "productid.0b0362f230eda6b12cec220c8a68ea4d",
                "name": "Mummy and Me Heart Message Biscuit",
                "gtin": "5054402323286",
                "image": "https://static.euw1.sustained.app/products/images/5054402323286_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 44.866185785241875
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b0362f230eda6b12cec220c8a68ea4d",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b0362f230eda6b12cec220c8a68ea4d/impacts"
                }
            },
            {
                "id": "productid.0b569d59bc9faa84c31022f2c03d7a5b",
                "name": "Butterscotch Crunch",
                "gtin": "0652655001088",
                "image": "https://static.euw1.sustained.app/products/images/0652655001088_0.jpg",
                "pack": "135g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b569d59bc9faa84c31022f2c03d7a5b",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b569d59bc9faa84c31022f2c03d7a5b/impacts"
                }
            },
            {
                "id": "productid.0bc36de53138d9acae5bb9a8af069035",
                "name": "Baked Snack Crackers Cheddar",
                "gtin": "0014100085393",
                "image": "https://static.euw1.sustained.app/products/images/0014100085393_0.jpg",
                "pack": "187g",
                "grade": "D",
                "points": {
                    "pef": 22.516785086530177
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bc36de53138d9acae5bb9a8af069035",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bc36de53138d9acae5bb9a8af069035/impacts"
                }
            },
            {
                "id": "productid.0e0ef2cf154f10fdee8197bf5cad9ef6",
                "name": "Original Cream Crackers",
                "gtin": "5000137415970",
                "image": "https://static.euw1.sustained.app/products/images/5000137415970_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e0ef2cf154f10fdee8197bf5cad9ef6",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e0ef2cf154f10fdee8197bf5cad9ef6/impacts"
                }
            },
            {
                "id": "productid.0e3f93969d0b0e4d032db6f5a346a825",
                "name": "4 Milk Chocolate Chunk Cookies",
                "gtin": "5055552401350",
                "image": "https://static.euw1.sustained.app/products/images/5055552401350_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e3f93969d0b0e4d032db6f5a346a825",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e3f93969d0b0e4d032db6f5a346a825/impacts"
                }
            },
            {
                "id": "productid.0e5b4807ddb1766c966000cf1ccca7ed",
                "name": "Duchy Organic Orange Biscuits Coated in Dark Chocolate",
                "gtin": "5000169149812",
                "image": "https://static.euw1.sustained.app/products/images/5000169149812_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 86.76746947023173
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e5b4807ddb1766c966000cf1ccca7ed",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e5b4807ddb1766c966000cf1ccca7ed/impacts"
                }
            },
            {
                "id": "productid.0e8dae52bff272f8435e96a667b1aef2",
                "name": "Vegan Coconut Crunch Biscuit Share Bag",
                "gtin": "7640155340533",
                "image": "https://static.euw1.sustained.app/products/images/7640155340533_0.jpg",
                "pack": "135g",
                "grade": "E",
                "points": {
                    "pef": 33.04037957649413
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e8dae52bff272f8435e96a667b1aef2",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e8dae52bff272f8435e96a667b1aef2/impacts"
                }
            },
            {
                "id": "productid.0ee96205206c538695b9ff57c360eb81",
                "name": "Free From Cheese Crackers",
                "gtin": "5010251782940",
                "image": "https://static.euw1.sustained.app/products/images/5010251782940_0.jpg",
                "pack": "125g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ee96205206c538695b9ff57c360eb81",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ee96205206c538695b9ff57c360eb81/impacts"
                }
            },
            {
                "id": "productid.0fa1becbddf24763c8971c40c0029dbf",
                "name": "Oats & Grains Peanut & Caramel Bars",
                "gtin": "5056357907597",
                "image": "https://static.euw1.sustained.app/products/images/5056357907597_0.jpg",
                "pack": "22.5g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fa1becbddf24763c8971c40c0029dbf",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fa1becbddf24763c8971c40c0029dbf/impacts"
                }
            },
            {
                "id": "productid.101bba50ce38ab1302d792bf6f687f71",
                "name": "Peanut Cookies",
                "gtin": "8710508913529",
                "image": "https://static.euw1.sustained.app/products/images/8710508913529_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.101bba50ce38ab1302d792bf6f687f71",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.101bba50ce38ab1302d792bf6f687f71/impacts"
                }
            },
            {
                "id": "productid.103bacf7dd8de642b375b3b309c93d51",
                "name": "of Yorkshire Mini Snackers Herb & Onion Crackers",
                "gtin": "5010112004785",
                "image": "https://static.euw1.sustained.app/products/images/5010112004785_0.jpg",
                "pack": "85g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.103bacf7dd8de642b375b3b309c93d51",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.103bacf7dd8de642b375b3b309c93d51/impacts"
                }
            },
            {
                "id": "productid.10bba8814fa61996417b3d759b5c70a4",
                "name": "Easter Bunny Gingerbread Biscuit",
                "gtin": "5010525165844",
                "image": "https://static.euw1.sustained.app/products/images/5010525165844_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10bba8814fa61996417b3d759b5c70a4",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10bba8814fa61996417b3d759b5c70a4/impacts"
                }
            },
            {
                "id": "productid.10bc4be1779a273ad09d30430506a2f0",
                "name": "SMA LITTLE STEPS Organic Tomato Savoury Crackers 100g, Baby Snack 12 Months+, 16 Portions Per Bag",
                "gtin": "8000300395471",
                "image": "https://static.euw1.sustained.app/products/images/8000300395471_0.jpg",
                "pack": "100g",
                "grade": "C",
                "points": {
                    "pef": 18.168430987904163
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10bc4be1779a273ad09d30430506a2f0",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10bc4be1779a273ad09d30430506a2f0/impacts"
                }
            },
            {
                "id": "productid.12e1851469a45dccce46a6772f259e6e",
                "name": "Food Crispy Cheese Crackers",
                "gtin": "0000000130912",
                "image": "https://static.euw1.sustained.app/products/images/0000000130912_0.jpg",
                "pack": "150g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12e1851469a45dccce46a6772f259e6e",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12e1851469a45dccce46a6772f259e6e/impacts"
                }
            },
            {
                "id": "productid.135be51b2a008a210722e0f14edca4d2",
                "name": "Organic Rye Kn\u00e4cke Sesame Crispbread",
                "gtin": "5032722318137",
                "image": "https://static.euw1.sustained.app/products/images/5032722318137_0.jpg",
                "pack": "200g",
                "grade": "B",
                "points": {
                    "pef": 14.056211268632643
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.135be51b2a008a210722e0f14edca4d2",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.135be51b2a008a210722e0f14edca4d2/impacts"
                }
            },
            {
                "id": "productid.136755ac70ebdda47d35213a43ecda50",
                "name": "Original Rye Crispbread",
                "gtin": "5057008761377",
                "image": "https://static.euw1.sustained.app/products/images/5057008761377_0.jpg",
                "pack": "250g",
                "grade": "A",
                "points": {
                    "pef": 10.509467853519002
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.136755ac70ebdda47d35213a43ecda50",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.136755ac70ebdda47d35213a43ecda50/impacts"
                }
            },
            {
                "id": "productid.1385af39bef9fa635a6665be841845b4",
                "name": "Mini Rice Crackers Seaweed Flavour",
                "gtin": "4710144900154",
                "image": "https://static.euw1.sustained.app/products/images/4710144900154_0.jpg",
                "pack": "60g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1385af39bef9fa635a6665be841845b4",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1385af39bef9fa635a6665be841845b4/impacts"
                }
            },
            {
                "id": "productid.14e91296861e0bc1ee71849899721900",
                "name": "Jaffa Cakes 100% Extra Free",
                "gtin": "5011005056362",
                "image": "https://static.euw1.sustained.app/products/images/5011005056362_0.jpg",
                "pack": "294g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14e91296861e0bc1ee71849899721900",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14e91296861e0bc1ee71849899721900/impacts"
                }
            },
            {
                "id": "productid.155c347919fe3771dff57d50cecb1f20",
                "name": "Peri Peri Spicy Oatcakes",
                "gtin": "0612322001216",
                "image": "https://static.euw1.sustained.app/products/images/0612322001216_0.jpg",
                "pack": "200g",
                "grade": "C",
                "points": {
                    "pef": 16.446323728188773
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.155c347919fe3771dff57d50cecb1f20",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.155c347919fe3771dff57d50cecb1f20/impacts"
                }
            },
            {
                "id": "productid.167bd0cee4db61d147347b3827c315f0",
                "name": "Choccy Rocks Moofreesas",
                "gtin": "5060235831454",
                "image": "https://static.euw1.sustained.app/products/images/5060235831454_0.jpg",
                "pack": "35g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Biscuits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.167bd0cee4db61d147347b3827c315f0",
                    "category": "https://api.sustained.com/choice/v1/categories/biscuits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.167bd0cee4db61d147347b3827c315f0/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.0012124ec5e5a585c220476b669fa06f",
                "name": "4 Hot Cross Buns",
                "gtin": "5099023123440",
                "image": "https://static.euw1.sustained.app/products/images/5099023123440_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0012124ec5e5a585c220476b669fa06f",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0012124ec5e5a585c220476b669fa06f/impacts"
                }
            },
            {
                "id": "productid.006dbef4da76143355781a6a92214619",
                "name": "White Family Pan",
                "gtin": "5052320937400",
                "image": "https://static.euw1.sustained.app/products/images/5052320937400_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 11.56958710985014
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.006dbef4da76143355781a6a92214619",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.006dbef4da76143355781a6a92214619/impacts"
                }
            },
            {
                "id": "productid.00e067714b610934b223439f3f274e51",
                "name": "Chosen by You White Tin Loaf",
                "gtin": "0230010000008",
                "image": "https://static.euw1.sustained.app/products/images/0230010000008_0.jpg",
                "pack": "800g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00e067714b610934b223439f3f274e51",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00e067714b610934b223439f3f274e51/impacts"
                }
            },
            {
                "id": "productid.018ee793a1e1a8b2dc2755f7e545a5fa",
                "name": "Taste the Difference 4 Sesame Bagels",
                "gtin": "00434737",
                "image": "https://static.euw1.sustained.app/products/images/00434737_0.jpg",
                "pack": "320g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.018ee793a1e1a8b2dc2755f7e545a5fa",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.018ee793a1e1a8b2dc2755f7e545a5fa/impacts"
                }
            },
            {
                "id": "productid.01ad0a6e9b65e46c508a8b3b5bfe2c4c",
                "name": "6 Large Flour Tortillas",
                "gtin": "8410076470300",
                "image": "https://static.euw1.sustained.app/products/images/8410076470300_0.jpg",
                "pack": "350g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01ad0a6e9b65e46c508a8b3b5bfe2c4c",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01ad0a6e9b65e46c508a8b3b5bfe2c4c/impacts"
                }
            },
            {
                "id": "productid.022015f8a91e14d51269573278607341",
                "name": "Multigrain Batch",
                "gtin": "5057545928912",
                "image": "https://static.euw1.sustained.app/products/images/5057545928912_0.jpg",
                "pack": "800g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.022015f8a91e14d51269573278607341",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.022015f8a91e14d51269573278607341/impacts"
                }
            },
            {
                "id": "productid.025d2048ebc3239b716daaf37dae4e85",
                "name": "Chosen by You Tiger Tin Loaf",
                "gtin": "0250600000003",
                "image": "https://static.euw1.sustained.app/products/images/0250600000003_0.jpg",
                "pack": "800g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.025d2048ebc3239b716daaf37dae4e85",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.025d2048ebc3239b716daaf37dae4e85/impacts"
                }
            },
            {
                "id": "productid.0282fbe0a342cbc415c6b4d092cea00d",
                "name": "Slimsters Squares White & Wholemeal Bread",
                "gtin": "5099268044906",
                "image": "https://static.euw1.sustained.app/products/images/5099268044906_0.jpg",
                "pack": "164g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0282fbe0a342cbc415c6b4d092cea00d",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0282fbe0a342cbc415c6b4d092cea00d/impacts"
                }
            },
            {
                "id": "productid.028f2cb2e4740a95f9254bc582be5855",
                "name": "Food 4 Five Grain Bagels",
                "gtin": "0000029143504",
                "image": "https://static.euw1.sustained.app/products/images/0000029143504_0.jpg",
                "pack": "300g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.028f2cb2e4740a95f9254bc582be5855",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.028f2cb2e4740a95f9254bc582be5855/impacts"
                }
            },
            {
                "id": "productid.029e068bb60dee84f984bafe115f0651",
                "name": "1 Elderflower, Lemon & Lovage Hot Cross Buns",
                "gtin": "5000169543153",
                "image": "https://static.euw1.sustained.app/products/images/5000169543153_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.029e068bb60dee84f984bafe115f0651",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.029e068bb60dee84f984bafe115f0651/impacts"
                }
            },
            {
                "id": "productid.0313aab51fcb99976861195147162459",
                "name": "Roasted Garlic Ciabatta",
                "gtin": "5000169137000",
                "image": "https://static.euw1.sustained.app/products/images/5000169137000_0.jpg",
                "pack": "275g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0313aab51fcb99976861195147162459",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0313aab51fcb99976861195147162459/impacts"
                }
            },
            {
                "id": "productid.031d01a220a08f3c12b4f86e0d5532b9",
                "name": "Bakery White Split Tin Loaf Sliced",
                "gtin": "0000001052220",
                "image": "https://static.euw1.sustained.app/products/images/0000001052220_0.jpg",
                "pack": "800g",
                "grade": "C",
                "points": {
                    "pef": 21.57773710653433
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.031d01a220a08f3c12b4f86e0d5532b9",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.031d01a220a08f3c12b4f86e0d5532b9/impacts"
                }
            },
            {
                "id": "productid.031fb05f59822df6f52ac547c055922f",
                "name": "White Quinoa Vegan Loaf",
                "gtin": "5060509390076",
                "image": "https://static.euw1.sustained.app/products/images/5060509390076_0.jpg",
                "pack": "550g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.031fb05f59822df6f52ac547c055922f",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.031fb05f59822df6f52ac547c055922f/impacts"
                }
            },
            {
                "id": "productid.038b2263d1c1ef6c24d7a1cc884cef0d",
                "name": "Gluten Free 4 Plain Bagels",
                "gtin": "5060195906360",
                "image": "https://static.euw1.sustained.app/products/images/5060195906360_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.038b2263d1c1ef6c24d7a1cc884cef0d",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.038b2263d1c1ef6c24d7a1cc884cef0d/impacts"
                }
            },
            {
                "id": "productid.03db6efef7f47caa73f389557ccb527a",
                "name": "Sticky Cinnamon Swirls",
                "gtin": "0000000336611",
                "image": "https://static.euw1.sustained.app/products/images/0000000336611_0.jpg",
                "pack": "349g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03db6efef7f47caa73f389557ccb527a",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03db6efef7f47caa73f389557ccb527a/impacts"
                }
            },
            {
                "id": "productid.03e53e22c9062fea3dba48166517cfb8",
                "name": "4 Hot Cross Buns",
                "gtin": "5010092705115",
                "image": "https://static.euw1.sustained.app/products/images/5010092705115_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03e53e22c9062fea3dba48166517cfb8",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03e53e22c9062fea3dba48166517cfb8/impacts"
                }
            },
            {
                "id": "productid.04ab4f88f284fce5a36b637dab09eb7e",
                "name": "Sliced White Pan",
                "gtin": "5099061000246",
                "image": "https://static.euw1.sustained.app/products/images/5099061000246_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04ab4f88f284fce5a36b637dab09eb7e",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04ab4f88f284fce5a36b637dab09eb7e/impacts"
                }
            },
            {
                "id": "productid.04ef6768e76e22cbdc1ee705cba949f4",
                "name": "Taste the Difference Summer Edition Soft Milk Rolls",
                "gtin": "0000000462167",
                "image": "https://static.euw1.sustained.app/products/images/0000000462167_0.jpg",
                "pack": "160g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04ef6768e76e22cbdc1ee705cba949f4",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04ef6768e76e22cbdc1ee705cba949f4/impacts"
                }
            },
            {
                "id": "productid.06029e5e4992e79bcc61b225e60ad915",
                "name": "Wholewheat Farmhouse Soda Bread",
                "gtin": "5099867000686",
                "image": "https://static.euw1.sustained.app/products/images/5099867000686_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06029e5e4992e79bcc61b225e60ad915",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06029e5e4992e79bcc61b225e60ad915/impacts"
                }
            },
            {
                "id": "productid.07e40c4a687df78cdc9ca9267c44e342",
                "name": "Bakery Poppy Seed Baguette",
                "gtin": "0000001053159",
                "image": "https://static.euw1.sustained.app/products/images/0000001053159_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 12.856364462263942
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07e40c4a687df78cdc9ca9267c44e342",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07e40c4a687df78cdc9ca9267c44e342/impacts"
                }
            },
            {
                "id": "productid.089e989eee7b637c3893364132e77e04",
                "name": "Essential Waitrose & Partners 9 Garlic Slices",
                "gtin": "5000169149393",
                "image": "https://static.euw1.sustained.app/products/images/5000169149393_0.jpg",
                "pack": "270g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.089e989eee7b637c3893364132e77e04",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.089e989eee7b637c3893364132e77e04/impacts"
                }
            },
            {
                "id": "productid.08b8e58de37b6dc605e5e3f64a72b2e3",
                "name": "6 Plain Wraps",
                "gtin": "5012121005630",
                "image": "https://static.euw1.sustained.app/products/images/5012121005630_0.jpg",
                "pack": "366g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08b8e58de37b6dc605e5e3f64a72b2e3",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08b8e58de37b6dc605e5e3f64a72b2e3/impacts"
                }
            },
            {
                "id": "productid.08bf465e0b22abc9097d065d77b12043",
                "name": "Seed Croutons with Caraway Seeds",
                "gtin": "5000169331385",
                "image": "https://static.euw1.sustained.app/products/images/5000169331385_0.jpg",
                "pack": "100g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08bf465e0b22abc9097d065d77b12043",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08bf465e0b22abc9097d065d77b12043/impacts"
                }
            },
            {
                "id": "productid.08d798a17f49c79e5fd155a1691d4c52",
                "name": "Taste the Difference Autumn Edition 2 Chilli & Coriander Wheat & Corn Flatbreads",
                "gtin": "00471060",
                "image": "https://static.euw1.sustained.app/products/images/00471060_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08d798a17f49c79e5fd155a1691d4c52",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08d798a17f49c79e5fd155a1691d4c52/impacts"
                }
            },
            {
                "id": "productid.08f78d3aac74a51e97e5aee9a66463a7",
                "name": "White Farmhouse Sliced",
                "gtin": "0282760000000",
                "image": "https://static.euw1.sustained.app/products/images/0282760000000_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 11.56958710985014
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08f78d3aac74a51e97e5aee9a66463a7",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08f78d3aac74a51e97e5aee9a66463a7/impacts"
                }
            },
            {
                "id": "productid.08fabac23a50c321778d0e52cc359294",
                "name": "Tiger Bloomer",
                "gtin": "0280880000000",
                "image": "https://static.euw1.sustained.app/products/images/0280880000000_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 11.56958710985014
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08fabac23a50c321778d0e52cc359294",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08fabac23a50c321778d0e52cc359294/impacts"
                }
            },
            {
                "id": "productid.08fb9540a980aa641d5e512f3f2e62bc",
                "name": "4 Skinny Italian Pizza Bases",
                "gtin": "5099077004061",
                "image": "https://static.euw1.sustained.app/products/images/5099077004061_0.jpg",
                "pack": "360g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08fb9540a980aa641d5e512f3f2e62bc",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08fb9540a980aa641d5e512f3f2e62bc/impacts"
                }
            },
            {
                "id": "productid.090d95ea1e7afb3c6aaafcb51832a36f",
                "name": "6 Chocolate Orange Hot Cross Buns",
                "gtin": "5010251778738",
                "image": "https://static.euw1.sustained.app/products/images/5010251778738_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.090d95ea1e7afb3c6aaafcb51832a36f",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.090d95ea1e7afb3c6aaafcb51832a36f/impacts"
                }
            },
            {
                "id": "productid.097c7571702a74fc42859e02f88e6cd3",
                "name": "Food Brewers Malted Barley Cob",
                "gtin": "0000029204908",
                "image": "https://static.euw1.sustained.app/products/images/0000029204908_0.jpg",
                "pack": "450g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.097c7571702a74fc42859e02f88e6cd3",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.097c7571702a74fc42859e02f88e6cd3/impacts"
                }
            },
            {
                "id": "productid.0aa582dbd984e53ba67a9fc5e1e26206",
                "name": "Free From 50/50 Loaf",
                "gtin": "5057172515684",
                "image": "https://static.euw1.sustained.app/products/images/5057172515684_0.jpg",
                "pack": "460g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0aa582dbd984e53ba67a9fc5e1e26206",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0aa582dbd984e53ba67a9fc5e1e26206/impacts"
                }
            },
            {
                "id": "productid.0ae9b23f021110b177fbe63c475120ca",
                "name": "12 Fruit Buns",
                "gtin": "5099823001245",
                "image": "https://static.euw1.sustained.app/products/images/5099823001245_0.jpg",
                "pack": "320g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ae9b23f021110b177fbe63c475120ca",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ae9b23f021110b177fbe63c475120ca/impacts"
                }
            },
            {
                "id": "productid.0b8348144c688cf3abaf015b4ab201e5",
                "name": "20 Cocktail Sausage Rolls",
                "gtin": "5027951007389",
                "image": "https://static.euw1.sustained.app/products/images/5027951007389_0.jpg",
                "pack": "320g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b8348144c688cf3abaf015b4ab201e5",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b8348144c688cf3abaf015b4ab201e5/impacts"
                }
            },
            {
                "id": "productid.0b8f499b8cd2cd429f035665530d5c12",
                "name": "Food Super Seeded Loaf",
                "gtin": "0000000642118",
                "image": "https://static.euw1.sustained.app/products/images/0000000642118_0.jpg",
                "pack": "800g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b8f499b8cd2cd429f035665530d5c12",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b8f499b8cd2cd429f035665530d5c12/impacts"
                }
            },
            {
                "id": "productid.0ba33de48ec0dc3dd3d8875e36399cee",
                "name": "Craft Bakery 4 Floury Buns",
                "gtin": "5029912003294",
                "image": "https://static.euw1.sustained.app/products/images/5029912003294_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 9.62384206102552
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ba33de48ec0dc3dd3d8875e36399cee",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ba33de48ec0dc3dd3d8875e36399cee/impacts"
                }
            },
            {
                "id": "productid.0ba8c8d2d1d1aef85337e90bac98ca8c",
                "name": "Essential Waitrose & Partners 6 White Floured Baps",
                "gtin": "5000169014721",
                "image": "https://static.euw1.sustained.app/products/images/5000169014721_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ba8c8d2d1d1aef85337e90bac98ca8c",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ba8c8d2d1d1aef85337e90bac98ca8c/impacts"
                }
            },
            {
                "id": "productid.0bdde412859351870a6f11c633b43944",
                "name": "Poppy Vienna",
                "gtin": "0289180000000",
                "image": "https://static.euw1.sustained.app/products/images/0289180000000_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 11.56958710985014
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bdde412859351870a6f11c633b43944",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bdde412859351870a6f11c633b43944/impacts"
                }
            },
            {
                "id": "productid.0c1f263a04a212d36557431d171a34e0",
                "name": "The Baker's On Market Street Golden Sunflower & Pumpkin Seed Boule",
                "gtin": "0202511001006",
                "image": "https://static.euw1.sustained.app/products/images/0202511001006_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c1f263a04a212d36557431d171a34e0",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c1f263a04a212d36557431d171a34e0/impacts"
                }
            },
            {
                "id": "productid.0d5ab54ffb66a292dcdea11434890be3",
                "name": "Traditional Brown Soda Sliced Brown Bread",
                "gtin": "5011041001692",
                "image": "https://static.euw1.sustained.app/products/images/5011041001692_0.jpg",
                "pack": "500g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d5ab54ffb66a292dcdea11434890be3",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d5ab54ffb66a292dcdea11434890be3/impacts"
                }
            },
            {
                "id": "productid.0e22dc226b58dc33b24c419ea94b0dd1",
                "name": "Family Bread Long Loaf Malted Grain Thick Sliced Bread",
                "gtin": "5011376000186",
                "image": "https://static.euw1.sustained.app/products/images/5011376000186_0.jpg",
                "pack": "800g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e22dc226b58dc33b24c419ea94b0dd1",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e22dc226b58dc33b24c419ea94b0dd1/impacts"
                }
            },
            {
                "id": "productid.0e54ec7228c8da30607e9e232b13b829",
                "name": "Chosen by You Burger Buns",
                "gtin": "0210890000008",
                "image": "https://static.euw1.sustained.app/products/images/0210890000008_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e54ec7228c8da30607e9e232b13b829",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e54ec7228c8da30607e9e232b13b829/impacts"
                }
            },
            {
                "id": "productid.0ea71a14457c2a78deb859185cf76c68",
                "name": "Karaway Bakery 100% Rye Bread with Grains",
                "gtin": "5060172065707",
                "image": "https://static.euw1.sustained.app/products/images/5060172065707_0.jpg",
                "pack": "600g",
                "grade": "B",
                "points": {
                    "pef": 15.634414176709257
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ea71a14457c2a78deb859185cf76c68",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ea71a14457c2a78deb859185cf76c68/impacts"
                }
            },
            {
                "id": "productid.0f16af38d0bbec3f1458009b87dcea0b",
                "name": "Elior White Sandwich Bread Thick 14 Slice",
                "gtin": "5000227203395",
                "image": "https://static.euw1.sustained.app/products/images/5000227203395_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f16af38d0bbec3f1458009b87dcea0b",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f16af38d0bbec3f1458009b87dcea0b/impacts"
                }
            },
            {
                "id": "productid.0f47a74007badc0c915cc2f17397f2aa",
                "name": "8 Brioche Rolls Milk Chocolate Chip",
                "gtin": "5010251758600",
                "image": "https://static.euw1.sustained.app/products/images/5010251758600_0.jpg",
                "pack": "280g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f47a74007badc0c915cc2f17397f2aa",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f47a74007badc0c915cc2f17397f2aa/impacts"
                }
            },
            {
                "id": "productid.0f66e9d9f0c482dce358e2faab465062",
                "name": "Gluten Free 3 Stonebaked Pitta Breads",
                "gtin": "5060195909002",
                "image": "https://static.euw1.sustained.app/products/images/5060195909002_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f66e9d9f0c482dce358e2faab465062",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f66e9d9f0c482dce358e2faab465062/impacts"
                }
            },
            {
                "id": "productid.0f7a54c75e5535dc2d8937df782b6d99",
                "name": "Gluten Free Spicy Fruit Loaf",
                "gtin": "5060195900498",
                "image": "https://static.euw1.sustained.app/products/images/5060195900498_0.jpg",
                "pack": "400g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f7a54c75e5535dc2d8937df782b6d99",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f7a54c75e5535dc2d8937df782b6d99/impacts"
                }
            },
            {
                "id": "productid.108bd5965011c9ab9758be67d3e0e2c7",
                "name": "4 Bundys Floury Baps",
                "gtin": "5099867002178",
                "image": "https://static.euw1.sustained.app/products/images/5099867002178_0.jpg",
                "pack": "320g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.108bd5965011c9ab9758be67d3e0e2c7",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.108bd5965011c9ab9758be67d3e0e2c7/impacts"
                }
            },
            {
                "id": "productid.112ac1ac828452c38704194a8b21a3d0",
                "name": "Softee Soft White Medium Sliced Bread",
                "gtin": "5013427014272",
                "image": "https://static.euw1.sustained.app/products/images/5013427014272_0.jpg",
                "pack": "800g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.112ac1ac828452c38704194a8b21a3d0",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.112ac1ac828452c38704194a8b21a3d0/impacts"
                }
            },
            {
                "id": "productid.1151db5c5bf08b41b49c41c8ba32e636",
                "name": "4 York Baps",
                "gtin": "5060047682800",
                "image": "https://static.euw1.sustained.app/products/images/5060047682800_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1151db5c5bf08b41b49c41c8ba32e636",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1151db5c5bf08b41b49c41c8ba32e636/impacts"
                }
            },
            {
                "id": "productid.11d85b59dc9e3fe17cb887e5ed00c4e8",
                "name": "Plain Nan",
                "gtin": "5030197123454",
                "image": "https://static.euw1.sustained.app/products/images/5030197123454_0.jpg",
                "pack": "375g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11d85b59dc9e3fe17cb887e5ed00c4e8",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11d85b59dc9e3fe17cb887e5ed00c4e8/impacts"
                }
            },
            {
                "id": "productid.1213e367403dd86253520094d2753d1f",
                "name": "Golden Bake Medium Sliced White Bread",
                "gtin": "5013427010342",
                "image": "https://static.euw1.sustained.app/products/images/5013427010342_0.jpg",
                "pack": "800g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Bread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1213e367403dd86253520094d2753d1f",
                    "category": "https://api.sustained.com/choice/v1/categories/bread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1213e367403dd86253520094d2753d1f/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.00b8855750fb9819de0fb987818151e9",
                "name": "Family Bread 4 Sultana Scones",
                "gtin": "5011376002753",
                "image": "https://static.euw1.sustained.app/products/images/5011376002753_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00b8855750fb9819de0fb987818151e9",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00b8855750fb9819de0fb987818151e9/impacts"
                }
            },
            {
                "id": "productid.01253d241ded98d81508a9065dfc4fff",
                "name": "2 Carrot & Walnut Muffins",
                "gtin": "5000169418864",
                "image": "https://static.euw1.sustained.app/products/images/5000169418864_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01253d241ded98d81508a9065dfc4fff",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01253d241ded98d81508a9065dfc4fff/impacts"
                }
            },
            {
                "id": "productid.02957a2122a4abfe6e75890ab45ce143",
                "name": "2 Croissant",
                "gtin": "0000001691849",
                "image": "https://static.euw1.sustained.app/products/images/0000001691849_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 23.007177503461328
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02957a2122a4abfe6e75890ab45ce143",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02957a2122a4abfe6e75890ab45ce143/impacts"
                }
            },
            {
                "id": "productid.0649bee7241e92912f9c4d23448e314b",
                "name": "Baby Beetroot",
                "gtin": "5060035409235",
                "image": "https://static.euw1.sustained.app/products/images/5060035409235_0.jpg",
                "pack": "150g",
                "grade": "A",
                "points": {
                    "pef": 5.321220567533148
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0649bee7241e92912f9c4d23448e314b",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0649bee7241e92912f9c4d23448e314b/impacts"
                }
            },
            {
                "id": "productid.06581ae394250cec2c9e0733db0d6fc4",
                "name": "The Baker's On Market Street Large Croissant",
                "gtin": "5010251735588",
                "image": "https://static.euw1.sustained.app/products/images/5010251735588_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 23.007177503461328
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06581ae394250cec2c9e0733db0d6fc4",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06581ae394250cec2c9e0733db0d6fc4/impacts"
                }
            },
            {
                "id": "productid.0b8b40990fffb7de970b0cd2263b4163",
                "name": "Belgian Style Sugar Waffle",
                "gtin": "5053526008253",
                "image": "https://static.euw1.sustained.app/products/images/5053526008253_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b8b40990fffb7de970b0cd2263b4163",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b8b40990fffb7de970b0cd2263b4163/impacts"
                }
            },
            {
                "id": "productid.0d0c328631c33716465c480210c186d4",
                "name": "Taste the Difference 4 All Butter Croissants",
                "gtin": "00318976",
                "image": "https://static.euw1.sustained.app/products/images/00318976_0.jpg",
                "pack": "256g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d0c328631c33716465c480210c186d4",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d0c328631c33716465c480210c186d4/impacts"
                }
            },
            {
                "id": "productid.0e4feee7470667aaacd4c20a4a14f105",
                "name": "Chocolate Brownies",
                "gtin": "8435201703559",
                "image": "https://static.euw1.sustained.app/products/images/8435201703559_0.jpg",
                "pack": "25g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e4feee7470667aaacd4c20a4a14f105",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e4feee7470667aaacd4c20a4a14f105/impacts"
                }
            },
            {
                "id": "productid.107b93a9ee446e6ac2f1a07ed9cedcc8",
                "name": "Oreo Muffin",
                "gtin": "5025183073745",
                "image": "https://static.euw1.sustained.app/products/images/5025183073745_0.jpg",
                "pack": "80g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.107b93a9ee446e6ac2f1a07ed9cedcc8",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.107b93a9ee446e6ac2f1a07ed9cedcc8/impacts"
                }
            },
            {
                "id": "productid.116b5b21d3b439dbf1cfdbd87d7d9db4",
                "name": "Golden Flapjack 30mg CBD",
                "gtin": "5060687310217",
                "image": "https://static.euw1.sustained.app/products/images/5060687310217_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.116b5b21d3b439dbf1cfdbd87d7d9db4",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.116b5b21d3b439dbf1cfdbd87d7d9db4/impacts"
                }
            },
            {
                "id": "productid.1f1ee8ae264cde33011072c1975e6792",
                "name": "4 Reduced Fat Butter Croissants",
                "gtin": "5000169075487",
                "image": "https://static.euw1.sustained.app/products/images/5000169075487_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1f1ee8ae264cde33011072c1975e6792",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1f1ee8ae264cde33011072c1975e6792/impacts"
                }
            },
            {
                "id": "productid.1f9e941aad35a48371a662cf00ef6961",
                "name": "4 Sultana Scones",
                "gtin": "5018156005485",
                "image": "https://static.euw1.sustained.app/products/images/5018156005485_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1f9e941aad35a48371a662cf00ef6961",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1f9e941aad35a48371a662cf00ef6961/impacts"
                }
            },
            {
                "id": "productid.22a8a51b12e82c961e45942a5b3bf409",
                "name": "4 Blueberry Muffins",
                "gtin": "5000169619162",
                "image": "https://static.euw1.sustained.app/products/images/5000169619162_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.22a8a51b12e82c961e45942a5b3bf409",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.22a8a51b12e82c961e45942a5b3bf409/impacts"
                }
            },
            {
                "id": "productid.239889166410da5d4a0234408586273e",
                "name": "8 Pancakes",
                "gtin": "8712941100924",
                "image": "https://static.euw1.sustained.app/products/images/8712941100924_0.jpg",
                "pack": "600g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.239889166410da5d4a0234408586273e",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.239889166410da5d4a0234408586273e/impacts"
                }
            },
            {
                "id": "productid.255614a311cc1aa404e25e302e0dd78b",
                "name": "The Cake Shop On Market Street 2 Fresh Cream Lemon Muffins",
                "gtin": "0204549002000",
                "image": "https://static.euw1.sustained.app/products/images/0204549002000_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.255614a311cc1aa404e25e302e0dd78b",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.255614a311cc1aa404e25e302e0dd78b/impacts"
                }
            },
            {
                "id": "productid.2a7b44bb08d8f9c4b4728cacae640aa5",
                "name": "Baker's Selection 4 Blueberry Muffins",
                "gtin": "27656884",
                "image": "https://static.euw1.sustained.app/products/images/27656884_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a7b44bb08d8f9c4b4728cacae640aa5",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a7b44bb08d8f9c4b4728cacae640aa5/impacts"
                }
            },
            {
                "id": "productid.2aecd7ca10e393794838bd4b494a0fc9",
                "name": "Free From Gluten 2 Dark Chocolate Chip Muffins",
                "gtin": "05010525187136",
                "image": "https://static.euw1.sustained.app/products/images/05010525187136_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2aecd7ca10e393794838bd4b494a0fc9",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2aecd7ca10e393794838bd4b494a0fc9/impacts"
                }
            },
            {
                "id": "productid.2c19e87c57b9e8b895373fb5ee0513ad",
                "name": "Taste the Difference 4 Sultana Scones",
                "gtin": "01077094",
                "image": "https://static.euw1.sustained.app/products/images/01077094_0.jpg",
                "pack": "280g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2c19e87c57b9e8b895373fb5ee0513ad",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2c19e87c57b9e8b895373fb5ee0513ad/impacts"
                }
            },
            {
                "id": "productid.2cea51ffaf010da36c6ba22b11850c98",
                "name": "Breakfast 4 Muffins",
                "gtin": "5010092152940",
                "image": "https://static.euw1.sustained.app/products/images/5010092152940_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2cea51ffaf010da36c6ba22b11850c98",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2cea51ffaf010da36c6ba22b11850c98/impacts"
                }
            },
            {
                "id": "productid.2e2f3e263cf1e060a6aabd3d8a54a128",
                "name": "All Butter Croissant",
                "gtin": "0000000002639",
                "image": "https://static.euw1.sustained.app/products/images/0000000002639_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 23.007177503461328
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2e2f3e263cf1e060a6aabd3d8a54a128",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2e2f3e263cf1e060a6aabd3d8a54a128/impacts"
                }
            },
            {
                "id": "productid.2e8ba7c65a2f28c41ac9c4ee71489f9b",
                "name": "4 Oven Bottom Muffins",
                "gtin": "5010044001135",
                "image": "https://static.euw1.sustained.app/products/images/5010044001135_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2e8ba7c65a2f28c41ac9c4ee71489f9b",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2e8ba7c65a2f28c41ac9c4ee71489f9b/impacts"
                }
            },
            {
                "id": "productid.2f0aee3cee1362580dbd92b114992340",
                "name": "Gluten Free 4 Gingerbread Muffins",
                "gtin": "7330242290116",
                "image": "https://static.euw1.sustained.app/products/images/7330242290116_0.jpg",
                "pack": "240g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2f0aee3cee1362580dbd92b114992340",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2f0aee3cee1362580dbd92b114992340/impacts"
                }
            },
            {
                "id": "productid.324b59ce2a7bd3738d6c1b1f2c50a521",
                "name": "6 Croissants",
                "gtin": "5057753254254",
                "image": "https://static.euw1.sustained.app/products/images/5057753254254_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 23.007177503461328
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.324b59ce2a7bd3738d6c1b1f2c50a521",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.324b59ce2a7bd3738d6c1b1f2c50a521/impacts"
                }
            },
            {
                "id": "productid.34a3f20fda31250180460c297bd9906b",
                "name": "6 Lemon & Raisin Pancakes",
                "gtin": "5054781728405",
                "image": "https://static.euw1.sustained.app/products/images/5054781728405_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.34a3f20fda31250180460c297bd9906b",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.34a3f20fda31250180460c297bd9906b/impacts"
                }
            },
            {
                "id": "productid.375de9610c5447e6af1b6722ea2585b5",
                "name": "12 Plain Fairy Cakes",
                "gtin": "5027738006024",
                "image": "https://static.euw1.sustained.app/products/images/5027738006024_0.jpg",
                "pack": "280g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.375de9610c5447e6af1b6722ea2585b5",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.375de9610c5447e6af1b6722ea2585b5/impacts"
                }
            },
            {
                "id": "productid.39def6835d9da1eb2adff6ef57ab196e",
                "name": "Almond Croissant with Butter",
                "gtin": "0284640000000",
                "image": "https://static.euw1.sustained.app/products/images/0284640000000_0.jpg",
                "pack": "85g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.39def6835d9da1eb2adff6ef57ab196e",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.39def6835d9da1eb2adff6ef57ab196e/impacts"
                }
            },
            {
                "id": "productid.3bcb0851602d0de45078463e692c2fa0",
                "name": "4 Blueberry Pancakes",
                "gtin": "5000169576366",
                "image": "https://static.euw1.sustained.app/products/images/5000169576366_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3bcb0851602d0de45078463e692c2fa0",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3bcb0851602d0de45078463e692c2fa0/impacts"
                }
            },
            {
                "id": "productid.3c590080605e471d9653e7b700263363",
                "name": "4 Original Potato Farls",
                "gtin": "5010896092206",
                "image": "https://static.euw1.sustained.app/products/images/5010896092206_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3c590080605e471d9653e7b700263363",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3c590080605e471d9653e7b700263363/impacts"
                }
            },
            {
                "id": "productid.3ed8efcea8c3e374754a74c1c0b48b77",
                "name": "6 Scottish Tattie Scones",
                "gtin": "5060086842159",
                "image": "https://static.euw1.sustained.app/products/images/5060086842159_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3ed8efcea8c3e374754a74c1c0b48b77",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3ed8efcea8c3e374754a74c1c0b48b77/impacts"
                }
            },
            {
                "id": "productid.41175c86405b68433e659625e361ab23",
                "name": "4 Muffins Wholemeal",
                "gtin": "5010251605508",
                "image": "https://static.euw1.sustained.app/products/images/5010251605508_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.41175c86405b68433e659625e361ab23",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.41175c86405b68433e659625e361ab23/impacts"
                }
            },
            {
                "id": "productid.423e7902fae6382bc12245d747fbc16e",
                "name": "Food 4 Sultana Scones",
                "gtin": "0000000103350",
                "image": "https://static.euw1.sustained.app/products/images/0000000103350_0.jpg",
                "pack": "280g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.423e7902fae6382bc12245d747fbc16e",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.423e7902fae6382bc12245d747fbc16e/impacts"
                }
            },
            {
                "id": "productid.4417e63f75cefa9955c3cd9e877c97da",
                "name": "Together 4 Buttermilk Muffins",
                "gtin": "5013427015606",
                "image": "https://static.euw1.sustained.app/products/images/5013427015606_0.jpg",
                "pack": "280g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4417e63f75cefa9955c3cd9e877c97da",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4417e63f75cefa9955c3cd9e877c97da/impacts"
                }
            },
            {
                "id": "productid.483367e70ffc717a75776f35b867fce3",
                "name": "4 White Butter Muffins",
                "gtin": "5000169202777",
                "image": "https://static.euw1.sustained.app/products/images/5000169202777_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.483367e70ffc717a75776f35b867fce3",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.483367e70ffc717a75776f35b867fce3/impacts"
                }
            },
            {
                "id": "productid.4a47f6f96d9bf9f1c7a23526d633e8d9",
                "name": "2 Tickled Pink Muffins",
                "gtin": "5057172284764",
                "image": "https://static.euw1.sustained.app/products/images/5057172284764_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4a47f6f96d9bf9f1c7a23526d633e8d9",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4a47f6f96d9bf9f1c7a23526d633e8d9/impacts"
                }
            },
            {
                "id": "productid.4a9e859774364b9c079b9d4f54045164",
                "name": "Multiseed Croissant",
                "gtin": "0000001144758",
                "image": "https://static.euw1.sustained.app/products/images/0000001144758_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 23.007177503461328
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4a9e859774364b9c079b9d4f54045164",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4a9e859774364b9c079b9d4f54045164/impacts"
                }
            },
            {
                "id": "productid.4acd6700fd24e09c65614dbc1a53c519",
                "name": "Scotland's Baker Iced Gingerbread",
                "gtin": "5011262881097",
                "image": "https://static.euw1.sustained.app/products/images/5011262881097_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4acd6700fd24e09c65614dbc1a53c519",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4acd6700fd24e09c65614dbc1a53c519/impacts"
                }
            },
            {
                "id": "productid.4c5f01931d25c101a1f6ccfb4ad17e4d",
                "name": "4 All Butter Scones",
                "gtin": "5025251012270",
                "image": "https://static.euw1.sustained.app/products/images/5025251012270_0.jpg",
                "pack": "280g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4c5f01931d25c101a1f6ccfb4ad17e4d",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4c5f01931d25c101a1f6ccfb4ad17e4d/impacts"
                }
            },
            {
                "id": "productid.4c71c4daa9234935351c88d9c75a7206",
                "name": "2 Iced & Spiced Fruited Buns",
                "gtin": "5000169608319",
                "image": "https://static.euw1.sustained.app/products/images/5000169608319_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4c71c4daa9234935351c88d9c75a7206",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4c71c4daa9234935351c88d9c75a7206/impacts"
                }
            },
            {
                "id": "productid.4d2bdb312e2c868c76f053a1e0471bdc",
                "name": "6 Pikelets",
                "gtin": "00204859",
                "image": "https://static.euw1.sustained.app/products/images/00204859_0.jpg",
                "pack": "160g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4d2bdb312e2c868c76f053a1e0471bdc",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4d2bdb312e2c868c76f053a1e0471bdc/impacts"
                }
            },
            {
                "id": "productid.4e97e3f04adcd27f9a7c136219880d40",
                "name": "Bakery 4 Blueberry Muffins",
                "gtin": "5000129321784",
                "image": "https://static.euw1.sustained.app/products/images/5000129321784_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4e97e3f04adcd27f9a7c136219880d40",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4e97e3f04adcd27f9a7c136219880d40/impacts"
                }
            },
            {
                "id": "productid.4efac406e48e982f2e7d66fc0e89b3f2",
                "name": "Finest 2 All Butter Fruit Scones",
                "gtin": "0282980000000",
                "image": "https://static.euw1.sustained.app/products/images/0282980000000_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4efac406e48e982f2e7d66fc0e89b3f2",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4efac406e48e982f2e7d66fc0e89b3f2/impacts"
                }
            },
            {
                "id": "productid.519b845c805f03874ea4d3a7f5091816",
                "name": "Extra Special All-Butter Cheese Scones 2 Pack",
                "gtin": "0219960000009",
                "image": "https://static.euw1.sustained.app/products/images/0219960000009_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.519b845c805f03874ea4d3a7f5091816",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.519b845c805f03874ea4d3a7f5091816/impacts"
                }
            },
            {
                "id": "productid.583a30496632b3da56e1dec431326735",
                "name": "Chocolate Coated Salted Dairy Fudge",
                "gtin": "5060180035716",
                "image": "https://static.euw1.sustained.app/products/images/5060180035716_0.jpg",
                "pack": "170g",
                "grade": "F",
                "points": {
                    "pef": 58.91128134461614
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.583a30496632b3da56e1dec431326735",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.583a30496632b3da56e1dec431326735/impacts"
                }
            },
            {
                "id": "productid.5855ef90ef4ed5618ab7639d8f92ac66",
                "name": "4 All Butter Croissants",
                "gtin": "5027952014928",
                "image": "https://static.euw1.sustained.app/products/images/5027952014928_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5855ef90ef4ed5618ab7639d8f92ac66",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5855ef90ef4ed5618ab7639d8f92ac66/impacts"
                }
            },
            {
                "id": "productid.58fa4e91d45358fc2bd8711aa22dd178",
                "name": "The Best 2 Lemon Curd Muffins",
                "gtin": "5010525066431",
                "image": "https://static.euw1.sustained.app/products/images/5010525066431_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.58fa4e91d45358fc2bd8711aa22dd178",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.58fa4e91d45358fc2bd8711aa22dd178/impacts"
                }
            },
            {
                "id": "productid.5bc094e8a36aab6c9df81e95a3f28a93",
                "name": "Double Chocolate Muffin Frozen Individually Wrapped 48 Units Per Case",
                "gtin": "5391524331004",
                "image": "https://static.euw1.sustained.app/products/images/5391524331004_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5bc094e8a36aab6c9df81e95a3f28a93",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5bc094e8a36aab6c9df81e95a3f28a93/impacts"
                }
            },
            {
                "id": "productid.64348b0e4f25b95617e0def6d39d06a7",
                "name": "Caramel Muffins",
                "gtin": "5025183063128",
                "image": "https://static.euw1.sustained.app/products/images/5025183063128_0.jpg",
                "pack": "110g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.64348b0e4f25b95617e0def6d39d06a7",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.64348b0e4f25b95617e0def6d39d06a7/impacts"
                }
            },
            {
                "id": "productid.654afe6ba1d3690a0ef8b1bc70a7a270",
                "name": "Free From Croissants",
                "gtin": "0000000378840",
                "image": "https://static.euw1.sustained.app/products/images/0000000378840_0.jpg",
                "pack": "130g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.654afe6ba1d3690a0ef8b1bc70a7a270",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.654afe6ba1d3690a0ef8b1bc70a7a270/impacts"
                }
            },
            {
                "id": "productid.670745c26fe0a6c074059e185c44b102",
                "name": "4 Girdle Scones",
                "gtin": "5028184300506",
                "image": "https://static.euw1.sustained.app/products/images/5028184300506_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.670745c26fe0a6c074059e185c44b102",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.670745c26fe0a6c074059e185c44b102/impacts"
                }
            },
            {
                "id": "productid.67294b2da37f7661a71f461f8b01fad2",
                "name": "Food 4 All Butter Pain Au Chocolat",
                "gtin": "0000000502610",
                "image": "https://static.euw1.sustained.app/products/images/0000000502610_0.jpg",
                "pack": "204g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Breakfast Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.67294b2da37f7661a71f461f8b01fad2",
                    "category": "https://api.sustained.com/choice/v1/categories/breakfast-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.67294b2da37f7661a71f461f8b01fad2/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.03092fecfb470b494448b298d8ff676b",
                "name": "The Best Brittany Butter with Sea Salt",
                "gtin": "5010251498681",
                "image": "https://static.euw1.sustained.app/products/images/5010251498681_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03092fecfb470b494448b298d8ff676b",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03092fecfb470b494448b298d8ff676b/impacts"
                }
            },
            {
                "id": "productid.05f0475eb3e08e2d74e527ec83e38d29",
                "name": "12 Large Free Range Eggs",
                "gtin": "5024263000046",
                "image": "https://static.euw1.sustained.app/products/images/5024263000046_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05f0475eb3e08e2d74e527ec83e38d29",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05f0475eb3e08e2d74e527ec83e38d29/impacts"
                }
            },
            {
                "id": "productid.070a0092187d88e575dcf5c789472986",
                "name": "Unsalted Block Butter",
                "gtin": "5707361100275",
                "image": "https://static.euw1.sustained.app/products/images/5707361100275_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 114.69756486953928
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.070a0092187d88e575dcf5c789472986",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.070a0092187d88e575dcf5c789472986/impacts"
                }
            },
            {
                "id": "productid.0a39b2c38d8d8af3e7b1166b4dc6865c",
                "name": "Soft Spread",
                "gtin": "01802757",
                "image": "https://static.euw1.sustained.app/products/images/01802757_0.jpg",
                "pack": "500g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a39b2c38d8d8af3e7b1166b4dc6865c",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a39b2c38d8d8af3e7b1166b4dc6865c/impacts"
                }
            },
            {
                "id": "productid.0cb9a3943509eb4d8aa70c750821c56f",
                "name": "British Unsalted Butter",
                "gtin": "5051413679975",
                "image": "https://static.euw1.sustained.app/products/images/5051413679975_0.jpg",
                "pack": "250g",
                "grade": "E",
                "points": {
                    "pef": 33.557541352967945
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0cb9a3943509eb4d8aa70c750821c56f",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0cb9a3943509eb4d8aa70c750821c56f/impacts"
                }
            },
            {
                "id": "productid.1165899b6811692b22c6a2d62594dc79",
                "name": "French Unsalted Butter",
                "gtin": "3228020481167",
                "image": "https://static.euw1.sustained.app/products/images/3228020481167_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 114.72450110221726
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1165899b6811692b22c6a2d62594dc79",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1165899b6811692b22c6a2d62594dc79/impacts"
                }
            },
            {
                "id": "productid.14377b9c9ffda171e1262b1e4e2db0c9",
                "name": "Dromona Spreadeasy",
                "gtin": "5010386105461",
                "image": "https://static.euw1.sustained.app/products/images/5010386105461_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14377b9c9ffda171e1262b1e4e2db0c9",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14377b9c9ffda171e1262b1e4e2db0c9/impacts"
                }
            },
            {
                "id": "productid.1578d0f649c58c1c40d3356e36a55e6d",
                "name": "Classic Churned Cornish Butter",
                "gtin": "5019608002502",
                "image": "https://static.euw1.sustained.app/products/images/5019608002502_0.jpg",
                "pack": "250g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1578d0f649c58c1c40d3356e36a55e6d",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1578d0f649c58c1c40d3356e36a55e6d/impacts"
                }
            },
            {
                "id": "productid.16b2b84eb6189739eaad3b56a6c2a6b6",
                "name": "Ingredients Chocolate Chip & Maple Syrup Butter",
                "gtin": "5054269765649",
                "image": "https://static.euw1.sustained.app/products/images/5054269765649_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 114.72450110221726
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16b2b84eb6189739eaad3b56a6c2a6b6",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16b2b84eb6189739eaad3b56a6c2a6b6/impacts"
                }
            },
            {
                "id": "productid.16fb68d851a3a38a249901bb63768bc1",
                "name": "Sainsbury's Lard",
                "gtin": "01596755",
                "image": "https://static.euw1.sustained.app/products/images/01596755_0.jpg",
                "pack": "250g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16fb68d851a3a38a249901bb63768bc1",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16fb68d851a3a38a249901bb63768bc1/impacts"
                }
            },
            {
                "id": "productid.1a5f3b795214455eab01ff3ee55b17c0",
                "name": "Pure & Natural Irish Butter",
                "gtin": "5011056900072",
                "image": "https://static.euw1.sustained.app/products/images/5011056900072_0.jpg",
                "pack": "6.34g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1a5f3b795214455eab01ff3ee55b17c0",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1a5f3b795214455eab01ff3ee55b17c0/impacts"
                }
            },
            {
                "id": "productid.1aa9db1da5e1ef9de67f0b118e2ce8ca",
                "name": "Dairy Free Slightly Salted",
                "gtin": "5057624513497",
                "image": "https://static.euw1.sustained.app/products/images/5057624513497_0.jpg",
                "pack": "250g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1aa9db1da5e1ef9de67f0b118e2ce8ca",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1aa9db1da5e1ef9de67f0b118e2ce8ca/impacts"
                }
            },
            {
                "id": "productid.26589c58e50dd0dc06ebd92e04a9081b",
                "name": "Pure Irish Butter",
                "gtin": "5011038138882",
                "image": "https://static.euw1.sustained.app/products/images/5011038138882_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 33.392562023778076
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.26589c58e50dd0dc06ebd92e04a9081b",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.26589c58e50dd0dc06ebd92e04a9081b/impacts"
                }
            },
            {
                "id": "productid.28af7db1c4caa8110d2c5719db2cb60b",
                "name": "Salted Butter",
                "gtin": "5011177000071",
                "image": "https://static.euw1.sustained.app/products/images/5011177000071_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.28af7db1c4caa8110d2c5719db2cb60b",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.28af7db1c4caa8110d2c5719db2cb60b/impacts"
                }
            },
            {
                "id": "productid.2aa3711e2be8e268a85b05d84257550b",
                "name": "Farmhouse Butter with Maldon Sea Salt Flakes",
                "gtin": "5026573000242",
                "image": "https://static.euw1.sustained.app/products/images/5026573000242_0.jpg",
                "pack": "125g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2aa3711e2be8e268a85b05d84257550b",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2aa3711e2be8e268a85b05d84257550b/impacts"
                }
            },
            {
                "id": "productid.2ae9442452639aa87f945a48ebd90658",
                "name": "British Unsalted Butter",
                "gtin": "5000246722112",
                "image": "https://static.euw1.sustained.app/products/images/5000246722112_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 114.67062863686127
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2ae9442452639aa87f945a48ebd90658",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2ae9442452639aa87f945a48ebd90658/impacts"
                }
            },
            {
                "id": "productid.30b0633f60caa488254f997434899f0f",
                "name": "Sainsbury's Butterlicious",
                "gtin": "01034752",
                "image": "https://static.euw1.sustained.app/products/images/01034752_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.30b0633f60caa488254f997434899f0f",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.30b0633f60caa488254f997434899f0f/impacts"
                }
            },
            {
                "id": "productid.353bba878d68b176862c4d9889e34cd2",
                "name": "Original Butter",
                "gtin": "5011069001551",
                "image": "https://static.euw1.sustained.app/products/images/5011069001551_0.jpg",
                "pack": "1kg",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.353bba878d68b176862c4d9889e34cd2",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.353bba878d68b176862c4d9889e34cd2/impacts"
                }
            },
            {
                "id": "productid.3a4de6fac24bf45e3c1949eb6ab1c0db",
                "name": "The Family Dairy Organic Slightly Salted Scottish Butter",
                "gtin": "5025840000923",
                "image": "https://static.euw1.sustained.app/products/images/5025840000923_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3a4de6fac24bf45e3c1949eb6ab1c0db",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3a4de6fac24bf45e3c1949eb6ab1c0db/impacts"
                }
            },
            {
                "id": "productid.3b6b29a0d5df681cc3b442ce364fa8a8",
                "name": "Salted Traditional Irish Butter",
                "gtin": "5391519491287",
                "image": "https://static.euw1.sustained.app/products/images/5391519491287_0.jpg",
                "pack": "454g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3b6b29a0d5df681cc3b442ce364fa8a8",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3b6b29a0d5df681cc3b442ce364fa8a8/impacts"
                }
            },
            {
                "id": "productid.3d89271bea0067fc33ecb0457cc7d670",
                "name": "Savers Lard",
                "gtin": "5010251425977",
                "image": "https://static.euw1.sustained.app/products/images/5010251425977_0.jpg",
                "pack": "250g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3d89271bea0067fc33ecb0457cc7d670",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3d89271bea0067fc33ecb0457cc7d670/impacts"
                }
            },
            {
                "id": "productid.3e5ddd48438209a9e23597010e07708f",
                "name": "Slightly Salted Spreadable",
                "gtin": "5711953085819",
                "image": "https://static.euw1.sustained.app/products/images/5711953085819_0.jpg",
                "pack": "2kg",
                "grade": "F",
                "points": {
                    "pef": 80.81762367903298
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3e5ddd48438209a9e23597010e07708f",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3e5ddd48438209a9e23597010e07708f/impacts"
                }
            },
            {
                "id": "productid.423fbfcc27d7fe74b9a1169686b32118",
                "name": "Ballyrashane Butter",
                "gtin": "5012006025081",
                "image": "https://static.euw1.sustained.app/products/images/5012006025081_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.423fbfcc27d7fe74b9a1169686b32118",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.423fbfcc27d7fe74b9a1169686b32118/impacts"
                }
            },
            {
                "id": "productid.43914b9e6a1ee87d76de56945a38418d",
                "name": "Organic Vegan Block Shea, Coconut, Rapeseed and Almond",
                "gtin": "5701977000097",
                "image": "https://static.euw1.sustained.app/products/images/5701977000097_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 73.87902210709234
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.43914b9e6a1ee87d76de56945a38418d",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.43914b9e6a1ee87d76de56945a38418d/impacts"
                }
            },
            {
                "id": "productid.4738879747f47446b76a05f01e2f67e2",
                "name": "French Unsalted Spreadable",
                "gtin": "3155250361818",
                "image": "https://static.euw1.sustained.app/products/images/3155250361818_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4738879747f47446b76a05f01e2f67e2",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4738879747f47446b76a05f01e2f67e2/impacts"
                }
            },
            {
                "id": "productid.4ca04180c46ac9cc70a92c1374b96cfe",
                "name": "Salted Farmhouse Butter",
                "gtin": "0000000646420",
                "image": "https://static.euw1.sustained.app/products/images/0000000646420_0.jpg",
                "pack": "125g",
                "grade": "F",
                "points": {
                    "pef": 59.8963762051838
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4ca04180c46ac9cc70a92c1374b96cfe",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4ca04180c46ac9cc70a92c1374b96cfe/impacts"
                }
            },
            {
                "id": "productid.56d8d08e588775d90f0c14f61eb2a69f",
                "name": "Food Collection Brandy Butter",
                "gtin": "0000000064101",
                "image": "https://static.euw1.sustained.app/products/images/0000000064101_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.56d8d08e588775d90f0c14f61eb2a69f",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.56d8d08e588775d90f0c14f61eb2a69f/impacts"
                }
            },
            {
                "id": "productid.5bf6fee6f39c53d5d56859fead51ca57",
                "name": "Organic British Slightly Salted Butter",
                "gtin": "5054781498681",
                "image": "https://static.euw1.sustained.app/products/images/5054781498681_0.jpg",
                "pack": "250g",
                "grade": "E",
                "points": {
                    "pef": 32.37630761611094
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5bf6fee6f39c53d5d56859fead51ca57",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5bf6fee6f39c53d5d56859fead51ca57/impacts"
                }
            },
            {
                "id": "productid.5d0ea3bad87aa5339d9a89b2013514b2",
                "name": "Irish Butter",
                "gtin": "5011056000048",
                "image": "https://static.euw1.sustained.app/products/images/5011056000048_0.jpg",
                "pack": "454g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5d0ea3bad87aa5339d9a89b2013514b2",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5d0ea3bad87aa5339d9a89b2013514b2/impacts"
                }
            },
            {
                "id": "productid.6288f50fc018886c8fdd888c0ffbcf0a",
                "name": "Handmade Country Butter",
                "gtin": "5319508960897",
                "image": "https://static.euw1.sustained.app/products/images/5319508960897_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 112.57680339884084
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.6288f50fc018886c8fdd888c0ffbcf0a",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.6288f50fc018886c8fdd888c0ffbcf0a/impacts"
                }
            },
            {
                "id": "productid.67e2e7ed21a6006212af1c7ee918ae3a",
                "name": "Extra Special West Country Butter",
                "gtin": "5054781649823",
                "image": "https://static.euw1.sustained.app/products/images/5054781649823_0.jpg",
                "pack": "250g",
                "grade": "F",
                "points": {
                    "pef": 68.77991529163765
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.67e2e7ed21a6006212af1c7ee918ae3a",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.67e2e7ed21a6006212af1c7ee918ae3a/impacts"
                }
            },
            {
                "id": "productid.6811b0d0e1bc51953246a9fd296d12e2",
                "name": "Golden Olive Spread",
                "gtin": "5011055404816",
                "image": "https://static.euw1.sustained.app/products/images/5011055404816_0.jpg",
                "pack": "250g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.6811b0d0e1bc51953246a9fd296d12e2",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.6811b0d0e1bc51953246a9fd296d12e2/impacts"
                }
            },
            {
                "id": "productid.70596ee53e1149d46141c8c6265a660d",
                "name": "Farmhouse Butter Salted",
                "gtin": "5021427100017",
                "image": "https://static.euw1.sustained.app/products/images/5021427100017_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 112.51133998837598
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.70596ee53e1149d46141c8c6265a660d",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.70596ee53e1149d46141c8c6265a660d/impacts"
                }
            },
            {
                "id": "productid.740c5428ce7b11a54bf35376867bdc1b",
                "name": "Pure Irish Butter Salted",
                "gtin": "5099869041694",
                "image": "https://static.euw1.sustained.app/products/images/5099869041694_0.jpg",
                "pack": "250g",
                "grade": "E",
                "points": {
                    "pef": 33.38662608108373
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.740c5428ce7b11a54bf35376867bdc1b",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.740c5428ce7b11a54bf35376867bdc1b/impacts"
                }
            },
            {
                "id": "productid.756511d7183aa1a8a0dcdb593658f691",
                "name": "Garnish and Go Herb Butter",
                "gtin": "05000169596845",
                "image": "https://static.euw1.sustained.app/products/images/05000169596845_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 114.72450110221726
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.756511d7183aa1a8a0dcdb593658f691",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.756511d7183aa1a8a0dcdb593658f691/impacts"
                }
            },
            {
                "id": "productid.75662d0d51e8d3b92cc09b7f299fecb3",
                "name": "British Butter Unsalted",
                "gtin": "5057545899144",
                "image": "https://static.euw1.sustained.app/products/images/5057545899144_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 114.69756486953928
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.75662d0d51e8d3b92cc09b7f299fecb3",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.75662d0d51e8d3b92cc09b7f299fecb3/impacts"
                }
            },
            {
                "id": "productid.766302537e132e4b9559bce3b2a6fec2",
                "name": "ProActiv Buttery Taste Spread",
                "gtin": "8719200079014",
                "image": "https://static.euw1.sustained.app/products/images/8719200079014_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 45.54681208609177
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.766302537e132e4b9559bce3b2a6fec2",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.766302537e132e4b9559bce3b2a6fec2/impacts"
                }
            },
            {
                "id": "productid.7afa80173e0ad20b6e6972e2e8f03ebb",
                "name": "Slightly Salted Butter Box",
                "gtin": "5740900403437",
                "image": "https://static.euw1.sustained.app/products/images/5740900403437_0.jpg",
                "pack": "225g",
                "grade": "G",
                "points": {
                    "pef": 107.81841610895435
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.7afa80173e0ad20b6e6972e2e8f03ebb",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.7afa80173e0ad20b6e6972e2e8f03ebb/impacts"
                }
            },
            {
                "id": "productid.8036c02eb208eb3e9b53d0c2d8b3428c",
                "name": "Organic Spreadable Butter",
                "gtin": "5060034200499",
                "image": "https://static.euw1.sustained.app/products/images/5060034200499_0.jpg",
                "pack": "250g",
                "grade": "F",
                "points": {
                    "pef": 80.41322285583513
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.8036c02eb208eb3e9b53d0c2d8b3428c",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.8036c02eb208eb3e9b53d0c2d8b3428c/impacts"
                }
            },
            {
                "id": "productid.83c5d65e1bb88f6a157b45334619c55b",
                "name": "Salted Pure Creamery Butter",
                "gtin": "5021254000054",
                "image": "https://static.euw1.sustained.app/products/images/5021254000054_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.83c5d65e1bb88f6a157b45334619c55b",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.83c5d65e1bb88f6a157b45334619c55b/impacts"
                }
            },
            {
                "id": "productid.83f4695cd96e2c89b39b35aea8bc929f",
                "name": "Sunflower Spread",
                "gtin": "00057516",
                "image": "https://static.euw1.sustained.app/products/images/00057516_0.jpg",
                "pack": "500g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.83f4695cd96e2c89b39b35aea8bc929f",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.83f4695cd96e2c89b39b35aea8bc929f/impacts"
                }
            },
            {
                "id": "productid.85916aa86b58fbcd964838abd8f8b566",
                "name": "Pro British Butter Portions",
                "gtin": "5000246554225",
                "image": "https://static.euw1.sustained.app/products/images/5000246554225_0.jpg",
                "pack": "7g",
                "grade": "F",
                "points": {
                    "pef": 59.68160643484617
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.85916aa86b58fbcd964838abd8f8b566",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.85916aa86b58fbcd964838abd8f8b566/impacts"
                }
            },
            {
                "id": "productid.86ef71d56597e17cc723ea603a02fde0",
                "name": "Baking Block",
                "gtin": "01691344",
                "image": "https://static.euw1.sustained.app/products/images/01691344_0.jpg",
                "pack": "250g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.86ef71d56597e17cc723ea603a02fde0",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.86ef71d56597e17cc723ea603a02fde0/impacts"
                }
            },
            {
                "id": "productid.889fe16935548b8952c0c0984f6c14d1",
                "name": "LowLow Spread",
                "gtin": "5011069001650",
                "image": "https://static.euw1.sustained.app/products/images/5011069001650_0.jpg",
                "pack": "10g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.889fe16935548b8952c0c0984f6c14d1",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.889fe16935548b8952c0c0984f6c14d1/impacts"
                }
            },
            {
                "id": "productid.8980fd6756d6561809e0497b658ad570",
                "name": "Food Organic British Salted Butter",
                "gtin": "0000000580458",
                "image": "https://static.euw1.sustained.app/products/images/0000000580458_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.8980fd6756d6561809e0497b658ad570",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.8980fd6756d6561809e0497b658ad570/impacts"
                }
            },
            {
                "id": "productid.89e78a091aba0e47f832a1f6108622e9",
                "name": "Toffee Apple Almond Butter",
                "gtin": "5060132284872",
                "image": "https://static.euw1.sustained.app/products/images/5060132284872_0.jpg",
                "pack": "170g",
                "grade": "G",
                "points": {
                    "pef": 159.76261248890546
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.89e78a091aba0e47f832a1f6108622e9",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.89e78a091aba0e47f832a1f6108622e9/impacts"
                }
            },
            {
                "id": "productid.8cde3401fcd76de40edcee6d11ff05b8",
                "name": "Salted Butter",
                "gtin": "5000295151987",
                "image": "https://static.euw1.sustained.app/products/images/5000295151987_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.8cde3401fcd76de40edcee6d11ff05b8",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.8cde3401fcd76de40edcee6d11ff05b8/impacts"
                }
            },
            {
                "id": "productid.8f4193e975e9ca0da4ada83176fe88c1",
                "name": "Finest Quality Pure Butter Ghee",
                "gtin": "5013635101061",
                "image": "https://static.euw1.sustained.app/products/images/5013635101061_0.jpg",
                "pack": "1kg",
                "grade": "E",
                "points": {
                    "pef": 33.930784479140115
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.8f4193e975e9ca0da4ada83176fe88c1",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.8f4193e975e9ca0da4ada83176fe88c1/impacts"
                }
            },
            {
                "id": "productid.90c62472b74b3f66a8f816d99ef74276",
                "name": "Pure Irish Butter",
                "gtin": "5000165024014",
                "image": "https://static.euw1.sustained.app/products/images/5000165024014_0.jpg",
                "pack": "250g",
                "grade": "E",
                "points": {
                    "pef": 33.392562023778076
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.90c62472b74b3f66a8f816d99ef74276",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.90c62472b74b3f66a8f816d99ef74276/impacts"
                }
            },
            {
                "id": "productid.9333d64144d0ff29ba3abc55d9a87f8d",
                "name": "Salted Pure Irish Butter",
                "gtin": "05099869745899",
                "image": "https://static.euw1.sustained.app/products/images/05099869745899_0.jpg",
                "pack": "454g",
                "grade": "G",
                "points": {
                    "pef": 113.67531491614561
                },
                "info_icons": [],
                "category": "Butter & Margarine",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.9333d64144d0ff29ba3abc55d9a87f8d",
                    "category": "https://api.sustained.com/choice/v1/categories/butter-and-margarine",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.9333d64144d0ff29ba3abc55d9a87f8d/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.004528e85111468bed9c2c4763a4ca1f",
                "name": "Cake Shop on Market Street Chocolate Cake",
                "gtin": "5010251603306",
                "image": "https://static.euw1.sustained.app/products/images/5010251603306_0.jpg",
                "pack": "455g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.004528e85111468bed9c2c4763a4ca1f",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.004528e85111468bed9c2c4763a4ca1f/impacts"
                }
            },
            {
                "id": "productid.006f7e8f47af71ef1d58268a19a30e0e",
                "name": "Fresh Bramley Apple Tart",
                "gtin": "5392000055391",
                "image": "https://static.euw1.sustained.app/products/images/5392000055391_0.jpg",
                "pack": "450g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.006f7e8f47af71ef1d58268a19a30e0e",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.006f7e8f47af71ef1d58268a19a30e0e/impacts"
                }
            },
            {
                "id": "productid.008b9f14451cf1bdd03b892abc073e0d",
                "name": "6 Sweet Dutch Pancakes",
                "gtin": "5060139991179",
                "image": "https://static.euw1.sustained.app/products/images/5060139991179_0.jpg",
                "pack": "375g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.008b9f14451cf1bdd03b892abc073e0d",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.008b9f14451cf1bdd03b892abc073e0d/impacts"
                }
            },
            {
                "id": "productid.0098323c411ae58da85ea31a7b03d2a5",
                "name": "M&K Bakery Jam & Cream Roll",
                "gtin": "5099823001269",
                "image": "https://static.euw1.sustained.app/products/images/5099823001269_0.jpg",
                "pack": "400g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0098323c411ae58da85ea31a7b03d2a5",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0098323c411ae58da85ea31a7b03d2a5/impacts"
                }
            },
            {
                "id": "productid.009e1d927d23cdaff5f0f70ec9795037",
                "name": "Cherry Madeira Cake",
                "gtin": "5000169302224",
                "image": "https://static.euw1.sustained.app/products/images/5000169302224_0.jpg",
                "pack": "350g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.009e1d927d23cdaff5f0f70ec9795037",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.009e1d927d23cdaff5f0f70ec9795037/impacts"
                }
            },
            {
                "id": "productid.00a5e51ea238be8d079cfd14491b31a7",
                "name": "12 Golden Sponge Fairy Cakes Christmas",
                "gtin": "5060056142548",
                "image": "https://static.euw1.sustained.app/products/images/5060056142548_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00a5e51ea238be8d079cfd14491b31a7",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00a5e51ea238be8d079cfd14491b31a7/impacts"
                }
            },
            {
                "id": "productid.011dfef57c468043c4f2bd80eab90cc3",
                "name": "Yule Dog",
                "gtin": "0000000404891",
                "image": "https://static.euw1.sustained.app/products/images/0000000404891_0.jpg",
                "pack": "631g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.011dfef57c468043c4f2bd80eab90cc3",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.011dfef57c468043c4f2bd80eab90cc3/impacts"
                }
            },
            {
                "id": "productid.0129ab8f1708df6c27bb27685f595339",
                "name": "Simnel Cake",
                "gtin": "01711769",
                "image": "https://static.euw1.sustained.app/products/images/01711769_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0129ab8f1708df6c27bb27685f595339",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0129ab8f1708df6c27bb27685f595339/impacts"
                }
            },
            {
                "id": "productid.018b48af8fc05d3a9b90025dc780317c",
                "name": "Sliced Fruit Loaf",
                "gtin": "5057545931219",
                "image": "https://static.euw1.sustained.app/products/images/5057545931219_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 37.910948333355066
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.018b48af8fc05d3a9b90025dc780317c",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.018b48af8fc05d3a9b90025dc780317c/impacts"
                }
            },
            {
                "id": "productid.01b320b00d54e6416734209650a6118c",
                "name": "Tesco Polar Bear Winter Scene Cake",
                "gtin": "5057753928476",
                "image": "https://static.euw1.sustained.app/products/images/5057753928476_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 41.14596972877065
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01b320b00d54e6416734209650a6118c",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01b320b00d54e6416734209650a6118c/impacts"
                }
            },
            {
                "id": "productid.01ff9bc2b0c1ecd2c798296371fe2c8b",
                "name": "Kitticorn Cake",
                "gtin": "5057172312399",
                "image": "https://static.euw1.sustained.app/products/images/5057172312399_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01ff9bc2b0c1ecd2c798296371fe2c8b",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01ff9bc2b0c1ecd2c798296371fe2c8b/impacts"
                }
            },
            {
                "id": "productid.02619afd96b94415180f35bcb6020083",
                "name": "6 Mince Pies",
                "gtin": "5060168300119",
                "image": "https://static.euw1.sustained.app/products/images/5060168300119_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02619afd96b94415180f35bcb6020083",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02619afd96b94415180f35bcb6020083/impacts"
                }
            },
            {
                "id": "productid.0271876ce73e1c931d1e5e27a163970b",
                "name": "Baker's Selection 18 Mini White Choc Rocky Road Bites",
                "gtin": "5054781901686",
                "image": "https://static.euw1.sustained.app/products/images/5054781901686_0.jpg",
                "pack": "180g",
                "grade": "F",
                "points": {
                    "pef": 91.11745680806557
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0271876ce73e1c931d1e5e27a163970b",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0271876ce73e1c931d1e5e27a163970b/impacts"
                }
            },
            {
                "id": "productid.032e19a831541116011008c2071e2828",
                "name": "Food Nuts for Chocolate Biscakes",
                "gtin": "0000029198818",
                "image": "https://static.euw1.sustained.app/products/images/0000029198818_0.jpg",
                "pack": "179g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.032e19a831541116011008c2071e2828",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.032e19a831541116011008c2071e2828/impacts"
                }
            },
            {
                "id": "productid.034c0bbcc3e8ecbc740240cdb06a5d22",
                "name": "90 Calories Chocolate Fudge Brownie",
                "gtin": "8410076910820",
                "image": "https://static.euw1.sustained.app/products/images/8410076910820_0.jpg",
                "pack": "24g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.034c0bbcc3e8ecbc740240cdb06a5d22",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.034c0bbcc3e8ecbc740240cdb06a5d22/impacts"
                }
            },
            {
                "id": "productid.043e21111626f5b0c4b5c5d92b495283",
                "name": "Finest 8 Meringue Nests",
                "gtin": "5051140095703",
                "image": "https://static.euw1.sustained.app/products/images/5051140095703_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.043e21111626f5b0c4b5c5d92b495283",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.043e21111626f5b0c4b5c5d92b495283/impacts"
                }
            },
            {
                "id": "productid.0478beab8b8700e6a484fd39fca545fe",
                "name": "Chocolate Celebration Cake",
                "gtin": "5000169330579",
                "image": "https://static.euw1.sustained.app/products/images/5000169330579_0.jpg",
                "pack": "752g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0478beab8b8700e6a484fd39fca545fe",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0478beab8b8700e6a484fd39fca545fe/impacts"
                }
            },
            {
                "id": "productid.04c2080eb1b89151f45e146e4689ce3d",
                "name": "8 Pains au Chocolat",
                "gtin": "5057545929407",
                "image": "https://static.euw1.sustained.app/products/images/5057545929407_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 54.782460410490486
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04c2080eb1b89151f45e146e4689ce3d",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04c2080eb1b89151f45e146e4689ce3d/impacts"
                }
            },
            {
                "id": "productid.05e76a9951096372a90d3eed89636538",
                "name": "3 Tartlets Raspberry",
                "gtin": "3178530408690",
                "image": "https://static.euw1.sustained.app/products/images/3178530408690_0.jpg",
                "pack": "45g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05e76a9951096372a90d3eed89636538",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05e76a9951096372a90d3eed89636538/impacts"
                }
            },
            {
                "id": "productid.067d85fd0da587cf9f36e59a50da76d0",
                "name": "Chocolate Cupcakes with Marbled Icing 4 Pack",
                "gtin": "5099839696688",
                "image": "https://static.euw1.sustained.app/products/images/5099839696688_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 80.98583982193439
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.067d85fd0da587cf9f36e59a50da76d0",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.067d85fd0da587cf9f36e59a50da76d0/impacts"
                }
            },
            {
                "id": "productid.06c60589c20fc87445c214f5e650bb41",
                "name": "The Best Hand Finished New York Style Cheesecake",
                "gtin": "5010525145839",
                "image": "https://static.euw1.sustained.app/products/images/5010525145839_0.jpg",
                "pack": "550g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06c60589c20fc87445c214f5e650bb41",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06c60589c20fc87445c214f5e650bb41/impacts"
                }
            },
            {
                "id": "productid.06ce6496a90ed45482e7776646687125",
                "name": "Free From Syrup Sponge Pudding",
                "gtin": "5054781952312",
                "image": "https://static.euw1.sustained.app/products/images/5054781952312_0.jpg",
                "pack": "110g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06ce6496a90ed45482e7776646687125",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06ce6496a90ed45482e7776646687125/impacts"
                }
            },
            {
                "id": "productid.07afee498b0519bd2359e9ae94927777",
                "name": "Salted Caramel Cheesecake",
                "gtin": "0793591912486",
                "image": "https://static.euw1.sustained.app/products/images/0793591912486_0.jpg",
                "pack": "90g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07afee498b0519bd2359e9ae94927777",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07afee498b0519bd2359e9ae94927777/impacts"
                }
            },
            {
                "id": "productid.07f7c34975697456b0d933009aefd0fd",
                "name": "Classic Christmas Pudding",
                "gtin": "5010744000742",
                "image": "https://static.euw1.sustained.app/products/images/5010744000742_0.jpg",
                "pack": "800g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07f7c34975697456b0d933009aefd0fd",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07f7c34975697456b0d933009aefd0fd/impacts"
                }
            },
            {
                "id": "productid.0875edced6fa9a0f44bdfcfa945cac3b",
                "name": "Multigrain Oatcakes",
                "gtin": "0612322001537",
                "image": "https://static.euw1.sustained.app/products/images/0612322001537_0.jpg",
                "pack": "200g",
                "grade": "B",
                "points": {
                    "pef": 12.52712818371661
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0875edced6fa9a0f44bdfcfa945cac3b",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0875edced6fa9a0f44bdfcfa945cac3b/impacts"
                }
            },
            {
                "id": "productid.08a72bd83117bbd87676f9321dbbe4d5",
                "name": "Christmas Pudding",
                "gtin": "5010744000544",
                "image": "https://static.euw1.sustained.app/products/images/5010744000544_0.jpg",
                "pack": "350g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08a72bd83117bbd87676f9321dbbe4d5",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08a72bd83117bbd87676f9321dbbe4d5/impacts"
                }
            },
            {
                "id": "productid.08a78892e72d9dd73e0716b2a70a99dd",
                "name": "4 Chorley Cakes",
                "gtin": "5010251265887",
                "image": "https://static.euw1.sustained.app/products/images/5010251265887_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08a78892e72d9dd73e0716b2a70a99dd",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08a78892e72d9dd73e0716b2a70a99dd/impacts"
                }
            },
            {
                "id": "productid.08b73631276ae49cf85f477c628084ce",
                "name": "Oatcake Tin",
                "gtin": "5015168003775",
                "image": "https://static.euw1.sustained.app/products/images/5015168003775_0.jpg",
                "pack": "100g",
                "grade": "B",
                "points": {
                    "pef": 15.804504052587838
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08b73631276ae49cf85f477c628084ce",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08b73631276ae49cf85f477c628084ce/impacts"
                }
            },
            {
                "id": "productid.09035cafcddf6b0569134340dbffdb4d",
                "name": "6 Viennese Whirls",
                "gtin": "5000221604549",
                "image": "https://static.euw1.sustained.app/products/images/5000221604549_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09035cafcddf6b0569134340dbffdb4d",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09035cafcddf6b0569134340dbffdb4d/impacts"
                }
            },
            {
                "id": "productid.099908cec8dda89554bdd627147a2c21",
                "name": "Chocolate & Orange Loaf",
                "gtin": "5000153007159",
                "image": "https://static.euw1.sustained.app/products/images/5000153007159_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.099908cec8dda89554bdd627147a2c21",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.099908cec8dda89554bdd627147a2c21/impacts"
                }
            },
            {
                "id": "productid.09a653b6af04aeb3182c6b7a1345008e",
                "name": "Food Christmas Cake",
                "gtin": "0000000271844",
                "image": "https://static.euw1.sustained.app/products/images/0000000271844_0.jpg",
                "pack": "155g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09a653b6af04aeb3182c6b7a1345008e",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09a653b6af04aeb3182c6b7a1345008e/impacts"
                }
            },
            {
                "id": "productid.0a21dfd083c249f0c62dd10390545ebf",
                "name": "Our Sundae Specials",
                "gtin": "5025210000034",
                "image": "https://static.euw1.sustained.app/products/images/5025210000034_0.jpg",
                "pack": "257g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a21dfd083c249f0c62dd10390545ebf",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a21dfd083c249f0c62dd10390545ebf/impacts"
                }
            },
            {
                "id": "productid.0a4b3239280a2f4407b56c54135fd7c9",
                "name": "12 Mini Vol au Vents",
                "gtin": "3528960007237",
                "image": "https://static.euw1.sustained.app/products/images/3528960007237_0.jpg",
                "pack": "54g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a4b3239280a2f4407b56c54135fd7c9",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a4b3239280a2f4407b56c54135fd7c9/impacts"
                }
            },
            {
                "id": "productid.0a5c585057035e2df9f13d5c1020f745",
                "name": "Food Made Without Wheat Gluten Free Lemon Drizzle Cake",
                "gtin": "0000000027298",
                "image": "https://static.euw1.sustained.app/products/images/0000000027298_0.jpg",
                "pack": "405g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a5c585057035e2df9f13d5c1020f745",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a5c585057035e2df9f13d5c1020f745/impacts"
                }
            },
            {
                "id": "productid.0a97ef0a738c91bb859107bf2733bdf3",
                "name": "8 Gingerbread Flapjacks Grab Bag",
                "gtin": "5059512741920",
                "image": "https://static.euw1.sustained.app/products/images/5059512741920_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 86.95340971583991
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a97ef0a738c91bb859107bf2733bdf3",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a97ef0a738c91bb859107bf2733bdf3/impacts"
                }
            },
            {
                "id": "productid.0aa67f8d6709e294c00a463a16d8c312",
                "name": "The Bakery Football Cake",
                "gtin": "5050854585852",
                "image": "https://static.euw1.sustained.app/products/images/5050854585852_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0aa67f8d6709e294c00a463a16d8c312",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0aa67f8d6709e294c00a463a16d8c312/impacts"
                }
            },
            {
                "id": "productid.0af8742f7e541ad24a804e03e37c979f",
                "name": "Food Blondie Smores Fudge",
                "gtin": "0000029177769",
                "image": "https://static.euw1.sustained.app/products/images/0000029177769_0.jpg",
                "pack": "300g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0af8742f7e541ad24a804e03e37c979f",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0af8742f7e541ad24a804e03e37c979f/impacts"
                }
            },
            {
                "id": "productid.0b66e3055a0e1fc6a54eb28ad613a095",
                "name": "Bakery Chocolate Iced Ring Doughnut",
                "gtin": "03063170",
                "image": "https://static.euw1.sustained.app/products/images/03063170_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 22.085649258254282
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b66e3055a0e1fc6a54eb28ad613a095",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b66e3055a0e1fc6a54eb28ad613a095/impacts"
                }
            },
            {
                "id": "productid.0b78802c02e16d742956aeaa74192d50",
                "name": "Chocolate & Coconut Cake",
                "gtin": "0077544163577",
                "image": "https://static.euw1.sustained.app/products/images/0077544163577_0.jpg",
                "pack": "250g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b78802c02e16d742956aeaa74192d50",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b78802c02e16d742956aeaa74192d50/impacts"
                }
            },
            {
                "id": "productid.0baf842bbbc90a1126dc45b2e1377dea",
                "name": "12 Hidden Centre Cupcakes",
                "gtin": "0000000385794",
                "image": "https://static.euw1.sustained.app/products/images/0000000385794_0.jpg",
                "pack": "636g",
                "grade": "E",
                "points": {
                    "pef": 41.14596972877065
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0baf842bbbc90a1126dc45b2e1377dea",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0baf842bbbc90a1126dc45b2e1377dea/impacts"
                }
            },
            {
                "id": "productid.0bb1d9986c4f4e93a7763e2b275392e7",
                "name": "Mini Danish Selection",
                "gtin": "03320075",
                "image": "https://static.euw1.sustained.app/products/images/03320075_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 49.870480268386025
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bb1d9986c4f4e93a7763e2b275392e7",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bb1d9986c4f4e93a7763e2b275392e7/impacts"
                }
            },
            {
                "id": "productid.0befee1c60c69565d37dea9800da7544",
                "name": "Food Golden Syrup Sponge Pudding",
                "gtin": "0000029021604",
                "image": "https://static.euw1.sustained.app/products/images/0000029021604_0.jpg",
                "pack": "400g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0befee1c60c69565d37dea9800da7544",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0befee1c60c69565d37dea9800da7544/impacts"
                }
            },
            {
                "id": "productid.0c49a0066a01264e5ba9c57027fcf07b",
                "name": "Platinum Jubilee Celebration Cake",
                "gtin": "5019485114718",
                "image": "https://static.euw1.sustained.app/products/images/5019485114718_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c49a0066a01264e5ba9c57027fcf07b",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c49a0066a01264e5ba9c57027fcf07b/impacts"
                }
            },
            {
                "id": "productid.0cd02d5af477852087af27893ff9fe88",
                "name": "Chocolate & Orange Cake",
                "gtin": "00441780",
                "image": "https://static.euw1.sustained.app/products/images/00441780_0.jpg",
                "pack": "410g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0cd02d5af477852087af27893ff9fe88",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0cd02d5af477852087af27893ff9fe88/impacts"
                }
            },
            {
                "id": "productid.0d1eab8b5f37898b15364add6dca9b25",
                "name": "Celebration Chocolate Cake",
                "gtin": "5060139661553",
                "image": "https://static.euw1.sustained.app/products/images/5060139661553_0.jpg",
                "pack": "650g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d1eab8b5f37898b15364add6dca9b25",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d1eab8b5f37898b15364add6dca9b25/impacts"
                }
            },
            {
                "id": "productid.0d36886cfd0f14950a34079fd0dc7425",
                "name": "Food Pressed Fruit & Nut Bars Raspberry & Almond Butter",
                "gtin": "0000029124473",
                "image": "https://static.euw1.sustained.app/products/images/0000029124473_0.jpg",
                "pack": "35g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d36886cfd0f14950a34079fd0dc7425",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d36886cfd0f14950a34079fd0dc7425/impacts"
                }
            },
            {
                "id": "productid.0d490531f1d518c19109262a86dfd095",
                "name": "5 Roses Strawberry Dream Cake Bars",
                "gtin": "5000221602828",
                "image": "https://static.euw1.sustained.app/products/images/5000221602828_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 69.08593237920569
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d490531f1d518c19109262a86dfd095",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d490531f1d518c19109262a86dfd095/impacts"
                }
            },
            {
                "id": "productid.0d4d5d917c7629fcd789d8fb7eee3878",
                "name": "Chocolate Sponge Roll",
                "gtin": "0000000619677",
                "image": "https://static.euw1.sustained.app/products/images/0000000619677_0.jpg",
                "pack": "225g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d4d5d917c7629fcd789d8fb7eee3878",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d4d5d917c7629fcd789d8fb7eee3878/impacts"
                }
            },
            {
                "id": "productid.0dd17ba6d03197900fd91e23fad16651",
                "name": "Finest Pease Pudding",
                "gtin": "5060084360006",
                "image": "https://static.euw1.sustained.app/products/images/5060084360006_0.jpg",
                "pack": "227g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0dd17ba6d03197900fd91e23fad16651",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0dd17ba6d03197900fd91e23fad16651/impacts"
                }
            },
            {
                "id": "productid.0ddc7141ee49577804eb4867172eb7ca",
                "name": "Jam & Cream Sponge",
                "gtin": "5099823001313",
                "image": "https://static.euw1.sustained.app/products/images/5099823001313_0.jpg",
                "pack": "350g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cakes",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ddc7141ee49577804eb4867172eb7ca",
                    "category": "https://api.sustained.com/choice/v1/categories/cakes",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ddc7141ee49577804eb4867172eb7ca/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.0062647108c4de56ffa6c75d6649a333",
                "name": "Rockstar Xdurance Blueberry, Pomegranate and Acai",
                "gtin": "5000382033615",
                "image": "https://static.euw1.sustained.app/products/images/5000382033615_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0062647108c4de56ffa6c75d6649a333",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0062647108c4de56ffa6c75d6649a333/impacts"
                }
            },
            {
                "id": "productid.0086c64b089f29ccec2bdf481e64b7a2",
                "name": "Skinny Tonic Raspberry",
                "gtin": "0634158993787",
                "image": "https://static.euw1.sustained.app/products/images/0634158993787_0.jpg",
                "pack": "150ml",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0086c64b089f29ccec2bdf481e64b7a2",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0086c64b089f29ccec2bdf481e64b7a2/impacts"
                }
            },
            {
                "id": "productid.012bf20117c9b872652341fc192c1da7",
                "name": "Lucozade Energy Drink Apple Blast",
                "gtin": "5054267010765",
                "image": "https://static.euw1.sustained.app/products/images/5054267010765_0.jpg",
                "pack": "1",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.012bf20117c9b872652341fc192c1da7",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.012bf20117c9b872652341fc192c1da7/impacts"
                }
            },
            {
                "id": "productid.03e5cc7fad20b65e88c4120a52e40777",
                "name": "Pepsi Max Cherry No Sugar Cola Can PMP",
                "gtin": "4060800306630",
                "image": "https://static.euw1.sustained.app/products/images/4060800306630_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03e5cc7fad20b65e88c4120a52e40777",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03e5cc7fad20b65e88c4120a52e40777/impacts"
                }
            },
            {
                "id": "productid.0433c0d1afa5b2da7cb5bc3fb397ae62",
                "name": "Diet Coke Twisted Strawberry",
                "gtin": "5449000263483",
                "image": "https://static.euw1.sustained.app/products/images/5449000263483_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0433c0d1afa5b2da7cb5bc3fb397ae62",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0433c0d1afa5b2da7cb5bc3fb397ae62/impacts"
                }
            },
            {
                "id": "productid.0505d87d3c6a9f82c29e829eb1493690",
                "name": "Coca-Cola Zero Sugar Cinnamon",
                "gtin": "5449000260550",
                "image": "https://static.euw1.sustained.app/products/images/5449000260550_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0505d87d3c6a9f82c29e829eb1493690",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0505d87d3c6a9f82c29e829eb1493690/impacts"
                }
            },
            {
                "id": "productid.052ba982c6ea871be83115d6df114342",
                "name": "Lucozade Energy Orange",
                "gtin": "5054267009400",
                "image": "https://static.euw1.sustained.app/products/images/5054267009400_0.jpg",
                "pack": "380ml",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.052ba982c6ea871be83115d6df114342",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.052ba982c6ea871be83115d6df114342/impacts"
                }
            },
            {
                "id": "productid.053d3ce066c4eb4e88ed543ac8fb28ac",
                "name": "Morrisons No Added Sugar Bitter Shandy",
                "gtin": "5010251628378",
                "image": "https://static.euw1.sustained.app/products/images/5010251628378_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.053d3ce066c4eb4e88ed543ac8fb28ac",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.053d3ce066c4eb4e88ed543ac8fb28ac/impacts"
                }
            },
            {
                "id": "productid.060bc46a3aa4197e604be75753b0222b",
                "name": "Maine Mini Orange Crush",
                "gtin": "5011475154988",
                "image": "https://static.euw1.sustained.app/products/images/5011475154988_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.060bc46a3aa4197e604be75753b0222b",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.060bc46a3aa4197e604be75753b0222b/impacts"
                }
            },
            {
                "id": "productid.06c05ccafeed4591ff1870205c2a51a9",
                "name": "Pepsi Diet Cola",
                "gtin": "4060800307149",
                "image": "https://static.euw1.sustained.app/products/images/4060800307149_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06c05ccafeed4591ff1870205c2a51a9",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06c05ccafeed4591ff1870205c2a51a9/impacts"
                }
            },
            {
                "id": "productid.06cdf7d1d3709fb0128f95bc1771e6ab",
                "name": "7UP Free Lemon & Lime Cans",
                "gtin": "4060800108791",
                "image": "https://static.euw1.sustained.app/products/images/4060800108791_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06cdf7d1d3709fb0128f95bc1771e6ab",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06cdf7d1d3709fb0128f95bc1771e6ab/impacts"
                }
            },
            {
                "id": "productid.071b441b9515a04c1bdf39694cb70699",
                "name": "Tango Apple",
                "gtin": "5010102244382",
                "image": "https://static.euw1.sustained.app/products/images/5010102244382_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.071b441b9515a04c1bdf39694cb70699",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.071b441b9515a04c1bdf39694cb70699/impacts"
                }
            },
            {
                "id": "productid.07d23d526fb92bc997a7316236aceb89",
                "name": "Pepsi Max Cherry No Sugar Cola Can PMP",
                "gtin": "04060800165992",
                "image": "https://static.euw1.sustained.app/products/images/04060800165992_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07d23d526fb92bc997a7316236aceb89",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07d23d526fb92bc997a7316236aceb89/impacts"
                }
            },
            {
                "id": "productid.07f2e7349ef838d8ac3fb06b82217c4b",
                "name": "Carters Royal American Ginger Ale",
                "gtin": "5000177427698",
                "image": "https://static.euw1.sustained.app/products/images/5000177427698_0.jpg",
                "pack": "1l",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07f2e7349ef838d8ac3fb06b82217c4b",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07f2e7349ef838d8ac3fb06b82217c4b/impacts"
                }
            },
            {
                "id": "productid.08688c2e73d18cd26de3a8f1643f7e71",
                "name": "Lucozade Energy Caribbean Crush",
                "gtin": "5054267001138",
                "image": "https://static.euw1.sustained.app/products/images/5054267001138_0.jpg",
                "pack": "500ml",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08688c2e73d18cd26de3a8f1643f7e71",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08688c2e73d18cd26de3a8f1643f7e71/impacts"
                }
            },
            {
                "id": "productid.0879a1aa4361f5fd2fa5feebd6e0379f",
                "name": "Diet Coke",
                "gtin": "5449000053879",
                "image": "https://static.euw1.sustained.app/products/images/5449000053879_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0879a1aa4361f5fd2fa5feebd6e0379f",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0879a1aa4361f5fd2fa5feebd6e0379f/impacts"
                }
            },
            {
                "id": "productid.09271bcb44cd9ec14b52307be7f74a39",
                "name": "Fentimans Sparkling Raspberry",
                "gtin": "5029396001090",
                "image": "https://static.euw1.sustained.app/products/images/5029396001090_0.jpg",
                "pack": "750ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09271bcb44cd9ec14b52307be7f74a39",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09271bcb44cd9ec14b52307be7f74a39/impacts"
                }
            },
            {
                "id": "productid.095debc2ae6b98c04819e030b285d6e7",
                "name": "Asda Ginger Ale",
                "gtin": "21009457",
                "image": "https://static.euw1.sustained.app/products/images/21009457_0.jpg",
                "pack": "1l",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.095debc2ae6b98c04819e030b285d6e7",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.095debc2ae6b98c04819e030b285d6e7/impacts"
                }
            },
            {
                "id": "productid.09ae1292c99e5ada08e7da9abefdc74e",
                "name": "Volvic L'mon & Grapefruit Sparkling Citrus Drink",
                "gtin": "5050432003211",
                "image": "https://static.euw1.sustained.app/products/images/5050432003211_0.jpg",
                "pack": "330ml",
                "grade": "A",
                "points": {
                    "pef": 4.089601108246962
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09ae1292c99e5ada08e7da9abefdc74e",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09ae1292c99e5ada08e7da9abefdc74e/impacts"
                }
            },
            {
                "id": "productid.0a7bca5b7ae1935c52b53e6217d40f9b",
                "name": "7UP Free Cherry Can",
                "gtin": "4060800303462",
                "image": "https://static.euw1.sustained.app/products/images/4060800303462_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a7bca5b7ae1935c52b53e6217d40f9b",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a7bca5b7ae1935c52b53e6217d40f9b/impacts"
                }
            },
            {
                "id": "productid.0b4f94d0c6fe4c06667c0e87437af520",
                "name": "Coca-Cola Original Taste",
                "gtin": "5449000015174",
                "image": "https://static.euw1.sustained.app/products/images/5449000015174_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b4f94d0c6fe4c06667c0e87437af520",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b4f94d0c6fe4c06667c0e87437af520/impacts"
                }
            },
            {
                "id": "productid.0c14d78ac7779d4d1b2849994abd9b55",
                "name": "Sainsbury's Blue Bolt",
                "gtin": "01651089",
                "image": "https://static.euw1.sustained.app/products/images/01651089_0.jpg",
                "pack": "250ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c14d78ac7779d4d1b2849994abd9b55",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c14d78ac7779d4d1b2849994abd9b55/impacts"
                }
            },
            {
                "id": "productid.0dc602be01a690cdad16992074ddffaf",
                "name": "Diet Coke",
                "gtin": "5449000052957",
                "image": "https://static.euw1.sustained.app/products/images/5449000052957_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0dc602be01a690cdad16992074ddffaf",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0dc602be01a690cdad16992074ddffaf/impacts"
                }
            },
            {
                "id": "productid.0e5166fcd80d6a487ac9622249414864",
                "name": "Old Jamaica Ginger Beer Regular",
                "gtin": "5029578000330",
                "image": "https://static.euw1.sustained.app/products/images/5029578000330_0.jpg",
                "pack": "1.5l",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e5166fcd80d6a487ac9622249414864",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e5166fcd80d6a487ac9622249414864/impacts"
                }
            },
            {
                "id": "productid.0f1a20ffff1cc41374ead23f7bd999f7",
                "name": "7UP Free Cherry Bottle",
                "gtin": "4060800303578",
                "image": "https://static.euw1.sustained.app/products/images/4060800303578_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f1a20ffff1cc41374ead23f7bd999f7",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f1a20ffff1cc41374ead23f7bd999f7/impacts"
                }
            },
            {
                "id": "productid.0f66c19eed87ebfa51784e6492b0e9f3",
                "name": "Genie Kombucha Sweet Citrus",
                "gtin": "5060629370248",
                "image": "https://static.euw1.sustained.app/products/images/5060629370248_0.jpg",
                "pack": "330ml",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f66c19eed87ebfa51784e6492b0e9f3",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f66c19eed87ebfa51784e6492b0e9f3/impacts"
                }
            },
            {
                "id": "productid.0fe1b4934d014e2b021c632f59fd3799",
                "name": "TRIP CBD Infused Elderflower Mint",
                "gtin": "5070000120706",
                "image": "https://static.euw1.sustained.app/products/images/5070000120706_0.jpg",
                "pack": "250ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fe1b4934d014e2b021c632f59fd3799",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fe1b4934d014e2b021c632f59fd3799/impacts"
                }
            },
            {
                "id": "productid.10030ca10c24b6473d1d61d0ed94ea72",
                "name": "Morrisons Savers Cola",
                "gtin": "5010251429821",
                "image": "https://static.euw1.sustained.app/products/images/5010251429821_0.jpg",
                "pack": "2l",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10030ca10c24b6473d1d61d0ed94ea72",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10030ca10c24b6473d1d61d0ed94ea72/impacts"
                }
            },
            {
                "id": "productid.106a4682da88e0727ef4a99ab7da97d9",
                "name": "Pepsi Max Cherry No Sugar Cola Bottle",
                "gtin": "4060800166531",
                "image": "https://static.euw1.sustained.app/products/images/4060800166531_0.jpg",
                "pack": "2l",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.106a4682da88e0727ef4a99ab7da97d9",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.106a4682da88e0727ef4a99ab7da97d9/impacts"
                }
            },
            {
                "id": "productid.13437c8b75304bbb84483659ae687224",
                "name": "Schweppes Lemonade",
                "gtin": "5000193900861",
                "image": "https://static.euw1.sustained.app/products/images/5000193900861_0.jpg",
                "pack": "2l",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.13437c8b75304bbb84483659ae687224",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.13437c8b75304bbb84483659ae687224/impacts"
                }
            },
            {
                "id": "productid.1386de66506e8f9034f555771a06264a",
                "name": "Britvic Indian Tonic Water Low Calorie Bottle",
                "gtin": "5010102225343",
                "image": "https://static.euw1.sustained.app/products/images/5010102225343_0.jpg",
                "pack": "200ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1386de66506e8f9034f555771a06264a",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1386de66506e8f9034f555771a06264a/impacts"
                }
            },
            {
                "id": "productid.13cb8859a0e007296e3f50e157f169d3",
                "name": "Ocean Spray Cranberry Sparkling Water",
                "gtin": "0031200458162",
                "image": "https://static.euw1.sustained.app/products/images/0031200458162_0.jpg",
                "pack": "1l",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.13cb8859a0e007296e3f50e157f169d3",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.13cb8859a0e007296e3f50e157f169d3/impacts"
                }
            },
            {
                "id": "productid.1429b8ee67fb230c1f17fdfe271dbc73",
                "name": "Coca-Cola Zero Sugar",
                "gtin": "5449000154750",
                "image": "https://static.euw1.sustained.app/products/images/5449000154750_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1429b8ee67fb230c1f17fdfe271dbc73",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1429b8ee67fb230c1f17fdfe271dbc73/impacts"
                }
            },
            {
                "id": "productid.1608772a5dcb23ec945e24dbf75084ba",
                "name": "Rubicon Sparkling Mango",
                "gtin": "5000382111597",
                "image": "https://static.euw1.sustained.app/products/images/5000382111597_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1608772a5dcb23ec945e24dbf75084ba",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1608772a5dcb23ec945e24dbf75084ba/impacts"
                }
            },
            {
                "id": "productid.16ef301a83d3a9a280d85d7caaf77b8a",
                "name": "Radnor Fruits Apple & Raspberry Sparkling",
                "gtin": "5026109010042",
                "image": "https://static.euw1.sustained.app/products/images/5026109010042_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16ef301a83d3a9a280d85d7caaf77b8a",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16ef301a83d3a9a280d85d7caaf77b8a/impacts"
                }
            },
            {
                "id": "productid.16fd63f2c4ce229f05c8c94580d6215c",
                "name": "Fever-Tree Premium Ginger Beer",
                "gtin": "5060108450386",
                "image": "https://static.euw1.sustained.app/products/images/5060108450386_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16fd63f2c4ce229f05c8c94580d6215c",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16fd63f2c4ce229f05c8c94580d6215c/impacts"
                }
            },
            {
                "id": "productid.171feae2f36e031122a7986c12ab2389",
                "name": "Tesco Low Calorie Soda Water with Lime",
                "gtin": "5052909510680",
                "image": "https://static.euw1.sustained.app/products/images/5052909510680_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 8.120011103499188
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.171feae2f36e031122a7986c12ab2389",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.171feae2f36e031122a7986c12ab2389/impacts"
                }
            },
            {
                "id": "productid.17354b15a9eeebf79cb6089ef103a80b",
                "name": "Asda Blue Charge Energy Drink",
                "gtin": "21165696",
                "image": "https://static.euw1.sustained.app/products/images/21165696_0.jpg",
                "pack": "1l",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.17354b15a9eeebf79cb6089ef103a80b",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.17354b15a9eeebf79cb6089ef103a80b/impacts"
                }
            },
            {
                "id": "productid.1809ad567ba7412ba4eb24ac32274da0",
                "name": "Whole Earth Organic Sparkling Elderflower",
                "gtin": "5011835102765",
                "image": "https://static.euw1.sustained.app/products/images/5011835102765_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1809ad567ba7412ba4eb24ac32274da0",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1809ad567ba7412ba4eb24ac32274da0/impacts"
                }
            },
            {
                "id": "productid.19523b5b4a16a51c09d27b5f958dcf8d",
                "name": "Thums Up",
                "gtin": "5017726417109",
                "image": "https://static.euw1.sustained.app/products/images/5017726417109_0.jpg",
                "pack": "300ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.19523b5b4a16a51c09d27b5f958dcf8d",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.19523b5b4a16a51c09d27b5f958dcf8d/impacts"
                }
            },
            {
                "id": "productid.19a3bebc598c0aa47f5cf9345b96d964",
                "name": "Club Zero Rock Shandy Cans",
                "gtin": "5391521831972",
                "image": "https://static.euw1.sustained.app/products/images/5391521831972_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.19a3bebc598c0aa47f5cf9345b96d964",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.19a3bebc598c0aa47f5cf9345b96d964/impacts"
                }
            },
            {
                "id": "productid.19afe5b669d0983d5a8a67dcc23414bf",
                "name": "7UP Lemon & Lime Cans",
                "gtin": "4060800304490",
                "image": "https://static.euw1.sustained.app/products/images/4060800304490_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.19afe5b669d0983d5a8a67dcc23414bf",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.19afe5b669d0983d5a8a67dcc23414bf/impacts"
                }
            },
            {
                "id": "productid.1aef72221b0c7c913a8a24f681108802",
                "name": "Tesco Peach & Passion Fruit Flavoured Sparkling Spring Water",
                "gtin": "5059316126022",
                "image": "https://static.euw1.sustained.app/products/images/5059316126022_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 2.6620068234190764
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1aef72221b0c7c913a8a24f681108802",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1aef72221b0c7c913a8a24f681108802/impacts"
                }
            },
            {
                "id": "productid.1b1a9161a3ef93f6dda721ec865b6d78",
                "name": "Pepsi Diet Cola Cans",
                "gtin": "4060800305497",
                "image": "https://static.euw1.sustained.app/products/images/4060800305497_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b1a9161a3ef93f6dda721ec865b6d78",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b1a9161a3ef93f6dda721ec865b6d78/impacts"
                }
            },
            {
                "id": "productid.1b9ce292c9a9fc6522c56099f0bbb651",
                "name": "Tesco Low Calorie Indian Tonic Water",
                "gtin": "5000119092274",
                "image": "https://static.euw1.sustained.app/products/images/5000119092274_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 8.120011103499188
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b9ce292c9a9fc6522c56099f0bbb651",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b9ce292c9a9fc6522c56099f0bbb651/impacts"
                }
            },
            {
                "id": "productid.1bc240168c28f1581b46c0f09d26fb35",
                "name": "M&S Diet Ginger Ale",
                "gtin": "29258970",
                "image": "https://static.euw1.sustained.app/products/images/29258970_0.jpg",
                "pack": "1l",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1bc240168c28f1581b46c0f09d26fb35",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1bc240168c28f1581b46c0f09d26fb35/impacts"
                }
            },
            {
                "id": "productid.1deccf1ecc3c911a67ac575c75c49d76",
                "name": "Asda Low Calorie Ginger Ale",
                "gtin": "21009471",
                "image": "https://static.euw1.sustained.app/products/images/21009471_0.jpg",
                "pack": "1l",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1deccf1ecc3c911a67ac575c75c49d76",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1deccf1ecc3c911a67ac575c75c49d76/impacts"
                }
            },
            {
                "id": "productid.1e47408db17888ddb60b9afe5f79ccd9",
                "name": "M&S Blood Orange Sparkling Water",
                "gtin": "0000029237418",
                "image": "https://static.euw1.sustained.app/products/images/0000029237418_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e47408db17888ddb60b9afe5f79ccd9",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e47408db17888ddb60b9afe5f79ccd9/impacts"
                }
            },
            {
                "id": "productid.1e91bd7293fa7845987fc067c9d55ecf",
                "name": "Maine Bubblegum",
                "gtin": "5011475359703",
                "image": "https://static.euw1.sustained.app/products/images/5011475359703_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e91bd7293fa7845987fc067c9d55ecf",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e91bd7293fa7845987fc067c9d55ecf/impacts"
                }
            },
            {
                "id": "productid.1ea5185d97ab7b02fa98c8b57cc2569d",
                "name": "SAVYLL BEVERAGE CO. Alcohol Free Cocktail Mojito Flavour",
                "gtin": "5060718680227",
                "image": "https://static.euw1.sustained.app/products/images/5060718680227_0.jpg",
                "pack": "250ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Carbonated Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1ea5185d97ab7b02fa98c8b57cc2569d",
                    "category": "https://api.sustained.com/choice/v1/categories/carbonated-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1ea5185d97ab7b02fa98c8b57cc2569d/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.00afb21e67ae04fef2f802fa6ab34826",
                "name": "Soft Baked Breakfast Biscuits Chocolate Chips",
                "gtin": "5054781542841",
                "image": "https://static.euw1.sustained.app/products/images/5054781542841_0.jpg",
                "pack": "250g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00afb21e67ae04fef2f802fa6ab34826",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00afb21e67ae04fef2f802fa6ab34826/impacts"
                }
            },
            {
                "id": "productid.00ea4150b15290bf1cb4fa38bca34bb0",
                "name": "Irish Oaty Flapjacks Multiseed",
                "gtin": "5099801002264",
                "image": "https://static.euw1.sustained.app/products/images/5099801002264_0.jpg",
                "pack": "40g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00ea4150b15290bf1cb4fa38bca34bb0",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00ea4150b15290bf1cb4fa38bca34bb0/impacts"
                }
            },
            {
                "id": "productid.0112395d8c7db776ae71fe80d3a0e514",
                "name": "Rice Krispies Squares Caramel & Chocolate",
                "gtin": "5053827171823",
                "image": "https://static.euw1.sustained.app/products/images/5053827171823_0.jpg",
                "pack": "36g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0112395d8c7db776ae71fe80d3a0e514",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0112395d8c7db776ae71fe80d3a0e514/impacts"
                }
            },
            {
                "id": "productid.0112eeabb08662627d69c5bff6092608",
                "name": "Vitamin Muesli Cereal Bar Raisins / Nuts",
                "gtin": "5908260252793",
                "image": "https://static.euw1.sustained.app/products/images/5908260252793_0.jpg",
                "pack": "40g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0112eeabb08662627d69c5bff6092608",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0112eeabb08662627d69c5bff6092608/impacts"
                }
            },
            {
                "id": "productid.012bfb2bf617baa22ab06087ff22ba49",
                "name": "Quaker Oat So Simple Original Porridge",
                "gtin": "5000108105145",
                "image": "https://static.euw1.sustained.app/products/images/5000108105145_0.jpg",
                "pack": "216g",
                "grade": "D",
                "points": {
                    "pef": 23.223260491395234
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.012bfb2bf617baa22ab06087ff22ba49",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.012bfb2bf617baa22ab06087ff22ba49/impacts"
                }
            },
            {
                "id": "productid.01cfbd317e365f7f3dc9ac01faddad57",
                "name": "Oat Bran",
                "gtin": "5099801000109",
                "image": "https://static.euw1.sustained.app/products/images/5099801000109_0.jpg",
                "pack": "750g",
                "grade": "B",
                "points": {
                    "pef": 14.212411339904271
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01cfbd317e365f7f3dc9ac01faddad57",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01cfbd317e365f7f3dc9ac01faddad57/impacts"
                }
            },
            {
                "id": "productid.03b9f3dfd3579c247cf8c0092a34e4de",
                "name": "Nutri-Grain Bars Strawberry",
                "gtin": "5053827186766",
                "image": "https://static.euw1.sustained.app/products/images/5053827186766_0.jpg",
                "pack": "37g",
                "grade": "E",
                "points": {
                    "pef": 41.337689818659605
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03b9f3dfd3579c247cf8c0092a34e4de",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03b9f3dfd3579c247cf8c0092a34e4de/impacts"
                }
            },
            {
                "id": "productid.0433de1819894ac8cf1b4ebc6e69efe5",
                "name": "High Bran",
                "gtin": "5057172086573",
                "image": "https://static.euw1.sustained.app/products/images/5057172086573_0.jpg",
                "pack": "500g",
                "grade": "A",
                "points": {
                    "pef": 9.276682319611124
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0433de1819894ac8cf1b4ebc6e69efe5",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0433de1819894ac8cf1b4ebc6e69efe5/impacts"
                }
            },
            {
                "id": "productid.043bc9b1fcd960759a4737474dfcb590",
                "name": "Express Quick Pre-Cooked Polenta with Cheese",
                "gtin": "8001840289398",
                "image": "https://static.euw1.sustained.app/products/images/8001840289398_0.jpg",
                "pack": "70g",
                "grade": "D",
                "points": {
                    "pef": 25.933558109798202
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.043bc9b1fcd960759a4737474dfcb590",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.043bc9b1fcd960759a4737474dfcb590/impacts"
                }
            },
            {
                "id": "productid.050ac3afebd4365545d5f1a11038f9cc",
                "name": "Protein Flapjacks Peanut Butter",
                "gtin": "5060088709962",
                "image": "https://static.euw1.sustained.app/products/images/5060088709962_0.jpg",
                "pack": "40g",
                "grade": "D",
                "points": {
                    "pef": 26.605136777677195
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.050ac3afebd4365545d5f1a11038f9cc",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.050ac3afebd4365545d5f1a11038f9cc/impacts"
                }
            },
            {
                "id": "productid.05c85fe8b8fef1934a0d15418bd61eb9",
                "name": "Food Chocolate & Orange Granola",
                "gtin": "29285723",
                "image": "https://static.euw1.sustained.app/products/images/29285723_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 43.832156224476755
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05c85fe8b8fef1934a0d15418bd61eb9",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05c85fe8b8fef1934a0d15418bd61eb9/impacts"
                }
            },
            {
                "id": "productid.0677cd9b934dfa9a683ee58cbfffbb71",
                "name": "Kellogg's Crunchy Nut Chocolate Clusters Cereal",
                "gtin": "05050083706813",
                "image": "https://static.euw1.sustained.app/products/images/05050083706813_0.jpg",
                "pack": "450g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0677cd9b934dfa9a683ee58cbfffbb71",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0677cd9b934dfa9a683ee58cbfffbb71/impacts"
                }
            },
            {
                "id": "productid.0791645ebc24dbffab5633e65ee54910",
                "name": "Shape & Slim Meal Replacement Bars Milk Chocolate Crunch",
                "gtin": "0000000447478",
                "image": "https://static.euw1.sustained.app/products/images/0000000447478_0.jpg",
                "pack": "60g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0791645ebc24dbffab5633e65ee54910",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0791645ebc24dbffab5633e65ee54910/impacts"
                }
            },
            {
                "id": "productid.084b17974b05da68eb5b0118a7f38846",
                "name": "Organic Gluten Free Maple Sunrise",
                "gtin": "0058449771999",
                "image": "https://static.euw1.sustained.app/products/images/0058449771999_0.jpg",
                "pack": "332g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.084b17974b05da68eb5b0118a7f38846",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.084b17974b05da68eb5b0118a7f38846/impacts"
                }
            },
            {
                "id": "productid.08c67eddb9d891946dfc4d2206ea2dc8",
                "name": "Kellogg's Crunchy Nut Granola Fusion Chocolate & Hazelnut",
                "gtin": "5059319001463",
                "image": "https://static.euw1.sustained.app/products/images/5059319001463_0.jpg",
                "pack": "65g",
                "grade": "D",
                "points": {
                    "pef": 31.215745656944538
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08c67eddb9d891946dfc4d2206ea2dc8",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08c67eddb9d891946dfc4d2206ea2dc8/impacts"
                }
            },
            {
                "id": "productid.08cfb38fb916a5ad80a0095fa030e0cc",
                "name": "Protein Crunch Cereal",
                "gtin": "5010029221084",
                "image": "https://static.euw1.sustained.app/products/images/5010029221084_0.jpg",
                "pack": "450g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08cfb38fb916a5ad80a0095fa030e0cc",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08cfb38fb916a5ad80a0095fa030e0cc/impacts"
                }
            },
            {
                "id": "productid.08d5ffe253ed0a262612473b46486d5f",
                "name": "Porridge Oats",
                "gtin": "5057545744765",
                "image": "https://static.euw1.sustained.app/products/images/5057545744765_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 23.223260491395234
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08d5ffe253ed0a262612473b46486d5f",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08d5ffe253ed0a262612473b46486d5f/impacts"
                }
            },
            {
                "id": "productid.09110aa8072b113f6fdbe00e35ce703b",
                "name": "Fruit 'n Fibre Cereal",
                "gtin": "5000127072091",
                "image": "https://static.euw1.sustained.app/products/images/5000127072091_0.jpg",
                "pack": "750g",
                "grade": "B",
                "points": {
                    "pef": 12.299573149806275
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09110aa8072b113f6fdbe00e35ce703b",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09110aa8072b113f6fdbe00e35ce703b/impacts"
                }
            },
            {
                "id": "productid.09a3b69a083906050194b443416f5c23",
                "name": "Raw Fruit & Nut Bars Peanut Cocoa",
                "gtin": "5060088709856",
                "image": "https://static.euw1.sustained.app/products/images/5060088709856_0.jpg",
                "pack": "30g",
                "grade": "F",
                "points": {
                    "pef": 61.155065498209254
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09a3b69a083906050194b443416f5c23",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09a3b69a083906050194b443416f5c23/impacts"
                }
            },
            {
                "id": "productid.0a44d3ff358a6284cce02871d2a81d88",
                "name": "The Original",
                "gtin": "7613036177719",
                "image": "https://static.euw1.sustained.app/products/images/7613036177719_0.jpg",
                "pack": "700g",
                "grade": "B",
                "points": {
                    "pef": 12.977049311807935
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a44d3ff358a6284cce02871d2a81d88",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a44d3ff358a6284cce02871d2a81d88/impacts"
                }
            },
            {
                "id": "productid.0a9ca3d130c691f907324f3363c628b8",
                "name": "Original Porridge",
                "gtin": "0000029032594",
                "image": "https://static.euw1.sustained.app/products/images/0000029032594_0.jpg",
                "pack": "70g",
                "grade": "F",
                "points": {
                    "pef": 53.93642493040931
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a9ca3d130c691f907324f3363c628b8",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a9ca3d130c691f907324f3363c628b8/impacts"
                }
            },
            {
                "id": "productid.0ae1a4f9b6e33056cec4a5230d682304",
                "name": "Corn Flakes Cereal",
                "gtin": "05053827190787",
                "image": "https://static.euw1.sustained.app/products/images/05053827190787_0.jpg",
                "pack": "1.2kg",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ae1a4f9b6e33056cec4a5230d682304",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ae1a4f9b6e33056cec4a5230d682304/impacts"
                }
            },
            {
                "id": "productid.0c358452ffffc7f61b8aad1012374991",
                "name": "Instant Oats Berry & Cherry",
                "gtin": "5057008101180",
                "image": "https://static.euw1.sustained.app/products/images/5057008101180_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 23.223260491395234
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c358452ffffc7f61b8aad1012374991",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c358452ffffc7f61b8aad1012374991/impacts"
                }
            },
            {
                "id": "productid.0c373f84275b56b5a5a67cf4043e34d1",
                "name": "Apple & Cinnamon Porridge Oat Bars",
                "gtin": "5060183673892",
                "image": "https://static.euw1.sustained.app/products/images/5060183673892_0.jpg",
                "pack": "42g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c373f84275b56b5a5a67cf4043e34d1",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c373f84275b56b5a5a67cf4043e34d1/impacts"
                }
            },
            {
                "id": "productid.0cce06f25b7a616da40c691a579c9942",
                "name": "Zesty & Crunchy Organic Orange Choc Cashew",
                "gtin": "0768114290297",
                "image": "https://static.euw1.sustained.app/products/images/0768114290297_0.jpg",
                "pack": "76g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0cce06f25b7a616da40c691a579c9942",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0cce06f25b7a616da40c691a579c9942/impacts"
                }
            },
            {
                "id": "productid.0eae2d74a5242f5e2a89f65d279e710d",
                "name": "Bircher Muesli",
                "gtin": "5039303005851",
                "image": "https://static.euw1.sustained.app/products/images/5039303005851_0.jpg",
                "pack": "450g",
                "grade": "E",
                "points": {
                    "pef": 31.56234279931717
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0eae2d74a5242f5e2a89f65d279e710d",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0eae2d74a5242f5e2a89f65d279e710d/impacts"
                }
            },
            {
                "id": "productid.0ebf4184b662fd915253d3b6eda08684",
                "name": "Kellogg's Special K Original Cereal",
                "gtin": "5053827213998",
                "image": "https://static.euw1.sustained.app/products/images/5053827213998_0.jpg",
                "pack": "375g",
                "grade": "C",
                "points": {
                    "pef": 20.326527816697812
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ebf4184b662fd915253d3b6eda08684",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ebf4184b662fd915253d3b6eda08684/impacts"
                }
            },
            {
                "id": "productid.0f0b6366a4cd446997eb53bb1ffbc673",
                "name": "Food Cereal Milk Clusters Mini Bites",
                "gtin": "0000029139170",
                "image": "https://static.euw1.sustained.app/products/images/0000029139170_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f0b6366a4cd446997eb53bb1ffbc673",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f0b6366a4cd446997eb53bb1ffbc673/impacts"
                }
            },
            {
                "id": "productid.0fd909ed35f1f47ea6fc43873dfa0327",
                "name": "Frosted Wheats Breakfast Cereal",
                "gtin": "5059319002781",
                "image": "https://static.euw1.sustained.app/products/images/5059319002781_0.jpg",
                "pack": "500g",
                "grade": "A",
                "points": {
                    "pef": 10.019491545117033
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fd909ed35f1f47ea6fc43873dfa0327",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fd909ed35f1f47ea6fc43873dfa0327/impacts"
                }
            },
            {
                "id": "productid.0ffc93c535a04f2180b73c0d193dda26",
                "name": "Oh Wow Cacao Chocolate",
                "gtin": "5060113081124",
                "image": "https://static.euw1.sustained.app/products/images/5060113081124_0.jpg",
                "pack": "38g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ffc93c535a04f2180b73c0d193dda26",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ffc93c535a04f2180b73c0d193dda26/impacts"
                }
            },
            {
                "id": "productid.104a9fe9385aae6f8648ad8b06c85d9e",
                "name": "Kellogg's Crunchy Nut Cereal",
                "gtin": "05050083392863",
                "image": "https://static.euw1.sustained.app/products/images/05050083392863_0.jpg",
                "pack": "35g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.104a9fe9385aae6f8648ad8b06c85d9e",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.104a9fe9385aae6f8648ad8b06c85d9e/impacts"
                }
            },
            {
                "id": "productid.1068d6d8bf640898dbb4088206083e2c",
                "name": "Mighty Oats Steel Cut Oatmeal Coarse",
                "gtin": "5010026518118",
                "image": "https://static.euw1.sustained.app/products/images/5010026518118_0.jpg",
                "pack": "750g",
                "grade": "D",
                "points": {
                    "pef": 27.883603540400742
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1068d6d8bf640898dbb4088206083e2c",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1068d6d8bf640898dbb4088206083e2c/impacts"
                }
            },
            {
                "id": "productid.113a0046d57125ce69625696082833d8",
                "name": "Food Sultana Bran Flakes",
                "gtin": "0000029058976",
                "image": "https://static.euw1.sustained.app/products/images/0000029058976_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 33.67553932313148
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.113a0046d57125ce69625696082833d8",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.113a0046d57125ce69625696082833d8/impacts"
                }
            },
            {
                "id": "productid.129901dbd8cef0d1579efd006ae13596",
                "name": "Rice Krispies Squares",
                "gtin": "5059321004353",
                "image": "https://static.euw1.sustained.app/products/images/5059321004353_0.jpg",
                "pack": "28g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.129901dbd8cef0d1579efd006ae13596",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.129901dbd8cef0d1579efd006ae13596/impacts"
                }
            },
            {
                "id": "productid.138a9ec0742a42e522f7d448595121de",
                "name": "SO Organic Cous Cous",
                "gtin": "0000000447775",
                "image": "https://static.euw1.sustained.app/products/images/0000000447775_0.jpg",
                "pack": "500g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.138a9ec0742a42e522f7d448595121de",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.138a9ec0742a42e522f7d448595121de/impacts"
                }
            },
            {
                "id": "productid.13ab2133772e8be118521ecd3915043a",
                "name": "Cheerios Multigrain",
                "gtin": "8445290301857",
                "image": "https://static.euw1.sustained.app/products/images/8445290301857_0.jpg",
                "pack": "390g",
                "grade": "C",
                "points": {
                    "pef": 19.10477697632755
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.13ab2133772e8be118521ecd3915043a",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.13ab2133772e8be118521ecd3915043a/impacts"
                }
            },
            {
                "id": "productid.13ba6bf35919a4aebdaef20db1e9962d",
                "name": "by Kids No Added Sugar Blueberry, Apple & Beetroot Multigrain Shapes Cereal",
                "gtin": "5059319000695",
                "image": "https://static.euw1.sustained.app/products/images/5059319000695_0.jpg",
                "pack": "350g",
                "grade": "E",
                "points": {
                    "pef": 37.97179981260792
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.13ba6bf35919a4aebdaef20db1e9962d",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.13ba6bf35919a4aebdaef20db1e9962d/impacts"
                }
            },
            {
                "id": "productid.152086bb2b1fa9e4270c9b304a409ba1",
                "name": "Granola Cranberry, Coconut & Chia Seed",
                "gtin": "0799439064164",
                "image": "https://static.euw1.sustained.app/products/images/0799439064164_0.jpg",
                "pack": "400g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.152086bb2b1fa9e4270c9b304a409ba1",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.152086bb2b1fa9e4270c9b304a409ba1/impacts"
                }
            },
            {
                "id": "productid.15df5e7ce44ae1f2146aceb81aafea25",
                "name": "Kefir Mango & Passion Fruit Granola Pot",
                "gtin": "0000029036622",
                "image": "https://static.euw1.sustained.app/products/images/0000029036622_0.jpg",
                "pack": "125g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.15df5e7ce44ae1f2146aceb81aafea25",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.15df5e7ce44ae1f2146aceb81aafea25/impacts"
                }
            },
            {
                "id": "productid.1652e4ade29a0c3d3fc4d991260e1410",
                "name": "Crunchy Nut Original Cereal",
                "gtin": "5053827209762",
                "image": "https://static.euw1.sustained.app/products/images/5053827209762_0.jpg",
                "pack": "300g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1652e4ade29a0c3d3fc4d991260e1410",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1652e4ade29a0c3d3fc4d991260e1410/impacts"
                }
            },
            {
                "id": "productid.1662b55969b30592194c6cedf305ccbf",
                "name": "Bouncy Berry Fruity Oat Bars",
                "gtin": "5060376811292",
                "image": "https://static.euw1.sustained.app/products/images/5060376811292_0.jpg",
                "pack": "20g",
                "grade": "E",
                "points": {
                    "pef": 32.77976484497667
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1662b55969b30592194c6cedf305ccbf",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1662b55969b30592194c6cedf305ccbf/impacts"
                }
            },
            {
                "id": "productid.176b63ed9fbb23069747dc8d43499790",
                "name": "Fantastically Fruity Muesli",
                "gtin": "5018357012459",
                "image": "https://static.euw1.sustained.app/products/images/5018357012459_0.jpg",
                "pack": "600g",
                "grade": "E",
                "points": {
                    "pef": 35.02302716468363
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.176b63ed9fbb23069747dc8d43499790",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.176b63ed9fbb23069747dc8d43499790/impacts"
                }
            },
            {
                "id": "productid.18c8c3dc8aee53650ce5c372df524cc5",
                "name": "Food Luxury Toasted Nut & Fruit Granola",
                "gtin": "00000029045143",
                "image": "https://static.euw1.sustained.app/products/images/00000029045143_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 46.74389862974191
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.18c8c3dc8aee53650ce5c372df524cc5",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.18c8c3dc8aee53650ce5c372df524cc5/impacts"
                }
            },
            {
                "id": "productid.1a4e1b36063bbb386914e485b6784ccb",
                "name": "Cheerios Multigrain",
                "gtin": "5011476100885",
                "image": "https://static.euw1.sustained.app/products/images/5011476100885_0.jpg",
                "pack": "600g",
                "grade": "C",
                "points": {
                    "pef": 19.10477697632755
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1a4e1b36063bbb386914e485b6784ccb",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1a4e1b36063bbb386914e485b6784ccb/impacts"
                }
            },
            {
                "id": "productid.1ab6b810a7c863dd0670ebc32e798732",
                "name": "Tasty Raspberry",
                "gtin": "5060376810356",
                "image": "https://static.euw1.sustained.app/products/images/5060376810356_0.jpg",
                "pack": "35g",
                "grade": "E",
                "points": {
                    "pef": 32.25286565559492
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1ab6b810a7c863dd0670ebc32e798732",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1ab6b810a7c863dd0670ebc32e798732/impacts"
                }
            },
            {
                "id": "productid.1ac559dee3f9e07c79021aa833ff643f",
                "name": "Cypressa Couscous",
                "gtin": "5000362250162",
                "image": "https://static.euw1.sustained.app/products/images/5000362250162_0.jpg",
                "pack": "1kg",
                "grade": "D",
                "points": {
                    "pef": 27.057244790621702
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1ac559dee3f9e07c79021aa833ff643f",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1ac559dee3f9e07c79021aa833ff643f/impacts"
                }
            },
            {
                "id": "productid.1af10bc89596dfb54b970f1d2e816c3e",
                "name": "Crunchy Nut Original Cereal",
                "gtin": "5053827209632",
                "image": "https://static.euw1.sustained.app/products/images/5053827209632_0.jpg",
                "pack": "840g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1af10bc89596dfb54b970f1d2e816c3e",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1af10bc89596dfb54b970f1d2e816c3e/impacts"
                }
            },
            {
                "id": "productid.1bd47e67ade8958468eeac74781f73b3",
                "name": "Cerealy Tasty Gluten Free Jumbo Oat Flakes",
                "gtin": "5390362006617",
                "image": "https://static.euw1.sustained.app/products/images/5390362006617_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 27.883603540400742
                },
                "info_icons": [],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1bd47e67ade8958468eeac74781f73b3",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1bd47e67ade8958468eeac74781f73b3/impacts"
                }
            },
            {
                "id": "productid.1c114219372b4072f3ab79c3f1b3ca44",
                "name": "The Original",
                "gtin": "7613036912617",
                "image": "https://static.euw1.sustained.app/products/images/7613036912617_0.jpg",
                "pack": "40g",
                "grade": "B",
                "points": {
                    "pef": 12.967259866123825
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c114219372b4072f3ab79c3f1b3ca44",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c114219372b4072f3ab79c3f1b3ca44/impacts"
                }
            },
            {
                "id": "productid.1ccc4b7ca1cd0809a2c87284bdd5be18",
                "name": "Bran Flakes",
                "gtin": "5054781745228",
                "image": "https://static.euw1.sustained.app/products/images/5054781745228_0.jpg",
                "pack": "1kg",
                "grade": "A",
                "points": {
                    "pef": 9.794572695451292
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cereals",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1ccc4b7ca1cd0809a2c87284bdd5be18",
                    "category": "https://api.sustained.com/choice/v1/categories/cereals",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1ccc4b7ca1cd0809a2c87284bdd5be18/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.000cbe34711a8fd6faed3ac200003aee",
                "name": "La Bellamente Pinot Grigio Ros\u00e9",
                "gtin": "5028235010125",
                "image": "https://static.euw1.sustained.app/products/images/5028235010125_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.000cbe34711a8fd6faed3ac200003aee",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.000cbe34711a8fd6faed3ac200003aee/impacts"
                }
            },
            {
                "id": "productid.0150c4ddf078e3ad7b9bfa99ace7c068",
                "name": "Santa Margherita Stilrose",
                "gtin": "8016729902008",
                "image": "https://static.euw1.sustained.app/products/images/8016729902008_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0150c4ddf078e3ad7b9bfa99ace7c068",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0150c4ddf078e3ad7b9bfa99ace7c068/impacts"
                }
            },
            {
                "id": "productid.037fbb016851b6b84c6d460fa70c0562",
                "name": "Blossom Hill White Zinfandel",
                "gtin": "5060078184113",
                "image": "https://static.euw1.sustained.app/products/images/5060078184113_0.jpg",
                "pack": "187ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.037fbb016851b6b84c6d460fa70c0562",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.037fbb016851b6b84c6d460fa70c0562/impacts"
                }
            },
            {
                "id": "productid.050f3fa47ea5ebf5bc89fee550dd3328",
                "name": "Berne Inspiration C\u00f4tes de Provence",
                "gtin": "3760033597584",
                "image": "https://static.euw1.sustained.app/products/images/3760033597584_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.050f3fa47ea5ebf5bc89fee550dd3328",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.050f3fa47ea5ebf5bc89fee550dd3328/impacts"
                }
            },
            {
                "id": "productid.05a9357f420e50e3f591b33d5140ff08",
                "name": "Bijou Horizon Grenache Ros\u00e9",
                "gtin": "3552657021213",
                "image": "https://static.euw1.sustained.app/products/images/3552657021213_0.jpg",
                "pack": "187ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05a9357f420e50e3f591b33d5140ff08",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05a9357f420e50e3f591b33d5140ff08/impacts"
                }
            },
            {
                "id": "productid.05af3ae25d0a196d7276e6a5a2e98957",
                "name": "Calafia Ros\u00e9",
                "gtin": "5052898004870",
                "image": "https://static.euw1.sustained.app/products/images/5052898004870_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05af3ae25d0a196d7276e6a5a2e98957",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05af3ae25d0a196d7276e6a5a2e98957/impacts"
                }
            },
            {
                "id": "productid.080b1cda361105de7971c02214291154",
                "name": "Barefoot Pink Moscato Ros\u00e9 Wine",
                "gtin": "0085000021095",
                "image": "https://static.euw1.sustained.app/products/images/0085000021095_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.080b1cda361105de7971c02214291154",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.080b1cda361105de7971c02214291154/impacts"
                }
            },
            {
                "id": "productid.08d65d07f58f22b216d7e53a3e63b223",
                "name": "La Gioiosa Rosea Brut",
                "gtin": "8006805159013",
                "image": "https://static.euw1.sustained.app/products/images/8006805159013_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08d65d07f58f22b216d7e53a3e63b223",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08d65d07f58f22b216d7e53a3e63b223/impacts"
                }
            },
            {
                "id": "productid.095c1787a3befd1c03d283b1049e07b5",
                "name": "Sensi 18k Pinot Noir-Ros\u00e8 Brut",
                "gtin": "8002477177560",
                "image": "https://static.euw1.sustained.app/products/images/8002477177560_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.095c1787a3befd1c03d283b1049e07b5",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.095c1787a3befd1c03d283b1049e07b5/impacts"
                }
            },
            {
                "id": "productid.0adabe547b3e64b9c8d1a30f11f6fbdb",
                "name": "Santa Julia Malbec Ros\u00e9",
                "gtin": "7791728010602",
                "image": "https://static.euw1.sustained.app/products/images/7791728010602_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0adabe547b3e64b9c8d1a30f11f6fbdb",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0adabe547b3e64b9c8d1a30f11f6fbdb/impacts"
                }
            },
            {
                "id": "productid.0ce2d9b00477c19fbe9e33bc2300faf6",
                "name": "Heidsieck & Co Champagne Monopole Rose Top Brut",
                "gtin": "3256931070545",
                "image": "https://static.euw1.sustained.app/products/images/3256931070545_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 19.291999921716172
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ce2d9b00477c19fbe9e33bc2300faf6",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ce2d9b00477c19fbe9e33bc2300faf6/impacts"
                }
            },
            {
                "id": "productid.0fe770ed2bc62a5be8a9d2828ad53c89",
                "name": "Mondelli Pinot Grigio",
                "gtin": "00367950",
                "image": "https://static.euw1.sustained.app/products/images/00367950_0.jpg",
                "pack": "2.25l",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fe770ed2bc62a5be8a9d2828ad53c89",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fe770ed2bc62a5be8a9d2828ad53c89/impacts"
                }
            },
            {
                "id": "productid.1063f6baa18322b45cc8da84594a211a",
                "name": "M&S Found Vinho Verde D.O.C. Ros\u00e9",
                "gtin": "0000029130337",
                "image": "https://static.euw1.sustained.app/products/images/0000029130337_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1063f6baa18322b45cc8da84594a211a",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1063f6baa18322b45cc8da84594a211a/impacts"
                }
            },
            {
                "id": "productid.10e82cb4c256acb754f5d11efca5ba68",
                "name": "Camel Valley Pinot Noir Ros\u00e9",
                "gtin": "5060031430080",
                "image": "https://static.euw1.sustained.app/products/images/5060031430080_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10e82cb4c256acb754f5d11efca5ba68",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10e82cb4c256acb754f5d11efca5ba68/impacts"
                }
            },
            {
                "id": "productid.12067e917d75e226d1f9c39934981d98",
                "name": "PinotPinot Pinot Grigio Pink Spritzer",
                "gtin": "5060152096363",
                "image": "https://static.euw1.sustained.app/products/images/5060152096363_0.jpg",
                "pack": "200ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12067e917d75e226d1f9c39934981d98",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12067e917d75e226d1f9c39934981d98/impacts"
                }
            },
            {
                "id": "productid.14ffbb144ff2270a891428178f9cba26",
                "name": "Luc Belaire Ros\u00e9",
                "gtin": "3304020003391",
                "image": "https://static.euw1.sustained.app/products/images/3304020003391_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14ffbb144ff2270a891428178f9cba26",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14ffbb144ff2270a891428178f9cba26/impacts"
                }
            },
            {
                "id": "productid.17b3dd5675e1505869450468b85d0334",
                "name": "M&S Cotes du Rhone Rose",
                "gtin": "0000029056279",
                "image": "https://static.euw1.sustained.app/products/images/0000029056279_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.17b3dd5675e1505869450468b85d0334",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.17b3dd5675e1505869450468b85d0334/impacts"
                }
            },
            {
                "id": "productid.18cea46ffaa6b328485059b90fa0d745",
                "name": "Freixenet Cordon Rosado Selecci\u00f3n Brut NV",
                "gtin": "8410036001094",
                "image": "https://static.euw1.sustained.app/products/images/8410036001094_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.18cea46ffaa6b328485059b90fa0d745",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.18cea46ffaa6b328485059b90fa0d745/impacts"
                }
            },
            {
                "id": "productid.1a53d10957097857fcf91d90dbda5666",
                "name": "Marqu\u00e9s de Leon Tempranillo Ros\u00e9 Wine",
                "gtin": "8410702054508",
                "image": "https://static.euw1.sustained.app/products/images/8410702054508_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1a53d10957097857fcf91d90dbda5666",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1a53d10957097857fcf91d90dbda5666/impacts"
                }
            },
            {
                "id": "productid.1a567ee607e4211bb7190f1952a206da",
                "name": "Asda Extra Special Pinot Grigio Ros\u00e9",
                "gtin": "5057172019540",
                "image": "https://static.euw1.sustained.app/products/images/5057172019540_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1a567ee607e4211bb7190f1952a206da",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1a567ee607e4211bb7190f1952a206da/impacts"
                }
            },
            {
                "id": "productid.1b31726478e76acd64039ee66c7bcc3c",
                "name": "Stowells White Zinfandel",
                "gtin": "5010186017087",
                "image": "https://static.euw1.sustained.app/products/images/5010186017087_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b31726478e76acd64039ee66c7bcc3c",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b31726478e76acd64039ee66c7bcc3c/impacts"
                }
            },
            {
                "id": "productid.1b5a69656447f5627538948ee6aa6541",
                "name": "Freixenet Ice Ros\u00e9 Cava",
                "gtin": "8410036806521",
                "image": "https://static.euw1.sustained.app/products/images/8410036806521_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b5a69656447f5627538948ee6aa6541",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b5a69656447f5627538948ee6aa6541/impacts"
                }
            },
            {
                "id": "productid.1b8da7fdcd6d82c47c05c745836d21f8",
                "name": "R\u00e9my Martin VSOP Cognac Fine Champagne",
                "gtin": "3024482235108",
                "image": "https://static.euw1.sustained.app/products/images/3024482235108_0.jpg",
                "pack": "35cl",
                "grade": "C",
                "points": {
                    "pef": 19.291999921716172
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b8da7fdcd6d82c47c05c745836d21f8",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b8da7fdcd6d82c47c05c745836d21f8/impacts"
                }
            },
            {
                "id": "productid.1baa20a4f28a9f808c9694f1941f772b",
                "name": "By. Ott Cotes De Provence",
                "gtin": "3383693000012",
                "image": "https://static.euw1.sustained.app/products/images/3383693000012_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1baa20a4f28a9f808c9694f1941f772b",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1baa20a4f28a9f808c9694f1941f772b/impacts"
                }
            },
            {
                "id": "productid.1d8302520fc470edc8818fe8340a2336",
                "name": "M&S Delacourt Medium Dry Champagne",
                "gtin": "0000000140515",
                "image": "https://static.euw1.sustained.app/products/images/0000000140515_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 19.291999921716172
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1d8302520fc470edc8818fe8340a2336",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1d8302520fc470edc8818fe8340a2336/impacts"
                }
            },
            {
                "id": "productid.1e227b60bee748f7131379edd0a8250d",
                "name": "Sartori Pinot Grigio Delle Venezie Doc Blush",
                "gtin": "8005390045114",
                "image": "https://static.euw1.sustained.app/products/images/8005390045114_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e227b60bee748f7131379edd0a8250d",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e227b60bee748f7131379edd0a8250d/impacts"
                }
            },
            {
                "id": "productid.24e380dedd14c51d15027fc5a1bbc94a",
                "name": "Esprit Provence",
                "gtin": "3181448210019",
                "image": "https://static.euw1.sustained.app/products/images/3181448210019_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.24e380dedd14c51d15027fc5a1bbc94a",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.24e380dedd14c51d15027fc5a1bbc94a/impacts"
                }
            },
            {
                "id": "productid.26347d1b9e4d59008c0f2f411f2fb7d5",
                "name": "Claude Carr\u00e9 et Fils Champagne Brut",
                "gtin": "3760017730891",
                "image": "https://static.euw1.sustained.app/products/images/3760017730891_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 19.291999921716172
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.26347d1b9e4d59008c0f2f411f2fb7d5",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.26347d1b9e4d59008c0f2f411f2fb7d5/impacts"
                }
            },
            {
                "id": "productid.287c3b0f6c1b4145bde40cb806e09cde",
                "name": "M&S Spanish Rose",
                "gtin": "0000029056194",
                "image": "https://static.euw1.sustained.app/products/images/0000029056194_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.287c3b0f6c1b4145bde40cb806e09cde",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.287c3b0f6c1b4145bde40cb806e09cde/impacts"
                }
            },
            {
                "id": "productid.288a930c6d3543c8bcf4c299385619ae",
                "name": "Piper-Heidsieck Champagne Ros\u00e9 Sauvage",
                "gtin": "3018336150006",
                "image": "https://static.euw1.sustained.app/products/images/3018336150006_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 19.291999921716172
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.288a930c6d3543c8bcf4c299385619ae",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.288a930c6d3543c8bcf4c299385619ae/impacts"
                }
            },
            {
                "id": "productid.2e4bd9a2d7a4515f410212f88663bbae",
                "name": "Invivo X Sarah Jessica Parker",
                "gtin": "9421905673105",
                "image": "https://static.euw1.sustained.app/products/images/9421905673105_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2e4bd9a2d7a4515f410212f88663bbae",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2e4bd9a2d7a4515f410212f88663bbae/impacts"
                }
            },
            {
                "id": "productid.2f5c6ad80f7751e7b7c024df9ebb26ed",
                "name": "The Modern Cocktail Champagne Toppers Make Your Own Cocktail Kit",
                "gtin": "0732346277500",
                "image": "https://static.euw1.sustained.app/products/images/0732346277500_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.291999921716172
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2f5c6ad80f7751e7b7c024df9ebb26ed",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2f5c6ad80f7751e7b7c024df9ebb26ed/impacts"
                }
            },
            {
                "id": "productid.323646580e3239108cfcb39e543ee7de",
                "name": "Perrier-Jou\u00ebt Belle Epoque Ros\u00e9 Vintage Champagne",
                "gtin": "3113880104519",
                "image": "https://static.euw1.sustained.app/products/images/3113880104519_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 19.291999921716172
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.323646580e3239108cfcb39e543ee7de",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.323646580e3239108cfcb39e543ee7de/impacts"
                }
            },
            {
                "id": "productid.331e7ded6b03a16fab02295b86215f7b",
                "name": "Bruce Jack Sauvignon Blush",
                "gtin": "6009802502020",
                "image": "https://static.euw1.sustained.app/products/images/6009802502020_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.331e7ded6b03a16fab02295b86215f7b",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.331e7ded6b03a16fab02295b86215f7b/impacts"
                }
            },
            {
                "id": "productid.3617f72e8b3a6190d66ff2c34ae3bf4c",
                "name": "Terre des Oliviers C\u00f4tes de Provence",
                "gtin": "3309200402346",
                "image": "https://static.euw1.sustained.app/products/images/3309200402346_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3617f72e8b3a6190d66ff2c34ae3bf4c",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3617f72e8b3a6190d66ff2c34ae3bf4c/impacts"
                }
            },
            {
                "id": "productid.386984e809127fdc980f0785b4d83b0f",
                "name": "Whispering Angel C\u00f4tes de Provence Ros\u00e9",
                "gtin": "3760167974718",
                "image": "https://static.euw1.sustained.app/products/images/3760167974718_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.386984e809127fdc980f0785b4d83b0f",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.386984e809127fdc980f0785b4d83b0f/impacts"
                }
            },
            {
                "id": "productid.3b0c80f1878a4048374b7dfa4619e9ab",
                "name": "Lanson Champagne Green Label Brut",
                "gtin": "3029440006356",
                "image": "https://static.euw1.sustained.app/products/images/3029440006356_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 19.291999921716172
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3b0c80f1878a4048374b7dfa4619e9ab",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3b0c80f1878a4048374b7dfa4619e9ab/impacts"
                }
            },
            {
                "id": "productid.3e12b110b381a0ff0fc813db4c7935c3",
                "name": "M&S Delacourt Champagne",
                "gtin": "0000000140614",
                "image": "https://static.euw1.sustained.app/products/images/0000000140614_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 19.291999921716172
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3e12b110b381a0ff0fc813db4c7935c3",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3e12b110b381a0ff0fc813db4c7935c3/impacts"
                }
            },
            {
                "id": "productid.3f887911939dd228e98e857b5a34e2bd",
                "name": "La Petite Laurette Du Midi Ros\u00e9",
                "gtin": "3257694002101",
                "image": "https://static.euw1.sustained.app/products/images/3257694002101_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3f887911939dd228e98e857b5a34e2bd",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3f887911939dd228e98e857b5a34e2bd/impacts"
                }
            },
            {
                "id": "productid.434b5b704953bca7b6889f5307202377",
                "name": "La Deliziosa Pinot Grigio Ros\u00e9",
                "gtin": "5011932011311",
                "image": "https://static.euw1.sustained.app/products/images/5011932011311_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.434b5b704953bca7b6889f5307202377",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.434b5b704953bca7b6889f5307202377/impacts"
                }
            },
            {
                "id": "productid.4610324ca719bf3d2117cc5d11c12579",
                "name": "The Uncommon Bubbly Ros\u00e9 Pinot Noir",
                "gtin": "0715235923733",
                "image": "https://static.euw1.sustained.app/products/images/0715235923733_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4610324ca719bf3d2117cc5d11c12579",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4610324ca719bf3d2117cc5d11c12579/impacts"
                }
            },
            {
                "id": "productid.462601ab677962522089d8d037bc90fa",
                "name": "Tesco Portuguese Ros\u00e9",
                "gtin": "5054775859481",
                "image": "https://static.euw1.sustained.app/products/images/5054775859481_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.462601ab677962522089d8d037bc90fa",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.462601ab677962522089d8d037bc90fa/impacts"
                }
            },
            {
                "id": "productid.4666d52fbdb2772522dcdc02cdbea087",
                "name": "The Ned Ros\u00e9",
                "gtin": "9421901182595",
                "image": "https://static.euw1.sustained.app/products/images/9421901182595_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4666d52fbdb2772522dcdc02cdbea087",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4666d52fbdb2772522dcdc02cdbea087/impacts"
                }
            },
            {
                "id": "productid.467620b94475b03a9480df0363f7a0fc",
                "name": "Lady A La Coste",
                "gtin": "3701215700248",
                "image": "https://static.euw1.sustained.app/products/images/3701215700248_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.467620b94475b03a9480df0363f7a0fc",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.467620b94475b03a9480df0363f7a0fc/impacts"
                }
            },
            {
                "id": "productid.469e0ddde5608fa76e302d7ee76af95e",
                "name": "Via Vincini Italian Blush Rubicone",
                "gtin": "5052898009561",
                "image": "https://static.euw1.sustained.app/products/images/5052898009561_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.469e0ddde5608fa76e302d7ee76af95e",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.469e0ddde5608fa76e302d7ee76af95e/impacts"
                }
            },
            {
                "id": "productid.4856659544aa1ebf2bca1383996b6ddd",
                "name": "Champagne Gartissier Brut",
                "gtin": "5060355570127",
                "image": "https://static.euw1.sustained.app/products/images/5060355570127_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.983874063521668
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4856659544aa1ebf2bca1383996b6ddd",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4856659544aa1ebf2bca1383996b6ddd/impacts"
                }
            },
            {
                "id": "productid.49667e6097a981c94b4bc102125eb775",
                "name": "Veuve Clicquot Yellow Label Brut Champagne",
                "gtin": "3049614182952",
                "image": "https://static.euw1.sustained.app/products/images/3049614182952_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 19.291999921716172
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.49667e6097a981c94b4bc102125eb775",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.49667e6097a981c94b4bc102125eb775/impacts"
                }
            },
            {
                "id": "productid.498d7c4c892a5d9b06eb5fca31a9486a",
                "name": "M&S Delacourt Champagne Cuv\u00e9e Ros\u00e9 Brut",
                "gtin": "0000000140522",
                "image": "https://static.euw1.sustained.app/products/images/0000000140522_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 19.291999921716172
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.498d7c4c892a5d9b06eb5fca31a9486a",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.498d7c4c892a5d9b06eb5fca31a9486a/impacts"
                }
            },
            {
                "id": "productid.4a8b94d54001b66f66a5ae887a6b187a",
                "name": "Freixenet Premium - Cava Ros\u00e9",
                "gtin": "8410036801212",
                "image": "https://static.euw1.sustained.app/products/images/8410036801212_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 18.09800542750526
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4a8b94d54001b66f66a5ae887a6b187a",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4a8b94d54001b66f66a5ae887a6b187a/impacts"
                }
            },
            {
                "id": "productid.4f4b39ec5a902e365ddb84ff7bd6aed5",
                "name": "Pommery Champagne Brut Royal",
                "gtin": "3352379006365",
                "image": "https://static.euw1.sustained.app/products/images/3352379006365_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 19.291999921716172
                },
                "info_icons": [],
                "category": "Champagne & Rose",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4f4b39ec5a902e365ddb84ff7bd6aed5",
                    "category": "https://api.sustained.com/choice/v1/categories/champagne-and-rose",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4f4b39ec5a902e365ddb84ff7bd6aed5/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.0029c960f2db8e1a9532b6fad9993d71",
                "name": "Mature Cheddar",
                "gtin": "5000165369078",
                "image": "https://static.euw1.sustained.app/products/images/5000165369078_0.jpg",
                "pack": "550g",
                "grade": "F",
                "points": {
                    "pef": 52.825800840049524
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0029c960f2db8e1a9532b6fad9993d71",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0029c960f2db8e1a9532b6fad9993d71/impacts"
                }
            },
            {
                "id": "productid.0033d5372652ceb3f650228501ddc2d6",
                "name": "Extra Special Truffle Brie",
                "gtin": "5057172781997",
                "image": "https://static.euw1.sustained.app/products/images/5057172781997_0.jpg",
                "pack": "130g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0033d5372652ceb3f650228501ddc2d6",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0033d5372652ceb3f650228501ddc2d6/impacts"
                }
            },
            {
                "id": "productid.00368119f199d1adc60e3a9b9a21b8c5",
                "name": "Natural Cottage Cheese",
                "gtin": "01862911",
                "image": "https://static.euw1.sustained.app/products/images/01862911_0.jpg",
                "pack": "600g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00368119f199d1adc60e3a9b9a21b8c5",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00368119f199d1adc60e3a9b9a21b8c5/impacts"
                }
            },
            {
                "id": "productid.0038c5473395f5ec431e9245cea79018",
                "name": "Cheese Selection",
                "gtin": "5014442007546",
                "image": "https://static.euw1.sustained.app/products/images/5014442007546_0.jpg",
                "pack": "230g",
                "grade": "E",
                "points": {
                    "pef": 49.17499672649642
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0038c5473395f5ec431e9245cea79018",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0038c5473395f5ec431e9245cea79018/impacts"
                }
            },
            {
                "id": "productid.0039d6321a6a224db4c7f0e9e3ee67c0",
                "name": "Beechwood Smoked Cheese Slices",
                "gtin": "5057008655577",
                "image": "https://static.euw1.sustained.app/products/images/5057008655577_0.jpg",
                "pack": "250g",
                "grade": "F",
                "points": {
                    "pef": 52.65835178117945
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0039d6321a6a224db4c7f0e9e3ee67c0",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0039d6321a6a224db4c7f0e9e3ee67c0/impacts"
                }
            },
            {
                "id": "productid.00490ed061621b20b3e258f01bcd2152",
                "name": "Welsh Caerphilly Deli",
                "gtin": "0214190000000",
                "image": "https://static.euw1.sustained.app/products/images/0214190000000_0.jpg",
                "pack": "1.1kg",
                "grade": "E",
                "points": {
                    "pef": 49.17499672649642
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00490ed061621b20b3e258f01bcd2152",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00490ed061621b20b3e258f01bcd2152/impacts"
                }
            },
            {
                "id": "productid.00676a07bcaf237cf3c7a2240fa7636f",
                "name": "Vintage Gouda",
                "gtin": "5711953132247",
                "image": "https://static.euw1.sustained.app/products/images/5711953132247_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 52.82580084004952
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00676a07bcaf237cf3c7a2240fa7636f",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00676a07bcaf237cf3c7a2240fa7636f/impacts"
                }
            },
            {
                "id": "productid.015f686b9c1f1aaef276fb078c9518c9",
                "name": "Mature Cheddar",
                "gtin": "5010525216645",
                "image": "https://static.euw1.sustained.app/products/images/5010525216645_0.jpg",
                "pack": "240g",
                "grade": "F",
                "points": {
                    "pef": 56.19076268100383
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.015f686b9c1f1aaef276fb078c9518c9",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.015f686b9c1f1aaef276fb078c9518c9/impacts"
                }
            },
            {
                "id": "productid.01b6b16d0da789ab565d449b6a25ab96",
                "name": "Red Leicester",
                "gtin": "5012179006306",
                "image": "https://static.euw1.sustained.app/products/images/5012179006306_0.jpg",
                "pack": "350g",
                "grade": "F",
                "points": {
                    "pef": 69.23580366731609
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01b6b16d0da789ab565d449b6a25ab96",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01b6b16d0da789ab565d449b6a25ab96/impacts"
                }
            },
            {
                "id": "productid.022d145df6f69f95f8cd7dcc05b966fd",
                "name": "Cricket St Thomas Camembert",
                "gtin": "5017838822969",
                "image": "https://static.euw1.sustained.app/products/images/5017838822969_0.jpg",
                "pack": "220g",
                "grade": "E",
                "points": {
                    "pef": 49.17499672649641
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.022d145df6f69f95f8cd7dcc05b966fd",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.022d145df6f69f95f8cd7dcc05b966fd/impacts"
                }
            },
            {
                "id": "productid.02716250affb528ba1809b5ce0a2202c",
                "name": "Mature Scottish Coloured Cheddar",
                "gtin": "5054070903810",
                "image": "https://static.euw1.sustained.app/products/images/5054070903810_0.jpg",
                "pack": "350g",
                "grade": "F",
                "points": {
                    "pef": 52.825800840049524
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02716250affb528ba1809b5ce0a2202c",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02716250affb528ba1809b5ce0a2202c/impacts"
                }
            },
            {
                "id": "productid.02fb01ba9623e2c0fbd049cfbc93e862",
                "name": "Mature Cheddar",
                "gtin": "5011255291674",
                "image": "https://static.euw1.sustained.app/products/images/5011255291674_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 52.825800840049524
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02fb01ba9623e2c0fbd049cfbc93e862",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02fb01ba9623e2c0fbd049cfbc93e862/impacts"
                }
            },
            {
                "id": "productid.034458c05085b78990092a4f6da1b4f1",
                "name": "N\u00ado1 Vintage Smoke Red Fox",
                "gtin": "5000169616451",
                "image": "https://static.euw1.sustained.app/products/images/5000169616451_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.034458c05085b78990092a4f6da1b4f1",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.034458c05085b78990092a4f6da1b4f1/impacts"
                }
            },
            {
                "id": "productid.034e6de9ce2b0b718d5e7298957cb024",
                "name": "Original Soft Cheese",
                "gtin": "7622201694081",
                "image": "https://static.euw1.sustained.app/products/images/7622201694081_0.jpg",
                "pack": "260g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.034e6de9ce2b0b718d5e7298957cb024",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.034e6de9ce2b0b718d5e7298957cb024/impacts"
                }
            },
            {
                "id": "productid.044614a2fab930740be009dd024fe1b0",
                "name": "Parmigiano Reggiano",
                "gtin": "8010721997048",
                "image": "https://static.euw1.sustained.app/products/images/8010721997048_0.jpg",
                "pack": "150g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.044614a2fab930740be009dd024fe1b0",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.044614a2fab930740be009dd024fe1b0/impacts"
                }
            },
            {
                "id": "productid.04be766a1edbccee72ff631aa455a266",
                "name": "Essential Waitrose & Partners British Cheddar",
                "gtin": "5000169342701",
                "image": "https://static.euw1.sustained.app/products/images/5000169342701_0.jpg",
                "pack": "350g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04be766a1edbccee72ff631aa455a266",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04be766a1edbccee72ff631aa455a266/impacts"
                }
            },
            {
                "id": "productid.04c594a48b776375e064f818379d1296",
                "name": "Cheese Selection",
                "gtin": "5011714405406",
                "image": "https://static.euw1.sustained.app/products/images/5011714405406_0.jpg",
                "pack": "485g",
                "grade": "E",
                "points": {
                    "pef": 49.17499672649642
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04c594a48b776375e064f818379d1296",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04c594a48b776375e064f818379d1296/impacts"
                }
            },
            {
                "id": "productid.05915e349f95f59491e799f280c7fa81",
                "name": "Essential Waitrose & Partners Parmigiano Reggiano DOP",
                "gtin": "5000169157404",
                "image": "https://static.euw1.sustained.app/products/images/5000169157404_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05915e349f95f59491e799f280c7fa81",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05915e349f95f59491e799f280c7fa81/impacts"
                }
            },
            {
                "id": "productid.06182563f1090e5df0ff9fc4bdf7c7f3",
                "name": "Creamfields Mild Coloured Cheddar",
                "gtin": "0224200000000",
                "image": "https://static.euw1.sustained.app/products/images/0224200000000_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 49.17499672649642
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06182563f1090e5df0ff9fc4bdf7c7f3",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06182563f1090e5df0ff9fc4bdf7c7f3/impacts"
                }
            },
            {
                "id": "productid.06bb9b1375630d1464005469f9540741",
                "name": "Essential Waitrose English Mild Grated Cheddar",
                "gtin": "5000169053980",
                "image": "https://static.euw1.sustained.app/products/images/5000169053980_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06bb9b1375630d1464005469f9540741",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06bb9b1375630d1464005469f9540741/impacts"
                }
            },
            {
                "id": "productid.06cb20bddc2589de2178eb9ca8934bb7",
                "name": "JuraFlore Comte 12 Month",
                "gtin": "3542864942111",
                "image": "https://static.euw1.sustained.app/products/images/3542864942111_0.jpg",
                "pack": "200g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06cb20bddc2589de2178eb9ca8934bb7",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06cb20bddc2589de2178eb9ca8934bb7/impacts"
                }
            },
            {
                "id": "productid.06d3b7cc6464da46a5eb3e6f2486512b",
                "name": "Epic Mature Cheddar Flavour Block Vegan Alternative to Cheese",
                "gtin": "5202390021640",
                "image": "https://static.euw1.sustained.app/products/images/5202390021640_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06d3b7cc6464da46a5eb3e6f2486512b",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06d3b7cc6464da46a5eb3e6f2486512b/impacts"
                }
            },
            {
                "id": "productid.07f7758f40c06259dfb69a817b4da53e",
                "name": "Our Extra Mature Cheddar",
                "gtin": "5000295141599",
                "image": "https://static.euw1.sustained.app/products/images/5000295141599_0.jpg",
                "pack": "350g",
                "grade": "F",
                "points": {
                    "pef": 56.47660495360264
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07f7758f40c06259dfb69a817b4da53e",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07f7758f40c06259dfb69a817b4da53e/impacts"
                }
            },
            {
                "id": "productid.087cf908a1667dfe28c4d201ddcd6914",
                "name": "Austrian Smoked Cheese",
                "gtin": "09016927",
                "image": "https://static.euw1.sustained.app/products/images/09016927_0.jpg",
                "pack": "125g",
                "grade": "F",
                "points": {
                    "pef": 60.23660109633931
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.087cf908a1667dfe28c4d201ddcd6914",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.087cf908a1667dfe28c4d201ddcd6914/impacts"
                }
            },
            {
                "id": "productid.08d807da730c5a21ea934223c8140888",
                "name": "Swiss Le Gruy\u00e8re",
                "gtin": "0000000812481",
                "image": "https://static.euw1.sustained.app/products/images/0000000812481_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 89.29661060813575
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08d807da730c5a21ea934223c8140888",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08d807da730c5a21ea934223c8140888/impacts"
                }
            },
            {
                "id": "productid.090e1e2f564c79b0231544f774dc4c28",
                "name": "Christmas Cheddar with Caramelised Onion Chutney",
                "gtin": "5000169612811",
                "image": "https://static.euw1.sustained.app/products/images/5000169612811_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.090e1e2f564c79b0231544f774dc4c28",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.090e1e2f564c79b0231544f774dc4c28/impacts"
                }
            },
            {
                "id": "productid.09dceeebb49d701f467d75e1816edb80",
                "name": "Somerset Nibbler Vintage Cheddar",
                "gtin": "0267420000007",
                "image": "https://static.euw1.sustained.app/products/images/0267420000007_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 56.47660495360264
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09dceeebb49d701f467d75e1816edb80",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09dceeebb49d701f467d75e1816edb80/impacts"
                }
            },
            {
                "id": "productid.0a3800b6595212005820cc57fe61f200",
                "name": "Soft Goats' Cheese",
                "gtin": "5000462346611",
                "image": "https://static.euw1.sustained.app/products/images/5000462346611_0.jpg",
                "pack": "125g",
                "grade": "E",
                "points": {
                    "pef": 49.17499672649643
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a3800b6595212005820cc57fe61f200",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a3800b6595212005820cc57fe61f200/impacts"
                }
            },
            {
                "id": "productid.0a4cccd920977da3664b593982d1ddc0",
                "name": "Cr\u00e8me de Brie Cheese Spread",
                "gtin": "3228024070282",
                "image": "https://static.euw1.sustained.app/products/images/3228024070282_0.jpg",
                "pack": "125g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a4cccd920977da3664b593982d1ddc0",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a4cccd920977da3664b593982d1ddc0/impacts"
                }
            },
            {
                "id": "productid.0adda62db9b01500867315ab48214cea",
                "name": "Sliced Mozzarella",
                "gtin": "5054781047742",
                "image": "https://static.euw1.sustained.app/products/images/5054781047742_0.jpg",
                "pack": "180g",
                "grade": "E",
                "points": {
                    "pef": 49.174996726496424
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0adda62db9b01500867315ab48214cea",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0adda62db9b01500867315ab48214cea/impacts"
                }
            },
            {
                "id": "productid.0b2a81ed3908fab06ffa723a9848976f",
                "name": "French Brie",
                "gtin": "5057753930370",
                "image": "https://static.euw1.sustained.app/products/images/5057753930370_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 49.17499672649641
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b2a81ed3908fab06ffa723a9848976f",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b2a81ed3908fab06ffa723a9848976f/impacts"
                }
            },
            {
                "id": "productid.0b56a549978a1f48868312244a383ed2",
                "name": "Greenside Deli Mature Coloured Cheddar",
                "gtin": "5010525216577",
                "image": "https://static.euw1.sustained.app/products/images/5010525216577_0.jpg",
                "pack": "625g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b56a549978a1f48868312244a383ed2",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b56a549978a1f48868312244a383ed2/impacts"
                }
            },
            {
                "id": "productid.0bd300f910a60e0619fa0e4415069873",
                "name": "Yorkshire Cream Cheese Reduced fat",
                "gtin": "5010578012195",
                "image": "https://static.euw1.sustained.app/products/images/5010578012195_0.jpg",
                "pack": "200g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bd300f910a60e0619fa0e4415069873",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bd300f910a60e0619fa0e4415069873/impacts"
                }
            },
            {
                "id": "productid.0c86eb18d5825025797c912660e5a362",
                "name": "Smartprice Mature White Cheddar",
                "gtin": "5057172111404",
                "image": "https://static.euw1.sustained.app/products/images/5057172111404_0.jpg",
                "pack": "400g",
                "grade": "F",
                "points": {
                    "pef": 52.825800840049524
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c86eb18d5825025797c912660e5a362",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c86eb18d5825025797c912660e5a362/impacts"
                }
            },
            {
                "id": "productid.0c932c8b64899a1446123ef35fc687d9",
                "name": "50% Lighter Cracked Black Pepper Soft Cheese",
                "gtin": "5010251907534",
                "image": "https://static.euw1.sustained.app/products/images/5010251907534_0.jpg",
                "pack": "250g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c932c8b64899a1446123ef35fc687d9",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c932c8b64899a1446123ef35fc687d9/impacts"
                }
            },
            {
                "id": "productid.0caf37e23f2f32a2cd7184d1cf162068",
                "name": "The Family Dairy Cottage Cheese Natural",
                "gtin": "5025840005195",
                "image": "https://static.euw1.sustained.app/products/images/5025840005195_0.jpg",
                "pack": "300g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0caf37e23f2f32a2cd7184d1cf162068",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0caf37e23f2f32a2cd7184d1cf162068/impacts"
                }
            },
            {
                "id": "productid.0ced9cff8fb2a8ce6883a95b6505c50a",
                "name": "Chosen by You 50% Less Fat Sweet Chilli Light Soft Cheese",
                "gtin": "5054070693827",
                "image": "https://static.euw1.sustained.app/products/images/5054070693827_0.jpg",
                "pack": "300g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ced9cff8fb2a8ce6883a95b6505c50a",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ced9cff8fb2a8ce6883a95b6505c50a/impacts"
                }
            },
            {
                "id": "productid.0db6fe632e28cddda6aee2f7564b5297",
                "name": "Light Soft Cheese",
                "gtin": "7622210585943",
                "image": "https://static.euw1.sustained.app/products/images/7622210585943_0.jpg",
                "pack": "340g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0db6fe632e28cddda6aee2f7564b5297",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0db6fe632e28cddda6aee2f7564b5297/impacts"
                }
            },
            {
                "id": "productid.0dc4319897823c9e9fe4a6cccbde5aa4",
                "name": "M&S Ricotta",
                "gtin": "0000000636155",
                "image": "https://static.euw1.sustained.app/products/images/0000000636155_0.jpg",
                "pack": "250g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0dc4319897823c9e9fe4a6cccbde5aa4",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0dc4319897823c9e9fe4a6cccbde5aa4/impacts"
                }
            },
            {
                "id": "productid.0df17e184e44f892c8a41ecccf98dd74",
                "name": "Original Cheese Block",
                "gtin": "3073780997027",
                "image": "https://static.euw1.sustained.app/products/images/3073780997027_0.jpg",
                "pack": "225g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0df17e184e44f892c8a41ecccf98dd74",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0df17e184e44f892c8a41ecccf98dd74/impacts"
                }
            },
            {
                "id": "productid.0e022998efbd2cda969f58efb9c319c7",
                "name": "Wensleydale with Cranberries",
                "gtin": "03062203",
                "image": "https://static.euw1.sustained.app/products/images/03062203_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 49.17499672649642
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e022998efbd2cda969f58efb9c319c7",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e022998efbd2cda969f58efb9c319c7/impacts"
                }
            },
            {
                "id": "productid.0e58de0fd3ed939fcd2a763447c388bd",
                "name": "Cheese Spread Triangles Mix Flavours",
                "gtin": "5902899101392",
                "image": "https://static.euw1.sustained.app/products/images/5902899101392_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 49.17499672649642
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e58de0fd3ed939fcd2a763447c388bd",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e58de0fd3ed939fcd2a763447c388bd/impacts"
                }
            },
            {
                "id": "productid.0ec6c72f81a9844000c81e2107f8b693",
                "name": "No1 Extra Mature Wookey Hole Cheddar Clothbound",
                "gtin": "5000169520857",
                "image": "https://static.euw1.sustained.app/products/images/5000169520857_0.jpg",
                "pack": "600g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ec6c72f81a9844000c81e2107f8b693",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ec6c72f81a9844000c81e2107f8b693/impacts"
                }
            },
            {
                "id": "productid.0f5ae37849c89fb7a58b57ce400152b4",
                "name": "Chosen by You Extra Mature Welsh Cheddar",
                "gtin": "5054070606919",
                "image": "https://static.euw1.sustained.app/products/images/5054070606919_0.jpg",
                "pack": "250g",
                "grade": "F",
                "points": {
                    "pef": 52.825800840049524
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f5ae37849c89fb7a58b57ce400152b4",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f5ae37849c89fb7a58b57ce400152b4/impacts"
                }
            },
            {
                "id": "productid.0fae69d3e8006212b7c5d33eced6b9fe",
                "name": "Medium Scottish Cheddar",
                "gtin": "5017838810164",
                "image": "https://static.euw1.sustained.app/products/images/5017838810164_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fae69d3e8006212b7c5d33eced6b9fe",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fae69d3e8006212b7c5d33eced6b9fe/impacts"
                }
            },
            {
                "id": "productid.110718a30bf894d52cf06ede75060e05",
                "name": "Chives Soft Cheese",
                "gtin": "7622201693831",
                "image": "https://static.euw1.sustained.app/products/images/7622201693831_0.jpg",
                "pack": "165g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.110718a30bf894d52cf06ede75060e05",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.110718a30bf894d52cf06ede75060e05/impacts"
                }
            },
            {
                "id": "productid.110a7253b071e6cd3d80f00da91f3831",
                "name": "Grated Italian Style Cheese",
                "gtin": "5010251460473",
                "image": "https://static.euw1.sustained.app/products/images/5010251460473_0.jpg",
                "pack": "100g",
                "grade": "E",
                "points": {
                    "pef": 49.17499672649642
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.110a7253b071e6cd3d80f00da91f3831",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.110a7253b071e6cd3d80f00da91f3831/impacts"
                }
            },
            {
                "id": "productid.1291e7d129d7dc12331deea85990afba",
                "name": "10 Gouda Slices Traditional Cheese",
                "gtin": "0000000728584",
                "image": "https://static.euw1.sustained.app/products/images/0000000728584_0.jpg",
                "pack": "250g",
                "grade": "E",
                "points": {
                    "pef": 49.17499672649642
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1291e7d129d7dc12331deea85990afba",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1291e7d129d7dc12331deea85990afba/impacts"
                }
            },
            {
                "id": "productid.141f683d1ea48664d922fd994aeeb17a",
                "name": "Coloured Medium Scottish Island Cheddar",
                "gtin": "5017838812762",
                "image": "https://static.euw1.sustained.app/products/images/5017838812762_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.141f683d1ea48664d922fd994aeeb17a",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.141f683d1ea48664d922fd994aeeb17a/impacts"
                }
            },
            {
                "id": "productid.1563688d659bbe9b82b3de58b8c30920",
                "name": "Parmigiano Reggiano",
                "gtin": "0000000562003",
                "image": "https://static.euw1.sustained.app/products/images/0000000562003_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 89.29661060813575
                },
                "info_icons": [],
                "category": "Cheese",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1563688d659bbe9b82b3de58b8c30920",
                    "category": "https://api.sustained.com/choice/v1/categories/cheese",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1563688d659bbe9b82b3de58b8c30920/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.01c32fda5ebb4a2591cfd05adf90ae32",
                "name": "Food Collection British Free Range Pembrokeshire Bronze Whole Turkey X-Large",
                "gtin": "00228312",
                "image": "https://static.euw1.sustained.app/products/images/00228312_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01c32fda5ebb4a2591cfd05adf90ae32",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01c32fda5ebb4a2591cfd05adf90ae32/impacts"
                }
            },
            {
                "id": "productid.03a242f81cb6f24bae8d903722b9cad9",
                "name": "Fresh Boneless & Skinless Chicken Thighs",
                "gtin": "0745114579386",
                "image": "https://static.euw1.sustained.app/products/images/0745114579386_0.jpg",
                "pack": "1kg",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03a242f81cb6f24bae8d903722b9cad9",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03a242f81cb6f24bae8d903722b9cad9/impacts"
                }
            },
            {
                "id": "productid.0437338a2d23d1fa68d88feb176768cd",
                "name": "Global Grill Hot Wings",
                "gtin": "5010525167527",
                "image": "https://static.euw1.sustained.app/products/images/5010525167527_0.jpg",
                "pack": "900g",
                "grade": "F",
                "points": {
                    "pef": 64.29374001952611
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0437338a2d23d1fa68d88feb176768cd",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0437338a2d23d1fa68d88feb176768cd/impacts"
                }
            },
            {
                "id": "productid.0550b7b093e8a3ab096248a96a06217a",
                "name": "Smoky & Spicy Chicken Breast Slices",
                "gtin": "5057172725557",
                "image": "https://static.euw1.sustained.app/products/images/5057172725557_0.jpg",
                "pack": "180g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0550b7b093e8a3ab096248a96a06217a",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0550b7b093e8a3ab096248a96a06217a/impacts"
                }
            },
            {
                "id": "productid.0809b21ee30a33ea2a0f5d26f7754d95",
                "name": "Simply Chicken Tenders",
                "gtin": "5011312000652",
                "image": "https://static.euw1.sustained.app/products/images/5011312000652_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0809b21ee30a33ea2a0f5d26f7754d95",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0809b21ee30a33ea2a0f5d26f7754d95/impacts"
                }
            },
            {
                "id": "productid.082ae5833da40b9473c1a696a55f70a7",
                "name": "The Butcher's Selection at Asda Medium Whole Turkey",
                "gtin": "5054781258421",
                "image": "https://static.euw1.sustained.app/products/images/5054781258421_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.082ae5833da40b9473c1a696a55f70a7",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.082ae5833da40b9473c1a696a55f70a7/impacts"
                }
            },
            {
                "id": "productid.0a482222fb9a863d9f89c7cb8752eb65",
                "name": "Food Select Farms Southern Fried Chicken Burgers",
                "gtin": "0000029147168",
                "image": "https://static.euw1.sustained.app/products/images/0000029147168_0.jpg",
                "pack": "282g",
                "grade": "F",
                "points": {
                    "pef": 61.703469716913574
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a482222fb9a863d9f89c7cb8752eb65",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a482222fb9a863d9f89c7cb8752eb65/impacts"
                }
            },
            {
                "id": "productid.0ad2cf9d5d1f77f4185094e923a567ea",
                "name": "Finest Irish Corn Fed Chicken Wings and Drumsticks",
                "gtin": "5059697717918",
                "image": "https://static.euw1.sustained.app/products/images/5059697717918_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ad2cf9d5d1f77f4185094e923a567ea",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ad2cf9d5d1f77f4185094e923a567ea/impacts"
                }
            },
            {
                "id": "productid.0b81dcdb21d1fdd57e278ffe5ad42afe",
                "name": "Smoked Turkey Breast 8 Slices",
                "gtin": "7291000367702",
                "image": "https://static.euw1.sustained.app/products/images/7291000367702_0.jpg",
                "pack": "142g",
                "grade": "E",
                "points": {
                    "pef": 51.58003716584842
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b81dcdb21d1fdd57e278ffe5ad42afe",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b81dcdb21d1fdd57e278ffe5ad42afe/impacts"
                }
            },
            {
                "id": "productid.0bd3114dda4accfcf0077010d75d72fd",
                "name": "Chicken Mince",
                "gtin": "0237960000000",
                "image": "https://static.euw1.sustained.app/products/images/0237960000000_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bd3114dda4accfcf0077010d75d72fd",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bd3114dda4accfcf0077010d75d72fd/impacts"
                }
            },
            {
                "id": "productid.0cd606776dbb21005d0a3714e23326f6",
                "name": "Tender Chicken Breast Fillets",
                "gtin": "5054781380016",
                "image": "https://static.euw1.sustained.app/products/images/5054781380016_0.jpg",
                "pack": "1.600kg",
                "grade": "F",
                "points": {
                    "pef": 80.99455205651091
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0cd606776dbb21005d0a3714e23326f6",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0cd606776dbb21005d0a3714e23326f6/impacts"
                }
            },
            {
                "id": "productid.0da99e65776c1e08166525f25ca19238",
                "name": "British Basted Turkey Crown",
                "gtin": "0279390000000",
                "image": "https://static.euw1.sustained.app/products/images/0279390000000_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 62.66141036884122
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0da99e65776c1e08166525f25ca19238",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0da99e65776c1e08166525f25ca19238/impacts"
                }
            },
            {
                "id": "productid.12c31cce4875553691b01fab49320a3c",
                "name": "British Roast Chicken Wholebird",
                "gtin": "0230643000000",
                "image": "https://static.euw1.sustained.app/products/images/0230643000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12c31cce4875553691b01fab49320a3c",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12c31cce4875553691b01fab49320a3c/impacts"
                }
            },
            {
                "id": "productid.12e731370fe30fe922f04dbecd8d3c74",
                "name": "Traditional Irish Chicken",
                "gtin": "5099822241918",
                "image": "https://static.euw1.sustained.app/products/images/5099822241918_0.jpg",
                "pack": "1900g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12e731370fe30fe922f04dbecd8d3c74",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12e731370fe30fe922f04dbecd8d3c74/impacts"
                }
            },
            {
                "id": "productid.139d3ca528831396701981a0f073689d",
                "name": "Smoked Turkey Salami",
                "gtin": "7290000367774",
                "image": "https://static.euw1.sustained.app/products/images/7290000367774_0.jpg",
                "pack": "142g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.139d3ca528831396701981a0f073689d",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.139d3ca528831396701981a0f073689d/impacts"
                }
            },
            {
                "id": "productid.1470e5cbeb2697a16466826768248ef1",
                "name": "Essential Waitrose & Partners Higher Welfare British Chicken Breast Fillets",
                "gtin": "5000169572467",
                "image": "https://static.euw1.sustained.app/products/images/5000169572467_0.jpg",
                "pack": "600g",
                "grade": "F",
                "points": {
                    "pef": 80.99455205651091
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1470e5cbeb2697a16466826768248ef1",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1470e5cbeb2697a16466826768248ef1/impacts"
                }
            },
            {
                "id": "productid.1492c46985a1531181aa8546b2e4041b",
                "name": "SO Organic British Free Range Chicken Breast Mini Fillets",
                "gtin": "0226350000006",
                "image": "https://static.euw1.sustained.app/products/images/0226350000006_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1492c46985a1531181aa8546b2e4041b",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1492c46985a1531181aa8546b2e4041b/impacts"
                }
            },
            {
                "id": "productid.155f71b9bc69d0ffe70a9e1e5cc29d3f",
                "name": "Sensationally Seasoned Southern Fried Chicken Drumsticks & Thighs",
                "gtin": "5010069708255",
                "image": "https://static.euw1.sustained.app/products/images/5010069708255_0.jpg",
                "pack": "1.2kg",
                "grade": "F",
                "points": {
                    "pef": 59.284854719875945
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.155f71b9bc69d0ffe70a9e1e5cc29d3f",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.155f71b9bc69d0ffe70a9e1e5cc29d3f/impacts"
                }
            },
            {
                "id": "productid.16e0bd407631981dc5301329c576ecf0",
                "name": "British Chicken Thigh Fillets",
                "gtin": "00296717",
                "image": "https://static.euw1.sustained.app/products/images/00296717_0.jpg",
                "pack": "640g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16e0bd407631981dc5301329c576ecf0",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16e0bd407631981dc5301329c576ecf0/impacts"
                }
            },
            {
                "id": "productid.173805b2344cc5d413655bb1e0efe915",
                "name": "Essential Waitrose British Chicken Breast Chunks",
                "gtin": "5000169322871",
                "image": "https://static.euw1.sustained.app/products/images/5000169322871_0.jpg",
                "pack": "330g",
                "grade": "F",
                "points": {
                    "pef": 80.99455205651091
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.173805b2344cc5d413655bb1e0efe915",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.173805b2344cc5d413655bb1e0efe915/impacts"
                }
            },
            {
                "id": "productid.18a405c1cf41db51770023238d9e645a",
                "name": "British Chicken Herb Crumbed Breast Fillets",
                "gtin": "5000169472231",
                "image": "https://static.euw1.sustained.app/products/images/5000169472231_0.jpg",
                "pack": "240g",
                "grade": "F",
                "points": {
                    "pef": 61.97541720797297
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.18a405c1cf41db51770023238d9e645a",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.18a405c1cf41db51770023238d9e645a/impacts"
                }
            },
            {
                "id": "productid.1992142a8730040ca0f26a184481f325",
                "name": "Fresh Irish Chicken",
                "gtin": "5099822042294",
                "image": "https://static.euw1.sustained.app/products/images/5099822042294_0.jpg",
                "pack": "1900g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1992142a8730040ca0f26a184481f325",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1992142a8730040ca0f26a184481f325/impacts"
                }
            },
            {
                "id": "productid.1b17775fb4ba6f76fa73ea18560f7628",
                "name": "Essential Waitrose British Chicken Medium without Giblets",
                "gtin": "0282520000000",
                "image": "https://static.euw1.sustained.app/products/images/0282520000000_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b17775fb4ba6f76fa73ea18560f7628",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b17775fb4ba6f76fa73ea18560f7628/impacts"
                }
            },
            {
                "id": "productid.1b6382cc7b49fcc35acc646a2ff91311",
                "name": "Irish Small Whole Chicken",
                "gtin": "5054269893618",
                "image": "https://static.euw1.sustained.app/products/images/5054269893618_0.jpg",
                "pack": "1300g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b6382cc7b49fcc35acc646a2ff91311",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b6382cc7b49fcc35acc646a2ff91311/impacts"
                }
            },
            {
                "id": "productid.1b675871f5527adbfa2fde1d1d517524",
                "name": "Smokey BBQ Chicken Drumsticks",
                "gtin": "5016901007074",
                "image": "https://static.euw1.sustained.app/products/images/5016901007074_0.jpg",
                "pack": "900g",
                "grade": "F",
                "points": {
                    "pef": 57.990257059091206
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b675871f5527adbfa2fde1d1d517524",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b675871f5527adbfa2fde1d1d517524/impacts"
                }
            },
            {
                "id": "productid.1bf7a80f787ddfbcaa0fbd17a4d1229c",
                "name": "Your Butcher Chicken Bites",
                "gtin": "5060234452070",
                "image": "https://static.euw1.sustained.app/products/images/5060234452070_0.jpg",
                "pack": "400g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1bf7a80f787ddfbcaa0fbd17a4d1229c",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1bf7a80f787ddfbcaa0fbd17a4d1229c/impacts"
                }
            },
            {
                "id": "productid.1d3a6df5e14f351db9c8a07d6aa2fae8",
                "name": "Shazans Jerk Chicken",
                "gtin": "5060388318857",
                "image": "https://static.euw1.sustained.app/products/images/5060388318857_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 60.63039509445488
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1d3a6df5e14f351db9c8a07d6aa2fae8",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1d3a6df5e14f351db9c8a07d6aa2fae8/impacts"
                }
            },
            {
                "id": "productid.1d885f1f557af77db915df55e4ad16ba",
                "name": "Piri Piri Thighs",
                "gtin": "5012648001641",
                "image": "https://static.euw1.sustained.app/products/images/5012648001641_0.jpg",
                "pack": "1.1kg",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1d885f1f557af77db915df55e4ad16ba",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1d885f1f557af77db915df55e4ad16ba/impacts"
                }
            },
            {
                "id": "productid.1e2be7ffbb988a0f0f4be45fc82af6aa",
                "name": "Peri Peri Chicken Wings",
                "gtin": "5016901004134",
                "image": "https://static.euw1.sustained.app/products/images/5016901004134_0.jpg",
                "pack": "600g",
                "grade": "F",
                "points": {
                    "pef": 60.82647257129394
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e2be7ffbb988a0f0f4be45fc82af6aa",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e2be7ffbb988a0f0f4be45fc82af6aa/impacts"
                }
            },
            {
                "id": "productid.1e80a9ed579f4209a49a48e4a9f0a35e",
                "name": "Summer Eating British Roast Chicken Drumsticks",
                "gtin": "5000169336366",
                "image": "https://static.euw1.sustained.app/products/images/5000169336366_0.jpg",
                "pack": "350g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e80a9ed579f4209a49a48e4a9f0a35e",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e80a9ed579f4209a49a48e4a9f0a35e/impacts"
                }
            },
            {
                "id": "productid.1f0b3c264125aaa24925ce81946fc311",
                "name": "Organic 2 Free Range British Chicken Leg Portions",
                "gtin": "2333028000000",
                "image": "https://static.euw1.sustained.app/products/images/2333028000000_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1f0b3c264125aaa24925ce81946fc311",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1f0b3c264125aaa24925ce81946fc311/impacts"
                }
            },
            {
                "id": "productid.1f0dd7b19abc0a1e9fd5bbfa1f7145fd",
                "name": "Sweet Chilli Chicken Breast Fillets",
                "gtin": "5054269959406",
                "image": "https://static.euw1.sustained.app/products/images/5054269959406_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 80.99455205651091
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1f0dd7b19abc0a1e9fd5bbfa1f7145fd",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1f0dd7b19abc0a1e9fd5bbfa1f7145fd/impacts"
                }
            },
            {
                "id": "productid.207bc2069ef841f78bcfe8da1a6ba6d6",
                "name": "Select Farms British Oakham Turkey Breast Mince 2% Fat",
                "gtin": "0000000083218",
                "image": "https://static.euw1.sustained.app/products/images/0000000083218_0.jpg",
                "pack": "400g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.207bc2069ef841f78bcfe8da1a6ba6d6",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.207bc2069ef841f78bcfe8da1a6ba6d6/impacts"
                }
            },
            {
                "id": "productid.21db6d0a13adf0d8979e1af99ba01ccd",
                "name": "The Butcher's On Market Street British Chicken Chinese Style Drumsticks & Thighs",
                "gtin": "5010251986447",
                "image": "https://static.euw1.sustained.app/products/images/5010251986447_0.jpg",
                "pack": "800g",
                "grade": "F",
                "points": {
                    "pef": 63.366247928962025
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.21db6d0a13adf0d8979e1af99ba01ccd",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.21db6d0a13adf0d8979e1af99ba01ccd/impacts"
                }
            },
            {
                "id": "productid.22e8e161c0e3186e7c59ecdfaba91c36",
                "name": "Crispy Crunchy Coated Tender Breaded Chicken Mini Fillets",
                "gtin": "5010069795699",
                "image": "https://static.euw1.sustained.app/products/images/5010069795699_0.jpg",
                "pack": "300g",
                "grade": "E",
                "points": {
                    "pef": 48.46601084279351
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.22e8e161c0e3186e7c59ecdfaba91c36",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.22e8e161c0e3186e7c59ecdfaba91c36/impacts"
                }
            },
            {
                "id": "productid.2344cdb808419e45286a3a8bf76c065a",
                "name": "Roast Tikka Chicken Fillet",
                "gtin": "5391530931267",
                "image": "https://static.euw1.sustained.app/products/images/5391530931267_0.jpg",
                "pack": "1.000kg",
                "grade": "F",
                "points": {
                    "pef": 63.0494302926062
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2344cdb808419e45286a3a8bf76c065a",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2344cdb808419e45286a3a8bf76c065a/impacts"
                }
            },
            {
                "id": "productid.242fc66fc7e6e711cb3883d3176f155d",
                "name": "Chicken Legs",
                "gtin": "5010069684221",
                "image": "https://static.euw1.sustained.app/products/images/5010069684221_0.jpg",
                "pack": "1.1kg",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.242fc66fc7e6e711cb3883d3176f155d",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.242fc66fc7e6e711cb3883d3176f155d/impacts"
                }
            },
            {
                "id": "productid.26f2b401d745a580d7dcbe69c6d87106",
                "name": "Tandoori Chicken Drumsticks",
                "gtin": "5016901004141",
                "image": "https://static.euw1.sustained.app/products/images/5016901004141_0.jpg",
                "pack": "650g",
                "grade": "F",
                "points": {
                    "pef": 62.3885545220765
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.26f2b401d745a580d7dcbe69c6d87106",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.26f2b401d745a580d7dcbe69c6d87106/impacts"
                }
            },
            {
                "id": "productid.281fdb43a7b2dc1c67859e7f97ae633b",
                "name": "Food Select Farms British Stuffed Oakham Turkey Breast Joint Boneless Medium",
                "gtin": "0000029106844",
                "image": "https://static.euw1.sustained.app/products/images/0000029106844_0.jpg",
                "pack": "1.075kg",
                "grade": "F",
                "points": {
                    "pef": 74.00612365590244
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.281fdb43a7b2dc1c67859e7f97ae633b",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.281fdb43a7b2dc1c67859e7f97ae633b/impacts"
                }
            },
            {
                "id": "productid.2cfdb9a721560085a5ed84c03d1a9f7c",
                "name": "British Fresh Chicken Thigh Fillets",
                "gtin": "0000000296717",
                "image": "https://static.euw1.sustained.app/products/images/0000000296717_0.jpg",
                "pack": "640g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2cfdb9a721560085a5ed84c03d1a9f7c",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2cfdb9a721560085a5ed84c03d1a9f7c/impacts"
                }
            },
            {
                "id": "productid.2d715809b5b41f9564637b6d33e976c4",
                "name": "Fresh Irish Chicken Part Boned Breasts",
                "gtin": "5099822361449",
                "image": "https://static.euw1.sustained.app/products/images/5099822361449_0.jpg",
                "pack": "870g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2d715809b5b41f9564637b6d33e976c4",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2d715809b5b41f9564637b6d33e976c4/impacts"
                }
            },
            {
                "id": "productid.2fcc61166fa3f63ed9669616c1e76f75",
                "name": "Cooked Chicken Breast Slices",
                "gtin": "5016901006701",
                "image": "https://static.euw1.sustained.app/products/images/5016901006701_0.jpg",
                "pack": "150g",
                "grade": "F",
                "points": {
                    "pef": 79.13407752936
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2fcc61166fa3f63ed9669616c1e76f75",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2fcc61166fa3f63ed9669616c1e76f75/impacts"
                }
            },
            {
                "id": "productid.320374aff77affcef06bbc8bc0f3f0ff",
                "name": "Free Range Chicken Mini Breast Fillets",
                "gtin": "2068130000008",
                "image": "https://static.euw1.sustained.app/products/images/2068130000008_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.320374aff77affcef06bbc8bc0f3f0ff",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.320374aff77affcef06bbc8bc0f3f0ff/impacts"
                }
            },
            {
                "id": "productid.336bd006fdab256fa87c4d0d19b8c37d",
                "name": "1 Free Range Roast Chicken Leg Quarters",
                "gtin": "5000169529669",
                "image": "https://static.euw1.sustained.app/products/images/5000169529669_0.jpg",
                "pack": "300g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.336bd006fdab256fa87c4d0d19b8c37d",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.336bd006fdab256fa87c4d0d19b8c37d/impacts"
                }
            },
            {
                "id": "productid.3410bc24fdf720c3e75f56a819f274c0",
                "name": "Just Essentials Chicken Thigh Portions",
                "gtin": "5057172667895",
                "image": "https://static.euw1.sustained.app/products/images/5057172667895_0.jpg",
                "pack": "1.1kg",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3410bc24fdf720c3e75f56a819f274c0",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3410bc24fdf720c3e75f56a819f274c0/impacts"
                }
            },
            {
                "id": "productid.349be2f8c2a912a1668e57cf3060b64e",
                "name": "Chicken Thigh Fillets Skinless & Boneless",
                "gtin": "5057753896072",
                "image": "https://static.euw1.sustained.app/products/images/5057753896072_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.349be2f8c2a912a1668e57cf3060b64e",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.349be2f8c2a912a1668e57cf3060b64e/impacts"
                }
            },
            {
                "id": "productid.351857ae546b513daaf11d8f5efbd017",
                "name": "Smoky Barbecue Chicken Drumsticks",
                "gtin": "5000169127520",
                "image": "https://static.euw1.sustained.app/products/images/5000169127520_0.jpg",
                "pack": "740g",
                "grade": "F",
                "points": {
                    "pef": 60.87025568770205
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.351857ae546b513daaf11d8f5efbd017",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.351857ae546b513daaf11d8f5efbd017/impacts"
                }
            },
            {
                "id": "productid.352a34b98ccc32fc64ee4da319426e9d",
                "name": "Cumin & Coriander Chicken",
                "gtin": "5010069505953",
                "image": "https://static.euw1.sustained.app/products/images/5010069505953_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.352a34b98ccc32fc64ee4da319426e9d",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.352a34b98ccc32fc64ee4da319426e9d/impacts"
                }
            },
            {
                "id": "productid.3549ab90d838bc97994a3d2737dcdba4",
                "name": "Slow Cooked Chicken Crown with Gravy",
                "gtin": "5010251764908",
                "image": "https://static.euw1.sustained.app/products/images/5010251764908_0.jpg",
                "pack": "520g",
                "grade": "F",
                "points": {
                    "pef": 72.03947470969972
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3549ab90d838bc97994a3d2737dcdba4",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3549ab90d838bc97994a3d2737dcdba4/impacts"
                }
            },
            {
                "id": "productid.36059d4c34b025113bbc4d74e6d1ef82",
                "name": "Roast in the Bag Sage & Onion Stuffed Irish Whole Chicken",
                "gtin": "5054775227044",
                "image": "https://static.euw1.sustained.app/products/images/5054775227044_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 64.63642954821987
                },
                "info_icons": [],
                "category": "Chicken & Turkey",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.36059d4c34b025113bbc4d74e6d1ef82",
                    "category": "https://api.sustained.com/choice/v1/categories/chicken-and-turkey",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.36059d4c34b025113bbc4d74e6d1ef82/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.0004571e1edd0330af3d975ea73bfc71",
                "name": "Strawberry & Chocolate Snack Bars",
                "gtin": "5056150601296",
                "image": "https://static.euw1.sustained.app/products/images/5056150601296_0.jpg",
                "pack": "25g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0004571e1edd0330af3d975ea73bfc71",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0004571e1edd0330af3d975ea73bfc71/impacts"
                }
            },
            {
                "id": "productid.001f0d72c7beffe39107bd8b006fb0a7",
                "name": "Glacier Mints",
                "gtin": "5010511479078",
                "image": "https://static.euw1.sustained.app/products/images/5010511479078_0.jpg",
                "pack": "1.7kg",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.001f0d72c7beffe39107bd8b006fb0a7",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.001f0d72c7beffe39107bd8b006fb0a7/impacts"
                }
            },
            {
                "id": "productid.005afd930fc5705289b5c5051c90b077",
                "name": "Food Plant Kitchen Vegan Salted Caramel Fudge",
                "gtin": "0000029262434",
                "image": "https://static.euw1.sustained.app/products/images/0000029262434_0.jpg",
                "pack": "300g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.005afd930fc5705289b5c5051c90b077",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.005afd930fc5705289b5c5051c90b077/impacts"
                }
            },
            {
                "id": "productid.00794c8e94193c19ead0761a6e998297",
                "name": "Darkmilk Chocolate Bar",
                "gtin": "7622210890214",
                "image": "https://static.euw1.sustained.app/products/images/7622210890214_0.jpg",
                "pack": "85g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00794c8e94193c19ead0761a6e998297",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00794c8e94193c19ead0761a6e998297/impacts"
                }
            },
            {
                "id": "productid.00894e0b7b7a3602ef37e86c23203b48",
                "name": "Metergum Strawberry Flavour",
                "gtin": "8690146619215",
                "image": "https://static.euw1.sustained.app/products/images/8690146619215_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00894e0b7b7a3602ef37e86c23203b48",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00894e0b7b7a3602ef37e86c23203b48/impacts"
                }
            },
            {
                "id": "productid.00b25049aa2fccbed935be9e886973c5",
                "name": "Refreshers & Wham Ice Duos",
                "gtin": "5031396072369",
                "image": "https://static.euw1.sustained.app/products/images/5031396072369_0.jpg",
                "pack": "80ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00b25049aa2fccbed935be9e886973c5",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00b25049aa2fccbed935be9e886973c5/impacts"
                }
            },
            {
                "id": "productid.00c31dc3be971ce1298e2e28985d1f65",
                "name": "Clotted Cream Fudge",
                "gtin": "5000169608395",
                "image": "https://static.euw1.sustained.app/products/images/5000169608395_0.jpg",
                "pack": "150g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00c31dc3be971ce1298e2e28985d1f65",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00c31dc3be971ce1298e2e28985d1f65/impacts"
                }
            },
            {
                "id": "productid.010a120f028980d923bd00273f0e8703",
                "name": "HARIBO Starmix",
                "gtin": "5012035969875",
                "image": "https://static.euw1.sustained.app/products/images/5012035969875_0.jpg",
                "pack": "800g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.010a120f028980d923bd00273f0e8703",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.010a120f028980d923bd00273f0e8703/impacts"
                }
            },
            {
                "id": "productid.0135cb97cfdfd0edb9ac8be5602afb05",
                "name": "Chewy Cinnamons",
                "gtin": "5026327007190",
                "image": "https://static.euw1.sustained.app/products/images/5026327007190_0.jpg",
                "pack": "120g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0135cb97cfdfd0edb9ac8be5602afb05",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0135cb97cfdfd0edb9ac8be5602afb05/impacts"
                }
            },
            {
                "id": "productid.01aad7641edecd0f60a8bd357af57d1b",
                "name": "Pink Marc de Champagne Mini-Truffles",
                "gtin": "5060047374071",
                "image": "https://static.euw1.sustained.app/products/images/5060047374071_0.jpg",
                "pack": "105g",
                "grade": "G",
                "points": {
                    "pef": 107.06284073755043
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01aad7641edecd0f60a8bd357af57d1b",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01aad7641edecd0f60a8bd357af57d1b/impacts"
                }
            },
            {
                "id": "productid.0216da321f8bfd2cf1725b00e9291171",
                "name": "Favourites Strawberry Delight",
                "gtin": "8445290019172",
                "image": "https://static.euw1.sustained.app/products/images/8445290019172_0.jpg",
                "pack": "344g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0216da321f8bfd2cf1725b00e9291171",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0216da321f8bfd2cf1725b00e9291171/impacts"
                }
            },
            {
                "id": "productid.0217e1aca6eaaa352fba2200b99874c4",
                "name": "2 Finger Dark Chocolate Biscuit Bar",
                "gtin": "7613035219465",
                "image": "https://static.euw1.sustained.app/products/images/7613035219465_0.jpg",
                "pack": "20.7g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0217e1aca6eaaa352fba2200b99874c4",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0217e1aca6eaaa352fba2200b99874c4/impacts"
                }
            },
            {
                "id": "productid.023048c7b9c96827636b7b663735a421",
                "name": "Nugglets Choc Brownie",
                "gtin": "5060426310423",
                "image": "https://static.euw1.sustained.app/products/images/5060426310423_0.jpg",
                "pack": "35g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.023048c7b9c96827636b7b663735a421",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.023048c7b9c96827636b7b663735a421/impacts"
                }
            },
            {
                "id": "productid.029387a5a5ef79e0610c96a8d5d0b7bb",
                "name": "Micha\u0142ki Bia\u0142e White Compound Coated Candies with Peanuts",
                "gtin": "5900102012138",
                "image": "https://static.euw1.sustained.app/products/images/5900102012138_0.jpg",
                "pack": "280g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.029387a5a5ef79e0610c96a8d5d0b7bb",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.029387a5a5ef79e0610c96a8d5d0b7bb/impacts"
                }
            },
            {
                "id": "productid.02956d6a3047b2f056341d4af77558fd",
                "name": "Giant Candy Cane",
                "gtin": "5057172209231",
                "image": "https://static.euw1.sustained.app/products/images/5057172209231_0.jpg",
                "pack": "80g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02956d6a3047b2f056341d4af77558fd",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02956d6a3047b2f056341d4af77558fd/impacts"
                }
            },
            {
                "id": "productid.02be27367c3d8e10b6e20bc897b13f97",
                "name": "Dark Chocolate Truffles",
                "gtin": "5056030508493",
                "image": "https://static.euw1.sustained.app/products/images/5056030508493_0.jpg",
                "pack": "130g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02be27367c3d8e10b6e20bc897b13f97",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02be27367c3d8e10b6e20bc897b13f97/impacts"
                }
            },
            {
                "id": "productid.02e3169c53049371d72d8c82e74b76b5",
                "name": "Glacier Dark",
                "gtin": "5000314009657",
                "image": "https://static.euw1.sustained.app/products/images/5000314009657_0.jpg",
                "pack": "130g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02e3169c53049371d72d8c82e74b76b5",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02e3169c53049371d72d8c82e74b76b5/impacts"
                }
            },
            {
                "id": "productid.02f8e199bab0af74fb22fc2908e04d57",
                "name": "Fruits Sweets Fun Size Bags Multipack",
                "gtin": "4009900542128",
                "image": "https://static.euw1.sustained.app/products/images/4009900542128_0.jpg",
                "pack": "18g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02f8e199bab0af74fb22fc2908e04d57",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02f8e199bab0af74fb22fc2908e04d57/impacts"
                }
            },
            {
                "id": "productid.02feccdac397754acbbf13724cd931ba",
                "name": "Rainbow Chewy Dragees",
                "gtin": "87311396",
                "image": "https://static.euw1.sustained.app/products/images/87311396_0.jpg",
                "pack": "37.5g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02feccdac397754acbbf13724cd931ba",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02feccdac397754acbbf13724cd931ba/impacts"
                }
            },
            {
                "id": "productid.03060ee093d540ecd5adfc49163164f8",
                "name": "Milk Chocolate Large Easter Egg",
                "gtin": "5000159539265",
                "image": "https://static.euw1.sustained.app/products/images/5000159539265_0.jpg",
                "pack": "252g",
                "grade": "G",
                "points": {
                    "pef": 131.97680711229503
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03060ee093d540ecd5adfc49163164f8",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03060ee093d540ecd5adfc49163164f8/impacts"
                }
            },
            {
                "id": "productid.0310c5078a2409cd0eb37ec70e1a4d3e",
                "name": "Crumbly Fudge Vanilla Fudge",
                "gtin": "5036854006023",
                "image": "https://static.euw1.sustained.app/products/images/5036854006023_0.jpg",
                "pack": "150g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0310c5078a2409cd0eb37ec70e1a4d3e",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0310c5078a2409cd0eb37ec70e1a4d3e/impacts"
                }
            },
            {
                "id": "productid.03260e002dc67024b2e0076c539b8200",
                "name": "The Best Belgian Milk Chocolate Egg",
                "gtin": "5010525164632",
                "image": "https://static.euw1.sustained.app/products/images/5010525164632_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 132.06480443983182
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03260e002dc67024b2e0076c539b8200",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03260e002dc67024b2e0076c539b8200/impacts"
                }
            },
            {
                "id": "productid.0329dac2a1e7b9d8cf79879316672aeb",
                "name": "Chocolate \u00a31 PMP Treat Bag",
                "gtin": "5000159504003",
                "image": "https://static.euw1.sustained.app/products/images/5000159504003_0.jpg",
                "pack": "82g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0329dac2a1e7b9d8cf79879316672aeb",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0329dac2a1e7b9d8cf79879316672aeb/impacts"
                }
            },
            {
                "id": "productid.035599849b4b7e1e263a27eb61965614",
                "name": "Randoms Sours Sweets Sharing Pouch",
                "gtin": "8593893765795",
                "image": "https://static.euw1.sustained.app/products/images/8593893765795_0.jpg",
                "pack": "140g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.035599849b4b7e1e263a27eb61965614",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.035599849b4b7e1e263a27eb61965614/impacts"
                }
            },
            {
                "id": "productid.037a166530715deb165ee5302f3a6c7d",
                "name": "Caramilk Golden Caramel Chocolate Bar 4 Pack",
                "gtin": "7622201788292",
                "image": "https://static.euw1.sustained.app/products/images/7622201788292_0.jpg",
                "pack": "28g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.037a166530715deb165ee5302f3a6c7d",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.037a166530715deb165ee5302f3a6c7d/impacts"
                }
            },
            {
                "id": "productid.0397634d7ae5a566fd7a7601fca55791",
                "name": "Double Chocolate Marshmallow Bar",
                "gtin": "5060374400238",
                "image": "https://static.euw1.sustained.app/products/images/5060374400238_0.jpg",
                "pack": "35g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0397634d7ae5a566fd7a7601fca55791",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0397634d7ae5a566fd7a7601fca55791/impacts"
                }
            },
            {
                "id": "productid.03a83fc99d3345ab4af3928e3042a2d3",
                "name": "Mega Ball Gum Tutti Frutti Flavour Bubble Gum 150 Pieces",
                "gtin": "5021202552079",
                "image": "https://static.euw1.sustained.app/products/images/5021202552079_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03a83fc99d3345ab4af3928e3042a2d3",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03a83fc99d3345ab4af3928e3042a2d3/impacts"
                }
            },
            {
                "id": "productid.042b28975c4fc7e702ec05efb2b08bb6",
                "name": "Counters Chocolate \u00a31 PMP Treat Bag",
                "gtin": "5000159491334",
                "image": "https://static.euw1.sustained.app/products/images/5000159491334_0.jpg",
                "pack": "78g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.042b28975c4fc7e702ec05efb2b08bb6",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.042b28975c4fc7e702ec05efb2b08bb6/impacts"
                }
            },
            {
                "id": "productid.045a3a82e81d895256378b1cdeb1565a",
                "name": "Christmas Dark Chocolate Cherry and Amaretto Fudge",
                "gtin": "5000169631096",
                "image": "https://static.euw1.sustained.app/products/images/5000169631096_0.jpg",
                "pack": "130g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.045a3a82e81d895256378b1cdeb1565a",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.045a3a82e81d895256378b1cdeb1565a/impacts"
                }
            },
            {
                "id": "productid.046320ad62be6ed1d8d285531a320ec6",
                "name": "Strawberry Fruit Strings",
                "gtin": "5060036011512",
                "image": "https://static.euw1.sustained.app/products/images/5060036011512_0.jpg",
                "pack": "20g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.046320ad62be6ed1d8d285531a320ec6",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.046320ad62be6ed1d8d285531a320ec6/impacts"
                }
            },
            {
                "id": "productid.047f22d790e158c61b3857a016040a57",
                "name": "Heroes Chocolate Tub",
                "gtin": "7622210954695",
                "image": "https://static.euw1.sustained.app/products/images/7622210954695_0.jpg",
                "pack": "600g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.047f22d790e158c61b3857a016040a57",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.047f22d790e158c61b3857a016040a57/impacts"
                }
            },
            {
                "id": "productid.0492121b490fa52baac00f550e5b3392",
                "name": "2 Milk Chocolate & Peanut Butter Trees King Size",
                "gtin": "0034000473700",
                "image": "https://static.euw1.sustained.app/products/images/0034000473700_0.jpg",
                "pack": "68g",
                "grade": "E",
                "points": {
                    "pef": 34.36003186663224
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0492121b490fa52baac00f550e5b3392",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0492121b490fa52baac00f550e5b3392/impacts"
                }
            },
            {
                "id": "productid.04a16033d4220b9c369fb334edb59541",
                "name": "5 Mini Rolls Mint Choc Ice Cream Flavour",
                "gtin": "5000221606536",
                "image": "https://static.euw1.sustained.app/products/images/5000221606536_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 62.71483249806075
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04a16033d4220b9c369fb334edb59541",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04a16033d4220b9c369fb334edb59541/impacts"
                }
            },
            {
                "id": "productid.04b6c8d7c7786203f3723112d72e4011",
                "name": "Watermelon Drops",
                "gtin": "5032860010702",
                "image": "https://static.euw1.sustained.app/products/images/5032860010702_0.jpg",
                "pack": "180g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04b6c8d7c7786203f3723112d72e4011",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04b6c8d7c7786203f3723112d72e4011/impacts"
                }
            },
            {
                "id": "productid.051740dd96f23849658870269588a54f",
                "name": "Choco & Fruity Jellies in Chocolate",
                "gtin": "5900102020096",
                "image": "https://static.euw1.sustained.app/products/images/5900102020096_0.jpg",
                "pack": "195g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.051740dd96f23849658870269588a54f",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.051740dd96f23849658870269588a54f/impacts"
                }
            },
            {
                "id": "productid.0536a44df433a189dee1158a51c20f7e",
                "name": "Organic Dark Collection Large Easter Egg",
                "gtin": "7622201099169",
                "image": "https://static.euw1.sustained.app/products/images/7622201099169_0.jpg",
                "pack": "345g",
                "grade": "G",
                "points": {
                    "pef": 134.92483553535095
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0536a44df433a189dee1158a51c20f7e",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0536a44df433a189dee1158a51c20f7e/impacts"
                }
            },
            {
                "id": "productid.055fdc98ccf300318981dc44e9b24af7",
                "name": "Signature Collection",
                "gtin": "5055037003123",
                "image": "https://static.euw1.sustained.app/products/images/5055037003123_0.jpg",
                "pack": "370g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.055fdc98ccf300318981dc44e9b24af7",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.055fdc98ccf300318981dc44e9b24af7/impacts"
                }
            },
            {
                "id": "productid.05a470f2767276379fb9a3f618d64c21",
                "name": "Food Turkish Delight",
                "gtin": "0000029099757",
                "image": "https://static.euw1.sustained.app/products/images/0000029099757_0.jpg",
                "pack": "250g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05a470f2767276379fb9a3f618d64c21",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05a470f2767276379fb9a3f618d64c21/impacts"
                }
            },
            {
                "id": "productid.05a4dfc53c94a02a41663345e13c3035",
                "name": "Milk Teeth",
                "gtin": "5010511477807",
                "image": "https://static.euw1.sustained.app/products/images/5010511477807_0.jpg",
                "pack": "190g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05a4dfc53c94a02a41663345e13c3035",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05a4dfc53c94a02a41663345e13c3035/impacts"
                }
            },
            {
                "id": "productid.05a53e9060710a73a736431361dc687d",
                "name": "MixUps Milk&White Chocolate Giant Easter Egg",
                "gtin": "7613037528183",
                "image": "https://static.euw1.sustained.app/products/images/7613037528183_0.jpg",
                "pack": "295g",
                "grade": "G",
                "points": {
                    "pef": 131.97680711229503
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05a53e9060710a73a736431361dc687d",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05a53e9060710a73a736431361dc687d/impacts"
                }
            },
            {
                "id": "productid.06063b1715eed7464247e112dd08b7c2",
                "name": "Jelly Babies \u00a31 Sweets Bag",
                "gtin": "7622210654199",
                "image": "https://static.euw1.sustained.app/products/images/7622210654199_0.jpg",
                "pack": "165g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06063b1715eed7464247e112dd08b7c2",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06063b1715eed7464247e112dd08b7c2/impacts"
                }
            },
            {
                "id": "productid.061c799d344a74cb6720777da0edc5ff",
                "name": "Fruit in Jelly Mandarin in Orange Jelly",
                "gtin": "5060391620787",
                "image": "https://static.euw1.sustained.app/products/images/5060391620787_0.jpg",
                "pack": "120g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.061c799d344a74cb6720777da0edc5ff",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.061c799d344a74cb6720777da0edc5ff/impacts"
                }
            },
            {
                "id": "productid.063061722d41ebba46b9c0b5b3caf056",
                "name": "Fried Eggs",
                "gtin": "5031396071188",
                "image": "https://static.euw1.sustained.app/products/images/5031396071188_0.jpg",
                "pack": "45g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.063061722d41ebba46b9c0b5b3caf056",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.063061722d41ebba46b9c0b5b3caf056/impacts"
                }
            },
            {
                "id": "productid.063c60d0d6f05826900ce897c517d6c6",
                "name": "Bites Double Choc Brownie",
                "gtin": "5391535677603",
                "image": "https://static.euw1.sustained.app/products/images/5391535677603_0.jpg",
                "pack": "162g",
                "grade": "F",
                "points": {
                    "pef": 75.09413759047233
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.063c60d0d6f05826900ce897c517d6c6",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.063c60d0d6f05826900ce897c517d6c6/impacts"
                }
            },
            {
                "id": "productid.0662b142077cf4b4703689890ad4d2da",
                "name": "Chocolate Pouch Bag",
                "gtin": "5000159530736",
                "image": "https://static.euw1.sustained.app/products/images/5000159530736_0.jpg",
                "pack": "102g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0662b142077cf4b4703689890ad4d2da",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0662b142077cf4b4703689890ad4d2da/impacts"
                }
            },
            {
                "id": "productid.067a2e709901d03836295d80868f58a7",
                "name": "Wafer Crunch",
                "gtin": "4260402484990",
                "image": "https://static.euw1.sustained.app/products/images/4260402484990_0.jpg",
                "pack": "80g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.067a2e709901d03836295d80868f58a7",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.067a2e709901d03836295d80868f58a7/impacts"
                }
            },
            {
                "id": "productid.06ac314c66aab6092ea2e2703a8e261b",
                "name": "Dragon Gummies Fruity",
                "gtin": "5900102022618",
                "image": "https://static.euw1.sustained.app/products/images/5900102022618_0.jpg",
                "pack": "100g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06ac314c66aab6092ea2e2703a8e261b",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06ac314c66aab6092ea2e2703a8e261b/impacts"
                }
            },
            {
                "id": "productid.07adeca3d7b68036f220e32d90b39a0c",
                "name": "Original Fruit Chews Sweets Multipack",
                "gtin": "4009900469630",
                "image": "https://static.euw1.sustained.app/products/images/4009900469630_0.jpg",
                "pack": "45g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07adeca3d7b68036f220e32d90b39a0c",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07adeca3d7b68036f220e32d90b39a0c/impacts"
                }
            },
            {
                "id": "productid.07cda26e7e14af53593ba30b4f37914d",
                "name": "Biscuits Classic",
                "gtin": "5900352002361",
                "image": "https://static.euw1.sustained.app/products/images/5900352002361_0.jpg",
                "pack": "140g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07cda26e7e14af53593ba30b4f37914d",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07cda26e7e14af53593ba30b4f37914d/impacts"
                }
            },
            {
                "id": "productid.07e410aabd56a23560771ab81eb65e9b",
                "name": "Giant Strawbs Bag",
                "gtin": "5012035962135",
                "image": "https://static.euw1.sustained.app/products/images/5012035962135_0.jpg",
                "pack": "160g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Chocolate & Sweets",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07e410aabd56a23560771ab81eb65e9b",
                    "category": "https://api.sustained.com/choice/v1/categories/chocolate-and-sweets",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07e410aabd56a23560771ab81eb65e9b/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.00db21d920635a0802ff14368df5a64c",
                "name": "Mealmaker Lamb Casserole",
                "gtin": "8712566143993",
                "image": "https://static.euw1.sustained.app/products/images/8712566143993_0.jpg",
                "pack": "47g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00db21d920635a0802ff14368df5a64c",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00db21d920635a0802ff14368df5a64c/impacts"
                }
            },
            {
                "id": "productid.00e05f453cef4da04989a736d8b9db2d",
                "name": "Food Made Without Wheat Gravy",
                "gtin": "0000000230728",
                "image": "https://static.euw1.sustained.app/products/images/0000000230728_0.jpg",
                "pack": "300g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00e05f453cef4da04989a736d8b9db2d",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00e05f453cef4da04989a736d8b9db2d/impacts"
                }
            },
            {
                "id": "productid.017a6eb4b90472455303bdbddc9c3d45",
                "name": "Mealmaker Pork Casserole",
                "gtin": "8712566144075",
                "image": "https://static.euw1.sustained.app/products/images/8712566144075_0.jpg",
                "pack": "44g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.017a6eb4b90472455303bdbddc9c3d45",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.017a6eb4b90472455303bdbddc9c3d45/impacts"
                }
            },
            {
                "id": "productid.01835278c7a5356236c4967438f2e2b2",
                "name": "Tesco Breadcrumbs",
                "gtin": "5057545079935",
                "image": "https://static.euw1.sustained.app/products/images/5057545079935_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 25.173899295232005
                },
                "info_icons": [],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01835278c7a5356236c4967438f2e2b2",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01835278c7a5356236c4967438f2e2b2/impacts"
                }
            },
            {
                "id": "productid.02a57916135c1fc22791284e3f97ad2c",
                "name": "Beef Taco Mix",
                "gtin": "5391512491581",
                "image": "https://static.euw1.sustained.app/products/images/5391512491581_0.jpg",
                "pack": "2.5kg",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02a57916135c1fc22791284e3f97ad2c",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02a57916135c1fc22791284e3f97ad2c/impacts"
                }
            },
            {
                "id": "productid.02c59ad928110c9c5910882da1b5bc58",
                "name": "Pesto Verde",
                "gtin": "5391501860060",
                "image": "https://static.euw1.sustained.app/products/images/5391501860060_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 48.12611360947992
                },
                "info_icons": [],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02c59ad928110c9c5910882da1b5bc58",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02c59ad928110c9c5910882da1b5bc58/impacts"
                }
            },
            {
                "id": "productid.05cfd5790214f4382be336c0ea0de930",
                "name": "Uncle Bens PMP \u00a31.89 Tikka Masala Sauce",
                "gtin": "4002359002533",
                "image": "https://static.euw1.sustained.app/products/images/4002359002533_0.jpg",
                "pack": "450g",
                "grade": "B",
                "points": {
                    "pef": 12.126191723358424
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05cfd5790214f4382be336c0ea0de930",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05cfd5790214f4382be336c0ea0de930/impacts"
                }
            },
            {
                "id": "productid.0823f1a78c38fcc35d071a8d4d3ffb0b",
                "name": "Professional Tikka Masala Sauce",
                "gtin": "8712566362035",
                "image": "https://static.euw1.sustained.app/products/images/8712566362035_0.jpg",
                "pack": "2.2l",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0823f1a78c38fcc35d071a8d4d3ffb0b",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0823f1a78c38fcc35d071a8d4d3ffb0b/impacts"
                }
            },
            {
                "id": "productid.08dde8e58fb88571acac53d928569307",
                "name": "Mattarello Arrabbiata Sauce with Hot Chillies",
                "gtin": "5060131680132",
                "image": "https://static.euw1.sustained.app/products/images/5060131680132_0.jpg",
                "pack": "230g",
                "grade": "B",
                "points": {
                    "pef": 12.108396953697923
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08dde8e58fb88571acac53d928569307",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08dde8e58fb88571acac53d928569307/impacts"
                }
            },
            {
                "id": "productid.0953e1549d9d06e60d37a1934b5c54e4",
                "name": "Best Turkey Gravy",
                "gtin": "5000354915635",
                "image": "https://static.euw1.sustained.app/products/images/5000354915635_0.jpg",
                "pack": "250g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0953e1549d9d06e60d37a1934b5c54e4",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0953e1549d9d06e60d37a1934b5c54e4/impacts"
                }
            },
            {
                "id": "productid.0a020b8d770d52bada6e687d482713b3",
                "name": "Sauce for Beef in Ale",
                "gtin": "01843903",
                "image": "https://static.euw1.sustained.app/products/images/01843903_0.jpg",
                "pack": "500g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a020b8d770d52bada6e687d482713b3",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a020b8d770d52bada6e687d482713b3/impacts"
                }
            },
            {
                "id": "productid.0a10dee9f5da8fc09d30ae2bbab3fc35",
                "name": "Cook With M&S Chicken Stock",
                "gtin": "0000000568296",
                "image": "https://static.euw1.sustained.app/products/images/0000000568296_0.jpg",
                "pack": "500ml",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a10dee9f5da8fc09d30ae2bbab3fc35",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a10dee9f5da8fc09d30ae2bbab3fc35/impacts"
                }
            },
            {
                "id": "productid.0a951b18063e1d2f8cb9a458453b0541",
                "name": "Italian Style Arrabbiatta Sauce",
                "gtin": "5057172106523",
                "image": "https://static.euw1.sustained.app/products/images/5057172106523_0.jpg",
                "pack": "300g",
                "grade": "A",
                "points": {
                    "pef": 10.951201783511591
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a951b18063e1d2f8cb9a458453b0541",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a951b18063e1d2f8cb9a458453b0541/impacts"
                }
            },
            {
                "id": "productid.0aed99a28487732a4347df0617b21115",
                "name": "Bolognese Pasta Sauce",
                "gtin": "8000139930645",
                "image": "https://static.euw1.sustained.app/products/images/8000139930645_0.jpg",
                "pack": "310g",
                "grade": "F",
                "points": {
                    "pef": 53.35127828781114
                },
                "info_icons": [],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0aed99a28487732a4347df0617b21115",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0aed99a28487732a4347df0617b21115/impacts"
                }
            },
            {
                "id": "productid.0b3f749abefc3d51f29d9f04b0e02e25",
                "name": "Chef's Favourites Bolognese Cook-In Sauce",
                "gtin": "5000354400131",
                "image": "https://static.euw1.sustained.app/products/images/5000354400131_0.jpg",
                "pack": "2kg",
                "grade": "A",
                "points": {
                    "pef": 8.353331854089708
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b3f749abefc3d51f29d9f04b0e02e25",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b3f749abefc3d51f29d9f04b0e02e25/impacts"
                }
            },
            {
                "id": "productid.0c765dac288b5fb8d653c91e84a93c86",
                "name": "Organic Tomato & Basil Pasta Sauce",
                "gtin": "5060132280072",
                "image": "https://static.euw1.sustained.app/products/images/5060132280072_0.jpg",
                "pack": "350g",
                "grade": "A",
                "points": {
                    "pef": 8.73594316573969
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c765dac288b5fb8d653c91e84a93c86",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c765dac288b5fb8d653c91e84a93c86/impacts"
                }
            },
            {
                "id": "productid.0e23fb404c83983345f7577b2274430b",
                "name": "Tomato and Parmesan Organic Pasta Sauce",
                "gtin": "4002359644498",
                "image": "https://static.euw1.sustained.app/products/images/4002359644498_0.jpg",
                "pack": "350g",
                "grade": "A",
                "points": {
                    "pef": 10.423971202072378
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e23fb404c83983345f7577b2274430b",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e23fb404c83983345f7577b2274430b/impacts"
                }
            },
            {
                "id": "productid.0fa9d943c6d22794d4ce1f9579f4f857",
                "name": "Red Curry Paste",
                "gtin": "0044738201698",
                "image": "https://static.euw1.sustained.app/products/images/0044738201698_0.jpg",
                "pack": "1000g",
                "grade": "A",
                "points": {
                    "pef": 10.657808603065485
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fa9d943c6d22794d4ce1f9579f4f857",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fa9d943c6d22794d4ce1f9579f4f857/impacts"
                }
            },
            {
                "id": "productid.10d6dd97c3451b11aeaeea471fcb7a1d",
                "name": "Uncle Bens Tikka Masala Curry Sauce",
                "gtin": "4002359661044",
                "image": "https://static.euw1.sustained.app/products/images/4002359661044_0.jpg",
                "pack": "2.24kg",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10d6dd97c3451b11aeaeea471fcb7a1d",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10d6dd97c3451b11aeaeea471fcb7a1d/impacts"
                }
            },
            {
                "id": "productid.1101b33bced7840700462f9541a2d9ef",
                "name": "Creamy Tomato & Herb Pasta Bake Sauce",
                "gtin": "5051898805821",
                "image": "https://static.euw1.sustained.app/products/images/5051898805821_0.jpg",
                "pack": "500g",
                "grade": "B",
                "points": {
                    "pef": 15.309833784187228
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1101b33bced7840700462f9541a2d9ef",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1101b33bced7840700462f9541a2d9ef/impacts"
                }
            },
            {
                "id": "productid.12ec83d27655f345f3d61ef8d6b50623",
                "name": "Napoletana Pasta Sauce",
                "gtin": "0000000021029",
                "image": "https://static.euw1.sustained.app/products/images/0000000021029_0.jpg",
                "pack": "340g",
                "grade": "A",
                "points": {
                    "pef": 10.209776543853309
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12ec83d27655f345f3d61ef8d6b50623",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12ec83d27655f345f3d61ef8d6b50623/impacts"
                }
            },
            {
                "id": "productid.1367e0c18c9ddf8ad821b4d4d6f9f3dd",
                "name": "Cook With M&S Food Mushroom & Roasted Garlic Paste",
                "gtin": "0000000832458",
                "image": "https://static.euw1.sustained.app/products/images/0000000832458_0.jpg",
                "pack": "190g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1367e0c18c9ddf8ad821b4d4d6f9f3dd",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1367e0c18c9ddf8ad821b4d4d6f9f3dd/impacts"
                }
            },
            {
                "id": "productid.1403070e21555074235fba08b3aae5b1",
                "name": "Passage to Asia Pad Thai Stir-Fry Sauce",
                "gtin": "9334279000258",
                "image": "https://static.euw1.sustained.app/products/images/9334279000258_0.jpg",
                "pack": "200g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1403070e21555074235fba08b3aae5b1",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1403070e21555074235fba08b3aae5b1/impacts"
                }
            },
            {
                "id": "productid.148e3be566cef900a011fc269be88295",
                "name": "Bens Original Katsu Curry Cooking Sauce",
                "gtin": "4002359016110",
                "image": "https://static.euw1.sustained.app/products/images/4002359016110_0.jpg",
                "pack": "2.23kg",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.148e3be566cef900a011fc269be88295",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.148e3be566cef900a011fc269be88295/impacts"
                }
            },
            {
                "id": "productid.14dcb61cee65026afbbe5a94ddee57f1",
                "name": "Food Sundried Tomato & Basil Stir in Sauce",
                "gtin": "0000000023702",
                "image": "https://static.euw1.sustained.app/products/images/0000000023702_0.jpg",
                "pack": "150g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14dcb61cee65026afbbe5a94ddee57f1",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14dcb61cee65026afbbe5a94ddee57f1/impacts"
                }
            },
            {
                "id": "productid.14ff419aaca6ea494a9a2602b70fde7e",
                "name": "Chicken Stock Pots",
                "gtin": "5010251920182",
                "image": "https://static.euw1.sustained.app/products/images/5010251920182_0.jpg",
                "pack": "28g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14ff419aaca6ea494a9a2602b70fde7e",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14ff419aaca6ea494a9a2602b70fde7e/impacts"
                }
            },
            {
                "id": "productid.16381ab1c193c81c7fbc8fd7943a4264",
                "name": "Food Chicken Stock",
                "gtin": "0000029159130",
                "image": "https://static.euw1.sustained.app/products/images/0000029159130_0.jpg",
                "pack": "500g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16381ab1c193c81c7fbc8fd7943a4264",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16381ab1c193c81c7fbc8fd7943a4264/impacts"
                }
            },
            {
                "id": "productid.16f0dc1d44570dcc5a7fba4461ed60bf",
                "name": "Onion Gravy Granules",
                "gtin": "5019989102709",
                "image": "https://static.euw1.sustained.app/products/images/5019989102709_0.jpg",
                "pack": "1.9kg",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16f0dc1d44570dcc5a7fba4461ed60bf",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16f0dc1d44570dcc5a7fba4461ed60bf/impacts"
                }
            },
            {
                "id": "productid.173b10d1fdfc89268fadf3a0e90ce398",
                "name": "Vegan Supergreen Sauce with Broccoli & Cavolo Nero",
                "gtin": "8001060036444",
                "image": "https://static.euw1.sustained.app/products/images/8001060036444_0.jpg",
                "pack": "350g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.173b10d1fdfc89268fadf3a0e90ce398",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.173b10d1fdfc89268fadf3a0e90ce398/impacts"
                }
            },
            {
                "id": "productid.178881742934da130aa5747cbe488d53",
                "name": "Katsu Curry Stir-In Sauce",
                "gtin": "5060524150167",
                "image": "https://static.euw1.sustained.app/products/images/5060524150167_0.jpg",
                "pack": "260g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.178881742934da130aa5747cbe488d53",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.178881742934da130aa5747cbe488d53/impacts"
                }
            },
            {
                "id": "productid.185ded6c3d6a9d19070dcab47eb70bd9",
                "name": "Authentic Italian Roasted Garlic Pasta Sauce",
                "gtin": "5060178073713",
                "image": "https://static.euw1.sustained.app/products/images/5060178073713_0.jpg",
                "pack": "350g",
                "grade": "A",
                "points": {
                    "pef": 9.00100831139985
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.185ded6c3d6a9d19070dcab47eb70bd9",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.185ded6c3d6a9d19070dcab47eb70bd9/impacts"
                }
            },
            {
                "id": "productid.186a852c75c40a6c942ae908887ed8fd",
                "name": "Bens Original Balti Curry Sauce",
                "gtin": "4002359016172",
                "image": "https://static.euw1.sustained.app/products/images/4002359016172_0.jpg",
                "pack": "2.24kg",
                "grade": "A",
                "points": {
                    "pef": 10.129076638501822
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.186a852c75c40a6c942ae908887ed8fd",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.186a852c75c40a6c942ae908887ed8fd/impacts"
                }
            },
            {
                "id": "productid.1906c9e410a0d912b7b8c65e972e7c59",
                "name": "Pepper Sauce Mix",
                "gtin": "8712566100279",
                "image": "https://static.euw1.sustained.app/products/images/8712566100279_0.jpg",
                "pack": "600g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1906c9e410a0d912b7b8c65e972e7c59",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1906c9e410a0d912b7b8c65e972e7c59/impacts"
                }
            },
            {
                "id": "productid.19ccfc30a35b38202ef9f2676c765b0e",
                "name": "24 Reduced Salt Chicken Stock Cubes",
                "gtin": "5000354916083",
                "image": "https://static.euw1.sustained.app/products/images/5000354916083_0.jpg",
                "pack": "142g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.19ccfc30a35b38202ef9f2676c765b0e",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.19ccfc30a35b38202ef9f2676c765b0e/impacts"
                }
            },
            {
                "id": "productid.1aa9d68683e3e946dff6ea8497d84b0d",
                "name": "Natural Breadcrumbs",
                "gtin": "5020933110671",
                "image": "https://static.euw1.sustained.app/products/images/5020933110671_0.jpg",
                "pack": "3kg",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1aa9d68683e3e946dff6ea8497d84b0d",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1aa9d68683e3e946dff6ea8497d84b0d/impacts"
                }
            },
            {
                "id": "productid.1ab6c8772544c1532cd4ccb07bee8f46",
                "name": "Cook With M&S Food Creamy Cheese, Leek & Bacon Sauce",
                "gtin": "0000000969178",
                "image": "https://static.euw1.sustained.app/products/images/0000000969178_0.jpg",
                "pack": "350g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1ab6c8772544c1532cd4ccb07bee8f46",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1ab6c8772544c1532cd4ccb07bee8f46/impacts"
                }
            },
            {
                "id": "productid.1b4a5e6857233563173955a5e56e8d84",
                "name": "Slow Cook Chicken Curry Recipe Mix",
                "gtin": "5000225038357",
                "image": "https://static.euw1.sustained.app/products/images/5000225038357_0.jpg",
                "pack": "33g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b4a5e6857233563173955a5e56e8d84",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b4a5e6857233563173955a5e56e8d84/impacts"
                }
            },
            {
                "id": "productid.1bab5cbfe8ca5b06accd2006ff199d56",
                "name": "Gravy Granules",
                "gtin": "5000354907609",
                "image": "https://static.euw1.sustained.app/products/images/5000354907609_0.jpg",
                "pack": "221g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1bab5cbfe8ca5b06accd2006ff199d56",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1bab5cbfe8ca5b06accd2006ff199d56/impacts"
                }
            },
            {
                "id": "productid.1bfd6d6ffddaa7f96e97fc3547254e24",
                "name": "Beef Casserole Recipe Mix",
                "gtin": "5000225005779",
                "image": "https://static.euw1.sustained.app/products/images/5000225005779_0.jpg",
                "pack": "43g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1bfd6d6ffddaa7f96e97fc3547254e24",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1bfd6d6ffddaa7f96e97fc3547254e24/impacts"
                }
            },
            {
                "id": "productid.1c5888e94f9737d794f8e34af115ca6b",
                "name": "BBQ Seasoning",
                "gtin": "0000003282342",
                "image": "https://static.euw1.sustained.app/products/images/0000003282342_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 22.91519445576034
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c5888e94f9737d794f8e34af115ca6b",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c5888e94f9737d794f8e34af115ca6b/impacts"
                }
            },
            {
                "id": "productid.1c74ce0c1ed37d815a2ac13d72afefdf",
                "name": "Vegetable Bouillon Paste",
                "gtin": "5019989100644",
                "image": "https://static.euw1.sustained.app/products/images/5019989100644_0.jpg",
                "pack": "1kg",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c74ce0c1ed37d815a2ac13d72afefdf",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c74ce0c1ed37d815a2ac13d72afefdf/impacts"
                }
            },
            {
                "id": "productid.1c9767e22d324a7226adb22a431a8b1a",
                "name": "Seasoning Mix for Taco Original Medium",
                "gtin": "7311311011191",
                "image": "https://static.euw1.sustained.app/products/images/7311311011191_0.jpg",
                "pack": "28g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c9767e22d324a7226adb22a431a8b1a",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c9767e22d324a7226adb22a431a8b1a/impacts"
                }
            },
            {
                "id": "productid.1ce0a28133c1ccc04f97aa605cdfc10e",
                "name": "Tomato & Roasted Garlic No Added Sugar Pasta Sauce",
                "gtin": "4002359008375",
                "image": "https://static.euw1.sustained.app/products/images/4002359008375_0.jpg",
                "pack": "350g",
                "grade": "A",
                "points": {
                    "pef": 9.981600274919437
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1ce0a28133c1ccc04f97aa605cdfc10e",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1ce0a28133c1ccc04f97aa605cdfc10e/impacts"
                }
            },
            {
                "id": "productid.1d35f95ea0f0b48c2553e4c9a34e40a2",
                "name": "Creamy Yellow Thai Curry Fresh Cooking Sauce",
                "gtin": "5060482841114",
                "image": "https://static.euw1.sustained.app/products/images/5060482841114_0.jpg",
                "pack": "240g",
                "grade": "A",
                "points": {
                    "pef": 10.441554583933842
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1d35f95ea0f0b48c2553e4c9a34e40a2",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1d35f95ea0f0b48c2553e4c9a34e40a2/impacts"
                }
            },
            {
                "id": "productid.1ee7fc2c6c7a03fb231d1ca63762db5a",
                "name": "Sweet & Sour Extra Pineapple",
                "gtin": "4002359014802",
                "image": "https://static.euw1.sustained.app/products/images/4002359014802_0.jpg",
                "pack": "450g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1ee7fc2c6c7a03fb231d1ca63762db5a",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1ee7fc2c6c7a03fb231d1ca63762db5a/impacts"
                }
            },
            {
                "id": "productid.1ef23d1f9b364e17d21be6a40d95c925",
                "name": "Chicken Stock Cubes",
                "gtin": "5010251556237",
                "image": "https://static.euw1.sustained.app/products/images/5010251556237_0.jpg",
                "pack": "120g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1ef23d1f9b364e17d21be6a40d95c925",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1ef23d1f9b364e17d21be6a40d95c925/impacts"
                }
            },
            {
                "id": "productid.1f64bfb722bc0ac6455432995d3221f3",
                "name": "White Miso Paste",
                "gtin": "5012836756278",
                "image": "https://static.euw1.sustained.app/products/images/5012836756278_0.jpg",
                "pack": "100g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1f64bfb722bc0ac6455432995d3221f3",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1f64bfb722bc0ac6455432995d3221f3/impacts"
                }
            },
            {
                "id": "productid.206513529452285acf60ea43a7c6d605",
                "name": "Cajun Rub",
                "gtin": "6009657342246",
                "image": "https://static.euw1.sustained.app/products/images/6009657342246_0.jpg",
                "pack": "65g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.206513529452285acf60ea43a7c6d605",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.206513529452285acf60ea43a7c6d605/impacts"
                }
            },
            {
                "id": "productid.20ab5a317e9a28b7f997250ddc5fc5bf",
                "name": "Squash 10 Cubes",
                "gtin": "5060503501461",
                "image": "https://static.euw1.sustained.app/products/images/5060503501461_0.jpg",
                "pack": "200g",
                "grade": "A",
                "points": {
                    "pef": 7.250791486678962
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.20ab5a317e9a28b7f997250ddc5fc5bf",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.20ab5a317e9a28b7f997250ddc5fc5bf/impacts"
                }
            },
            {
                "id": "productid.20d1e34aa65a4922231d8f6967319a62",
                "name": "Kansas City Classic BBQ Sauce",
                "gtin": "0041500936420",
                "image": "https://static.euw1.sustained.app/products/images/0041500936420_0.jpg",
                "pack": "396g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Cooking Sauces",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.20d1e34aa65a4922231d8f6967319a62",
                    "category": "https://api.sustained.com/choice/v1/categories/cooking-sauces",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.20d1e34aa65a4922231d8f6967319a62/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.00085f32bb97e9322c9328916b17119c",
                "name": "Sausage Roll Flavour Crisps",
                "gtin": "5000328251479",
                "image": "https://static.euw1.sustained.app/products/images/5000328251479_0.jpg",
                "pack": "25g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00085f32bb97e9322c9328916b17119c",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00085f32bb97e9322c9328916b17119c/impacts"
                }
            },
            {
                "id": "productid.00286c45eee72aa7b6dd1d250dfdbd4b",
                "name": "Loaded Fries Seaside Salt & Vinegar Flavour",
                "gtin": "5016451343592",
                "image": "https://static.euw1.sustained.app/products/images/5016451343592_0.jpg",
                "pack": "110g",
                "grade": "D",
                "points": {
                    "pef": 26.1806512483015
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00286c45eee72aa7b6dd1d250dfdbd4b",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00286c45eee72aa7b6dd1d250dfdbd4b/impacts"
                }
            },
            {
                "id": "productid.004e8433903f3a9f74091c742dc48acc",
                "name": "Espress So & So Fruit, Nut & Choc Mix",
                "gtin": "5060422870167",
                "image": "https://static.euw1.sustained.app/products/images/5060422870167_0.jpg",
                "pack": "35g",
                "grade": "G",
                "points": {
                    "pef": 110.09285900335232
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.004e8433903f3a9f74091c742dc48acc",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.004e8433903f3a9f74091c742dc48acc/impacts"
                }
            },
            {
                "id": "productid.006f21924c52ad01b2c26d6d15076d8c",
                "name": "Smoky Paprika Nut Mix",
                "gtin": "5059512738500",
                "image": "https://static.euw1.sustained.app/products/images/5059512738500_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 90.97423757982614
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.006f21924c52ad01b2c26d6d15076d8c",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.006f21924c52ad01b2c26d6d15076d8c/impacts"
                }
            },
            {
                "id": "productid.008b65438c384347d9ef22d29c31c26d",
                "name": "Buffalo Chicken Wings Crispy Coated Peanuts",
                "gtin": "5060036016548",
                "image": "https://static.euw1.sustained.app/products/images/5060036016548_0.jpg",
                "pack": "60g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.008b65438c384347d9ef22d29c31c26d",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.008b65438c384347d9ef22d29c31c26d/impacts"
                }
            },
            {
                "id": "productid.011b98284c7f41bf2bbaea6dae04e1fc",
                "name": "Crunchy Roasted Fava Beans Lightly Salted Flavour",
                "gtin": "9318471000964",
                "image": "https://static.euw1.sustained.app/products/images/9318471000964_0.jpg",
                "pack": "25g",
                "grade": "C",
                "points": {
                    "pef": 19.01340815713412
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.011b98284c7f41bf2bbaea6dae04e1fc",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.011b98284c7f41bf2bbaea6dae04e1fc/impacts"
                }
            },
            {
                "id": "productid.013bffed79044b7fa9017beebc191df2",
                "name": "West Country Sour Cream & Chive Sourdough Bites",
                "gtin": "5060198820779",
                "image": "https://static.euw1.sustained.app/products/images/5060198820779_0.jpg",
                "pack": "90g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.013bffed79044b7fa9017beebc191df2",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.013bffed79044b7fa9017beebc191df2/impacts"
                }
            },
            {
                "id": "productid.016cfa5055de9468c8f41b0bade853e3",
                "name": "Oven Baked Cheese & Onion Multipack Snacks",
                "gtin": "5000328308012",
                "image": "https://static.euw1.sustained.app/products/images/5000328308012_0.jpg",
                "pack": "25g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.016cfa5055de9468c8f41b0bade853e3",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.016cfa5055de9468c8f41b0bade853e3/impacts"
                }
            },
            {
                "id": "productid.01abb7ab4abd7214569b886105bbb18f",
                "name": "Three Seed Mix",
                "gtin": "5057172859818",
                "image": "https://static.euw1.sustained.app/products/images/5057172859818_0.jpg",
                "pack": "150g",
                "grade": "D",
                "points": {
                    "pef": 25.27762755657446
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01abb7ab4abd7214569b886105bbb18f",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01abb7ab4abd7214569b886105bbb18f/impacts"
                }
            },
            {
                "id": "productid.026e7497eb0b71184ba0c24c821a5fd0",
                "name": "The Best Belgian Dark Chocolate & Mint Rice Cakes",
                "gtin": "5010251836094",
                "image": "https://static.euw1.sustained.app/products/images/5010251836094_0.jpg",
                "pack": "102g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.026e7497eb0b71184ba0c24c821a5fd0",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.026e7497eb0b71184ba0c24c821a5fd0/impacts"
                }
            },
            {
                "id": "productid.03463b41119aba24b9c9b5387309d502",
                "name": "Taste the Difference Indian Spiced Flatbread Chips",
                "gtin": "00332095",
                "image": "https://static.euw1.sustained.app/products/images/00332095_0.jpg",
                "pack": "150g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03463b41119aba24b9c9b5387309d502",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03463b41119aba24b9c9b5387309d502/impacts"
                }
            },
            {
                "id": "productid.037904cc67af01dc529ba518b732b6f6",
                "name": "Sea Salt & Vinegar Flavour Potato Snacks",
                "gtin": "5060292302508",
                "image": "https://static.euw1.sustained.app/products/images/5060292302508_0.jpg",
                "pack": "23g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.037904cc67af01dc529ba518b732b6f6",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.037904cc67af01dc529ba518b732b6f6/impacts"
                }
            },
            {
                "id": "productid.0431f81be53267c9fb29b3dbe54545fe",
                "name": "Bacon Flavour Bacon Fries",
                "gtin": "5011096009193",
                "image": "https://static.euw1.sustained.app/products/images/5011096009193_0.jpg",
                "pack": "85g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0431f81be53267c9fb29b3dbe54545fe",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0431f81be53267c9fb29b3dbe54545fe/impacts"
                }
            },
            {
                "id": "productid.0443cecd124bd3b597e396eae7e5c6a8",
                "name": "Sweet Chilli Flavour Crisps",
                "gtin": "5011096006963",
                "image": "https://static.euw1.sustained.app/products/images/5011096006963_0.jpg",
                "pack": "125g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0443cecd124bd3b597e396eae7e5c6a8",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0443cecd124bd3b597e396eae7e5c6a8/impacts"
                }
            },
            {
                "id": "productid.0488e385427b40aea0619eae918c7485",
                "name": "Crispy Smoked Bacon Flavour Potato Crisps",
                "gtin": "5010052113479",
                "image": "https://static.euw1.sustained.app/products/images/5010052113479_0.jpg",
                "pack": "65g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0488e385427b40aea0619eae918c7485",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0488e385427b40aea0619eae918c7485/impacts"
                }
            },
            {
                "id": "productid.04e2e992648446c8d28164f0533940b8",
                "name": "Hand Cooked Potato Crisp Seasoned with Sea Salt",
                "gtin": "5035336003826",
                "image": "https://static.euw1.sustained.app/products/images/5035336003826_0.jpg",
                "pack": "40g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04e2e992648446c8d28164f0533940b8",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04e2e992648446c8d28164f0533940b8/impacts"
                }
            },
            {
                "id": "productid.04ec978b476d972c9678fd53d1991e89",
                "name": "Taste the Difference Limited Edition Mac & Cheese Flavour Hand Cooked Potato Crisps",
                "gtin": "0000000432542",
                "image": "https://static.euw1.sustained.app/products/images/0000000432542_0.jpg",
                "pack": "150g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04ec978b476d972c9678fd53d1991e89",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04ec978b476d972c9678fd53d1991e89/impacts"
                }
            },
            {
                "id": "productid.053f8788eef6502a6079ffb0c5290c03",
                "name": "Banshee Bones Salt & Vinegar 12 pack flashed \u20ac3",
                "gtin": "5011096009940",
                "image": "https://static.euw1.sustained.app/products/images/5011096009940_0.jpg",
                "pack": "17g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.053f8788eef6502a6079ffb0c5290c03",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.053f8788eef6502a6079ffb0c5290c03/impacts"
                }
            },
            {
                "id": "productid.05a42ad73f44408e72b247d77ae6cfc9",
                "name": "Tangy Tomato Flavour Potato Crisps",
                "gtin": "5039378000935",
                "image": "https://static.euw1.sustained.app/products/images/5039378000935_0.jpg",
                "pack": "40g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05a42ad73f44408e72b247d77ae6cfc9",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05a42ad73f44408e72b247d77ae6cfc9/impacts"
                }
            },
            {
                "id": "productid.05b2b911ab1ab6d3ec3155e910c9e6fe",
                "name": "Sweet BBQ Flavour Popped Crisps",
                "gtin": "5000328015644",
                "image": "https://static.euw1.sustained.app/products/images/5000328015644_0.jpg",
                "pack": "28g",
                "grade": "D",
                "points": {
                    "pef": 22.769038504290773
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05b2b911ab1ab6d3ec3155e910c9e6fe",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05b2b911ab1ab6d3ec3155e910c9e6fe/impacts"
                }
            },
            {
                "id": "productid.060d54de226cdd3e88c4d7d65dec03cc",
                "name": "Peanut Butter Popcorn",
                "gtin": "0609132436024",
                "image": "https://static.euw1.sustained.app/products/images/0609132436024_0.jpg",
                "pack": "80g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.060d54de226cdd3e88c4d7d65dec03cc",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.060d54de226cdd3e88c4d7d65dec03cc/impacts"
                }
            },
            {
                "id": "productid.066cf8c617d76ea549d6f6412873e87a",
                "name": "Organic Wholegrain Rice Cakes No Added Salt",
                "gtin": "5021554000242",
                "image": "https://static.euw1.sustained.app/products/images/5021554000242_0.jpg",
                "pack": "130g",
                "grade": "D",
                "points": {
                    "pef": 31.227682434520275
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.066cf8c617d76ea549d6f6412873e87a",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.066cf8c617d76ea549d6f6412873e87a/impacts"
                }
            },
            {
                "id": "productid.06a3a270bcde7beaad0ad6d99da3c80b",
                "name": "Chilli and Lime",
                "gtin": "5060536110678",
                "image": "https://static.euw1.sustained.app/products/images/5060536110678_0.jpg",
                "pack": "80g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06a3a270bcde7beaad0ad6d99da3c80b",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06a3a270bcde7beaad0ad6d99da3c80b/impacts"
                }
            },
            {
                "id": "productid.0748760ecc198445b8ff1df3c5a14d0d",
                "name": "Snaps Spicy Tomato Multipack Snacks",
                "gtin": "5000328514376",
                "image": "https://static.euw1.sustained.app/products/images/5000328514376_0.jpg",
                "pack": "13g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0748760ecc198445b8ff1df3c5a14d0d",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0748760ecc198445b8ff1df3c5a14d0d/impacts"
                }
            },
            {
                "id": "productid.077016c61d2561c4cf431c54935274ea",
                "name": "Texas BBQ Sauce Crisps",
                "gtin": "5053990168521",
                "image": "https://static.euw1.sustained.app/products/images/5053990168521_0.jpg",
                "pack": "165g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.077016c61d2561c4cf431c54935274ea",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.077016c61d2561c4cf431c54935274ea/impacts"
                }
            },
            {
                "id": "productid.077996478e8a485e45de24168b6f5faf",
                "name": "Choccy Apricot",
                "gtin": "5060013263088",
                "image": "https://static.euw1.sustained.app/products/images/5060013263088_0.jpg",
                "pack": "475g",
                "grade": "G",
                "points": {
                    "pef": 111.38014742485761
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.077996478e8a485e45de24168b6f5faf",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.077996478e8a485e45de24168b6f5faf/impacts"
                }
            },
            {
                "id": "productid.07904802c4de607c125d3e68d6c12b12",
                "name": "Food Keralan Curry Dhal Chips",
                "gtin": "0000000377690",
                "image": "https://static.euw1.sustained.app/products/images/0000000377690_0.jpg",
                "pack": "150g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07904802c4de607c125d3e68d6c12b12",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07904802c4de607c125d3e68d6c12b12/impacts"
                }
            },
            {
                "id": "productid.09b5e1c2bb61c4fb901116a37cd21912",
                "name": "Footballs Cheese Multipack Snacks",
                "gtin": "5000328268002",
                "image": "https://static.euw1.sustained.app/products/images/5000328268002_0.jpg",
                "pack": "340g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09b5e1c2bb61c4fb901116a37cd21912",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09b5e1c2bb61c4fb901116a37cd21912/impacts"
                }
            },
            {
                "id": "productid.09c7352d1136e6bbf38fac8a4b59f9d4",
                "name": "Ridge Cut Salted Flavour Potato Crisps",
                "gtin": "5000237138564",
                "image": "https://static.euw1.sustained.app/products/images/5000237138564_0.jpg",
                "pack": "45g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09c7352d1136e6bbf38fac8a4b59f9d4",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09c7352d1136e6bbf38fac8a4b59f9d4/impacts"
                }
            },
            {
                "id": "productid.0a1f6aeaaba8503033e623e64290fae6",
                "name": "Cheese & Onion Crisps",
                "gtin": "5057753195847",
                "image": "https://static.euw1.sustained.app/products/images/5057753195847_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 28.958249836205717
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a1f6aeaaba8503033e623e64290fae6",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a1f6aeaaba8503033e623e64290fae6/impacts"
                }
            },
            {
                "id": "productid.0aa225d5019bbdbb04aad7c5d96a1b05",
                "name": "4 Cheese & Bean Bakes",
                "gtin": "0000000384469",
                "image": "https://static.euw1.sustained.app/products/images/0000000384469_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0aa225d5019bbdbb04aad7c5d96a1b05",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0aa225d5019bbdbb04aad7c5d96a1b05/impacts"
                }
            },
            {
                "id": "productid.0b442a673f3b8e244d404e17b80ab2ae",
                "name": "Party Pack Mature Cheese & Red Onion Crisps",
                "gtin": "5011096009537",
                "image": "https://static.euw1.sustained.app/products/images/5011096009537_0.jpg",
                "pack": "170g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b442a673f3b8e244d404e17b80ab2ae",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b442a673f3b8e244d404e17b80ab2ae/impacts"
                }
            },
            {
                "id": "productid.0b517a536774b61ccfe6031a9cc3996b",
                "name": "Stephenson's Bakery 6 Chocolate Mini Rolls",
                "gtin": "5010251427780",
                "image": "https://static.euw1.sustained.app/products/images/5010251427780_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b517a536774b61ccfe6031a9cc3996b",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b517a536774b61ccfe6031a9cc3996b/impacts"
                }
            },
            {
                "id": "productid.0b7bc23eae71d7546c4619d3b660ae04",
                "name": "Oven Baked with Veg Beetroot & Sweet Chilli Jam Snacks",
                "gtin": "5000328137247",
                "image": "https://static.euw1.sustained.app/products/images/5000328137247_0.jpg",
                "pack": "35g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b7bc23eae71d7546c4619d3b660ae04",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b7bc23eae71d7546c4619d3b660ae04/impacts"
                }
            },
            {
                "id": "productid.0bb7354b7cc3129b16eae97b28466042",
                "name": "Mixed Nuts",
                "gtin": "5000234053310",
                "image": "https://static.euw1.sustained.app/products/images/5000234053310_0.jpg",
                "pack": "40g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bb7354b7cc3129b16eae97b28466042",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bb7354b7cc3129b16eae97b28466042/impacts"
                }
            },
            {
                "id": "productid.0c1bc39244940a737741675e9e1cb85a",
                "name": "6 Ridged Cut Flavour Crisps",
                "gtin": "5054781699453",
                "image": "https://static.euw1.sustained.app/products/images/5054781699453_0.jpg",
                "pack": "27g",
                "grade": "D",
                "points": {
                    "pef": 28.958249836205717
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c1bc39244940a737741675e9e1cb85a",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c1bc39244940a737741675e9e1cb85a/impacts"
                }
            },
            {
                "id": "productid.0c29618891694109e4fb9bb09a904d80",
                "name": "Sweet Chipotle Chilli Flavour Popped Crisps",
                "gtin": "5000328017082",
                "image": "https://static.euw1.sustained.app/products/images/5000328017082_0.jpg",
                "pack": "28g",
                "grade": "D",
                "points": {
                    "pef": 24.1139893662245
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c29618891694109e4fb9bb09a904d80",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c29618891694109e4fb9bb09a904d80/impacts"
                }
            },
            {
                "id": "productid.0c62040da884943064cf2bc7b0cd9d0d",
                "name": "Fruit Snacks Banana Chips",
                "gtin": "5000328232737",
                "image": "https://static.euw1.sustained.app/products/images/5000328232737_0.jpg",
                "pack": "24g",
                "grade": "A",
                "points": {
                    "pef": 9.48529019471244
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c62040da884943064cf2bc7b0cd9d0d",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c62040da884943064cf2bc7b0cd9d0d/impacts"
                }
            },
            {
                "id": "productid.0cc048e3b2d40e5929ba8b31bbb882dd",
                "name": "Food Made Without Wheat Salt & Pepper Lentil Crackers",
                "gtin": "0000000891585",
                "image": "https://static.euw1.sustained.app/products/images/0000000891585_0.jpg",
                "pack": "150g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0cc048e3b2d40e5929ba8b31bbb882dd",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0cc048e3b2d40e5929ba8b31bbb882dd/impacts"
                }
            },
            {
                "id": "productid.0d3016118b795ef80392897893ab9676",
                "name": "Naked Oven Chips Straight",
                "gtin": "5010228002170",
                "image": "https://static.euw1.sustained.app/products/images/5010228002170_0.jpg",
                "pack": "2.25kg",
                "grade": "A",
                "points": {
                    "pef": 8.182594139214448
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d3016118b795ef80392897893ab9676",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d3016118b795ef80392897893ab9676/impacts"
                }
            },
            {
                "id": "productid.0d4ad4a5bdc6efadb1ec2bcb43b12918",
                "name": "Fish 'n' Chips Lashings of Salt & Vinegar Flavour Baked Snacks",
                "gtin": "0072417196973",
                "image": "https://static.euw1.sustained.app/products/images/0072417196973_0.jpg",
                "pack": "40g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d4ad4a5bdc6efadb1ec2bcb43b12918",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d4ad4a5bdc6efadb1ec2bcb43b12918/impacts"
                }
            },
            {
                "id": "productid.0dd707da805028adbc2daa1af8a6632c",
                "name": "Crinkle Crisps Cream Cheese and Chives Flavour",
                "gtin": "5016451062172",
                "image": "https://static.euw1.sustained.app/products/images/5016451062172_0.jpg",
                "pack": "25g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0dd707da805028adbc2daa1af8a6632c",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0dd707da805028adbc2daa1af8a6632c/impacts"
                }
            },
            {
                "id": "productid.0e5a269c0fddb4ff142243bca949215c",
                "name": "Funyuns Sharing Onion Rings",
                "gtin": "5000328261539",
                "image": "https://static.euw1.sustained.app/products/images/5000328261539_0.jpg",
                "pack": "125g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e5a269c0fddb4ff142243bca949215c",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e5a269c0fddb4ff142243bca949215c/impacts"
                }
            },
            {
                "id": "productid.0f5dd2a95f01414e08a7656af7243709",
                "name": "- Sea Salted Mix",
                "gtin": "5060530210305",
                "image": "https://static.euw1.sustained.app/products/images/5060530210305_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 78.78357434447878
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f5dd2a95f01414e08a7656af7243709",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f5dd2a95f01414e08a7656af7243709/impacts"
                }
            },
            {
                "id": "productid.0f7964f7b7532e40d5fdf7b8ba2151cd",
                "name": "Saucers Sour Cream & Onion Flavour Snacks",
                "gtin": "5010052112540",
                "image": "https://static.euw1.sustained.app/products/images/5010052112540_0.jpg",
                "pack": "65g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f7964f7b7532e40d5fdf7b8ba2151cd",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f7964f7b7532e40d5fdf7b8ba2151cd/impacts"
                }
            },
            {
                "id": "productid.0fdfa9f620962eed50648cfb47eeb71c",
                "name": "Masala Munch Sharing Snacks",
                "gtin": "5000328314822",
                "image": "https://static.euw1.sustained.app/products/images/5000328314822_0.jpg",
                "pack": "100g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fdfa9f620962eed50648cfb47eeb71c",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fdfa9f620962eed50648cfb47eeb71c/impacts"
                }
            },
            {
                "id": "productid.0fe928dca82f1cc6fe666329396fb830",
                "name": "Salt & Vinegar Rice Cakes",
                "gtin": "5010251519638",
                "image": "https://static.euw1.sustained.app/products/images/5010251519638_0.jpg",
                "pack": "129g",
                "grade": "D",
                "points": {
                    "pef": 29.323596695375
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fe928dca82f1cc6fe666329396fb830",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fe928dca82f1cc6fe666329396fb830/impacts"
                }
            },
            {
                "id": "productid.102204a3497ef99fc1ffb29cfd8e6078",
                "name": "Sour Cream & Chive Multipack Rice Cakes",
                "gtin": "5000328250762",
                "image": "https://static.euw1.sustained.app/products/images/5000328250762_0.jpg",
                "pack": "19g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.102204a3497ef99fc1ffb29cfd8e6078",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.102204a3497ef99fc1ffb29cfd8e6078/impacts"
                }
            },
            {
                "id": "productid.1025c991efa66d9372dbd5532592a613",
                "name": "Cheese & Onion Crisps",
                "gtin": "5000328698823",
                "image": "https://static.euw1.sustained.app/products/images/5000328698823_0.jpg",
                "pack": "32.5g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1025c991efa66d9372dbd5532592a613",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1025c991efa66d9372dbd5532592a613/impacts"
                }
            },
            {
                "id": "productid.10c1c127be9dccbe90f23827ca3a5e10",
                "name": "Roast Chicken Flavour Potato Crisps",
                "gtin": "5012159037962",
                "image": "https://static.euw1.sustained.app/products/images/5012159037962_0.jpg",
                "pack": "32.5g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Crisps & Snacks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10c1c127be9dccbe90f23827ca3a5e10",
                    "category": "https://api.sustained.com/choice/v1/categories/crisps-and-snacks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10c1c127be9dccbe90f23827ca3a5e10/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.0010e845ad4cf0763008a4fb9cb4db84",
                "name": "Cheese Coleslaw",
                "gtin": "5013377011086",
                "image": "https://static.euw1.sustained.app/products/images/5013377011086_0.jpg",
                "pack": "227g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0010e845ad4cf0763008a4fb9cb4db84",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0010e845ad4cf0763008a4fb9cb4db84/impacts"
                }
            },
            {
                "id": "productid.005ae724bb8dc4cb664e1e72ac272337",
                "name": "Food House Salad",
                "gtin": "0000000611251",
                "image": "https://static.euw1.sustained.app/products/images/0000000611251_0.jpg",
                "pack": "310g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.005ae724bb8dc4cb664e1e72ac272337",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.005ae724bb8dc4cb664e1e72ac272337/impacts"
                }
            },
            {
                "id": "productid.01044f4f9d204627d4fed43b1a48790c",
                "name": "Crumbed Ham 18 Slices",
                "gtin": "5057753895549",
                "image": "https://static.euw1.sustained.app/products/images/5057753895549_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 191.38927381473994
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01044f4f9d204627d4fed43b1a48790c",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01044f4f9d204627d4fed43b1a48790c/impacts"
                }
            },
            {
                "id": "productid.011fadff770c7abb1f7e237cb66eeea2",
                "name": "Finest Coleslaw",
                "gtin": "0000003330074",
                "image": "https://static.euw1.sustained.app/products/images/0000003330074_0.jpg",
                "pack": "300g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.011fadff770c7abb1f7e237cb66eeea2",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.011fadff770c7abb1f7e237cb66eeea2/impacts"
                }
            },
            {
                "id": "productid.01b109092cf9dbdf2fa68a561bc87147",
                "name": "5 Deli Ham Slices",
                "gtin": "5060055251852",
                "image": "https://static.euw1.sustained.app/products/images/5060055251852_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 126.60616487444538
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01b109092cf9dbdf2fa68a561bc87147",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01b109092cf9dbdf2fa68a561bc87147/impacts"
                }
            },
            {
                "id": "productid.01c127bc256974e177c8047ed28ff523",
                "name": "Global Grill Chilli Halloumi Skewers",
                "gtin": "5010525179841",
                "image": "https://static.euw1.sustained.app/products/images/5010525179841_0.jpg",
                "pack": "225g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01c127bc256974e177c8047ed28ff523",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01c127bc256974e177c8047ed28ff523/impacts"
                }
            },
            {
                "id": "productid.01c7f0817375024d36df6fcddee32424",
                "name": "Chicken and Bacon",
                "gtin": "5054781713883",
                "image": "https://static.euw1.sustained.app/products/images/5054781713883_0.jpg",
                "pack": "225g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01c7f0817375024d36df6fcddee32424",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01c7f0817375024d36df6fcddee32424/impacts"
                }
            },
            {
                "id": "productid.02eeba43e81be5db33dac8568a726d46",
                "name": "Smoked Ham & Mustard Mayo on Oatmeal Bread",
                "gtin": "5000169465776",
                "image": "https://static.euw1.sustained.app/products/images/5000169465776_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02eeba43e81be5db33dac8568a726d46",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02eeba43e81be5db33dac8568a726d46/impacts"
                }
            },
            {
                "id": "productid.031285cf65f26d81c3654bf4b803db56",
                "name": "Tabbouleh Salad",
                "gtin": "5000169603864",
                "image": "https://static.euw1.sustained.app/products/images/5000169603864_0.jpg",
                "pack": "185g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.031285cf65f26d81c3654bf4b803db56",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.031285cf65f26d81c3654bf4b803db56/impacts"
                }
            },
            {
                "id": "productid.035b3826f48bce62e926b1cec3e272c7",
                "name": "Vegan Coleslaw",
                "gtin": "5013377011727",
                "image": "https://static.euw1.sustained.app/products/images/5013377011727_0.jpg",
                "pack": "225g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.035b3826f48bce62e926b1cec3e272c7",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.035b3826f48bce62e926b1cec3e272c7/impacts"
                }
            },
            {
                "id": "productid.036234ee3a02abf28fc8ad5c61171c42",
                "name": "Classic Kabanos Chicken",
                "gtin": "5908230529917",
                "image": "https://static.euw1.sustained.app/products/images/5908230529917_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.036234ee3a02abf28fc8ad5c61171c42",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.036234ee3a02abf28fc8ad5c61171c42/impacts"
                }
            },
            {
                "id": "productid.0366271eb229f598a8aca0b2996d7722",
                "name": "On the Go Red Leicester Ploughmans",
                "gtin": "0000000237758",
                "image": "https://static.euw1.sustained.app/products/images/0000000237758_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0366271eb229f598a8aca0b2996d7722",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0366271eb229f598a8aca0b2996d7722/impacts"
                }
            },
            {
                "id": "productid.03a9232c938d865774e76f7042af1e30",
                "name": "Orange & Honey Roast Ham",
                "gtin": "5000169258125",
                "image": "https://static.euw1.sustained.app/products/images/5000169258125_0.jpg",
                "pack": "1.1kg",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03a9232c938d865774e76f7042af1e30",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03a9232c938d865774e76f7042af1e30/impacts"
                }
            },
            {
                "id": "productid.03cc0a1ba1e13700a9a2120adb1c44a0",
                "name": "Food Avocado, Feta & Grain Salad",
                "gtin": "0000000623643",
                "image": "https://static.euw1.sustained.app/products/images/0000000623643_0.jpg",
                "pack": "320g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03cc0a1ba1e13700a9a2120adb1c44a0",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03cc0a1ba1e13700a9a2120adb1c44a0/impacts"
                }
            },
            {
                "id": "productid.0406451e4c69bbd278f491718c552b29",
                "name": "Brunch Triple Sandwich",
                "gtin": "5010251658986",
                "image": "https://static.euw1.sustained.app/products/images/5010251658986_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0406451e4c69bbd278f491718c552b29",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0406451e4c69bbd278f491718c552b29/impacts"
                }
            },
            {
                "id": "productid.05b8ec9209f04917ac8b75f2ef5c18ce",
                "name": "Smokin' BBQ Free Range Pulled Pork, Bacon & Bourbon Patties",
                "gtin": "5060095862773",
                "image": "https://static.euw1.sustained.app/products/images/5060095862773_0.jpg",
                "pack": "120g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05b8ec9209f04917ac8b75f2ef5c18ce",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05b8ec9209f04917ac8b75f2ef5c18ce/impacts"
                }
            },
            {
                "id": "productid.05ee5c0e0eaecbbefcc52027d3a94e2e",
                "name": "Reduced Fat Coleslaw",
                "gtin": "5010251551720",
                "image": "https://static.euw1.sustained.app/products/images/5010251551720_0.jpg",
                "pack": "180g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05ee5c0e0eaecbbefcc52027d3a94e2e",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05ee5c0e0eaecbbefcc52027d3a94e2e/impacts"
                }
            },
            {
                "id": "productid.06a407506240c2cde82c917b0881d220",
                "name": "Food Collection Jamon De Bellota Iberico",
                "gtin": "0000000061629",
                "image": "https://static.euw1.sustained.app/products/images/0000000061629_0.jpg",
                "pack": "60g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06a407506240c2cde82c917b0881d220",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06a407506240c2cde82c917b0881d220/impacts"
                }
            },
            {
                "id": "productid.076928dfddf2ae3573be0fe7db34a5ae",
                "name": "Wiltshire Cured Ham Wafer Thin 6 Slices",
                "gtin": "5057545756195",
                "image": "https://static.euw1.sustained.app/products/images/5057545756195_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 191.38927381473994
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.076928dfddf2ae3573be0fe7db34a5ae",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.076928dfddf2ae3573be0fe7db34a5ae/impacts"
                }
            },
            {
                "id": "productid.076eb73e4f9b5ee5a86828823290a406",
                "name": "Chicken & Sweetcorn Pasta Salad",
                "gtin": "5057172341184",
                "image": "https://static.euw1.sustained.app/products/images/5057172341184_0.jpg",
                "pack": "500g",
                "grade": "D",
                "points": {
                    "pef": 30.937393713781738
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.076eb73e4f9b5ee5a86828823290a406",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.076eb73e4f9b5ee5a86828823290a406/impacts"
                }
            },
            {
                "id": "productid.077652cdcd4acc826ffffcc0e5aeae83",
                "name": "Greenside Deli Breaded Ham 15 Slices",
                "gtin": "5010525118314",
                "image": "https://static.euw1.sustained.app/products/images/5010525118314_0.jpg",
                "pack": "370g",
                "grade": "G",
                "points": {
                    "pef": 130.0635881620822
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.077652cdcd4acc826ffffcc0e5aeae83",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.077652cdcd4acc826ffffcc0e5aeae83/impacts"
                }
            },
            {
                "id": "productid.081950898988d6ea14cd921527ff6b51",
                "name": "Delicatessen Shaved Honey Roast Sliced Ham",
                "gtin": "5099338026429",
                "image": "https://static.euw1.sustained.app/products/images/5099338026429_0.jpg",
                "pack": "180g",
                "grade": "G",
                "points": {
                    "pef": 141.99543765510938
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.081950898988d6ea14cd921527ff6b51",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.081950898988d6ea14cd921527ff6b51/impacts"
                }
            },
            {
                "id": "productid.0841c238a832086c46c53b14c46b34b8",
                "name": "Gourmet Sliced Turkey Breast",
                "gtin": "5030450713910",
                "image": "https://static.euw1.sustained.app/products/images/5030450713910_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0841c238a832086c46c53b14c46b34b8",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0841c238a832086c46c53b14c46b34b8/impacts"
                }
            },
            {
                "id": "productid.094c48cfcc706dc2b8589804fde5f0b2",
                "name": "Tomato & Basil Chicken Pasta Salad",
                "gtin": "5057172567904",
                "image": "https://static.euw1.sustained.app/products/images/5057172567904_0.jpg",
                "pack": "230g",
                "grade": "E",
                "points": {
                    "pef": 32.46628472329349
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.094c48cfcc706dc2b8589804fde5f0b2",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.094c48cfcc706dc2b8589804fde5f0b2/impacts"
                }
            },
            {
                "id": "productid.096511975ffaee366f8ba9c232960001",
                "name": "The Collection Ib\u00e9rico Ham",
                "gtin": "0000000027304",
                "image": "https://static.euw1.sustained.app/products/images/0000000027304_0.jpg",
                "pack": "70g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.096511975ffaee366f8ba9c232960001",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.096511975ffaee366f8ba9c232960001/impacts"
                }
            },
            {
                "id": "productid.09cbc4daddbb026264de0b0d80c9e710",
                "name": "Macsween Haggis",
                "gtin": "5026018000271",
                "image": "https://static.euw1.sustained.app/products/images/5026018000271_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 280.4483401118093
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09cbc4daddbb026264de0b0d80c9e710",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09cbc4daddbb026264de0b0d80c9e710/impacts"
                }
            },
            {
                "id": "productid.0b105a4e019f65eb96076b9f9ebfc2e9",
                "name": "Sainsbury's Coleslaw",
                "gtin": "0000000413664",
                "image": "https://static.euw1.sustained.app/products/images/0000000413664_0.jpg",
                "pack": "300g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b105a4e019f65eb96076b9f9ebfc2e9",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b105a4e019f65eb96076b9f9ebfc2e9/impacts"
                }
            },
            {
                "id": "productid.0b1fff5815646dfd32559039f78d6a41",
                "name": "Chicken Tomato & Basil Pasta Meal Deal",
                "gtin": "5057753653774",
                "image": "https://static.euw1.sustained.app/products/images/5057753653774_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 31.689515767636028
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b1fff5815646dfd32559039f78d6a41",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b1fff5815646dfd32559039f78d6a41/impacts"
                }
            },
            {
                "id": "productid.0b5de7c62ddda9c79e186ff16e3afee0",
                "name": "Cooked Ham Wafer Thin",
                "gtin": "5054781710899",
                "image": "https://static.euw1.sustained.app/products/images/5054781710899_0.jpg",
                "pack": "420g",
                "grade": "G",
                "points": {
                    "pef": 119.8206893628237
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b5de7c62ddda9c79e186ff16e3afee0",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b5de7c62ddda9c79e186ff16e3afee0/impacts"
                }
            },
            {
                "id": "productid.0ba345c7ba88b81c600bca4111179e8a",
                "name": "British Cooked Beef 10 Slices",
                "gtin": "01687651",
                "image": "https://static.euw1.sustained.app/products/images/01687651_0.jpg",
                "pack": "115g",
                "grade": "G",
                "points": {
                    "pef": 243.01383905668388
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ba345c7ba88b81c600bca4111179e8a",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ba345c7ba88b81c600bca4111179e8a/impacts"
                }
            },
            {
                "id": "productid.0c10b4c047cddfc171486a57b3dd42b3",
                "name": "Italian Ham Crudo",
                "gtin": "8019063000041",
                "image": "https://static.euw1.sustained.app/products/images/8019063000041_0.jpg",
                "pack": "0.100kg",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c10b4c047cddfc171486a57b3dd42b3",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c10b4c047cddfc171486a57b3dd42b3/impacts"
                }
            },
            {
                "id": "productid.0c170b2445f33c8fab7632ef3e34361c",
                "name": "Cooked Ham 16 Slices",
                "gtin": "5010251889847",
                "image": "https://static.euw1.sustained.app/products/images/5010251889847_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 127.73358584298731
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c170b2445f33c8fab7632ef3e34361c",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c170b2445f33c8fab7632ef3e34361c/impacts"
                }
            },
            {
                "id": "productid.0cf7eed4ba04006c9478e0574f4bdc06",
                "name": "Haslet 10 Slices",
                "gtin": "5010251715467",
                "image": "https://static.euw1.sustained.app/products/images/5010251715467_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 122.51060572332983
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0cf7eed4ba04006c9478e0574f4bdc06",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0cf7eed4ba04006c9478e0574f4bdc06/impacts"
                }
            },
            {
                "id": "productid.0d4c48be59fe5d3c095836ff7c529cfd",
                "name": "Food Our Best Ever Scottish Smoked Salmon Pate",
                "gtin": "0000029034437",
                "image": "https://static.euw1.sustained.app/products/images/0000029034437_0.jpg",
                "pack": "120g",
                "grade": "G",
                "points": {
                    "pef": 95.05515278860302
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d4c48be59fe5d3c095836ff7c529cfd",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d4c48be59fe5d3c095836ff7c529cfd/impacts"
                }
            },
            {
                "id": "productid.0d614e61568865cd5e24f28439670f62",
                "name": "Family Pack Thinly Sliced Irish Ham",
                "gtin": "5391512926106",
                "image": "https://static.euw1.sustained.app/products/images/5391512926106_0.jpg",
                "pack": "140g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d614e61568865cd5e24f28439670f62",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d614e61568865cd5e24f28439670f62/impacts"
                }
            },
            {
                "id": "productid.0dd87531cf255fba9d4fb03db595bc7a",
                "name": "The Best Roast Chicken & Bacon Club",
                "gtin": "5010525203348",
                "image": "https://static.euw1.sustained.app/products/images/5010525203348_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0dd87531cf255fba9d4fb03db595bc7a",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0dd87531cf255fba9d4fb03db595bc7a/impacts"
                }
            },
            {
                "id": "productid.0e27831fde8b4925352281dc397c5769",
                "name": "Food Collection Whole Truffle Ham",
                "gtin": "0000029103492",
                "image": "https://static.euw1.sustained.app/products/images/0000029103492_0.jpg",
                "pack": "1kg",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e27831fde8b4925352281dc397c5769",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e27831fde8b4925352281dc397c5769/impacts"
                }
            },
            {
                "id": "productid.0e3cab90c5586597e0d44f813d4bc93c",
                "name": "On the Go Brie & Cranberry with Rocket and Black Pepper",
                "gtin": "0000000258913",
                "image": "https://static.euw1.sustained.app/products/images/0000000258913_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e3cab90c5586597e0d44f813d4bc93c",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e3cab90c5586597e0d44f813d4bc93c/impacts"
                }
            },
            {
                "id": "productid.0ed1b4953ae52f4c46e69f5f1308dbcb",
                "name": "Steak Slices",
                "gtin": "5060534670525",
                "image": "https://static.euw1.sustained.app/products/images/5060534670525_0.jpg",
                "pack": "50g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ed1b4953ae52f4c46e69f5f1308dbcb",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ed1b4953ae52f4c46e69f5f1308dbcb/impacts"
                }
            },
            {
                "id": "productid.0fd0d7ac16f254ce7d28de661633534f",
                "name": "Organic Wafer Thin Ham",
                "gtin": "5036557001271",
                "image": "https://static.euw1.sustained.app/products/images/5036557001271_0.jpg",
                "pack": "110g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fd0d7ac16f254ce7d28de661633534f",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fd0d7ac16f254ce7d28de661633534f/impacts"
                }
            },
            {
                "id": "productid.0fe83bdd224b586e4e8cdc7f84469b75",
                "name": "Taste the Difference Unsmoked British Outdoor Bred Gammon Joint",
                "gtin": "0000000373623",
                "image": "https://static.euw1.sustained.app/products/images/0000000373623_0.jpg",
                "pack": "1.3kg",
                "grade": "G",
                "points": {
                    "pef": 171.5901789387806
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fe83bdd224b586e4e8cdc7f84469b75",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fe83bdd224b586e4e8cdc7f84469b75/impacts"
                }
            },
            {
                "id": "productid.100ac0fb5be704b65b94aac651c53d05",
                "name": "The Deli On Market Street Cooked Sandwich Ham",
                "gtin": "5010525042978",
                "image": "https://static.euw1.sustained.app/products/images/5010525042978_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 134.61062572411936
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.100ac0fb5be704b65b94aac651c53d05",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.100ac0fb5be704b65b94aac651c53d05/impacts"
                }
            },
            {
                "id": "productid.1036471b55119521a95c57722cfebb7f",
                "name": "Salmon & Cucumber Meal Deal",
                "gtin": "03034316",
                "image": "https://static.euw1.sustained.app/products/images/03034316_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1036471b55119521a95c57722cfebb7f",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1036471b55119521a95c57722cfebb7f/impacts"
                }
            },
            {
                "id": "productid.10ef647681bc3f9de4e0446434bcbbf0",
                "name": "Cook It Cheese & Bacon Chicken Breast Fillets",
                "gtin": "5010251718710",
                "image": "https://static.euw1.sustained.app/products/images/5010251718710_0.jpg",
                "pack": "395g",
                "grade": "F",
                "points": {
                    "pef": 85.86258543164456
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10ef647681bc3f9de4e0446434bcbbf0",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10ef647681bc3f9de4e0446434bcbbf0/impacts"
                }
            },
            {
                "id": "productid.11b857fb0d6e3c8f6fc90e63df5fec1c",
                "name": "Old Style Cooked Ham",
                "gtin": "2058709000000",
                "image": "https://static.euw1.sustained.app/products/images/2058709000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 191.38927381473994
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11b857fb0d6e3c8f6fc90e63df5fec1c",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11b857fb0d6e3c8f6fc90e63df5fec1c/impacts"
                }
            },
            {
                "id": "productid.121eb7d7732cad06e2ca13528bac9a5e",
                "name": "Christmas Chicken Liver P\u00e2t\u00e9",
                "gtin": "5000169303016",
                "image": "https://static.euw1.sustained.app/products/images/5000169303016_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.121eb7d7732cad06e2ca13528bac9a5e",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.121eb7d7732cad06e2ca13528bac9a5e/impacts"
                }
            },
            {
                "id": "productid.122fdc9fbaeb1f86f45d039e8e457a8a",
                "name": "Organic Sauerkraut",
                "gtin": "5032722301160",
                "image": "https://static.euw1.sustained.app/products/images/5032722301160_0.jpg",
                "pack": "360g",
                "grade": "A",
                "points": {
                    "pef": 6.643212560955255
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.122fdc9fbaeb1f86f45d039e8e457a8a",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.122fdc9fbaeb1f86f45d039e8e457a8a/impacts"
                }
            },
            {
                "id": "productid.1243e829970ec90c4ff2d82a48b805f9",
                "name": "Hot & Spicy Chicken Slices",
                "gtin": "5010251638452",
                "image": "https://static.euw1.sustained.app/products/images/5010251638452_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1243e829970ec90c4ff2d82a48b805f9",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1243e829970ec90c4ff2d82a48b805f9/impacts"
                }
            },
            {
                "id": "productid.125d7e02b95ec701265d7e3160a3f558",
                "name": "Deli Style Coleslaw",
                "gtin": "0000000378253",
                "image": "https://static.euw1.sustained.app/products/images/0000000378253_0.jpg",
                "pack": "300g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.125d7e02b95ec701265d7e3160a3f558",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.125d7e02b95ec701265d7e3160a3f558/impacts"
                }
            },
            {
                "id": "productid.12e0de7f625e9376aea5b3e3ca5431a1",
                "name": "Beef & Haggis Olives",
                "gtin": "0247380000000",
                "image": "https://static.euw1.sustained.app/products/images/0247380000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 396.95209916330606
                },
                "info_icons": [],
                "category": "Deli",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12e0de7f625e9376aea5b3e3ca5431a1",
                    "category": "https://api.sustained.com/choice/v1/categories/deli",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12e0de7f625e9376aea5b3e3ca5431a1/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.00698d4381b5a0c5440b1df22dc70dfb",
                "name": "Asda Cranberries",
                "gtin": "5054781718512",
                "image": "https://static.euw1.sustained.app/products/images/5054781718512_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00698d4381b5a0c5440b1df22dc70dfb",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00698d4381b5a0c5440b1df22dc70dfb/impacts"
                }
            },
            {
                "id": "productid.00776bf82dc2562b250f0bcaad069d5d",
                "name": "Sunny Fruits Juicy Jumbo Raisin Mix",
                "gtin": "5000234044028",
                "image": "https://static.euw1.sustained.app/products/images/5000234044028_0.jpg",
                "pack": "25g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00776bf82dc2562b250f0bcaad069d5d",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00776bf82dc2562b250f0bcaad069d5d/impacts"
                }
            },
            {
                "id": "productid.018021cba027e54d067638f13147eff9",
                "name": "Flame Raisins",
                "gtin": "5000169218372",
                "image": "https://static.euw1.sustained.app/products/images/5000169218372_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.018021cba027e54d067638f13147eff9",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.018021cba027e54d067638f13147eff9/impacts"
                }
            },
            {
                "id": "productid.0258563abc77f760816a52071c6c58bb",
                "name": "Organic Toasted Coconut Flakes",
                "gtin": "0779140007612",
                "image": "https://static.euw1.sustained.app/products/images/0779140007612_0.jpg",
                "pack": "100g",
                "grade": "D",
                "points": {
                    "pef": 23.77707938241796
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0258563abc77f760816a52071c6c58bb",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0258563abc77f760816a52071c6c58bb/impacts"
                }
            },
            {
                "id": "productid.0284a80ae65580085791d5d5fca593bf",
                "name": "Food Organic Hazelnuts",
                "gtin": "0000029088614",
                "image": "https://static.euw1.sustained.app/products/images/0000029088614_0.jpg",
                "pack": "150g",
                "grade": "D",
                "points": {
                    "pef": 29.7610119776575
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0284a80ae65580085791d5d5fca593bf",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0284a80ae65580085791d5d5fca593bf/impacts"
                }
            },
            {
                "id": "productid.02d49d39e8ebcb854969245fa4f7ac69",
                "name": "Food Natural Pecans",
                "gtin": "0000000333467",
                "image": "https://static.euw1.sustained.app/products/images/0000000333467_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 86.8671120329742
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02d49d39e8ebcb854969245fa4f7ac69",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02d49d39e8ebcb854969245fa4f7ac69/impacts"
                }
            },
            {
                "id": "productid.0342946f8eca3774627a79be31e0f74c",
                "name": "Tesco Pistachios",
                "gtin": "10001400",
                "image": "https://static.euw1.sustained.app/products/images/10001400_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 35.75999939824057
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0342946f8eca3774627a79be31e0f74c",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0342946f8eca3774627a79be31e0f74c/impacts"
                }
            },
            {
                "id": "productid.03838aaa9148544cec8cc71068945d71",
                "name": "Chia Seeds",
                "gtin": "5054781053002",
                "image": "https://static.euw1.sustained.app/products/images/5054781053002_0.jpg",
                "pack": "150g",
                "grade": "E",
                "points": {
                    "pef": 42.69907625616115
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03838aaa9148544cec8cc71068945d71",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03838aaa9148544cec8cc71068945d71/impacts"
                }
            },
            {
                "id": "productid.04e68318099200895ac39f7af572e051",
                "name": "Baked Truffle Nuts",
                "gtin": "5060310130786",
                "image": "https://static.euw1.sustained.app/products/images/5060310130786_0.jpg",
                "pack": "80g",
                "grade": "G",
                "points": {
                    "pef": 125.77379141551368
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04e68318099200895ac39f7af572e051",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04e68318099200895ac39f7af572e051/impacts"
                }
            },
            {
                "id": "productid.04ff3a24fc02150f4f5466a377bae12f",
                "name": "Pine Nuts Kernels Chinese",
                "gtin": "5060090200105",
                "image": "https://static.euw1.sustained.app/products/images/5060090200105_0.jpg",
                "pack": "1kg",
                "grade": "F",
                "points": {
                    "pef": 63.05972283404185
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04ff3a24fc02150f4f5466a377bae12f",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04ff3a24fc02150f4f5466a377bae12f/impacts"
                }
            },
            {
                "id": "productid.0585eb994b7103b9e3ad5a2286259dba",
                "name": "Gourmet Golden Raisins",
                "gtin": "5099809126542",
                "image": "https://static.euw1.sustained.app/products/images/5099809126542_0.jpg",
                "pack": "375g",
                "grade": "F",
                "points": {
                    "pef": 64.42668027423886
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0585eb994b7103b9e3ad5a2286259dba",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0585eb994b7103b9e3ad5a2286259dba/impacts"
                }
            },
            {
                "id": "productid.080b0da2659951d1e5174343f7ea03a3",
                "name": "Safawi Dates",
                "gtin": "5034525012779",
                "image": "https://static.euw1.sustained.app/products/images/5034525012779_0.jpg",
                "pack": "450g",
                "grade": "E",
                "points": {
                    "pef": 45.24410103585034
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.080b0da2659951d1e5174343f7ea03a3",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.080b0da2659951d1e5174343f7ea03a3/impacts"
                }
            },
            {
                "id": "productid.09b0638240a09abcdcc5449f82c14126",
                "name": "Power Pack Fruit & Nut",
                "gtin": "5022374042795",
                "image": "https://static.euw1.sustained.app/products/images/5022374042795_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 54.18291261592684
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09b0638240a09abcdcc5449f82c14126",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09b0638240a09abcdcc5449f82c14126/impacts"
                }
            },
            {
                "id": "productid.09d60e8660f0a526f1365a7f92f707a6",
                "name": "Itac Almond",
                "gtin": "8420063023716",
                "image": "https://static.euw1.sustained.app/products/images/8420063023716_0.jpg",
                "pack": "60g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09d60e8660f0a526f1365a7f92f707a6",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09d60e8660f0a526f1365a7f92f707a6/impacts"
                }
            },
            {
                "id": "productid.0a4bef0285c99612baa7f88481fc6f82",
                "name": "Food Soft Eating Apricots",
                "gtin": "0000000463072",
                "image": "https://static.euw1.sustained.app/products/images/0000000463072_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a4bef0285c99612baa7f88481fc6f82",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a4bef0285c99612baa7f88481fc6f82/impacts"
                }
            },
            {
                "id": "productid.0f62d798f9c4b259931ad6d16b5f667b",
                "name": "Organic Walnut Halves",
                "gtin": "5033729010178",
                "image": "https://static.euw1.sustained.app/products/images/5033729010178_0.jpg",
                "pack": "125g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f62d798f9c4b259931ad6d16b5f667b",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f62d798f9c4b259931ad6d16b5f667b/impacts"
                }
            },
            {
                "id": "productid.0feb4925e07df5eb011e5ab55ad406f9",
                "name": "Signature Fruit, Nuts & Seeds",
                "gtin": "5010251618188",
                "image": "https://static.euw1.sustained.app/products/images/5010251618188_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 92.24270844646541
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0feb4925e07df5eb011e5ab55ad406f9",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0feb4925e07df5eb011e5ab55ad406f9/impacts"
                }
            },
            {
                "id": "productid.10769667061935c83324335783e76831",
                "name": "Mixed Nuts & Raisins",
                "gtin": "5000362129154",
                "image": "https://static.euw1.sustained.app/products/images/5000362129154_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10769667061935c83324335783e76831",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10769667061935c83324335783e76831/impacts"
                }
            },
            {
                "id": "productid.108b65cfa8bc508ebc7d1604d6498d32",
                "name": "Food Soft Eating Prunes",
                "gtin": "0000000267564",
                "image": "https://static.euw1.sustained.app/products/images/0000000267564_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 136.55242901551372
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.108b65cfa8bc508ebc7d1604d6498d32",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.108b65cfa8bc508ebc7d1604d6498d32/impacts"
                }
            },
            {
                "id": "productid.11f19bf401a0e0f7b14a389b8ce371a5",
                "name": "Cashew Nuts",
                "gtin": "5054070710692",
                "image": "https://static.euw1.sustained.app/products/images/5054070710692_0.jpg",
                "pack": "150g",
                "grade": "F",
                "points": {
                    "pef": 71.78088079562058
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11f19bf401a0e0f7b14a389b8ce371a5",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11f19bf401a0e0f7b14a389b8ce371a5/impacts"
                }
            },
            {
                "id": "productid.125d66014e6d58a09002bb91c304fded",
                "name": "Just for Kids Squishy Raisins & Sultanas",
                "gtin": "5010251633624",
                "image": "https://static.euw1.sustained.app/products/images/5010251633624_0.jpg",
                "pack": "30g",
                "grade": "F",
                "points": {
                    "pef": 64.31919215670747
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.125d66014e6d58a09002bb91c304fded",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.125d66014e6d58a09002bb91c304fded/impacts"
                }
            },
            {
                "id": "productid.13d4e99373ec0a3036de61d19773e33d",
                "name": "Fruit Nut & Seed Medley",
                "gtin": "00325356",
                "image": "https://static.euw1.sustained.app/products/images/00325356_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 79.62907215602054
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.13d4e99373ec0a3036de61d19773e33d",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.13d4e99373ec0a3036de61d19773e33d/impacts"
                }
            },
            {
                "id": "productid.153015339967a4664ee083c081965a3f",
                "name": "Food Hickory BBQ Flavour Nut Mix",
                "gtin": "0000029097685",
                "image": "https://static.euw1.sustained.app/products/images/0000029097685_0.jpg",
                "pack": "300g",
                "grade": "G",
                "points": {
                    "pef": 105.03935801629754
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.153015339967a4664ee083c081965a3f",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.153015339967a4664ee083c081965a3f/impacts"
                }
            },
            {
                "id": "productid.165f86ac074f3e59c7d849b86e31a373",
                "name": "Pistachio Nut Kernels",
                "gtin": "5000169218310",
                "image": "https://static.euw1.sustained.app/products/images/5000169218310_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.165f86ac074f3e59c7d849b86e31a373",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.165f86ac074f3e59c7d849b86e31a373/impacts"
                }
            },
            {
                "id": "productid.17c2472ce7ae30c2503bb3b10b05f73b",
                "name": "Asda Figs",
                "gtin": "5054070630051",
                "image": "https://static.euw1.sustained.app/products/images/5054070630051_0.jpg",
                "pack": "500g",
                "grade": "C",
                "points": {
                    "pef": 16.817115232387568
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.17c2472ce7ae30c2503bb3b10b05f73b",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.17c2472ce7ae30c2503bb3b10b05f73b/impacts"
                }
            },
            {
                "id": "productid.18c21ee18bf22bd5d88fc3bac364da58",
                "name": "Shamrock Raisins",
                "gtin": "5011005055730",
                "image": "https://static.euw1.sustained.app/products/images/5011005055730_0.jpg",
                "pack": "350g",
                "grade": "F",
                "points": {
                    "pef": 64.42668027423886
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.18c21ee18bf22bd5d88fc3bac364da58",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.18c21ee18bf22bd5d88fc3bac364da58/impacts"
                }
            },
            {
                "id": "productid.18da00b9d92ad6e73c3d034e245e087f",
                "name": "Glac\u00e9 Ginger",
                "gtin": "0000001205732",
                "image": "https://static.euw1.sustained.app/products/images/0000001205732_0.jpg",
                "pack": "200g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.18da00b9d92ad6e73c3d034e245e087f",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.18da00b9d92ad6e73c3d034e245e087f/impacts"
                }
            },
            {
                "id": "productid.19b960535634158861ffdbe8cb2a5be7",
                "name": "The Greengrocer's On Market Street Brazils",
                "gtin": "5010251618058",
                "image": "https://static.euw1.sustained.app/products/images/5010251618058_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 63.019128922690854
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.19b960535634158861ffdbe8cb2a5be7",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.19b960535634158861ffdbe8cb2a5be7/impacts"
                }
            },
            {
                "id": "productid.1b1d2033add4f10481602fd9a1abe4f9",
                "name": "Organic Whole Almonds",
                "gtin": "5099809148704",
                "image": "https://static.euw1.sustained.app/products/images/5099809148704_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 208.26769747312414
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b1d2033add4f10481602fd9a1abe4f9",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b1d2033add4f10481602fd9a1abe4f9/impacts"
                }
            },
            {
                "id": "productid.1c844a5e3bb73f7941fb13e34e2d9cfe",
                "name": "Taste Buds Raisins",
                "gtin": "0000000599306",
                "image": "https://static.euw1.sustained.app/products/images/0000000599306_0.jpg",
                "pack": "14g",
                "grade": "F",
                "points": {
                    "pef": 64.47618354605218
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c844a5e3bb73f7941fb13e34e2d9cfe",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c844a5e3bb73f7941fb13e34e2d9cfe/impacts"
                }
            },
            {
                "id": "productid.1cf06cd60b7ae12efc81dd79f573af29",
                "name": "Organic Pine Nuts",
                "gtin": "5012246516882",
                "image": "https://static.euw1.sustained.app/products/images/5012246516882_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 63.05972283404185
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1cf06cd60b7ae12efc81dd79f573af29",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1cf06cd60b7ae12efc81dd79f573af29/impacts"
                }
            },
            {
                "id": "productid.1da7b6638c716ad7f9774f0a1d66f2d4",
                "name": "Mixed Nuts",
                "gtin": "0000010112298",
                "image": "https://static.euw1.sustained.app/products/images/0000010112298_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 90.97423757982614
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1da7b6638c716ad7f9774f0a1d66f2d4",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1da7b6638c716ad7f9774f0a1d66f2d4/impacts"
                }
            },
            {
                "id": "productid.1e9fac505e14d0b64462cb3933377bad",
                "name": "Strawberry Infused Apple Pieces",
                "gtin": "5059697390319",
                "image": "https://static.euw1.sustained.app/products/images/5059697390319_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.656019205950824
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e9fac505e14d0b64462cb3933377bad",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e9fac505e14d0b64462cb3933377bad/impacts"
                }
            },
            {
                "id": "productid.23c4d3a6694c12b07dc681d9b904a8a7",
                "name": "Mixed Peel",
                "gtin": "3612645429503",
                "image": "https://static.euw1.sustained.app/products/images/3612645429503_0.jpg",
                "pack": "100g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.23c4d3a6694c12b07dc681d9b904a8a7",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.23c4d3a6694c12b07dc681d9b904a8a7/impacts"
                }
            },
            {
                "id": "productid.24bc5d45b48a78e73fad8bbd82d40473",
                "name": "Organic Brazil Nuts",
                "gtin": "5059697403989",
                "image": "https://static.euw1.sustained.app/products/images/5059697403989_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 63.019128922690854
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.24bc5d45b48a78e73fad8bbd82d40473",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.24bc5d45b48a78e73fad8bbd82d40473/impacts"
                }
            },
            {
                "id": "productid.254bc2bec0a270ef534e5dbf651ec07a",
                "name": "Chewy Banana",
                "gtin": "5059697256820",
                "image": "https://static.euw1.sustained.app/products/images/5059697256820_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 9.48529019471244
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.254bc2bec0a270ef534e5dbf651ec07a",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.254bc2bec0a270ef534e5dbf651ec07a/impacts"
                }
            },
            {
                "id": "productid.255b426e657d79643c4a60fd6bc45e27",
                "name": "LoveLife Soft Apricots",
                "gtin": "5000169164914",
                "image": "https://static.euw1.sustained.app/products/images/5000169164914_0.jpg",
                "pack": "30g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.255b426e657d79643c4a60fd6bc45e27",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.255b426e657d79643c4a60fd6bc45e27/impacts"
                }
            },
            {
                "id": "productid.25d93fbd82cc2bb61c0be49ca105dc10",
                "name": "Stoned Sayer Dates",
                "gtin": "5000234049795",
                "image": "https://static.euw1.sustained.app/products/images/5000234049795_0.jpg",
                "pack": "300g",
                "grade": "E",
                "points": {
                    "pef": 45.35792616628001
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.25d93fbd82cc2bb61c0be49ca105dc10",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.25d93fbd82cc2bb61c0be49ca105dc10/impacts"
                }
            },
            {
                "id": "productid.26e374c1fb484de031e18d04f28ee9c2",
                "name": "Organic Sunflower Seeds",
                "gtin": "5012246214283",
                "image": "https://static.euw1.sustained.app/products/images/5012246214283_0.jpg",
                "pack": "100g",
                "grade": "A",
                "points": {
                    "pef": 10.588192970268036
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.26e374c1fb484de031e18d04f28ee9c2",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.26e374c1fb484de031e18d04f28ee9c2/impacts"
                }
            },
            {
                "id": "productid.270e064ed29925676af23abc50cd9efc",
                "name": "Blanched Hazelnuts",
                "gtin": "5000169153451",
                "image": "https://static.euw1.sustained.app/products/images/5000169153451_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.270e064ed29925676af23abc50cd9efc",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.270e064ed29925676af23abc50cd9efc/impacts"
                }
            },
            {
                "id": "productid.27e01d8c513d69f27e595b78b2f4096e",
                "name": "Sun Exotics Prunes",
                "gtin": "5390193600220",
                "image": "https://static.euw1.sustained.app/products/images/5390193600220_0.jpg",
                "pack": "275g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.27e01d8c513d69f27e595b78b2f4096e",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.27e01d8c513d69f27e595b78b2f4096e/impacts"
                }
            },
            {
                "id": "productid.2967933cde1912bd70befbc56813070e",
                "name": "On the Go Almonds",
                "gtin": "00370547",
                "image": "https://static.euw1.sustained.app/products/images/00370547_0.jpg",
                "pack": "25g",
                "grade": "G",
                "points": {
                    "pef": 208.26769747312414
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2967933cde1912bd70befbc56813070e",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2967933cde1912bd70befbc56813070e/impacts"
                }
            },
            {
                "id": "productid.299c689cf4bf4f9e17defcde74189beb",
                "name": "Food Blanched Hazelnuts",
                "gtin": "0000029157549",
                "image": "https://static.euw1.sustained.app/products/images/0000029157549_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 62.31255490348775
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.299c689cf4bf4f9e17defcde74189beb",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.299c689cf4bf4f9e17defcde74189beb/impacts"
                }
            },
            {
                "id": "productid.299f650567762164adb55f46e1ff5001",
                "name": "Honey Roasted Cashew Nuts",
                "gtin": "5000169630747",
                "image": "https://static.euw1.sustained.app/products/images/5000169630747_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 64.0034320246787
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.299f650567762164adb55f46e1ff5001",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.299f650567762164adb55f46e1ff5001/impacts"
                }
            },
            {
                "id": "productid.29feb882ecd4e42722bc008cab4a693d",
                "name": "Organic Cashew Nuts",
                "gtin": "5059697398957",
                "image": "https://static.euw1.sustained.app/products/images/5059697398957_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 71.78088079562058
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.29feb882ecd4e42722bc008cab4a693d",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.29feb882ecd4e42722bc008cab4a693d/impacts"
                }
            },
            {
                "id": "productid.2a13a1e45b441d575413939ca7514f49",
                "name": "Organic Mixed Nuts",
                "gtin": "10063972",
                "image": "https://static.euw1.sustained.app/products/images/10063972_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a13a1e45b441d575413939ca7514f49",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a13a1e45b441d575413939ca7514f49/impacts"
                }
            },
            {
                "id": "productid.2a4fe51e9738f350326dc7e29e77e32d",
                "name": "Nut & Fruit Mix",
                "gtin": "5000169625927",
                "image": "https://static.euw1.sustained.app/products/images/5000169625927_0.jpg",
                "pack": "60g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a4fe51e9738f350326dc7e29e77e32d",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a4fe51e9738f350326dc7e29e77e32d/impacts"
                }
            },
            {
                "id": "productid.2afd9c1d2e0deee7094eb011eea9e343",
                "name": "Just Essentials Sultanas",
                "gtin": "0000021114694",
                "image": "https://static.euw1.sustained.app/products/images/0000021114694_0.jpg",
                "pack": "500g",
                "grade": "F",
                "points": {
                    "pef": 64.37717700242554
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2afd9c1d2e0deee7094eb011eea9e343",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2afd9c1d2e0deee7094eb011eea9e343/impacts"
                }
            },
            {
                "id": "productid.2b9569e8e7ca0033f6294d990efb407f",
                "name": "Seedless Raisins",
                "gtin": "5054070629987",
                "image": "https://static.euw1.sustained.app/products/images/5054070629987_0.jpg",
                "pack": "1kg",
                "grade": "F",
                "points": {
                    "pef": 64.37717700242554
                },
                "info_icons": [],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2b9569e8e7ca0033f6294d990efb407f",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2b9569e8e7ca0033f6294d990efb407f/impacts"
                }
            },
            {
                "id": "productid.2c2d33edf6598d0a5e30ee29f03fb4b1",
                "name": "Cypressa Prunes",
                "gtin": "5000362285096",
                "image": "https://static.euw1.sustained.app/products/images/5000362285096_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Dried Fruits & Nuts",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2c2d33edf6598d0a5e30ee29f03fb4b1",
                    "category": "https://api.sustained.com/choice/v1/categories/dried-fruits-and-nuts",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2c2d33edf6598d0a5e30ee29f03fb4b1/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.01f9c6f93a1f900cfcb7cef48cfaae65",
                "name": "Medium 12 Northern Irish Free Range Eggs",
                "gtin": "01890273",
                "image": "https://static.euw1.sustained.app/products/images/01890273_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01f9c6f93a1f900cfcb7cef48cfaae65",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01f9c6f93a1f900cfcb7cef48cfaae65/impacts"
                }
            },
            {
                "id": "productid.0449f4ffef5a216904b922d2e285e2de",
                "name": "British 6 Large Free Range Eggs",
                "gtin": "5010251788607",
                "image": "https://static.euw1.sustained.app/products/images/5010251788607_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0449f4ffef5a216904b922d2e285e2de",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0449f4ffef5a216904b922d2e285e2de/impacts"
                }
            },
            {
                "id": "productid.09486a713bda6f1dba3272e5195c4b57",
                "name": "15 Free Range Eggs Mixed Weight",
                "gtin": "5000326010665",
                "image": "https://static.euw1.sustained.app/products/images/5000326010665_0.jpg",
                "pack": "655g",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09486a713bda6f1dba3272e5195c4b57",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09486a713bda6f1dba3272e5195c4b57/impacts"
                }
            },
            {
                "id": "productid.0ad609b7e42eb33615ce7c3f5bc06a81",
                "name": "12 Small Free Range Mini Pullet Eggs",
                "gtin": "5098930025182",
                "image": "https://static.euw1.sustained.app/products/images/5098930025182_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ad609b7e42eb33615ce7c3f5bc06a81",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ad609b7e42eb33615ce7c3f5bc06a81/impacts"
                }
            },
            {
                "id": "productid.0c04a8063d531524095399c12ffcf21f",
                "name": "6 Fresh Very Large Eggs",
                "gtin": "5099583001066",
                "image": "https://static.euw1.sustained.app/products/images/5099583001066_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c04a8063d531524095399c12ffcf21f",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c04a8063d531524095399c12ffcf21f/impacts"
                }
            },
            {
                "id": "productid.145f5199199b76641c33dc81618b8416",
                "name": "Mabel Pearman's Burford Browns 5 Dozen Free Range Eggs",
                "gtin": "5000306120537",
                "image": "https://static.euw1.sustained.app/products/images/5000306120537_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.145f5199199b76641c33dc81618b8416",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.145f5199199b76641c33dc81618b8416/impacts"
                }
            },
            {
                "id": "productid.15aebce0f39c4a816dbe405b04219d76",
                "name": "10 Big Tasty Caged Eggs",
                "gtin": "5060060966451",
                "image": "https://static.euw1.sustained.app/products/images/5060060966451_0.jpg",
                "pack": "680g",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.15aebce0f39c4a816dbe405b04219d76",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.15aebce0f39c4a816dbe405b04219d76/impacts"
                }
            },
            {
                "id": "productid.193265c65d577946a20c7fdf5b4a08b2",
                "name": "10 Bloomin' Big Eggs",
                "gtin": "5024263001203",
                "image": "https://static.euw1.sustained.app/products/images/5024263001203_0.jpg",
                "pack": "660g",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.193265c65d577946a20c7fdf5b4a08b2",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.193265c65d577946a20c7fdf5b4a08b2/impacts"
                }
            },
            {
                "id": "productid.1e596847d5d1c7b8f3742d486231eb8a",
                "name": "6 Organic Eggs L & M",
                "gtin": "5099690100942",
                "image": "https://static.euw1.sustained.app/products/images/5099690100942_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e596847d5d1c7b8f3742d486231eb8a",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e596847d5d1c7b8f3742d486231eb8a/impacts"
                }
            },
            {
                "id": "productid.287c2893c95e2b0894bb455545b67c30",
                "name": "6 British Eggs Mixed",
                "gtin": "5057545900802",
                "image": "https://static.euw1.sustained.app/products/images/5057545900802_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.287c2893c95e2b0894bb455545b67c30",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.287c2893c95e2b0894bb455545b67c30/impacts"
                }
            },
            {
                "id": "productid.29fb50021d8f3189f2443624c623a2eb",
                "name": "6 Free Range Eggs Mixed Sizes",
                "gtin": "5000326011051",
                "image": "https://static.euw1.sustained.app/products/images/5000326011051_0.jpg",
                "pack": "298g",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.29fb50021d8f3189f2443624c623a2eb",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.29fb50021d8f3189f2443624c623a2eb/impacts"
                }
            },
            {
                "id": "productid.2ba6b8a6012295bed45ca32166900164",
                "name": "6 Medium Local & Fresh Eggs",
                "gtin": "5060125220030",
                "image": "https://static.euw1.sustained.app/products/images/5060125220030_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2ba6b8a6012295bed45ca32166900164",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2ba6b8a6012295bed45ca32166900164/impacts"
                }
            },
            {
                "id": "productid.2ed3ae15e244f39a750498daa06c75fc",
                "name": "2 Hard Boiled Eggs",
                "gtin": "5010251419112",
                "image": "https://static.euw1.sustained.app/products/images/5010251419112_0.jpg",
                "pack": "100g",
                "grade": "E",
                "points": {
                    "pef": 42.03199801113882
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2ed3ae15e244f39a750498daa06c75fc",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2ed3ae15e244f39a750498daa06c75fc/impacts"
                }
            },
            {
                "id": "productid.3052838aab06ba0f27aa887bc404cea7",
                "name": "6 Derwent Valley Free Range Eggs Medium",
                "gtin": "5060125220108",
                "image": "https://static.euw1.sustained.app/products/images/5060125220108_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3052838aab06ba0f27aa887bc404cea7",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3052838aab06ba0f27aa887bc404cea7/impacts"
                }
            },
            {
                "id": "productid.31dfa1c7cd169a79a12cbe2d105bcda1",
                "name": "6 Free Range Eggs",
                "gtin": "5012829006755",
                "image": "https://static.euw1.sustained.app/products/images/5012829006755_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.31dfa1c7cd169a79a12cbe2d105bcda1",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.31dfa1c7cd169a79a12cbe2d105bcda1/impacts"
                }
            },
            {
                "id": "productid.336b45256d2b93ca327be58de5a0299b",
                "name": "Old Cotswold Legbar 6 Large Free Range Eggs",
                "gtin": "5000306004448",
                "image": "https://static.euw1.sustained.app/products/images/5000306004448_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.336b45256d2b93ca327be58de5a0299b",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.336b45256d2b93ca327be58de5a0299b/impacts"
                }
            },
            {
                "id": "productid.368330c159b566323ce12508b0a935b1",
                "name": "6 Large Welsh Free Range Eggs",
                "gtin": "5060002470060",
                "image": "https://static.euw1.sustained.app/products/images/5060002470060_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.368330c159b566323ce12508b0a935b1",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.368330c159b566323ce12508b0a935b1/impacts"
                }
            },
            {
                "id": "productid.3aac40569e28cd273f366604cff17a6e",
                "name": "Free Range 6 Medium Eggs",
                "gtin": "5098899120034",
                "image": "https://static.euw1.sustained.app/products/images/5098899120034_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3aac40569e28cd273f366604cff17a6e",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3aac40569e28cd273f366604cff17a6e/impacts"
                }
            },
            {
                "id": "productid.3be02b80b2103b8256e11b5da2d41241",
                "name": "Organic 12 Mixed Sizes Free Range Eggs",
                "gtin": "5057753908522",
                "image": "https://static.euw1.sustained.app/products/images/5057753908522_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3be02b80b2103b8256e11b5da2d41241",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3be02b80b2103b8256e11b5da2d41241/impacts"
                }
            },
            {
                "id": "productid.428c2008ed525561cbf7d33d7620dbb6",
                "name": "6 Derwent Valley Free Range Eggs Large",
                "gtin": "5060125220092",
                "image": "https://static.euw1.sustained.app/products/images/5060125220092_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.428c2008ed525561cbf7d33d7620dbb6",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.428c2008ed525561cbf7d33d7620dbb6/impacts"
                }
            },
            {
                "id": "productid.42923c579a25c94a3823f4c88380e1b3",
                "name": "'Free to Fly' 12 Quail Eggs",
                "gtin": "5000306002024",
                "image": "https://static.euw1.sustained.app/products/images/5000306002024_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.42923c579a25c94a3823f4c88380e1b3",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.42923c579a25c94a3823f4c88380e1b3/impacts"
                }
            },
            {
                "id": "productid.47725d0874ea240688a122e1e420f890",
                "name": "Fresh Free Range 6 Large/Medium Eggs",
                "gtin": "5098930700249",
                "image": "https://static.euw1.sustained.app/products/images/5098930700249_0.jpg",
                "pack": "348g",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.47725d0874ea240688a122e1e420f890",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.47725d0874ea240688a122e1e420f890/impacts"
                }
            },
            {
                "id": "productid.4c9561aa856efa8147000f2292656345",
                "name": "Green Field 20 Large Fresh Brown Eggs",
                "gtin": "5099690402206",
                "image": "https://static.euw1.sustained.app/products/images/5099690402206_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4c9561aa856efa8147000f2292656345",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4c9561aa856efa8147000f2292656345/impacts"
                }
            },
            {
                "id": "productid.5333eaa56665e282c984cb2a72929f4a",
                "name": "Belview Eggs Medium 6 Eggs",
                "gtin": "5099594006036",
                "image": "https://static.euw1.sustained.app/products/images/5099594006036_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5333eaa56665e282c984cb2a72929f4a",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5333eaa56665e282c984cb2a72929f4a/impacts"
                }
            },
            {
                "id": "productid.54291c7173c0464fe8579bc2dd6400a8",
                "name": "Fresh Free Range 6 Very Large/Large Eggs",
                "gtin": "5098930700232",
                "image": "https://static.euw1.sustained.app/products/images/5098930700232_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.54291c7173c0464fe8579bc2dd6400a8",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.54291c7173c0464fe8579bc2dd6400a8/impacts"
                }
            },
            {
                "id": "productid.567e2f0daf63aa702ebf3c6056d06432",
                "name": "Fresh Free Range 10 Medium Eggs",
                "gtin": "5098930823146",
                "image": "https://static.euw1.sustained.app/products/images/5098930823146_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.567e2f0daf63aa702ebf3c6056d06432",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.567e2f0daf63aa702ebf3c6056d06432/impacts"
                }
            },
            {
                "id": "productid.5be78474887d96d2568cf086fdb2591f",
                "name": "6 Medium Free Range Eggs",
                "gtin": "5060454383109",
                "image": "https://static.euw1.sustained.app/products/images/5060454383109_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5be78474887d96d2568cf086fdb2591f",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5be78474887d96d2568cf086fdb2591f/impacts"
                }
            },
            {
                "id": "productid.5dbe40d1a8ef352d29ed0703a8163161",
                "name": "Free Range, 12 Medium Eggs",
                "gtin": "5000326011099",
                "image": "https://static.euw1.sustained.app/products/images/5000326011099_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5dbe40d1a8ef352d29ed0703a8163161",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5dbe40d1a8ef352d29ed0703a8163161/impacts"
                }
            },
            {
                "id": "productid.5f7699be9691a3be68e052d3e22d4b80",
                "name": "6 Medium Free Range Hen Eggs",
                "gtin": "5000306002314",
                "image": "https://static.euw1.sustained.app/products/images/5000306002314_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5f7699be9691a3be68e052d3e22d4b80",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5f7699be9691a3be68e052d3e22d4b80/impacts"
                }
            },
            {
                "id": "productid.62a074843aea9ce995f660744e94bea6",
                "name": "12 Medium Free Range Eggs",
                "gtin": "5000306004769",
                "image": "https://static.euw1.sustained.app/products/images/5000306004769_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.62a074843aea9ce995f660744e94bea6",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.62a074843aea9ce995f660744e94bea6/impacts"
                }
            },
            {
                "id": "productid.66c8290c1356a90a5d6712098ecc7c54",
                "name": "6 Large Free Range Eggs",
                "gtin": "5010251564836",
                "image": "https://static.euw1.sustained.app/products/images/5010251564836_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.66c8290c1356a90a5d6712098ecc7c54",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.66c8290c1356a90a5d6712098ecc7c54/impacts"
                }
            },
            {
                "id": "productid.6a5ea2413eb826d9059b8e5eda8d000f",
                "name": "Bird Bros 15 Medium Eggs",
                "gtin": "5024263040400",
                "image": "https://static.euw1.sustained.app/products/images/5024263040400_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.6a5ea2413eb826d9059b8e5eda8d000f",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.6a5ea2413eb826d9059b8e5eda8d000f/impacts"
                }
            },
            {
                "id": "productid.6c27534f371dd28d250d956a5f86eabe",
                "name": "Smart Price 15 Egg from Caged Hens",
                "gtin": "20408466",
                "image": "https://static.euw1.sustained.app/products/images/20408466_0.jpg",
                "pack": "805g",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.6c27534f371dd28d250d956a5f86eabe",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.6c27534f371dd28d250d956a5f86eabe/impacts"
                }
            },
            {
                "id": "productid.7069ded8b0bc7efdca43b9e7692e8821",
                "name": "Traditional Breed Collection",
                "gtin": "5000306002253",
                "image": "https://static.euw1.sustained.app/products/images/5000306002253_0.jpg",
                "pack": "300g",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.7069ded8b0bc7efdca43b9e7692e8821",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.7069ded8b0bc7efdca43b9e7692e8821/impacts"
                }
            },
            {
                "id": "productid.719fecbef0008adb85aae356e34602ce",
                "name": "Extra Special Free Range 6 Chesnut Maran Eggs",
                "gtin": "5054781873389",
                "image": "https://static.euw1.sustained.app/products/images/5054781873389_0.jpg",
                "pack": "300g",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.719fecbef0008adb85aae356e34602ce",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.719fecbef0008adb85aae356e34602ce/impacts"
                }
            },
            {
                "id": "productid.7694b86f3027bd25fa6f779f46fe324d",
                "name": "Liquid Whole Egg",
                "gtin": "5018095008189",
                "image": "https://static.euw1.sustained.app/products/images/5018095008189_0.jpg",
                "pack": "1kg",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.7694b86f3027bd25fa6f779f46fe324d",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.7694b86f3027bd25fa6f779f46fe324d/impacts"
                }
            },
            {
                "id": "productid.7753f6d4092ebfb49946751c67ca439d",
                "name": "20 Large Barn Eggs",
                "gtin": "5099690101147",
                "image": "https://static.euw1.sustained.app/products/images/5099690101147_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.7753f6d4092ebfb49946751c67ca439d",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.7753f6d4092ebfb49946751c67ca439d/impacts"
                }
            },
            {
                "id": "productid.79ed03e046c770083b813502cd5c3431",
                "name": "The Collection British Farmed 6 Free Range White Eggs",
                "gtin": "0000000695343",
                "image": "https://static.euw1.sustained.app/products/images/0000000695343_0.jpg",
                "pack": "328g",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.79ed03e046c770083b813502cd5c3431",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.79ed03e046c770083b813502cd5c3431/impacts"
                }
            },
            {
                "id": "productid.7ab5dff1cdaed9b033df24addd8a2cc3",
                "name": "10 Big Organic Eggs",
                "gtin": "5034729070018",
                "image": "https://static.euw1.sustained.app/products/images/5034729070018_0.jpg",
                "pack": "660g",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.7ab5dff1cdaed9b033df24addd8a2cc3",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.7ab5dff1cdaed9b033df24addd8a2cc3/impacts"
                }
            },
            {
                "id": "productid.851f162c47814113915e61e7d765ce0f",
                "name": "+ Defence 6 British Free Range Medium Eggs",
                "gtin": "5000306123507",
                "image": "https://static.euw1.sustained.app/products/images/5000306123507_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.851f162c47814113915e61e7d765ce0f",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.851f162c47814113915e61e7d765ce0f/impacts"
                }
            },
            {
                "id": "productid.8a8c8441108b9eb88b462cc41ddfac7e",
                "name": "The Collection 6 British Farmed Free Range Duck Eggs",
                "gtin": "0000000585965",
                "image": "https://static.euw1.sustained.app/products/images/0000000585965_0.jpg",
                "pack": "360g",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.8a8c8441108b9eb88b462cc41ddfac7e",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.8a8c8441108b9eb88b462cc41ddfac7e/impacts"
                }
            },
            {
                "id": "productid.93c7117c1823ab156b224d849214bd3f",
                "name": "Seasonal Goose Egg",
                "gtin": "5000306003120",
                "image": "https://static.euw1.sustained.app/products/images/5000306003120_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.93c7117c1823ab156b224d849214bd3f",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.93c7117c1823ab156b224d849214bd3f/impacts"
                }
            },
            {
                "id": "productid.a6a55db873e074dcfd0d9b3c1ceef1ae",
                "name": "6 Free Range Omega 3 Rich Eggs",
                "gtin": "5000169482094",
                "image": "https://static.euw1.sustained.app/products/images/5000169482094_0.jpg",
                "pack": "318g",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.a6a55db873e074dcfd0d9b3c1ceef1ae",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.a6a55db873e074dcfd0d9b3c1ceef1ae/impacts"
                }
            },
            {
                "id": "productid.ac945a679680e4282dfd20c4b158a3c3",
                "name": "12 Ready to Eat Quail Eggs",
                "gtin": "5000306004189",
                "image": "https://static.euw1.sustained.app/products/images/5000306004189_0.jpg",
                "pack": "120g",
                "grade": "E",
                "points": {
                    "pef": 39.035956011552265
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.ac945a679680e4282dfd20c4b158a3c3",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.ac945a679680e4282dfd20c4b158a3c3/impacts"
                }
            },
            {
                "id": "productid.ace6032b0ba389af9704c63ce5ea055a",
                "name": "Belview Eggs 6 Eggs Large",
                "gtin": "5099594006029",
                "image": "https://static.euw1.sustained.app/products/images/5099594006029_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.ace6032b0ba389af9704c63ce5ea055a",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.ace6032b0ba389af9704c63ce5ea055a/impacts"
                }
            },
            {
                "id": "productid.b1fd70490c399c0b2080385262799ee9",
                "name": "Small 18 Eggs",
                "gtin": "5099583011867",
                "image": "https://static.euw1.sustained.app/products/images/5099583011867_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.b1fd70490c399c0b2080385262799ee9",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.b1fd70490c399c0b2080385262799ee9/impacts"
                }
            },
            {
                "id": "productid.b4ef65819a9085e7c096ad19d50aa138",
                "name": "Yorkshire Bee Friendly Free Range Large (6 per pack)",
                "gtin": "5034729050003",
                "image": "https://static.euw1.sustained.app/products/images/5034729050003_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.b4ef65819a9085e7c096ad19d50aa138",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.b4ef65819a9085e7c096ad19d50aa138/impacts"
                }
            },
            {
                "id": "productid.b5ad518f91ef15014c2608bafc885b08",
                "name": "Bird Bros 12 Large Eggs",
                "gtin": "5024263001210",
                "image": "https://static.euw1.sustained.app/products/images/5024263001210_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.b5ad518f91ef15014c2608bafc885b08",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.b5ad518f91ef15014c2608bafc885b08/impacts"
                }
            },
            {
                "id": "productid.b87344728dbda3e0e11c269b62c13554",
                "name": "6 Medium Free Range Eggs",
                "gtin": "5022115000343",
                "image": "https://static.euw1.sustained.app/products/images/5022115000343_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.b87344728dbda3e0e11c269b62c13554",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.b87344728dbda3e0e11c269b62c13554/impacts"
                }
            },
            {
                "id": "productid.b964bc65f004f02ec000cd61f54117bf",
                "name": "6 Free Range Eggs Large",
                "gtin": "5391242000015",
                "image": "https://static.euw1.sustained.app/products/images/5391242000015_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.173703118520265
                },
                "info_icons": [],
                "category": "Eggs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.b964bc65f004f02ec000cd61f54117bf",
                    "category": "https://api.sustained.com/choice/v1/categories/eggs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.b964bc65f004f02ec000cd61f54117bf/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.00acdb0b06e59ecca057c572a49f4b06",
                "name": "Food Collection Vietnamese Black Tiger Prawns in Garlic & Herb Butter",
                "gtin": "0000029148745",
                "image": "https://static.euw1.sustained.app/products/images/0000029148745_0.jpg",
                "pack": "335g",
                "grade": "E",
                "points": {
                    "pef": 36.28019420208062
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00acdb0b06e59ecca057c572a49f4b06",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00acdb0b06e59ecca057c572a49f4b06/impacts"
                }
            },
            {
                "id": "productid.00f708846ae784267be4b243397bde6d",
                "name": "Sliced Smoked Salmon",
                "gtin": "5011082002085",
                "image": "https://static.euw1.sustained.app/products/images/5011082002085_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 126.60431764520172
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00f708846ae784267be4b243397bde6d",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00f708846ae784267be4b243397bde6d/impacts"
                }
            },
            {
                "id": "productid.014c539912d0ef2ed38fdb2ea32836b7",
                "name": "Golden Shell Mussels with a White Wine, Cream & Garlic Sauce",
                "gtin": "5030098060070",
                "image": "https://static.euw1.sustained.app/products/images/5030098060070_0.jpg",
                "pack": "640g",
                "grade": "C",
                "points": {
                    "pef": 18.175943376240653
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.014c539912d0ef2ed38fdb2ea32836b7",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.014c539912d0ef2ed38fdb2ea32836b7/impacts"
                }
            },
            {
                "id": "productid.02ec6e94b60239ea01ff0819df1a2ad3",
                "name": "Sea Bass Fillets",
                "gtin": "0248190000008",
                "image": "https://static.euw1.sustained.app/products/images/0248190000008_0.jpg",
                "pack": "90g",
                "grade": "G",
                "points": {
                    "pef": 130.15709599603773
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02ec6e94b60239ea01ff0819df1a2ad3",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02ec6e94b60239ea01ff0819df1a2ad3/impacts"
                }
            },
            {
                "id": "productid.04706231e98de809d928643ff8862b22",
                "name": "Food 2 Scottish Honey Roast Hot Smoked Salmon Fillets",
                "gtin": "0000029150144",
                "image": "https://static.euw1.sustained.app/products/images/0000029150144_0.jpg",
                "pack": "160g",
                "grade": "G",
                "points": {
                    "pef": 123.88179692681248
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04706231e98de809d928643ff8862b22",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04706231e98de809d928643ff8862b22/impacts"
                }
            },
            {
                "id": "productid.07014a46c57293efd7ae04e432cb4515",
                "name": "Market St Whole Sea Bass",
                "gtin": "5000144727882",
                "image": "https://static.euw1.sustained.app/products/images/5000144727882_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 130.15709599603773
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07014a46c57293efd7ae04e432cb4515",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07014a46c57293efd7ae04e432cb4515/impacts"
                }
            },
            {
                "id": "productid.08f762284867ebbde850425ffb4235c8",
                "name": "Flavoursome 2 Salmon Fillets",
                "gtin": "5054070136010",
                "image": "https://static.euw1.sustained.app/products/images/5054070136010_0.jpg",
                "pack": "240g",
                "grade": "G",
                "points": {
                    "pef": 129.6623869198634
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08f762284867ebbde850425ffb4235c8",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08f762284867ebbde850425ffb4235c8/impacts"
                }
            },
            {
                "id": "productid.0a32d9ad43a147d166bb0141b6e55c96",
                "name": "Taste the Difference British Smoked Mackerel",
                "gtin": "0211156002415",
                "image": "https://static.euw1.sustained.app/products/images/0211156002415_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 28.258277236827666
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a32d9ad43a147d166bb0141b6e55c96",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a32d9ad43a147d166bb0141b6e55c96/impacts"
                }
            },
            {
                "id": "productid.0af06f9eb79bb670dc52411d5885e601",
                "name": "Food 2 Breaded Haddock Fillets",
                "gtin": "0000029119103",
                "image": "https://static.euw1.sustained.app/products/images/0000029119103_0.jpg",
                "pack": "300g",
                "grade": "G",
                "points": {
                    "pef": 186.46710439519825
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0af06f9eb79bb670dc52411d5885e601",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0af06f9eb79bb670dc52411d5885e601/impacts"
                }
            },
            {
                "id": "productid.0b83d37c66f14fd702cdc31b10b84861",
                "name": "2 Breaded Chunky Cod Fillets",
                "gtin": "5052910096050",
                "image": "https://static.euw1.sustained.app/products/images/5052910096050_0.jpg",
                "pack": "350g",
                "grade": "G",
                "points": {
                    "pef": 186.46710439519825
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b83d37c66f14fd702cdc31b10b84861",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b83d37c66f14fd702cdc31b10b84861/impacts"
                }
            },
            {
                "id": "productid.0ccb2bb64c9e2343a7bcb208575f1277",
                "name": "2 Cod Fillets Boneless",
                "gtin": "5057545986844",
                "image": "https://static.euw1.sustained.app/products/images/5057545986844_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 186.46710439519825
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ccb2bb64c9e2343a7bcb208575f1277",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ccb2bb64c9e2343a7bcb208575f1277/impacts"
                }
            },
            {
                "id": "productid.0ced42c3d0a873ba09c2e11cdcd4572c",
                "name": "6 Boneless Salmon Fillets",
                "gtin": "5057545734414",
                "image": "https://static.euw1.sustained.app/products/images/5057545734414_0.jpg",
                "pack": "780g",
                "grade": "G",
                "points": {
                    "pef": 129.6623869198634
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ced42c3d0a873ba09c2e11cdcd4572c",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ced42c3d0a873ba09c2e11cdcd4572c/impacts"
                }
            },
            {
                "id": "productid.0d521d26f02d105fbea59a07a563e2ca",
                "name": "The Fishmonger's On Market Street Smoked Mackerel Fillets",
                "gtin": "2335300002088",
                "image": "https://static.euw1.sustained.app/products/images/2335300002088_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 28.060729804907645
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d521d26f02d105fbea59a07a563e2ca",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d521d26f02d105fbea59a07a563e2ca/impacts"
                }
            },
            {
                "id": "productid.0dacaea4bfefea89e34a927054ed8ca6",
                "name": "Finest Raw Jumbo King Prawns",
                "gtin": "5057008546240",
                "image": "https://static.euw1.sustained.app/products/images/5057008546240_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 34.60611012848653
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0dacaea4bfefea89e34a927054ed8ca6",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0dacaea4bfefea89e34a927054ed8ca6/impacts"
                }
            },
            {
                "id": "productid.0fc403e5c7cfb50d8a2f6944af533e1f",
                "name": "Select Farm Salmon Fillet Rosemary Garlic",
                "gtin": "0231460000006",
                "image": "https://static.euw1.sustained.app/products/images/0231460000006_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 129.6623869198634
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fc403e5c7cfb50d8a2f6944af533e1f",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fc403e5c7cfb50d8a2f6944af533e1f/impacts"
                }
            },
            {
                "id": "productid.10d0bb57f65e1473a417e7d111fa4740",
                "name": "Tesco Whole Sea Bream 220-400g",
                "gtin": "5057753905965",
                "image": "https://static.euw1.sustained.app/products/images/5057753905965_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 130.15709599603773
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10d0bb57f65e1473a417e7d111fa4740",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10d0bb57f65e1473a417e7d111fa4740/impacts"
                }
            },
            {
                "id": "productid.12c4450495310a851e8607686f668483",
                "name": "Lightly Dusted 2 Lemon Sole Fillets",
                "gtin": "01773019",
                "image": "https://static.euw1.sustained.app/products/images/01773019_0.jpg",
                "pack": "265g",
                "grade": "G",
                "points": {
                    "pef": 141.49395701473796
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12c4450495310a851e8607686f668483",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12c4450495310a851e8607686f668483/impacts"
                }
            },
            {
                "id": "productid.16a6ef8273914fd99f42aaa18ec52eee",
                "name": "Westcountry Smoked Peppered Mackerel",
                "gtin": "2058201000000",
                "image": "https://static.euw1.sustained.app/products/images/2058201000000_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 28.061450711301525
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16a6ef8273914fd99f42aaa18ec52eee",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16a6ef8273914fd99f42aaa18ec52eee/impacts"
                }
            },
            {
                "id": "productid.1759e70a2731a1f8a6a15d70f880536a",
                "name": "Smoked Scottish Salmon Fillet",
                "gtin": "0231720000005",
                "image": "https://static.euw1.sustained.app/products/images/0231720000005_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 129.6623869198634
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1759e70a2731a1f8a6a15d70f880536a",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1759e70a2731a1f8a6a15d70f880536a/impacts"
                }
            },
            {
                "id": "productid.1b64a05c62f01a101126581f3027edeb",
                "name": "Whisky Infused Scottish Salmon",
                "gtin": "5030450733178",
                "image": "https://static.euw1.sustained.app/products/images/5030450733178_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 123.71016126225962
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b64a05c62f01a101126581f3027edeb",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b64a05c62f01a101126581f3027edeb/impacts"
                }
            },
            {
                "id": "productid.1c0345b66bb5ea0e461dfb48444acc68",
                "name": "No1 Tsar Cut Fillet of Scottish Salmon",
                "gtin": "5000169407707",
                "image": "https://static.euw1.sustained.app/products/images/5000169407707_0.jpg",
                "pack": "140g",
                "grade": "G",
                "points": {
                    "pef": 125.94659420965203
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c0345b66bb5ea0e461dfb48444acc68",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c0345b66bb5ea0e461dfb48444acc68/impacts"
                }
            },
            {
                "id": "productid.1c78522d1788b482ed3f031f833a8758",
                "name": "Crayfish Tails",
                "gtin": "5060044533549",
                "image": "https://static.euw1.sustained.app/products/images/5060044533549_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 70.22833694387589
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c78522d1788b482ed3f031f833a8758",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c78522d1788b482ed3f031f833a8758/impacts"
                }
            },
            {
                "id": "productid.1e52cd37aad01f75509c00f986ecb0ed",
                "name": "Live Oysters",
                "gtin": "9993510309934",
                "image": "https://static.euw1.sustained.app/products/images/9993510309934_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 10.128367619953577
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e52cd37aad01f75509c00f986ecb0ed",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e52cd37aad01f75509c00f986ecb0ed/impacts"
                }
            },
            {
                "id": "productid.1f7a644a925af906eb6d5c84ab472407",
                "name": "Food Black Pepper, Mustard & Dill Hot Smoked Scottish Salmon",
                "gtin": "0000000378154",
                "image": "https://static.euw1.sustained.app/products/images/0000000378154_0.jpg",
                "pack": "125g",
                "grade": "G",
                "points": {
                    "pef": 127.1911158383455
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1f7a644a925af906eb6d5c84ab472407",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1f7a644a925af906eb6d5c84ab472407/impacts"
                }
            },
            {
                "id": "productid.21a87bdce809a5f9b89dfeb318d85844",
                "name": "Food Atlantic Peeled Prawns",
                "gtin": "0000000280976",
                "image": "https://static.euw1.sustained.app/products/images/0000000280976_0.jpg",
                "pack": "175g",
                "grade": "E",
                "points": {
                    "pef": 34.06078024458473
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.21a87bdce809a5f9b89dfeb318d85844",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.21a87bdce809a5f9b89dfeb318d85844/impacts"
                }
            },
            {
                "id": "productid.2382fcd7705f39659cf49a8eca111e4e",
                "name": "6 Boneless Sea Bass Fillets",
                "gtin": "0000000400909",
                "image": "https://static.euw1.sustained.app/products/images/0000000400909_0.jpg",
                "pack": "540g",
                "grade": "G",
                "points": {
                    "pef": 130.15709599603773
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2382fcd7705f39659cf49a8eca111e4e",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2382fcd7705f39659cf49a8eca111e4e/impacts"
                }
            },
            {
                "id": "productid.245ea2dbbdd52f46d239bd3f3d5e59da",
                "name": "Sweet Chilli Hot Smoked Salmon Fillets",
                "gtin": "5057753305130",
                "image": "https://static.euw1.sustained.app/products/images/5057753305130_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 129.6623869198634
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.245ea2dbbdd52f46d239bd3f3d5e59da",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.245ea2dbbdd52f46d239bd3f3d5e59da/impacts"
                }
            },
            {
                "id": "productid.251356ff07fca8e0594ecae2bf63c4d1",
                "name": "2 Boneless Salmon Fillets",
                "gtin": "5054269381351",
                "image": "https://static.euw1.sustained.app/products/images/5054269381351_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 129.6623869198634
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.251356ff07fca8e0594ecae2bf63c4d1",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.251356ff07fca8e0594ecae2bf63c4d1/impacts"
                }
            },
            {
                "id": "productid.25d4e91d9a63f8ed9178663487b62568",
                "name": "Cod Skinless FAS 16/32",
                "gtin": "15694310800041",
                "image": "https://static.euw1.sustained.app/products/images/15694310800041_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 186.46710439519825
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.25d4e91d9a63f8ed9178663487b62568",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.25d4e91d9a63f8ed9178663487b62568/impacts"
                }
            },
            {
                "id": "productid.26321d1fb41efb698524c36d30990da5",
                "name": "LEAP Keta Salmon Fillets",
                "gtin": "5036820001434",
                "image": "https://static.euw1.sustained.app/products/images/5036820001434_0.jpg",
                "pack": "220g",
                "grade": "G",
                "points": {
                    "pef": 129.6623869198634
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.26321d1fb41efb698524c36d30990da5",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.26321d1fb41efb698524c36d30990da5/impacts"
                }
            },
            {
                "id": "productid.28ea7db844d9c336cc9b74f864cce7ca",
                "name": "Food Scottish Oak Smoked Salmon Ribbons",
                "gtin": "0000000611046",
                "image": "https://static.euw1.sustained.app/products/images/0000000611046_0.jpg",
                "pack": "50g",
                "grade": "G",
                "points": {
                    "pef": 127.7691548279286
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.28ea7db844d9c336cc9b74f864cce7ca",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.28ea7db844d9c336cc9b74f864cce7ca/impacts"
                }
            },
            {
                "id": "productid.2b0736ff3f14d8604aac516ebb168afa",
                "name": "16 Seafood Sticks",
                "gtin": "5000436868941",
                "image": "https://static.euw1.sustained.app/products/images/5000436868941_0.jpg",
                "pack": "250g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2b0736ff3f14d8604aac516ebb168afa",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2b0736ff3f14d8604aac516ebb168afa/impacts"
                }
            },
            {
                "id": "productid.2bf24b9b718c370ead15477c61108b59",
                "name": "Market St Scallop Meat",
                "gtin": "5000144727844",
                "image": "https://static.euw1.sustained.app/products/images/5000144727844_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 20.15576709164735
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2bf24b9b718c370ead15477c61108b59",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2bf24b9b718c370ead15477c61108b59/impacts"
                }
            },
            {
                "id": "productid.2bfec800014e44143a0ffe1195f694ad",
                "name": "Salmon Darnes",
                "gtin": "5391511399123",
                "image": "https://static.euw1.sustained.app/products/images/5391511399123_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 129.6623869198634
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2bfec800014e44143a0ffe1195f694ad",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2bfec800014e44143a0ffe1195f694ad/impacts"
                }
            },
            {
                "id": "productid.2d6f992ef59acfa1d37a1c8a7f6d22d6",
                "name": "Salmon Tails",
                "gtin": "5059697751226",
                "image": "https://static.euw1.sustained.app/products/images/5059697751226_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 129.6623869198634
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2d6f992ef59acfa1d37a1c8a7f6d22d6",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2d6f992ef59acfa1d37a1c8a7f6d22d6/impacts"
                }
            },
            {
                "id": "productid.2eedaf3e5104edfe67f5b44347a934cb",
                "name": "Extra Large Raw King Prawns",
                "gtin": "5000169279335",
                "image": "https://static.euw1.sustained.app/products/images/5000169279335_0.jpg",
                "pack": "180g",
                "grade": "E",
                "points": {
                    "pef": 34.33344518653562
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2eedaf3e5104edfe67f5b44347a934cb",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2eedaf3e5104edfe67f5b44347a934cb/impacts"
                }
            },
            {
                "id": "productid.2f022ba6b387a49757f000b9b4a43e84",
                "name": "Superior Scottish Salmon Fillets",
                "gtin": "5055946000770",
                "image": "https://static.euw1.sustained.app/products/images/5055946000770_0.jpg",
                "pack": "230g",
                "grade": "G",
                "points": {
                    "pef": 129.6623869198634
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2f022ba6b387a49757f000b9b4a43e84",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2f022ba6b387a49757f000b9b4a43e84/impacts"
                }
            },
            {
                "id": "productid.305c393503ad14349f0136139a00dc4f",
                "name": "White Crab",
                "gtin": "5060212470232",
                "image": "https://static.euw1.sustained.app/products/images/5060212470232_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 70.22833694387589
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.305c393503ad14349f0136139a00dc4f",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.305c393503ad14349f0136139a00dc4f/impacts"
                }
            },
            {
                "id": "productid.321ee7b463bd0d742f3062f0016a4cce",
                "name": "Tuna Steak",
                "gtin": "5060177151443",
                "image": "https://static.euw1.sustained.app/products/images/5060177151443_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.321ee7b463bd0d742f3062f0016a4cce",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.321ee7b463bd0d742f3062f0016a4cce/impacts"
                }
            },
            {
                "id": "productid.3284347865268c33e7bed4a546ea3069",
                "name": "Market Street Smoked Haddock Loin",
                "gtin": "0281660000009",
                "image": "https://static.euw1.sustained.app/products/images/0281660000009_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3284347865268c33e7bed4a546ea3069",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3284347865268c33e7bed4a546ea3069/impacts"
                }
            },
            {
                "id": "productid.3382500cc9bea75f61f343c9c2435574",
                "name": "Squid Antipasti",
                "gtin": "5010292953767",
                "image": "https://static.euw1.sustained.app/products/images/5010292953767_0.jpg",
                "pack": "110g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3382500cc9bea75f61f343c9c2435574",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3382500cc9bea75f61f343c9c2435574/impacts"
                }
            },
            {
                "id": "productid.355c2d0cd3ae00b0535c777c59c4b59f",
                "name": "Bradan Orach Dry Cured, Cold Smoked Scottish Salmon",
                "gtin": "5030098012550",
                "image": "https://static.euw1.sustained.app/products/images/5030098012550_0.jpg",
                "pack": "125g",
                "grade": "G",
                "points": {
                    "pef": 126.23734933224235
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.355c2d0cd3ae00b0535c777c59c4b59f",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.355c2d0cd3ae00b0535c777c59c4b59f/impacts"
                }
            },
            {
                "id": "productid.3665a6f88265295fc9c3ee8ef0feb0e9",
                "name": "Air-Dried Smoked Salmon Pellicle",
                "gtin": "5018427151002",
                "image": "https://static.euw1.sustained.app/products/images/5018427151002_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3665a6f88265295fc9c3ee8ef0feb0e9",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3665a6f88265295fc9c3ee8ef0feb0e9/impacts"
                }
            },
            {
                "id": "productid.3849778c61f30fa42bcb6c55ce4d270c",
                "name": "Finest MSC Wild Sockeye Salmon Fillet",
                "gtin": "0293370000000",
                "image": "https://static.euw1.sustained.app/products/images/0293370000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 129.6623869198634
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3849778c61f30fa42bcb6c55ce4d270c",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3849778c61f30fa42bcb6c55ce4d270c/impacts"
                }
            },
            {
                "id": "productid.3abe9a7c8b926d9ceb372769c5d51a19",
                "name": "Raw King Prawns",
                "gtin": "01804706",
                "image": "https://static.euw1.sustained.app/products/images/01804706_0.jpg",
                "pack": "165g",
                "grade": "E",
                "points": {
                    "pef": 34.33344518653562
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3abe9a7c8b926d9ceb372769c5d51a19",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3abe9a7c8b926d9ceb372769c5d51a19/impacts"
                }
            },
            {
                "id": "productid.3b8b4be70ed9d6d74acdc844a82be598",
                "name": "Finest Scottish Smoked Salmon",
                "gtin": "5057753638337",
                "image": "https://static.euw1.sustained.app/products/images/5057753638337_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 129.6623869198634
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3b8b4be70ed9d6d74acdc844a82be598",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3b8b4be70ed9d6d74acdc844a82be598/impacts"
                }
            },
            {
                "id": "productid.3ba0cd0c09eda500e930212fc75b1c3a",
                "name": "Cooked & Peeled Coldwater Prawns",
                "gtin": "5060021927187",
                "image": "https://static.euw1.sustained.app/products/images/5060021927187_0.jpg",
                "pack": "150g",
                "grade": "E",
                "points": {
                    "pef": 34.06078024458473
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3ba0cd0c09eda500e930212fc75b1c3a",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3ba0cd0c09eda500e930212fc75b1c3a/impacts"
                }
            },
            {
                "id": "productid.3bc8ee4e34c9f6f9ef87ab4c9b1d9afe",
                "name": "Hooked Wakame Tuna Burger",
                "gtin": "8714543020921",
                "image": "https://static.euw1.sustained.app/products/images/8714543020921_0.jpg",
                "pack": "80g",
                "grade": "G",
                "points": {
                    "pef": 95.88471606316085
                },
                "info_icons": [],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3bc8ee4e34c9f6f9ef87ab4c9b1d9afe",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3bc8ee4e34c9f6f9ef87ab4c9b1d9afe/impacts"
                }
            },
            {
                "id": "productid.3cba94f5c8baf0317f4b3b14dcd56359",
                "name": "Smoked British Mackerel",
                "gtin": "2058205000000",
                "image": "https://static.euw1.sustained.app/products/images/2058205000000_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 27.97573028105698
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3cba94f5c8baf0317f4b3b14dcd56359",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3cba94f5c8baf0317f4b3b14dcd56359/impacts"
                }
            },
            {
                "id": "productid.3da316f6b32b7da089a589ca09cac15b",
                "name": "2 Boneless Cod Fillets",
                "gtin": "5057008059573",
                "image": "https://static.euw1.sustained.app/products/images/5057008059573_0.jpg",
                "pack": "280g",
                "grade": "G",
                "points": {
                    "pef": 186.46710439519825
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fish & Seafood",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3da316f6b32b7da089a589ca09cac15b",
                    "category": "https://api.sustained.com/choice/v1/categories/fish-and-seafood",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3da316f6b32b7da089a589ca09cac15b/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.000a384509ba4684735565ad6c4cc882",
                "name": "Finest Free Range Bronze Whole Turkey with Giblets 3.2 -",
                "gtin": "5057753912741",
                "image": "https://static.euw1.sustained.app/products/images/5057753912741_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.000a384509ba4684735565ad6c4cc882",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.000a384509ba4684735565ad6c4cc882/impacts"
                }
            },
            {
                "id": "productid.00bbbc4d7f0c13deb91c4097ebbc1879",
                "name": "Beef Spaghetti Bolognese",
                "gtin": "5000116103843",
                "image": "https://static.euw1.sustained.app/products/images/5000116103843_0.jpg",
                "pack": "340g",
                "grade": "F",
                "points": {
                    "pef": 61.74896570846128
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00bbbc4d7f0c13deb91c4097ebbc1879",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00bbbc4d7f0c13deb91c4097ebbc1879/impacts"
                }
            },
            {
                "id": "productid.01396aa22f03e5d2ad1b26f42cd0e18f",
                "name": "Cornetto Choc 'n' Ball Ice Cream Cone",
                "gtin": "8712100486517",
                "image": "https://static.euw1.sustained.app/products/images/8712100486517_0.jpg",
                "pack": "160ml",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01396aa22f03e5d2ad1b26f42cd0e18f",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01396aa22f03e5d2ad1b26f42cd0e18f/impacts"
                }
            },
            {
                "id": "productid.0174d78a751d21ac1307cffed1a7ed1a",
                "name": "Beef Lasagne",
                "gtin": "0000000123051",
                "image": "https://static.euw1.sustained.app/products/images/0000000123051_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0174d78a751d21ac1307cffed1a7ed1a",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0174d78a751d21ac1307cffed1a7ed1a/impacts"
                }
            },
            {
                "id": "productid.01e01e8136b5f15e54e7b4848009a372",
                "name": "Microwaveable White Long Grain Rice",
                "gtin": "01615630",
                "image": "https://static.euw1.sustained.app/products/images/01615630_0.jpg",
                "pack": "200g",
                "grade": "D",
                "points": {
                    "pef": 31.227682434520275
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01e01e8136b5f15e54e7b4848009a372",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01e01e8136b5f15e54e7b4848009a372/impacts"
                }
            },
            {
                "id": "productid.01e3ba5ec840b917e72959f73d9d9d5f",
                "name": "Profiteroles & Chocolate Sauce",
                "gtin": "5051413184660",
                "image": "https://static.euw1.sustained.app/products/images/5051413184660_0.jpg",
                "pack": "235g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01e3ba5ec840b917e72959f73d9d9d5f",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01e3ba5ec840b917e72959f73d9d9d5f/impacts"
                }
            },
            {
                "id": "productid.01f4bb953ebd682acee712e2bcd67aa7",
                "name": "Food 12 Oriental Prawn Selection",
                "gtin": "0000000683449",
                "image": "https://static.euw1.sustained.app/products/images/0000000683449_0.jpg",
                "pack": "270g",
                "grade": "E",
                "points": {
                    "pef": 34.60611012848653
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01f4bb953ebd682acee712e2bcd67aa7",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01f4bb953ebd682acee712e2bcd67aa7/impacts"
                }
            },
            {
                "id": "productid.020205d4121169aeab9895bee66647a5",
                "name": "Summer Cup Sorbet Minis",
                "gtin": "5000169620809",
                "image": "https://static.euw1.sustained.app/products/images/5000169620809_0.jpg",
                "pack": "100ml",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.020205d4121169aeab9895bee66647a5",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.020205d4121169aeab9895bee66647a5/impacts"
                }
            },
            {
                "id": "productid.0234988bdfff65c98c798db5008e4af0",
                "name": "Mediterranean Vegetable Tart (Vegan)",
                "gtin": "5060253235944",
                "image": "https://static.euw1.sustained.app/products/images/5060253235944_0.jpg",
                "pack": "260g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0234988bdfff65c98c798db5008e4af0",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0234988bdfff65c98c798db5008e4af0/impacts"
                }
            },
            {
                "id": "productid.026a7bb77db379855fe031604d2bc557",
                "name": "Foodservice Hot 'N' Spicy Chicken Fillets",
                "gtin": "5060021944108",
                "image": "https://static.euw1.sustained.app/products/images/5060021944108_0.jpg",
                "pack": "2kg",
                "grade": "F",
                "points": {
                    "pef": 57.84564247908655
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.026a7bb77db379855fe031604d2bc557",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.026a7bb77db379855fe031604d2bc557/impacts"
                }
            },
            {
                "id": "productid.02cd31bf8c2c4d1dc1f1a1534b44193c",
                "name": "Frozen Croissant Fine Butter Ready to Bake",
                "gtin": "0000000145749",
                "image": "https://static.euw1.sustained.app/products/images/0000000145749_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02cd31bf8c2c4d1dc1f1a1534b44193c",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02cd31bf8c2c4d1dc1f1a1534b44193c/impacts"
                }
            },
            {
                "id": "productid.02eab352e34edc0453d77906769c723a",
                "name": "Festive Potato Shapes",
                "gtin": "5057172606894",
                "image": "https://static.euw1.sustained.app/products/images/5057172606894_0.jpg",
                "pack": "500g",
                "grade": "D",
                "points": {
                    "pef": 30.38645314141577
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02eab352e34edc0453d77906769c723a",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02eab352e34edc0453d77906769c723a/impacts"
                }
            },
            {
                "id": "productid.0331a1ce9a59fece2fde83c0c816e53d",
                "name": "Essential Waitrose & Partners Brussels Sprouts",
                "gtin": "5000169515532",
                "image": "https://static.euw1.sustained.app/products/images/5000169515532_0.jpg",
                "pack": "500g",
                "grade": "A",
                "points": {
                    "pef": 6.628853728533776
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0331a1ce9a59fece2fde83c0c816e53d",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0331a1ce9a59fece2fde83c0c816e53d/impacts"
                }
            },
            {
                "id": "productid.033b43dd86bfd51bba214ce43cf6f881",
                "name": "Funny Feet Strawberry",
                "gtin": "8711327510906",
                "image": "https://static.euw1.sustained.app/products/images/8711327510906_0.jpg",
                "pack": "79ml",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.033b43dd86bfd51bba214ce43cf6f881",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.033b43dd86bfd51bba214ce43cf6f881/impacts"
                }
            },
            {
                "id": "productid.03736f15f861541d66f47eef771b4654",
                "name": "4 Chunky Battered Cod Fillets",
                "gtin": "0000000346832",
                "image": "https://static.euw1.sustained.app/products/images/0000000346832_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03736f15f861541d66f47eef771b4654",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03736f15f861541d66f47eef771b4654/impacts"
                }
            },
            {
                "id": "productid.03cb588477d1aa3685579105b36791bb",
                "name": "Chicken Gyoza 12 Pieces",
                "gtin": "5014276706721",
                "image": "https://static.euw1.sustained.app/products/images/5014276706721_0.jpg",
                "pack": "240g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03cb588477d1aa3685579105b36791bb",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03cb588477d1aa3685579105b36791bb/impacts"
                }
            },
            {
                "id": "productid.03cc7f5d7b124199a9056de51730c737",
                "name": "Essential Waitrose & Partners Alaskan Pollock Fillets",
                "gtin": "5000169606025",
                "image": "https://static.euw1.sustained.app/products/images/5000169606025_0.jpg",
                "pack": "1kg",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03cc7f5d7b124199a9056de51730c737",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03cc7f5d7b124199a9056de51730c737/impacts"
                }
            },
            {
                "id": "productid.043948bab6f33f25d7d9e3e122ad3330",
                "name": "Salt & Chilli Crispy Shredded Chicken",
                "gtin": "5391511149902",
                "image": "https://static.euw1.sustained.app/products/images/5391511149902_0.jpg",
                "pack": "320g",
                "grade": "F",
                "points": {
                    "pef": 60.68372208730325
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.043948bab6f33f25d7d9e3e122ad3330",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.043948bab6f33f25d7d9e3e122ad3330/impacts"
                }
            },
            {
                "id": "productid.04499a854eb520fd2319a773e6d10863",
                "name": "Big & Tasty Vietnamese White Clams",
                "gtin": "5060044533525",
                "image": "https://static.euw1.sustained.app/products/images/5060044533525_0.jpg",
                "pack": "600g",
                "grade": "B",
                "points": {
                    "pef": 12.072428167364565
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04499a854eb520fd2319a773e6d10863",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04499a854eb520fd2319a773e6d10863/impacts"
                }
            },
            {
                "id": "productid.0459a6c644bc53480912dbe83269251f",
                "name": "Mint & Chilli Peas",
                "gtin": "5010525051093",
                "image": "https://static.euw1.sustained.app/products/images/5010525051093_0.jpg",
                "pack": "100g",
                "grade": "A",
                "points": {
                    "pef": 9.728203673754429
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0459a6c644bc53480912dbe83269251f",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0459a6c644bc53480912dbe83269251f/impacts"
                }
            },
            {
                "id": "productid.0468029133ac0a225aca17ccba4d51c0",
                "name": "The Kale & Quinoa Burger",
                "gtin": "5391528181056",
                "image": "https://static.euw1.sustained.app/products/images/5391528181056_0.jpg",
                "pack": "300g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0468029133ac0a225aca17ccba4d51c0",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0468029133ac0a225aca17ccba4d51c0/impacts"
                }
            },
            {
                "id": "productid.047f164a67806a36797534dab7722283",
                "name": "Mythicools Phoenix",
                "gtin": "8711327539020",
                "image": "https://static.euw1.sustained.app/products/images/8711327539020_0.jpg",
                "pack": "70ml",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.047f164a67806a36797534dab7722283",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.047f164a67806a36797534dab7722283/impacts"
                }
            },
            {
                "id": "productid.049e15e33bb83d0507532363dbe56845",
                "name": "Organic Vanilla Ice Cream Tub",
                "gtin": "5010238017584",
                "image": "https://static.euw1.sustained.app/products/images/5010238017584_0.jpg",
                "pack": "500ml",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.049e15e33bb83d0507532363dbe56845",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.049e15e33bb83d0507532363dbe56845/impacts"
                }
            },
            {
                "id": "productid.04ac80d63cb8254c196fc992f86eb73c",
                "name": "Fruity Indulgence Tropical Sorbet",
                "gtin": "5011035502020",
                "image": "https://static.euw1.sustained.app/products/images/5011035502020_0.jpg",
                "pack": "4l",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04ac80d63cb8254c196fc992f86eb73c",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04ac80d63cb8254c196fc992f86eb73c/impacts"
                }
            },
            {
                "id": "productid.04d4b3cc1cad15bb4d4f661f5df6a1fe",
                "name": "True Barbecue Divine Rack of Pork Ribs with Glorious Kansas City Style BBQ Sauce",
                "gtin": "5060736491027",
                "image": "https://static.euw1.sustained.app/products/images/5060736491027_0.jpg",
                "pack": "600g",
                "grade": "G",
                "points": {
                    "pef": 95.4047161803316
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04d4b3cc1cad15bb4d4f661f5df6a1fe",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04d4b3cc1cad15bb4d4f661f5df6a1fe/impacts"
                }
            },
            {
                "id": "productid.054cf2e8e07ce3e318df46930c570883",
                "name": "Gluten Free Toad in the Hole",
                "gtin": "5021746001156",
                "image": "https://static.euw1.sustained.app/products/images/5021746001156_0.jpg",
                "pack": "190g",
                "grade": "G",
                "points": {
                    "pef": 107.2618525128868
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.054cf2e8e07ce3e318df46930c570883",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.054cf2e8e07ce3e318df46930c570883/impacts"
                }
            },
            {
                "id": "productid.055923b2cd53988fe3e8c1988c583d56",
                "name": "Lemon Sorbet",
                "gtin": "5057545777305",
                "image": "https://static.euw1.sustained.app/products/images/5057545777305_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.714972323269464
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.055923b2cd53988fe3e8c1988c583d56",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.055923b2cd53988fe3e8c1988c583d56/impacts"
                }
            },
            {
                "id": "productid.057d20c92c6f9ae66071f377b6d99397",
                "name": "Vanilla Ice Cream",
                "gtin": "8712100728136",
                "image": "https://static.euw1.sustained.app/products/images/8712100728136_0.jpg",
                "pack": "100millilitre",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.057d20c92c6f9ae66071f377b6d99397",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.057d20c92c6f9ae66071f377b6d99397/impacts"
                }
            },
            {
                "id": "productid.059436865fbb8b5fe64b4f0fafce9af3",
                "name": "Steamfresh Green Medley",
                "gtin": "5000116108428",
                "image": "https://static.euw1.sustained.app/products/images/5000116108428_0.jpg",
                "pack": "300g",
                "grade": "B",
                "points": {
                    "pef": 14.82606921863707
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.059436865fbb8b5fe64b4f0fafce9af3",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.059436865fbb8b5fe64b4f0fafce9af3/impacts"
                }
            },
            {
                "id": "productid.05b623510a25212a116f188ca0100b9f",
                "name": "Non-Dairy Cookie on Cookie Dough Ice Cream",
                "gtin": "8711327538290",
                "image": "https://static.euw1.sustained.app/products/images/8711327538290_0.jpg",
                "pack": "100millilitre",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05b623510a25212a116f188ca0100b9f",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05b623510a25212a116f188ca0100b9f/impacts"
                }
            },
            {
                "id": "productid.05ca87a7fc5679280cc08d4224e680cd",
                "name": "Extra Special Red Cabbage and Apple in a Mulled Wine Sauce",
                "gtin": "5057172169818",
                "image": "https://static.euw1.sustained.app/products/images/5057172169818_0.jpg",
                "pack": "400g",
                "grade": "B",
                "points": {
                    "pef": 13.310581783759815
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05ca87a7fc5679280cc08d4224e680cd",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05ca87a7fc5679280cc08d4224e680cd/impacts"
                }
            },
            {
                "id": "productid.0628be6fb426d6c3996c01e4bfb77e27",
                "name": "8 Vegan Italia Chipolatas",
                "gtin": "5060317353386",
                "image": "https://static.euw1.sustained.app/products/images/5060317353386_0.jpg",
                "pack": "340g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0628be6fb426d6c3996c01e4bfb77e27",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0628be6fb426d6c3996c01e4bfb77e27/impacts"
                }
            },
            {
                "id": "productid.0631ebb2235032c821fa2068fbee8e5f",
                "name": "Slow Cooked Lamb Shanks with a Mint Gravy",
                "gtin": "0000000135498",
                "image": "https://static.euw1.sustained.app/products/images/0000000135498_0.jpg",
                "pack": "750g",
                "grade": "G",
                "points": {
                    "pef": 355.14991233657804
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0631ebb2235032c821fa2068fbee8e5f",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0631ebb2235032c821fa2068fbee8e5f/impacts"
                }
            },
            {
                "id": "productid.06371982b93dd70cf7695d3ae7b5e1c0",
                "name": "8 Vegetarian Burgers",
                "gtin": "5056053306526",
                "image": "https://static.euw1.sustained.app/products/images/5056053306526_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06371982b93dd70cf7695d3ae7b5e1c0",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06371982b93dd70cf7695d3ae7b5e1c0/impacts"
                }
            },
            {
                "id": "productid.066009a0e2740e6c42d0ea24d4ea3d4c",
                "name": "12 Onion Bhajis",
                "gtin": "5010525050775",
                "image": "https://static.euw1.sustained.app/products/images/5010525050775_0.jpg",
                "pack": "240g",
                "grade": "A",
                "points": {
                    "pef": 9.583538835259093
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.066009a0e2740e6c42d0ea24d4ea3d4c",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.066009a0e2740e6c42d0ea24d4ea3d4c/impacts"
                }
            },
            {
                "id": "productid.0697b7c5193c8ec1b7bddb59bf792f77",
                "name": "32 Beef Meatballs",
                "gtin": "5054781663713",
                "image": "https://static.euw1.sustained.app/products/images/5054781663713_0.jpg",
                "pack": "640g",
                "grade": "G",
                "points": {
                    "pef": 228.35376462818414
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0697b7c5193c8ec1b7bddb59bf792f77",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0697b7c5193c8ec1b7bddb59bf792f77/impacts"
                }
            },
            {
                "id": "productid.069c937541a59d7c9e6422a5b466f725",
                "name": "Deep Pan Baked Spicy Chicken Sizzler",
                "gtin": "5391527075844",
                "image": "https://static.euw1.sustained.app/products/images/5391527075844_0.jpg",
                "pack": "438g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.069c937541a59d7c9e6422a5b466f725",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.069c937541a59d7c9e6422a5b466f725/impacts"
                }
            },
            {
                "id": "productid.06c5bd1fcb822fd50f85f17a8fce1017",
                "name": "14 Onion Bhajis",
                "gtin": "5053947409141",
                "image": "https://static.euw1.sustained.app/products/images/5053947409141_0.jpg",
                "pack": "210g",
                "grade": "A",
                "points": {
                    "pef": 10.542802463714871
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06c5bd1fcb822fd50f85f17a8fce1017",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06c5bd1fcb822fd50f85f17a8fce1017/impacts"
                }
            },
            {
                "id": "productid.06e49f10a187a28eb1fd9245e6223d36",
                "name": "Food Rice & Vegetables",
                "gtin": "0000000314237",
                "image": "https://static.euw1.sustained.app/products/images/0000000314237_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06e49f10a187a28eb1fd9245e6223d36",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06e49f10a187a28eb1fd9245e6223d36/impacts"
                }
            },
            {
                "id": "productid.0771a39def767e606647a5c39fb007e6",
                "name": "12 Southern Fried Chicken Breast Strips",
                "gtin": "5052449623864",
                "image": "https://static.euw1.sustained.app/products/images/5052449623864_0.jpg",
                "pack": "350g",
                "grade": "F",
                "points": {
                    "pef": 61.616479755485756
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0771a39def767e606647a5c39fb007e6",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0771a39def767e606647a5c39fb007e6/impacts"
                }
            },
            {
                "id": "productid.077a99f3302ae0bed4b6010d440fe64a",
                "name": "Food 4 Battered Cod Fillets",
                "gtin": "0000000360005",
                "image": "https://static.euw1.sustained.app/products/images/0000000360005_0.jpg",
                "pack": "540g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.077a99f3302ae0bed4b6010d440fe64a",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.077a99f3302ae0bed4b6010d440fe64a/impacts"
                }
            },
            {
                "id": "productid.078bda5049c82fec3510ab7aae521cf7",
                "name": "Toffee Swirl Cheesecake",
                "gtin": "4008577088144",
                "image": "https://static.euw1.sustained.app/products/images/4008577088144_0.jpg",
                "pack": "405g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.078bda5049c82fec3510ab7aae521cf7",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.078bda5049c82fec3510ab7aae521cf7/impacts"
                }
            },
            {
                "id": "productid.07d977ec69402106c8b79e76d003ad12",
                "name": "Seeded Artisan Style 2 Fish Fillets",
                "gtin": "5000116125302",
                "image": "https://static.euw1.sustained.app/products/images/5000116125302_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07d977ec69402106c8b79e76d003ad12",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07d977ec69402106c8b79e76d003ad12/impacts"
                }
            },
            {
                "id": "productid.084a08774ca56cf95ec0e42a8ace72f2",
                "name": "Triple Chocolate Ice Cream",
                "gtin": "00415729",
                "image": "https://static.euw1.sustained.app/products/images/00415729_0.jpg",
                "pack": "900ml",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.084a08774ca56cf95ec0e42a8ace72f2",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.084a08774ca56cf95ec0e42a8ace72f2/impacts"
                }
            },
            {
                "id": "productid.084c14b921073f454ac5b5457f7c7c71",
                "name": "6 Cheese & Garlic Slices",
                "gtin": "0000000310628",
                "image": "https://static.euw1.sustained.app/products/images/0000000310628_0.jpg",
                "pack": "200g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.084c14b921073f454ac5b5457f7c7c71",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.084c14b921073f454ac5b5457f7c7c71/impacts"
                }
            },
            {
                "id": "productid.0900db4396cec0282023d796ac3996dc",
                "name": "Soft Chocolatey Ice Cream",
                "gtin": "8711327489608",
                "image": "https://static.euw1.sustained.app/products/images/8711327489608_0.jpg",
                "pack": "140ml",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0900db4396cec0282023d796ac3996dc",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0900db4396cec0282023d796ac3996dc/impacts"
                }
            },
            {
                "id": "productid.09111cec9d8c33a31b6d4624e1f76a6a",
                "name": "The Original Strawberry and Blue Raspberry Splits",
                "gtin": "5060202713226",
                "image": "https://static.euw1.sustained.app/products/images/5060202713226_0.jpg",
                "pack": "65ml",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09111cec9d8c33a31b6d4624e1f76a6a",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09111cec9d8c33a31b6d4624e1f76a6a/impacts"
                }
            },
            {
                "id": "productid.09183b03a4001101760e01b0410a7bd8",
                "name": "Scorpion Chilli Chicken Wings",
                "gtin": "5010525123431",
                "image": "https://static.euw1.sustained.app/products/images/5010525123431_0.jpg",
                "pack": "750g",
                "grade": "F",
                "points": {
                    "pef": 62.71432257280309
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09183b03a4001101760e01b0410a7bd8",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09183b03a4001101760e01b0410a7bd8/impacts"
                }
            },
            {
                "id": "productid.0958536f79fe140cd69ece36ed509613",
                "name": "Duo Dark Chocolate & Salted Caramel Crunch Ice Cream",
                "gtin": "3415582301921",
                "image": "https://static.euw1.sustained.app/products/images/3415582301921_0.jpg",
                "pack": "420ml",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0958536f79fe140cd69ece36ed509613",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0958536f79fe140cd69ece36ed509613/impacts"
                }
            },
            {
                "id": "productid.098a0838eb6400ee94daeee2aaa992f7",
                "name": "Ice Cream Cones",
                "gtin": "5010693000107",
                "image": "https://static.euw1.sustained.app/products/images/5010693000107_0.jpg",
                "pack": "100ml",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Frozen Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.098a0838eb6400ee94daeee2aaa992f7",
                    "category": "https://api.sustained.com/choice/v1/categories/frozen-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.098a0838eb6400ee94daeee2aaa992f7/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.0021e89db1142aec30c16ecd8edf9ab1",
                "name": "Innocent Coconut Water",
                "gtin": "5038862136174",
                "image": "https://static.euw1.sustained.app/products/images/5038862136174_0.jpg",
                "pack": "1l",
                "grade": "A",
                "points": {
                    "pef": 6.815352505087667
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0021e89db1142aec30c16ecd8edf9ab1",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0021e89db1142aec30c16ecd8edf9ab1/impacts"
                }
            },
            {
                "id": "productid.0099b592976b987d9a98b4b6c619d011",
                "name": "Tropicana + Power Punch Juice",
                "gtin": "5022313238593",
                "image": "https://static.euw1.sustained.app/products/images/5022313238593_0.jpg",
                "pack": "750ml",
                "grade": "B",
                "points": {
                    "pef": 14.372374608823892
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0099b592976b987d9a98b4b6c619d011",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0099b592976b987d9a98b4b6c619d011/impacts"
                }
            },
            {
                "id": "productid.011a4b67b31370a255e354d74c059fb7",
                "name": "Tesco The Grower's Harvest Apple Juice",
                "gtin": "5057753911478",
                "image": "https://static.euw1.sustained.app/products/images/5057753911478_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 8.273408758766587
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.011a4b67b31370a255e354d74c059fb7",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.011a4b67b31370a255e354d74c059fb7/impacts"
                }
            },
            {
                "id": "productid.0197a50ff7074d2227c396ffabc5bc47",
                "name": "Capri-Sun Fruit Crush Nothing Artificial No Added Sugar Tropical",
                "gtin": "4000177024300",
                "image": "https://static.euw1.sustained.app/products/images/4000177024300_0.jpg",
                "pack": "200ml",
                "grade": "B",
                "points": {
                    "pef": 14.372374608823892
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0197a50ff7074d2227c396ffabc5bc47",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0197a50ff7074d2227c396ffabc5bc47/impacts"
                }
            },
            {
                "id": "productid.01cb47dc25af65bcbc24d182b2e4bddc",
                "name": "Waitrose 50% Pink Grapefruit Juice Squash",
                "gtin": "5000169792360",
                "image": "https://static.euw1.sustained.app/products/images/5000169792360_0.jpg",
                "pack": "1l",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01cb47dc25af65bcbc24d182b2e4bddc",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01cb47dc25af65bcbc24d182b2e4bddc/impacts"
                }
            },
            {
                "id": "productid.02331daff76d69d68b48ea4519bc49b4",
                "name": "The Cracker Drinks Co Pure Squeezed Orange Juice",
                "gtin": "5060170370414",
                "image": "https://static.euw1.sustained.app/products/images/5060170370414_0.jpg",
                "pack": "1l",
                "grade": "D",
                "points": {
                    "pef": 26.30127092870307
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02331daff76d69d68b48ea4519bc49b4",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02331daff76d69d68b48ea4519bc49b4/impacts"
                }
            },
            {
                "id": "productid.0262183247e2ad61499ffffc05c18197",
                "name": "M&S Ginger Zinger Apple, Cold-Pressed Ginger",
                "gtin": "0000000300339",
                "image": "https://static.euw1.sustained.app/products/images/0000000300339_0.jpg",
                "pack": "100ml",
                "grade": "A",
                "points": {
                    "pef": 8.273408758766587
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0262183247e2ad61499ffffc05c18197",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0262183247e2ad61499ffffc05c18197/impacts"
                }
            },
            {
                "id": "productid.03afbe490a8986bf6f6eb34b5ba6a8c4",
                "name": "Asda No Added Sugar Fruit & Barley Pink Grapefruit",
                "gtin": "5057172004874",
                "image": "https://static.euw1.sustained.app/products/images/5057172004874_0.jpg",
                "pack": "1l",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321831
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03afbe490a8986bf6f6eb34b5ba6a8c4",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03afbe490a8986bf6f6eb34b5ba6a8c4/impacts"
                }
            },
            {
                "id": "productid.03d3577d46b2300da60531d53eb2d9fb",
                "name": "Robinsons Superfruit Cordial Raspberry & Goji Berry",
                "gtin": "05010102231719",
                "image": "https://static.euw1.sustained.app/products/images/05010102231719_0.jpg",
                "pack": "500ml",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321838
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03d3577d46b2300da60531d53eb2d9fb",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03d3577d46b2300da60531d53eb2d9fb/impacts"
                }
            },
            {
                "id": "productid.04b92d8b40419b97674c85809793431e",
                "name": "Sunny D Tangy Florida Citrus Fusion",
                "gtin": "5010663815335",
                "image": "https://static.euw1.sustained.app/products/images/5010663815335_0.jpg",
                "pack": "1l",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321838
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04b92d8b40419b97674c85809793431e",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04b92d8b40419b97674c85809793431e/impacts"
                }
            },
            {
                "id": "productid.04cda584dc8887123ed1fa82a24d3c75",
                "name": "Ocean Spray Cranberry Mango",
                "gtin": "0031200468956",
                "image": "https://static.euw1.sustained.app/products/images/0031200468956_0.jpg",
                "pack": "1l",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321838
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04cda584dc8887123ed1fa82a24d3c75",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04cda584dc8887123ed1fa82a24d3c75/impacts"
                }
            },
            {
                "id": "productid.053e13dde2194f2a95e0d740adc2f28b",
                "name": "Oasis Summer Fruits",
                "gtin": "0000090490118",
                "image": "https://static.euw1.sustained.app/products/images/0000090490118_0.jpg",
                "pack": "375ml",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321842
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.053e13dde2194f2a95e0d740adc2f28b",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.053e13dde2194f2a95e0d740adc2f28b/impacts"
                }
            },
            {
                "id": "productid.05789bd21c5e9164d2ba1d0bce3c3fc1",
                "name": "Tesco Apple Juice",
                "gtin": "5000119010674",
                "image": "https://static.euw1.sustained.app/products/images/5000119010674_0.jpg",
                "pack": "1l",
                "grade": "A",
                "points": {
                    "pef": 8.273408758766587
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05789bd21c5e9164d2ba1d0bce3c3fc1",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05789bd21c5e9164d2ba1d0bce3c3fc1/impacts"
                }
            },
            {
                "id": "productid.061ce0f76448c794d2b2c8e0a5259714",
                "name": "Supligen Vanilla Flavour",
                "gtin": "0752046156028",
                "image": "https://static.euw1.sustained.app/products/images/0752046156028_0.jpg",
                "pack": "290ml",
                "grade": "A",
                "points": {
                    "pef": 4.071213095303334
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.061ce0f76448c794d2b2c8e0a5259714",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.061ce0f76448c794d2b2c8e0a5259714/impacts"
                }
            },
            {
                "id": "productid.064c6ddacf0dfe5dd0dcc883cc8fb6b4",
                "name": "Morrisons No Added Sugar Cranberry Juice Drink",
                "gtin": "5010251765226",
                "image": "https://static.euw1.sustained.app/products/images/5010251765226_0.jpg",
                "pack": "1l",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321831
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.064c6ddacf0dfe5dd0dcc883cc8fb6b4",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.064c6ddacf0dfe5dd0dcc883cc8fb6b4/impacts"
                }
            },
            {
                "id": "productid.068f30308807e20eaa35a1a55b08248c",
                "name": "Robinsons Crushed Lime & Mint Fruit Cordial",
                "gtin": "5010102240490",
                "image": "https://static.euw1.sustained.app/products/images/5010102240490_0.jpg",
                "pack": "500ml",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321842
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.068f30308807e20eaa35a1a55b08248c",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.068f30308807e20eaa35a1a55b08248c/impacts"
                }
            },
            {
                "id": "productid.0697307ac2779462d4498a14b9e78af8",
                "name": "Grace Tropical Rhythms Fruit Punch",
                "gtin": "5035139213200",
                "image": "https://static.euw1.sustained.app/products/images/5035139213200_0.jpg",
                "pack": "475ml",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321842
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0697307ac2779462d4498a14b9e78af8",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0697307ac2779462d4498a14b9e78af8/impacts"
                }
            },
            {
                "id": "productid.06bf8391e0fce6a52ca8a3096cdab495",
                "name": "Niru Pulled Jackfruit",
                "gtin": "5060468011128",
                "image": "https://static.euw1.sustained.app/products/images/5060468011128_0.jpg",
                "pack": "540g",
                "grade": "C",
                "points": {
                    "pef": 18.060639701364263
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06bf8391e0fce6a52ca8a3096cdab495",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06bf8391e0fce6a52ca8a3096cdab495/impacts"
                }
            },
            {
                "id": "productid.07343cc461021fad451156f7b177f5b0",
                "name": "The Berry Company Blueberry Juice Drink",
                "gtin": "5034525006624",
                "image": "https://static.euw1.sustained.app/products/images/5034525006624_0.jpg",
                "pack": "1l",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321838
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07343cc461021fad451156f7b177f5b0",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07343cc461021fad451156f7b177f5b0/impacts"
                }
            },
            {
                "id": "productid.07755d756d6460fb81921db0fb6cd96f",
                "name": "innocent Super Smoothie Recharge, Mandarin & Carrot",
                "gtin": "5038862237420",
                "image": "https://static.euw1.sustained.app/products/images/5038862237420_0.jpg",
                "pack": "300ml",
                "grade": "A",
                "points": {
                    "pef": 10.888949090000986
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07755d756d6460fb81921db0fb6cd96f",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07755d756d6460fb81921db0fb6cd96f/impacts"
                }
            },
            {
                "id": "productid.0795c48730cde6d7c688a53a74f45288",
                "name": "Sainsbury's Apple Juice from Concentrate",
                "gtin": "01508550",
                "image": "https://static.euw1.sustained.app/products/images/01508550_0.jpg",
                "pack": "200ml",
                "grade": "A",
                "points": {
                    "pef": 8.273408758766587
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0795c48730cde6d7c688a53a74f45288",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0795c48730cde6d7c688a53a74f45288/impacts"
                }
            },
            {
                "id": "productid.07f85932baa19f4d255b21213b2a0e7c",
                "name": "Robinsons Fruit Shoot Apple & Blackcurrant",
                "gtin": "5010102109353",
                "image": "https://static.euw1.sustained.app/products/images/5010102109353_0.jpg",
                "pack": "200ml",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321842
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07f85932baa19f4d255b21213b2a0e7c",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07f85932baa19f4d255b21213b2a0e7c/impacts"
                }
            },
            {
                "id": "productid.083d43850024943fb712dd34a74581cc",
                "name": "Clamato The Original Tomato Cocktail from Concentrate",
                "gtin": "0014800513240",
                "image": "https://static.euw1.sustained.app/products/images/0014800513240_0.jpg",
                "pack": "946ml",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321838
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.083d43850024943fb712dd34a74581cc",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.083d43850024943fb712dd34a74581cc/impacts"
                }
            },
            {
                "id": "productid.08877a7074364103a367b8fd14ddabf1",
                "name": "Ribena Crisp Pear and Rose Cordial",
                "gtin": "5054267009783",
                "image": "https://static.euw1.sustained.app/products/images/5054267009783_0.jpg",
                "pack": "500ml",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321838
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08877a7074364103a367b8fd14ddabf1",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08877a7074364103a367b8fd14ddabf1/impacts"
                }
            },
            {
                "id": "productid.088e89f6fcf8fdd3d6b1cbfcd29a543f",
                "name": "Lucozade Sport Raspberry",
                "gtin": "5054267006812",
                "image": "https://static.euw1.sustained.app/products/images/5054267006812_0.jpg",
                "pack": "500ml",
                "grade": "A",
                "points": {
                    "pef": 4.071213095303334
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.088e89f6fcf8fdd3d6b1cbfcd29a543f",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.088e89f6fcf8fdd3d6b1cbfcd29a543f/impacts"
                }
            },
            {
                "id": "productid.08b3cb362c86bee72da1c91d364ebaf4",
                "name": "Dr Pepper Zero",
                "gtin": "5017726169312",
                "image": "https://static.euw1.sustained.app/products/images/5017726169312_0.jpg",
                "pack": "330ml",
                "grade": "A",
                "points": {
                    "pef": 4.702260191550754
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08b3cb362c86bee72da1c91d364ebaf4",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08b3cb362c86bee72da1c91d364ebaf4/impacts"
                }
            },
            {
                "id": "productid.0907212769911d1b3543c3b77cdb24fb",
                "name": "Lucozade Sport Raspberry",
                "gtin": "5054267011830",
                "image": "https://static.euw1.sustained.app/products/images/5054267011830_0.jpg",
                "pack": "750ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0907212769911d1b3543c3b77cdb24fb",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0907212769911d1b3543c3b77cdb24fb/impacts"
                }
            },
            {
                "id": "productid.0935678b433f6a45f949f7f4a4af4489",
                "name": "San Pellegrino Grapefruit",
                "gtin": "8002270676772",
                "image": "https://static.euw1.sustained.app/products/images/8002270676772_0.jpg",
                "pack": "330ml",
                "grade": "A",
                "points": {
                    "pef": 4.702260191550754
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0935678b433f6a45f949f7f4a4af4489",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0935678b433f6a45f949f7f4a4af4489/impacts"
                }
            },
            {
                "id": "productid.0a621ba7f54f405a2fb641bf638ce687",
                "name": "Morrisons No Added Sugar Cranberry Juice Drink",
                "gtin": "5010251766452",
                "image": "https://static.euw1.sustained.app/products/images/5010251766452_0.jpg",
                "pack": "1l",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321831
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a621ba7f54f405a2fb641bf638ce687",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a621ba7f54f405a2fb641bf638ce687/impacts"
                }
            },
            {
                "id": "productid.0a94bf9c8d065efef7d4d5b8300a7737",
                "name": "What a Juice Tropical",
                "gtin": "5060167500534",
                "image": "https://static.euw1.sustained.app/products/images/5060167500534_0.jpg",
                "pack": "750ml",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321838
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a94bf9c8d065efef7d4d5b8300a7737",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a94bf9c8d065efef7d4d5b8300a7737/impacts"
                }
            },
            {
                "id": "productid.0ae44bad50dc63394c15cbf4ce3fd97a",
                "name": "Sainsbury's Apple Strawberry Squash",
                "gtin": "01649291",
                "image": "https://static.euw1.sustained.app/products/images/01649291_0.jpg",
                "pack": "1.5l",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ae44bad50dc63394c15cbf4ce3fd97a",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ae44bad50dc63394c15cbf4ce3fd97a/impacts"
                }
            },
            {
                "id": "productid.0b099efb26c4bdce526dbbb5d5553cea",
                "name": "Up & Go Breakfast Drink Vanilla",
                "gtin": "5060406460032",
                "image": "https://static.euw1.sustained.app/products/images/5060406460032_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655623
                },
                "info_icons": [],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b099efb26c4bdce526dbbb5d5553cea",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b099efb26c4bdce526dbbb5d5553cea/impacts"
                }
            },
            {
                "id": "productid.0b37da774a9d9836d851a38dbbde39e1",
                "name": "M&S British Pressed Apple Juice",
                "gtin": "0000000431866",
                "image": "https://static.euw1.sustained.app/products/images/0000000431866_0.jpg",
                "pack": "250ml",
                "grade": "A",
                "points": {
                    "pef": 8.273408758766587
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b37da774a9d9836d851a38dbbde39e1",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b37da774a9d9836d851a38dbbde39e1/impacts"
                }
            },
            {
                "id": "productid.0e1ce4a13fc0325d426123e772f9af96",
                "name": "Vit Hit Sparkling Mango & Pineapple with Rooibos Tea",
                "gtin": "5034033000169",
                "image": "https://static.euw1.sustained.app/products/images/5034033000169_0.jpg",
                "pack": "330ml",
                "grade": "A",
                "points": {
                    "pef": 4.702260191550754
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e1ce4a13fc0325d426123e772f9af96",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e1ce4a13fc0325d426123e772f9af96/impacts"
                }
            },
            {
                "id": "productid.0e288fed535ab27020c4b4d886fd080b",
                "name": "Morrisons Savers No Added Sugar Orange Squash",
                "gtin": "5010251747239",
                "image": "https://static.euw1.sustained.app/products/images/5010251747239_0.jpg",
                "pack": "750ml",
                "grade": "A",
                "points": {
                    "pef": 4.071213095303334
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e288fed535ab27020c4b4d886fd080b",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e288fed535ab27020c4b4d886fd080b/impacts"
                }
            },
            {
                "id": "productid.0ea8cbc5457fbf0d4bb2bc317e8f1df0",
                "name": "James White British Carrot Classic Vegetable Juice",
                "gtin": "5020934520073",
                "image": "https://static.euw1.sustained.app/products/images/5020934520073_0.jpg",
                "pack": "1l",
                "grade": "A",
                "points": {
                    "pef": 8.312495597280835
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ea8cbc5457fbf0d4bb2bc317e8f1df0",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ea8cbc5457fbf0d4bb2bc317e8f1df0/impacts"
                }
            },
            {
                "id": "productid.0ec3d54eb8d06708b6989a7d138f34cf",
                "name": "Sunpride Orange Juice",
                "gtin": "5010663250259",
                "image": "https://static.euw1.sustained.app/products/images/5010663250259_0.jpg",
                "pack": "1l",
                "grade": "B",
                "points": {
                    "pef": 14.372374608823892
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ec3d54eb8d06708b6989a7d138f34cf",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ec3d54eb8d06708b6989a7d138f34cf/impacts"
                }
            },
            {
                "id": "productid.0ee0478f07a1bf90cc0dba8ad1ef60cd",
                "name": "Just Juice Oranges",
                "gtin": "5010663816554",
                "image": "https://static.euw1.sustained.app/products/images/5010663816554_0.jpg",
                "pack": "200ml",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321842
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ee0478f07a1bf90cc0dba8ad1ef60cd",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ee0478f07a1bf90cc0dba8ad1ef60cd/impacts"
                }
            },
            {
                "id": "productid.101d26e69f068c0e376c1cc551637959",
                "name": "Sunmagic Orange & Carrot Juice Drink",
                "gtin": "5013804000744",
                "image": "https://static.euw1.sustained.app/products/images/5013804000744_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.101d26e69f068c0e376c1cc551637959",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.101d26e69f068c0e376c1cc551637959/impacts"
                }
            },
            {
                "id": "productid.10956f2fa5bc6ff8be96e57ff34de7d9",
                "name": "Shloer Zero White Grape Sparkling Zero Calorie Fruit Drink",
                "gtin": "5010388803501",
                "image": "https://static.euw1.sustained.app/products/images/5010388803501_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10956f2fa5bc6ff8be96e57ff34de7d9",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10956f2fa5bc6ff8be96e57ff34de7d9/impacts"
                }
            },
            {
                "id": "productid.1182ab93531605f0cba70961ae5209b8",
                "name": "FIREFLY Lemon, Lime & Ginger Revitalising Botanical Drinks",
                "gtin": "5038512001258",
                "image": "https://static.euw1.sustained.app/products/images/5038512001258_0.jpg",
                "pack": "250ml",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321842
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1182ab93531605f0cba70961ae5209b8",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1182ab93531605f0cba70961ae5209b8/impacts"
                }
            },
            {
                "id": "productid.11988f95d3bd4fc36ec0a738a26ce02a",
                "name": "innocent Smoothies Just for Kids Cherries, Strawberries & Apples",
                "gtin": "5038862630962",
                "image": "https://static.euw1.sustained.app/products/images/5038862630962_0.jpg",
                "pack": "150ml",
                "grade": "A",
                "points": {
                    "pef": 10.888949090000986
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11988f95d3bd4fc36ec0a738a26ce02a",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11988f95d3bd4fc36ec0a738a26ce02a/impacts"
                }
            },
            {
                "id": "productid.12303f0ae07be11c16e8987c0761d2b1",
                "name": "Ribena Strawberry Juice Drink No Added Sugar",
                "gtin": "5054267002319",
                "image": "https://static.euw1.sustained.app/products/images/5054267002319_0.jpg",
                "pack": "500",
                "grade": "A",
                "points": {
                    "pef": 4.071213095303334
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12303f0ae07be11c16e8987c0761d2b1",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12303f0ae07be11c16e8987c0761d2b1/impacts"
                }
            },
            {
                "id": "productid.13d6e9d7d6b2a6f64cfe2a12b46ab2c0",
                "name": "Walshe's 100% Orange Juice",
                "gtin": "5391501240039",
                "image": "https://static.euw1.sustained.app/products/images/5391501240039_0.jpg",
                "pack": "2l",
                "grade": "D",
                "points": {
                    "pef": 26.30127092870307
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.13d6e9d7d6b2a6f64cfe2a12b46ab2c0",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.13d6e9d7d6b2a6f64cfe2a12b46ab2c0/impacts"
                }
            },
            {
                "id": "productid.140d5b55a4a1f8121c154ee425ebe87f",
                "name": "Jack's Orange High Juice",
                "gtin": "5057753082055",
                "image": "https://static.euw1.sustained.app/products/images/5057753082055_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321838
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.140d5b55a4a1f8121c154ee425ebe87f",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.140d5b55a4a1f8121c154ee425ebe87f/impacts"
                }
            },
            {
                "id": "productid.152d0f274eb495c15eb886f46e776b0b",
                "name": "Jaffa Gold Splash No Added Sugar Apple & Blackcurrant Juice Drink",
                "gtin": "5011064011166",
                "image": "https://static.euw1.sustained.app/products/images/5011064011166_0.jpg",
                "pack": "200ml",
                "grade": "A",
                "points": {
                    "pef": 4.071213095303334
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.152d0f274eb495c15eb886f46e776b0b",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.152d0f274eb495c15eb886f46e776b0b/impacts"
                }
            },
            {
                "id": "productid.153718ca837bcd4c0c6f177fd75f64ea",
                "name": "Tesco Tropical",
                "gtin": "5059697756016",
                "image": "https://static.euw1.sustained.app/products/images/5059697756016_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321842
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.153718ca837bcd4c0c6f177fd75f64ea",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.153718ca837bcd4c0c6f177fd75f64ea/impacts"
                }
            },
            {
                "id": "productid.15e879b6a7cd509052b58e6bc5d6b2d1",
                "name": "Ribena Sparkling Raspberry",
                "gtin": "5054267010178",
                "image": "https://static.euw1.sustained.app/products/images/5054267010178_0.jpg",
                "pack": "000",
                "grade": "A",
                "points": {
                    "pef": 4.702260191550754
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.15e879b6a7cd509052b58e6bc5d6b2d1",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.15e879b6a7cd509052b58e6bc5d6b2d1/impacts"
                }
            },
            {
                "id": "productid.16a15c18685fd6e8a4f8c1a1bb8fe202",
                "name": "Carabao Energy Drink Original",
                "gtin": "8855790000868",
                "image": "https://static.euw1.sustained.app/products/images/8855790000868_0.jpg",
                "pack": "330ml",
                "grade": "A",
                "points": {
                    "pef": 5.865153271591771
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16a15c18685fd6e8a4f8c1a1bb8fe202",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16a15c18685fd6e8a4f8c1a1bb8fe202/impacts"
                }
            },
            {
                "id": "productid.16f1a8347ec5f53a3c879b865558e149",
                "name": "Vimto Fizzy",
                "gtin": "5010438012808",
                "image": "https://static.euw1.sustained.app/products/images/5010438012808_0.jpg",
                "pack": "250ml",
                "grade": "A",
                "points": {
                    "pef": 7.922956132321842
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruit Juice & Drinks",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16f1a8347ec5f53a3c879b865558e149",
                    "category": "https://api.sustained.com/choice/v1/categories/fruit-juice-and-drinks",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16f1a8347ec5f53a3c879b865558e149/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.00f452d326ec08ce4554332c0d2d0152",
                "name": "Braeburn Apple",
                "gtin": "03259139",
                "image": "https://static.euw1.sustained.app/products/images/03259139_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.656019205950824
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00f452d326ec08ce4554332c0d2d0152",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00f452d326ec08ce4554332c0d2d0152/impacts"
                }
            },
            {
                "id": "productid.024ad08c7c9163f79fcca6481de1bf0e",
                "name": "Sweet Strawberries",
                "gtin": "53994110",
                "image": "https://static.euw1.sustained.app/products/images/53994110_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 41.016539401995416
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.024ad08c7c9163f79fcca6481de1bf0e",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.024ad08c7c9163f79fcca6481de1bf0e/impacts"
                }
            },
            {
                "id": "productid.028ce39514895716835a0e04214a7d98",
                "name": "Pineapple in Juice",
                "gtin": "5010188000940",
                "image": "https://static.euw1.sustained.app/products/images/5010188000940_0.jpg",
                "pack": "200g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.028ce39514895716835a0e04214a7d98",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.028ce39514895716835a0e04214a7d98/impacts"
                }
            },
            {
                "id": "productid.02a855863b9bf4f64eee4db949ff00ae",
                "name": "Food Select Farms Lemons",
                "gtin": "0000000811705",
                "image": "https://static.euw1.sustained.app/products/images/0000000811705_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 11.488669846592286
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02a855863b9bf4f64eee4db949ff00ae",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02a855863b9bf4f64eee4db949ff00ae/impacts"
                }
            },
            {
                "id": "productid.033a429ce84e45728e6d804cb75f5e0e",
                "name": "Market St Granny Smith Apples x 6",
                "gtin": "5010251731603",
                "image": "https://static.euw1.sustained.app/products/images/5010251731603_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.656019205950824
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.033a429ce84e45728e6d804cb75f5e0e",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.033a429ce84e45728e6d804cb75f5e0e/impacts"
                }
            },
            {
                "id": "productid.0357ae8a8668651d20fc0b95fbcf0318",
                "name": "Taste the Difference Vitoria Seedless Grapes",
                "gtin": "00292177",
                "image": "https://static.euw1.sustained.app/products/images/00292177_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 8.137557667185739
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0357ae8a8668651d20fc0b95fbcf0318",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0357ae8a8668651d20fc0b95fbcf0318/impacts"
                }
            },
            {
                "id": "productid.0443b934f7316d6a9555b778b8f8d2e6",
                "name": "Strawberries",
                "gtin": "5054781593539",
                "image": "https://static.euw1.sustained.app/products/images/5054781593539_0.jpg",
                "pack": "227g",
                "grade": "E",
                "points": {
                    "pef": 41.016539401995416
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0443b934f7316d6a9555b778b8f8d2e6",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0443b934f7316d6a9555b778b8f8d2e6/impacts"
                }
            },
            {
                "id": "productid.05d93b8eef6a503dc37e72c77c79dc06",
                "name": "Figs",
                "gtin": "5000169636794",
                "image": "https://static.euw1.sustained.app/products/images/5000169636794_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 18.060639701364263
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05d93b8eef6a503dc37e72c77c79dc06",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05d93b8eef6a503dc37e72c77c79dc06/impacts"
                }
            },
            {
                "id": "productid.05e4b902794b14b8b92c08fa37bc753b",
                "name": "The Best King Cherries",
                "gtin": "5010251748533",
                "image": "https://static.euw1.sustained.app/products/images/5010251748533_0.jpg",
                "pack": "350g",
                "grade": "E",
                "points": {
                    "pef": 36.72952886718111
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05e4b902794b14b8b92c08fa37bc753b",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05e4b902794b14b8b92c08fa37bc753b/impacts"
                }
            },
            {
                "id": "productid.0695a105725c56b9d71f548d599d9e2b",
                "name": "Apple Slices",
                "gtin": "0000001489613",
                "image": "https://static.euw1.sustained.app/products/images/0000001489613_0.jpg",
                "pack": "80g",
                "grade": "A",
                "points": {
                    "pef": 5.656019205950824
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0695a105725c56b9d71f548d599d9e2b",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0695a105725c56b9d71f548d599d9e2b/impacts"
                }
            },
            {
                "id": "productid.0733c7d195756c162851117c06c2abe5",
                "name": "Food Luxury Fruit Salad Selection",
                "gtin": "0000000886635",
                "image": "https://static.euw1.sustained.app/products/images/0000000886635_0.jpg",
                "pack": "1kg",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0733c7d195756c162851117c06c2abe5",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0733c7d195756c162851117c06c2abe5/impacts"
                }
            },
            {
                "id": "productid.075f54bd60bed7f556aae0239f5872c0",
                "name": "Rosedene Farms Tangy Apple",
                "gtin": "03303733",
                "image": "https://static.euw1.sustained.app/products/images/03303733_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.656019205950824
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.075f54bd60bed7f556aae0239f5872c0",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.075f54bd60bed7f556aae0239f5872c0/impacts"
                }
            },
            {
                "id": "productid.0855dbc5a11b21cf6e89e1691696f932",
                "name": "Loose Peaches",
                "gtin": "03202319",
                "image": "https://static.euw1.sustained.app/products/images/03202319_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 17.936466001782396
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0855dbc5a11b21cf6e89e1691696f932",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0855dbc5a11b21cf6e89e1691696f932/impacts"
                }
            },
            {
                "id": "productid.094944ccf314f1cfd1cccc27c5371ac6",
                "name": "Blackberries",
                "gtin": "5000169113363",
                "image": "https://static.euw1.sustained.app/products/images/5000169113363_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 75.4776037622606
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.094944ccf314f1cfd1cccc27c5371ac6",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.094944ccf314f1cfd1cccc27c5371ac6/impacts"
                }
            },
            {
                "id": "productid.0a0fc012bfb23940c690b61c09dd1820",
                "name": "Grower's Selection Golden Kiwi Fruit",
                "gtin": "5054781883210",
                "image": "https://static.euw1.sustained.app/products/images/5054781883210_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 9.715700409717142
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a0fc012bfb23940c690b61c09dd1820",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a0fc012bfb23940c690b61c09dd1820/impacts"
                }
            },
            {
                "id": "productid.0a28181eb5918360772dc1b9e9243a5e",
                "name": "Mango",
                "gtin": "5000169079256",
                "image": "https://static.euw1.sustained.app/products/images/5000169079256_0.jpg",
                "pack": "280g",
                "grade": "C",
                "points": {
                    "pef": 18.681459534431912
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a28181eb5918360772dc1b9e9243a5e",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a28181eb5918360772dc1b9e9243a5e/impacts"
                }
            },
            {
                "id": "productid.0a63cbe89501f1e2af83ab636f38d7af",
                "name": "Frozen British Blackberries",
                "gtin": "5000169139080",
                "image": "https://static.euw1.sustained.app/products/images/5000169139080_0.jpg",
                "pack": "300g",
                "grade": "F",
                "points": {
                    "pef": 75.4776037622606
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a63cbe89501f1e2af83ab636f38d7af",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a63cbe89501f1e2af83ab636f38d7af/impacts"
                }
            },
            {
                "id": "productid.0a6ac3466200de00276ac258ef941563",
                "name": "Oranges Navel Cambria",
                "gtin": "8431603005895",
                "image": "https://static.euw1.sustained.app/products/images/8431603005895_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 11.4979989778862
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a6ac3466200de00276ac258ef941563",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a6ac3466200de00276ac258ef941563/impacts"
                }
            },
            {
                "id": "productid.0a72c270d9ddcec50a079371d82effc7",
                "name": "Supersweet Peaches",
                "gtin": "10004746",
                "image": "https://static.euw1.sustained.app/products/images/10004746_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 17.936466001782396
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a72c270d9ddcec50a079371d82effc7",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a72c270d9ddcec50a079371d82effc7/impacts"
                }
            },
            {
                "id": "productid.0ab5431dca75aac9aa9a18faa54ca1df",
                "name": "Grapefruit",
                "gtin": "5000169602591",
                "image": "https://static.euw1.sustained.app/products/images/5000169602591_0.jpg",
                "pack": "300g",
                "grade": "A",
                "points": {
                    "pef": 9.224394231023714
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ab5431dca75aac9aa9a18faa54ca1df",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ab5431dca75aac9aa9a18faa54ca1df/impacts"
                }
            },
            {
                "id": "productid.0c65181a55cb5959a6fe667aeb802dca",
                "name": "Food Select Farms Granny Smith Apples",
                "gtin": "0000029092857",
                "image": "https://static.euw1.sustained.app/products/images/0000029092857_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.656019205950824
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c65181a55cb5959a6fe667aeb802dca",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c65181a55cb5959a6fe667aeb802dca/impacts"
                }
            },
            {
                "id": "productid.0d111a06f4a82eb33eca9087ebeee4ab",
                "name": "Blueberries",
                "gtin": "5099400001927",
                "image": "https://static.euw1.sustained.app/products/images/5099400001927_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 21.253186202543986
                },
                "info_icons": [],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d111a06f4a82eb33eca9087ebeee4ab",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d111a06f4a82eb33eca9087ebeee4ab/impacts"
                }
            },
            {
                "id": "productid.0e583360d6c3cfbc663b041e469227e8",
                "name": "Cooks' Ingredients Gooseberries",
                "gtin": "5000169351154",
                "image": "https://static.euw1.sustained.app/products/images/5000169351154_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 29.550664001554424
                },
                "info_icons": [],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e583360d6c3cfbc663b041e469227e8",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e583360d6c3cfbc663b041e469227e8/impacts"
                }
            },
            {
                "id": "productid.0ec552f0ca6c4f077b7bd1c434466b65",
                "name": "Pitted Green Olives with Chilli",
                "gtin": "5057008254398",
                "image": "https://static.euw1.sustained.app/products/images/5057008254398_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 13.787332805957924
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ec552f0ca6c4f077b7bd1c434466b65",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ec552f0ca6c4f077b7bd1c434466b65/impacts"
                }
            },
            {
                "id": "productid.0f03b93f1a9e81b82ec12bd41a0e5ac8",
                "name": "Raspberries",
                "gtin": "03257227",
                "image": "https://static.euw1.sustained.app/products/images/03257227_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 41.06584566346667
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f03b93f1a9e81b82ec12bd41a0e5ac8",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f03b93f1a9e81b82ec12bd41a0e5ac8/impacts"
                }
            },
            {
                "id": "productid.0f2c9db928afa99be7f24b5517dd2649",
                "name": "Melon & Grape",
                "gtin": "0000000200400",
                "image": "https://static.euw1.sustained.app/products/images/0000000200400_0.jpg",
                "pack": "160g",
                "grade": "A",
                "points": {
                    "pef": 8.791786837533769
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f2c9db928afa99be7f24b5517dd2649",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f2c9db928afa99be7f24b5517dd2649/impacts"
                }
            },
            {
                "id": "productid.0fdf0262a2d5755afaf2e059ce0ad01b",
                "name": "Grower's Selection Blueberries",
                "gtin": "5052449404746",
                "image": "https://static.euw1.sustained.app/products/images/5052449404746_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 21.253186202543986
                },
                "info_icons": [],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fdf0262a2d5755afaf2e059ce0ad01b",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fdf0262a2d5755afaf2e059ce0ad01b/impacts"
                }
            },
            {
                "id": "productid.1232ccadf567d3e16a8870600888981f",
                "name": "Finest Charentais Melon",
                "gtin": "03285879",
                "image": "https://static.euw1.sustained.app/products/images/03285879_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 8.94524800094874
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1232ccadf567d3e16a8870600888981f",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1232ccadf567d3e16a8870600888981f/impacts"
                }
            },
            {
                "id": "productid.13154c3e86e3d5c1655e49eb9cdb3d6a",
                "name": "Sweet Strawberries",
                "gtin": "0000053993939",
                "image": "https://static.euw1.sustained.app/products/images/0000053993939_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 41.016539401995416
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.13154c3e86e3d5c1655e49eb9cdb3d6a",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.13154c3e86e3d5c1655e49eb9cdb3d6a/impacts"
                }
            },
            {
                "id": "productid.131ab5b1d8b5354249e0f5e38f99d182",
                "name": "Rainier Cherries",
                "gtin": "5000169292174",
                "image": "https://static.euw1.sustained.app/products/images/5000169292174_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 36.72952886718111
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.131ab5b1d8b5354249e0f5e38f99d182",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.131ab5b1d8b5354249e0f5e38f99d182/impacts"
                }
            },
            {
                "id": "productid.1414e01cc3b0a21d1136c03e28db7c34",
                "name": "The Greengrocer's On Market Street Seasonal Fruit Bowl",
                "gtin": "5010525189185",
                "image": "https://static.euw1.sustained.app/products/images/5010525189185_0.jpg",
                "pack": "400g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1414e01cc3b0a21d1136c03e28db7c34",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1414e01cc3b0a21d1136c03e28db7c34/impacts"
                }
            },
            {
                "id": "productid.141deee6cef5816f6432f34f7d33988e",
                "name": "4 Best of British Apples",
                "gtin": "5000169002742",
                "image": "https://static.euw1.sustained.app/products/images/5000169002742_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.656019205950824
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.141deee6cef5816f6432f34f7d33988e",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.141deee6cef5816f6432f34f7d33988e/impacts"
                }
            },
            {
                "id": "productid.144b537ef3e449b2429d248356a9116a",
                "name": "Mixed Peel",
                "gtin": "5011005010807",
                "image": "https://static.euw1.sustained.app/products/images/5011005010807_0.jpg",
                "pack": "200g",
                "grade": "B",
                "points": {
                    "pef": 11.496133151627415
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.144b537ef3e449b2429d248356a9116a",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.144b537ef3e449b2429d248356a9116a/impacts"
                }
            },
            {
                "id": "productid.150aa8572cfa911d0bd4b1e98910b30b",
                "name": "Blue Whale 6 Organic Gala Apples",
                "gtin": "3430067836552",
                "image": "https://static.euw1.sustained.app/products/images/3430067836552_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.656019205950824
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.150aa8572cfa911d0bd4b1e98910b30b",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.150aa8572cfa911d0bd4b1e98910b30b/impacts"
                }
            },
            {
                "id": "productid.153a21f927a4536b5d026fa09651c9b8",
                "name": "Melon, Stone Fruit & Raspberry",
                "gtin": "0000000425230",
                "image": "https://static.euw1.sustained.app/products/images/0000000425230_0.jpg",
                "pack": "220g",
                "grade": "C",
                "points": {
                    "pef": 16.491346727604824
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.153a21f927a4536b5d026fa09651c9b8",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.153a21f927a4536b5d026fa09651c9b8/impacts"
                }
            },
            {
                "id": "productid.15ba84d430ea87cd92b9ca74f314eaeb",
                "name": "Food Pink Lady Apple",
                "gtin": "0000029025305",
                "image": "https://static.euw1.sustained.app/products/images/0000029025305_0.jpg",
                "pack": "90g",
                "grade": "A",
                "points": {
                    "pef": 5.656019205950824
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.15ba84d430ea87cd92b9ca74f314eaeb",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.15ba84d430ea87cd92b9ca74f314eaeb/impacts"
                }
            },
            {
                "id": "productid.15f2b2bc3f2be3c821509d05a2d76023",
                "name": "Berry Best Twin Pack",
                "gtin": "53993991",
                "image": "https://static.euw1.sustained.app/products/images/53993991_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 41.06584566346667
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.15f2b2bc3f2be3c821509d05a2d76023",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.15f2b2bc3f2be3c821509d05a2d76023/impacts"
                }
            },
            {
                "id": "productid.1677c7ca0a6b1c89ff9dc95543e899e0",
                "name": "Galia Melon",
                "gtin": "10021170",
                "image": "https://static.euw1.sustained.app/products/images/10021170_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 8.94524800094874
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1677c7ca0a6b1c89ff9dc95543e899e0",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1677c7ca0a6b1c89ff9dc95543e899e0/impacts"
                }
            },
            {
                "id": "productid.16ad32cbf57e3f97f41b088dcd1e4b58",
                "name": "Mango",
                "gtin": "03308288",
                "image": "https://static.euw1.sustained.app/products/images/03308288_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 18.681459534431912
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16ad32cbf57e3f97f41b088dcd1e4b58",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16ad32cbf57e3f97f41b088dcd1e4b58/impacts"
                }
            },
            {
                "id": "productid.16af5e119e274df8e59244727441db9f",
                "name": "Nectarine, Black Grape and Strawberry in Season",
                "gtin": "5000169559970",
                "image": "https://static.euw1.sustained.app/products/images/5000169559970_0.jpg",
                "pack": "200g",
                "grade": "D",
                "points": {
                    "pef": 23.73113885978422
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16af5e119e274df8e59244727441db9f",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16af5e119e274df8e59244727441db9f/impacts"
                }
            },
            {
                "id": "productid.1753bb2ca7cf480ae9f5fa9413b47299",
                "name": "The Collection Greek Double Stuffed Halkidiki Olives",
                "gtin": "0000000157070",
                "image": "https://static.euw1.sustained.app/products/images/0000000157070_0.jpg",
                "pack": "150g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1753bb2ca7cf480ae9f5fa9413b47299",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1753bb2ca7cf480ae9f5fa9413b47299/impacts"
                }
            },
            {
                "id": "productid.178f5bd71bd47b908534dd240f5822d9",
                "name": "Mandarin in Juice",
                "gtin": "9310006007868",
                "image": "https://static.euw1.sustained.app/products/images/9310006007868_0.jpg",
                "pack": "390g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.178f5bd71bd47b908534dd240f5822d9",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.178f5bd71bd47b908534dd240f5822d9/impacts"
                }
            },
            {
                "id": "productid.17d3d3ff543f74bc8b6c2af751d3411a",
                "name": "The Greengrocer's On Market Street Melon & Grape",
                "gtin": "09018068",
                "image": "https://static.euw1.sustained.app/products/images/09018068_0.jpg",
                "pack": "320g",
                "grade": "A",
                "points": {
                    "pef": 8.783709934196139
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.17d3d3ff543f74bc8b6c2af751d3411a",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.17d3d3ff543f74bc8b6c2af751d3411a/impacts"
                }
            },
            {
                "id": "productid.1845d2dff3c2d015fd7832feb332f7aa",
                "name": "Seville Loose Orange",
                "gtin": "5000169977583",
                "image": "https://static.euw1.sustained.app/products/images/5000169977583_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 11.4979989778862
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1845d2dff3c2d015fd7832feb332f7aa",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1845d2dff3c2d015fd7832feb332f7aa/impacts"
                }
            },
            {
                "id": "productid.1a9d83f09588e5a73adba49d05141520",
                "name": "Halkidiki Olives Stuffed with Garlic Cloves",
                "gtin": "5057545898826",
                "image": "https://static.euw1.sustained.app/products/images/5057545898826_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 13.787332805957924
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1a9d83f09588e5a73adba49d05141520",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1a9d83f09588e5a73adba49d05141520/impacts"
                }
            },
            {
                "id": "productid.1b867551faa422665b3f81eebd8a3fd5",
                "name": "No1 Soft Pineapple Rings",
                "gtin": "5000169590140",
                "image": "https://static.euw1.sustained.app/products/images/5000169590140_0.jpg",
                "pack": "125g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b867551faa422665b3f81eebd8a3fd5",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b867551faa422665b3f81eebd8a3fd5/impacts"
                }
            },
            {
                "id": "productid.1c910efdc6c29c0701d1835e46f13d4c",
                "name": "Extra Special Blueberries",
                "gtin": "5054781593478",
                "image": "https://static.euw1.sustained.app/products/images/5054781593478_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 21.253186202543986
                },
                "info_icons": [],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c910efdc6c29c0701d1835e46f13d4c",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c910efdc6c29c0701d1835e46f13d4c/impacts"
                }
            },
            {
                "id": "productid.1ca4acf378bed1c67f1e22bd2b4c6769",
                "name": "Raspberries",
                "gtin": "03272671",
                "image": "https://static.euw1.sustained.app/products/images/03272671_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 41.06584566346667
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1ca4acf378bed1c67f1e22bd2b4c6769",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1ca4acf378bed1c67f1e22bd2b4c6769/impacts"
                }
            },
            {
                "id": "productid.1cb0785fd1c99cdff55a7ad98e01e27e",
                "name": "Naturally Wonky Berries",
                "gtin": "5010251850564",
                "image": "https://static.euw1.sustained.app/products/images/5010251850564_0.jpg",
                "pack": "125g",
                "grade": "E",
                "points": {
                    "pef": 41.06584566346667
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1cb0785fd1c99cdff55a7ad98e01e27e",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1cb0785fd1c99cdff55a7ad98e01e27e/impacts"
                }
            },
            {
                "id": "productid.1d4bb0306f3f849af9bfdc3b043e87d6",
                "name": "Rosedene Farms Small Apples 6 Pack",
                "gtin": "0000003266090",
                "image": "https://static.euw1.sustained.app/products/images/0000003266090_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.656019205950824
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Fruits",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1d4bb0306f3f849af9bfdc3b043e87d6",
                    "category": "https://api.sustained.com/choice/v1/categories/fruits",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1d4bb0306f3f849af9bfdc3b043e87d6/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.0c6455a6961c38770a908804ed04ca98",
                "name": "Roast in Bag Whole Duck",
                "gtin": "5034231004365",
                "image": "https://static.euw1.sustained.app/products/images/5034231004365_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c6455a6961c38770a908804ed04ca98",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c6455a6961c38770a908804ed04ca98/impacts"
                }
            },
            {
                "id": "productid.1935a41307e845d4189e66c6b422aaa7",
                "name": "2 Pheasant Breast Fillets",
                "gtin": "5030823002016",
                "image": "https://static.euw1.sustained.app/products/images/5030823002016_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 63.76599528811803
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1935a41307e845d4189e66c6b422aaa7",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1935a41307e845d4189e66c6b422aaa7/impacts"
                }
            },
            {
                "id": "productid.5b5d487857c259ba3ba62860c30abeb9",
                "name": "Part Deboned Stuffed Whole Duck",
                "gtin": "5099459714410",
                "image": "https://static.euw1.sustained.app/products/images/5099459714410_0.jpg",
                "pack": "1.2kg",
                "grade": "F",
                "points": {
                    "pef": 64.22654912169463
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5b5d487857c259ba3ba62860c30abeb9",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5b5d487857c259ba3ba62860c30abeb9/impacts"
                }
            },
            {
                "id": "productid.6cf3fadb6795470f152a6f5281825a21",
                "name": "Roast Half Duck",
                "gtin": "5099459813007",
                "image": "https://static.euw1.sustained.app/products/images/5099459813007_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.29873617290066
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.6cf3fadb6795470f152a6f5281825a21",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.6cf3fadb6795470f152a6f5281825a21/impacts"
                }
            },
            {
                "id": "productid.709e3a0483fa9e61dcaf96c1305b552e",
                "name": "2 Duck Breast Fillets",
                "gtin": "5099459760240",
                "image": "https://static.euw1.sustained.app/products/images/5099459760240_0.jpg",
                "pack": "350g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.709e3a0483fa9e61dcaf96c1305b552e",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.709e3a0483fa9e61dcaf96c1305b552e/impacts"
                }
            },
            {
                "id": "productid.7aee26889ebae96dfdcb0fa47f8f80cd",
                "name": "2 British Fresh Duck Breast Fillets",
                "gtin": "0000001800906",
                "image": "https://static.euw1.sustained.app/products/images/0000001800906_0.jpg",
                "pack": "340g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.7aee26889ebae96dfdcb0fa47f8f80cd",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.7aee26889ebae96dfdcb0fa47f8f80cd/impacts"
                }
            },
            {
                "id": "productid.8cc0b28c5a5234d718c40e0e82d2fbc4",
                "name": "Food Effortless Entertaining Slow Cooked Lightly Smoked Duck Legs",
                "gtin": "29282920",
                "image": "https://static.euw1.sustained.app/products/images/29282920_0.jpg",
                "pack": "1.07kg",
                "grade": "F",
                "points": {
                    "pef": 56.52027869923324
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.8cc0b28c5a5234d718c40e0e82d2fbc4",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.8cc0b28c5a5234d718c40e0e82d2fbc4/impacts"
                }
            },
            {
                "id": "productid.a6516c93686f3422d88b684d6d40dac7",
                "name": "No1 Game Casserole",
                "gtin": "5000169027684",
                "image": "https://static.euw1.sustained.app/products/images/5000169027684_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.a6516c93686f3422d88b684d6d40dac7",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.a6516c93686f3422d88b684d6d40dac7/impacts"
                }
            },
            {
                "id": "productid.ae5678094ffc31da591785ab7651b624",
                "name": "Venison Fillet",
                "gtin": "5060120623164",
                "image": "https://static.euw1.sustained.app/products/images/5060120623164_0.jpg",
                "pack": "170g",
                "grade": "F",
                "points": {
                    "pef": 76.97412842299914
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.ae5678094ffc31da591785ab7651b624",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.ae5678094ffc31da591785ab7651b624/impacts"
                }
            },
            {
                "id": "productid.c56e11bb7c30420dc9cc19b278a40f95",
                "name": "Free Range Goose",
                "gtin": "0279580000001",
                "image": "https://static.euw1.sustained.app/products/images/0279580000001_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.c56e11bb7c30420dc9cc19b278a40f95",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.c56e11bb7c30420dc9cc19b278a40f95/impacts"
                }
            },
            {
                "id": "productid.d0955baff71d51fbce8b67c0f99a5640",
                "name": "Gressingham Duck",
                "gtin": "5034231521206",
                "image": "https://static.euw1.sustained.app/products/images/5034231521206_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.d0955baff71d51fbce8b67c0f99a5640",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.d0955baff71d51fbce8b67c0f99a5640/impacts"
                }
            },
            {
                "id": "productid.eaddfa175b9e2c953cea1ed856cfcad6",
                "name": "Half Duck",
                "gtin": "5099459760349",
                "image": "https://static.euw1.sustained.app/products/images/5099459760349_0.jpg",
                "pack": "750g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.eaddfa175b9e2c953cea1ed856cfcad6",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.eaddfa175b9e2c953cea1ed856cfcad6/impacts"
                }
            },
            {
                "id": "productid.4264fa6819c906c039ed2eb4c0ab7e78",
                "name": "Young German Duck",
                "gtin": "4005614000282",
                "image": "https://static.euw1.sustained.app/products/images/4005614000282_0.jpg",
                "pack": "2800g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4264fa6819c906c039ed2eb4c0ab7e78",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4264fa6819c906c039ed2eb4c0ab7e78/impacts"
                }
            },
            {
                "id": "productid.56a7368bc7513344d6cdaea401f61e27",
                "name": "Whole Duck Without Giblets",
                "gtin": "5099459760356",
                "image": "https://static.euw1.sustained.app/products/images/5099459760356_0.jpg",
                "pack": "2200g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.56a7368bc7513344d6cdaea401f61e27",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.56a7368bc7513344d6cdaea401f61e27/impacts"
                }
            },
            {
                "id": "productid.5a2c4d229a1babbfcd941e533217681b",
                "name": "Duck Legs",
                "gtin": "5016157075827",
                "image": "https://static.euw1.sustained.app/products/images/5016157075827_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5a2c4d229a1babbfcd941e533217681b",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5a2c4d229a1babbfcd941e533217681b/impacts"
                }
            },
            {
                "id": "productid.81f8230d55b7b167c00fdffe6c03edd1",
                "name": "Free Range Goose",
                "gtin": "0261520000004",
                "image": "https://static.euw1.sustained.app/products/images/0261520000004_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.81f8230d55b7b167c00fdffe6c03edd1",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.81f8230d55b7b167c00fdffe6c03edd1/impacts"
                }
            },
            {
                "id": "productid.84c7201c3a79099cdfbdc2a67874802b",
                "name": "Free Range Goose",
                "gtin": "5016157078200",
                "image": "https://static.euw1.sustained.app/products/images/5016157078200_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.84c7201c3a79099cdfbdc2a67874802b",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.84c7201c3a79099cdfbdc2a67874802b/impacts"
                }
            },
            {
                "id": "productid.8504faac761e553e0bd40f0a52e151c3",
                "name": "2 Deliciously Simple Duck Legs",
                "gtin": "5034231003924",
                "image": "https://static.euw1.sustained.app/products/images/5034231003924_0.jpg",
                "pack": "462g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.8504faac761e553e0bd40f0a52e151c3",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.8504faac761e553e0bd40f0a52e151c3/impacts"
                }
            },
            {
                "id": "productid.87f1222ad34a498d76fe7db9939a87d3",
                "name": "Whole Duck Without Giblets",
                "gtin": "5099459905078",
                "image": "https://static.euw1.sustained.app/products/images/5099459905078_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.87f1222ad34a498d76fe7db9939a87d3",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.87f1222ad34a498d76fe7db9939a87d3/impacts"
                }
            },
            {
                "id": "productid.a76bdbc29cefc1a3d191963871b7cb19",
                "name": "Whole Duck",
                "gtin": "5099459714106",
                "image": "https://static.euw1.sustained.app/products/images/5099459714106_0.jpg",
                "pack": "2000g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.a76bdbc29cefc1a3d191963871b7cb19",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.a76bdbc29cefc1a3d191963871b7cb19/impacts"
                }
            },
            {
                "id": "productid.5a0183f887bd09fb2d072fe711aca99f",
                "name": "Stir Fry Duck Breast",
                "gtin": "5016157078347",
                "image": "https://static.euw1.sustained.app/products/images/5016157078347_0.jpg",
                "pack": "175g",
                "grade": "F",
                "points": {
                    "pef": 80.99455205651094
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5a0183f887bd09fb2d072fe711aca99f",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5a0183f887bd09fb2d072fe711aca99f/impacts"
                }
            },
            {
                "id": "productid.5b8584895ae291adce269fd77fb705fa",
                "name": "Gressingham Duck",
                "gtin": "5034231521107",
                "image": "https://static.euw1.sustained.app/products/images/5034231521107_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5b8584895ae291adce269fd77fb705fa",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5b8584895ae291adce269fd77fb705fa/impacts"
                }
            },
            {
                "id": "productid.7ea19dfcb84b2c1c3c38a0ef19698357",
                "name": "Easy Carve Duck with Seasonal Sausage Stuffing & Orange Glaze",
                "gtin": "5099874050759",
                "image": "https://static.euw1.sustained.app/products/images/5099874050759_0.jpg",
                "pack": "1400g",
                "grade": "F",
                "points": {
                    "pef": 72.36818941140339
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.7ea19dfcb84b2c1c3c38a0ef19698357",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.7ea19dfcb84b2c1c3c38a0ef19698357/impacts"
                }
            },
            {
                "id": "productid.84503599c4b45ed53592a0db2252b54d",
                "name": "Aromatic Duck Legs",
                "gtin": "5099459002753",
                "image": "https://static.euw1.sustained.app/products/images/5099459002753_0.jpg",
                "pack": "500g",
                "grade": "F",
                "points": {
                    "pef": 64.12616391554963
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.84503599c4b45ed53592a0db2252b54d",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.84503599c4b45ed53592a0db2252b54d/impacts"
                }
            },
            {
                "id": "productid.88a57d55b8c861503eb6b686c8169acd",
                "name": "Food Collection British Wild Game Pheasant Breast Joint",
                "gtin": "0000000760096",
                "image": "https://static.euw1.sustained.app/products/images/0000000760096_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 75.88308448940792
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.88a57d55b8c861503eb6b686c8169acd",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.88a57d55b8c861503eb6b686c8169acd/impacts"
                }
            },
            {
                "id": "productid.8ddc407f74fee83492c636d1e5bfade6",
                "name": "2 Deliciously Simple Duck Legs",
                "gtin": "5016157076060",
                "image": "https://static.euw1.sustained.app/products/images/5016157076060_0.jpg",
                "pack": "440g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.8ddc407f74fee83492c636d1e5bfade6",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.8ddc407f74fee83492c636d1e5bfade6/impacts"
                }
            },
            {
                "id": "productid.a0c30463f0d12788a7cbd59ca487e1a1",
                "name": "Diced Game Casserole Mix",
                "gtin": "5030823002023",
                "image": "https://static.euw1.sustained.app/products/images/5030823002023_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.a0c30463f0d12788a7cbd59ca487e1a1",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.a0c30463f0d12788a7cbd59ca487e1a1/impacts"
                }
            },
            {
                "id": "productid.d53122e28c56f044d3edcec28b9a8a32",
                "name": "No1 Partridge Breast Fillets",
                "gtin": "5000169350621",
                "image": "https://static.euw1.sustained.app/products/images/5000169350621_0.jpg",
                "pack": "150g",
                "grade": "F",
                "points": {
                    "pef": 63.76599528811803
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.d53122e28c56f044d3edcec28b9a8a32",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.d53122e28c56f044d3edcec28b9a8a32/impacts"
                }
            },
            {
                "id": "productid.d9aacafd72f7aa18b5acc054d8a38387",
                "name": "Food Collection British Whole Duck",
                "gtin": "0000000962834",
                "image": "https://static.euw1.sustained.app/products/images/0000000962834_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.d9aacafd72f7aa18b5acc054d8a38387",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.d9aacafd72f7aa18b5acc054d8a38387/impacts"
                }
            },
            {
                "id": "productid.e40978d37899f7c20025455505af5fd5",
                "name": "No1 Easy to Carve Pheasant",
                "gtin": "5000169027653",
                "image": "https://static.euw1.sustained.app/products/images/5000169027653_0.jpg",
                "pack": "510g",
                "grade": "F",
                "points": {
                    "pef": 63.0962712356359
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.e40978d37899f7c20025455505af5fd5",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.e40978d37899f7c20025455505af5fd5/impacts"
                }
            },
            {
                "id": "productid.22362eab3485d227f5287456d9dbf59e",
                "name": "Duck Breast",
                "gtin": "5034231002460",
                "image": "https://static.euw1.sustained.app/products/images/5034231002460_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 80.99455205651094
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.22362eab3485d227f5287456d9dbf59e",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.22362eab3485d227f5287456d9dbf59e/impacts"
                }
            },
            {
                "id": "productid.27a95211b82b205356fcf4a0c5dc3f07",
                "name": "Young German Duck",
                "gtin": "4005614000268",
                "image": "https://static.euw1.sustained.app/products/images/4005614000268_0.jpg",
                "pack": "2600g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.27a95211b82b205356fcf4a0c5dc3f07",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.27a95211b82b205356fcf4a0c5dc3f07/impacts"
                }
            },
            {
                "id": "productid.4d32a89c5a32e9ee18437b66b2b2e656",
                "name": "No1 Pheasant Breast Fillets",
                "gtin": "5000169027646",
                "image": "https://static.euw1.sustained.app/products/images/5000169027646_0.jpg",
                "pack": "150g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4d32a89c5a32e9ee18437b66b2b2e656",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4d32a89c5a32e9ee18437b66b2b2e656/impacts"
                }
            },
            {
                "id": "productid.777ebee4731a321e7013bb9416871cac",
                "name": "Fresh Irish Duck",
                "gtin": "5099459763029",
                "image": "https://static.euw1.sustained.app/products/images/5099459763029_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.777ebee4731a321e7013bb9416871cac",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.777ebee4731a321e7013bb9416871cac/impacts"
                }
            },
            {
                "id": "productid.7e61821a5badb113b1be96338fcba84d",
                "name": "Roast in the Bag Duck Crown",
                "gtin": "5016157075995",
                "image": "https://static.euw1.sustained.app/products/images/5016157075995_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.7e61821a5badb113b1be96338fcba84d",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.7e61821a5badb113b1be96338fcba84d/impacts"
                }
            },
            {
                "id": "productid.83a24ca0af5055d5a098258918c5a9ca",
                "name": "No1 Pheasant Breast Joint",
                "gtin": "5000169350584",
                "image": "https://static.euw1.sustained.app/products/images/5000169350584_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 60.03698465038208
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.83a24ca0af5055d5a098258918c5a9ca",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.83a24ca0af5055d5a098258918c5a9ca/impacts"
                }
            },
            {
                "id": "productid.88ab7a8704d35deb4877eaf212f836a8",
                "name": "Impressively Easy Duck Breast Portions",
                "gtin": "5016157077586",
                "image": "https://static.euw1.sustained.app/products/images/5016157077586_0.jpg",
                "pack": "225g",
                "grade": "F",
                "points": {
                    "pef": 80.99455205651094
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.88ab7a8704d35deb4877eaf212f836a8",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.88ab7a8704d35deb4877eaf212f836a8/impacts"
                }
            },
            {
                "id": "productid.92c1c3630fc59b24a19c6583b8b671cd",
                "name": "Food Collection British Wild Game 2 Mallard Duck Breast Fillets",
                "gtin": "0000000760102",
                "image": "https://static.euw1.sustained.app/products/images/0000000760102_0.jpg",
                "pack": "140g",
                "grade": "F",
                "points": {
                    "pef": 80.99455205651094
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.92c1c3630fc59b24a19c6583b8b671cd",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.92c1c3630fc59b24a19c6583b8b671cd/impacts"
                }
            },
            {
                "id": "productid.984a1714ff9bc1006bf858211a9e460f",
                "name": "Deliciously Simple Whole Duck",
                "gtin": "5016157078880",
                "image": "https://static.euw1.sustained.app/products/images/5016157078880_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.984a1714ff9bc1006bf858211a9e460f",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.984a1714ff9bc1006bf858211a9e460f/impacts"
                }
            },
            {
                "id": "productid.9b77f2fdba1b6ecab49581ede1646fb0",
                "name": "Aromatic Shredded Duck",
                "gtin": "5016157078323",
                "image": "https://static.euw1.sustained.app/products/images/5016157078323_0.jpg",
                "pack": "210g",
                "grade": "E",
                "points": {
                    "pef": 46.11598487600095
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.9b77f2fdba1b6ecab49581ede1646fb0",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.9b77f2fdba1b6ecab49581ede1646fb0/impacts"
                }
            },
            {
                "id": "productid.b47cd4dff4bd1342e8017dc43b1a2019",
                "name": "2 Duck Breast Fillets",
                "gtin": "5099459914612",
                "image": "https://static.euw1.sustained.app/products/images/5099459914612_0.jpg",
                "pack": "350g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.b47cd4dff4bd1342e8017dc43b1a2019",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.b47cd4dff4bd1342e8017dc43b1a2019/impacts"
                }
            },
            {
                "id": "productid.bf8ae693ce36e45e9e10eb88b75b8f4f",
                "name": "No1 Partridge Topped with Bacon",
                "gtin": "5000169027721",
                "image": "https://static.euw1.sustained.app/products/images/5000169027721_0.jpg",
                "pack": "225g",
                "grade": "F",
                "points": {
                    "pef": 67.05255114833294
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.bf8ae693ce36e45e9e10eb88b75b8f4f",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.bf8ae693ce36e45e9e10eb88b75b8f4f/impacts"
                }
            },
            {
                "id": "productid.ce27549c156dc8f8bc5167a14390fe3b",
                "name": "Roast Half Duck",
                "gtin": "5099459760103",
                "image": "https://static.euw1.sustained.app/products/images/5099459760103_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.29873617290066
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.ce27549c156dc8f8bc5167a14390fe3b",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.ce27549c156dc8f8bc5167a14390fe3b/impacts"
                }
            },
            {
                "id": "productid.f545d6b1ec1ec2f15bf0aa8069866031",
                "name": "Whole Duck",
                "gtin": "5034231550787",
                "image": "https://static.euw1.sustained.app/products/images/5034231550787_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.f545d6b1ec1ec2f15bf0aa8069866031",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.f545d6b1ec1ec2f15bf0aa8069866031/impacts"
                }
            },
            {
                "id": "productid.046872d7a94f4b47e07d3b26e9032802",
                "name": "2 British Fresh Duck Legs",
                "gtin": "0000001800913",
                "image": "https://static.euw1.sustained.app/products/images/0000001800913_0.jpg",
                "pack": "440g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.046872d7a94f4b47e07d3b26e9032802",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.046872d7a94f4b47e07d3b26e9032802/impacts"
                }
            },
            {
                "id": "productid.092b40c4277dd7ca28fbe36332394edd",
                "name": "No1 Free Range Guinea Fowl",
                "gtin": "5000169218600",
                "image": "https://static.euw1.sustained.app/products/images/5000169218600_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 80.99455205651097
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.092b40c4277dd7ca28fbe36332394edd",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.092b40c4277dd7ca28fbe36332394edd/impacts"
                }
            },
            {
                "id": "productid.122f0278c4b00dcadfd9b3ee969d17ff",
                "name": "Impressively Easy Duck Breasts",
                "gtin": "5016157077890",
                "image": "https://static.euw1.sustained.app/products/images/5016157077890_0.jpg",
                "pack": "300g",
                "grade": "F",
                "points": {
                    "pef": 80.99455205651094
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.122f0278c4b00dcadfd9b3ee969d17ff",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.122f0278c4b00dcadfd9b3ee969d17ff/impacts"
                }
            },
            {
                "id": "productid.2418afe18b7af8c3bc066fdfa19ccea6",
                "name": "Impressively Easy Duck Breasts",
                "gtin": "5016157074714",
                "image": "https://static.euw1.sustained.app/products/images/5016157074714_0.jpg",
                "pack": "340g",
                "grade": "F",
                "points": {
                    "pef": 80.99455205651094
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2418afe18b7af8c3bc066fdfa19ccea6",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2418afe18b7af8c3bc066fdfa19ccea6/impacts"
                }
            },
            {
                "id": "productid.4206fa55c6b95fe4783f4f29f25c1c70",
                "name": "2 Duck Breast Fillets",
                "gtin": "5099839319518",
                "image": "https://static.euw1.sustained.app/products/images/5099839319518_0.jpg",
                "pack": "350g",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908454
                },
                "info_icons": [],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4206fa55c6b95fe4783f4f29f25c1c70",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4206fa55c6b95fe4783f4f29f25c1c70/impacts"
                }
            },
            {
                "id": "productid.515da1bfbad74a501307b2406b4fdbc0",
                "name": "Taste the Difference British Whole Pheasant",
                "gtin": "01440447",
                "image": "https://static.euw1.sustained.app/products/images/01440447_0.jpg",
                "pack": "500g",
                "grade": "F",
                "points": {
                    "pef": 63.76599528811803
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Game",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.515da1bfbad74a501307b2406b4fdbc0",
                    "category": "https://api.sustained.com/choice/v1/categories/game",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.515da1bfbad74a501307b2406b4fdbc0/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.02ca209e2db813503d73c09678c17e87",
                "name": "Grower's Selection Sage",
                "gtin": "5051413651940",
                "image": "https://static.euw1.sustained.app/products/images/5051413651940_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.013312798574262
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02ca209e2db813503d73c09678c17e87",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02ca209e2db813503d73c09678c17e87/impacts"
                }
            },
            {
                "id": "productid.077127d4ff046dc0bb3a05594470453a",
                "name": "Food Select Farms Basil",
                "gtin": "0000000833196",
                "image": "https://static.euw1.sustained.app/products/images/0000000833196_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.013312798574258
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.077127d4ff046dc0bb3a05594470453a",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.077127d4ff046dc0bb3a05594470453a/impacts"
                }
            },
            {
                "id": "productid.09020d218a769968ddedb0c14bd606bb",
                "name": "Irish Fresh Herbs Flat Leaf Parsley Delicate and Fresh Flavour",
                "gtin": "10079171",
                "image": "https://static.euw1.sustained.app/products/images/10079171_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.321220567533148
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09020d218a769968ddedb0c14bd606bb",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09020d218a769968ddedb0c14bd606bb/impacts"
                }
            },
            {
                "id": "productid.0afa1ba91ea1d61db4bcb31125d5f6e7",
                "name": "Mint",
                "gtin": "0000003282854",
                "image": "https://static.euw1.sustained.app/products/images/0000003282854_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0afa1ba91ea1d61db4bcb31125d5f6e7",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0afa1ba91ea1d61db4bcb31125d5f6e7/impacts"
                }
            },
            {
                "id": "productid.15f5c096dfcbb5c1b7cf24dd2da4c5e3",
                "name": "The Greengrocer's On Market Street Fresh Flat-Leaf Parsley",
                "gtin": "5010251970040",
                "image": "https://static.euw1.sustained.app/products/images/5010251970040_0.jpg",
                "pack": "30g",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.15f5c096dfcbb5c1b7cf24dd2da4c5e3",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.15f5c096dfcbb5c1b7cf24dd2da4c5e3/impacts"
                }
            },
            {
                "id": "productid.19ad3a138cd0feb7111377a6d49401e8",
                "name": "Fresh Mint",
                "gtin": "5010251717010",
                "image": "https://static.euw1.sustained.app/products/images/5010251717010_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.19ad3a138cd0feb7111377a6d49401e8",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.19ad3a138cd0feb7111377a6d49401e8/impacts"
                }
            },
            {
                "id": "productid.1b3e39f885148f40bda33c6605e7404f",
                "name": "British Coriander Small Pot",
                "gtin": "0000003283288",
                "image": "https://static.euw1.sustained.app/products/images/0000003283288_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 10.81169255215334
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b3e39f885148f40bda33c6605e7404f",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b3e39f885148f40bda33c6605e7404f/impacts"
                }
            },
            {
                "id": "productid.1d11ca7ded1d53eab98ab1231d3eea3f",
                "name": "Irish Fresh Herbs Coriander Aromatic & Citrusy Flavour",
                "gtin": "10079195",
                "image": "https://static.euw1.sustained.app/products/images/10079195_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 10.81169255215334
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1d11ca7ded1d53eab98ab1231d3eea3f",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1d11ca7ded1d53eab98ab1231d3eea3f/impacts"
                }
            },
            {
                "id": "productid.3e9b8f0e59b9ddbf6b5cc690f2e45ee0",
                "name": "The Greengrocer's On Market Street Fresh Dill",
                "gtin": "5010251961710",
                "image": "https://static.euw1.sustained.app/products/images/5010251961710_0.jpg",
                "pack": "20g",
                "grade": "A",
                "points": {
                    "pef": 11.013312798574258
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3e9b8f0e59b9ddbf6b5cc690f2e45ee0",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3e9b8f0e59b9ddbf6b5cc690f2e45ee0/impacts"
                }
            },
            {
                "id": "productid.40a11f7e556aa2e760dfe7972fd50ac7",
                "name": "Curled Leaf Parsley",
                "gtin": "10053447",
                "image": "https://static.euw1.sustained.app/products/images/10053447_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.40a11f7e556aa2e760dfe7972fd50ac7",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.40a11f7e556aa2e760dfe7972fd50ac7/impacts"
                }
            },
            {
                "id": "productid.40a9da455e72c51ba5d5049530c1b11f",
                "name": "Herbs Mint",
                "gtin": "5011182003913",
                "image": "https://static.euw1.sustained.app/products/images/5011182003913_0.jpg",
                "pack": "20g",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.40a9da455e72c51ba5d5049530c1b11f",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.40a9da455e72c51ba5d5049530c1b11f/impacts"
                }
            },
            {
                "id": "productid.43dfcc5988232dae7a6b39e23f7bc186",
                "name": "Oregano",
                "gtin": "5012836753826",
                "image": "https://static.euw1.sustained.app/products/images/5012836753826_0.jpg",
                "pack": "40g",
                "grade": "C",
                "points": {
                    "pef": 16.783131490079057
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.43dfcc5988232dae7a6b39e23f7bc186",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.43dfcc5988232dae7a6b39e23f7bc186/impacts"
                }
            },
            {
                "id": "productid.49b578b7f22105af5fae04033a26814e",
                "name": "Capers Surfine",
                "gtin": "5391508622616",
                "image": "https://static.euw1.sustained.app/products/images/5391508622616_0.jpg",
                "pack": "1.52kg",
                "grade": "C",
                "points": {
                    "pef": 16.518778815799617
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.49b578b7f22105af5fae04033a26814e",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.49b578b7f22105af5fae04033a26814e/impacts"
                }
            },
            {
                "id": "productid.49fee0c5131948c146eb36106c37f0e3",
                "name": "Grower's Selection British Chives",
                "gtin": "23037328",
                "image": "https://static.euw1.sustained.app/products/images/23037328_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 7.092734896173699
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.49fee0c5131948c146eb36106c37f0e3",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.49fee0c5131948c146eb36106c37f0e3/impacts"
                }
            },
            {
                "id": "productid.4b8a50c1b1b90828bd41357d6eb16f5d",
                "name": "Mint",
                "gtin": "0000003280805",
                "image": "https://static.euw1.sustained.app/products/images/0000003280805_0.jpg",
                "pack": "100g",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4b8a50c1b1b90828bd41357d6eb16f5d",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4b8a50c1b1b90828bd41357d6eb16f5d/impacts"
                }
            },
            {
                "id": "productid.57825328eadb4aae25a49f55601baedc",
                "name": "British Curled Parsley Small Pot",
                "gtin": "0000003283271",
                "image": "https://static.euw1.sustained.app/products/images/0000003283271_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.57825328eadb4aae25a49f55601baedc",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.57825328eadb4aae25a49f55601baedc/impacts"
                }
            },
            {
                "id": "productid.6bfd11fb0c26147d8eebb11d132071be",
                "name": "Cooks' Ingredients Zingy Mint",
                "gtin": "5000169001844",
                "image": "https://static.euw1.sustained.app/products/images/5000169001844_0.jpg",
                "pack": "100g",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.6bfd11fb0c26147d8eebb11d132071be",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.6bfd11fb0c26147d8eebb11d132071be/impacts"
                }
            },
            {
                "id": "productid.6c9bd60cb8137efce5ad74da82740c30",
                "name": "British Mint",
                "gtin": "10070482",
                "image": "https://static.euw1.sustained.app/products/images/10070482_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.6c9bd60cb8137efce5ad74da82740c30",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.6c9bd60cb8137efce5ad74da82740c30/impacts"
                }
            },
            {
                "id": "productid.6cc88a91a75c10b13b3b7ff0b64bacec",
                "name": "Fresh Herbs",
                "gtin": "03316375",
                "image": "https://static.euw1.sustained.app/products/images/03316375_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 16.858248802583155
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.6cc88a91a75c10b13b3b7ff0b64bacec",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.6cc88a91a75c10b13b3b7ff0b64bacec/impacts"
                }
            },
            {
                "id": "productid.715e5456c91d37e1c5ae391f2bed7d87",
                "name": "Aromatic & Zesty Coriander",
                "gtin": "5057172439775",
                "image": "https://static.euw1.sustained.app/products/images/5057172439775_0.jpg",
                "pack": "30g",
                "grade": "A",
                "points": {
                    "pef": 10.81169255215334
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.715e5456c91d37e1c5ae391f2bed7d87",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.715e5456c91d37e1c5ae391f2bed7d87/impacts"
                }
            },
            {
                "id": "productid.7280685192684cd4e63a97673baa41a7",
                "name": "Delicately Sweet Dill",
                "gtin": "5054070690468",
                "image": "https://static.euw1.sustained.app/products/images/5054070690468_0.jpg",
                "pack": "25g",
                "grade": "A",
                "points": {
                    "pef": 11.013312798574258
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.7280685192684cd4e63a97673baa41a7",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.7280685192684cd4e63a97673baa41a7/impacts"
                }
            },
            {
                "id": "productid.7e7e2949ca2d15772963bb66e0875521",
                "name": "Sage",
                "gtin": "0000000599184",
                "image": "https://static.euw1.sustained.app/products/images/0000000599184_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.013312798574262
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.7e7e2949ca2d15772963bb66e0875521",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.7e7e2949ca2d15772963bb66e0875521/impacts"
                }
            },
            {
                "id": "productid.842479983a35ab4dd2a8747b35baf95f",
                "name": "Grower's Selection Dill",
                "gtin": "5051413653227",
                "image": "https://static.euw1.sustained.app/products/images/5051413653227_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.013312798574258
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.842479983a35ab4dd2a8747b35baf95f",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.842479983a35ab4dd2a8747b35baf95f/impacts"
                }
            },
            {
                "id": "productid.af949e4445bc8065998fce36e42aadac",
                "name": "Fresh Herbs",
                "gtin": "03333846",
                "image": "https://static.euw1.sustained.app/products/images/03333846_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 16.858248802583155
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.af949e4445bc8065998fce36e42aadac",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.af949e4445bc8065998fce36e42aadac/impacts"
                }
            },
            {
                "id": "productid.c0a7cdca02aafb8c0cb417d8718641bf",
                "name": "Select Farms Mint",
                "gtin": "0000029092673",
                "image": "https://static.euw1.sustained.app/products/images/0000029092673_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.c0a7cdca02aafb8c0cb417d8718641bf",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.c0a7cdca02aafb8c0cb417d8718641bf/impacts"
                }
            },
            {
                "id": "productid.c97ce7ab3ff1385344f9052b0f57d4fb",
                "name": "Cool & Refreshing Growing Garden Mint",
                "gtin": "23037427",
                "image": "https://static.euw1.sustained.app/products/images/23037427_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.c97ce7ab3ff1385344f9052b0f57d4fb",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.c97ce7ab3ff1385344f9052b0f57d4fb/impacts"
                }
            },
            {
                "id": "productid.e8c1632c58145694081d0a5f384bd7d9",
                "name": "The Greengrocer's On Market Street Fresh Thyme",
                "gtin": "5010251961703",
                "image": "https://static.euw1.sustained.app/products/images/5010251961703_0.jpg",
                "pack": "20g",
                "grade": "A",
                "points": {
                    "pef": 11.013312798574258
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.e8c1632c58145694081d0a5f384bd7d9",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.e8c1632c58145694081d0a5f384bd7d9/impacts"
                }
            },
            {
                "id": "productid.ea36d6814547fc8c31c2116638f70499",
                "name": "The Greengrocer's On Market Street Fresh Chives",
                "gtin": "5010251961680",
                "image": "https://static.euw1.sustained.app/products/images/5010251961680_0.jpg",
                "pack": "20g",
                "grade": "A",
                "points": {
                    "pef": 7.092734896173699
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.ea36d6814547fc8c31c2116638f70499",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.ea36d6814547fc8c31c2116638f70499/impacts"
                }
            },
            {
                "id": "productid.eee38639d607180a225ceb99fa911757",
                "name": "Flat Leaf Parsley",
                "gtin": "03268551",
                "image": "https://static.euw1.sustained.app/products/images/03268551_0.jpg",
                "pack": "30g",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.eee38639d607180a225ceb99fa911757",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.eee38639d607180a225ceb99fa911757/impacts"
                }
            },
            {
                "id": "productid.f6bff52d22c77d5c58b6a959525ae6e9",
                "name": "Fresh Herbs",
                "gtin": "03306697",
                "image": "https://static.euw1.sustained.app/products/images/03306697_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 16.858248802583155
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.f6bff52d22c77d5c58b6a959525ae6e9",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.f6bff52d22c77d5c58b6a959525ae6e9/impacts"
                }
            },
            {
                "id": "productid.f9da19853b7fef5d8de20a641d59070a",
                "name": "Thyme Pot",
                "gtin": "10023907",
                "image": "https://static.euw1.sustained.app/products/images/10023907_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.013312798574258
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.f9da19853b7fef5d8de20a641d59070a",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.f9da19853b7fef5d8de20a641d59070a/impacts"
                }
            },
            {
                "id": "productid.ffce92f390ef109cdd14dcc96cc1565d",
                "name": "Grower's Selection Growing Coriander",
                "gtin": "23037335",
                "image": "https://static.euw1.sustained.app/products/images/23037335_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 10.81169255215334
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.ffce92f390ef109cdd14dcc96cc1565d",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.ffce92f390ef109cdd14dcc96cc1565d/impacts"
                }
            },
            {
                "id": "productid.026a9df24930374ee601900808d7e21c",
                "name": "British Thyme",
                "gtin": "10053263",
                "image": "https://static.euw1.sustained.app/products/images/10053263_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.013312798574258
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.026a9df24930374ee601900808d7e21c",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.026a9df24930374ee601900808d7e21c/impacts"
                }
            },
            {
                "id": "productid.03419e393d90a69c1d23e3308e4de117",
                "name": "Irish Basil Pot",
                "gtin": "10023679",
                "image": "https://static.euw1.sustained.app/products/images/10023679_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.013312798574258
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03419e393d90a69c1d23e3308e4de117",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03419e393d90a69c1d23e3308e4de117/impacts"
                }
            },
            {
                "id": "productid.132ebcdfa86bdda92859888fee99bfed",
                "name": "Basil",
                "gtin": "01291476",
                "image": "https://static.euw1.sustained.app/products/images/01291476_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.013312798574258
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.132ebcdfa86bdda92859888fee99bfed",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.132ebcdfa86bdda92859888fee99bfed/impacts"
                }
            },
            {
                "id": "productid.136828ec191e290a929d5262712baca6",
                "name": "Irish Fresh Herbs Coriander Aromatic & Citrusy Flavour",
                "gtin": "10079089",
                "image": "https://static.euw1.sustained.app/products/images/10079089_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 10.81169255215334
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.136828ec191e290a929d5262712baca6",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.136828ec191e290a929d5262712baca6/impacts"
                }
            },
            {
                "id": "productid.164c01d40b3adaee189a8c5aeb1af893",
                "name": "Market St Living Coriander",
                "gtin": "5010251734284",
                "image": "https://static.euw1.sustained.app/products/images/5010251734284_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 10.81169255215334
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.164c01d40b3adaee189a8c5aeb1af893",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.164c01d40b3adaee189a8c5aeb1af893/impacts"
                }
            },
            {
                "id": "productid.16a434eddeb05b2f886ea837475aaf9a",
                "name": "Delightfully Fragrant Growing Rosemary",
                "gtin": "5052449479430",
                "image": "https://static.euw1.sustained.app/products/images/5052449479430_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 16.783131490079054
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16a434eddeb05b2f886ea837475aaf9a",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16a434eddeb05b2f886ea837475aaf9a/impacts"
                }
            },
            {
                "id": "productid.2a08570cc95838fed754303448ba2541",
                "name": "Chives",
                "gtin": "01673517",
                "image": "https://static.euw1.sustained.app/products/images/01673517_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 7.092734896173699
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a08570cc95838fed754303448ba2541",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a08570cc95838fed754303448ba2541/impacts"
                }
            },
            {
                "id": "productid.30480e833e775d7dff73b5b15aac7a1a",
                "name": "Grower's Selection Growing Curly Parsley",
                "gtin": "23037311",
                "image": "https://static.euw1.sustained.app/products/images/23037311_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.30480e833e775d7dff73b5b15aac7a1a",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.30480e833e775d7dff73b5b15aac7a1a/impacts"
                }
            },
            {
                "id": "productid.33ecc31cf16487959778c0233916f63a",
                "name": "British Coriander",
                "gtin": "10053492",
                "image": "https://static.euw1.sustained.app/products/images/10053492_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 10.81169255215334
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.33ecc31cf16487959778c0233916f63a",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.33ecc31cf16487959778c0233916f63a/impacts"
                }
            },
            {
                "id": "productid.33ed4c072eb1c53e254714b1494c6809",
                "name": "British Chives",
                "gtin": "10053232",
                "image": "https://static.euw1.sustained.app/products/images/10053232_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 7.092734896173699
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.33ed4c072eb1c53e254714b1494c6809",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.33ed4c072eb1c53e254714b1494c6809/impacts"
                }
            },
            {
                "id": "productid.3d56932b5a6a93cd340526e905ed9226",
                "name": "Dill",
                "gtin": "00310994",
                "image": "https://static.euw1.sustained.app/products/images/00310994_0.jpg",
                "pack": "20g",
                "grade": "A",
                "points": {
                    "pef": 11.013312798574258
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3d56932b5a6a93cd340526e905ed9226",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3d56932b5a6a93cd340526e905ed9226/impacts"
                }
            },
            {
                "id": "productid.3fdaf05c27ebf5d3966fe25c35f9c275",
                "name": "Market St Living Mint",
                "gtin": "5010251734307",
                "image": "https://static.euw1.sustained.app/products/images/5010251734307_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3fdaf05c27ebf5d3966fe25c35f9c275",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3fdaf05c27ebf5d3966fe25c35f9c275/impacts"
                }
            },
            {
                "id": "productid.40a04fe178428b3d3a4dd6d93bf6c371",
                "name": "The Greengrocer's On Market Street Fresh Coriander",
                "gtin": "5010251745396",
                "image": "https://static.euw1.sustained.app/products/images/5010251745396_0.jpg",
                "pack": "100g",
                "grade": "A",
                "points": {
                    "pef": 10.81169255215334
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.40a04fe178428b3d3a4dd6d93bf6c371",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.40a04fe178428b3d3a4dd6d93bf6c371/impacts"
                }
            },
            {
                "id": "productid.40bcfb8f21d7233bfc5aba93de9edd28",
                "name": "Grower's Selection Large Flat Leaf Parsley",
                "gtin": "5051413834442",
                "image": "https://static.euw1.sustained.app/products/images/5051413834442_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.40bcfb8f21d7233bfc5aba93de9edd28",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.40bcfb8f21d7233bfc5aba93de9edd28/impacts"
                }
            },
            {
                "id": "productid.430233b3215a294eef7ba85e10224ff5",
                "name": "Cooks' Ingredients Mash Avocado with Guacamole Recipe Herb Kit",
                "gtin": "5000169518663",
                "image": "https://static.euw1.sustained.app/products/images/5000169518663_0.jpg",
                "pack": "75g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.430233b3215a294eef7ba85e10224ff5",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.430233b3215a294eef7ba85e10224ff5/impacts"
                }
            },
            {
                "id": "productid.4483c3e64aae495848636a70bfd6eb6b",
                "name": "Dill Bunches",
                "gtin": "0000003235393",
                "image": "https://static.euw1.sustained.app/products/images/0000003235393_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.013312798574258
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4483c3e64aae495848636a70bfd6eb6b",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4483c3e64aae495848636a70bfd6eb6b/impacts"
                }
            },
            {
                "id": "productid.45d1ee3bcc6313f468992fcb35bc2b79",
                "name": "Wonderfully Aromatic Growing Basil",
                "gtin": "23037342",
                "image": "https://static.euw1.sustained.app/products/images/23037342_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.013312798574258
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.45d1ee3bcc6313f468992fcb35bc2b79",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.45d1ee3bcc6313f468992fcb35bc2b79/impacts"
                }
            },
            {
                "id": "productid.4f50b119426cd89bd341e9c6229106c9",
                "name": "Fresh Herbs Curled Parsley Fresh and Vibrant Flavour",
                "gtin": "10079140",
                "image": "https://static.euw1.sustained.app/products/images/10079140_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 11.01331279857426
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Herbs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4f50b119426cd89bd341e9c6229106c9",
                    "category": "https://api.sustained.com/choice/v1/categories/herbs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4f50b119426cd89bd341e9c6229106c9/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.000029ebf224b190fde987b99d40f824",
                "name": "Soft White Ready to Roll Icing",
                "gtin": "00430067",
                "image": "https://static.euw1.sustained.app/products/images/00430067_0.jpg",
                "pack": "1kg",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.000029ebf224b190fde987b99d40f824",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.000029ebf224b190fde987b99d40f824/impacts"
                }
            },
            {
                "id": "productid.001a6b62354c72e3a69ab152ec94491a",
                "name": "Golden Syrup Maple Flavour",
                "gtin": "5010115825899",
                "image": "https://static.euw1.sustained.app/products/images/5010115825899_0.jpg",
                "pack": "2.80kg",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.001a6b62354c72e3a69ab152ec94491a",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.001a6b62354c72e3a69ab152ec94491a/impacts"
                }
            },
            {
                "id": "productid.005b59cba84bed177ac77d130b5b109f",
                "name": "Raspberry Flavour Jelly",
                "gtin": "5012991001459",
                "image": "https://static.euw1.sustained.app/products/images/5012991001459_0.jpg",
                "pack": "3.5kg",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.005b59cba84bed177ac77d130b5b109f",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.005b59cba84bed177ac77d130b5b109f/impacts"
                }
            },
            {
                "id": "productid.007a724ab37ba569b74e7826f3aa1042",
                "name": "Wellcare Gluten Free Sponge and Cupcake Mix",
                "gtin": "5000254502706",
                "image": "https://static.euw1.sustained.app/products/images/5000254502706_0.jpg",
                "pack": "1kg",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.007a724ab37ba569b74e7826f3aa1042",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.007a724ab37ba569b74e7826f3aa1042/impacts"
                }
            },
            {
                "id": "productid.009536834df3043353588cf960cdcf3a",
                "name": "Hartleys Jelly Orange Flavour Pots",
                "gtin": "5060391622415",
                "image": "https://static.euw1.sustained.app/products/images/5060391622415_0.jpg",
                "pack": "125g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.009536834df3043353588cf960cdcf3a",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.009536834df3043353588cf960cdcf3a/impacts"
                }
            },
            {
                "id": "productid.009aa8f0bc260aa4e34ec576dd16ac62",
                "name": "Organic Coconut Syrup",
                "gtin": "7791400067146",
                "image": "https://static.euw1.sustained.app/products/images/7791400067146_0.jpg",
                "pack": "250g",
                "grade": "D",
                "points": {
                    "pef": 23.77707938241796
                },
                "info_icons": [],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.009aa8f0bc260aa4e34ec576dd16ac62",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.009aa8f0bc260aa4e34ec576dd16ac62/impacts"
                }
            },
            {
                "id": "productid.011236226eea98d8a1c9ef9de237691d",
                "name": "Guinness Ale & Steak Pie Mix",
                "gtin": "5032457717175",
                "image": "https://static.euw1.sustained.app/products/images/5032457717175_0.jpg",
                "pack": "40g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.011236226eea98d8a1c9ef9de237691d",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.011236226eea98d8a1c9ef9de237691d/impacts"
                }
            },
            {
                "id": "productid.01c4bf3cce7c0a1e2242912b0ea95cc0",
                "name": "Premium Slush Syrup Strawberry",
                "gtin": "5060343701847",
                "image": "https://static.euw1.sustained.app/products/images/5060343701847_0.jpg",
                "pack": "5l",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01c4bf3cce7c0a1e2242912b0ea95cc0",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01c4bf3cce7c0a1e2242912b0ea95cc0/impacts"
                }
            },
            {
                "id": "productid.01d45e1b0667d7c8cc44abd07708c351",
                "name": "Vegeta Natur",
                "gtin": "3850104257933",
                "image": "https://static.euw1.sustained.app/products/images/3850104257933_0.jpg",
                "pack": "75g",
                "grade": "A",
                "points": {
                    "pef": 9.974333066355086
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01d45e1b0667d7c8cc44abd07708c351",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01d45e1b0667d7c8cc44abd07708c351/impacts"
                }
            },
            {
                "id": "productid.02cdeffaf2f5dd9bad2f37920d80e930",
                "name": "Cooks' Ingredients Rainbow Stars and Confetti",
                "gtin": "5000169600481",
                "image": "https://static.euw1.sustained.app/products/images/5000169600481_0.jpg",
                "pack": "65g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02cdeffaf2f5dd9bad2f37920d80e930",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02cdeffaf2f5dd9bad2f37920d80e930/impacts"
                }
            },
            {
                "id": "productid.02ebef83a707bcab2b3a6828f1376a5d",
                "name": "Elderflower Jelly",
                "gtin": "5060360380445",
                "image": "https://static.euw1.sustained.app/products/images/5060360380445_0.jpg",
                "pack": "150g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02ebef83a707bcab2b3a6828f1376a5d",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02ebef83a707bcab2b3a6828f1376a5d/impacts"
                }
            },
            {
                "id": "productid.0305d7a906b32854b50f711ee953d400",
                "name": "Royal Icing",
                "gtin": "5010251826606",
                "image": "https://static.euw1.sustained.app/products/images/5010251826606_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0305d7a906b32854b50f711ee953d400",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0305d7a906b32854b50f711ee953d400/impacts"
                }
            },
            {
                "id": "productid.03506ca06193a08f61974406cf04f96e",
                "name": "Ready Rolled 2 Puff Pastry Sheets",
                "gtin": "5010084103882",
                "image": "https://static.euw1.sustained.app/products/images/5010084103882_0.jpg",
                "pack": "320g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03506ca06193a08f61974406cf04f96e",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03506ca06193a08f61974406cf04f96e/impacts"
                }
            },
            {
                "id": "productid.0382ef7df7671e2d40dbb7316b7043d7",
                "name": "Almond Syrup",
                "gtin": "05010102222717",
                "image": "https://static.euw1.sustained.app/products/images/05010102222717_0.jpg",
                "pack": "700ml",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0382ef7df7671e2d40dbb7316b7043d7",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0382ef7df7671e2d40dbb7316b7043d7/impacts"
                }
            },
            {
                "id": "productid.03ca470abfd1f9a144c2cee25f3513fb",
                "name": "Chocolatey Strands",
                "gtin": "50254484",
                "image": "https://static.euw1.sustained.app/products/images/50254484_0.jpg",
                "pack": "55g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03ca470abfd1f9a144c2cee25f3513fb",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03ca470abfd1f9a144c2cee25f3513fb/impacts"
                }
            },
            {
                "id": "productid.046c68d27cc628f424f78fa214f36e70",
                "name": "Plain Cookie Mix",
                "gtin": "5012991008458",
                "image": "https://static.euw1.sustained.app/products/images/5012991008458_0.jpg",
                "pack": "3.5kg",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.046c68d27cc628f424f78fa214f36e70",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.046c68d27cc628f424f78fa214f36e70/impacts"
                }
            },
            {
                "id": "productid.054e9452f7147f2fdba9d9f606239aa2",
                "name": "Bakery Granary Bread Flour",
                "gtin": "5010024118082",
                "image": "https://static.euw1.sustained.app/products/images/5010024118082_0.jpg",
                "pack": "1kg",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.054e9452f7147f2fdba9d9f606239aa2",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.054e9452f7147f2fdba9d9f606239aa2/impacts"
                }
            },
            {
                "id": "productid.0592f53aab209eaffcd705f9248e8fd5",
                "name": "LoveLife Yorkshire Pudding & Pancake Mix",
                "gtin": "5000169353561",
                "image": "https://static.euw1.sustained.app/products/images/5000169353561_0.jpg",
                "pack": "130g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0592f53aab209eaffcd705f9248e8fd5",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0592f53aab209eaffcd705f9248e8fd5/impacts"
                }
            },
            {
                "id": "productid.05ae80259d2de9f3ee6e74f484a9b230",
                "name": "Baking Powder",
                "gtin": "5057172722990",
                "image": "https://static.euw1.sustained.app/products/images/5057172722990_0.jpg",
                "pack": "170g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05ae80259d2de9f3ee6e74f484a9b230",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05ae80259d2de9f3ee6e74f484a9b230/impacts"
                }
            },
            {
                "id": "productid.07e829604aa74d6a83c47d199f0d8ec8",
                "name": "Multi-Seed Malted Bread Mix",
                "gtin": "5000169613573",
                "image": "https://static.euw1.sustained.app/products/images/5000169613573_0.jpg",
                "pack": "500g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07e829604aa74d6a83c47d199f0d8ec8",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07e829604aa74d6a83c47d199f0d8ec8/impacts"
                }
            },
            {
                "id": "productid.085ba23557a8c8a4fe395ff2c187f2a5",
                "name": "Cooks' Ingredients Bronze Crunch",
                "gtin": "5000169218907",
                "image": "https://static.euw1.sustained.app/products/images/5000169218907_0.jpg",
                "pack": "87g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.085ba23557a8c8a4fe395ff2c187f2a5",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.085ba23557a8c8a4fe395ff2c187f2a5/impacts"
                }
            },
            {
                "id": "productid.090a46ecf31fa20b49b79c0462294aef",
                "name": "Liquid Stevia Drops Sweetener Vanilla Flavour",
                "gtin": "5060457840364",
                "image": "https://static.euw1.sustained.app/products/images/5060457840364_0.jpg",
                "pack": "50ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.090a46ecf31fa20b49b79c0462294aef",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.090a46ecf31fa20b49b79c0462294aef/impacts"
                }
            },
            {
                "id": "productid.0988ab6f11037f2c149d305cc41f0754",
                "name": "Hearty & Homely Dumpling Mix",
                "gtin": "5050665011847",
                "image": "https://static.euw1.sustained.app/products/images/5050665011847_0.jpg",
                "pack": "140g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0988ab6f11037f2c149d305cc41f0754",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0988ab6f11037f2c149d305cc41f0754/impacts"
                }
            },
            {
                "id": "productid.0a61ae7ef55d0262cd7a15a29582aae0",
                "name": "Fast Action Dried Yeast",
                "gtin": "5051790493904",
                "image": "https://static.euw1.sustained.app/products/images/5051790493904_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.57222732805323
                },
                "info_icons": [],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a61ae7ef55d0262cd7a15a29582aae0",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a61ae7ef55d0262cd7a15a29582aae0/impacts"
                }
            },
            {
                "id": "productid.0a85e3c347532ed7e6b58c94aa07216c",
                "name": "Smash Original Instant Mash Potato",
                "gtin": "5000354163562",
                "image": "https://static.euw1.sustained.app/products/images/5000354163562_0.jpg",
                "pack": "2kg",
                "grade": "D",
                "points": {
                    "pef": 26.232119444122375
                },
                "info_icons": [],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a85e3c347532ed7e6b58c94aa07216c",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a85e3c347532ed7e6b58c94aa07216c/impacts"
                }
            },
            {
                "id": "productid.0be86b4230a73de4e151eee2ae7038dc",
                "name": "Altar Bread Flour",
                "gtin": "5011040001594",
                "image": "https://static.euw1.sustained.app/products/images/5011040001594_0.jpg",
                "pack": "25kg",
                "grade": "A",
                "points": {
                    "pef": 9.091181326934837
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0be86b4230a73de4e151eee2ae7038dc",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0be86b4230a73de4e151eee2ae7038dc/impacts"
                }
            },
            {
                "id": "productid.0cbbd09a6606918ea33eb7f6cbaf5c89",
                "name": "Saucy Toffee Pudding Kit",
                "gtin": "5010525159409",
                "image": "https://static.euw1.sustained.app/products/images/5010525159409_0.jpg",
                "pack": "260g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0cbbd09a6606918ea33eb7f6cbaf5c89",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0cbbd09a6606918ea33eb7f6cbaf5c89/impacts"
                }
            },
            {
                "id": "productid.0ce4a68fb0feb713b46d1076c85ff37e",
                "name": "Rich Cream Cheese Style Icing",
                "gtin": "5010084902706",
                "image": "https://static.euw1.sustained.app/products/images/5010084902706_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ce4a68fb0feb713b46d1076c85ff37e",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ce4a68fb0feb713b46d1076c85ff37e/impacts"
                }
            },
            {
                "id": "productid.0d0d1228cf1960504324a1b595a56e1a",
                "name": "Scrumptious Cider Apple Jelly",
                "gtin": "5032714013521",
                "image": "https://static.euw1.sustained.app/products/images/5032714013521_0.jpg",
                "pack": "200g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d0d1228cf1960504324a1b595a56e1a",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d0d1228cf1960504324a1b595a56e1a/impacts"
                }
            },
            {
                "id": "productid.0d38837c575742f6b8acb79928c0ca15",
                "name": "Baking Garlic & Rosemary Focaccia Bread Mix",
                "gtin": "5020387001389",
                "image": "https://static.euw1.sustained.app/products/images/5020387001389_0.jpg",
                "pack": "500g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d38837c575742f6b8acb79928c0ca15",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d38837c575742f6b8acb79928c0ca15/impacts"
                }
            },
            {
                "id": "productid.0d4016772a822bb579209718bb7ebc04",
                "name": "Taste the Difference Wholegrain Seeded Bread Flour",
                "gtin": "01728637",
                "image": "https://static.euw1.sustained.app/products/images/01728637_0.jpg",
                "pack": "1kg",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d4016772a822bb579209718bb7ebc04",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d4016772a822bb579209718bb7ebc04/impacts"
                }
            },
            {
                "id": "productid.0d8ac5a0ce2d27a5a1180752d3f4a47d",
                "name": "Extra Special 6 All-Butter Pastry Cases",
                "gtin": "5057172090495",
                "image": "https://static.euw1.sustained.app/products/images/5057172090495_0.jpg",
                "pack": "210g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d8ac5a0ce2d27a5a1180752d3f4a47d",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d8ac5a0ce2d27a5a1180752d3f4a47d/impacts"
                }
            },
            {
                "id": "productid.0dda598dc73eefd4d8aa4f99688d4bba",
                "name": "Caputo Pizzeria",
                "gtin": "8014601150202",
                "image": "https://static.euw1.sustained.app/products/images/8014601150202_0.jpg",
                "pack": "15kg",
                "grade": "A",
                "points": {
                    "pef": 9.091181326934837
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0dda598dc73eefd4d8aa4f99688d4bba",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0dda598dc73eefd4d8aa4f99688d4bba/impacts"
                }
            },
            {
                "id": "productid.0f62aa5b992bd4d2ff6d92fae48a2e3b",
                "name": "Forest Fruits Flavoured Kissel with Pieces of Fruits",
                "gtin": "5900437037240",
                "image": "https://static.euw1.sustained.app/products/images/5900437037240_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f62aa5b992bd4d2ff6d92fae48a2e3b",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f62aa5b992bd4d2ff6d92fae48a2e3b/impacts"
                }
            },
            {
                "id": "productid.11687ab0140dffa4ddf40ff2f8b5644e",
                "name": "Yorkshire Pudding Mix",
                "gtin": "5050665028098",
                "image": "https://static.euw1.sustained.app/products/images/5050665028098_0.jpg",
                "pack": "120g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11687ab0140dffa4ddf40ff2f8b5644e",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11687ab0140dffa4ddf40ff2f8b5644e/impacts"
                }
            },
            {
                "id": "productid.12a35770ae828eaeb34ed6b542f150f3",
                "name": "Gluten & Dairy Free Middle Eastern Style Flatbread Mix",
                "gtin": "5015285000060",
                "image": "https://static.euw1.sustained.app/products/images/5015285000060_0.jpg",
                "pack": "250g",
                "grade": "D",
                "points": {
                    "pef": 23.70102396160815
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12a35770ae828eaeb34ed6b542f150f3",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12a35770ae828eaeb34ed6b542f150f3/impacts"
                }
            },
            {
                "id": "productid.12bd953b8f78d3ba4f7ae04cc7cc8011",
                "name": "Peach in Mango Flavoured Jelly",
                "gtin": "5052910476081",
                "image": "https://static.euw1.sustained.app/products/images/5052910476081_0.jpg",
                "pack": "120g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12bd953b8f78d3ba4f7ae04cc7cc8011",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12bd953b8f78d3ba4f7ae04cc7cc8011/impacts"
                }
            },
            {
                "id": "productid.12c096f0fa8f6b9586bd5980a8c2bc23",
                "name": "Sainsbury's Sweetener",
                "gtin": "0000000318938",
                "image": "https://static.euw1.sustained.app/products/images/0000000318938_0.jpg",
                "pack": "75g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12c096f0fa8f6b9586bd5980a8c2bc23",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12c096f0fa8f6b9586bd5980a8c2bc23/impacts"
                }
            },
            {
                "id": "productid.12d48a9b068d405f4e1f3bc24dadbd4f",
                "name": "Chia Seeds",
                "gtin": "5059512742170",
                "image": "https://static.euw1.sustained.app/products/images/5059512742170_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 42.69907625616115
                },
                "info_icons": [],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12d48a9b068d405f4e1f3bc24dadbd4f",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12d48a9b068d405f4e1f3bc24dadbd4f/impacts"
                }
            },
            {
                "id": "productid.14d242bff2d71496cc40cbf1f2316b87",
                "name": "Premium Potato Starch",
                "gtin": "5010928367265",
                "image": "https://static.euw1.sustained.app/products/images/5010928367265_0.jpg",
                "pack": "25kg",
                "grade": "E",
                "points": {
                    "pef": 42.184491323662485
                },
                "info_icons": [],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14d242bff2d71496cc40cbf1f2316b87",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14d242bff2d71496cc40cbf1f2316b87/impacts"
                }
            },
            {
                "id": "productid.1587fc5a368c2baf814c27f74aabb8a6",
                "name": "Seeded Wholemeal Bread Flour",
                "gtin": "5000169614402",
                "image": "https://static.euw1.sustained.app/products/images/5000169614402_0.jpg",
                "pack": "1.5kg",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1587fc5a368c2baf814c27f74aabb8a6",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1587fc5a368c2baf814c27f74aabb8a6/impacts"
                }
            },
            {
                "id": "productid.16a6a3e1a80e19f2f7ae69aec7cf0c39",
                "name": "Fairtrade Cane Sugar White Cubes",
                "gtin": "5010115820863",
                "image": "https://static.euw1.sustained.app/products/images/5010115820863_0.jpg",
                "pack": "500g",
                "grade": "A",
                "points": {
                    "pef": 4.8367850908573295
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16a6a3e1a80e19f2f7ae69aec7cf0c39",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16a6a3e1a80e19f2f7ae69aec7cf0c39/impacts"
                }
            },
            {
                "id": "productid.16dd57051dadc9bb1ce5192dca085786",
                "name": "ProGel Concentrated Food Colour Baby Blue",
                "gtin": "5060197937850",
                "image": "https://static.euw1.sustained.app/products/images/5060197937850_0.jpg",
                "pack": "25g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16dd57051dadc9bb1ce5192dca085786",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16dd57051dadc9bb1ce5192dca085786/impacts"
                }
            },
            {
                "id": "productid.16ef57d0d922b0eb16c6d52d66e226ab",
                "name": "Ready-to-Roll Icing Orange",
                "gtin": "5015462209798",
                "image": "https://static.euw1.sustained.app/products/images/5015462209798_0.jpg",
                "pack": "250g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16ef57d0d922b0eb16c6d52d66e226ab",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16ef57d0d922b0eb16c6d52d66e226ab/impacts"
                }
            },
            {
                "id": "productid.16f87c06a8cbb820d3f3a4fbf8280802",
                "name": "Multiseed Bread Mix",
                "gtin": "5060201625087",
                "image": "https://static.euw1.sustained.app/products/images/5060201625087_0.jpg",
                "pack": "500g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16f87c06a8cbb820d3f3a4fbf8280802",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16f87c06a8cbb820d3f3a4fbf8280802/impacts"
                }
            },
            {
                "id": "productid.17d16ac1b7f4b2efe750081857ecbd38",
                "name": "Lemon Flavour Jelly",
                "gtin": "5000354160905",
                "image": "https://static.euw1.sustained.app/products/images/5000354160905_0.jpg",
                "pack": "135g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.17d16ac1b7f4b2efe750081857ecbd38",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.17d16ac1b7f4b2efe750081857ecbd38/impacts"
                }
            },
            {
                "id": "productid.18bef32f184ffadea66f95cf857707b1",
                "name": "10 Cal Rhubarb & Custard Flavour Jelly",
                "gtin": "5060391626529",
                "image": "https://static.euw1.sustained.app/products/images/5060391626529_0.jpg",
                "pack": "175g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.18bef32f184ffadea66f95cf857707b1",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.18bef32f184ffadea66f95cf857707b1/impacts"
                }
            },
            {
                "id": "productid.18d2e4f6c17b4d8e418c972be8c0432c",
                "name": "Food British Granulated Sugar",
                "gtin": "0000000963602",
                "image": "https://static.euw1.sustained.app/products/images/0000000963602_0.jpg",
                "pack": "1kg",
                "grade": "B",
                "points": {
                    "pef": 14.551829669183038
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.18d2e4f6c17b4d8e418c972be8c0432c",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.18d2e4f6c17b4d8e418c972be8c0432c/impacts"
                }
            },
            {
                "id": "productid.1a496e75c83e8effb796a162900d4bea",
                "name": "Golden Granulated Sugar",
                "gtin": "5391521480385",
                "image": "https://static.euw1.sustained.app/products/images/5391521480385_0.jpg",
                "pack": "1kg",
                "grade": "B",
                "points": {
                    "pef": 14.551829669183038
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1a496e75c83e8effb796a162900d4bea",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1a496e75c83e8effb796a162900d4bea/impacts"
                }
            },
            {
                "id": "productid.1b2182f8fc7b0d0bd255d1e64bb1c9fa",
                "name": "Marshalls Semolina",
                "gtin": "5010223229749",
                "image": "https://static.euw1.sustained.app/products/images/5010223229749_0.jpg",
                "pack": "500g",
                "grade": "D",
                "points": {
                    "pef": 27.0572447906217
                },
                "info_icons": [],
                "category": "Home Baking & Sugar",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b2182f8fc7b0d0bd255d1e64bb1c9fa",
                    "category": "https://api.sustained.com/choice/v1/categories/home-baking-and-sugar",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b2182f8fc7b0d0bd255d1e64bb1c9fa/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.018cd2a67167b5e3817b7d1cfd2aa40b",
                "name": "Golden 100 Tea Bags",
                "gtin": "0078876501358",
                "image": "https://static.euw1.sustained.app/products/images/0078876501358_0.jpg",
                "pack": "290g",
                "grade": "G",
                "points": {
                    "pef": 94.05161293443493
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.018cd2a67167b5e3817b7d1cfd2aa40b",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.018cd2a67167b5e3817b7d1cfd2aa40b/impacts"
                }
            },
            {
                "id": "productid.022be7641db1f1525314f41344922c8a",
                "name": "Classic Full Roast Coffee",
                "gtin": "5010251501183",
                "image": "https://static.euw1.sustained.app/products/images/5010251501183_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.022be7641db1f1525314f41344922c8a",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.022be7641db1f1525314f41344922c8a/impacts"
                }
            },
            {
                "id": "productid.024f9f78e1e13e82e2e6f8cc0ab32751",
                "name": "Fairtrade Melrose Bean Nespresso Compatible Ground Coffee Capsules",
                "gtin": "5016311000399",
                "image": "https://static.euw1.sustained.app/products/images/5016311000399_0.jpg",
                "pack": "54g",
                "grade": "G",
                "points": {
                    "pef": 178.99052475008824
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.024f9f78e1e13e82e2e6f8cc0ab32751",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.024f9f78e1e13e82e2e6f8cc0ab32751/impacts"
                }
            },
            {
                "id": "productid.02563ebbe48b371f813e8d9ea5d5625c",
                "name": "BMX Ground Coffee",
                "gtin": "5392000150973",
                "image": "https://static.euw1.sustained.app/products/images/5392000150973_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02563ebbe48b371f813e8d9ea5d5625c",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02563ebbe48b371f813e8d9ea5d5625c/impacts"
                }
            },
            {
                "id": "productid.03681fc67914f0af4d495fcc89bfa1a1",
                "name": "Beans Blend No.9 Roast Coffee",
                "gtin": "5052274351086",
                "image": "https://static.euw1.sustained.app/products/images/5052274351086_0.jpg",
                "pack": "227g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03681fc67914f0af4d495fcc89bfa1a1",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03681fc67914f0af4d495fcc89bfa1a1/impacts"
                }
            },
            {
                "id": "productid.049e2ce3f76206eefca8dc136e30ba70",
                "name": "Taste the Difference Fairtrade 200 Earl Grey Fine Tea Bags",
                "gtin": "00202916",
                "image": "https://static.euw1.sustained.app/products/images/00202916_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 94.05161293443493
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.049e2ce3f76206eefca8dc136e30ba70",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.049e2ce3f76206eefca8dc136e30ba70/impacts"
                }
            },
            {
                "id": "productid.04da5a8c5835c2080a56b0a9d7b25e1e",
                "name": "Brazil: Santa Lucia Ground Coffee",
                "gtin": "5060433880148",
                "image": "https://static.euw1.sustained.app/products/images/5060433880148_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04da5a8c5835c2080a56b0a9d7b25e1e",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04da5a8c5835c2080a56b0a9d7b25e1e/impacts"
                }
            },
            {
                "id": "productid.06c1d3cdd22431fd92c90411fb11a4c5",
                "name": "Food Fairtrade 50 Pure Origin Kenyan Teabags",
                "gtin": "0000000730884",
                "image": "https://static.euw1.sustained.app/products/images/0000000730884_0.jpg",
                "pack": "125g",
                "grade": "G",
                "points": {
                    "pef": 94.05161293443493
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06c1d3cdd22431fd92c90411fb11a4c5",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06c1d3cdd22431fd92c90411fb11a4c5/impacts"
                }
            },
            {
                "id": "productid.06eb562b81244984e5e1f7bac85332f8",
                "name": "Tea Bags Original Biodegradable 240",
                "gtin": "8712100712494",
                "image": "https://static.euw1.sustained.app/products/images/8712100712494_0.jpg",
                "pack": "696g",
                "grade": "G",
                "points": {
                    "pef": 94.05161293443493
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06eb562b81244984e5e1f7bac85332f8",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06eb562b81244984e5e1f7bac85332f8/impacts"
                }
            },
            {
                "id": "productid.079cc93a157e66613bf7ae188b393564",
                "name": "Green Jasmine 20s Pyramid Teabags",
                "gtin": "8718114888651",
                "image": "https://static.euw1.sustained.app/products/images/8718114888651_0.jpg",
                "pack": "34g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.079cc93a157e66613bf7ae188b393564",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.079cc93a157e66613bf7ae188b393564/impacts"
                }
            },
            {
                "id": "productid.081aefb7bfd195738f51b512d5920642",
                "name": "Coffee Co Las Vegas All In Ground Coffee",
                "gtin": "5060313402019",
                "image": "https://static.euw1.sustained.app/products/images/5060313402019_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.081aefb7bfd195738f51b512d5920642",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.081aefb7bfd195738f51b512d5920642/impacts"
                }
            },
            {
                "id": "productid.087cd9d71c26fe03b6e9499337105c8d",
                "name": "Highlights Bournville Hot Chocolate",
                "gtin": "7622210833006",
                "image": "https://static.euw1.sustained.app/products/images/7622210833006_0.jpg",
                "pack": "220g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.087cd9d71c26fe03b6e9499337105c8d",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.087cd9d71c26fe03b6e9499337105c8d/impacts"
                }
            },
            {
                "id": "productid.08acf2badd148826b055bee83b7c3a39",
                "name": "Instant Hot Chocolate",
                "gtin": "5023471105512",
                "image": "https://static.euw1.sustained.app/products/images/5023471105512_0.jpg",
                "pack": "1kg",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08acf2badd148826b055bee83b7c3a39",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08acf2badd148826b055bee83b7c3a39/impacts"
                }
            },
            {
                "id": "productid.08b93dd2a311c94e5b62e3c0c4f0f71e",
                "name": "Taste the Difference Oolong Tea Loose Leaf",
                "gtin": "0000000305150",
                "image": "https://static.euw1.sustained.app/products/images/0000000305150_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 94.05161293443493
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08b93dd2a311c94e5b62e3c0c4f0f71e",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08b93dd2a311c94e5b62e3c0c4f0f71e/impacts"
                }
            },
            {
                "id": "productid.08c4780021d2993fbab7cbf85755f0b3",
                "name": "Lemon & Ginger 80 Single Tea Bags",
                "gtin": "0070177270322",
                "image": "https://static.euw1.sustained.app/products/images/0070177270322_0.jpg",
                "pack": "120g",
                "grade": "A",
                "points": {
                    "pef": 10.247267507707502
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08c4780021d2993fbab7cbf85755f0b3",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08c4780021d2993fbab7cbf85755f0b3/impacts"
                }
            },
            {
                "id": "productid.0908e8d9aba48db14335052826dda756",
                "name": "Source Organic Rwanda Gishwati Cloud Forest Coffee",
                "gtin": "5060433880186",
                "image": "https://static.euw1.sustained.app/products/images/5060433880186_0.jpg",
                "pack": "227g",
                "grade": "G",
                "points": {
                    "pef": 178.99052475008824
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0908e8d9aba48db14335052826dda756",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0908e8d9aba48db14335052826dda756/impacts"
                }
            },
            {
                "id": "productid.096f756a52b72d8e8ac38e3c6c7e047b",
                "name": "Organic Japanese Matcha Turmeric Tea",
                "gtin": "5021554001522",
                "image": "https://static.euw1.sustained.app/products/images/5021554001522_0.jpg",
                "pack": "1.8g",
                "grade": "F",
                "points": {
                    "pef": 66.97346988739992
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.096f756a52b72d8e8ac38e3c6c7e047b",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.096f756a52b72d8e8ac38e3c6c7e047b/impacts"
                }
            },
            {
                "id": "productid.0a18e508bae0d2d224f8a8689d880083",
                "name": "Espresso Roasted Coffee Beans",
                "gtin": "5010251732761",
                "image": "https://static.euw1.sustained.app/products/images/5010251732761_0.jpg",
                "pack": "227g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a18e508bae0d2d224f8a8689d880083",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a18e508bae0d2d224f8a8689d880083/impacts"
                }
            },
            {
                "id": "productid.0a573db69923b6d9da1cac0f28ffd60f",
                "name": "Gold Blend Instant Coffee",
                "gtin": "7613036077514",
                "image": "https://static.euw1.sustained.app/products/images/7613036077514_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a573db69923b6d9da1cac0f28ffd60f",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a573db69923b6d9da1cac0f28ffd60f/impacts"
                }
            },
            {
                "id": "productid.0ac1af7c30cedcc84a3d8a7036082e12",
                "name": "Bold Morning Shot Ground Coffee",
                "gtin": "0745125437064",
                "image": "https://static.euw1.sustained.app/products/images/0745125437064_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ac1af7c30cedcc84a3d8a7036082e12",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ac1af7c30cedcc84a3d8a7036082e12/impacts"
                }
            },
            {
                "id": "productid.0aecb47ecc9c96795d1f0ba505606357",
                "name": "Caramel Latte Premium Instant Coffee",
                "gtin": "7613038559087",
                "image": "https://static.euw1.sustained.app/products/images/7613038559087_0.jpg",
                "pack": "21.5g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0aecb47ecc9c96795d1f0ba505606357",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0aecb47ecc9c96795d1f0ba505606357/impacts"
                }
            },
            {
                "id": "productid.0b3dc28532e58d1f35a5106f912bc866",
                "name": "Organic Fairtrade Decaf Wholebean Coffee",
                "gtin": "5060399772105",
                "image": "https://static.euw1.sustained.app/products/images/5060399772105_0.jpg",
                "pack": "227g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b3dc28532e58d1f35a5106f912bc866",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b3dc28532e58d1f35a5106f912bc866/impacts"
                }
            },
            {
                "id": "productid.0b8c3aae30247a0b8f194c1d25f2167e",
                "name": "Gold Flat White Instant Coffee",
                "gtin": "7613287184238",
                "image": "https://static.euw1.sustained.app/products/images/7613287184238_0.jpg",
                "pack": "12.5g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b8c3aae30247a0b8f194c1d25f2167e",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b8c3aae30247a0b8f194c1d25f2167e/impacts"
                }
            },
            {
                "id": "productid.0b9f4ae559401c5a5144bc91714b6a34",
                "name": "iTierra! For Amazonia Organic Coffee Beans",
                "gtin": "8000070049628",
                "image": "https://static.euw1.sustained.app/products/images/8000070049628_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b9f4ae559401c5a5144bc91714b6a34",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b9f4ae559401c5a5144bc91714b6a34/impacts"
                }
            },
            {
                "id": "productid.0be477726dda7b9a74851c0eb4544094",
                "name": "Just Essentials 40 Tea Bags",
                "gtin": "5057172733286",
                "image": "https://static.euw1.sustained.app/products/images/5057172733286_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0be477726dda7b9a74851c0eb4544094",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0be477726dda7b9a74851c0eb4544094/impacts"
                }
            },
            {
                "id": "productid.0c2545e1e1586cc30d01673aa060f9a0",
                "name": "Chocolate Caramel Nespresso Compatible Capsules",
                "gtin": "5034718051011",
                "image": "https://static.euw1.sustained.app/products/images/5034718051011_0.jpg",
                "pack": "52g",
                "grade": "G",
                "points": {
                    "pef": 178.99052475008824
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c2545e1e1586cc30d01673aa060f9a0",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c2545e1e1586cc30d01673aa060f9a0/impacts"
                }
            },
            {
                "id": "productid.0c8b92ed877115ffae08a66def1fed4c",
                "name": "Caff\u00e8 Latte by Nescafe Dolce Gusto Coffee Pods (Pack of 6, Total 72 Capsules, 36 Servings)",
                "gtin": "07613287139887",
                "image": "https://static.euw1.sustained.app/products/images/07613287139887_0.jpg",
                "pack": "10.1g",
                "grade": "G",
                "points": {
                    "pef": 144.5671338647068
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c8b92ed877115ffae08a66def1fed4c",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c8b92ed877115ffae08a66def1fed4c/impacts"
                }
            },
            {
                "id": "productid.0e3d5258d338c620b7fa1d5f973dfe3b",
                "name": "Fairtrade Original Filter",
                "gtin": "5054073045814",
                "image": "https://static.euw1.sustained.app/products/images/5054073045814_0.jpg",
                "pack": "65g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e3d5258d338c620b7fa1d5f973dfe3b",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e3d5258d338c620b7fa1d5f973dfe3b/impacts"
                }
            },
            {
                "id": "productid.0e5594ed1721627cf2e1b1950ca7d56d",
                "name": "Fairtrade Colombia Coffee Capsule",
                "gtin": "5099589226524",
                "image": "https://static.euw1.sustained.app/products/images/5099589226524_0.jpg",
                "pack": "5.3g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e5594ed1721627cf2e1b1950ca7d56d",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e5594ed1721627cf2e1b1950ca7d56d/impacts"
                }
            },
            {
                "id": "productid.0e72ec6b8950d357fe86582f04f45b4c",
                "name": "Finest Americano Colombian Microground Coffee",
                "gtin": "5059697396267",
                "image": "https://static.euw1.sustained.app/products/images/5059697396267_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 494.29431094128455
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e72ec6b8950d357fe86582f04f45b4c",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e72ec6b8950d357fe86582f04f45b4c/impacts"
                }
            },
            {
                "id": "productid.0f02a972f83d66812c606cf6a5befb39",
                "name": "Fairtade Organic Honduran Wholebean Coffee",
                "gtin": "5060399772020",
                "image": "https://static.euw1.sustained.app/products/images/5060399772020_0.jpg",
                "pack": "227g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f02a972f83d66812c606cf6a5befb39",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f02a972f83d66812c606cf6a5befb39/impacts"
                }
            },
            {
                "id": "productid.0f2c5333b611cde24acbce52ec1069f4",
                "name": "L'Or Espresso Decaffeinato Coffee Pods x16",
                "gtin": "8711000520093",
                "image": "https://static.euw1.sustained.app/products/images/8711000520093_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 506.8710723372568
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f2c5333b611cde24acbce52ec1069f4",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f2c5333b611cde24acbce52ec1069f4/impacts"
                }
            },
            {
                "id": "productid.0fc058ec1fad574c40bc43c4ca97cd5d",
                "name": "Craft Coffee SW18 Daily Grind Whole Bean",
                "gtin": "5060313403160",
                "image": "https://static.euw1.sustained.app/products/images/5060313403160_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fc058ec1fad574c40bc43c4ca97cd5d",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fc058ec1fad574c40bc43c4ca97cd5d/impacts"
                }
            },
            {
                "id": "productid.1003545d11a4456220c14469501cf194",
                "name": "Green Tea Apple & Pear 40 Tea Bags",
                "gtin": "5099810150109",
                "image": "https://static.euw1.sustained.app/products/images/5099810150109_0.jpg",
                "pack": "80g",
                "grade": "F",
                "points": {
                    "pef": 93.09574862664874
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1003545d11a4456220c14469501cf194",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1003545d11a4456220c14469501cf194/impacts"
                }
            },
            {
                "id": "productid.1016a250782014c659c945dd5116f70d",
                "name": "Finest Assam Loose Leaf",
                "gtin": "5053947622229",
                "image": "https://static.euw1.sustained.app/products/images/5053947622229_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 94.05161293443493
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1016a250782014c659c945dd5116f70d",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1016a250782014c659c945dd5116f70d/impacts"
                }
            },
            {
                "id": "productid.10d7f72a9f2fd38ab316bd14c1f5745a",
                "name": "UTZ Italia Espresso Coffee Beans",
                "gtin": "8711000297803",
                "image": "https://static.euw1.sustained.app/products/images/8711000297803_0.jpg",
                "pack": "1kg",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10d7f72a9f2fd38ab316bd14c1f5745a",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10d7f72a9f2fd38ab316bd14c1f5745a/impacts"
                }
            },
            {
                "id": "productid.111617794e265588f62ce560bad3b52c",
                "name": "Instant Coffee Powder",
                "gtin": "8711000533260",
                "image": "https://static.euw1.sustained.app/products/images/8711000533260_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 494.29431094128455
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.111617794e265588f62ce560bad3b52c",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.111617794e265588f62ce560bad3b52c/impacts"
                }
            },
            {
                "id": "productid.11288a0e197efbc9caa347ebb1b5b565",
                "name": "Chai Loose Leaf Spiced Black Tea",
                "gtin": "9330462001934",
                "image": "https://static.euw1.sustained.app/products/images/9330462001934_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11288a0e197efbc9caa347ebb1b5b565",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11288a0e197efbc9caa347ebb1b5b565/impacts"
                }
            },
            {
                "id": "productid.11f49167bd5e51f173b6d7a9b0878f94",
                "name": "18 No. 4 Dark & Intense Coffee Bags",
                "gtin": "5011667883351",
                "image": "https://static.euw1.sustained.app/products/images/5011667883351_0.jpg",
                "pack": "125g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11f49167bd5e51f173b6d7a9b0878f94",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11f49167bd5e51f173b6d7a9b0878f94/impacts"
                }
            },
            {
                "id": "productid.122faec388d4990af8f1ec0c5b18d1cd",
                "name": "Fairtrade Smooth Colombian Coffee Bags",
                "gtin": "5016311613490",
                "image": "https://static.euw1.sustained.app/products/images/5016311613490_0.jpg",
                "pack": "8g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.122faec388d4990af8f1ec0c5b18d1cd",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.122faec388d4990af8f1ec0c5b18d1cd/impacts"
                }
            },
            {
                "id": "productid.12438ed37bd4a6ceb7f1fa67cd74f940",
                "name": "Gold Chocolate & Hazelnut Latte Instant Coffee",
                "gtin": "7613287184085",
                "image": "https://static.euw1.sustained.app/products/images/7613287184085_0.jpg",
                "pack": "18.5g",
                "grade": "G",
                "points": {
                    "pef": 136.65651036394976
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12438ed37bd4a6ceb7f1fa67cd74f940",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12438ed37bd4a6ceb7f1fa67cd74f940/impacts"
                }
            },
            {
                "id": "productid.126b174b92fe14c602272dfd19066557",
                "name": "Original Tea 240 Tea Bags",
                "gtin": "5000119098665",
                "image": "https://static.euw1.sustained.app/products/images/5000119098665_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 94.05161293443493
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.126b174b92fe14c602272dfd19066557",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.126b174b92fe14c602272dfd19066557/impacts"
                }
            },
            {
                "id": "productid.13d64951fb657dd3ae05776bb83540de",
                "name": "Gold Blend Tea 80 Tea Bags",
                "gtin": "5099589220386",
                "image": "https://static.euw1.sustained.app/products/images/5099589220386_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 94.05161293443493
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.13d64951fb657dd3ae05776bb83540de",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.13d64951fb657dd3ae05776bb83540de/impacts"
                }
            },
            {
                "id": "productid.146d15dce9c5bd68171f30cbb5579e9b",
                "name": "No15 Knightsbridge Blend 50 Tea Bags",
                "gtin": "0190817162180",
                "image": "https://static.euw1.sustained.app/products/images/0190817162180_0.jpg",
                "pack": "125g",
                "grade": "G",
                "points": {
                    "pef": 94.05161293443493
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.146d15dce9c5bd68171f30cbb5579e9b",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.146d15dce9c5bd68171f30cbb5579e9b/impacts"
                }
            },
            {
                "id": "productid.1481eee018ef415337b0d74d239f28e6",
                "name": "Taste the Difference Colombian Freeze Dried Instant Coffee",
                "gtin": "0000000359818",
                "image": "https://static.euw1.sustained.app/products/images/0000000359818_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 494.29431094128455
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1481eee018ef415337b0d74d239f28e6",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1481eee018ef415337b0d74d239f28e6/impacts"
                }
            },
            {
                "id": "productid.1594d15640a78de3a20cc00c2f6ae4a0",
                "name": "Extra Special Fairtrade Ethiopian Mocha Limu Roast & Ground Coffee",
                "gtin": "5051413713112",
                "image": "https://static.euw1.sustained.app/products/images/5051413713112_0.jpg",
                "pack": "227g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1594d15640a78de3a20cc00c2f6ae4a0",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1594d15640a78de3a20cc00c2f6ae4a0/impacts"
                }
            },
            {
                "id": "productid.16722cb75d79fc613b0d7b07b283bb43",
                "name": "Mocha Italia Signature Blend Coffee Beans",
                "gtin": "5039303004151",
                "image": "https://static.euw1.sustained.app/products/images/5039303004151_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 212.8623454776422
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16722cb75d79fc613b0d7b07b283bb43",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16722cb75d79fc613b0d7b07b283bb43/impacts"
                }
            },
            {
                "id": "productid.174e86ebc5fdc4870df9c57a28dfac44",
                "name": "Silver Blend 1 Cup Tea Bags 600 Tea Bags",
                "gtin": "5397136005975",
                "image": "https://static.euw1.sustained.app/products/images/5397136005975_0.jpg",
                "pack": "1.5kg",
                "grade": "G",
                "points": {
                    "pef": 94.05161293443493
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.174e86ebc5fdc4870df9c57a28dfac44",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.174e86ebc5fdc4870df9c57a28dfac44/impacts"
                }
            },
            {
                "id": "productid.17f24349467fd9a80fd1a3c6635523d3",
                "name": "Premium Tea",
                "gtin": "5018605231687",
                "image": "https://static.euw1.sustained.app/products/images/5018605231687_0.jpg",
                "pack": "1kg",
                "grade": "G",
                "points": {
                    "pef": 94.05161293443493
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.17f24349467fd9a80fd1a3c6635523d3",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.17f24349467fd9a80fd1a3c6635523d3/impacts"
                }
            },
            {
                "id": "productid.17fa8eb68e733aa7bc623af07839d6ee",
                "name": "Moroccan Mint 15 Proper Tea Bags",
                "gtin": "5060331430278",
                "image": "https://static.euw1.sustained.app/products/images/5060331430278_0.jpg",
                "pack": "37.5g",
                "grade": "G",
                "points": {
                    "pef": 94.05161293443493
                },
                "info_icons": [],
                "category": "Hot Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.17fa8eb68e733aa7bc623af07839d6ee",
                    "category": "https://api.sustained.com/choice/v1/categories/hot-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.17fa8eb68e733aa7bc623af07839d6ee/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.015aa9984783a2ab71b1d6293932a459",
                "name": "Tiptree Strawberry Conserve",
                "gtin": "0043647900012",
                "image": "https://static.euw1.sustained.app/products/images/0043647900012_0.jpg",
                "pack": "340g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.015aa9984783a2ab71b1d6293932a459",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.015aa9984783a2ab71b1d6293932a459/impacts"
                }
            },
            {
                "id": "productid.01c9c18201a5cb97647b5a60ba366f84",
                "name": "Strawberry Jam",
                "gtin": "5391508580190",
                "image": "https://static.euw1.sustained.app/products/images/5391508580190_0.jpg",
                "pack": "30g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01c9c18201a5cb97647b5a60ba366f84",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01c9c18201a5cb97647b5a60ba366f84/impacts"
                }
            },
            {
                "id": "productid.0219ab2bb67ea40e45c277e7fc0cf818",
                "name": "Crunchy Almond Butter",
                "gtin": "5060401053895",
                "image": "https://static.euw1.sustained.app/products/images/5060401053895_0.jpg",
                "pack": "170g",
                "grade": "G",
                "points": {
                    "pef": 208.26769747312414
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0219ab2bb67ea40e45c277e7fc0cf818",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0219ab2bb67ea40e45c277e7fc0cf818/impacts"
                }
            },
            {
                "id": "productid.02fae8342067e36be84b82d2636d60e5",
                "name": "Smooth Peanut Butter",
                "gtin": "5011835101058",
                "image": "https://static.euw1.sustained.app/products/images/5011835101058_0.jpg",
                "pack": "340g",
                "grade": "F",
                "points": {
                    "pef": 62.215104876640375
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02fae8342067e36be84b82d2636d60e5",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02fae8342067e36be84b82d2636d60e5/impacts"
                }
            },
            {
                "id": "productid.0459e81fd64f009f9617a8982e34cd55",
                "name": "Stockwell & Co. Orange Marmalade",
                "gtin": "5057545986721",
                "image": "https://static.euw1.sustained.app/products/images/5057545986721_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 20.674442801122233
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0459e81fd64f009f9617a8982e34cd55",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0459e81fd64f009f9617a8982e34cd55/impacts"
                }
            },
            {
                "id": "productid.049b32690559fdbf9eaeeed380e6ad80",
                "name": "Crunchy Peanut Butter",
                "gtin": "5098732003814",
                "image": "https://static.euw1.sustained.app/products/images/5098732003814_0.jpg",
                "pack": "170g",
                "grade": "F",
                "points": {
                    "pef": 63.05972283404185
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.049b32690559fdbf9eaeeed380e6ad80",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.049b32690559fdbf9eaeeed380e6ad80/impacts"
                }
            },
            {
                "id": "productid.04fa2abd6bd35601013b7ffb4d77f737",
                "name": "Irish Blackberry & Apple Jam",
                "gtin": "5099350000025",
                "image": "https://static.euw1.sustained.app/products/images/5099350000025_0.jpg",
                "pack": "370g",
                "grade": "B",
                "points": {
                    "pef": 13.977658730371338
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04fa2abd6bd35601013b7ffb4d77f737",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04fa2abd6bd35601013b7ffb4d77f737/impacts"
                }
            },
            {
                "id": "productid.071ff4c0bb01b0f497b7cb5e66603107",
                "name": "Margetts Strawberry",
                "gtin": "5018010053096",
                "image": "https://static.euw1.sustained.app/products/images/5018010053096_0.jpg",
                "pack": "2.5kg",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.071ff4c0bb01b0f497b7cb5e66603107",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.071ff4c0bb01b0f497b7cb5e66603107/impacts"
                }
            },
            {
                "id": "productid.077cbc74bff0b786efa8222eda5d652b",
                "name": "Sweet Chilli Jam",
                "gtin": "6003770009307",
                "image": "https://static.euw1.sustained.app/products/images/6003770009307_0.jpg",
                "pack": "510g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.077cbc74bff0b786efa8222eda5d652b",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.077cbc74bff0b786efa8222eda5d652b/impacts"
                }
            },
            {
                "id": "productid.094df2031292016c2d3aa0c71bf8459c",
                "name": "Food Collection Scottish Honey",
                "gtin": "0000029184040",
                "image": "https://static.euw1.sustained.app/products/images/0000029184040_0.jpg",
                "pack": "340g",
                "grade": "D",
                "points": {
                    "pef": 26.010147905192987
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.094df2031292016c2d3aa0c71bf8459c",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.094df2031292016c2d3aa0c71bf8459c/impacts"
                }
            },
            {
                "id": "productid.09c26a619476be5139c42f1c29d31ace",
                "name": "Duchy Organic Peanut Butter Smooth",
                "gtin": "5000169604427",
                "image": "https://static.euw1.sustained.app/products/images/5000169604427_0.jpg",
                "pack": "340g",
                "grade": "F",
                "points": {
                    "pef": 62.50252176503541
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09c26a619476be5139c42f1c29d31ace",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09c26a619476be5139c42f1c29d31ace/impacts"
                }
            },
            {
                "id": "productid.09fa60b876bfcd079f3ef8c65fe59875",
                "name": "Clover Spread",
                "gtin": "5000295148611",
                "image": "https://static.euw1.sustained.app/products/images/5000295148611_0.jpg",
                "pack": "500g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09fa60b876bfcd079f3ef8c65fe59875",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09fa60b876bfcd079f3ef8c65fe59875/impacts"
                }
            },
            {
                "id": "productid.0a3d796eaf708849900c3f02758c8a65",
                "name": "Hazelnut and Chocolate Spread",
                "gtin": "8000500131329",
                "image": "https://static.euw1.sustained.app/products/images/8000500131329_0.jpg",
                "pack": "3kg",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a3d796eaf708849900c3f02758c8a65",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a3d796eaf708849900c3f02758c8a65/impacts"
                }
            },
            {
                "id": "productid.0b15ea1355645782200838d3474cddc9",
                "name": "Spreadable Honey",
                "gtin": "5011273030651",
                "image": "https://static.euw1.sustained.app/products/images/5011273030651_0.jpg",
                "pack": "1.36kg",
                "grade": "D",
                "points": {
                    "pef": 26.010147905192987
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b15ea1355645782200838d3474cddc9",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b15ea1355645782200838d3474cddc9/impacts"
                }
            },
            {
                "id": "productid.0b474d9a312b3320fd2ae1850c873a9d",
                "name": "Maca, Pecan & Almond",
                "gtin": "0760970151811",
                "image": "https://static.euw1.sustained.app/products/images/0760970151811_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 194.7153071696884
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b474d9a312b3320fd2ae1850c873a9d",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b474d9a312b3320fd2ae1850c873a9d/impacts"
                }
            },
            {
                "id": "productid.0c318c750da15812d3d366fb4018582d",
                "name": "Food Organic Zambian Honey",
                "gtin": "29255504",
                "image": "https://static.euw1.sustained.app/products/images/29255504_0.jpg",
                "pack": "340g",
                "grade": "D",
                "points": {
                    "pef": 26.010147905192987
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c318c750da15812d3d366fb4018582d",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c318c750da15812d3d366fb4018582d/impacts"
                }
            },
            {
                "id": "productid.0c411d5ab2750403c5def62abfae738a",
                "name": "Campfire Salted Caramel",
                "gtin": "5037158006726",
                "image": "https://static.euw1.sustained.app/products/images/5037158006726_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c411d5ab2750403c5def62abfae738a",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c411d5ab2750403c5def62abfae738a/impacts"
                }
            },
            {
                "id": "productid.0c793e4eaff4d29128b88a906146dcc3",
                "name": "Summer Fruits & Irish Whiskey Preserve",
                "gtin": "5099303001857",
                "image": "https://static.euw1.sustained.app/products/images/5099303001857_0.jpg",
                "pack": "113g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c793e4eaff4d29128b88a906146dcc3",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c793e4eaff4d29128b88a906146dcc3/impacts"
                }
            },
            {
                "id": "productid.0d052b6d977e131211ac20c35337593a",
                "name": "Smooth Peanut Butter",
                "gtin": "5057172515424",
                "image": "https://static.euw1.sustained.app/products/images/5057172515424_0.jpg",
                "pack": "340g",
                "grade": "C",
                "points": {
                    "pef": 21.42336604344041
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d052b6d977e131211ac20c35337593a",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d052b6d977e131211ac20c35337593a/impacts"
                }
            },
            {
                "id": "productid.0d08398f785e2e969076b54876ccbde3",
                "name": "100% Organic Honey",
                "gtin": "5099440000072",
                "image": "https://static.euw1.sustained.app/products/images/5099440000072_0.jpg",
                "pack": "340g",
                "grade": "D",
                "points": {
                    "pef": 26.010147905192987
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d08398f785e2e969076b54876ccbde3",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d08398f785e2e969076b54876ccbde3/impacts"
                }
            },
            {
                "id": "productid.0e819af9ef4f12b76d68da0cf1f3f1a5",
                "name": "Organic Honey",
                "gtin": "5011273040339",
                "image": "https://static.euw1.sustained.app/products/images/5011273040339_0.jpg",
                "pack": "340g",
                "grade": "D",
                "points": {
                    "pef": 26.010147905192987
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e819af9ef4f12b76d68da0cf1f3f1a5",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e819af9ef4f12b76d68da0cf1f3f1a5/impacts"
                }
            },
            {
                "id": "productid.0eb98b0408c83c393bb1eb900ba0e0c2",
                "name": "Bitter Orange Intense Less Sugar Marmalade",
                "gtin": "3608580930204",
                "image": "https://static.euw1.sustained.app/products/images/3608580930204_0.jpg",
                "pack": "235g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0eb98b0408c83c393bb1eb900ba0e0c2",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0eb98b0408c83c393bb1eb900ba0e0c2/impacts"
                }
            },
            {
                "id": "productid.0ebf4f8275b91a92f40a05e54c749809",
                "name": "Crunchy Organic Peanut Butter",
                "gtin": "5013665108320",
                "image": "https://static.euw1.sustained.app/products/images/5013665108320_0.jpg",
                "pack": "340g",
                "grade": "F",
                "points": {
                    "pef": 63.05972283404185
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ebf4f8275b91a92f40a05e54c749809",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ebf4f8275b91a92f40a05e54c749809/impacts"
                }
            },
            {
                "id": "productid.0edde9a733e6d07810e3a0e718152e88",
                "name": "Pure English Honey",
                "gtin": "5060118170014",
                "image": "https://static.euw1.sustained.app/products/images/5060118170014_0.jpg",
                "pack": "340g",
                "grade": "D",
                "points": {
                    "pef": 26.010147905192987
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0edde9a733e6d07810e3a0e718152e88",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0edde9a733e6d07810e3a0e718152e88/impacts"
                }
            },
            {
                "id": "productid.0f393627898138660dd989aa71b21245",
                "name": "Savers Orange Marmalade",
                "gtin": "5010251555131",
                "image": "https://static.euw1.sustained.app/products/images/5010251555131_0.jpg",
                "pack": "454g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f393627898138660dd989aa71b21245",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f393627898138660dd989aa71b21245/impacts"
                }
            },
            {
                "id": "productid.0f78716086a49b715a62ff3f9d62f8f4",
                "name": "Premium Baking Block",
                "gtin": "5010219601467",
                "image": "https://static.euw1.sustained.app/products/images/5010219601467_0.jpg",
                "pack": "250g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f78716086a49b715a62ff3f9d62f8f4",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f78716086a49b715a62ff3f9d62f8f4/impacts"
                }
            },
            {
                "id": "productid.0fc47514203609e387c503e5ebf2fa29",
                "name": "Organic Smooth Pumpkin Seed Butter",
                "gtin": "5032722313910",
                "image": "https://static.euw1.sustained.app/products/images/5032722313910_0.jpg",
                "pack": "170g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fc47514203609e387c503e5ebf2fa29",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fc47514203609e387c503e5ebf2fa29/impacts"
                }
            },
            {
                "id": "productid.0fdfdac5306980d4aaf24d0fc7d0c394",
                "name": "Organic Smooth Cashew Butter",
                "gtin": "5060132280447",
                "image": "https://static.euw1.sustained.app/products/images/5060132280447_0.jpg",
                "pack": "170g",
                "grade": "F",
                "points": {
                    "pef": 71.22429700521384
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fdfdac5306980d4aaf24d0fc7d0c394",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fdfdac5306980d4aaf24d0fc7d0c394/impacts"
                }
            },
            {
                "id": "productid.11adbfcac7e627566940d2b79038630a",
                "name": "Wexford Blackcurrant Jam",
                "gtin": "5099350000766",
                "image": "https://static.euw1.sustained.app/products/images/5099350000766_0.jpg",
                "pack": "260g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11adbfcac7e627566940d2b79038630a",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11adbfcac7e627566940d2b79038630a/impacts"
                }
            },
            {
                "id": "productid.11c120be91a5c8031459173d4777e1e7",
                "name": "Pure Organic Honey",
                "gtin": "5099303000669",
                "image": "https://static.euw1.sustained.app/products/images/5099303000669_0.jpg",
                "pack": "225g",
                "grade": "D",
                "points": {
                    "pef": 26.010147905192987
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11c120be91a5c8031459173d4777e1e7",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11c120be91a5c8031459173d4777e1e7/impacts"
                }
            },
            {
                "id": "productid.121f29381d36617f95dd28e13cc93a72",
                "name": "Nothing But Fruit Orange Marmalade",
                "gtin": "5099682001394",
                "image": "https://static.euw1.sustained.app/products/images/5099682001394_0.jpg",
                "pack": "340g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.121f29381d36617f95dd28e13cc93a72",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.121f29381d36617f95dd28e13cc93a72/impacts"
                }
            },
            {
                "id": "productid.1338c48b8d2235e4b25804fb2040044c",
                "name": "Essential Waitrose & Partners Wholenut Peanut Butter",
                "gtin": "5000169153734",
                "image": "https://static.euw1.sustained.app/products/images/5000169153734_0.jpg",
                "pack": "340g",
                "grade": "F",
                "points": {
                    "pef": 62.60434630893096
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1338c48b8d2235e4b25804fb2040044c",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1338c48b8d2235e4b25804fb2040044c/impacts"
                }
            },
            {
                "id": "productid.14a03e0589bf9bbc515b8f8c898c767e",
                "name": "Nut-Tastic Crunchy Peanut Butter",
                "gtin": "8710532462390",
                "image": "https://static.euw1.sustained.app/products/images/8710532462390_0.jpg",
                "pack": "340g",
                "grade": "F",
                "points": {
                    "pef": 66.1524877714119
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14a03e0589bf9bbc515b8f8c898c767e",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14a03e0589bf9bbc515b8f8c898c767e/impacts"
                }
            },
            {
                "id": "productid.14ad39ef5f7f7249bb7418aa05dc43b1",
                "name": "Acacia Honey",
                "gtin": "5000169170397",
                "image": "https://static.euw1.sustained.app/products/images/5000169170397_0.jpg",
                "pack": "340g",
                "grade": "D",
                "points": {
                    "pef": 26.010147905192987
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14ad39ef5f7f7249bb7418aa05dc43b1",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14ad39ef5f7f7249bb7418aa05dc43b1/impacts"
                }
            },
            {
                "id": "productid.14fdcd6e51bfc23fd5474a1603f2d095",
                "name": "Organic Canadian Honey",
                "gtin": "5060584300243",
                "image": "https://static.euw1.sustained.app/products/images/5060584300243_0.jpg",
                "pack": "330g",
                "grade": "D",
                "points": {
                    "pef": 26.010147905192987
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14fdcd6e51bfc23fd5474a1603f2d095",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14fdcd6e51bfc23fd5474a1603f2d095/impacts"
                }
            },
            {
                "id": "productid.15bec049cffce1b3cf254f06dc2acd92",
                "name": "Taste the Difference Blueberry Conserve",
                "gtin": "00457675",
                "image": "https://static.euw1.sustained.app/products/images/00457675_0.jpg",
                "pack": "340g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.15bec049cffce1b3cf254f06dc2acd92",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.15bec049cffce1b3cf254f06dc2acd92/impacts"
                }
            },
            {
                "id": "productid.172a62abcbc16a5d83699fb6ae832224",
                "name": "Onion Marmalade",
                "gtin": "5019378000166",
                "image": "https://static.euw1.sustained.app/products/images/5019378000166_0.jpg",
                "pack": "725g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.172a62abcbc16a5d83699fb6ae832224",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.172a62abcbc16a5d83699fb6ae832224/impacts"
                }
            },
            {
                "id": "productid.17a1d6f86fdeb4c9912a0ab7d3d61629",
                "name": "Blackcurrant Preserve",
                "gtin": "5391508580282",
                "image": "https://static.euw1.sustained.app/products/images/5391508580282_0.jpg",
                "pack": "340g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.17a1d6f86fdeb4c9912a0ab7d3d61629",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.17a1d6f86fdeb4c9912a0ab7d3d61629/impacts"
                }
            },
            {
                "id": "productid.17dab6f17ec6de2fd8a15bbfbf20b1c8",
                "name": "Strawberry Jam",
                "gtin": "5725112014015",
                "image": "https://static.euw1.sustained.app/products/images/5725112014015_0.jpg",
                "pack": "227g",
                "grade": "E",
                "points": {
                    "pef": 36.49051597920625
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.17dab6f17ec6de2fd8a15bbfbf20b1c8",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.17dab6f17ec6de2fd8a15bbfbf20b1c8/impacts"
                }
            },
            {
                "id": "productid.18439f100ffd2b58a8589952f9529fd3",
                "name": "Hartley's Pineapple",
                "gtin": "5060391625119",
                "image": "https://static.euw1.sustained.app/products/images/5060391625119_0.jpg",
                "pack": "300g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.18439f100ffd2b58a8589952f9529fd3",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.18439f100ffd2b58a8589952f9529fd3/impacts"
                }
            },
            {
                "id": "productid.1928327bf29ea273d5fb50c3eb1c5fe1",
                "name": "Multiflora Honey",
                "gtin": "5099440000294",
                "image": "https://static.euw1.sustained.app/products/images/5099440000294_0.jpg",
                "pack": "450g",
                "grade": "D",
                "points": {
                    "pef": 26.010147905192987
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1928327bf29ea273d5fb50c3eb1c5fe1",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1928327bf29ea273d5fb50c3eb1c5fe1/impacts"
                }
            },
            {
                "id": "productid.198bfa3ee4ba708bba2c4ace303d408e",
                "name": "Jam Strawberry",
                "gtin": "5900397006744",
                "image": "https://static.euw1.sustained.app/products/images/5900397006744_0.jpg",
                "pack": "280g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.198bfa3ee4ba708bba2c4ace303d408e",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.198bfa3ee4ba708bba2c4ace303d408e/impacts"
                }
            },
            {
                "id": "productid.1a647cd3ebf124a1df93155a676e088e",
                "name": "Organic Smooth Cashew Nut Butter",
                "gtin": "5032722305892",
                "image": "https://static.euw1.sustained.app/products/images/5032722305892_0.jpg",
                "pack": "170g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1a647cd3ebf124a1df93155a676e088e",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1a647cd3ebf124a1df93155a676e088e/impacts"
                }
            },
            {
                "id": "productid.1c7ccacf3c2ad36a2b4c29978b3ef243",
                "name": "Duchy Organic Extra Jam Preserve Strawberry",
                "gtin": "5000169151228",
                "image": "https://static.euw1.sustained.app/products/images/5000169151228_0.jpg",
                "pack": "340g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c7ccacf3c2ad36a2b4c29978b3ef243",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c7ccacf3c2ad36a2b4c29978b3ef243/impacts"
                }
            },
            {
                "id": "productid.1cc0732a3fb8b3ce7a7008e5cc35d50b",
                "name": "UMF 5+ Manuka Honey",
                "gtin": "9400501001109",
                "image": "https://static.euw1.sustained.app/products/images/9400501001109_0.jpg",
                "pack": "250g",
                "grade": "D",
                "points": {
                    "pef": 26.010147905192987
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1cc0732a3fb8b3ce7a7008e5cc35d50b",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1cc0732a3fb8b3ce7a7008e5cc35d50b/impacts"
                }
            },
            {
                "id": "productid.1dba9c29dc2714cbea6c9972cd4f6781",
                "name": "Green Fig and Earl Grey Preserve",
                "gtin": "0190817003261",
                "image": "https://static.euw1.sustained.app/products/images/0190817003261_0.jpg",
                "pack": "320g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1dba9c29dc2714cbea6c9972cd4f6781",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1dba9c29dc2714cbea6c9972cd4f6781/impacts"
                }
            },
            {
                "id": "productid.1e99dbc8ebb0970dead90d0e0a4325bc",
                "name": "Natural Gold Blend Honey",
                "gtin": "5099440000096",
                "image": "https://static.euw1.sustained.app/products/images/5099440000096_0.jpg",
                "pack": "454g",
                "grade": "D",
                "points": {
                    "pef": 26.010147905192987
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e99dbc8ebb0970dead90d0e0a4325bc",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e99dbc8ebb0970dead90d0e0a4325bc/impacts"
                }
            },
            {
                "id": "productid.1ef7bb4eeb0b6877624c52f44f54514d",
                "name": "Biscoff Crunchy Spread",
                "gtin": "5410126726947",
                "image": "https://static.euw1.sustained.app/products/images/5410126726947_0.jpg",
                "pack": "380g",
                "grade": "E",
                "points": {
                    "pef": 41.89653570034384
                },
                "info_icons": [],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1ef7bb4eeb0b6877624c52f44f54514d",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1ef7bb4eeb0b6877624c52f44f54514d/impacts"
                }
            },
            {
                "id": "productid.1f6f2161e5dd5e4e1cc460cb97faf499",
                "name": "Organic Pear Puree",
                "gtin": "5032722316119",
                "image": "https://static.euw1.sustained.app/products/images/5032722316119_0.jpg",
                "pack": "360g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1f6f2161e5dd5e4e1cc460cb97faf499",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1f6f2161e5dd5e4e1cc460cb97faf499/impacts"
                }
            },
            {
                "id": "productid.1fd11a03a8d48ac9927dd4ab47a2d7d8",
                "name": "Raspberries with Rum",
                "gtin": "5033403014690",
                "image": "https://static.euw1.sustained.app/products/images/5033403014690_0.jpg",
                "pack": "360g",
                "grade": "E",
                "points": {
                    "pef": 31.67486831275432
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Jams & Spread",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1fd11a03a8d48ac9927dd4ab47a2d7d8",
                    "category": "https://api.sustained.com/choice/v1/categories/jams-and-spread",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1fd11a03a8d48ac9927dd4ab47a2d7d8/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.010b99c560fefc7b6ff00ad98466ccb6",
                "name": "British Lamb's Liver",
                "gtin": "0236370000000",
                "image": "https://static.euw1.sustained.app/products/images/0236370000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.010b99c560fefc7b6ff00ad98466ccb6",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.010b99c560fefc7b6ff00ad98466ccb6/impacts"
                }
            },
            {
                "id": "productid.0ac606361a56e046bf6b8530391bedc2",
                "name": "10% Fat Lamb Mince",
                "gtin": "00255967",
                "image": "https://static.euw1.sustained.app/products/images/00255967_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ac606361a56e046bf6b8530391bedc2",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ac606361a56e046bf6b8530391bedc2/impacts"
                }
            },
            {
                "id": "productid.0aed963055c49e11651a8a789d83d7b1",
                "name": "Essential Waitrose & Partners British Lamb Half Shoulder",
                "gtin": "0296100000006",
                "image": "https://static.euw1.sustained.app/products/images/0296100000006_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0aed963055c49e11651a8a789d83d7b1",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0aed963055c49e11651a8a789d83d7b1/impacts"
                }
            },
            {
                "id": "productid.0c7f918d225faca2f200ce9fe81ce50d",
                "name": "Welsh Lamb Leg",
                "gtin": "0252770000005",
                "image": "https://static.euw1.sustained.app/products/images/0252770000005_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c7f918d225faca2f200ce9fe81ce50d",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c7f918d225faca2f200ce9fe81ce50d/impacts"
                }
            },
            {
                "id": "productid.12c3f3c3f11df3a69d3399954f769b3f",
                "name": "Duchy Organic British Lamb Half Leg",
                "gtin": "0296210000002",
                "image": "https://static.euw1.sustained.app/products/images/0296210000002_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12c3f3c3f11df3a69d3399954f769b3f",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12c3f3c3f11df3a69d3399954f769b3f/impacts"
                }
            },
            {
                "id": "productid.13a28cc887b2674cfdadd7c00bfab780",
                "name": "British Lamb Cutlets",
                "gtin": "2049423000004",
                "image": "https://static.euw1.sustained.app/products/images/2049423000004_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.13a28cc887b2674cfdadd7c00bfab780",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.13a28cc887b2674cfdadd7c00bfab780/impacts"
                }
            },
            {
                "id": "productid.1409f3b639a99fe4b3633aa8380f4c11",
                "name": "Tender Lamb Shoulder",
                "gtin": "0238710000007",
                "image": "https://static.euw1.sustained.app/products/images/0238710000007_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1409f3b639a99fe4b3633aa8380f4c11",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1409f3b639a99fe4b3633aa8380f4c11/impacts"
                }
            },
            {
                "id": "productid.1708a2e4b89fc77d8719c0ae6986f130",
                "name": "British Sliced Lamb Liver",
                "gtin": "0246100000000",
                "image": "https://static.euw1.sustained.app/products/images/0246100000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1708a2e4b89fc77d8719c0ae6986f130",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1708a2e4b89fc77d8719c0ae6986f130/impacts"
                }
            },
            {
                "id": "productid.1769b72fa83836db2ba66926d634359b",
                "name": "The Butcher's On Market Street British Lamb Stuffed Breast",
                "gtin": "0258820000001",
                "image": "https://static.euw1.sustained.app/products/images/0258820000001_0.jpg",
                "pack": "0.34kg",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1769b72fa83836db2ba66926d634359b",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1769b72fa83836db2ba66926d634359b/impacts"
                }
            },
            {
                "id": "productid.1b90f6a6273602611818ce2f7c1fd78d",
                "name": "British Lamb Whole Shoulder",
                "gtin": "02115300000006",
                "image": "https://static.euw1.sustained.app/products/images/02115300000006_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b90f6a6273602611818ce2f7c1fd78d",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b90f6a6273602611818ce2f7c1fd78d/impacts"
                }
            },
            {
                "id": "productid.1c496f0cc6fb4330d3269230ab60fa1e",
                "name": "Welsh Lamb Rack",
                "gtin": "0289770000001",
                "image": "https://static.euw1.sustained.app/products/images/0289770000001_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c496f0cc6fb4330d3269230ab60fa1e",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c496f0cc6fb4330d3269230ab60fa1e/impacts"
                }
            },
            {
                "id": "productid.211ad851fab6489d78e124a662bf3040",
                "name": "Counter West Country Lamb Loin",
                "gtin": "0238860000000",
                "image": "https://static.euw1.sustained.app/products/images/0238860000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.211ad851fab6489d78e124a662bf3040",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.211ad851fab6489d78e124a662bf3040/impacts"
                }
            },
            {
                "id": "productid.22ed806291e91652b5596730b20b70be",
                "name": "2 Welsh Lamb Leg Steaks",
                "gtin": "5000169504888",
                "image": "https://static.euw1.sustained.app/products/images/5000169504888_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 458.75468282299516
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.22ed806291e91652b5596730b20b70be",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.22ed806291e91652b5596730b20b70be/impacts"
                }
            },
            {
                "id": "productid.2315c15ffe7448f93ea845e234ad20ec",
                "name": "Our Family Butcher Boneless Pork Shoulder Joint Large",
                "gtin": "2331702000000",
                "image": "https://static.euw1.sustained.app/products/images/2331702000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2315c15ffe7448f93ea845e234ad20ec",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2315c15ffe7448f93ea845e234ad20ec/impacts"
                }
            },
            {
                "id": "productid.231b2af9b16e4bb91aa79023bed1c1a0",
                "name": "Scotch Lamb Shoulder Joint",
                "gtin": "0288220000000",
                "image": "https://static.euw1.sustained.app/products/images/0288220000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.231b2af9b16e4bb91aa79023bed1c1a0",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.231b2af9b16e4bb91aa79023bed1c1a0/impacts"
                }
            },
            {
                "id": "productid.243ac4fdeee70623efed9817e03fd1d9",
                "name": "Butcher's Selection Lamb Escalopes",
                "gtin": "5054781688327",
                "image": "https://static.euw1.sustained.app/products/images/5054781688327_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.243ac4fdeee70623efed9817e03fd1d9",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.243ac4fdeee70623efed9817e03fd1d9/impacts"
                }
            },
            {
                "id": "productid.2aa0078171bbc9be9457c3bd8b9ee48d",
                "name": "Taste the Difference Welsh Hill Lamb Rack",
                "gtin": "0228450000009",
                "image": "https://static.euw1.sustained.app/products/images/0228450000009_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2aa0078171bbc9be9457c3bd8b9ee48d",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2aa0078171bbc9be9457c3bd8b9ee48d/impacts"
                }
            },
            {
                "id": "productid.2dcb2924fa17fd8dfbee334738d82859",
                "name": "Easy Carve Lamb Shoulder",
                "gtin": "0223930000000",
                "image": "https://static.euw1.sustained.app/products/images/0223930000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2dcb2924fa17fd8dfbee334738d82859",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2dcb2924fa17fd8dfbee334738d82859/impacts"
                }
            },
            {
                "id": "productid.33b910599975d267b710ad6be271f863",
                "name": "Our Family Butcher Lamb Guard of Honour",
                "gtin": "2334077000000",
                "image": "https://static.euw1.sustained.app/products/images/2334077000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.33b910599975d267b710ad6be271f863",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.33b910599975d267b710ad6be271f863/impacts"
                }
            },
            {
                "id": "productid.3ca049f7239add7acafed39a6cc14658",
                "name": "The Butcher's on Market Street British Lamb Shank",
                "gtin": "0299050000003",
                "image": "https://static.euw1.sustained.app/products/images/0299050000003_0.jpg",
                "pack": "0.456kg",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3ca049f7239add7acafed39a6cc14658",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3ca049f7239add7acafed39a6cc14658/impacts"
                }
            },
            {
                "id": "productid.3e2c82b9d70e1c54dc6e446600ce61d9",
                "name": "Haji Baba Masala Mutton Chops",
                "gtin": "0209030000008",
                "image": "https://static.euw1.sustained.app/products/images/0209030000008_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3e2c82b9d70e1c54dc6e446600ce61d9",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3e2c82b9d70e1c54dc6e446600ce61d9/impacts"
                }
            },
            {
                "id": "productid.4121f2a675ae57e216fa47a6188daa8e",
                "name": "12 Lamb Koftas",
                "gtin": "5016901002857",
                "image": "https://static.euw1.sustained.app/products/images/5016901002857_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 422.3568008134784
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4121f2a675ae57e216fa47a6188daa8e",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4121f2a675ae57e216fa47a6188daa8e/impacts"
                }
            },
            {
                "id": "productid.450216081abe8c64d9009a4e49f27274",
                "name": "Welsh Counter Lamb Leg Shanks",
                "gtin": "0237270000000",
                "image": "https://static.euw1.sustained.app/products/images/0237270000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.450216081abe8c64d9009a4e49f27274",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.450216081abe8c64d9009a4e49f27274/impacts"
                }
            },
            {
                "id": "productid.455e44b2ea72718e6e3364c93ba82fc5",
                "name": "Easy Carve Lamb Leg Joint",
                "gtin": "0245100000000",
                "image": "https://static.euw1.sustained.app/products/images/0245100000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.455e44b2ea72718e6e3364c93ba82fc5",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.455e44b2ea72718e6e3364c93ba82fc5/impacts"
                }
            },
            {
                "id": "productid.4cbb96afb8370255952b03b520511604",
                "name": "Sliced Lamb Liver",
                "gtin": "0236350000000",
                "image": "https://static.euw1.sustained.app/products/images/0236350000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4cbb96afb8370255952b03b520511604",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4cbb96afb8370255952b03b520511604/impacts"
                }
            },
            {
                "id": "productid.4fc0efca4c74cdd34a7b584bddbdc13c",
                "name": "Counter British Lamb Liver",
                "gtin": "0238360000000",
                "image": "https://static.euw1.sustained.app/products/images/0238360000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4fc0efca4c74cdd34a7b584bddbdc13c",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4fc0efca4c74cdd34a7b584bddbdc13c/impacts"
                }
            },
            {
                "id": "productid.52c9253ceeb0f094e90494601326726c",
                "name": "Welsh Lamb Hind Shanks",
                "gtin": "0272010000008",
                "image": "https://static.euw1.sustained.app/products/images/0272010000008_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.52c9253ceeb0f094e90494601326726c",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.52c9253ceeb0f094e90494601326726c/impacts"
                }
            },
            {
                "id": "productid.5986ea0410d09b03d4ef08467ca9ce64",
                "name": "Mutton Mix",
                "gtin": "0208670000003",
                "image": "https://static.euw1.sustained.app/products/images/0208670000003_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5986ea0410d09b03d4ef08467ca9ce64",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5986ea0410d09b03d4ef08467ca9ce64/impacts"
                }
            },
            {
                "id": "productid.5be6cd6fe75dcc871f2692c10eea3830",
                "name": "4 Bone Rack of Lamb",
                "gtin": "5027615060651",
                "image": "https://static.euw1.sustained.app/products/images/5027615060651_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5be6cd6fe75dcc871f2692c10eea3830",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5be6cd6fe75dcc871f2692c10eea3830/impacts"
                }
            },
            {
                "id": "productid.604bbb6c96efad2c397eb00f8f04c70a",
                "name": "Our Family Butcher Lamb Cutlets 4 Pack",
                "gtin": "2331689000000",
                "image": "https://static.euw1.sustained.app/products/images/2331689000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.604bbb6c96efad2c397eb00f8f04c70a",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.604bbb6c96efad2c397eb00f8f04c70a/impacts"
                }
            },
            {
                "id": "productid.64d73f7f92b710db244bc90473b56447",
                "name": "2 Welsh Lamb Leg Shanks",
                "gtin": "0279670000003",
                "image": "https://static.euw1.sustained.app/products/images/0279670000003_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.64d73f7f92b710db244bc90473b56447",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.64d73f7f92b710db244bc90473b56447/impacts"
                }
            },
            {
                "id": "productid.67f43d341c51d817a8bb4e130762a334",
                "name": "Irish Lamb Loin Chops",
                "gtin": "5057753153014",
                "image": "https://static.euw1.sustained.app/products/images/5057753153014_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.67f43d341c51d817a8bb4e130762a334",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.67f43d341c51d817a8bb4e130762a334/impacts"
                }
            },
            {
                "id": "productid.75970330adcd1de07e9a93e6db31172b",
                "name": "SO Organic British Lamb Steaks",
                "gtin": "0229310000009",
                "image": "https://static.euw1.sustained.app/products/images/0229310000009_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.75970330adcd1de07e9a93e6db31172b",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.75970330adcd1de07e9a93e6db31172b/impacts"
                }
            },
            {
                "id": "productid.7868e5aeae0abc99e367e7c873cb359a",
                "name": "Irish Lamb Leg with Garlic and Pepper Butter",
                "gtin": "0248890000000",
                "image": "https://static.euw1.sustained.app/products/images/0248890000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.7868e5aeae0abc99e367e7c873cb359a",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.7868e5aeae0abc99e367e7c873cb359a/impacts"
                }
            },
            {
                "id": "productid.786c63f73731c658821e7fbd2e071153",
                "name": "Irish Lamb Boneless Shoulder Chops",
                "gtin": "5057753600150",
                "image": "https://static.euw1.sustained.app/products/images/5057753600150_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.786c63f73731c658821e7fbd2e071153",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.786c63f73731c658821e7fbd2e071153/impacts"
                }
            },
            {
                "id": "productid.7dd8defcf0c9db48eae32bd395f29dc7",
                "name": "Market Street Flavoured Lamb Ribs",
                "gtin": "0229940000004",
                "image": "https://static.euw1.sustained.app/products/images/0229940000004_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.7dd8defcf0c9db48eae32bd395f29dc7",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.7dd8defcf0c9db48eae32bd395f29dc7/impacts"
                }
            },
            {
                "id": "productid.804b1648a257a91659a9d2be13649a9a",
                "name": "Lamb Cutlets",
                "gtin": "2078353000006",
                "image": "https://static.euw1.sustained.app/products/images/2078353000006_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.804b1648a257a91659a9d2be13649a9a",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.804b1648a257a91659a9d2be13649a9a/impacts"
                }
            },
            {
                "id": "productid.85336f008214f453f03f2fe2746b921a",
                "name": "Counter West Country Lamb Rump",
                "gtin": "0238780000000",
                "image": "https://static.euw1.sustained.app/products/images/0238780000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.85336f008214f453f03f2fe2746b921a",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.85336f008214f453f03f2fe2746b921a/impacts"
                }
            },
            {
                "id": "productid.85d7bc164b0f3fd1b6d222529a8dc26d",
                "name": "Boneless Lamb Leg Joint",
                "gtin": "2059339000000",
                "image": "https://static.euw1.sustained.app/products/images/2059339000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.85d7bc164b0f3fd1b6d222529a8dc26d",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.85d7bc164b0f3fd1b6d222529a8dc26d/impacts"
                }
            },
            {
                "id": "productid.883cce920cba2cb147fe3b23d68c4bec",
                "name": "Flavourful Diced Lamb",
                "gtin": "5052449451740",
                "image": "https://static.euw1.sustained.app/products/images/5052449451740_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.883cce920cba2cb147fe3b23d68c4bec",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.883cce920cba2cb147fe3b23d68c4bec/impacts"
                }
            },
            {
                "id": "productid.8cbd3040be8b4705d3459389f6b62334",
                "name": "Welsh Thin Cut Lamb Chops",
                "gtin": "0000000347037",
                "image": "https://static.euw1.sustained.app/products/images/0000000347037_0.jpg",
                "pack": "425g",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.8cbd3040be8b4705d3459389f6b62334",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.8cbd3040be8b4705d3459389f6b62334/impacts"
                }
            },
            {
                "id": "productid.932382445c82b7f406e25422e19ee03b",
                "name": "The Butcher's On Market Street British Lamb Chops",
                "gtin": "5010525065083",
                "image": "https://static.euw1.sustained.app/products/images/5010525065083_0.jpg",
                "pack": "0.360kg",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.932382445c82b7f406e25422e19ee03b",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.932382445c82b7f406e25422e19ee03b/impacts"
                }
            },
            {
                "id": "productid.97b96343e5153045662ba4270a168f7f",
                "name": "Butcher's Selection 8 Minted Lamb Kebabs",
                "gtin": "5054781379836",
                "image": "https://static.euw1.sustained.app/products/images/5054781379836_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 375.86751666478153
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.97b96343e5153045662ba4270a168f7f",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.97b96343e5153045662ba4270a168f7f/impacts"
                }
            },
            {
                "id": "productid.98a193680ee68df24aaa6abd5481b23b",
                "name": "Butcher's Selection Northern Ireland Lamb Boneless Rolled Breast",
                "gtin": "2093800000002",
                "image": "https://static.euw1.sustained.app/products/images/2093800000002_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.98a193680ee68df24aaa6abd5481b23b",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.98a193680ee68df24aaa6abd5481b23b/impacts"
                }
            },
            {
                "id": "productid.9a394219d94afa35c0af090ca57124b0",
                "name": "The Butcher's On Market Street British Lamb Leg Joint",
                "gtin": "2334475000000",
                "image": "https://static.euw1.sustained.app/products/images/2334475000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.9a394219d94afa35c0af090ca57124b0",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.9a394219d94afa35c0af090ca57124b0/impacts"
                }
            },
            {
                "id": "productid.9dc6945878af17c1ca5ff5442fc46006",
                "name": "British Lamb Shanks",
                "gtin": "0247760000000",
                "image": "https://static.euw1.sustained.app/products/images/0247760000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.9dc6945878af17c1ca5ff5442fc46006",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.9dc6945878af17c1ca5ff5442fc46006/impacts"
                }
            },
            {
                "id": "productid.9fe244be014c9b04e9024a86998001c4",
                "name": "Diced Shoulder of Lamb",
                "gtin": "00328388",
                "image": "https://static.euw1.sustained.app/products/images/00328388_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.9fe244be014c9b04e9024a86998001c4",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.9fe244be014c9b04e9024a86998001c4/impacts"
                }
            },
            {
                "id": "productid.a25fe1d5cf763431a7294f417a5d260d",
                "name": "Counter British Lamb Loin",
                "gtin": "0233740000000",
                "image": "https://static.euw1.sustained.app/products/images/0233740000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.a25fe1d5cf763431a7294f417a5d260d",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.a25fe1d5cf763431a7294f417a5d260d/impacts"
                }
            },
            {
                "id": "productid.a6e81e7809eb84a4a823afcf8fd8f2d1",
                "name": "20% Fat Lamb Mince",
                "gtin": "01308693",
                "image": "https://static.euw1.sustained.app/products/images/01308693_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.a6e81e7809eb84a4a823afcf8fd8f2d1",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.a6e81e7809eb84a4a823afcf8fd8f2d1/impacts"
                }
            },
            {
                "id": "productid.a85d150b9acc3c1e5d81e2f0a5554822",
                "name": "The Butcher's On Market Street British Lamb Mince",
                "gtin": "5010251986584",
                "image": "https://static.euw1.sustained.app/products/images/5010251986584_0.jpg",
                "pack": "0.250kg",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [],
                "category": "Lamb",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.a85d150b9acc3c1e5d81e2f0a5554822",
                    "category": "https://api.sustained.com/choice/v1/categories/lamb",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.a85d150b9acc3c1e5d81e2f0a5554822/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.502185c7f3f963e4a1a004440e35d6f3",
                "name": "Classic Chilli",
                "gtin": "5019503038576",
                "image": "https://static.euw1.sustained.app/products/images/5019503038576_0.jpg",
                "pack": "225g",
                "grade": "A",
                "points": {
                    "pef": 9.801256065752332
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Meat Free Alternatives",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.502185c7f3f963e4a1a004440e35d6f3",
                    "category": "https://api.sustained.com/choice/v1/categories/meat-free-alternatives",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.502185c7f3f963e4a1a004440e35d6f3/impacts"
                }
            },
            {
                "id": "productid.dae3e5f831e6e3e078e5ae393e68123c",
                "name": "Crispy Nuggets",
                "gtin": "5019503012019",
                "image": "https://static.euw1.sustained.app/products/images/5019503012019_0.jpg",
                "pack": "300g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Meat Free Alternatives",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.dae3e5f831e6e3e078e5ae393e68123c",
                    "category": "https://api.sustained.com/choice/v1/categories/meat-free-alternatives",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.dae3e5f831e6e3e078e5ae393e68123c/impacts"
                }
            },
            {
                "id": "productid.284065b42f65f5ae99dafaf16909f8cf",
                "name": "Brilliant Burgers",
                "gtin": "5019503001327",
                "image": "https://static.euw1.sustained.app/products/images/5019503001327_0.jpg",
                "pack": "200g",
                "grade": "",
                "points": {
                    "pef": 0
                },
                "info_icons": [],
                "category": "Meat Free Alternatives",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.284065b42f65f5ae99dafaf16909f8cf",
                    "category": "https://api.sustained.com/choice/v1/categories/meat-free-alternatives",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.284065b42f65f5ae99dafaf16909f8cf/impacts"
                }
            },
            {
                "id": "productid.2e40f0a6699ce9d8cc05b48abfabeaca",
                "name": "Vegetarian Pepperoni Slices",
                "gtin": "5019503022117",
                "image": "https://static.euw1.sustained.app/products/images/5019503022117_0.jpg",
                "pack": "100g",
                "grade": "",
                "points": {
                    "pef": 0
                },
                "info_icons": [],
                "category": "Meat Free Alternatives",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2e40f0a6699ce9d8cc05b48abfabeaca",
                    "category": "https://api.sustained.com/choice/v1/categories/meat-free-alternatives",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2e40f0a6699ce9d8cc05b48abfabeaca/impacts"
                }
            },
            {
                "id": "productid.ab62a65a7d936a5797b6cc4c3ac29977",
                "name": "Vegetarian Ham Slices",
                "gtin": "5019503039917",
                "image": "https://static.euw1.sustained.app/products/images/5019503039917_0.jpg",
                "pack": "170g",
                "grade": "",
                "points": {
                    "pef": 0
                },
                "info_icons": [],
                "category": "Meat Free Alternatives",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.ab62a65a7d936a5797b6cc4c3ac29977",
                    "category": "https://api.sustained.com/choice/v1/categories/meat-free-alternatives",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.ab62a65a7d936a5797b6cc4c3ac29977/impacts"
                }
            },
            {
                "id": "productid.0f6622daab7df9fa1ba3c57d12238e53",
                "name": "Vegetarian Chicken Slices",
                "gtin": "5019503039924",
                "image": "https://static.euw1.sustained.app/products/images/5019503039924_0.jpg",
                "pack": "170g",
                "grade": "",
                "points": {
                    "pef": 0
                },
                "info_icons": [],
                "category": "Meat Free Alternatives",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f6622daab7df9fa1ba3c57d12238e53",
                    "category": "https://api.sustained.com/choice/v1/categories/meat-free-alternatives",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f6622daab7df9fa1ba3c57d12238e53/impacts"
                }
            },
            {
                "id": "productid.ba56c716dbd798605d0ed6624de6f87a",
                "name": "Brilliant Burgers",
                "gtin": "5019503018097",
                "image": "https://static.euw1.sustained.app/products/images/5019503018097_0.jpg",
                "pack": "300g",
                "grade": "",
                "points": {
                    "pef": 0
                },
                "info_icons": [],
                "category": "Meat Free Alternatives",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.ba56c716dbd798605d0ed6624de6f87a",
                    "category": "https://api.sustained.com/choice/v1/categories/meat-free-alternatives",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.ba56c716dbd798605d0ed6624de6f87a/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.02ea27b8d9c88d082b7268e23bd65fd6",
                "name": "Organic Fresh Whole Milk 3.52 Pints/2L",
                "gtin": "5000245006640",
                "image": "https://static.euw1.sustained.app/products/images/5000245006640_0.jpg",
                "pack": "2l",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02ea27b8d9c88d082b7268e23bd65fd6",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02ea27b8d9c88d082b7268e23bd65fd6/impacts"
                }
            },
            {
                "id": "productid.0456c64289c7429791d327e7eb707ccb",
                "name": "Fresh Cream",
                "gtin": "5011056030410",
                "image": "https://static.euw1.sustained.app/products/images/5011056030410_0.jpg",
                "pack": "170ml",
                "grade": "E",
                "points": {
                    "pef": 33.930784479140115
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0456c64289c7429791d327e7eb707ccb",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0456c64289c7429791d327e7eb707ccb/impacts"
                }
            },
            {
                "id": "productid.055b74231645188da55b745280a93947",
                "name": "Ballinahina Fresh Milk",
                "gtin": "5099534000551",
                "image": "https://static.euw1.sustained.app/products/images/5099534000551_0.jpg",
                "pack": "2l",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.055b74231645188da55b745280a93947",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.055b74231645188da55b745280a93947/impacts"
                }
            },
            {
                "id": "productid.05ba8f4e16649f992ff896c874e660c9",
                "name": "White Chocolate Mocha Iced Coffee Drink",
                "gtin": "5711953002342",
                "image": "https://static.euw1.sustained.app/products/images/5711953002342_0.jpg",
                "pack": "220ml",
                "grade": "E",
                "points": {
                    "pef": 40.2760007161979
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05ba8f4e16649f992ff896c874e660c9",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05ba8f4e16649f992ff896c874e660c9/impacts"
                }
            },
            {
                "id": "productid.05cc46b7fe0aebcea82ed132a02c7ef2",
                "name": "Barista Whole Milk",
                "gtin": "5099534001619",
                "image": "https://static.euw1.sustained.app/products/images/5099534001619_0.jpg",
                "pack": "3l",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05cc46b7fe0aebcea82ed132a02c7ef2",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05cc46b7fe0aebcea82ed132a02c7ef2/impacts"
                }
            },
            {
                "id": "productid.07476dea35ddfd177121e016cf881da9",
                "name": "Coconut Milk",
                "gtin": "8710605096170",
                "image": "https://static.euw1.sustained.app/products/images/8710605096170_0.jpg",
                "pack": "250ml",
                "grade": "A",
                "points": {
                    "pef": 6.815352505087667
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07476dea35ddfd177121e016cf881da9",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07476dea35ddfd177121e016cf881da9/impacts"
                }
            },
            {
                "id": "productid.074d10ae4d72e96893c4747c2301206d",
                "name": "Salted Caramel",
                "gtin": "5060342470638",
                "image": "https://static.euw1.sustained.app/products/images/5060342470638_0.jpg",
                "pack": "750ml",
                "grade": "B",
                "points": {
                    "pef": 15.289428675570663
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.074d10ae4d72e96893c4747c2301206d",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.074d10ae4d72e96893c4747c2301206d/impacts"
                }
            },
            {
                "id": "productid.07d9db52c6161e10d074c2646fc21553",
                "name": "Organic Coconut Milk Light",
                "gtin": "5032722311534",
                "image": "https://static.euw1.sustained.app/products/images/5032722311534_0.jpg",
                "pack": "400ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07d9db52c6161e10d074c2646fc21553",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07d9db52c6161e10d074c2646fc21553/impacts"
                }
            },
            {
                "id": "productid.07f4881f4bd074e3ec7ae1c29eb56241",
                "name": "Food Select Farms British Whole Milk 1 Pint/568ml",
                "gtin": "0000000187213",
                "image": "https://static.euw1.sustained.app/products/images/0000000187213_0.jpg",
                "pack": "568ml",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07f4881f4bd074e3ec7ae1c29eb56241",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07f4881f4bd074e3ec7ae1c29eb56241/impacts"
                }
            },
            {
                "id": "productid.093ea7ac2f5e254d11ea07572461ce17",
                "name": "Custard UHT",
                "gtin": "5000401001328",
                "image": "https://static.euw1.sustained.app/products/images/5000401001328_0.jpg",
                "pack": "1kg",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.093ea7ac2f5e254d11ea07572461ce17",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.093ea7ac2f5e254d11ea07572461ce17/impacts"
                }
            },
            {
                "id": "productid.09e2bffeb6cc95f6bdcd91ef5a9bbe08",
                "name": "British Single Cream",
                "gtin": "5010251724322",
                "image": "https://static.euw1.sustained.app/products/images/5010251724322_0.jpg",
                "pack": "300ml",
                "grade": "E",
                "points": {
                    "pef": 33.930784479140115
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09e2bffeb6cc95f6bdcd91ef5a9bbe08",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09e2bffeb6cc95f6bdcd91ef5a9bbe08/impacts"
                }
            },
            {
                "id": "productid.0a7d8d12123804f0ff01e8c5a6c6ea60",
                "name": "Coconut Drink with Cocoa",
                "gtin": "5016860000857",
                "image": "https://static.euw1.sustained.app/products/images/5016860000857_0.jpg",
                "pack": "330ml",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a7d8d12123804f0ff01e8c5a6c6ea60",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a7d8d12123804f0ff01e8c5a6c6ea60/impacts"
                }
            },
            {
                "id": "productid.0cd37e299f61e7bf8c343f0eccb81b0c",
                "name": "Filtered Fresh Semi Skimmed Milk",
                "gtin": "5000181036886",
                "image": "https://static.euw1.sustained.app/products/images/5000181036886_0.jpg",
                "pack": "250ml",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0cd37e299f61e7bf8c343f0eccb81b0c",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0cd37e299f61e7bf8c343f0eccb81b0c/impacts"
                }
            },
            {
                "id": "productid.0fd7fecb3ad6751a1017272c08ebcb15",
                "name": "Fresh Double Cream",
                "gtin": "5050854280047",
                "image": "https://static.euw1.sustained.app/products/images/5050854280047_0.jpg",
                "pack": "250ml",
                "grade": "E",
                "points": {
                    "pef": 33.73830213307577
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fd7fecb3ad6751a1017272c08ebcb15",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fd7fecb3ad6751a1017272c08ebcb15/impacts"
                }
            },
            {
                "id": "productid.0ff7be494c37ff1feaf82dbab56e3f55",
                "name": "Lactofree Cream",
                "gtin": "7310865075376",
                "image": "https://static.euw1.sustained.app/products/images/7310865075376_0.jpg",
                "pack": "250ml",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ff7be494c37ff1feaf82dbab56e3f55",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ff7be494c37ff1feaf82dbab56e3f55/impacts"
                }
            },
            {
                "id": "productid.108ee16b44cc97fe62773a0a409efbc0",
                "name": "Original Milk",
                "gtin": "5060674960005",
                "image": "https://static.euw1.sustained.app/products/images/5060674960005_0.jpg",
                "pack": "1l",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.108ee16b44cc97fe62773a0a409efbc0",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.108ee16b44cc97fe62773a0a409efbc0/impacts"
                }
            },
            {
                "id": "productid.10eaad88a9b101cbea1df196c1924cc1",
                "name": "Soy Drink",
                "gtin": "4891028707585",
                "image": "https://static.euw1.sustained.app/products/images/4891028707585_0.jpg",
                "pack": "1l",
                "grade": "C",
                "points": {
                    "pef": 17.45362855199412
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10eaad88a9b101cbea1df196c1924cc1",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10eaad88a9b101cbea1df196c1924cc1/impacts"
                }
            },
            {
                "id": "productid.131be407bbd2e49322a2b5831c14ed7a",
                "name": "Breakfast Drink Banana Flavour Milk Drink",
                "gtin": "5060201623878",
                "image": "https://static.euw1.sustained.app/products/images/5060201623878_0.jpg",
                "pack": "330ml",
                "grade": "C",
                "points": {
                    "pef": 16.740651140436256
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.131be407bbd2e49322a2b5831c14ed7a",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.131be407bbd2e49322a2b5831c14ed7a/impacts"
                }
            },
            {
                "id": "productid.137b26869cbb7dd91e33c296405d1782",
                "name": "Fat Free Milk",
                "gtin": "5099534001275",
                "image": "https://static.euw1.sustained.app/products/images/5099534001275_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655623
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.137b26869cbb7dd91e33c296405d1782",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.137b26869cbb7dd91e33c296405d1782/impacts"
                }
            },
            {
                "id": "productid.1406aa522672b73795e2f4d68da518e8",
                "name": "Whole Super Milk",
                "gtin": "5011056021760",
                "image": "https://static.euw1.sustained.app/products/images/5011056021760_0.jpg",
                "pack": "1l",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1406aa522672b73795e2f4d68da518e8",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1406aa522672b73795e2f4d68da518e8/impacts"
                }
            },
            {
                "id": "productid.14db03db90a4dc3a976fe467cf17a7c0",
                "name": "Semi-Skimmed Milk",
                "gtin": "5000119110541",
                "image": "https://static.euw1.sustained.app/products/images/5000119110541_0.jpg",
                "pack": "568ml",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14db03db90a4dc3a976fe467cf17a7c0",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14db03db90a4dc3a976fe467cf17a7c0/impacts"
                }
            },
            {
                "id": "productid.1625f274c6faeb81e37f211c62eed506",
                "name": "Food Made Without Lactose Semi-Skimmed Milk",
                "gtin": "0000000915847",
                "image": "https://static.euw1.sustained.app/products/images/0000000915847_0.jpg",
                "pack": "1l",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1625f274c6faeb81e37f211c62eed506",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1625f274c6faeb81e37f211c62eed506/impacts"
                }
            },
            {
                "id": "productid.167299ef1963fb93c7fc4c274e3d01d9",
                "name": "Banana Milkshake Powder",
                "gtin": "8445290293312",
                "image": "https://static.euw1.sustained.app/products/images/8445290293312_0.jpg",
                "pack": "500g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.167299ef1963fb93c7fc4c274e3d01d9",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.167299ef1963fb93c7fc4c274e3d01d9/impacts"
                }
            },
            {
                "id": "productid.16cf4258fccbe6b9fd3604a45c62ca5c",
                "name": "Best of British Fresh Double Cream",
                "gtin": "5050854830709",
                "image": "https://static.euw1.sustained.app/products/images/5050854830709_0.jpg",
                "pack": "600ml",
                "grade": "E",
                "points": {
                    "pef": 33.73830213307577
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16cf4258fccbe6b9fd3604a45c62ca5c",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16cf4258fccbe6b9fd3604a45c62ca5c/impacts"
                }
            },
            {
                "id": "productid.183daf726b18fb401c1a69604a7df228",
                "name": "The Locals' Milk Cornish Whole Milk",
                "gtin": "5019608600050",
                "image": "https://static.euw1.sustained.app/products/images/5019608600050_0.jpg",
                "pack": "2l",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.183daf726b18fb401c1a69604a7df228",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.183daf726b18fb401c1a69604a7df228/impacts"
                }
            },
            {
                "id": "productid.1947ab90db58ac3bae299c5c107bdd07",
                "name": "Gentle Skimmed Fresh Pasteurised Goats' Milk",
                "gtin": "5015326100025",
                "image": "https://static.euw1.sustained.app/products/images/5015326100025_0.jpg",
                "pack": "1l",
                "grade": "D",
                "points": {
                    "pef": 22.59806356588644
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1947ab90db58ac3bae299c5c107bdd07",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1947ab90db58ac3bae299c5c107bdd07/impacts"
                }
            },
            {
                "id": "productid.1ad3fc3d64d20839b3ba5763600db0c6",
                "name": "Ice Cream Mix",
                "gtin": "5010652989924",
                "image": "https://static.euw1.sustained.app/products/images/5010652989924_0.jpg",
                "pack": "1l",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1ad3fc3d64d20839b3ba5763600db0c6",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1ad3fc3d64d20839b3ba5763600db0c6/impacts"
                }
            },
            {
                "id": "productid.1d043db562631db164bfae549b459bff",
                "name": "Chocolate Milkshake Drink",
                "gtin": "5056357907368",
                "image": "https://static.euw1.sustained.app/products/images/5056357907368_0.jpg",
                "pack": "376ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1d043db562631db164bfae549b459bff",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1d043db562631db164bfae549b459bff/impacts"
                }
            },
            {
                "id": "productid.1d4ca5394f6327f3b18afaeb8fe187b3",
                "name": "Yorkshire Semi Skimmed Milk 2 Pints/1.136 Litres",
                "gtin": "5010251879268",
                "image": "https://static.euw1.sustained.app/products/images/5010251879268_0.jpg",
                "pack": "1.136l",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1d4ca5394f6327f3b18afaeb8fe187b3",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1d4ca5394f6327f3b18afaeb8fe187b3/impacts"
                }
            },
            {
                "id": "productid.1dd513a48225e8b353ccb3af1ee2c343",
                "name": "Whole Milk",
                "gtin": "5099603111287",
                "image": "https://static.euw1.sustained.app/products/images/5099603111287_0.jpg",
                "pack": "1l",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1dd513a48225e8b353ccb3af1ee2c343",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1dd513a48225e8b353ccb3af1ee2c343/impacts"
                }
            },
            {
                "id": "productid.1f3b806ca6eb2dfb3b82cf61f2fae576",
                "name": "Creamery Fresh Milk",
                "gtin": "5098931000010",
                "image": "https://static.euw1.sustained.app/products/images/5098931000010_0.jpg",
                "pack": "1l",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1f3b806ca6eb2dfb3b82cf61f2fae576",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1f3b806ca6eb2dfb3b82cf61f2fae576/impacts"
                }
            },
            {
                "id": "productid.20bea6dd528edfb43a65ef481e13eba9",
                "name": "SO Organic Skimmed British Milk 2 Pints/1.136L",
                "gtin": "01219500",
                "image": "https://static.euw1.sustained.app/products/images/01219500_0.jpg",
                "pack": "1.136l",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655623
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.20bea6dd528edfb43a65ef481e13eba9",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.20bea6dd528edfb43a65ef481e13eba9/impacts"
                }
            },
            {
                "id": "productid.216c40a104cfc73cda8826ffe46adf70",
                "name": "Swords Whole Milk",
                "gtin": "5015496006134",
                "image": "https://static.euw1.sustained.app/products/images/5015496006134_0.jpg",
                "pack": "1l",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.216c40a104cfc73cda8826ffe46adf70",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.216c40a104cfc73cda8826ffe46adf70/impacts"
                }
            },
            {
                "id": "productid.219e8a1ab6c7c2fb103399e93b91df00",
                "name": "Pasteurised Skimmed Milk",
                "gtin": "5015496003003",
                "image": "https://static.euw1.sustained.app/products/images/5015496003003_0.jpg",
                "pack": "2l",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655623
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.219e8a1ab6c7c2fb103399e93b91df00",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.219e8a1ab6c7c2fb103399e93b91df00/impacts"
                }
            },
            {
                "id": "productid.21c95a791033ac0325ea30737ac6f307",
                "name": "Coconut Milk",
                "gtin": "5012389005014",
                "image": "https://static.euw1.sustained.app/products/images/5012389005014_0.jpg",
                "pack": "400ml",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.21c95a791033ac0325ea30737ac6f307",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.21c95a791033ac0325ea30737ac6f307/impacts"
                }
            },
            {
                "id": "productid.221cfc2ebc4696e9caffeb03273b81b5",
                "name": "Creamfields Semi-Skimmed UHT Milk 1.76 Pints/1 Litre",
                "gtin": "5057545828601",
                "image": "https://static.euw1.sustained.app/products/images/5057545828601_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.221cfc2ebc4696e9caffeb03273b81b5",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.221cfc2ebc4696e9caffeb03273b81b5/impacts"
                }
            },
            {
                "id": "productid.2278f1c883572f0d6cc9aeba6396ac05",
                "name": "Butter Milk",
                "gtin": "5011055602489",
                "image": "https://static.euw1.sustained.app/products/images/5011055602489_0.jpg",
                "pack": "1l",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2278f1c883572f0d6cc9aeba6396ac05",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2278f1c883572f0d6cc9aeba6396ac05/impacts"
                }
            },
            {
                "id": "productid.22ac556705fbe06a665f76c05fc7eb82",
                "name": "Food 4 Tropical Smoothies",
                "gtin": "0000000036757",
                "image": "https://static.euw1.sustained.app/products/images/0000000036757_0.jpg",
                "pack": "110g",
                "grade": "B",
                "points": {
                    "pef": 11.363272511198394
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.22ac556705fbe06a665f76c05fc7eb82",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.22ac556705fbe06a665f76c05fc7eb82/impacts"
                }
            },
            {
                "id": "productid.235a37d359b2baafa5f57cf470568939",
                "name": "Food Organic Coconut Cream",
                "gtin": "0000029074761",
                "image": "https://static.euw1.sustained.app/products/images/0000029074761_0.jpg",
                "pack": "160ml",
                "grade": "C",
                "points": {
                    "pef": 20.02867559853766
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.235a37d359b2baafa5f57cf470568939",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.235a37d359b2baafa5f57cf470568939/impacts"
                }
            },
            {
                "id": "productid.238c8325c378ada7e3178e3d64e8ac70",
                "name": "Barista Edition The Original Oat Drink",
                "gtin": "7394376619427",
                "image": "https://static.euw1.sustained.app/products/images/7394376619427_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.238c8325c378ada7e3178e3d64e8ac70",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.238c8325c378ada7e3178e3d64e8ac70/impacts"
                }
            },
            {
                "id": "productid.238c96e6a428b27df1bd8582dc29727b",
                "name": "Fresh Milk",
                "gtin": "5099817001497",
                "image": "https://static.euw1.sustained.app/products/images/5099817001497_0.jpg",
                "pack": "3l",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.238c96e6a428b27df1bd8582dc29727b",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.238c96e6a428b27df1bd8582dc29727b/impacts"
                }
            },
            {
                "id": "productid.253fb8902a063915f1f3df36e8217748",
                "name": "Vanilla Milk Drink",
                "gtin": "5410438043015",
                "image": "https://static.euw1.sustained.app/products/images/5410438043015_0.jpg",
                "pack": "400ml",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.253fb8902a063915f1f3df36e8217748",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.253fb8902a063915f1f3df36e8217748/impacts"
                }
            },
            {
                "id": "productid.266ec625b03f890418f5c68e7c07fa84",
                "name": "Fresh Milk",
                "gtin": "5011056020978",
                "image": "https://static.euw1.sustained.app/products/images/5011056020978_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.266ec625b03f890418f5c68e7c07fa84",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.266ec625b03f890418f5c68e7c07fa84/impacts"
                }
            },
            {
                "id": "productid.26910f5cdf7d958e1195a0e786da83f0",
                "name": "Coconut Milk",
                "gtin": "9556041131001",
                "image": "https://static.euw1.sustained.app/products/images/9556041131001_0.jpg",
                "pack": "200ml",
                "grade": "C",
                "points": {
                    "pef": 18.51670268243486
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.26910f5cdf7d958e1195a0e786da83f0",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.26910f5cdf7d958e1195a0e786da83f0/impacts"
                }
            },
            {
                "id": "productid.27031ec2aaa755aa81e84ec4834229f0",
                "name": "UHT Milk Junior",
                "gtin": "5900820012519",
                "image": "https://static.euw1.sustained.app/products/images/5900820012519_0.jpg",
                "pack": "1l",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.27031ec2aaa755aa81e84ec4834229f0",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.27031ec2aaa755aa81e84ec4834229f0/impacts"
                }
            },
            {
                "id": "productid.2804088f064e1b51b02f11b0c38a0912",
                "name": "DoubleShot Espresso No Added Sugar Iced Coffee Drink",
                "gtin": "5711953079481",
                "image": "https://static.euw1.sustained.app/products/images/5711953079481_0.jpg",
                "pack": "200ml",
                "grade": "E",
                "points": {
                    "pef": 46.614634547005664
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2804088f064e1b51b02f11b0c38a0912",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2804088f064e1b51b02f11b0c38a0912/impacts"
                }
            },
            {
                "id": "productid.28282769316f9ce776fbd0e8d1dfa0a2",
                "name": "Brownie Flavour Milk",
                "gtin": "5057753686284",
                "image": "https://static.euw1.sustained.app/products/images/5057753686284_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.28282769316f9ce776fbd0e8d1dfa0a2",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.28282769316f9ce776fbd0e8d1dfa0a2/impacts"
                }
            },
            {
                "id": "productid.28ce8035d24579fb8ffa8952ceed1293",
                "name": "Whole Milk",
                "gtin": "5099416000914",
                "image": "https://static.euw1.sustained.app/products/images/5099416000914_0.jpg",
                "pack": "2l",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.28ce8035d24579fb8ffa8952ceed1293",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.28ce8035d24579fb8ffa8952ceed1293/impacts"
                }
            },
            {
                "id": "productid.2a43574bdbe5cca02517e323261a9e0e",
                "name": "Milk Whole 2 Pints",
                "gtin": "5017600010204",
                "image": "https://static.euw1.sustained.app/products/images/5017600010204_0.jpg",
                "pack": "1.136l",
                "grade": "B",
                "points": {
                    "pef": 13.265801415655627
                },
                "info_icons": [],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a43574bdbe5cca02517e323261a9e0e",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a43574bdbe5cca02517e323261a9e0e/impacts"
                }
            },
            {
                "id": "productid.2a524784f38f12d8572c2493ea0410c3",
                "name": "Cold Brew Coffee with Coconut Milk",
                "gtin": "5060232811978",
                "image": "https://static.euw1.sustained.app/products/images/5060232811978_0.jpg",
                "pack": "1l",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Milk & Cream",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a524784f38f12d8572c2493ea0410c3",
                    "category": "https://api.sustained.com/choice/v1/categories/milk-and-cream",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a524784f38f12d8572c2493ea0410c3/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.00e759830163464ce0e04e672f174d24",
                "name": "The Best Piccolo Tomatoes",
                "gtin": "5010251752042",
                "image": "https://static.euw1.sustained.app/products/images/5010251752042_0.jpg",
                "pack": "400g",
                "grade": "A",
                "points": {
                    "pef": 7.515362193288447
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00e759830163464ce0e04e672f174d24",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00e759830163464ce0e04e672f174d24/impacts"
                }
            },
            {
                "id": "productid.056d6081682c135d6ca7af336231afda",
                "name": "Organic Kale",
                "gtin": "0000000187138",
                "image": "https://static.euw1.sustained.app/products/images/0000000187138_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 6.883204177819301
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.056d6081682c135d6ca7af336231afda",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.056d6081682c135d6ca7af336231afda/impacts"
                }
            },
            {
                "id": "productid.0577282e60dcd38c26412307994191e5",
                "name": "Arreton Jewel",
                "gtin": "5060192822236",
                "image": "https://static.euw1.sustained.app/products/images/5060192822236_0.jpg",
                "pack": "250g",
                "grade": "A",
                "points": {
                    "pef": 7.515362193288447
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0577282e60dcd38c26412307994191e5",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0577282e60dcd38c26412307994191e5/impacts"
                }
            },
            {
                "id": "productid.087f76646ec14db2265279e8a0427760",
                "name": "Organic Holland Red Onions",
                "gtin": "5010251771654",
                "image": "https://static.euw1.sustained.app/products/images/5010251771654_0.jpg",
                "pack": "500g",
                "grade": "A",
                "points": {
                    "pef": 5.632161628095959
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.087f76646ec14db2265279e8a0427760",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.087f76646ec14db2265279e8a0427760/impacts"
                }
            },
            {
                "id": "productid.097e2228c0ecc39a9a2bb9bec9fc704c",
                "name": "The Greengrocer's On Market Street Seedless Green Grapes",
                "gtin": "5010251748182",
                "image": "https://static.euw1.sustained.app/products/images/5010251748182_0.jpg",
                "pack": "500g",
                "grade": "A",
                "points": {
                    "pef": 8.137557667185739
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.097e2228c0ecc39a9a2bb9bec9fc704c",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.097e2228c0ecc39a9a2bb9bec9fc704c/impacts"
                }
            },
            {
                "id": "productid.0e247fc45382861c20a1e78a6b520eea",
                "name": "Sicilian Blood Oranges",
                "gtin": "5060294429418",
                "image": "https://static.euw1.sustained.app/products/images/5060294429418_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 11.4979989778862
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e247fc45382861c20a1e78a6b520eea",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e247fc45382861c20a1e78a6b520eea/impacts"
                }
            },
            {
                "id": "productid.10eaa87db8de23acb61600aa5daa32d1",
                "name": "Creamy & Nutty Avocados",
                "gtin": "0000023066755",
                "image": "https://static.euw1.sustained.app/products/images/0000023066755_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.454065438207856
                },
                "info_icons": [],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10eaa87db8de23acb61600aa5daa32d1",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10eaa87db8de23acb61600aa5daa32d1/impacts"
                }
            },
            {
                "id": "productid.11e6096545bfcd09c9c3cfa9e0df248e",
                "name": "Organic Spinach",
                "gtin": "5010251852841",
                "image": "https://static.euw1.sustained.app/products/images/5010251852841_0.jpg",
                "pack": "160g",
                "grade": "A",
                "points": {
                    "pef": 4.735835369432604
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11e6096545bfcd09c9c3cfa9e0df248e",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11e6096545bfcd09c9c3cfa9e0df248e/impacts"
                }
            },
            {
                "id": "productid.122814b99a8253986a14c6f065c5d086",
                "name": "Irish Closed Cup Mushrooms",
                "gtin": "03279397",
                "image": "https://static.euw1.sustained.app/products/images/03279397_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 4.424270487544751
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.122814b99a8253986a14c6f065c5d086",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.122814b99a8253986a14c6f065c5d086/impacts"
                }
            },
            {
                "id": "productid.1751d21011ea93d5ebaebdc855c05366",
                "name": "Organic Sweet Potatoes",
                "gtin": "10045725",
                "image": "https://static.euw1.sustained.app/products/images/10045725_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 6.6843254812885915
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1751d21011ea93d5ebaebdc855c05366",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1751d21011ea93d5ebaebdc855c05366/impacts"
                }
            },
            {
                "id": "productid.25f116a55fa0153635c55813d8179a5b",
                "name": "The Best Sweetelle Plum Tomatoes",
                "gtin": "5010251752066",
                "image": "https://static.euw1.sustained.app/products/images/5010251752066_0.jpg",
                "pack": "220g",
                "grade": "A",
                "points": {
                    "pef": 7.515362193288447
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.25f116a55fa0153635c55813d8179a5b",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.25f116a55fa0153635c55813d8179a5b/impacts"
                }
            },
            {
                "id": "productid.268e0b50bd0a4e3cf8abfdd74bc6cd57",
                "name": "Conference Pears",
                "gtin": "0000003250020",
                "image": "https://static.euw1.sustained.app/products/images/0000003250020_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 8.479902645832293
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.268e0b50bd0a4e3cf8abfdd74bc6cd57",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.268e0b50bd0a4e3cf8abfdd74bc6cd57/impacts"
                }
            },
            {
                "id": "productid.269d9442fc3eaffb8ff12c49abf66c68",
                "name": "Fresh White Mushrooms",
                "gtin": "0000000005852",
                "image": "https://static.euw1.sustained.app/products/images/0000000005852_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 4.424270487544751
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.269d9442fc3eaffb8ff12c49abf66c68",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.269d9442fc3eaffb8ff12c49abf66c68/impacts"
                }
            },
            {
                "id": "productid.2efa67d6519ec05a84e98baa337dab24",
                "name": "Organic Prunes",
                "gtin": "0000029088669",
                "image": "https://static.euw1.sustained.app/products/images/0000029088669_0.jpg",
                "pack": "250g",
                "grade": "G",
                "points": {
                    "pef": 136.55242901551372
                },
                "info_icons": [],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2efa67d6519ec05a84e98baa337dab24",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2efa67d6519ec05a84e98baa337dab24/impacts"
                }
            },
            {
                "id": "productid.31614c5c1384a5feb0e5255d93cd0520",
                "name": "Cook With M&S Food Leek & Potato Soup Mix",
                "gtin": "0000000909655",
                "image": "https://static.euw1.sustained.app/products/images/0000000909655_0.jpg",
                "pack": "600g",
                "grade": "A",
                "points": {
                    "pef": 7.213037125783733
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.31614c5c1384a5feb0e5255d93cd0520",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.31614c5c1384a5feb0e5255d93cd0520/impacts"
                }
            },
            {
                "id": "productid.31aff93695e7931e6659b5459351be2d",
                "name": "Organic Chestnut Mushrooms",
                "gtin": "10050033",
                "image": "https://static.euw1.sustained.app/products/images/10050033_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 4.424270487544751
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.31aff93695e7931e6659b5459351be2d",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.31aff93695e7931e6659b5459351be2d/impacts"
                }
            },
            {
                "id": "productid.3ac27ef9aeb4b2f102995da0e56a8070",
                "name": "Duchy Organic Maris Piper Potatoes",
                "gtin": "5000169601808",
                "image": "https://static.euw1.sustained.app/products/images/5000169601808_0.jpg",
                "pack": "1kg",
                "grade": "A",
                "points": {
                    "pef": 6.6843254812885915
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3ac27ef9aeb4b2f102995da0e56a8070",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3ac27ef9aeb4b2f102995da0e56a8070/impacts"
                }
            },
            {
                "id": "productid.43e29e82e47e00a8d2311ab0bb899d51",
                "name": "Redmere Farms Cooked Beetroot",
                "gtin": "03334294",
                "image": "https://static.euw1.sustained.app/products/images/03334294_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.321220567533148
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.43e29e82e47e00a8d2311ab0bb899d51",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.43e29e82e47e00a8d2311ab0bb899d51/impacts"
                }
            },
            {
                "id": "productid.474f05216aaf4ac3ae984952a504b181",
                "name": "Organic Carrots",
                "gtin": "5054781511946",
                "image": "https://static.euw1.sustained.app/products/images/5054781511946_0.jpg",
                "pack": "500g",
                "grade": "A",
                "points": {
                    "pef": 5.321220567533144
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.474f05216aaf4ac3ae984952a504b181",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.474f05216aaf4ac3ae984952a504b181/impacts"
                }
            },
            {
                "id": "productid.49a4db3b7084ac976c8b6afa6bd9b575",
                "name": "Organic Cabbage",
                "gtin": "0000023064096",
                "image": "https://static.euw1.sustained.app/products/images/0000023064096_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 6.628853728533776
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.49a4db3b7084ac976c8b6afa6bd9b575",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.49a4db3b7084ac976c8b6afa6bd9b575/impacts"
                }
            },
            {
                "id": "productid.4bc955f9874ec533659dcede03cc74ff",
                "name": "Organic Cherry on the Vine",
                "gtin": "5060192822366",
                "image": "https://static.euw1.sustained.app/products/images/5060192822366_0.jpg",
                "pack": "250g",
                "grade": "A",
                "points": {
                    "pef": 7.515362193288447
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4bc955f9874ec533659dcede03cc74ff",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4bc955f9874ec533659dcede03cc74ff/impacts"
                }
            },
            {
                "id": "productid.5083332e35d2ab34167b87da91538c6e",
                "name": "Arreton Sweet Vine Selection",
                "gtin": "5060192823554",
                "image": "https://static.euw1.sustained.app/products/images/5060192823554_0.jpg",
                "pack": "300g",
                "grade": "A",
                "points": {
                    "pef": 7.515362193288447
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5083332e35d2ab34167b87da91538c6e",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5083332e35d2ab34167b87da91538c6e/impacts"
                }
            },
            {
                "id": "productid.54927d08c705c5638f9eb8b6db118052",
                "name": "Organic Gala Apples",
                "gtin": "03256558",
                "image": "https://static.euw1.sustained.app/products/images/03256558_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.656019205950824
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.54927d08c705c5638f9eb8b6db118052",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.54927d08c705c5638f9eb8b6db118052/impacts"
                }
            },
            {
                "id": "productid.56b2d10013ad40ae18900b86811709b5",
                "name": "Duchy Organic Green Kiwi",
                "gtin": "5000169040034",
                "image": "https://static.euw1.sustained.app/products/images/5000169040034_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 9.715700409717142
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.56b2d10013ad40ae18900b86811709b5",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.56b2d10013ad40ae18900b86811709b5/impacts"
                }
            },
            {
                "id": "productid.5ab2d9d5e3d44083654ec48bc91c8612",
                "name": "Market St Mixed Peppers",
                "gtin": "5010251733805",
                "image": "https://static.euw1.sustained.app/products/images/5010251733805_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 9.832826505421094
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5ab2d9d5e3d44083654ec48bc91c8612",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5ab2d9d5e3d44083654ec48bc91c8612/impacts"
                }
            },
            {
                "id": "productid.5b77e57e50f73790edb689c5b25d41be",
                "name": "SO Organic Apples",
                "gtin": "01755084",
                "image": "https://static.euw1.sustained.app/products/images/01755084_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.656019205950824
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5b77e57e50f73790edb689c5b25d41be",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5b77e57e50f73790edb689c5b25d41be/impacts"
                }
            },
            {
                "id": "productid.5e421db1e145c92250f7621ee8f6c517",
                "name": "Organic Large Vine Tomatoes",
                "gtin": "5010251745976",
                "image": "https://static.euw1.sustained.app/products/images/5010251745976_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 7.515362193288447
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5e421db1e145c92250f7621ee8f6c517",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5e421db1e145c92250f7621ee8f6c517/impacts"
                }
            },
            {
                "id": "productid.5fa909ceb7d5cf1a35c081f23feb3a90",
                "name": "Extra Special Sweet & Spicy Infused Baby Beetroot",
                "gtin": "5057172549993",
                "image": "https://static.euw1.sustained.app/products/images/5057172549993_0.jpg",
                "pack": "175g",
                "grade": "A",
                "points": {
                    "pef": 5.46974199336841
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5fa909ceb7d5cf1a35c081f23feb3a90",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5fa909ceb7d5cf1a35c081f23feb3a90/impacts"
                }
            },
            {
                "id": "productid.5fc9b1004dada1f216b2c0fff6b1dda9",
                "name": "Food Organic Avocado",
                "gtin": "0000000167086",
                "image": "https://static.euw1.sustained.app/products/images/0000000167086_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.454065438207856
                },
                "info_icons": [],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5fc9b1004dada1f216b2c0fff6b1dda9",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5fc9b1004dada1f216b2c0fff6b1dda9/impacts"
                }
            },
            {
                "id": "productid.5fefd1fe6decbc22434803394c049bc6",
                "name": "Plum Tomatoes",
                "gtin": "0000000454520",
                "image": "https://static.euw1.sustained.app/products/images/0000000454520_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 7.515362193288447
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5fefd1fe6decbc22434803394c049bc6",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5fefd1fe6decbc22434803394c049bc6/impacts"
                }
            },
            {
                "id": "productid.6181a2301b66f66ce1be748e35713dbb",
                "name": "Organic Blueberries",
                "gtin": "10055212",
                "image": "https://static.euw1.sustained.app/products/images/10055212_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 21.253186202543986
                },
                "info_icons": [],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.6181a2301b66f66ce1be748e35713dbb",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.6181a2301b66f66ce1be748e35713dbb/impacts"
                }
            },
            {
                "id": "productid.62b363516d17cb5473144a4056d129bf",
                "name": "SO Organic Cos Lettuce",
                "gtin": "01639711",
                "image": "https://static.euw1.sustained.app/products/images/01639711_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 7.303232188917164
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.62b363516d17cb5473144a4056d129bf",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.62b363516d17cb5473144a4056d129bf/impacts"
                }
            },
            {
                "id": "productid.638ad1a805f4b1fa63cccabccd25f857",
                "name": "Organic Parsnips",
                "gtin": "5391537730061",
                "image": "https://static.euw1.sustained.app/products/images/5391537730061_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.321220567533148
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.638ad1a805f4b1fa63cccabccd25f857",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.638ad1a805f4b1fa63cccabccd25f857/impacts"
                }
            },
            {
                "id": "productid.648dbc1dde182befff08cd7a755a56e7",
                "name": "Green Celery",
                "gtin": "0278090000006",
                "image": "https://static.euw1.sustained.app/products/images/0278090000006_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 7.689862412941972
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.648dbc1dde182befff08cd7a755a56e7",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.648dbc1dde182befff08cd7a755a56e7/impacts"
                }
            },
            {
                "id": "productid.64901bdad35b1bafe9c0404d42351fed",
                "name": "Organic Ripe & Ready Pears",
                "gtin": "03256602",
                "image": "https://static.euw1.sustained.app/products/images/03256602_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 8.479902645832293
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.64901bdad35b1bafe9c0404d42351fed",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.64901bdad35b1bafe9c0404d42351fed/impacts"
                }
            },
            {
                "id": "productid.67725564e4df019bc24a448baed62c97",
                "name": "Organic 4 Seasonal Apples",
                "gtin": "0000003298572",
                "image": "https://static.euw1.sustained.app/products/images/0000003298572_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.656019205950824
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.67725564e4df019bc24a448baed62c97",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.67725564e4df019bc24a448baed62c97/impacts"
                }
            },
            {
                "id": "productid.6975401d5ddfd85b597d131ca817f91c",
                "name": "Duchy Organic Strawberries",
                "gtin": "5000169291580",
                "image": "https://static.euw1.sustained.app/products/images/5000169291580_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 41.016539401995416
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.6975401d5ddfd85b597d131ca817f91c",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.6975401d5ddfd85b597d131ca817f91c/impacts"
                }
            },
            {
                "id": "productid.6ab1a45edfeebf1b929a1081e2d56902",
                "name": "SO Organic Aubergines",
                "gtin": "0000000323154",
                "image": "https://static.euw1.sustained.app/products/images/0000000323154_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 48.82992516181264
                },
                "info_icons": [],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.6ab1a45edfeebf1b929a1081e2d56902",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.6ab1a45edfeebf1b929a1081e2d56902/impacts"
                }
            },
            {
                "id": "productid.6f7f98fde2a6331a8be0dfaef6eb78c7",
                "name": "Extra Special Petite Plum Vine Tomatoes",
                "gtin": "5057172012121",
                "image": "https://static.euw1.sustained.app/products/images/5057172012121_0.jpg",
                "pack": "180g",
                "grade": "A",
                "points": {
                    "pef": 7.515362193288447
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.6f7f98fde2a6331a8be0dfaef6eb78c7",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.6f7f98fde2a6331a8be0dfaef6eb78c7/impacts"
                }
            },
            {
                "id": "productid.6fdbed0c2c2085370af7b95ddc4a0bff",
                "name": "Duchy Organic Blueberries",
                "gtin": "5000169713402",
                "image": "https://static.euw1.sustained.app/products/images/5000169713402_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 21.253186202543986
                },
                "info_icons": [],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.6fdbed0c2c2085370af7b95ddc4a0bff",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.6fdbed0c2c2085370af7b95ddc4a0bff/impacts"
                }
            },
            {
                "id": "productid.742f80c4f321139425aa2b96083b8188",
                "name": "Organic Unwaxed Lemons",
                "gtin": "10045428",
                "image": "https://static.euw1.sustained.app/products/images/10045428_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 11.488669846592286
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.742f80c4f321139425aa2b96083b8188",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.742f80c4f321139425aa2b96083b8188/impacts"
                }
            },
            {
                "id": "productid.78c45b578a1d1373029d5d841ca26602",
                "name": "Organic Carrots",
                "gtin": "5391537730320",
                "image": "https://static.euw1.sustained.app/products/images/5391537730320_0.jpg",
                "pack": "700g",
                "grade": "A",
                "points": {
                    "pef": 5.321220567533144
                },
                "info_icons": [
                    {
                        "name": "Seasonal food",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/seasonal.svg",
                        "description": "In season now! Less storage, travel and pollution which is good for everyone."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.78c45b578a1d1373029d5d841ca26602",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.78c45b578a1d1373029d5d841ca26602/impacts"
                }
            },
            {
                "id": "productid.7fd1b9a04cd5b51bd80ceb655590ee73",
                "name": "Food Organic Spinach",
                "gtin": "0000000248549",
                "image": "https://static.euw1.sustained.app/products/images/0000000248549_0.jpg",
                "pack": "200g",
                "grade": "A",
                "points": {
                    "pef": 4.735835369432604
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.7fd1b9a04cd5b51bd80ceb655590ee73",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.7fd1b9a04cd5b51bd80ceb655590ee73/impacts"
                }
            },
            {
                "id": "productid.81a092aff5f72440412441c55f60ad82",
                "name": "Sweet & Fruity Baby Plum Tomatoes",
                "gtin": "5054070628485",
                "image": "https://static.euw1.sustained.app/products/images/5054070628485_0.jpg",
                "pack": "325g",
                "grade": "A",
                "points": {
                    "pef": 7.515362193288447
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.81a092aff5f72440412441c55f60ad82",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.81a092aff5f72440412441c55f60ad82/impacts"
                }
            },
            {
                "id": "productid.900110796e9d554807c21348fdfa1c3d",
                "name": "Large Avocado",
                "gtin": "5000169179697",
                "image": "https://static.euw1.sustained.app/products/images/5000169179697_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.454065438207856
                },
                "info_icons": [],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.900110796e9d554807c21348fdfa1c3d",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.900110796e9d554807c21348fdfa1c3d/impacts"
                }
            },
            {
                "id": "productid.913ad6fba6fd8875be69555ad17dd801",
                "name": "Organic Celery",
                "gtin": "0000003242636",
                "image": "https://static.euw1.sustained.app/products/images/0000003242636_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 7.689862412941972
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.913ad6fba6fd8875be69555ad17dd801",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.913ad6fba6fd8875be69555ad17dd801/impacts"
                }
            },
            {
                "id": "productid.91daffdb182c1a950663c643c3cf8034",
                "name": "Organic Parsnips",
                "gtin": "5052909528142",
                "image": "https://static.euw1.sustained.app/products/images/5052909528142_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.321220567533148
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.91daffdb182c1a950663c643c3cf8034",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.91daffdb182c1a950663c643c3cf8034/impacts"
                }
            },
            {
                "id": "productid.9379a0ff901732256e7ff76e72729fbf",
                "name": "Grower's Selection Organic Vine Ripened Cherry Tomatoes",
                "gtin": "5054781859857",
                "image": "https://static.euw1.sustained.app/products/images/5054781859857_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 7.515362193288447
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.9379a0ff901732256e7ff76e72729fbf",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.9379a0ff901732256e7ff76e72729fbf/impacts"
                }
            },
            {
                "id": "productid.9ae1da0f6450b882209cfb1da1db1504",
                "name": "Grower's Selection Organic Avocado",
                "gtin": "5054781593720",
                "image": "https://static.euw1.sustained.app/products/images/5054781593720_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.454065438207856
                },
                "info_icons": [],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.9ae1da0f6450b882209cfb1da1db1504",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.9ae1da0f6450b882209cfb1da1db1504/impacts"
                }
            },
            {
                "id": "productid.a0aea06f78f579356d5c5f022a32a739",
                "name": "Organic Bunched Beetroot",
                "gtin": "5016143006644",
                "image": "https://static.euw1.sustained.app/products/images/5016143006644_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 5.321220567533148
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Organic Produce",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.a0aea06f78f579356d5c5f022a32a739",
                    "category": "https://api.sustained.com/choice/v1/categories/organic-produce",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.a0aea06f78f579356d5c5f022a32a739/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.0048c6ebd00adfc5aea4e7c0c50cd44f",
                "name": "Sainsbury's Aperitivo",
                "gtin": "0000000356305",
                "image": "https://static.euw1.sustained.app/products/images/0000000356305_0.jpg",
                "pack": "70cl",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0048c6ebd00adfc5aea4e7c0c50cd44f",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0048c6ebd00adfc5aea4e7c0c50cd44f/impacts"
                }
            },
            {
                "id": "productid.0429f0fddd26572b9071019f7e61b782",
                "name": "Cherry B Cherry Wine",
                "gtin": "5010134918312",
                "image": "https://static.euw1.sustained.app/products/images/5010134918312_0.jpg",
                "pack": "200ml",
                "grade": "C",
                "points": {
                    "pef": 18.535946971015374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0429f0fddd26572b9071019f7e61b782",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0429f0fddd26572b9071019f7e61b782/impacts"
                }
            },
            {
                "id": "productid.0b7bc400c093cf3708479d5f214dc2bc",
                "name": "Piranha Cherry Schnapps",
                "gtin": "5391515830158",
                "image": "https://static.euw1.sustained.app/products/images/5391515830158_0.jpg",
                "pack": "50cl",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b7bc400c093cf3708479d5f214dc2bc",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b7bc400c093cf3708479d5f214dc2bc/impacts"
                }
            },
            {
                "id": "productid.0be2126525327d1600e4e03643aba298",
                "name": "Authentic Cocktail Co Forest Fruits Gin Bramble",
                "gtin": "5060193936932",
                "image": "https://static.euw1.sustained.app/products/images/5060193936932_0.jpg",
                "pack": "250ml",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0be2126525327d1600e4e03643aba298",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0be2126525327d1600e4e03643aba298/impacts"
                }
            },
            {
                "id": "productid.0ca7b45b4a6f53f0be37611575258c5a",
                "name": "Glassbox Spirits Vodka Soda Cranberry",
                "gtin": "5391528252206",
                "image": "https://static.euw1.sustained.app/products/images/5391528252206_0.jpg",
                "pack": "330ml",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ca7b45b4a6f53f0be37611575258c5a",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ca7b45b4a6f53f0be37611575258c5a/impacts"
                }
            },
            {
                "id": "productid.0ce73daf40ac4af28335d3236a1f346e",
                "name": "Don Simon Sangr\u00eda Organic",
                "gtin": "8410261151847",
                "image": "https://static.euw1.sustained.app/products/images/8410261151847_0.jpg",
                "pack": "1l",
                "grade": "C",
                "points": {
                    "pef": 21.74037697439984
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ce73daf40ac4af28335d3236a1f346e",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ce73daf40ac4af28335d3236a1f346e/impacts"
                }
            },
            {
                "id": "productid.0d649932d925a8dfb1a66c73d8d2cf2e",
                "name": "Mike's Hard Seltzer Lime",
                "gtin": "5014379010053",
                "image": "https://static.euw1.sustained.app/products/images/5014379010053_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d649932d925a8dfb1a66c73d8d2cf2e",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d649932d925a8dfb1a66c73d8d2cf2e/impacts"
                }
            },
            {
                "id": "productid.0ed6e8f930edfc9598d2326f5aca9dc8",
                "name": "Sincerely Sabella Blood Orange Aperitivo Italian Wine Spritz",
                "gtin": "5014379008821",
                "image": "https://static.euw1.sustained.app/products/images/5014379008821_0.jpg",
                "pack": "250ml",
                "grade": "C",
                "points": {
                    "pef": 19.291999921716172
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ed6e8f930edfc9598d2326f5aca9dc8",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ed6e8f930edfc9598d2326f5aca9dc8/impacts"
                }
            },
            {
                "id": "productid.12ad8a6a84bb3a246dd6c0242e52f8da",
                "name": "All Shook Up Strawberry Daiquiri Premium Cocktail",
                "gtin": "5032678009226",
                "image": "https://static.euw1.sustained.app/products/images/5032678009226_0.jpg",
                "pack": "250ml",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12ad8a6a84bb3a246dd6c0242e52f8da",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12ad8a6a84bb3a246dd6c0242e52f8da/impacts"
                }
            },
            {
                "id": "productid.135f48877ed67cd8f1cfb1bb7b2c0dff",
                "name": "Trooper Premium British Beer",
                "gtin": "5056499910110",
                "image": "https://static.euw1.sustained.app/products/images/5056499910110_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 12.11410917421006
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.135f48877ed67cd8f1cfb1bb7b2c0dff",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.135f48877ed67cd8f1cfb1bb7b2c0dff/impacts"
                }
            },
            {
                "id": "productid.139ae9cd14bc4cc89c57a1a9492c8d5f",
                "name": "MD 20/20 Electric Melon",
                "gtin": "0085904364403",
                "image": "https://static.euw1.sustained.app/products/images/0085904364403_0.jpg",
                "pack": "75cl",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.139ae9cd14bc4cc89c57a1a9492c8d5f",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.139ae9cd14bc4cc89c57a1a9492c8d5f/impacts"
                }
            },
            {
                "id": "productid.173aca69ed46cfe6b39d9314a6f03c4c",
                "name": "Seven Summits Lemon Hard Seltzer",
                "gtin": "5060133560920",
                "image": "https://static.euw1.sustained.app/products/images/5060133560920_0.jpg",
                "pack": "250ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.173aca69ed46cfe6b39d9314a6f03c4c",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.173aca69ed46cfe6b39d9314a6f03c4c/impacts"
                }
            },
            {
                "id": "productid.1a3064dc4013d094e2848fb563120298",
                "name": "G:N Long Drink",
                "gtin": "4740098090588",
                "image": "https://static.euw1.sustained.app/products/images/4740098090588_0.jpg",
                "pack": "0.33l",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1a3064dc4013d094e2848fb563120298",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1a3064dc4013d094e2848fb563120298/impacts"
                }
            },
            {
                "id": "productid.1a7743751937fdd8778b6d6bb6acf317",
                "name": "Sainsbury's Bourbon and Cola",
                "gtin": "01784572",
                "image": "https://static.euw1.sustained.app/products/images/01784572_0.jpg",
                "pack": "250ml",
                "grade": "A",
                "points": {
                    "pef": 9.435954568941742
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1a7743751937fdd8778b6d6bb6acf317",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1a7743751937fdd8778b6d6bb6acf317/impacts"
                }
            },
            {
                "id": "productid.1be999a9918ea73c3e2275a2299b7e09",
                "name": "Baileys Orange Hot Chocolate Set",
                "gtin": "5038635076508",
                "image": "https://static.euw1.sustained.app/products/images/5038635076508_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 14.551829669183038
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1be999a9918ea73c3e2275a2299b7e09",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1be999a9918ea73c3e2275a2299b7e09/impacts"
                }
            },
            {
                "id": "productid.1d5feb6508525cf443990e428997cc5e",
                "name": "Jura Journey Malt Whisky Craft Beer Gift Pack",
                "gtin": "5013967014657",
                "image": "https://static.euw1.sustained.app/products/images/5013967014657_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 4.310709028491551
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1d5feb6508525cf443990e428997cc5e",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1d5feb6508525cf443990e428997cc5e/impacts"
                }
            },
            {
                "id": "productid.1e6a139e281d465609ae0b12541fa8f9",
                "name": "Echo Falls Strawberry & Pink Pepper Ros\u00e9 Seltzer",
                "gtin": "5010134919043",
                "image": "https://static.euw1.sustained.app/products/images/5010134919043_0.jpg",
                "pack": "250ml",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e6a139e281d465609ae0b12541fa8f9",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e6a139e281d465609ae0b12541fa8f9/impacts"
                }
            },
            {
                "id": "productid.1e853ce6edac3d09690101b0fdf0bbae",
                "name": "Jawbox Pineapple & Ginger Gin & Ginger Ale",
                "gtin": "5060434132710",
                "image": "https://static.euw1.sustained.app/products/images/5060434132710_0.jpg",
                "pack": "250ml",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e853ce6edac3d09690101b0fdf0bbae",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e853ce6edac3d09690101b0fdf0bbae/impacts"
                }
            },
            {
                "id": "productid.1ece0a1aec66247ae0c6bb54e933dd0d",
                "name": "Kopparberg Hard Seltzer Mixed Berries",
                "gtin": "7393714646866",
                "image": "https://static.euw1.sustained.app/products/images/7393714646866_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1ece0a1aec66247ae0c6bb54e933dd0d",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1ece0a1aec66247ae0c6bb54e933dd0d/impacts"
                }
            },
            {
                "id": "productid.223312fe48a36185fd9624dbc66901bf",
                "name": "Cactus Jack's Schnapps Passion Fruit",
                "gtin": "5021932022408",
                "image": "https://static.euw1.sustained.app/products/images/5021932022408_0.jpg",
                "pack": "50cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753957
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.223312fe48a36185fd9624dbc66901bf",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.223312fe48a36185fd9624dbc66901bf/impacts"
                }
            },
            {
                "id": "productid.25de504acf1723cef587b5861f6dfc42",
                "name": "Three Mills Varietals Pinot Grigio Blush",
                "gtin": "5018970094382",
                "image": "https://static.euw1.sustained.app/products/images/5018970094382_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.535946971015374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.25de504acf1723cef587b5861f6dfc42",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.25de504acf1723cef587b5861f6dfc42/impacts"
                }
            },
            {
                "id": "productid.2644fa88007fc8f6d6e803eef4be5083",
                "name": "Echo Falls Raspberry & Cassis Fruit Fusion",
                "gtin": "5010134918589",
                "image": "https://static.euw1.sustained.app/products/images/5010134918589_0.jpg",
                "pack": "750ml",
                "grade": "C",
                "points": {
                    "pef": 18.535946971015374
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2644fa88007fc8f6d6e803eef4be5083",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2644fa88007fc8f6d6e803eef4be5083/impacts"
                }
            },
            {
                "id": "productid.27071e65a123efb5c68a48eb716a44ce",
                "name": "Tesco Low Alcohol G&T Refreshing Drink & Tonic",
                "gtin": "5057753896973",
                "image": "https://static.euw1.sustained.app/products/images/5057753896973_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.27071e65a123efb5c68a48eb716a44ce",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.27071e65a123efb5c68a48eb716a44ce/impacts"
                }
            },
            {
                "id": "productid.2e720795883bb63fce4d75e03436d3d8",
                "name": "Sperone Aperitivo Spritz",
                "gtin": "8008513008799",
                "image": "https://static.euw1.sustained.app/products/images/8008513008799_0.jpg",
                "pack": "70cl",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2e720795883bb63fce4d75e03436d3d8",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2e720795883bb63fce4d75e03436d3d8/impacts"
                }
            },
            {
                "id": "productid.2ed529677f97ea87d66b9a9d4d41ea10",
                "name": "Phillips of Bristol Lovage Old English Alcoholic Cordial",
                "gtin": "5010696813100",
                "image": "https://static.euw1.sustained.app/products/images/5010696813100_0.jpg",
                "pack": "70cl",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2ed529677f97ea87d66b9a9d4d41ea10",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2ed529677f97ea87d66b9a9d4d41ea10/impacts"
                }
            },
            {
                "id": "productid.32a01fd51e5798533a0a197efe8b4692",
                "name": "Ace+Freak Mint + Elderflower Spritz",
                "gtin": "0735850989846",
                "image": "https://static.euw1.sustained.app/products/images/0735850989846_0.jpg",
                "pack": "250ml",
                "grade": "C",
                "points": {
                    "pef": 19.291999921716172
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.32a01fd51e5798533a0a197efe8b4692",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.32a01fd51e5798533a0a197efe8b4692/impacts"
                }
            },
            {
                "id": "productid.39fafe420487bf983827a0225c19da48",
                "name": "Captain Morgan Original Spiced Gold and Cola",
                "gtin": "5000281045429",
                "image": "https://static.euw1.sustained.app/products/images/5000281045429_0.jpg",
                "pack": "250ml",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.39fafe420487bf983827a0225c19da48",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.39fafe420487bf983827a0225c19da48/impacts"
                }
            },
            {
                "id": "productid.3a19dbd0d9e9d3ccd928f88727f04f2b",
                "name": "Mike's Hard Seltzer Variety Pack",
                "gtin": "5014379010527",
                "image": "https://static.euw1.sustained.app/products/images/5014379010527_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3a19dbd0d9e9d3ccd928f88727f04f2b",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3a19dbd0d9e9d3ccd928f88727f04f2b/impacts"
                }
            },
            {
                "id": "productid.3c723880f0839df878e5ea57acddebe9",
                "name": "Asda Spiced Rum & Cola",
                "gtin": "5057172123018",
                "image": "https://static.euw1.sustained.app/products/images/5057172123018_0.jpg",
                "pack": "250ml",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3c723880f0839df878e5ea57acddebe9",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3c723880f0839df878e5ea57acddebe9/impacts"
                }
            },
            {
                "id": "productid.3c9459b9b4204a36e63044e1cc2a237f",
                "name": "Asda Mulled Wine",
                "gtin": "5010658318360",
                "image": "https://static.euw1.sustained.app/products/images/5010658318360_0.jpg",
                "pack": "75cl",
                "grade": "C",
                "points": {
                    "pef": 18.535946971015374
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3c9459b9b4204a36e63044e1cc2a237f",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3c9459b9b4204a36e63044e1cc2a237f/impacts"
                }
            },
            {
                "id": "productid.3e085800bf51b2ed99ce576e312e492b",
                "name": "Cactus Jack's Schnapps Dark Fruits",
                "gtin": "5021932022309",
                "image": "https://static.euw1.sustained.app/products/images/5021932022309_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753957
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3e085800bf51b2ed99ce576e312e492b",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3e085800bf51b2ed99ce576e312e492b/impacts"
                }
            },
            {
                "id": "productid.3f60c758383f930fc5cba592e73fc355",
                "name": "Kubota Junmai Daiginjo",
                "gtin": "04984283002097",
                "image": "https://static.euw1.sustained.app/products/images/04984283002097_0.jpg",
                "pack": "720ml",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753984
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3f60c758383f930fc5cba592e73fc355",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3f60c758383f930fc5cba592e73fc355/impacts"
                }
            },
            {
                "id": "productid.3f880eb5ba0812def2930c42653a427d",
                "name": "Rockshore Seltzer Raspberry",
                "gtin": "5000213025734",
                "image": "https://static.euw1.sustained.app/products/images/5000213025734_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3f880eb5ba0812def2930c42653a427d",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3f880eb5ba0812def2930c42653a427d/impacts"
                }
            },
            {
                "id": "productid.418b319ffe3619a4eda877cecf69ee90",
                "name": "Truly Wild Berry Hard Seltzer",
                "gtin": "0087692006770",
                "image": "https://static.euw1.sustained.app/products/images/0087692006770_0.jpg",
                "pack": "355ml",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.418b319ffe3619a4eda877cecf69ee90",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.418b319ffe3619a4eda877cecf69ee90/impacts"
                }
            },
            {
                "id": "productid.4398cbbe117f3e4bdeebfd0cdedfc95a",
                "name": "Gordon's Pink Berry Martini",
                "gtin": "5000289932882",
                "image": "https://static.euw1.sustained.app/products/images/5000289932882_0.jpg",
                "pack": "0.250l",
                "grade": "B",
                "points": {
                    "pef": 13.61455246629356
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4398cbbe117f3e4bdeebfd0cdedfc95a",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4398cbbe117f3e4bdeebfd0cdedfc95a/impacts"
                }
            },
            {
                "id": "productid.43b819b5245565a3dc45689697cb3973",
                "name": "Seven Summits Wild Berry Hard Seltzer",
                "gtin": "5060133561033",
                "image": "https://static.euw1.sustained.app/products/images/5060133561033_0.jpg",
                "pack": "250ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.43b819b5245565a3dc45689697cb3973",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.43b819b5245565a3dc45689697cb3973/impacts"
                }
            },
            {
                "id": "productid.462b6132757366c0cd2248225a4f590a",
                "name": "MD 20/20 Orange Jubilee",
                "gtin": "5010319203066",
                "image": "https://static.euw1.sustained.app/products/images/5010319203066_0.jpg",
                "pack": "75cl",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.462b6132757366c0cd2248225a4f590a",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.462b6132757366c0cd2248225a4f590a/impacts"
                }
            },
            {
                "id": "productid.49104dd81c44ffdace71231166e0a560",
                "name": "Bodega Bay Hard Seltzer Cherry Mango & Goji Berry",
                "gtin": "0745760440047",
                "image": "https://static.euw1.sustained.app/products/images/0745760440047_0.jpg",
                "pack": "250ml",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.49104dd81c44ffdace71231166e0a560",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.49104dd81c44ffdace71231166e0a560/impacts"
                }
            },
            {
                "id": "productid.4a7c33c454f5466a02bc7615f666841f",
                "name": "Wilde Child Brewing Co. Opaque Reality Mango and Passionfruit Milkshake IPA",
                "gtin": "5060472970091",
                "image": "https://static.euw1.sustained.app/products/images/5060472970091_0.jpg",
                "pack": "440ml",
                "grade": "A",
                "points": {
                    "pef": 4.310709028491551
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4a7c33c454f5466a02bc7615f666841f",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4a7c33c454f5466a02bc7615f666841f/impacts"
                }
            },
            {
                "id": "productid.4a993c0c31fe6ecc7fb81b97672e7f0c",
                "name": "Mionetto Prosecco Goody Bag",
                "gtin": "5060038024473",
                "image": "https://static.euw1.sustained.app/products/images/5060038024473_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 14.551829669183038
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4a993c0c31fe6ecc7fb81b97672e7f0c",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4a993c0c31fe6ecc7fb81b97672e7f0c/impacts"
                }
            },
            {
                "id": "productid.4cc9da7c9ad75ef2e215913320458d25",
                "name": "Gordon's Premium Pink Distilled Gin, Glass & Drinks Shimmer",
                "gtin": "5038635066639",
                "image": "https://static.euw1.sustained.app/products/images/5038635066639_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 14.551829669183038
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4cc9da7c9ad75ef2e215913320458d25",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4cc9da7c9ad75ef2e215913320458d25/impacts"
                }
            },
            {
                "id": "productid.4db497970a05d2d98dd44d7be356e8a0",
                "name": "Bols Grenadine Syrup",
                "gtin": "8716000959270",
                "image": "https://static.euw1.sustained.app/products/images/8716000959270_0.jpg",
                "pack": "750ml",
                "grade": "B",
                "points": {
                    "pef": 14.551829669183038
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4db497970a05d2d98dd44d7be356e8a0",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4db497970a05d2d98dd44d7be356e8a0/impacts"
                }
            },
            {
                "id": "productid.4e28aa9a81dd50a97433ca2fa9cf0711",
                "name": "Bottega Gold Prosecco DOC Spumante Brut and Godiva Milk Chocolates",
                "gtin": "5038635081328",
                "image": "https://static.euw1.sustained.app/products/images/5038635081328_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 14.551829669183038
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4e28aa9a81dd50a97433ca2fa9cf0711",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4e28aa9a81dd50a97433ca2fa9cf0711/impacts"
                }
            },
            {
                "id": "productid.4f3a70a9d66c456dc6188b1168462e12",
                "name": "Cocktail Chic Cranberry Martini Shimmer Handbag",
                "gtin": "5056499910387",
                "image": "https://static.euw1.sustained.app/products/images/5056499910387_0.jpg",
                "pack": "500ml",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753957
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4f3a70a9d66c456dc6188b1168462e12",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4f3a70a9d66c456dc6188b1168462e12/impacts"
                }
            },
            {
                "id": "productid.4fcf79c922e2df1cd5f15c8992fbbc8e",
                "name": "Gordon's Premium Pink Gin & Tonic",
                "gtin": "5000289935562",
                "image": "https://static.euw1.sustained.app/products/images/5000289935562_0.jpg",
                "pack": "0.250l",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4fcf79c922e2df1cd5f15c8992fbbc8e",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4fcf79c922e2df1cd5f15c8992fbbc8e/impacts"
                }
            },
            {
                "id": "productid.50216b512d67f1ebe118eca12ef2915e",
                "name": "M&S Original Green Ginger Wine",
                "gtin": "0000000891530",
                "image": "https://static.euw1.sustained.app/products/images/0000000891530_0.jpg",
                "pack": "70cl",
                "grade": "C",
                "points": {
                    "pef": 18.535946971015374
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.50216b512d67f1ebe118eca12ef2915e",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.50216b512d67f1ebe118eca12ef2915e/impacts"
                }
            },
            {
                "id": "productid.548fb3c523a1581cd88613012fe4409c",
                "name": "Trulo Coconut Rum Flavoured",
                "gtin": "5060641130219",
                "image": "https://static.euw1.sustained.app/products/images/5060641130219_0.jpg",
                "pack": "50cl",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.548fb3c523a1581cd88613012fe4409c",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.548fb3c523a1581cd88613012fe4409c/impacts"
                }
            },
            {
                "id": "productid.584badd35cd2deb9d2b8b71153738d8c",
                "name": "Kopparberg Hard Seltzer PassionFruit",
                "gtin": "7393714646200",
                "image": "https://static.euw1.sustained.app/products/images/7393714646200_0.jpg",
                "pack": "330ml",
                "grade": "B",
                "points": {
                    "pef": 13.508656700297166
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.584badd35cd2deb9d2b8b71153738d8c",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.584badd35cd2deb9d2b8b71153738d8c/impacts"
                }
            },
            {
                "id": "productid.589c0521aa0f7913a72c8fec03f2a923",
                "name": "Havana Club Cuban Rum, Tropical Fruit Flavours, Spices & Cola",
                "gtin": "8501110084192",
                "image": "https://static.euw1.sustained.app/products/images/8501110084192_0.jpg",
                "pack": "250ml",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.589c0521aa0f7913a72c8fec03f2a923",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.589c0521aa0f7913a72c8fec03f2a923/impacts"
                }
            },
            {
                "id": "productid.5acc90b41e7385ad7f3f83df4575c921",
                "name": "Tullamore D.E.W.",
                "gtin": "5011026108903",
                "image": "https://static.euw1.sustained.app/products/images/5011026108903_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Other Alcoholic Beverages",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5acc90b41e7385ad7f3f83df4575c921",
                    "category": "https://api.sustained.com/choice/v1/categories/other-alcoholic-beverages",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5acc90b41e7385ad7f3f83df4575c921/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.01b8f0294732462a16c667ce3cf52e31",
                "name": "Pork Exclusive Kabanos Sausages",
                "gtin": "5908230517099",
                "image": "https://static.euw1.sustained.app/products/images/5908230517099_0.jpg",
                "pack": "50g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01b8f0294732462a16c667ce3cf52e31",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01b8f0294732462a16c667ce3cf52e31/impacts"
                }
            },
            {
                "id": "productid.0281f8cbfb2c53eac2d11ba2d37cd0cc",
                "name": "Pigolatas Chipolatas 12 Chipolata Sausages",
                "gtin": "5028068000423",
                "image": "https://static.euw1.sustained.app/products/images/5028068000423_0.jpg",
                "pack": "340g",
                "grade": "G",
                "points": {
                    "pef": 107.7395793740598
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0281f8cbfb2c53eac2d11ba2d37cd0cc",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0281f8cbfb2c53eac2d11ba2d37cd0cc/impacts"
                }
            },
            {
                "id": "productid.02a266f1ed8b50a3a6fb4fd17a4bb121",
                "name": "British Corned Beef 9 Slices",
                "gtin": "5010525137063",
                "image": "https://static.euw1.sustained.app/products/images/5010525137063_0.jpg",
                "pack": "225g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02a266f1ed8b50a3a6fb4fd17a4bb121",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02a266f1ed8b50a3a6fb4fd17a4bb121/impacts"
                }
            },
            {
                "id": "productid.03a91ac1f7f9e4397de81a974b627f79",
                "name": "The Best British Pork, Chestnut & Bacon Stuffing",
                "gtin": "5010251929413",
                "image": "https://static.euw1.sustained.app/products/images/5010251929413_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 97.95206010196269
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03a91ac1f7f9e4397de81a974b627f79",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03a91ac1f7f9e4397de81a974b627f79/impacts"
                }
            },
            {
                "id": "productid.03c42a7a427ccc551a60bf390e222adb",
                "name": "Hamburger Patties",
                "gtin": "5900751012701",
                "image": "https://static.euw1.sustained.app/products/images/5900751012701_0.jpg",
                "pack": "250g",
                "grade": "F",
                "points": {
                    "pef": 54.41929581065375
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03c42a7a427ccc551a60bf390e222adb",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03c42a7a427ccc551a60bf390e222adb/impacts"
                }
            },
            {
                "id": "productid.058970631de5389481f63cc1265c2244",
                "name": "6 Gluten Free Sweet Chilli Pork Sausages",
                "gtin": "5060084348059",
                "image": "https://static.euw1.sustained.app/products/images/5060084348059_0.jpg",
                "pack": "375g",
                "grade": "G",
                "points": {
                    "pef": 109.25518836159709
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.058970631de5389481f63cc1265c2244",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.058970631de5389481f63cc1265c2244/impacts"
                }
            },
            {
                "id": "productid.065f2bb633f0e99e50d9bbb74e633fe0",
                "name": "Finest Italian Antipasto",
                "gtin": "5057545008591",
                "image": "https://static.euw1.sustained.app/products/images/5057545008591_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 191.38927381473994
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.065f2bb633f0e99e50d9bbb74e633fe0",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.065f2bb633f0e99e50d9bbb74e633fe0/impacts"
                }
            },
            {
                "id": "productid.07d358007804a9287cea7541159804e0",
                "name": "Fire Pit 8 Jumbo Taco Pork Sausages",
                "gtin": "5059316486317",
                "image": "https://static.euw1.sustained.app/products/images/5059316486317_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 83.55303379841666
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07d358007804a9287cea7541159804e0",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07d358007804a9287cea7541159804e0/impacts"
                }
            },
            {
                "id": "productid.0b2ed9f6862a890ffd60013a1c194a05",
                "name": "Beyond Meatballs Plant-Based Meatballs",
                "gtin": "1230000068536",
                "image": "https://static.euw1.sustained.app/products/images/1230000068536_0.jpg",
                "pack": "25g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b2ed9f6862a890ffd60013a1c194a05",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b2ed9f6862a890ffd60013a1c194a05/impacts"
                }
            },
            {
                "id": "productid.0daff7588fada50a93c754003931d22d",
                "name": "Selection Trio Salami 20 Slices",
                "gtin": "5099474510004",
                "image": "https://static.euw1.sustained.app/products/images/5099474510004_0.jpg",
                "pack": "100g",
                "grade": "F",
                "points": {
                    "pef": 83.55303379841666
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0daff7588fada50a93c754003931d22d",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0daff7588fada50a93c754003931d22d/impacts"
                }
            },
            {
                "id": "productid.0dcdd80df3723a9549aeb46fb1118cd2",
                "name": "Plant-Based Burger",
                "gtin": "5391512911379",
                "image": "https://static.euw1.sustained.app/products/images/5391512911379_0.jpg",
                "pack": "230g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0dcdd80df3723a9549aeb46fb1118cd2",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0dcdd80df3723a9549aeb46fb1118cd2/impacts"
                }
            },
            {
                "id": "productid.0ed0d2f9bbf6038ce6b79b598948f6d1",
                "name": "2 Peri Peri Chicken Burgers",
                "gtin": "5060059530762",
                "image": "https://static.euw1.sustained.app/products/images/5060059530762_0.jpg",
                "pack": "250g",
                "grade": "F",
                "points": {
                    "pef": 56.116880127575136
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ed0d2f9bbf6038ce6b79b598948f6d1",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ed0d2f9bbf6038ce6b79b598948f6d1/impacts"
                }
            },
            {
                "id": "productid.0ed3b31a4d5bf20d7df4cf862e2009ff",
                "name": "The Best 6 Thick British Pork Sausages",
                "gtin": "5010251715726",
                "image": "https://static.euw1.sustained.app/products/images/5010251715726_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 125.2143783265617
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ed3b31a4d5bf20d7df4cf862e2009ff",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ed3b31a4d5bf20d7df4cf862e2009ff/impacts"
                }
            },
            {
                "id": "productid.107cb8b7c45819021cb88c34a158dff6",
                "name": "Lorne Sausage",
                "gtin": "5032992917078",
                "image": "https://static.euw1.sustained.app/products/images/5032992917078_0.jpg",
                "pack": "300g",
                "grade": "G",
                "points": {
                    "pef": 163.28905693178558
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.107cb8b7c45819021cb88c34a158dff6",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.107cb8b7c45819021cb88c34a158dff6/impacts"
                }
            },
            {
                "id": "productid.116fb8da5365ef22c77575d0ed1aa84d",
                "name": "The Best 2 British Beef 6oz Chuck Steak Burgers",
                "gtin": "5010251785552",
                "image": "https://static.euw1.sustained.app/products/images/5010251785552_0.jpg",
                "pack": "0.340kg",
                "grade": "G",
                "points": {
                    "pef": 250.6559530361387
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.116fb8da5365ef22c77575d0ed1aa84d",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.116fb8da5365ef22c77575d0ed1aa84d/impacts"
                }
            },
            {
                "id": "productid.11b494161d89876dafc3da0ffa201d4a",
                "name": "Vegetarian Sausage Meat",
                "gtin": "5391514640895",
                "image": "https://static.euw1.sustained.app/products/images/5391514640895_0.jpg",
                "pack": "454g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.11b494161d89876dafc3da0ffa201d4a",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.11b494161d89876dafc3da0ffa201d4a/impacts"
                }
            },
            {
                "id": "productid.1256bb6ff914205e015ae39270b80c7f",
                "name": "24 Aberdeen Angus Beef Meatballs",
                "gtin": "5000169223604",
                "image": "https://static.euw1.sustained.app/products/images/5000169223604_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 263.32198604669793
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1256bb6ff914205e015ae39270b80c7f",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1256bb6ff914205e015ae39270b80c7f/impacts"
                }
            },
            {
                "id": "productid.126b81b1bc137face0facc3815cce218",
                "name": "Chorizo Ring Mild",
                "gtin": "8411034239304",
                "image": "https://static.euw1.sustained.app/products/images/8411034239304_0.jpg",
                "pack": "200g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.126b81b1bc137face0facc3815cce218",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.126b81b1bc137face0facc3815cce218/impacts"
                }
            },
            {
                "id": "productid.1307643b1955de10c7790d5551ebdf80",
                "name": "Salame Milano",
                "gtin": "8002469691043",
                "image": "https://static.euw1.sustained.app/products/images/8002469691043_0.jpg",
                "pack": "70g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1307643b1955de10c7790d5551ebdf80",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1307643b1955de10c7790d5551ebdf80/impacts"
                }
            },
            {
                "id": "productid.134b3de2262b394ecea05f6ca1e9a916",
                "name": "Taste the Difference 12 Cumberland Sausages",
                "gtin": "0000000236843",
                "image": "https://static.euw1.sustained.app/products/images/0000000236843_0.jpg",
                "pack": "795g",
                "grade": "G",
                "points": {
                    "pef": 130.42715899532882
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.134b3de2262b394ecea05f6ca1e9a916",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.134b3de2262b394ecea05f6ca1e9a916/impacts"
                }
            },
            {
                "id": "productid.149d80c16ab3cb933b64500124b69c90",
                "name": "Salami Pepperoni Piccante",
                "gtin": "5010292965982",
                "image": "https://static.euw1.sustained.app/products/images/5010292965982_0.jpg",
                "pack": "180g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.149d80c16ab3cb933b64500124b69c90",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.149d80c16ab3cb933b64500124b69c90/impacts"
                }
            },
            {
                "id": "productid.1568a789bc2836094d6cc3ed65ea70d3",
                "name": "Go Mediterranean Longaniza",
                "gtin": "8436030673310",
                "image": "https://static.euw1.sustained.app/products/images/8436030673310_0.jpg",
                "pack": "40g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1568a789bc2836094d6cc3ed65ea70d3",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1568a789bc2836094d6cc3ed65ea70d3/impacts"
                }
            },
            {
                "id": "productid.1582e3d119815235702d4af9060c9af6",
                "name": "Pork Kabanos Exclusive",
                "gtin": "5908230520747",
                "image": "https://static.euw1.sustained.app/products/images/5908230520747_0.jpg",
                "pack": "50g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1582e3d119815235702d4af9060c9af6",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1582e3d119815235702d4af9060c9af6/impacts"
                }
            },
            {
                "id": "productid.165bb35eadcd1a0b3c9fde58f0399610",
                "name": "Food Italian Cured Meat Selection 22 Slices",
                "gtin": "0000000778558",
                "image": "https://static.euw1.sustained.app/products/images/0000000778558_0.jpg",
                "pack": "205g",
                "grade": "G",
                "points": {
                    "pef": 191.38927381473994
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.165bb35eadcd1a0b3c9fde58f0399610",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.165bb35eadcd1a0b3c9fde58f0399610/impacts"
                }
            },
            {
                "id": "productid.168a2d02d9219233447a736689481777",
                "name": "Deli On Market Street Continental Meat Platter",
                "gtin": "0231117004005",
                "image": "https://static.euw1.sustained.app/products/images/0231117004005_0.jpg",
                "pack": "240g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.168a2d02d9219233447a736689481777",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.168a2d02d9219233447a736689481777/impacts"
                }
            },
            {
                "id": "productid.16f5f1cd1aba0bbcc0f43ec1b2324c99",
                "name": "6 Prime Pork Sausages",
                "gtin": "5057753224424",
                "image": "https://static.euw1.sustained.app/products/images/5057753224424_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 83.55303379841666
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16f5f1cd1aba0bbcc0f43ec1b2324c99",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16f5f1cd1aba0bbcc0f43ec1b2324c99/impacts"
                }
            },
            {
                "id": "productid.1876fb9148836912905de1cd39b63800",
                "name": "Extra Special 10 Chicken Chipolata Sausages",
                "gtin": "5054781711841",
                "image": "https://static.euw1.sustained.app/products/images/5054781711841_0.jpg",
                "pack": "340g",
                "grade": "F",
                "points": {
                    "pef": 74.38788443062806
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1876fb9148836912905de1cd39b63800",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1876fb9148836912905de1cd39b63800/impacts"
                }
            },
            {
                "id": "productid.19bcb528bf1726946609f67445379e82",
                "name": "Meat & Veg 12 Beef Meatballs",
                "gtin": "5057753906665",
                "image": "https://static.euw1.sustained.app/products/images/5057753906665_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 170.9551962332579
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.19bcb528bf1726946609f67445379e82",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.19bcb528bf1726946609f67445379e82/impacts"
                }
            },
            {
                "id": "productid.19cf36e03f77eedaf021bb7e4a8a744d",
                "name": "Summer Edition 20 Mini BBQ Pork Sausage Rolls",
                "gtin": "00460040",
                "image": "https://static.euw1.sustained.app/products/images/00460040_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.19cf36e03f77eedaf021bb7e4a8a744d",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.19cf36e03f77eedaf021bb7e4a8a744d/impacts"
                }
            },
            {
                "id": "productid.1c28b1173fb9388a2a620dabc67fb783",
                "name": "British Chicken and Chorizo Burgers",
                "gtin": "5000169473498",
                "image": "https://static.euw1.sustained.app/products/images/5000169473498_0.jpg",
                "pack": "454g",
                "grade": "F",
                "points": {
                    "pef": 60.57335110659294
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c28b1173fb9388a2a620dabc67fb783",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c28b1173fb9388a2a620dabc67fb783/impacts"
                }
            },
            {
                "id": "productid.1c6b5de9d1969c440a1fde3166c3fb52",
                "name": "Soko\u0142\u00f3w Salami",
                "gtin": "5902310014539",
                "image": "https://static.euw1.sustained.app/products/images/5902310014539_0.jpg",
                "pack": "100g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c6b5de9d1969c440a1fde3166c3fb52",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c6b5de9d1969c440a1fde3166c3fb52/impacts"
                }
            },
            {
                "id": "productid.1cbefc6a20ed6757f2d88b0aba8f80e9",
                "name": "Sweet Chilli Chicken Sausages",
                "gtin": "5057753914424",
                "image": "https://static.euw1.sustained.app/products/images/5057753914424_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 83.55303379841666
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1cbefc6a20ed6757f2d88b0aba8f80e9",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1cbefc6a20ed6757f2d88b0aba8f80e9/impacts"
                }
            },
            {
                "id": "productid.1d888c1038b4dd22e7bcd365e3c1bb9b",
                "name": "Full-Flavoured 12 Beef Meatballs",
                "gtin": "5054781688075",
                "image": "https://static.euw1.sustained.app/products/images/5054781688075_0.jpg",
                "pack": "340g",
                "grade": "G",
                "points": {
                    "pef": 240.435113536299
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1d888c1038b4dd22e7bcd365e3c1bb9b",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1d888c1038b4dd22e7bcd365e3c1bb9b/impacts"
                }
            },
            {
                "id": "productid.1f826082b43a9f049b7992d53f3549d3",
                "name": "Traditional Irish Pork Skinless Sausages",
                "gtin": "5099632000200",
                "image": "https://static.euw1.sustained.app/products/images/5099632000200_0.jpg",
                "pack": "227g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1f826082b43a9f049b7992d53f3549d3",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1f826082b43a9f049b7992d53f3549d3/impacts"
                }
            },
            {
                "id": "productid.20571c4d205df44afaaf6a4773234e70",
                "name": "30 Hot & Spicy Pork Cocktail Sausages",
                "gtin": "5057172122424",
                "image": "https://static.euw1.sustained.app/products/images/5057172122424_0.jpg",
                "pack": "300g",
                "grade": "G",
                "points": {
                    "pef": 104.51800633797691
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.20571c4d205df44afaaf6a4773234e70",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.20571c4d205df44afaaf6a4773234e70/impacts"
                }
            },
            {
                "id": "productid.21683a892d25bbaffc6034c1a9b59d6c",
                "name": "Mini Salamis",
                "gtin": "5059512735141",
                "image": "https://static.euw1.sustained.app/products/images/5059512735141_0.jpg",
                "pack": "85g",
                "grade": "G",
                "points": {
                    "pef": 141.97639807491635
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.21683a892d25bbaffc6034c1a9b59d6c",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.21683a892d25bbaffc6034c1a9b59d6c/impacts"
                }
            },
            {
                "id": "productid.24a154eeae41472a14aa4fb89417df95",
                "name": "8 Cumberland Pork Sausages",
                "gtin": "5060044510236",
                "image": "https://static.euw1.sustained.app/products/images/5060044510236_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 107.69555493908398
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.24a154eeae41472a14aa4fb89417df95",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.24a154eeae41472a14aa4fb89417df95/impacts"
                }
            },
            {
                "id": "productid.2515ab2c33402aca60a6855436e02f3c",
                "name": "Ox Tongue 4 Slices",
                "gtin": "0244320000000",
                "image": "https://static.euw1.sustained.app/products/images/0244320000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 142.75897835906363
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2515ab2c33402aca60a6855436e02f3c",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2515ab2c33402aca60a6855436e02f3c/impacts"
                }
            },
            {
                "id": "productid.25a1332669d48285bdf3a15aa93bdc44",
                "name": "18 Lincolnshire Pork Sausages",
                "gtin": "5037918201187",
                "image": "https://static.euw1.sustained.app/products/images/5037918201187_0.jpg",
                "pack": "1.2kg",
                "grade": "G",
                "points": {
                    "pef": 117.6187723062655
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.25a1332669d48285bdf3a15aa93bdc44",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.25a1332669d48285bdf3a15aa93bdc44/impacts"
                }
            },
            {
                "id": "productid.28ddd0a7adb8a320304b362dca1c6dfa",
                "name": "Fire Pit 6 Beef Burgers",
                "gtin": "5054775859344",
                "image": "https://static.euw1.sustained.app/products/images/5054775859344_0.jpg",
                "pack": "730g",
                "grade": "G",
                "points": {
                    "pef": 241.36634427208367
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.28ddd0a7adb8a320304b362dca1c6dfa",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.28ddd0a7adb8a320304b362dca1c6dfa/impacts"
                }
            },
            {
                "id": "productid.2a092ce5f1b469939854cda058a9b942",
                "name": "The Best Corned Beef",
                "gtin": "5010251889694",
                "image": "https://static.euw1.sustained.app/products/images/5010251889694_0.jpg",
                "pack": "130g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a092ce5f1b469939854cda058a9b942",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a092ce5f1b469939854cda058a9b942/impacts"
                }
            },
            {
                "id": "productid.2a34ba18730eeb0e528518b262d08419",
                "name": "4 Traditional Black Pudding Slices",
                "gtin": "5060051271069",
                "image": "https://static.euw1.sustained.app/products/images/5060051271069_0.jpg",
                "pack": "230g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a34ba18730eeb0e528518b262d08419",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a34ba18730eeb0e528518b262d08419/impacts"
                }
            },
            {
                "id": "productid.2a54c80bd21bf433d13a60ac0e761ba6",
                "name": "Welsh Beef Steak Burgers",
                "gtin": "5060059532377",
                "image": "https://static.euw1.sustained.app/products/images/5060059532377_0.jpg",
                "pack": "320g",
                "grade": "G",
                "points": {
                    "pef": 237.28232635333399
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a54c80bd21bf433d13a60ac0e761ba6",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a54c80bd21bf433d13a60ac0e761ba6/impacts"
                }
            },
            {
                "id": "productid.2ac1427fd79c28e43e705d434fb92761",
                "name": "The Best 12 Pork, Sage & Onion Stuffing Balls",
                "gtin": "5010251929390",
                "image": "https://static.euw1.sustained.app/products/images/5010251929390_0.jpg",
                "pack": "360g",
                "grade": "F",
                "points": {
                    "pef": 79.88426389354447
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2ac1427fd79c28e43e705d434fb92761",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2ac1427fd79c28e43e705d434fb92761/impacts"
                }
            },
            {
                "id": "productid.2ade09f34c6aa55efb1a9a8efdfa26e0",
                "name": "Food Select Farms British Outdoor Bred Pork Butchers Sausagemeat",
                "gtin": "0000029254149",
                "image": "https://static.euw1.sustained.app/products/images/0000029254149_0.jpg",
                "pack": "454g",
                "grade": "G",
                "points": {
                    "pef": 115.18566471169206
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2ade09f34c6aa55efb1a9a8efdfa26e0",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2ade09f34c6aa55efb1a9a8efdfa26e0/impacts"
                }
            },
            {
                "id": "productid.2afc7bb02e3932ce450573bffa198151",
                "name": "Black Pudding",
                "gtin": "5020341800164",
                "image": "https://static.euw1.sustained.app/products/images/5020341800164_0.jpg",
                "pack": "454g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2afc7bb02e3932ce450573bffa198151",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2afc7bb02e3932ce450573bffa198151/impacts"
                }
            },
            {
                "id": "productid.2b8b3e4f48f064d4dbf1e3f8b239dca2",
                "name": "Global Grill 4 Tikka Style Chicken Thigh Fillet Kebabs",
                "gtin": "5010525100470",
                "image": "https://static.euw1.sustained.app/products/images/5010525100470_0.jpg",
                "pack": "350g",
                "grade": "F",
                "points": {
                    "pef": 63.48759063923715
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2b8b3e4f48f064d4dbf1e3f8b239dca2",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2b8b3e4f48f064d4dbf1e3f8b239dca2/impacts"
                }
            },
            {
                "id": "productid.2bad067e77f34492f4a4ad5199e3cc6c",
                "name": "18 Chicken Italia",
                "gtin": "5060317353898",
                "image": "https://static.euw1.sustained.app/products/images/5060317353898_0.jpg",
                "pack": "612g",
                "grade": "F",
                "points": {
                    "pef": 60.13381829621582
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2bad067e77f34492f4a4ad5199e3cc6c",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2bad067e77f34492f4a4ad5199e3cc6c/impacts"
                }
            },
            {
                "id": "productid.2c754433bbd0b46a8886cd611d5c2dca",
                "name": "Extra Special 10 Pigs in Blankets",
                "gtin": "5057172487776",
                "image": "https://static.euw1.sustained.app/products/images/5057172487776_0.jpg",
                "pack": "210g",
                "grade": "G",
                "points": {
                    "pef": 131.2271836706521
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2c754433bbd0b46a8886cd611d5c2dca",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2c754433bbd0b46a8886cd611d5c2dca/impacts"
                }
            },
            {
                "id": "productid.2d6df5de536b00b0dea03b4c547cd4ed",
                "name": "Chicken with Tikka",
                "gtin": "5034525012076",
                "image": "https://static.euw1.sustained.app/products/images/5034525012076_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 43.31592943892597
                },
                "info_icons": [],
                "category": "Other Fresh Meat & Fish",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2d6df5de536b00b0dea03b4c547cd4ed",
                    "category": "https://api.sustained.com/choice/v1/categories/other-fresh-meat-and-fish",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2d6df5de536b00b0dea03b4c547cd4ed/impacts"
                }
            }
        ],
        [],
        [
            {
                "id": "productid.019caba3af77ae1f5d7eb3955185e019",
                "name": "British Smoked Bacon Lardons",
                "gtin": "5057753225971",
                "image": "https://static.euw1.sustained.app/products/images/5057753225971_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.019caba3af77ae1f5d7eb3955185e019",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.019caba3af77ae1f5d7eb3955185e019/impacts"
                }
            },
            {
                "id": "productid.060c14aefa4c99831a5ef474a36fc6c9",
                "name": "Unsmoked Back Bacon Joint",
                "gtin": "5099839663642",
                "image": "https://static.euw1.sustained.app/products/images/5099839663642_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 132.2242617838962
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.060c14aefa4c99831a5ef474a36fc6c9",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.060c14aefa4c99831a5ef474a36fc6c9/impacts"
                }
            },
            {
                "id": "productid.067688cd12a72482098671b83fa95e68",
                "name": "British Large Pork Shoulder Joint",
                "gtin": "0253840000000",
                "image": "https://static.euw1.sustained.app/products/images/0253840000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.067688cd12a72482098671b83fa95e68",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.067688cd12a72482098671b83fa95e68/impacts"
                }
            },
            {
                "id": "productid.068ca0616ea60d405c53faee04d5665e",
                "name": "Smoked Back Bacon Joint",
                "gtin": "5099839663659",
                "image": "https://static.euw1.sustained.app/products/images/5099839663659_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 132.2242617838962
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.068ca0616ea60d405c53faee04d5665e",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.068ca0616ea60d405c53faee04d5665e/impacts"
                }
            },
            {
                "id": "productid.0702368611635c0651d682ba77c9f732",
                "name": "British Pork Shoulder Joint",
                "gtin": "0218630000000",
                "image": "https://static.euw1.sustained.app/products/images/0218630000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0702368611635c0651d682ba77c9f732",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0702368611635c0651d682ba77c9f732/impacts"
                }
            },
            {
                "id": "productid.070ed6d7ffad269b79af791225528fb1",
                "name": "Boneless British Pork Loin Crackling Joint",
                "gtin": "0284960000007",
                "image": "https://static.euw1.sustained.app/products/images/0284960000007_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.070ed6d7ffad269b79af791225528fb1",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.070ed6d7ffad269b79af791225528fb1/impacts"
                }
            },
            {
                "id": "productid.0b72e1192bb2d31d7780a403cf6d2818",
                "name": "Simply Roast Crackling Gammon Joint",
                "gtin": "5057172255252",
                "image": "https://static.euw1.sustained.app/products/images/5057172255252_0.jpg",
                "pack": "1.2kg",
                "grade": "G",
                "points": {
                    "pef": 129.3427502094445
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0b72e1192bb2d31d7780a403cf6d2818",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0b72e1192bb2d31d7780a403cf6d2818/impacts"
                }
            },
            {
                "id": "productid.0ee1ee7a8325ddd69492855973fbf30f",
                "name": "Our Family Butcher Pork Boneless Loin Large",
                "gtin": "2327864000000",
                "image": "https://static.euw1.sustained.app/products/images/2327864000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ee1ee7a8325ddd69492855973fbf30f",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ee1ee7a8325ddd69492855973fbf30f/impacts"
                }
            },
            {
                "id": "productid.0f3ab205d2cfa57875b3bc42b1835d29",
                "name": "Pork Loin Roast",
                "gtin": "5060551219097",
                "image": "https://static.euw1.sustained.app/products/images/5060551219097_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f3ab205d2cfa57875b3bc42b1835d29",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f3ab205d2cfa57875b3bc42b1835d29/impacts"
                }
            },
            {
                "id": "productid.0fa5741b6129b6ad9c47f13c8333a7fb",
                "name": "No1 Free Range Unsmoked Bacon Lardons",
                "gtin": "5000169097151",
                "image": "https://static.euw1.sustained.app/products/images/5000169097151_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 82.06194886728561
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fa5741b6129b6ad9c47f13c8333a7fb",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fa5741b6129b6ad9c47f13c8333a7fb/impacts"
                }
            },
            {
                "id": "productid.135888dce3077404924aca1ca5eedaa5",
                "name": "Unsmoked Gammon Joint Boneless",
                "gtin": "0249402005828",
                "image": "https://static.euw1.sustained.app/products/images/0249402005828_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.135888dce3077404924aca1ca5eedaa5",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.135888dce3077404924aca1ca5eedaa5/impacts"
                }
            },
            {
                "id": "productid.148613339ab2211617d6d756042fc15e",
                "name": "British Pork Loin Steaks",
                "gtin": "5057753226053",
                "image": "https://static.euw1.sustained.app/products/images/5057753226053_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.148613339ab2211617d6d756042fc15e",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.148613339ab2211617d6d756042fc15e/impacts"
                }
            },
            {
                "id": "productid.14e0c8e9b2e4d4d3871fc33a3b5784fe",
                "name": "Unsmoked 2 Gammon Steaks",
                "gtin": "0000000119962",
                "image": "https://static.euw1.sustained.app/products/images/0000000119962_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 129.44718283434185
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14e0c8e9b2e4d4d3871fc33a3b5784fe",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14e0c8e9b2e4d4d3871fc33a3b5784fe/impacts"
                }
            },
            {
                "id": "productid.1849284b9c95f154c53088434a6d8ece",
                "name": "The Butcher's On Market Street British Pork Fillet Medallion",
                "gtin": "0276110000005",
                "image": "https://static.euw1.sustained.app/products/images/0276110000005_0.jpg",
                "pack": "0.420kg",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1849284b9c95f154c53088434a6d8ece",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1849284b9c95f154c53088434a6d8ece/impacts"
                }
            },
            {
                "id": "productid.193658dee7f36e6fc4ac567b35dca26d",
                "name": "Irish Basted Pork Chops",
                "gtin": "5052910567628",
                "image": "https://static.euw1.sustained.app/products/images/5052910567628_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.193658dee7f36e6fc4ac567b35dca26d",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.193658dee7f36e6fc4ac567b35dca26d/impacts"
                }
            },
            {
                "id": "productid.1974d4670cf2d1bc4c4dbfb567af85e4",
                "name": "Smoked Gammon Joint Boneless",
                "gtin": "0220330000000",
                "image": "https://static.euw1.sustained.app/products/images/0220330000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1974d4670cf2d1bc4c4dbfb567af85e4",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1974d4670cf2d1bc4c4dbfb567af85e4/impacts"
                }
            },
            {
                "id": "productid.19ce7d70b35647a2c4d4193761b03c6d",
                "name": "Butcher's On Market Street British Pork Loin Steak",
                "gtin": "0290171004505",
                "image": "https://static.euw1.sustained.app/products/images/0290171004505_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.19ce7d70b35647a2c4d4193761b03c6d",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.19ce7d70b35647a2c4d4193761b03c6d/impacts"
                }
            },
            {
                "id": "productid.1dc7ca918d3cfa93fcc2579e1307e30c",
                "name": "Fire Pit Mexican Style Pork Shoulder Steaks",
                "gtin": "5057753905354",
                "image": "https://static.euw1.sustained.app/products/images/5057753905354_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1dc7ca918d3cfa93fcc2579e1307e30c",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1dc7ca918d3cfa93fcc2579e1307e30c/impacts"
                }
            },
            {
                "id": "productid.1efa16af74421700b2453709d10f90a7",
                "name": "Finest Irish Porchetta",
                "gtin": "0270120000000",
                "image": "https://static.euw1.sustained.app/products/images/0270120000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1efa16af74421700b2453709d10f90a7",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1efa16af74421700b2453709d10f90a7/impacts"
                }
            },
            {
                "id": "productid.1f35a7050a905ca9e8e2c6016dfa5fac",
                "name": "British Pork Shoulder Joint",
                "gtin": "0218630000004",
                "image": "https://static.euw1.sustained.app/products/images/0218630000004_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1f35a7050a905ca9e8e2c6016dfa5fac",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1f35a7050a905ca9e8e2c6016dfa5fac/impacts"
                }
            },
            {
                "id": "productid.21bda63e26877ca39562719cd83ec4f0",
                "name": "Extra Special Festive Orange Topside Gammon Joint",
                "gtin": "5054070266472",
                "image": "https://static.euw1.sustained.app/products/images/5054070266472_0.jpg",
                "pack": "1.5g",
                "grade": "G",
                "points": {
                    "pef": 133.8021443221304
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.21bda63e26877ca39562719cd83ec4f0",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.21bda63e26877ca39562719cd83ec4f0/impacts"
                }
            },
            {
                "id": "productid.259cfd7bb5898553e709d2acf371206d",
                "name": "Meadowfields Unsmoked Back Joint with Added Water",
                "gtin": "5391505654443",
                "image": "https://static.euw1.sustained.app/products/images/5391505654443_0.jpg",
                "pack": "1kg",
                "grade": "G",
                "points": {
                    "pef": 131.5512534412327
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.259cfd7bb5898553e709d2acf371206d",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.259cfd7bb5898553e709d2acf371206d/impacts"
                }
            },
            {
                "id": "productid.270c7d1bdcdf9e9acc8f45da790793ea",
                "name": "Smoked Diced Pancetta",
                "gtin": "4010352512508",
                "image": "https://static.euw1.sustained.app/products/images/4010352512508_0.jpg",
                "pack": "80g",
                "grade": "G",
                "points": {
                    "pef": 140.10386380058418
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.270c7d1bdcdf9e9acc8f45da790793ea",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.270c7d1bdcdf9e9acc8f45da790793ea/impacts"
                }
            },
            {
                "id": "productid.278feca53bc07b9e13a45d277ec30e6b",
                "name": "British Pork Crackling Leg Joint",
                "gtin": "0220990000000",
                "image": "https://static.euw1.sustained.app/products/images/0220990000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.278feca53bc07b9e13a45d277ec30e6b",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.278feca53bc07b9e13a45d277ec30e6b/impacts"
                }
            },
            {
                "id": "productid.2e6331662f4795b5f30799547f152e8a",
                "name": "Maple Cure Back Bacon",
                "gtin": "5017326133263",
                "image": "https://static.euw1.sustained.app/products/images/5017326133263_0.jpg",
                "pack": "150g",
                "grade": "G",
                "points": {
                    "pef": 129.03550788381133
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2e6331662f4795b5f30799547f152e8a",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2e6331662f4795b5f30799547f152e8a/impacts"
                }
            },
            {
                "id": "productid.2ee8e3b30fdb4275c27be3220cda0ef3",
                "name": "Maple BBQ Pork Loin Steaks",
                "gtin": "5054775466276",
                "image": "https://static.euw1.sustained.app/products/images/5054775466276_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2ee8e3b30fdb4275c27be3220cda0ef3",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2ee8e3b30fdb4275c27be3220cda0ef3/impacts"
                }
            },
            {
                "id": "productid.31fb2527f56b5068b5eb74049923266f",
                "name": "Diced Pork",
                "gtin": "5052910452856",
                "image": "https://static.euw1.sustained.app/products/images/5052910452856_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.31fb2527f56b5068b5eb74049923266f",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.31fb2527f56b5068b5eb74049923266f/impacts"
                }
            },
            {
                "id": "productid.3286b3979335834de6006c36d3ccb960",
                "name": "Oriental Mini Spare Ribs",
                "gtin": "5000169327562",
                "image": "https://static.euw1.sustained.app/products/images/5000169327562_0.jpg",
                "pack": "290g",
                "grade": "B",
                "points": {
                    "pef": 16.08159577232546
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3286b3979335834de6006c36d3ccb960",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3286b3979335834de6006c36d3ccb960/impacts"
                }
            },
            {
                "id": "productid.33b2088f5aecd04d8cd19c99191f42ac",
                "name": "Woodside Farms Pork Chops",
                "gtin": "5054775750511",
                "image": "https://static.euw1.sustained.app/products/images/5054775750511_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.33b2088f5aecd04d8cd19c99191f42ac",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.33b2088f5aecd04d8cd19c99191f42ac/impacts"
                }
            },
            {
                "id": "productid.3517f4dcfc4ff662ab8ef6845bb77837",
                "name": "The Butcher's On Market Street British Pork Loin Steak",
                "gtin": "0237060000002",
                "image": "https://static.euw1.sustained.app/products/images/0237060000002_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3517f4dcfc4ff662ab8ef6845bb77837",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3517f4dcfc4ff662ab8ef6845bb77837/impacts"
                }
            },
            {
                "id": "productid.3b125b845689d74c62cde20950eb9fa5",
                "name": "Woodland Free Range Hampshire Easy Carve Leg",
                "gtin": "0268730000008",
                "image": "https://static.euw1.sustained.app/products/images/0268730000008_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3b125b845689d74c62cde20950eb9fa5",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3b125b845689d74c62cde20950eb9fa5/impacts"
                }
            },
            {
                "id": "productid.3d3f97330350b43adce1f488306aea84",
                "name": "Butcher's Selection Unsmoked Boneless Gammon Joint",
                "gtin": "0257350000000",
                "image": "https://static.euw1.sustained.app/products/images/0257350000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 130.44284217458087
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3d3f97330350b43adce1f488306aea84",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3d3f97330350b43adce1f488306aea84/impacts"
                }
            },
            {
                "id": "productid.3e3abbb7d40fc17f28c93a6290bd3a09",
                "name": "British Pork Fillet",
                "gtin": "0233120000000",
                "image": "https://static.euw1.sustained.app/products/images/0233120000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3e3abbb7d40fc17f28c93a6290bd3a09",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3e3abbb7d40fc17f28c93a6290bd3a09/impacts"
                }
            },
            {
                "id": "productid.3f332d24178e0215987fc76d4b2d2bd4",
                "name": "Pork Fillet",
                "gtin": "5060551216560",
                "image": "https://static.euw1.sustained.app/products/images/5060551216560_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3f332d24178e0215987fc76d4b2d2bd4",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3f332d24178e0215987fc76d4b2d2bd4/impacts"
                }
            },
            {
                "id": "productid.3fa4fc5ec0126bb36ca5572e712ce00c",
                "name": "Unsmoked Gammon Joint Boneless",
                "gtin": "0246640000000",
                "image": "https://static.euw1.sustained.app/products/images/0246640000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3fa4fc5ec0126bb36ca5572e712ce00c",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3fa4fc5ec0126bb36ca5572e712ce00c/impacts"
                }
            },
            {
                "id": "productid.41d475835092a7b99d685060882a4911",
                "name": "Delicious Unsmoked Gammon Joint",
                "gtin": "5057172487868",
                "image": "https://static.euw1.sustained.app/products/images/5057172487868_0.jpg",
                "pack": "1.3kg",
                "grade": "G",
                "points": {
                    "pef": 127.93710230090875
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.41d475835092a7b99d685060882a4911",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.41d475835092a7b99d685060882a4911/impacts"
                }
            },
            {
                "id": "productid.4b8998ae67ab48bc3a45ccff32293c77",
                "name": "Irish Stuffed Pork Loin Joint with Garlic and Herb Stuffing",
                "gtin": "5057753721794",
                "image": "https://static.euw1.sustained.app/products/images/5057753721794_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4b8998ae67ab48bc3a45ccff32293c77",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4b8998ae67ab48bc3a45ccff32293c77/impacts"
                }
            },
            {
                "id": "productid.4e40c2dd451affa47857a9cdc9b32867",
                "name": "Irish Unsmoked Ham Fillet",
                "gtin": "5057967879403",
                "image": "https://static.euw1.sustained.app/products/images/5057967879403_0.jpg",
                "pack": "1.8kg",
                "grade": "G",
                "points": {
                    "pef": 130.59533350112147
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4e40c2dd451affa47857a9cdc9b32867",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4e40c2dd451affa47857a9cdc9b32867/impacts"
                }
            },
            {
                "id": "productid.4f585783dd3557bc0f80829d93cc2296",
                "name": "Fire Pit Salt & Chilli Pork Belly Slices",
                "gtin": "5057373266538",
                "image": "https://static.euw1.sustained.app/products/images/5057373266538_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.4f585783dd3557bc0f80829d93cc2296",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.4f585783dd3557bc0f80829d93cc2296/impacts"
                }
            },
            {
                "id": "productid.53b45171efa209975dac231e8fdf58c5",
                "name": "Asian Style Irish Pork Fillet",
                "gtin": "5059697776649",
                "image": "https://static.euw1.sustained.app/products/images/5059697776649_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.53b45171efa209975dac231e8fdf58c5",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.53b45171efa209975dac231e8fdf58c5/impacts"
                }
            },
            {
                "id": "productid.58af6e27a921e11a817dabeb9a27a354",
                "name": "Butcher's On Market Street British Lamb Smoked Garlic & Mint Chops",
                "gtin": "5010525130392",
                "image": "https://static.euw1.sustained.app/products/images/5010525130392_0.jpg",
                "pack": "0.375kg",
                "grade": "G",
                "points": {
                    "pef": 461.2611335869978
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.58af6e27a921e11a817dabeb9a27a354",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.58af6e27a921e11a817dabeb9a27a354/impacts"
                }
            },
            {
                "id": "productid.58fe7b244be1226086200ea67c477d35",
                "name": "BBQ Pork Belly",
                "gtin": "5057753935009",
                "image": "https://static.euw1.sustained.app/products/images/5057753935009_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.58fe7b244be1226086200ea67c477d35",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.58fe7b244be1226086200ea67c477d35/impacts"
                }
            },
            {
                "id": "productid.5fbfb2245b31cf3193b4aa4ec70d8ce7",
                "name": "The Best Dry Cured 2 Hampshire Breed Unsmoked Gammon Steaks",
                "gtin": "5010251906513",
                "image": "https://static.euw1.sustained.app/products/images/5010251906513_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 140.67344728062534
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.5fbfb2245b31cf3193b4aa4ec70d8ce7",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.5fbfb2245b31cf3193b4aa4ec70d8ce7/impacts"
                }
            },
            {
                "id": "productid.620e8f29efbc542f2a8c9feef6d027b4",
                "name": "Northern Irish Fresh 20% Fat Pork Mince",
                "gtin": "00328012",
                "image": "https://static.euw1.sustained.app/products/images/00328012_0.jpg",
                "pack": "500g",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.620e8f29efbc542f2a8c9feef6d027b4",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.620e8f29efbc542f2a8c9feef6d027b4/impacts"
                }
            },
            {
                "id": "productid.625d0c5072d4d18fe0961866e59b9cd2",
                "name": "Memphis Inspired BBQ Pork Shoulder Steaks",
                "gtin": "5057753918606",
                "image": "https://static.euw1.sustained.app/products/images/5057753918606_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.625d0c5072d4d18fe0961866e59b9cd2",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.625d0c5072d4d18fe0961866e59b9cd2/impacts"
                }
            },
            {
                "id": "productid.63f29c3ebc91b090d1f185a21f8efe18",
                "name": "Pepper Crusted Pork Chops with Garlic Butter",
                "gtin": "5057753896249",
                "image": "https://static.euw1.sustained.app/products/images/5057753896249_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.63f29c3ebc91b090d1f185a21f8efe18",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.63f29c3ebc91b090d1f185a21f8efe18/impacts"
                }
            },
            {
                "id": "productid.6459b5c27dc651884a6aad834d5eb0e6",
                "name": "Our Family Butcher British Pork Tenderloin",
                "gtin": "2330108000000",
                "image": "https://static.euw1.sustained.app/products/images/2330108000000_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.6459b5c27dc651884a6aad834d5eb0e6",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.6459b5c27dc651884a6aad834d5eb0e6/impacts"
                }
            },
            {
                "id": "productid.645d5092bf0769e47ee23dfb4fd02297",
                "name": "British Fresh Large Pork Crackling Leg Joint",
                "gtin": "0220120000005",
                "image": "https://static.euw1.sustained.app/products/images/0220120000005_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.645d5092bf0769e47ee23dfb4fd02297",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.645d5092bf0769e47ee23dfb4fd02297/impacts"
                }
            },
            {
                "id": "productid.65d1c2562ffd0b089a8b011bc6b46505",
                "name": "Unsmoked Gammon Joint",
                "gtin": "5010251716297",
                "image": "https://static.euw1.sustained.app/products/images/5010251716297_0.jpg",
                "pack": "750g",
                "grade": "G",
                "points": {
                    "pef": 129.37513166377127
                },
                "info_icons": [],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.65d1c2562ffd0b089a8b011bc6b46505",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.65d1c2562ffd0b089a8b011bc6b46505/impacts"
                }
            },
            {
                "id": "productid.67343c66509239588fbeab042aa58fb2",
                "name": "Pork Fillet",
                "gtin": "5011182004552",
                "image": "https://static.euw1.sustained.app/products/images/5011182004552_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 145.92989179349095
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Pork",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.67343c66509239588fbeab042aa58fb2",
                    "category": "https://api.sustained.com/choice/v1/categories/pork",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.67343c66509239588fbeab042aa58fb2/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.004aa2595539079aa4dea83152d3ccb6",
                "name": "The Best Roast Garlic Pizzabread",
                "gtin": "5010525017549",
                "image": "https://static.euw1.sustained.app/products/images/5010525017549_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.004aa2595539079aa4dea83152d3ccb6",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.004aa2595539079aa4dea83152d3ccb6/impacts"
                }
            },
            {
                "id": "productid.00586138754364518b3ffec78950af70",
                "name": "2 Bramley Apple & Cream Turnovers",
                "gtin": "5000169616970",
                "image": "https://static.euw1.sustained.app/products/images/5000169616970_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00586138754364518b3ffec78950af70",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00586138754364518b3ffec78950af70/impacts"
                }
            },
            {
                "id": "productid.0067e4f794ad02c19d2b5b2b44af8331",
                "name": "Fish & Chips",
                "gtin": "5010525022987",
                "image": "https://static.euw1.sustained.app/products/images/5010525022987_0.jpg",
                "pack": "400g",
                "grade": "E",
                "points": {
                    "pef": 50.39840558254548
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0067e4f794ad02c19d2b5b2b44af8331",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0067e4f794ad02c19d2b5b2b44af8331/impacts"
                }
            },
            {
                "id": "productid.0086132403e052e064b20624c9a7c589",
                "name": "Spanish Inspired 4 Chicken & Chorizo Empanadas",
                "gtin": "5057172845316",
                "image": "https://static.euw1.sustained.app/products/images/5057172845316_0.jpg",
                "pack": "108g",
                "grade": "D",
                "points": {
                    "pef": 30.73593466622645
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0086132403e052e064b20624c9a7c589",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0086132403e052e064b20624c9a7c589/impacts"
                }
            },
            {
                "id": "productid.009082f274e3e52677101783de070c6d",
                "name": "Indian Collection Chicken Tikka Masala with Pilau Rice",
                "gtin": "5010893792918",
                "image": "https://static.euw1.sustained.app/products/images/5010893792918_0.jpg",
                "pack": "450g",
                "grade": "E",
                "points": {
                    "pef": 46.840818002203626
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.009082f274e3e52677101783de070c6d",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.009082f274e3e52677101783de070c6d/impacts"
                }
            },
            {
                "id": "productid.00aa2e1386e2701d9a1dd4c1096560a0",
                "name": "Kerala Aubergine Curry",
                "gtin": "5391510682752",
                "image": "https://static.euw1.sustained.app/products/images/5391510682752_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00aa2e1386e2701d9a1dd4c1096560a0",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00aa2e1386e2701d9a1dd4c1096560a0/impacts"
                }
            },
            {
                "id": "productid.00c4c38e6791012fd8ef69414ce8748d",
                "name": "Mashed Potato",
                "gtin": "5052910938930",
                "image": "https://static.euw1.sustained.app/products/images/5052910938930_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 6.6843254812885915
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00c4c38e6791012fd8ef69414ce8748d",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00c4c38e6791012fd8ef69414ce8748d/impacts"
                }
            },
            {
                "id": "productid.00d8d1371be7061a24b1825e31e66241",
                "name": "Cheese & Bacon Quiche",
                "gtin": "5027951008409",
                "image": "https://static.euw1.sustained.app/products/images/5027951008409_0.jpg",
                "pack": "400g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00d8d1371be7061a24b1825e31e66241",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00d8d1371be7061a24b1825e31e66241/impacts"
                }
            },
            {
                "id": "productid.00f2d16666935db0c7ea140d6957ae62",
                "name": "Essential Waitrose & Partners Black Salad Olives in Sunflower Oil",
                "gtin": "5000169526262",
                "image": "https://static.euw1.sustained.app/products/images/5000169526262_0.jpg",
                "pack": "130g",
                "grade": "C",
                "points": {
                    "pef": 18.928046558701034
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00f2d16666935db0c7ea140d6957ae62",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00f2d16666935db0c7ea140d6957ae62/impacts"
                }
            },
            {
                "id": "productid.00f300e221e10a5bda4e5a195b96fcf2",
                "name": "Curried Chickpea Salad",
                "gtin": "5057172567751",
                "image": "https://static.euw1.sustained.app/products/images/5057172567751_0.jpg",
                "pack": "230g",
                "grade": "B",
                "points": {
                    "pef": 16.160784743348465
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00f300e221e10a5bda4e5a195b96fcf2",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00f300e221e10a5bda4e5a195b96fcf2/impacts"
                }
            },
            {
                "id": "productid.01023c518cd4e02e4fd14810671f5f3e",
                "name": "Sweet & Sour Chicken Balls",
                "gtin": "5051873003747",
                "image": "https://static.euw1.sustained.app/products/images/5051873003747_0.jpg",
                "pack": "",
                "grade": "F",
                "points": {
                    "pef": 65.33806408908453
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01023c518cd4e02e4fd14810671f5f3e",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01023c518cd4e02e4fd14810671f5f3e/impacts"
                }
            },
            {
                "id": "productid.0126a1a17f805dab80cb577a34fdf16c",
                "name": "Food Count On Us Smoky Pulled Pork Bowl",
                "gtin": "0000000264174",
                "image": "https://static.euw1.sustained.app/products/images/0000000264174_0.jpg",
                "pack": "300g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0126a1a17f805dab80cb577a34fdf16c",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0126a1a17f805dab80cb577a34fdf16c/impacts"
                }
            },
            {
                "id": "productid.013d79f98536852521b056ec4c6e419e",
                "name": "The Best Mediterranean Style King Prawns",
                "gtin": "5010525125510",
                "image": "https://static.euw1.sustained.app/products/images/5010525125510_0.jpg",
                "pack": "120g",
                "grade": "D",
                "points": {
                    "pef": 28.68183726564067
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.013d79f98536852521b056ec4c6e419e",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.013d79f98536852521b056ec4c6e419e/impacts"
                }
            },
            {
                "id": "productid.0168a52b41505e63b34b0754d33ec499",
                "name": "Taste the Difference Italian Inspired Al Taglio Salami Pizza",
                "gtin": "00442183",
                "image": "https://static.euw1.sustained.app/products/images/00442183_0.jpg",
                "pack": "607g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0168a52b41505e63b34b0754d33ec499",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0168a52b41505e63b34b0754d33ec499/impacts"
                }
            },
            {
                "id": "productid.019bdfd62f05adae6b6ba7e243dd84c2",
                "name": "Pizza Plait",
                "gtin": "0000003286197",
                "image": "https://static.euw1.sustained.app/products/images/0000003286197_0.jpg",
                "pack": "",
                "grade": "A",
                "points": {
                    "pef": 9.091181326934837
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.019bdfd62f05adae6b6ba7e243dd84c2",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.019bdfd62f05adae6b6ba7e243dd84c2/impacts"
                }
            },
            {
                "id": "productid.01a6fc9628b961a719a35bde5d19a105",
                "name": "Classic Kitchen Five Bean Chilli & Potato Bake",
                "gtin": "5057753904180",
                "image": "https://static.euw1.sustained.app/products/images/5057753904180_0.jpg",
                "pack": "800g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01a6fc9628b961a719a35bde5d19a105",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01a6fc9628b961a719a35bde5d19a105/impacts"
                }
            },
            {
                "id": "productid.01b3fe8f29084d248c0ba06cce0e8506",
                "name": "The Best 2 Haddock Fishcakes with Applewood Smoked Cheddar Melting Centre",
                "gtin": "5010251871316",
                "image": "https://static.euw1.sustained.app/products/images/5010251871316_0.jpg",
                "pack": "290g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01b3fe8f29084d248c0ba06cce0e8506",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01b3fe8f29084d248c0ba06cce0e8506/impacts"
                }
            },
            {
                "id": "productid.01c5fae4f2b7d1439dad001f45d51e01",
                "name": "Sun Drenched Tomatoes, Mozzarella & Pesto",
                "gtin": "5000169579732",
                "image": "https://static.euw1.sustained.app/products/images/5000169579732_0.jpg",
                "pack": "220g",
                "grade": "E",
                "points": {
                    "pef": 32.1287768029289
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01c5fae4f2b7d1439dad001f45d51e01",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01c5fae4f2b7d1439dad001f45d51e01/impacts"
                }
            },
            {
                "id": "productid.01c7f27ffcc134457ec44dc51a5ebcff",
                "name": "An Authentic Beef Lasagne",
                "gtin": "5391511420292",
                "image": "https://static.euw1.sustained.app/products/images/5391511420292_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01c7f27ffcc134457ec44dc51a5ebcff",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01c7f27ffcc134457ec44dc51a5ebcff/impacts"
                }
            },
            {
                "id": "productid.01c98bd3c0e30136e3c529fa130b2c2e",
                "name": "Minced Steak & Onion Pie",
                "gtin": "5032181030182",
                "image": "https://static.euw1.sustained.app/products/images/5032181030182_0.jpg",
                "pack": "140g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01c98bd3c0e30136e3c529fa130b2c2e",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01c98bd3c0e30136e3c529fa130b2c2e/impacts"
                }
            },
            {
                "id": "productid.01d36dc7d59c8b3345c05bacdb9589c4",
                "name": "No1 Chateaubriand with Truffle & Porcini Butter",
                "gtin": "5000169591864",
                "image": "https://static.euw1.sustained.app/products/images/5000169591864_0.jpg",
                "pack": "682g",
                "grade": "G",
                "points": {
                    "pef": 266.53427166770285
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01d36dc7d59c8b3345c05bacdb9589c4",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01d36dc7d59c8b3345c05bacdb9589c4/impacts"
                }
            },
            {
                "id": "productid.01d4a35a4110138bcda97776afd27989",
                "name": "Food Plant Kitchen Blueberry & Coconut Pot",
                "gtin": "0000000136044",
                "image": "https://static.euw1.sustained.app/products/images/0000000136044_0.jpg",
                "pack": "120g",
                "grade": "A",
                "points": {
                    "pef": 9.927897934637521
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01d4a35a4110138bcda97776afd27989",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01d4a35a4110138bcda97776afd27989/impacts"
                }
            },
            {
                "id": "productid.023639ec38cafdb132de25be6be67bf2",
                "name": "Chicken Hotpot",
                "gtin": "5010525096995",
                "image": "https://static.euw1.sustained.app/products/images/5010525096995_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.023639ec38cafdb132de25be6be67bf2",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.023639ec38cafdb132de25be6be67bf2/impacts"
                }
            },
            {
                "id": "productid.0263f8ec3915dde38d9ebfcc96ba24df",
                "name": "King Prawns with Prosecco Sauce",
                "gtin": "5057172816392",
                "image": "https://static.euw1.sustained.app/products/images/5057172816392_0.jpg",
                "pack": "430g",
                "grade": "C",
                "points": {
                    "pef": 21.808675686037812
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0263f8ec3915dde38d9ebfcc96ba24df",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0263f8ec3915dde38d9ebfcc96ba24df/impacts"
                }
            },
            {
                "id": "productid.027bda8bb0ade3e3a8e9fb0c1baef0a1",
                "name": "Creamy Mushroom Pie",
                "gtin": "5031102000020",
                "image": "https://static.euw1.sustained.app/products/images/5031102000020_0.jpg",
                "pack": "235g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.027bda8bb0ade3e3a8e9fb0c1baef0a1",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.027bda8bb0ade3e3a8e9fb0c1baef0a1/impacts"
                }
            },
            {
                "id": "productid.027c2a0f75492403ac29dbfad28adbb7",
                "name": "Seasoned Southern Fried Chicken Thighs & Drumsticks",
                "gtin": "5057172466689",
                "image": "https://static.euw1.sustained.app/products/images/5057172466689_0.jpg",
                "pack": "950g",
                "grade": "F",
                "points": {
                    "pef": 58.61733401971765
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.027c2a0f75492403ac29dbfad28adbb7",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.027c2a0f75492403ac29dbfad28adbb7/impacts"
                }
            },
            {
                "id": "productid.02e324e65c80da5f869343d59288517d",
                "name": "PlantLiving: Chocolate & Cherry Pots",
                "gtin": "5000169613054",
                "image": "https://static.euw1.sustained.app/products/images/5000169613054_0.jpg",
                "pack": "90g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02e324e65c80da5f869343d59288517d",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02e324e65c80da5f869343d59288517d/impacts"
                }
            },
            {
                "id": "productid.02ea8c7657ad887f9abc84207b2245b2",
                "name": "Chilli Cheese Quiche",
                "gtin": "5010525066097",
                "image": "https://static.euw1.sustained.app/products/images/5010525066097_0.jpg",
                "pack": "170g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02ea8c7657ad887f9abc84207b2245b2",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02ea8c7657ad887f9abc84207b2245b2/impacts"
                }
            },
            {
                "id": "productid.032071662944d414da606366945aec38",
                "name": "Meatloaf Marinara",
                "gtin": "5059512739774",
                "image": "https://static.euw1.sustained.app/products/images/5059512739774_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 276.32121622322865
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.032071662944d414da606366945aec38",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.032071662944d414da606366945aec38/impacts"
                }
            },
            {
                "id": "productid.0326977bd6f517ce036dc0756dd1745d",
                "name": "Chicken Korma with Pulao Rice",
                "gtin": "5391517071078",
                "image": "https://static.euw1.sustained.app/products/images/5391517071078_0.jpg",
                "pack": "450g",
                "grade": "F",
                "points": {
                    "pef": 61.85443811295039
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0326977bd6f517ce036dc0756dd1745d",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0326977bd6f517ce036dc0756dd1745d/impacts"
                }
            },
            {
                "id": "productid.033ffcba5f3ca93d51f55c0f5a4c7d2d",
                "name": "The Bakery Apple Crumbles",
                "gtin": "5054781497462",
                "image": "https://static.euw1.sustained.app/products/images/5054781497462_0.jpg",
                "pack": "120g",
                "grade": "C",
                "points": {
                    "pef": 17.203446958184845
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.033ffcba5f3ca93d51f55c0f5a4c7d2d",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.033ffcba5f3ca93d51f55c0f5a4c7d2d/impacts"
                }
            },
            {
                "id": "productid.03496b573bad3f077f7bc45059329ba0",
                "name": "Food Count On Us Braised Steak",
                "gtin": "0000000206778",
                "image": "https://static.euw1.sustained.app/products/images/0000000206778_0.jpg",
                "pack": "380g",
                "grade": "F",
                "points": {
                    "pef": 66.35952374142312
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03496b573bad3f077f7bc45059329ba0",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03496b573bad3f077f7bc45059329ba0/impacts"
                }
            },
            {
                "id": "productid.03579e1aabe834b7dfc3fb631551f969",
                "name": "Gala Pie Pork & Egg",
                "gtin": "5057172730919",
                "image": "https://static.euw1.sustained.app/products/images/5057172730919_0.jpg",
                "pack": "320g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03579e1aabe834b7dfc3fb631551f969",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03579e1aabe834b7dfc3fb631551f969/impacts"
                }
            },
            {
                "id": "productid.0370cf02f151c046ae3d860d735a7aea",
                "name": "Carrot & Reduced Fat Houmous",
                "gtin": "5000169158692",
                "image": "https://static.euw1.sustained.app/products/images/5000169158692_0.jpg",
                "pack": "125g",
                "grade": "B",
                "points": {
                    "pef": 12.600821905922286
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0370cf02f151c046ae3d860d735a7aea",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0370cf02f151c046ae3d860d735a7aea/impacts"
                }
            },
            {
                "id": "productid.037c98bc7b1994b261532847eff48a2c",
                "name": "On the Go This Isn't a Chicken Tikka Roll",
                "gtin": "0000000453462",
                "image": "https://static.euw1.sustained.app/products/images/0000000453462_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.037c98bc7b1994b261532847eff48a2c",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.037c98bc7b1994b261532847eff48a2c/impacts"
                }
            },
            {
                "id": "productid.038201b8ab1a5400f51cdff5a30f4b6d",
                "name": "Burrito Chipotle BBQ Pork",
                "gtin": "5038794000062",
                "image": "https://static.euw1.sustained.app/products/images/5038794000062_0.jpg",
                "pack": "205g",
                "grade": "E",
                "points": {
                    "pef": 47.05778250243114
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.038201b8ab1a5400f51cdff5a30f4b6d",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.038201b8ab1a5400f51cdff5a30f4b6d/impacts"
                }
            },
            {
                "id": "productid.03822003f6e34a13a221b36c0604e885",
                "name": "Southern Style Chicken Bites Mini Packs",
                "gtin": "5000178113026",
                "image": "https://static.euw1.sustained.app/products/images/5000178113026_0.jpg",
                "pack": "22.5g",
                "grade": "F",
                "points": {
                    "pef": 76.77090052327148
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03822003f6e34a13a221b36c0604e885",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03822003f6e34a13a221b36c0604e885/impacts"
                }
            },
            {
                "id": "productid.038e2d321232701f954071e7732bd855",
                "name": "Prawn Cocktail",
                "gtin": "5057545986905",
                "image": "https://static.euw1.sustained.app/products/images/5057545986905_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 34.60611012848653
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.038e2d321232701f954071e7732bd855",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.038e2d321232701f954071e7732bd855/impacts"
                }
            },
            {
                "id": "productid.03a6ad2d4b4c6ddbf582f05113d07807",
                "name": "Rack of Smoky BBQ Pork Ribs",
                "gtin": "5010251946083",
                "image": "https://static.euw1.sustained.app/products/images/5010251946083_0.jpg",
                "pack": "550g",
                "grade": "G",
                "points": {
                    "pef": 97.55889369924553
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03a6ad2d4b4c6ddbf582f05113d07807",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03a6ad2d4b4c6ddbf582f05113d07807/impacts"
                }
            },
            {
                "id": "productid.03c74e7d41a21a45aab88e707bf458fe",
                "name": "Christmas Scottish Smoked Salmon P\u00e2t\u00e9 with Breadsticks",
                "gtin": "5000169612095",
                "image": "https://static.euw1.sustained.app/products/images/5000169612095_0.jpg",
                "pack": "70g",
                "grade": "F",
                "points": {
                    "pef": 71.08291820414351
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03c74e7d41a21a45aab88e707bf458fe",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03c74e7d41a21a45aab88e707bf458fe/impacts"
                }
            },
            {
                "id": "productid.041d45e1dd74fdc987b550c2cfe704d4",
                "name": "Avocado & Feta Side Salad",
                "gtin": "5000169437605",
                "image": "https://static.euw1.sustained.app/products/images/5000169437605_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.041d45e1dd74fdc987b550c2cfe704d4",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.041d45e1dd74fdc987b550c2cfe704d4/impacts"
                }
            },
            {
                "id": "productid.0437bba8e6038c09231769b0e25a6f4f",
                "name": "Sweet Chilli Chicken Breast Slices",
                "gtin": "5010251985839",
                "image": "https://static.euw1.sustained.app/products/images/5010251985839_0.jpg",
                "pack": "180g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0437bba8e6038c09231769b0e25a6f4f",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0437bba8e6038c09231769b0e25a6f4f/impacts"
                }
            },
            {
                "id": "productid.0441110a466365927b438ae2063f4753",
                "name": "Chargrilled Piri Piri Mini Fillets",
                "gtin": "5010292966507",
                "image": "https://static.euw1.sustained.app/products/images/5010292966507_0.jpg",
                "pack": "120g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0441110a466365927b438ae2063f4753",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0441110a466365927b438ae2063f4753/impacts"
                }
            },
            {
                "id": "productid.04b0dce703351fffede1a5e5ea989746",
                "name": "Ham, Mushroom & Olive Medium Woodfired Sourdough Pizza",
                "gtin": "0219560000003",
                "image": "https://static.euw1.sustained.app/products/images/0219560000003_0.jpg",
                "pack": "",
                "grade": "G",
                "points": {
                    "pef": 477.3726877900245
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04b0dce703351fffede1a5e5ea989746",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04b0dce703351fffede1a5e5ea989746/impacts"
                }
            },
            {
                "id": "productid.04bf4f1f090cdcac65bea44f241a0561",
                "name": "Chicken Teriyaki",
                "gtin": "5024593000051",
                "image": "https://static.euw1.sustained.app/products/images/5024593000051_0.jpg",
                "pack": "450g",
                "grade": "D",
                "points": {
                    "pef": 26.67117613621418
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04bf4f1f090cdcac65bea44f241a0561",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04bf4f1f090cdcac65bea44f241a0561/impacts"
                }
            },
            {
                "id": "productid.04f4f28eddbae64064946a055b587f0e",
                "name": "Pork Meatballs with Vegetables in Tomato Sauce",
                "gtin": "5907501002036",
                "image": "https://static.euw1.sustained.app/products/images/5907501002036_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04f4f28eddbae64064946a055b587f0e",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04f4f28eddbae64064946a055b587f0e/impacts"
                }
            },
            {
                "id": "productid.04fd40c98223fb6b3b740e81f0c8d2d5",
                "name": "British Fresh Roast in the Bag Chicken Thighs & Drumsticks",
                "gtin": "00338448",
                "image": "https://static.euw1.sustained.app/products/images/00338448_0.jpg",
                "pack": "600g",
                "grade": "F",
                "points": {
                    "pef": 59.1213143797725
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04fd40c98223fb6b3b740e81f0c8d2d5",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04fd40c98223fb6b3b740e81f0c8d2d5/impacts"
                }
            },
            {
                "id": "productid.0525d6e2d4a96f92f21566550ec5dc57",
                "name": "World Caf\u00e9 American Inspired BBQ Chicken with Potatoes",
                "gtin": "5000169362426",
                "image": "https://static.euw1.sustained.app/products/images/5000169362426_0.jpg",
                "pack": "350g",
                "grade": "D",
                "points": {
                    "pef": 25.32238895175792
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0525d6e2d4a96f92f21566550ec5dc57",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0525d6e2d4a96f92f21566550ec5dc57/impacts"
                }
            },
            {
                "id": "productid.054126248c6794dca3a6a06a5c3c7693",
                "name": "Barbecue Flavoured Pulled Pork",
                "gtin": "5391511052127",
                "image": "https://static.euw1.sustained.app/products/images/5391511052127_0.jpg",
                "pack": "360g",
                "grade": "G",
                "points": {
                    "pef": 119.88478103261961
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.054126248c6794dca3a6a06a5c3c7693",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.054126248c6794dca3a6a06a5c3c7693/impacts"
                }
            },
            {
                "id": "productid.0542d0ddcbf35e7faceee3226997df15",
                "name": "Bens Original Wholegrain Savoury Chicken Microwave Rice",
                "gtin": "5010034011359",
                "image": "https://static.euw1.sustained.app/products/images/5010034011359_0.jpg",
                "pack": "250g",
                "grade": "D",
                "points": {
                    "pef": 30.501067365225985
                },
                "info_icons": [],
                "category": "Ready Made Foods",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0542d0ddcbf35e7faceee3226997df15",
                    "category": "https://api.sustained.com/choice/v1/categories/ready-made-foods",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0542d0ddcbf35e7faceee3226997df15/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.006532ac65a84182a503987cb5888680",
                "name": "Food Fusilli Pasta",
                "gtin": "0000029074891",
                "image": "https://static.euw1.sustained.app/products/images/0000029074891_0.jpg",
                "pack": "1kg",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.006532ac65a84182a503987cb5888680",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.006532ac65a84182a503987cb5888680/impacts"
                }
            },
            {
                "id": "productid.00c350a62aff391a35b3b2b99728b6ee",
                "name": "Organic Basmati Rice",
                "gtin": "5010251492450",
                "image": "https://static.euw1.sustained.app/products/images/5010251492450_0.jpg",
                "pack": "500g",
                "grade": "D",
                "points": {
                    "pef": 31.227682434520272
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00c350a62aff391a35b3b2b99728b6ee",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00c350a62aff391a35b3b2b99728b6ee/impacts"
                }
            },
            {
                "id": "productid.014bb261d5cf09ce12e7c3cb6fb0c383",
                "name": "Spanish Paella Rice",
                "gtin": "5054781951896",
                "image": "https://static.euw1.sustained.app/products/images/5054781951896_0.jpg",
                "pack": "500g",
                "grade": "D",
                "points": {
                    "pef": 31.227682434520275
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.014bb261d5cf09ce12e7c3cb6fb0c383",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.014bb261d5cf09ce12e7c3cb6fb0c383/impacts"
                }
            },
            {
                "id": "productid.01eb14f5ec140ee77b8126f5d071ef94",
                "name": "Garofalo Tricolore",
                "gtin": "8000139912078",
                "image": "https://static.euw1.sustained.app/products/images/8000139912078_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01eb14f5ec140ee77b8126f5d071ef94",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01eb14f5ec140ee77b8126f5d071ef94/impacts"
                }
            },
            {
                "id": "productid.02726fb1f1038429778c6b20d5088fed",
                "name": "Easy Cook Long Grain Rice",
                "gtin": "8901047011184",
                "image": "https://static.euw1.sustained.app/products/images/8901047011184_0.jpg",
                "pack": "5kg",
                "grade": "D",
                "points": {
                    "pef": 31.227682434520275
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02726fb1f1038429778c6b20d5088fed",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02726fb1f1038429778c6b20d5088fed/impacts"
                }
            },
            {
                "id": "productid.02a740c61930937ea517277617d973ad",
                "name": "Noodles Japanese Style Katsu Curry",
                "gtin": "5098732011932",
                "image": "https://static.euw1.sustained.app/products/images/5098732011932_0.jpg",
                "pack": "85g",
                "grade": "F",
                "points": {
                    "pef": 55.55206119412428
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02a740c61930937ea517277617d973ad",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02a740c61930937ea517277617d973ad/impacts"
                }
            },
            {
                "id": "productid.03b34e6e613e0af4c3c54ec120d71835",
                "name": "Organic Red Lentil Spaghetti",
                "gtin": "5032722316973",
                "image": "https://static.euw1.sustained.app/products/images/5032722316973_0.jpg",
                "pack": "250g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03b34e6e613e0af4c3c54ec120d71835",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03b34e6e613e0af4c3c54ec120d71835/impacts"
                }
            },
            {
                "id": "productid.043553a7914bff98dc75a93edeb0a38f",
                "name": "Easy Cook Long Grain Rice",
                "gtin": "01072662",
                "image": "https://static.euw1.sustained.app/products/images/01072662_0.jpg",
                "pack": "500g",
                "grade": "D",
                "points": {
                    "pef": 31.227682434520275
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.043553a7914bff98dc75a93edeb0a38f",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.043553a7914bff98dc75a93edeb0a38f/impacts"
                }
            },
            {
                "id": "productid.05f294a130f43355390de6315a406494",
                "name": "Jollof Rice with Vegetables Mild",
                "gtin": "5060387830084",
                "image": "https://static.euw1.sustained.app/products/images/5060387830084_0.jpg",
                "pack": "250g",
                "grade": "D",
                "points": {
                    "pef": 25.658001187512333
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05f294a130f43355390de6315a406494",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05f294a130f43355390de6315a406494/impacts"
                }
            },
            {
                "id": "productid.068e347118bc68b6c407c5738106af6b",
                "name": "Wholewheat Penne",
                "gtin": "5054781025276",
                "image": "https://static.euw1.sustained.app/products/images/5054781025276_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 31.689515767636028
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.068e347118bc68b6c407c5738106af6b",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.068e347118bc68b6c407c5738106af6b/impacts"
                }
            },
            {
                "id": "productid.08a372b23b7a197d14ce0df057e6c9c8",
                "name": "On Market Street Classic Ramen Bowl",
                "gtin": "5010525094939",
                "image": "https://static.euw1.sustained.app/products/images/5010525094939_0.jpg",
                "pack": "320g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08a372b23b7a197d14ce0df057e6c9c8",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08a372b23b7a197d14ce0df057e6c9c8/impacts"
                }
            },
            {
                "id": "productid.09012ecc01b52d75c17c9894ae8846b9",
                "name": "Rice Stick Noodles",
                "gtin": "5020580019792",
                "image": "https://static.euw1.sustained.app/products/images/5020580019792_0.jpg",
                "pack": "400g",
                "grade": "E",
                "points": {
                    "pef": 35.20154044397951
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09012ecc01b52d75c17c9894ae8846b9",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09012ecc01b52d75c17c9894ae8846b9/impacts"
                }
            },
            {
                "id": "productid.09a497f620f4dcca7d4f9df2f56f3310",
                "name": "Orecchiette No 91 Integrali",
                "gtin": "8001250005984",
                "image": "https://static.euw1.sustained.app/products/images/8001250005984_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 31.689515767636028
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09a497f620f4dcca7d4f9df2f56f3310",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09a497f620f4dcca7d4f9df2f56f3310/impacts"
                }
            },
            {
                "id": "productid.09d93ab7f3f540f1a01813154737151f",
                "name": "Rustichella d'Abruzzo Spaghetti",
                "gtin": "8009452227012",
                "image": "https://static.euw1.sustained.app/products/images/8009452227012_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09d93ab7f3f540f1a01813154737151f",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09d93ab7f3f540f1a01813154737151f/impacts"
                }
            },
            {
                "id": "productid.0a1c79fa8642922a80f1be2ed3c6235c",
                "name": "Long Grain Rice",
                "gtin": "5010893022541",
                "image": "https://static.euw1.sustained.app/products/images/5010893022541_0.jpg",
                "pack": "1kg",
                "grade": "D",
                "points": {
                    "pef": 31.227682434520275
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a1c79fa8642922a80f1be2ed3c6235c",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a1c79fa8642922a80f1be2ed3c6235c/impacts"
                }
            },
            {
                "id": "productid.0a37912d0c8814327db02d61c069e51b",
                "name": "Wholegrain Brown Basmati Rice",
                "gtin": "5011157070438",
                "image": "https://static.euw1.sustained.app/products/images/5011157070438_0.jpg",
                "pack": "5kg",
                "grade": "D",
                "points": {
                    "pef": 31.227682434520275
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a37912d0c8814327db02d61c069e51b",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a37912d0c8814327db02d61c069e51b/impacts"
                }
            },
            {
                "id": "productid.0bcc7b2b38ecf445067072888853a1eb",
                "name": "Green Lentils",
                "gtin": "5020933015594",
                "image": "https://static.euw1.sustained.app/products/images/5020933015594_0.jpg",
                "pack": "3kg",
                "grade": "B",
                "points": {
                    "pef": 12.829575775056949
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bcc7b2b38ecf445067072888853a1eb",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bcc7b2b38ecf445067072888853a1eb/impacts"
                }
            },
            {
                "id": "productid.0bea2e22fb1587808c49af2c509c7579",
                "name": "Sweet Chilli Noodle",
                "gtin": "0000000273084",
                "image": "https://static.euw1.sustained.app/products/images/0000000273084_0.jpg",
                "pack": "250g",
                "grade": "C",
                "points": {
                    "pef": 16.992545305339583
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bea2e22fb1587808c49af2c509c7579",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bea2e22fb1587808c49af2c509c7579/impacts"
                }
            },
            {
                "id": "productid.0bef2ff8c110e9fda8012e1f3f258594",
                "name": "The Original Curry Flavour Oriental Instant Noodles",
                "gtin": "8888056103850",
                "image": "https://static.euw1.sustained.app/products/images/8888056103850_0.jpg",
                "pack": "85g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bef2ff8c110e9fda8012e1f3f258594",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bef2ff8c110e9fda8012e1f3f258594/impacts"
                }
            },
            {
                "id": "productid.0c7ff5cc21003d3cca7e1c31541667bf",
                "name": "Italian Lasagne Sheets",
                "gtin": "5010251595519",
                "image": "https://static.euw1.sustained.app/products/images/5010251595519_0.jpg",
                "pack": "250g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c7ff5cc21003d3cca7e1c31541667bf",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c7ff5cc21003d3cca7e1c31541667bf/impacts"
                }
            },
            {
                "id": "productid.0cda9344870f077ea377a75e443d9db3",
                "name": "Street Food Green Curry Noodle Kit",
                "gtin": "5000354915475",
                "image": "https://static.euw1.sustained.app/products/images/5000354915475_0.jpg",
                "pack": "110g",
                "grade": "E",
                "points": {
                    "pef": 42.26262511402318
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0cda9344870f077ea377a75e443d9db3",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0cda9344870f077ea377a75e443d9db3/impacts"
                }
            },
            {
                "id": "productid.0d05e1ec16000a2b00643906253c5778",
                "name": "Charcoal Noodles with Epic Mushroom",
                "gtin": "5015821158613",
                "image": "https://static.euw1.sustained.app/products/images/5015821158613_0.jpg",
                "pack": "80g",
                "grade": "A",
                "points": {
                    "pef": 9.420036666174834
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d05e1ec16000a2b00643906253c5778",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d05e1ec16000a2b00643906253c5778/impacts"
                }
            },
            {
                "id": "productid.0d9e69ada013a252168763cc2c3a4ce7",
                "name": "Food High Fibre Noodles",
                "gtin": "0000000594257",
                "image": "https://static.euw1.sustained.app/products/images/0000000594257_0.jpg",
                "pack": "250g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d9e69ada013a252168763cc2c3a4ce7",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d9e69ada013a252168763cc2c3a4ce7/impacts"
                }
            },
            {
                "id": "productid.0ddf688bea8559e9060b546a3916a7d2",
                "name": "Bombay Bad Boy Flavour",
                "gtin": "8720182371003",
                "image": "https://static.euw1.sustained.app/products/images/8720182371003_0.jpg",
                "pack": "90g",
                "grade": "F",
                "points": {
                    "pef": 67.83097184357618
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ddf688bea8559e9060b546a3916a7d2",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ddf688bea8559e9060b546a3916a7d2/impacts"
                }
            },
            {
                "id": "productid.0ebb8218ef9cce77bb27f06d4255f3dd",
                "name": "No 108 Ondine Spinach Lasagne",
                "gtin": "8006680121082",
                "image": "https://static.euw1.sustained.app/products/images/8006680121082_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ebb8218ef9cce77bb27f06d4255f3dd",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ebb8218ef9cce77bb27f06d4255f3dd/impacts"
                }
            },
            {
                "id": "productid.0fb05a62c550c3d89a41dfb9180275d0",
                "name": "Long Grain Rice",
                "gtin": "5410673003560",
                "image": "https://static.euw1.sustained.app/products/images/5410673003560_0.jpg",
                "pack": "1kg",
                "grade": "D",
                "points": {
                    "pef": 31.227682434520275
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fb05a62c550c3d89a41dfb9180275d0",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fb05a62c550c3d89a41dfb9180275d0/impacts"
                }
            },
            {
                "id": "productid.0fb9ac5fc540746ae35cdd34d05cb03f",
                "name": "Essential Waitrose & Partners Fusilli",
                "gtin": "5000169014301",
                "image": "https://static.euw1.sustained.app/products/images/5000169014301_0.jpg",
                "pack": "1kg",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fb9ac5fc540746ae35cdd34d05cb03f",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fb9ac5fc540746ae35cdd34d05cb03f/impacts"
                }
            },
            {
                "id": "productid.10a33bea03d454c043a5f61185257264",
                "name": "Organic Long Grain Brown Rice",
                "gtin": "5032722318144",
                "image": "https://static.euw1.sustained.app/products/images/5032722318144_0.jpg",
                "pack": "1kg",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10a33bea03d454c043a5f61185257264",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10a33bea03d454c043a5f61185257264/impacts"
                }
            },
            {
                "id": "productid.125cff345b4b64c42b6a5484a8bb94c5",
                "name": "Pasta Tagliatelle",
                "gtin": "8056324825081",
                "image": "https://static.euw1.sustained.app/products/images/8056324825081_0.jpg",
                "pack": "250g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.125cff345b4b64c42b6a5484a8bb94c5",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.125cff345b4b64c42b6a5484a8bb94c5/impacts"
                }
            },
            {
                "id": "productid.12681feb1556dc400aad3b6472ed3114",
                "name": "Lasagne Pasta",
                "gtin": "5000232027306",
                "image": "https://static.euw1.sustained.app/products/images/5000232027306_0.jpg",
                "pack": "375g",
                "grade": "D",
                "points": {
                    "pef": 27.0572447906217
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12681feb1556dc400aad3b6472ed3114",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12681feb1556dc400aad3b6472ed3114/impacts"
                }
            },
            {
                "id": "productid.126b6eb9cf545e2bb320c703d4911d39",
                "name": "Multiboost Organic Hemp Milled Whole Seed with Flax & Chia Seeds",
                "gtin": "5016887005187",
                "image": "https://static.euw1.sustained.app/products/images/5016887005187_0.jpg",
                "pack": "200g",
                "grade": "D",
                "points": {
                    "pef": 26.31671571521658
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.126b6eb9cf545e2bb320c703d4911d39",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.126b6eb9cf545e2bb320c703d4911d39/impacts"
                }
            },
            {
                "id": "productid.128de1e203d8cb14d2e82a9c9f8df5d3",
                "name": "PASTAVILLA Rotini",
                "gtin": "8690684002098",
                "image": "https://static.euw1.sustained.app/products/images/8690684002098_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.128de1e203d8cb14d2e82a9c9f8df5d3",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.128de1e203d8cb14d2e82a9c9f8df5d3/impacts"
                }
            },
            {
                "id": "productid.12c729363f31932fea2e196f3343d3a4",
                "name": "Original Udon Ramen Kit",
                "gtin": "0634158582868",
                "image": "https://static.euw1.sustained.app/products/images/0634158582868_0.jpg",
                "pack": "486g",
                "grade": "F",
                "points": {
                    "pef": 67.83097184357618
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12c729363f31932fea2e196f3343d3a4",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12c729363f31932fea2e196f3343d3a4/impacts"
                }
            },
            {
                "id": "productid.12d4b707e2e8e40ea7271e7685a58644",
                "name": "Collection Italian Single Grain Spaghettoni",
                "gtin": "0000029129485",
                "image": "https://static.euw1.sustained.app/products/images/0000029129485_0.jpg",
                "pack": "500g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12d4b707e2e8e40ea7271e7685a58644",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12d4b707e2e8e40ea7271e7685a58644/impacts"
                }
            },
            {
                "id": "productid.13045b543f9a8afdc9d71a5ac5a987b4",
                "name": "Stir-Fry On Market Street Soy, Ginger & Garlic Special Noodles",
                "gtin": "5010251733942",
                "image": "https://static.euw1.sustained.app/products/images/5010251733942_0.jpg",
                "pack": "410g",
                "grade": "F",
                "points": {
                    "pef": 52.02579012525998
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.13045b543f9a8afdc9d71a5ac5a987b4",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.13045b543f9a8afdc9d71a5ac5a987b4/impacts"
                }
            },
            {
                "id": "productid.13276057701a89e24030f2b7164bee1e",
                "name": "Coriander Whole",
                "gtin": "5018605899566",
                "image": "https://static.euw1.sustained.app/products/images/5018605899566_0.jpg",
                "pack": "300g",
                "grade": "A",
                "points": {
                    "pef": 10.81169255215334
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.13276057701a89e24030f2b7164bee1e",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.13276057701a89e24030f2b7164bee1e/impacts"
                }
            },
            {
                "id": "productid.134c9f2a0852604a5e06c79163a212ae",
                "name": "Scotch Broth Mix",
                "gtin": "5010148111051",
                "image": "https://static.euw1.sustained.app/products/images/5010148111051_0.jpg",
                "pack": "500g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.134c9f2a0852604a5e06c79163a212ae",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.134c9f2a0852604a5e06c79163a212ae/impacts"
                }
            },
            {
                "id": "productid.135c9117e2e73566aa04fa917c47eab1",
                "name": "Ramen Chinese Stir-Fry Noodles",
                "gtin": "5024448562307",
                "image": "https://static.euw1.sustained.app/products/images/5024448562307_0.jpg",
                "pack": "200g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.135c9117e2e73566aa04fa917c47eab1",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.135c9117e2e73566aa04fa917c47eab1/impacts"
                }
            },
            {
                "id": "productid.1373d131117d831c17c2f200c348bb51",
                "name": "Basmati Rice",
                "gtin": "5011157992860",
                "image": "https://static.euw1.sustained.app/products/images/5011157992860_0.jpg",
                "pack": "5kg",
                "grade": "D",
                "points": {
                    "pef": 31.227682434520272
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1373d131117d831c17c2f200c348bb51",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1373d131117d831c17c2f200c348bb51/impacts"
                }
            },
            {
                "id": "productid.13ab0f5448f9b3e664578295d109c2b2",
                "name": "Green Lentils",
                "gtin": "5013635352470",
                "image": "https://static.euw1.sustained.app/products/images/5013635352470_0.jpg",
                "pack": "1kg",
                "grade": "B",
                "points": {
                    "pef": 12.829575775056949
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.13ab0f5448f9b3e664578295d109c2b2",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.13ab0f5448f9b3e664578295d109c2b2/impacts"
                }
            },
            {
                "id": "productid.14184894b741a149b7548e1e405bd87a",
                "name": "Tomato & Pancetta Ravioli",
                "gtin": "5057545807231",
                "image": "https://static.euw1.sustained.app/products/images/5057545807231_0.jpg",
                "pack": "300g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14184894b741a149b7548e1e405bd87a",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14184894b741a149b7548e1e405bd87a/impacts"
                }
            },
            {
                "id": "productid.14392dd369ae3f4b6fc22693f2c12887",
                "name": "PlantLiving: No Chorizo Tortelloni",
                "gtin": "5000169612224",
                "image": "https://static.euw1.sustained.app/products/images/5000169612224_0.jpg",
                "pack": "250g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14392dd369ae3f4b6fc22693f2c12887",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14392dd369ae3f4b6fc22693f2c12887/impacts"
                }
            },
            {
                "id": "productid.149204c7452a1168110d9f0b89e1c076",
                "name": "Basmati Rice",
                "gtin": "5010398010074",
                "image": "https://static.euw1.sustained.app/products/images/5010398010074_0.jpg",
                "pack": "500g",
                "grade": "D",
                "points": {
                    "pef": 31.227682434520275
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.149204c7452a1168110d9f0b89e1c076",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.149204c7452a1168110d9f0b89e1c076/impacts"
                }
            },
            {
                "id": "productid.149c52586841dd1e22dca788ad19ef17",
                "name": "Hearty Food Co. Spaghetti",
                "gtin": "5057545092514",
                "image": "https://static.euw1.sustained.app/products/images/5057545092514_0.jpg",
                "pack": "",
                "grade": "E",
                "points": {
                    "pef": 31.689515767636028
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.149c52586841dd1e22dca788ad19ef17",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.149c52586841dd1e22dca788ad19ef17/impacts"
                }
            },
            {
                "id": "productid.15c1beb466197ee6d8393fdebb054709",
                "name": "Spaghetti Pasta",
                "gtin": "5000232020345",
                "image": "https://static.euw1.sustained.app/products/images/5000232020345_0.jpg",
                "pack": "3kg",
                "grade": "D",
                "points": {
                    "pef": 27.0572447906217
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.15c1beb466197ee6d8393fdebb054709",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.15c1beb466197ee6d8393fdebb054709/impacts"
                }
            },
            {
                "id": "productid.15c2a1d0393502cd3b86075fbda7b14e",
                "name": "Curry Flavour Noodle Pot",
                "gtin": "5010251447191",
                "image": "https://static.euw1.sustained.app/products/images/5010251447191_0.jpg",
                "pack": "90g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.15c2a1d0393502cd3b86075fbda7b14e",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.15c2a1d0393502cd3b86075fbda7b14e/impacts"
                }
            },
            {
                "id": "productid.162e52b6b247262f1a72f5ca17d93e95",
                "name": "Super Noodles Mexican Chicken Fajita Flavour",
                "gtin": "5000354921087",
                "image": "https://static.euw1.sustained.app/products/images/5000354921087_0.jpg",
                "pack": "90g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.162e52b6b247262f1a72f5ca17d93e95",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.162e52b6b247262f1a72f5ca17d93e95/impacts"
                }
            },
            {
                "id": "productid.16c051efeec3a204e1cdb86f782502a2",
                "name": "Instant Noodles Chicken Flavor",
                "gtin": "0089686130010",
                "image": "https://static.euw1.sustained.app/products/images/0089686130010_0.jpg",
                "pack": "70g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16c051efeec3a204e1cdb86f782502a2",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16c051efeec3a204e1cdb86f782502a2/impacts"
                }
            },
            {
                "id": "productid.16cbe33d7eb64ee821ec96f06bf23520",
                "name": "Free From Macaroni",
                "gtin": "5057172234004",
                "image": "https://static.euw1.sustained.app/products/images/5057172234004_0.jpg",
                "pack": "500g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16cbe33d7eb64ee821ec96f06bf23520",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16cbe33d7eb64ee821ec96f06bf23520/impacts"
                }
            },
            {
                "id": "productid.16fab2f79481ea53f23bf9c62093af8d",
                "name": "Golden Linseeds",
                "gtin": "5054070674857",
                "image": "https://static.euw1.sustained.app/products/images/5054070674857_0.jpg",
                "pack": "150g",
                "grade": "E",
                "points": {
                    "pef": 38.03148739229502
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Rice & Pasta",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.16fab2f79481ea53f23bf9c62093af8d",
                    "category": "https://api.sustained.com/choice/v1/categories/rice-and-pasta",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.16fab2f79481ea53f23bf9c62093af8d/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.004d51ca1c930c32f48c2dcb551f3dfb",
                "name": "Favourites Cock-a-Leekie Soup",
                "gtin": "5012427291409",
                "image": "https://static.euw1.sustained.app/products/images/5012427291409_0.jpg",
                "pack": "400g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.004d51ca1c930c32f48c2dcb551f3dfb",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.004d51ca1c930c32f48c2dcb551f3dfb/impacts"
                }
            },
            {
                "id": "productid.0175f3dcde33c34d6fa40dec31cb4e7c",
                "name": "Chicken & Veg Noodle Cup Soup",
                "gtin": "5000157141576",
                "image": "https://static.euw1.sustained.app/products/images/5000157141576_0.jpg",
                "pack": "18g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0175f3dcde33c34d6fa40dec31cb4e7c",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0175f3dcde33c34d6fa40dec31cb4e7c/impacts"
                }
            },
            {
                "id": "productid.01ef178bd3b0943cd838a50c28f86c29",
                "name": "Potato Soup 1.5pt",
                "gtin": "8712566301751",
                "image": "https://static.euw1.sustained.app/products/images/8712566301751_0.jpg",
                "pack": "78g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01ef178bd3b0943cd838a50c28f86c29",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01ef178bd3b0943cd838a50c28f86c29/impacts"
                }
            },
            {
                "id": "productid.020bb29f157c27d462c207310502243f",
                "name": "Taste the Difference Sundried Tomato, Chorizo & Orzo Soup",
                "gtin": "00415101",
                "image": "https://static.euw1.sustained.app/products/images/00415101_0.jpg",
                "pack": "400g",
                "grade": "C",
                "points": {
                    "pef": 20.37428809190838
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.020bb29f157c27d462c207310502243f",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.020bb29f157c27d462c207310502243f/impacts"
                }
            },
            {
                "id": "productid.02704b9bd9ecf40e4cbf9be88fccced2",
                "name": "Soup Cream of Chicken",
                "gtin": "5000232900623",
                "image": "https://static.euw1.sustained.app/products/images/5000232900623_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02704b9bd9ecf40e4cbf9be88fccced2",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02704b9bd9ecf40e4cbf9be88fccced2/impacts"
                }
            },
            {
                "id": "productid.0342b286e2389a142027d5fcbc6807b5",
                "name": "Cream of Roasted Mushroom",
                "gtin": "5000232024008",
                "image": "https://static.euw1.sustained.app/products/images/5000232024008_0.jpg",
                "pack": "400g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0342b286e2389a142027d5fcbc6807b5",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0342b286e2389a142027d5fcbc6807b5/impacts"
                }
            },
            {
                "id": "productid.038992eff03877f9a91ca6d1c910eb2e",
                "name": "Super Good Tomato, Orange and Ginger",
                "gtin": "5012427212602",
                "image": "https://static.euw1.sustained.app/products/images/5012427212602_0.jpg",
                "pack": "400g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.038992eff03877f9a91ca6d1c910eb2e",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.038992eff03877f9a91ca6d1c910eb2e/impacts"
                }
            },
            {
                "id": "productid.047013d50a2519bdd7805d83fd8c3bf2",
                "name": "Chunky Soup Minted Lamb Hotpot",
                "gtin": "5051413578698",
                "image": "https://static.euw1.sustained.app/products/images/5051413578698_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.047013d50a2519bdd7805d83fd8c3bf2",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.047013d50a2519bdd7805d83fd8c3bf2/impacts"
                }
            },
            {
                "id": "productid.0767a1fe3ffed364b7ea93a6f1ae5eef",
                "name": "Food Souper Spicy Tomato & Lentil Soup",
                "gtin": "00220552",
                "image": "https://static.euw1.sustained.app/products/images/00220552_0.jpg",
                "pack": "600g",
                "grade": "A",
                "points": {
                    "pef": 9.247373534218864
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0767a1fe3ffed364b7ea93a6f1ae5eef",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0767a1fe3ffed364b7ea93a6f1ae5eef/impacts"
                }
            },
            {
                "id": "productid.0895203f3a9e9dcc4aaa653d3a4fd736",
                "name": "Taste the Difference French Onion Soup with Gruy\u00e8re",
                "gtin": "00442954",
                "image": "https://static.euw1.sustained.app/products/images/00442954_0.jpg",
                "pack": "400g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0895203f3a9e9dcc4aaa653d3a4fd736",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0895203f3a9e9dcc4aaa653d3a4fd736/impacts"
                }
            },
            {
                "id": "productid.0898dbad0ab206e745a520076d92932a",
                "name": "Scottish Style Chicken & Leek Cup Soup",
                "gtin": "5050665028357",
                "image": "https://static.euw1.sustained.app/products/images/5050665028357_0.jpg",
                "pack": "60g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0898dbad0ab206e745a520076d92932a",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0898dbad0ab206e745a520076d92932a/impacts"
                }
            },
            {
                "id": "productid.0ae8232f6fd1f2b5a024effb23f2fc49",
                "name": "Spicy Red Lentil Soup",
                "gtin": "5000169573976",
                "image": "https://static.euw1.sustained.app/products/images/5000169573976_0.jpg",
                "pack": "600g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ae8232f6fd1f2b5a024effb23f2fc49",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ae8232f6fd1f2b5a024effb23f2fc49/impacts"
                }
            },
            {
                "id": "productid.0c892338b296747e454e447adeb6949a",
                "name": "Classic Lentil Soup",
                "gtin": "5000157075550",
                "image": "https://static.euw1.sustained.app/products/images/5000157075550_0.jpg",
                "pack": "400g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0c892338b296747e454e447adeb6949a",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0c892338b296747e454e447adeb6949a/impacts"
                }
            },
            {
                "id": "productid.0ca35c8c233d45370dec93d6f2d4a348",
                "name": "Carrot Parsnip Soup 1.5pt",
                "gtin": "8712566301881",
                "image": "https://static.euw1.sustained.app/products/images/8712566301881_0.jpg",
                "pack": "73g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0ca35c8c233d45370dec93d6f2d4a348",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0ca35c8c233d45370dec93d6f2d4a348/impacts"
                }
            },
            {
                "id": "productid.0d69348ebc8360184e7b54de175cdefa",
                "name": "Taste the Difference Petits Pois & Ham Soup",
                "gtin": "01538205",
                "image": "https://static.euw1.sustained.app/products/images/01538205_0.jpg",
                "pack": "600g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d69348ebc8360184e7b54de175cdefa",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d69348ebc8360184e7b54de175cdefa/impacts"
                }
            },
            {
                "id": "productid.140bdd014ba88fa4db541508b8dcc143",
                "name": "Chicken Noodle Soup",
                "gtin": "3850104003189",
                "image": "https://static.euw1.sustained.app/products/images/3850104003189_0.jpg",
                "pack": "62g",
                "grade": "F",
                "points": {
                    "pef": 56.52541430721243
                },
                "info_icons": [],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.140bdd014ba88fa4db541508b8dcc143",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.140bdd014ba88fa4db541508b8dcc143/impacts"
                }
            },
            {
                "id": "productid.146e8e069fe935420d11413abe93cb17",
                "name": "Sweet Potato & Chilli Soup",
                "gtin": "5054781965107",
                "image": "https://static.euw1.sustained.app/products/images/5054781965107_0.jpg",
                "pack": "600g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.146e8e069fe935420d11413abe93cb17",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.146e8e069fe935420d11413abe93cb17/impacts"
                }
            },
            {
                "id": "productid.18851270186bb8f48fbc67a8ed7c3b9d",
                "name": "Big Soup Sausage & Vegetable",
                "gtin": "5000157078407",
                "image": "https://static.euw1.sustained.app/products/images/5000157078407_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.18851270186bb8f48fbc67a8ed7c3b9d",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.18851270186bb8f48fbc67a8ed7c3b9d/impacts"
                }
            },
            {
                "id": "productid.1b1c584d2f3bb09f69736b996f45474e",
                "name": "Made Without Wheat Gluten Free Maple Syrup Flavour Porridge Pot",
                "gtin": "0000000230582",
                "image": "https://static.euw1.sustained.app/products/images/0000000230582_0.jpg",
                "pack": "70g",
                "grade": "E",
                "points": {
                    "pef": 50.823321111286404
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b1c584d2f3bb09f69736b996f45474e",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b1c584d2f3bb09f69736b996f45474e/impacts"
                }
            },
            {
                "id": "productid.1b7da102da5d533a51654fc129110d03",
                "name": "Favourites Pea & Ham Soup",
                "gtin": "5012427296800",
                "image": "https://static.euw1.sustained.app/products/images/5012427296800_0.jpg",
                "pack": "400g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1b7da102da5d533a51654fc129110d03",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1b7da102da5d533a51654fc129110d03/impacts"
                }
            },
            {
                "id": "productid.1c27da8c9ceb882cee74425634d8ca8d",
                "name": "Just Essentials Vegetable Soup",
                "gtin": "5057172516056",
                "image": "https://static.euw1.sustained.app/products/images/5057172516056_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1c27da8c9ceb882cee74425634d8ca8d",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1c27da8c9ceb882cee74425634d8ca8d/impacts"
                }
            },
            {
                "id": "productid.1cdfd36ca54691de5ccc52dee2020447",
                "name": "Cream of Mushroom Soup",
                "gtin": "5054070650219",
                "image": "https://static.euw1.sustained.app/products/images/5054070650219_0.jpg",
                "pack": "400g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1cdfd36ca54691de5ccc52dee2020447",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1cdfd36ca54691de5ccc52dee2020447/impacts"
                }
            },
            {
                "id": "productid.1e6eb89823d3d081be8fb78d2635a98b",
                "name": "Cup a Soup Minestrone with Croutons",
                "gtin": "5000354921407",
                "image": "https://static.euw1.sustained.app/products/images/5000354921407_0.jpg",
                "pack": "94g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1e6eb89823d3d081be8fb78d2635a98b",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1e6eb89823d3d081be8fb78d2635a98b/impacts"
                }
            },
            {
                "id": "productid.1ef542e1b8acda289c67321562986b5d",
                "name": "Fresh Grab & Go Soup Butternut Squash & Sweet Potato",
                "gtin": "5019624002432",
                "image": "https://static.euw1.sustained.app/products/images/5019624002432_0.jpg",
                "pack": "380g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1ef542e1b8acda289c67321562986b5d",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1ef542e1b8acda289c67321562986b5d/impacts"
                }
            },
            {
                "id": "productid.21172518d74df7543410a9247405b27c",
                "name": "Spanish Inspired Smoky Tomato + Paprika Soup",
                "gtin": "5034124006261",
                "image": "https://static.euw1.sustained.app/products/images/5034124006261_0.jpg",
                "pack": "600g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.21172518d74df7543410a9247405b27c",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.21172518d74df7543410a9247405b27c/impacts"
                }
            },
            {
                "id": "productid.212d8f85909ca2e7282883b8c0e0e0a8",
                "name": "Leek & Potato Cup Soup",
                "gtin": "5000169100943",
                "image": "https://static.euw1.sustained.app/products/images/5000169100943_0.jpg",
                "pack": "25g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.212d8f85909ca2e7282883b8c0e0e0a8",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.212d8f85909ca2e7282883b8c0e0e0a8/impacts"
                }
            },
            {
                "id": "productid.215f1b761bb0b92cb58e8aff73687a7f",
                "name": "Silky Potato & Celeriac with Harrogate Blue",
                "gtin": "5056053306847",
                "image": "https://static.euw1.sustained.app/products/images/5056053306847_0.jpg",
                "pack": "600g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.215f1b761bb0b92cb58e8aff73687a7f",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.215f1b761bb0b92cb58e8aff73687a7f/impacts"
                }
            },
            {
                "id": "productid.21edbab1e7ac4e7423f4606bacad0b03",
                "name": "Organic Spiced Lentil Soup",
                "gtin": "5060083950697",
                "image": "https://static.euw1.sustained.app/products/images/5060083950697_0.jpg",
                "pack": "600g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.21edbab1e7ac4e7423f4606bacad0b03",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.21edbab1e7ac4e7423f4606bacad0b03/impacts"
                }
            },
            {
                "id": "productid.22251119086e1938c3a3383cb10e5a79",
                "name": "Classic Cream of Mushroom Soup",
                "gtin": "5010251476658",
                "image": "https://static.euw1.sustained.app/products/images/5010251476658_0.jpg",
                "pack": "400g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.22251119086e1938c3a3383cb10e5a79",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.22251119086e1938c3a3383cb10e5a79/impacts"
                }
            },
            {
                "id": "productid.2291af67e16a7f1e991fbaa7a7abd9d0",
                "name": "Plant Based Jackfruit, Three Bean & Chipotle Chilli",
                "gtin": "5012427309708",
                "image": "https://static.euw1.sustained.app/products/images/5012427309708_0.jpg",
                "pack": "380g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2291af67e16a7f1e991fbaa7a7abd9d0",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2291af67e16a7f1e991fbaa7a7abd9d0/impacts"
                }
            },
            {
                "id": "productid.2389278bf1abaee52d8d70eff9bef308",
                "name": "Professional 100% Soup Leek & Potato",
                "gtin": "8711200302741",
                "image": "https://static.euw1.sustained.app/products/images/8711200302741_0.jpg",
                "pack": "2.5kg",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2389278bf1abaee52d8d70eff9bef308",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2389278bf1abaee52d8d70eff9bef308/impacts"
                }
            },
            {
                "id": "productid.24654de8437ce9a42e81545178f75916",
                "name": "Hearty Organic Rustic Italian Vegetable Soup",
                "gtin": "0042272005987",
                "image": "https://static.euw1.sustained.app/products/images/0042272005987_0.jpg",
                "pack": "397g",
                "grade": "A",
                "points": {
                    "pef": 9.194641846990544
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.24654de8437ce9a42e81545178f75916",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.24654de8437ce9a42e81545178f75916/impacts"
                }
            },
            {
                "id": "productid.246d6764fcb6d49a0121320f2bafaa10",
                "name": "Hotcup Original Chicken",
                "gtin": "5098732002374",
                "image": "https://static.euw1.sustained.app/products/images/5098732002374_0.jpg",
                "pack": "53g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.246d6764fcb6d49a0121320f2bafaa10",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.246d6764fcb6d49a0121320f2bafaa10/impacts"
                }
            },
            {
                "id": "productid.248888db55ee42ecaa9c0b2e21a51088",
                "name": "Soup of the Day Cauliflower, Onion & Potato Soup",
                "gtin": "5000157141095",
                "image": "https://static.euw1.sustained.app/products/images/5000157141095_0.jpg",
                "pack": "400g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.248888db55ee42ecaa9c0b2e21a51088",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.248888db55ee42ecaa9c0b2e21a51088/impacts"
                }
            },
            {
                "id": "productid.24ea9c1b2651a84166a85369710195f1",
                "name": "Professional Chicken & Vegetable Soup",
                "gtin": "8712566473243",
                "image": "https://static.euw1.sustained.app/products/images/8712566473243_0.jpg",
                "pack": "0.882kg",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.24ea9c1b2651a84166a85369710195f1",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.24ea9c1b2651a84166a85369710195f1/impacts"
                }
            },
            {
                "id": "productid.2592a2ecee25b0ce62ed8e9bafe87e8a",
                "name": "Minestrone Soup",
                "gtin": "5000157078353",
                "image": "https://static.euw1.sustained.app/products/images/5000157078353_0.jpg",
                "pack": "400g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2592a2ecee25b0ce62ed8e9bafe87e8a",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2592a2ecee25b0ce62ed8e9bafe87e8a/impacts"
                }
            },
            {
                "id": "productid.268bbc8f2120e92ad03dfd3e3f4ff8cd",
                "name": "Wholesome Chicken & Vegetable Fresh Soup",
                "gtin": "5011056072151",
                "image": "https://static.euw1.sustained.app/products/images/5011056072151_0.jpg",
                "pack": "1kg",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.268bbc8f2120e92ad03dfd3e3f4ff8cd",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.268bbc8f2120e92ad03dfd3e3f4ff8cd/impacts"
                }
            },
            {
                "id": "productid.26b3af30afc0b519662fe48683e26a56",
                "name": "Soup-Er Creamy Tomato & Basil Soup",
                "gtin": "5391511421510",
                "image": "https://static.euw1.sustained.app/products/images/5391511421510_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.26b3af30afc0b519662fe48683e26a56",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.26b3af30afc0b519662fe48683e26a56/impacts"
                }
            },
            {
                "id": "productid.2a80909b26005e9544ac2d62fe699032",
                "name": "Thick Vegetable Dry Packet Soup",
                "gtin": "8712566247851",
                "image": "https://static.euw1.sustained.app/products/images/8712566247851_0.jpg",
                "pack": "75g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2a80909b26005e9544ac2d62fe699032",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2a80909b26005e9544ac2d62fe699032/impacts"
                }
            },
            {
                "id": "productid.2c9f8fbff1be3886380bd70688564f1a",
                "name": "Cup a Soup Roast Chicken & Grain",
                "gtin": "5000354917516",
                "image": "https://static.euw1.sustained.app/products/images/5000354917516_0.jpg",
                "pack": "64g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2c9f8fbff1be3886380bd70688564f1a",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2c9f8fbff1be3886380bd70688564f1a/impacts"
                }
            },
            {
                "id": "productid.2ce7d1a81f95450342d15ee74511bbdf",
                "name": "Spring Vegetable Soup",
                "gtin": "5039023036593",
                "image": "https://static.euw1.sustained.app/products/images/5039023036593_0.jpg",
                "pack": "95g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2ce7d1a81f95450342d15ee74511bbdf",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2ce7d1a81f95450342d15ee74511bbdf/impacts"
                }
            },
            {
                "id": "productid.2d2ddc7b7a71f2fb94d01ef8e96d30c9",
                "name": "Big Soup Vegetable & Bean Stew",
                "gtin": "5000157079503",
                "image": "https://static.euw1.sustained.app/products/images/5000157079503_0.jpg",
                "pack": "500g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2d2ddc7b7a71f2fb94d01ef8e96d30c9",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2d2ddc7b7a71f2fb94d01ef8e96d30c9/impacts"
                }
            },
            {
                "id": "productid.2d9351efa4ece4b65d5e38fb0c719935",
                "name": "Cream of Tomato Soup in a Cup",
                "gtin": "00413367",
                "image": "https://static.euw1.sustained.app/products/images/00413367_0.jpg",
                "pack": "24g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2d9351efa4ece4b65d5e38fb0c719935",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2d9351efa4ece4b65d5e38fb0c719935/impacts"
                }
            },
            {
                "id": "productid.2dd8c15b4b42948b31f0bcafaeb221bd",
                "name": "Soup Roast Chicken",
                "gtin": "5050665041264",
                "image": "https://static.euw1.sustained.app/products/images/5050665041264_0.jpg",
                "pack": "75g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2dd8c15b4b42948b31f0bcafaeb221bd",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2dd8c15b4b42948b31f0bcafaeb221bd/impacts"
                }
            },
            {
                "id": "productid.2e20861f173728d49ed5e801d35b9532",
                "name": "Chef Selections Creamy Cauliflower with Parmesan & Wholegrain Mustard",
                "gtin": "5012427182707",
                "image": "https://static.euw1.sustained.app/products/images/5012427182707_0.jpg",
                "pack": "400g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2e20861f173728d49ed5e801d35b9532",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2e20861f173728d49ed5e801d35b9532/impacts"
                }
            },
            {
                "id": "productid.2e6b1f0653bef0c7a77b4b7ffa7f7663",
                "name": "Soup Broccoli & Stilton",
                "gtin": "5050665041141",
                "image": "https://static.euw1.sustained.app/products/images/5050665041141_0.jpg",
                "pack": "72g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2e6b1f0653bef0c7a77b4b7ffa7f7663",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2e6b1f0653bef0c7a77b4b7ffa7f7663/impacts"
                }
            },
            {
                "id": "productid.2ed9eba40f3ea495722b49986e3eb9e1",
                "name": "Chicken Noodle Soup",
                "gtin": "0000001851779",
                "image": "https://static.euw1.sustained.app/products/images/0000001851779_0.jpg",
                "pack": "400g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.2ed9eba40f3ea495722b49986e3eb9e1",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.2ed9eba40f3ea495722b49986e3eb9e1/impacts"
                }
            },
            {
                "id": "productid.3152745c9b48f5ba4c4c63820e7e3a86",
                "name": "Mandlen Soup Nuts",
                "gtin": "0072700102179",
                "image": "https://static.euw1.sustained.app/products/images/0072700102179_0.jpg",
                "pack": "28g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.3152745c9b48f5ba4c4c63820e7e3a86",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.3152745c9b48f5ba4c4c63820e7e3a86/impacts"
                }
            },
            {
                "id": "productid.33b6221bfdb8a0a7920c02c128622ba9",
                "name": "Vine Tomato and Aged Italian Parmesan Soup",
                "gtin": "5011056073042",
                "image": "https://static.euw1.sustained.app/products/images/5011056073042_0.jpg",
                "pack": "400g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.33b6221bfdb8a0a7920c02c128622ba9",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.33b6221bfdb8a0a7920c02c128622ba9/impacts"
                }
            },
            {
                "id": "productid.34c48af7240d1f04bf15c8f85555ac1b",
                "name": "Organic Chunky Tomato Soup",
                "gtin": "0042272005567",
                "image": "https://static.euw1.sustained.app/products/images/0042272005567_0.jpg",
                "pack": "411g",
                "grade": "C",
                "points": {
                    "pef": 18.15489401537111
                },
                "info_icons": [],
                "category": "Soups",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.34c48af7240d1f04bf15c8f85555ac1b",
                    "category": "https://api.sustained.com/choice/v1/categories/soups",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.34c48af7240d1f04bf15c8f85555ac1b/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.00b3bf5c013b165663d8fdfb1db683d1",
                "name": "Chapel Down Bacchus Tenterden Gin and Tonic",
                "gtin": "5026608000674",
                "image": "https://static.euw1.sustained.app/products/images/5026608000674_0.jpg",
                "pack": "250ml",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753968
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00b3bf5c013b165663d8fdfb1db683d1",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00b3bf5c013b165663d8fdfb1db683d1/impacts"
                }
            },
            {
                "id": "productid.00b46d8ecc35e43038c775134f395b5c",
                "name": "Writers Tears Red Head Irish Whiskey",
                "gtin": "5099811906156",
                "image": "https://static.euw1.sustained.app/products/images/5099811906156_0.jpg",
                "pack": "700ml",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00b46d8ecc35e43038c775134f395b5c",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00b46d8ecc35e43038c775134f395b5c/impacts"
                }
            },
            {
                "id": "productid.00b8e764d333e6dfb308a9e65e08034a",
                "name": "M\u00edl Gin",
                "gtin": "5391528372300",
                "image": "https://static.euw1.sustained.app/products/images/5391528372300_0.jpg",
                "pack": "700ml",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753968
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00b8e764d333e6dfb308a9e65e08034a",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00b8e764d333e6dfb308a9e65e08034a/impacts"
                }
            },
            {
                "id": "productid.011df82a440238e14f50413fe7ab78e0",
                "name": "Ciroc Peach Flavoured Vodka,",
                "gtin": "5010103934374",
                "image": "https://static.euw1.sustained.app/products/images/5010103934374_0.jpg",
                "pack": "0.700l",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753957
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.011df82a440238e14f50413fe7ab78e0",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.011df82a440238e14f50413fe7ab78e0/impacts"
                }
            },
            {
                "id": "productid.014c5ffecc9dfaf760e50f9386dbe554",
                "name": "Moskovskaya Osobaya Vodka",
                "gtin": "4750021001208",
                "image": "https://static.euw1.sustained.app/products/images/4750021001208_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753957
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.014c5ffecc9dfaf760e50f9386dbe554",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.014c5ffecc9dfaf760e50f9386dbe554/impacts"
                }
            },
            {
                "id": "productid.01ab075dcb09d0a15f06c92be787f564",
                "name": "Tesco Grain Vodka with Lime & Lemonade",
                "gtin": "5054268979368",
                "image": "https://static.euw1.sustained.app/products/images/5054268979368_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753957
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01ab075dcb09d0a15f06c92be787f564",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01ab075dcb09d0a15f06c92be787f564/impacts"
                }
            },
            {
                "id": "productid.01d4023b474494846b11cbe5ee9f9b99",
                "name": "Appleton Estate 8 Year Old Reserve",
                "gtin": "5024576207101",
                "image": "https://static.euw1.sustained.app/products/images/5024576207101_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753962
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01d4023b474494846b11cbe5ee9f9b99",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01d4023b474494846b11cbe5ee9f9b99/impacts"
                }
            },
            {
                "id": "productid.01e2bce37e5ccab6503423da15e31112",
                "name": "Bulleit Bourbon Frontier Whiskey",
                "gtin": "5000281038018",
                "image": "https://static.euw1.sustained.app/products/images/5000281038018_0.jpg",
                "pack": "0.700l",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01e2bce37e5ccab6503423da15e31112",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01e2bce37e5ccab6503423da15e31112/impacts"
                }
            },
            {
                "id": "productid.0220cff9c6e886115f1019660fb65e26",
                "name": "Glenfiddich Malt Master's Edition Single Malt Scotch Whisky",
                "gtin": "5010327325545",
                "image": "https://static.euw1.sustained.app/products/images/5010327325545_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0220cff9c6e886115f1019660fb65e26",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0220cff9c6e886115f1019660fb65e26/impacts"
                }
            },
            {
                "id": "productid.02e223a8b52a0dce38d7d8e548e02411",
                "name": "El Jimador Tequila Blanco",
                "gtin": "7501145268100",
                "image": "https://static.euw1.sustained.app/products/images/7501145268100_0.jpg",
                "pack": "70cl",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02e223a8b52a0dce38d7d8e548e02411",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02e223a8b52a0dce38d7d8e548e02411/impacts"
                }
            },
            {
                "id": "productid.037b5f1627af9731faa222dd8d0901ec",
                "name": "M&S Clover Club",
                "gtin": "0000029181476",
                "image": "https://static.euw1.sustained.app/products/images/0000029181476_0.jpg",
                "pack": "50cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753968
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.037b5f1627af9731faa222dd8d0901ec",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.037b5f1627af9731faa222dd8d0901ec/impacts"
                }
            },
            {
                "id": "productid.03e6316f13d10157e0e1b187a45d298f",
                "name": "Conker Cold Brew Coffee Liqueur",
                "gtin": "5060432770044",
                "image": "https://static.euw1.sustained.app/products/images/5060432770044_0.jpg",
                "pack": "70cl",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03e6316f13d10157e0e1b187a45d298f",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03e6316f13d10157e0e1b187a45d298f/impacts"
                }
            },
            {
                "id": "productid.05f0fb4cad5eda4319b937895078ee51",
                "name": "Rolov Mild Imperial Spirit",
                "gtin": "5016840004127",
                "image": "https://static.euw1.sustained.app/products/images/5016840004127_0.jpg",
                "pack": "35cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753962
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05f0fb4cad5eda4319b937895078ee51",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05f0fb4cad5eda4319b937895078ee51/impacts"
                }
            },
            {
                "id": "productid.06335a1eaf2f0aa93ad2f34a73630cdd",
                "name": "Jameson Wire Pack",
                "gtin": "5011007023058",
                "image": "https://static.euw1.sustained.app/products/images/5011007023058_0.jpg",
                "pack": "200ml",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06335a1eaf2f0aa93ad2f34a73630cdd",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06335a1eaf2f0aa93ad2f34a73630cdd/impacts"
                }
            },
            {
                "id": "productid.06686d9809a6e7ff5a750443e6d0e06b",
                "name": "Knob Creek Small Batch Kentucky Straight Bourbon Whiskey",
                "gtin": "5060045580184",
                "image": "https://static.euw1.sustained.app/products/images/5060045580184_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06686d9809a6e7ff5a750443e6d0e06b",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06686d9809a6e7ff5a750443e6d0e06b/impacts"
                }
            },
            {
                "id": "productid.066f22be4f9b2d7301aeb27d5116108f",
                "name": "Wyborowa Limited Edition Polish Vodka",
                "gtin": "5900685005244",
                "image": "https://static.euw1.sustained.app/products/images/5900685005244_0.jpg",
                "pack": "700ml",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753957
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.066f22be4f9b2d7301aeb27d5116108f",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.066f22be4f9b2d7301aeb27d5116108f/impacts"
                }
            },
            {
                "id": "productid.06706b97ff0a8db6659fc96bd26340c3",
                "name": "Dead Man's Fingers Coffee Rum",
                "gtin": "5011166056607",
                "image": "https://static.euw1.sustained.app/products/images/5011166056607_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753959
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06706b97ff0a8db6659fc96bd26340c3",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06706b97ff0a8db6659fc96bd26340c3/impacts"
                }
            },
            {
                "id": "productid.069c96d9ab5273681acc056cddab85f5",
                "name": "RedLeg Spiced Rum with Caramelised Pineapple",
                "gtin": "5032482000037",
                "image": "https://static.euw1.sustained.app/products/images/5032482000037_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753962
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.069c96d9ab5273681acc056cddab85f5",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.069c96d9ab5273681acc056cddab85f5/impacts"
                }
            },
            {
                "id": "productid.06d24f4d4b9a6d3d3e7aabee10b94018",
                "name": "Greenall's Wild Berry Gin",
                "gtin": "5010296004670",
                "image": "https://static.euw1.sustained.app/products/images/5010296004670_0.jpg",
                "pack": "1l",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753968
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06d24f4d4b9a6d3d3e7aabee10b94018",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06d24f4d4b9a6d3d3e7aabee10b94018/impacts"
                }
            },
            {
                "id": "productid.070c67dacce595e8e16a07b4e21cf957",
                "name": "Special Reserve Blended Scotch Whisky",
                "gtin": "5013967012318",
                "image": "https://static.euw1.sustained.app/products/images/5013967012318_0.jpg",
                "pack": "35cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.070c67dacce595e8e16a07b4e21cf957",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.070c67dacce595e8e16a07b4e21cf957/impacts"
                }
            },
            {
                "id": "productid.0777199ef282d70ebf88711f7cf4d38a",
                "name": "24 Ice Frozen Cocktails Strawberry Daiquiri Ice & Alcohol",
                "gtin": "8718868983039",
                "image": "https://static.euw1.sustained.app/products/images/8718868983039_0.jpg",
                "pack": "65ml",
                "grade": "B",
                "points": {
                    "pef": 13.61455246629356
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0777199ef282d70ebf88711f7cf4d38a",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0777199ef282d70ebf88711f7cf4d38a/impacts"
                }
            },
            {
                "id": "productid.07d8e191b9be576875ae3e0c229b6356",
                "name": "Black Cow Vodka English Strawberries",
                "gtin": "5060323760307",
                "image": "https://static.euw1.sustained.app/products/images/5060323760307_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753957
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07d8e191b9be576875ae3e0c229b6356",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07d8e191b9be576875ae3e0c229b6356/impacts"
                }
            },
            {
                "id": "productid.08368c261ed131066db5bef5db17ab81",
                "name": "Coleraine Irish Whiskey",
                "gtin": "5011007008017",
                "image": "https://static.euw1.sustained.app/products/images/5011007008017_0.jpg",
                "pack": "700ml",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.08368c261ed131066db5bef5db17ab81",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.08368c261ed131066db5bef5db17ab81/impacts"
                }
            },
            {
                "id": "productid.0862ebe9ebe15690e4d8e11d4f8ffbe2",
                "name": "Captain Morgan Dark Rum",
                "gtin": "5000281072418",
                "image": "https://static.euw1.sustained.app/products/images/5000281072418_0.jpg",
                "pack": "0.700l",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753962
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0862ebe9ebe15690e4d8e11d4f8ffbe2",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0862ebe9ebe15690e4d8e11d4f8ffbe2/impacts"
                }
            },
            {
                "id": "productid.093ceeef9ade324209b593c67e4f7931",
                "name": "Lagavulin 8 Year Old Single Malt Scotch Whisky",
                "gtin": "5000281045146",
                "image": "https://static.euw1.sustained.app/products/images/5000281045146_0.jpg",
                "pack": "0.700l",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.093ceeef9ade324209b593c67e4f7931",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.093ceeef9ade324209b593c67e4f7931/impacts"
                }
            },
            {
                "id": "productid.0a61b9effa1f435e9ad95a2643902b47",
                "name": "Glen's Strawberry & Apple Fruit Flavoured Spirit Drink",
                "gtin": "5016840168317",
                "image": "https://static.euw1.sustained.app/products/images/5016840168317_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753962
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a61b9effa1f435e9ad95a2643902b47",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a61b9effa1f435e9ad95a2643902b47/impacts"
                }
            },
            {
                "id": "productid.0bd44b6cb4b6eedac0bc05b565576e70",
                "name": "Rooster Rojo Tequila with Smoked Pineapple",
                "gtin": "7503023613767",
                "image": "https://static.euw1.sustained.app/products/images/7503023613767_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753957
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0bd44b6cb4b6eedac0bc05b565576e70",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0bd44b6cb4b6eedac0bc05b565576e70/impacts"
                }
            },
            {
                "id": "productid.0d78a0cecb064b3577a8fdea5afd170b",
                "name": "Bombay Bramble Distilled Gin Gift Box",
                "gtin": "7640175742775",
                "image": "https://static.euw1.sustained.app/products/images/7640175742775_0.jpg",
                "pack": "700ml",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753968
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0d78a0cecb064b3577a8fdea5afd170b",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0d78a0cecb064b3577a8fdea5afd170b/impacts"
                }
            },
            {
                "id": "productid.0de713a995eb59ca428bc1f15e7b9cf2",
                "name": "Fair Spirits Gin",
                "gtin": "3760227760060",
                "image": "https://static.euw1.sustained.app/products/images/3760227760060_0.jpg",
                "pack": "50cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753968
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0de713a995eb59ca428bc1f15e7b9cf2",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0de713a995eb59ca428bc1f15e7b9cf2/impacts"
                }
            },
            {
                "id": "productid.0e1918da5532f243daf8b8f679db37de",
                "name": "Boulard Calvados Pays d'Auge Grand Solage",
                "gtin": "3152900001155",
                "image": "https://static.euw1.sustained.app/products/images/3152900001155_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753962
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e1918da5532f243daf8b8f679db37de",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e1918da5532f243daf8b8f679db37de/impacts"
                }
            },
            {
                "id": "productid.0e50b3d53546d0e5c7416c8d16abedd2",
                "name": "Captain Morgan Dark Rum",
                "gtin": "5000281071145",
                "image": "https://static.euw1.sustained.app/products/images/5000281071145_0.jpg",
                "pack": "0.350l",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753962
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e50b3d53546d0e5c7416c8d16abedd2",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e50b3d53546d0e5c7416c8d16abedd2/impacts"
                }
            },
            {
                "id": "productid.0e7abf01ef25d3e9fac1cb6a7e6a8936",
                "name": "Dewar's Japanese Smooth Blended Scotch Whisky",
                "gtin": "7640171038001",
                "image": "https://static.euw1.sustained.app/products/images/7640171038001_0.jpg",
                "pack": "700ml",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0e7abf01ef25d3e9fac1cb6a7e6a8936",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0e7abf01ef25d3e9fac1cb6a7e6a8936/impacts"
                }
            },
            {
                "id": "productid.0eb6a5714bb25104409bac1a11657bd7",
                "name": "Jura French Oak Single Malt Whisky",
                "gtin": "5013967015388",
                "image": "https://static.euw1.sustained.app/products/images/5013967015388_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0eb6a5714bb25104409bac1a11657bd7",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0eb6a5714bb25104409bac1a11657bd7/impacts"
                }
            },
            {
                "id": "productid.0f2cd620bfea08be7601d80279013c72",
                "name": "Jamaica Cove Black Spiced Rum Infused with Pineapple",
                "gtin": "5060119770428",
                "image": "https://static.euw1.sustained.app/products/images/5060119770428_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753962
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0f2cd620bfea08be7601d80279013c72",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0f2cd620bfea08be7601d80279013c72/impacts"
                }
            },
            {
                "id": "productid.0fccc7d9d8657dadf2e1baf25073f4a3",
                "name": "Hennessy Very Special Cognac",
                "gtin": "3245990250005",
                "image": "https://static.euw1.sustained.app/products/images/3245990250005_0.jpg",
                "pack": "1.50l",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753962
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0fccc7d9d8657dadf2e1baf25073f4a3",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0fccc7d9d8657dadf2e1baf25073f4a3/impacts"
                }
            },
            {
                "id": "productid.102e0b00fbc4eb48e7885036c8d39353",
                "name": "Jack Daniel's Legacy 3 Ltd Edition Tennessee Whiskey",
                "gtin": "5099873018583",
                "image": "https://static.euw1.sustained.app/products/images/5099873018583_0.jpg",
                "pack": "700ml",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.102e0b00fbc4eb48e7885036c8d39353",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.102e0b00fbc4eb48e7885036c8d39353/impacts"
                }
            },
            {
                "id": "productid.105ec831dbcfc9f93e0a4633c98b6009",
                "name": "Glendalough Double Barrel Irish Whiskey",
                "gtin": "0799439682160",
                "image": "https://static.euw1.sustained.app/products/images/0799439682160_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.105ec831dbcfc9f93e0a4633c98b6009",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.105ec831dbcfc9f93e0a4633c98b6009/impacts"
                }
            },
            {
                "id": "productid.10c10becad900357e272176ce4c5ce71",
                "name": "BREWDOG DISTILLING CO Seven Day Vodka",
                "gtin": "5056025460218",
                "image": "https://static.euw1.sustained.app/products/images/5056025460218_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753957
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.10c10becad900357e272176ce4c5ce71",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.10c10becad900357e272176ce4c5ce71/impacts"
                }
            },
            {
                "id": "productid.1205b0fa8ddf920896f63d12afc2a824",
                "name": "Togouchi Premium Japanese Blended Whisky",
                "gtin": "4901903064105",
                "image": "https://static.euw1.sustained.app/products/images/4901903064105_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1205b0fa8ddf920896f63d12afc2a824",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1205b0fa8ddf920896f63d12afc2a824/impacts"
                }
            },
            {
                "id": "productid.1277a2e3c3031923568c4e3f7847d4b3",
                "name": "High Commissioner Blended Scotch Whisky",
                "gtin": "5016840000129",
                "image": "https://static.euw1.sustained.app/products/images/5016840000129_0.jpg",
                "pack": "35cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.1277a2e3c3031923568c4e3f7847d4b3",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.1277a2e3c3031923568c4e3f7847d4b3/impacts"
                }
            },
            {
                "id": "productid.127d69401bc7594712652f34432e7bb9",
                "name": "Malibu Original White Rum with Coconut Flavour",
                "gtin": "5010284100056",
                "image": "https://static.euw1.sustained.app/products/images/5010284100056_0.jpg",
                "pack": "35cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753959
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.127d69401bc7594712652f34432e7bb9",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.127d69401bc7594712652f34432e7bb9/impacts"
                }
            },
            {
                "id": "productid.129f949c3f5e70919034f15f374b6549",
                "name": "Adnams Southwold Single Malt Whisky",
                "gtin": "5016878600605",
                "image": "https://static.euw1.sustained.app/products/images/5016878600605_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.129f949c3f5e70919034f15f374b6549",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.129f949c3f5e70919034f15f374b6549/impacts"
                }
            },
            {
                "id": "productid.12f07fc393a12fe47ce90f9d6839b1f1",
                "name": "Highclere Castle London Dry Gin",
                "gtin": "0702669708340",
                "image": "https://static.euw1.sustained.app/products/images/0702669708340_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753968
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.12f07fc393a12fe47ce90f9d6839b1f1",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.12f07fc393a12fe47ce90f9d6839b1f1/impacts"
                }
            },
            {
                "id": "productid.133688025372e4f8331e57ea62c15e82",
                "name": "Expertly Blended for M&S Flavoured Vodka Selection",
                "gtin": "0000000942836",
                "image": "https://static.euw1.sustained.app/products/images/0000000942836_0.jpg",
                "pack": "5cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753957
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.133688025372e4f8331e57ea62c15e82",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.133688025372e4f8331e57ea62c15e82/impacts"
                }
            },
            {
                "id": "productid.142a7472d21a582ce3cf83dc008de884",
                "name": "Jack Daniel's Tennessee Whiskey Blended with Honey Liqueur",
                "gtin": "5099873009116",
                "image": "https://static.euw1.sustained.app/products/images/5099873009116_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.142a7472d21a582ce3cf83dc008de884",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.142a7472d21a582ce3cf83dc008de884/impacts"
                }
            },
            {
                "id": "productid.144fa11c52c8baa30a81aacc5658a12f",
                "name": "Hennessy Very Special Cognac",
                "gtin": "3245990250302",
                "image": "https://static.euw1.sustained.app/products/images/3245990250302_0.jpg",
                "pack": "35cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753962
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.144fa11c52c8baa30a81aacc5658a12f",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.144fa11c52c8baa30a81aacc5658a12f/impacts"
                }
            },
            {
                "id": "productid.14bfec93063909106e52b7c0101fe894",
                "name": "Cotswolds Distillery",
                "gtin": "5060404511293",
                "image": "https://static.euw1.sustained.app/products/images/5060404511293_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753982
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14bfec93063909106e52b7c0101fe894",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14bfec93063909106e52b7c0101fe894/impacts"
                }
            },
            {
                "id": "productid.14ccf7d0001e46aba3d16aeae194aabb",
                "name": "Eden Mill St Andrews Golf Gin",
                "gtin": "5060334032011",
                "image": "https://static.euw1.sustained.app/products/images/5060334032011_0.jpg",
                "pack": "50cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753968
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.14ccf7d0001e46aba3d16aeae194aabb",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.14ccf7d0001e46aba3d16aeae194aabb/impacts"
                }
            },
            {
                "id": "productid.150b1a7c5b1124e02ade533ab6ff71dc",
                "name": "Whitley Neill Limited Edition Mango & Lime Handcrafted Gin",
                "gtin": "5011166064718",
                "image": "https://static.euw1.sustained.app/products/images/5011166064718_0.jpg",
                "pack": "70cl",
                "grade": "B",
                "points": {
                    "pef": 12.029173771753968
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.150b1a7c5b1124e02ade533ab6ff71dc",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.150b1a7c5b1124e02ade533ab6ff71dc/impacts"
                }
            },
            {
                "id": "productid.150e9453c9578a3087d0e54ba08a7a68",
                "name": "Pernod Aniseed Liqueur",
                "gtin": "3047100090316",
                "image": "https://static.euw1.sustained.app/products/images/3047100090316_0.jpg",
                "pack": "70cl",
                "grade": "D",
                "points": {
                    "pef": 24.875937288789206
                },
                "info_icons": [],
                "category": "Spirit & Liqueurs",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.150e9453c9578a3087d0e54ba08a7a68",
                    "category": "https://api.sustained.com/choice/v1/categories/spirit-and-liqueurs",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.150e9453c9578a3087d0e54ba08a7a68/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.0021d9b815577f6545ae604f982e8850",
                "name": "Salad Cream",
                "gtin": "5060154035520",
                "image": "https://static.euw1.sustained.app/products/images/5060154035520_0.jpg",
                "pack": "2.27l",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0021d9b815577f6545ae604f982e8850",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0021d9b815577f6545ae604f982e8850/impacts"
                }
            },
            {
                "id": "productid.0054646071e4fd372fb6f94c1b08a3be",
                "name": "Dr. Will's All Natural BBQ Sauce",
                "gtin": "0793052122713",
                "image": "https://static.euw1.sustained.app/products/images/0793052122713_0.jpg",
                "pack": "250g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0054646071e4fd372fb6f94c1b08a3be",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0054646071e4fd372fb6f94c1b08a3be/impacts"
                }
            },
            {
                "id": "productid.00624fa6d2e0d8ebbb9f28b8eeeceb1a",
                "name": "Organic Olive Ghee",
                "gtin": "5032722317284",
                "image": "https://static.euw1.sustained.app/products/images/5032722317284_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 85.29402242792136
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00624fa6d2e0d8ebbb9f28b8eeeceb1a",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00624fa6d2e0d8ebbb9f28b8eeeceb1a/impacts"
                }
            },
            {
                "id": "productid.007e32014e6557384adfe3bda38e13df",
                "name": "Rogan Josh Sauce",
                "gtin": "5010525091433",
                "image": "https://static.euw1.sustained.app/products/images/5010525091433_0.jpg",
                "pack": "500g",
                "grade": "B",
                "points": {
                    "pef": 11.21503662961149
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.007e32014e6557384adfe3bda38e13df",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.007e32014e6557384adfe3bda38e13df/impacts"
                }
            },
            {
                "id": "productid.008e71f2c2a062d77a596508f0f23d42",
                "name": "Pesto Sun-Dried Tomato",
                "gtin": "8001060039957",
                "image": "https://static.euw1.sustained.app/products/images/8001060039957_0.jpg",
                "pack": "190g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.008e71f2c2a062d77a596508f0f23d42",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.008e71f2c2a062d77a596508f0f23d42/impacts"
                }
            },
            {
                "id": "productid.009b7c275899ca2c133409e2d2464956",
                "name": "Sweet Soy Sauce",
                "gtin": "0078895147308",
                "image": "https://static.euw1.sustained.app/products/images/0078895147308_0.jpg",
                "pack": "207ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.009b7c275899ca2c133409e2d2464956",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.009b7c275899ca2c133409e2d2464956/impacts"
                }
            },
            {
                "id": "productid.00a12197417f31240274641ae7f7fa83",
                "name": "White Miso",
                "gtin": "5014276701139",
                "image": "https://static.euw1.sustained.app/products/images/5014276701139_0.jpg",
                "pack": "100g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00a12197417f31240274641ae7f7fa83",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00a12197417f31240274641ae7f7fa83/impacts"
                }
            },
            {
                "id": "productid.00a1b68721eb25daefc52752e8580bcd",
                "name": "Smokey Baconnaise",
                "gtin": "5000157076649",
                "image": "https://static.euw1.sustained.app/products/images/5000157076649_0.jpg",
                "pack": "880g",
                "grade": "D",
                "points": {
                    "pef": 28.113349216159165
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00a1b68721eb25daefc52752e8580bcd",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00a1b68721eb25daefc52752e8580bcd/impacts"
                }
            },
            {
                "id": "productid.011cdfabb8c5ad5e39de6757edc83b0a",
                "name": "Cooks' Ingredients Garam Masala",
                "gtin": "5000169625156",
                "image": "https://static.euw1.sustained.app/products/images/5000169625156_0.jpg",
                "pack": "38g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.011cdfabb8c5ad5e39de6757edc83b0a",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.011cdfabb8c5ad5e39de6757edc83b0a/impacts"
                }
            },
            {
                "id": "productid.01211121ed41bc68bd3866a99b3dc17f",
                "name": "Bramley Apple Sauce",
                "gtin": "5099229002013",
                "image": "https://static.euw1.sustained.app/products/images/5099229002013_0.jpg",
                "pack": "1.3kg",
                "grade": "A",
                "points": {
                    "pef": 8.263238889181993
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01211121ed41bc68bd3866a99b3dc17f",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01211121ed41bc68bd3866a99b3dc17f/impacts"
                }
            },
            {
                "id": "productid.013c39fc8b87df39af2be0608210e34c",
                "name": "Christmas Ham Seasoning",
                "gtin": "0000000368919",
                "image": "https://static.euw1.sustained.app/products/images/0000000368919_0.jpg",
                "pack": "45g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.013c39fc8b87df39af2be0608210e34c",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.013c39fc8b87df39af2be0608210e34c/impacts"
                }
            },
            {
                "id": "productid.01c3751a76bb68c82f0f5e2b083200c3",
                "name": "Peri-Peri Marinade Mango & Lime",
                "gtin": "6003770007587",
                "image": "https://static.euw1.sustained.app/products/images/6003770007587_0.jpg",
                "pack": "270g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01c3751a76bb68c82f0f5e2b083200c3",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01c3751a76bb68c82f0f5e2b083200c3/impacts"
                }
            },
            {
                "id": "productid.01c7b899aeb9ea2f55129941525aaba3",
                "name": "Garlic Mayo",
                "gtin": "5391518711126",
                "image": "https://static.euw1.sustained.app/products/images/5391518711126_0.jpg",
                "pack": "450ml",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01c7b899aeb9ea2f55129941525aaba3",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01c7b899aeb9ea2f55129941525aaba3/impacts"
                }
            },
            {
                "id": "productid.01f00a8c3a1efb8e4c999b39cf662f77",
                "name": "Natural Fine Sea Salt",
                "gtin": "0794711000038",
                "image": "https://static.euw1.sustained.app/products/images/0794711000038_0.jpg",
                "pack": "500g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.01f00a8c3a1efb8e4c999b39cf662f77",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.01f00a8c3a1efb8e4c999b39cf662f77/impacts"
                }
            },
            {
                "id": "productid.02097f11e5e636c30bb5af771beebe85",
                "name": "Extra Virgin Olive Oil",
                "gtin": "5057172338689",
                "image": "https://static.euw1.sustained.app/products/images/5057172338689_0.jpg",
                "pack": "250ml",
                "grade": "F",
                "points": {
                    "pef": 62.773641301787386
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02097f11e5e636c30bb5af771beebe85",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02097f11e5e636c30bb5af771beebe85/impacts"
                }
            },
            {
                "id": "productid.020de543d9de3da941317da470f21e7d",
                "name": "Harissa Seasoning",
                "gtin": "0000003282304",
                "image": "https://static.euw1.sustained.app/products/images/0000003282304_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 22.91519445576034
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.020de543d9de3da941317da470f21e7d",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.020de543d9de3da941317da470f21e7d/impacts"
                }
            },
            {
                "id": "productid.02364a8423dd90ccb26d23eb6753b741",
                "name": "Aceto Balsamico",
                "gtin": "8022717102886",
                "image": "https://static.euw1.sustained.app/products/images/8022717102886_0.jpg",
                "pack": "0.250l",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02364a8423dd90ccb26d23eb6753b741",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02364a8423dd90ccb26d23eb6753b741/impacts"
                }
            },
            {
                "id": "productid.025b0a914ed7bdddd2e402f601c74694",
                "name": "Cold Pressed Rapeseed Oil Blended with Jalape\u00f1o Chilli",
                "gtin": "5060239130058",
                "image": "https://static.euw1.sustained.app/products/images/5060239130058_0.jpg",
                "pack": "250ml",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.025b0a914ed7bdddd2e402f601c74694",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.025b0a914ed7bdddd2e402f601c74694/impacts"
                }
            },
            {
                "id": "productid.027f76a5b30b0d424cd3857147a3a587",
                "name": "Jalfrezi Batch-Crafted Spice Mix",
                "gtin": "5060579990053",
                "image": "https://static.euw1.sustained.app/products/images/5060579990053_0.jpg",
                "pack": "80g",
                "grade": "E",
                "points": {
                    "pef": 32.38469397804565
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.027f76a5b30b0d424cd3857147a3a587",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.027f76a5b30b0d424cd3857147a3a587/impacts"
                }
            },
            {
                "id": "productid.028a2598bc7539985b1ed9d2c7ab351d",
                "name": "Sun Dried Tomato Puree with Oil, Herbs and Spices",
                "gtin": "8001830533920",
                "image": "https://static.euw1.sustained.app/products/images/8001830533920_0.jpg",
                "pack": "80g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.028a2598bc7539985b1ed9d2c7ab351d",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.028a2598bc7539985b1ed9d2c7ab351d/impacts"
                }
            },
            {
                "id": "productid.02a588231f56d5017758b8e40608dcdd",
                "name": "Organic Plum Tomatoes",
                "gtin": "0628110454478",
                "image": "https://static.euw1.sustained.app/products/images/0628110454478_0.jpg",
                "pack": "400g",
                "grade": "A",
                "points": {
                    "pef": 9.067646966410383
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02a588231f56d5017758b8e40608dcdd",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02a588231f56d5017758b8e40608dcdd/impacts"
                }
            },
            {
                "id": "productid.02cd57cf8d3c0173978e055077b53b70",
                "name": "White Pepper Ground",
                "gtin": "5410531547939",
                "image": "https://static.euw1.sustained.app/products/images/5410531547939_0.jpg",
                "pack": "43g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02cd57cf8d3c0173978e055077b53b70",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02cd57cf8d3c0173978e055077b53b70/impacts"
                }
            },
            {
                "id": "productid.03013127ead77a439d8382b8e6dba5fa",
                "name": "Medium Curry Powder",
                "gtin": "50197637",
                "image": "https://static.euw1.sustained.app/products/images/50197637_0.jpg",
                "pack": "102g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03013127ead77a439d8382b8e6dba5fa",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03013127ead77a439d8382b8e6dba5fa/impacts"
                }
            },
            {
                "id": "productid.032b3acb803cbb6b16c1d9b41ac5862b",
                "name": "Sun-Dried Tomato Pur\u00e9e with Onion & Garlic",
                "gtin": "0000050449323",
                "image": "https://static.euw1.sustained.app/products/images/0000050449323_0.jpg",
                "pack": "90g",
                "grade": "B",
                "points": {
                    "pef": 15.73678105513983
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.032b3acb803cbb6b16c1d9b41ac5862b",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.032b3acb803cbb6b16c1d9b41ac5862b/impacts"
                }
            },
            {
                "id": "productid.034f5fe06de5625a973319e5cc07e54d",
                "name": "Mango and Habanero Hot Sauce",
                "gtin": "5060183367104",
                "image": "https://static.euw1.sustained.app/products/images/5060183367104_0.jpg",
                "pack": "100ml",
                "grade": "C",
                "points": {
                    "pef": 17.63162440298512
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.034f5fe06de5625a973319e5cc07e54d",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.034f5fe06de5625a973319e5cc07e54d/impacts"
                }
            },
            {
                "id": "productid.03774175ad88fb02194f773ab71d0e36",
                "name": "Toasted Sesame Oil",
                "gtin": "0000000804998",
                "image": "https://static.euw1.sustained.app/products/images/0000000804998_0.jpg",
                "pack": "250ml",
                "grade": "F",
                "points": {
                    "pef": 51.98228494808682
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03774175ad88fb02194f773ab71d0e36",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03774175ad88fb02194f773ab71d0e36/impacts"
                }
            },
            {
                "id": "productid.038334af9763abfe25b1e7070686011c",
                "name": "Cranberry Sauce",
                "gtin": "5010421074141",
                "image": "https://static.euw1.sustained.app/products/images/5010421074141_0.jpg",
                "pack": "2.6kg",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.038334af9763abfe25b1e7070686011c",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.038334af9763abfe25b1e7070686011c/impacts"
                }
            },
            {
                "id": "productid.03ac71974b59a9f8697ec2b9da03e8d7",
                "name": "The Best Italian Passata",
                "gtin": "5010525015699",
                "image": "https://static.euw1.sustained.app/products/images/5010525015699_0.jpg",
                "pack": "430g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03ac71974b59a9f8697ec2b9da03e8d7",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03ac71974b59a9f8697ec2b9da03e8d7/impacts"
                }
            },
            {
                "id": "productid.03b0f80af59e8cf00b488e23f1e04647",
                "name": "P Smoked Paprika",
                "gtin": "5010204914442",
                "image": "https://static.euw1.sustained.app/products/images/5010204914442_0.jpg",
                "pack": "",
                "grade": "D",
                "points": {
                    "pef": 27.753545398808697
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03b0f80af59e8cf00b488e23f1e04647",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03b0f80af59e8cf00b488e23f1e04647/impacts"
                }
            },
            {
                "id": "productid.03bf0aad87c4924a30e9f8dfb1f29c9e",
                "name": "Season + Shake Paprika Chicken",
                "gtin": "8718114885414",
                "image": "https://static.euw1.sustained.app/products/images/8718114885414_0.jpg",
                "pack": "24g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03bf0aad87c4924a30e9f8dfb1f29c9e",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03bf0aad87c4924a30e9f8dfb1f29c9e/impacts"
                }
            },
            {
                "id": "productid.03f782fb3e83cf908417c89818e2b715",
                "name": "Reduced Fat Houmous",
                "gtin": "5391510683216",
                "image": "https://static.euw1.sustained.app/products/images/5391510683216_0.jpg",
                "pack": "300g",
                "grade": "B",
                "points": {
                    "pef": 14.471486963871067
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03f782fb3e83cf908417c89818e2b715",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03f782fb3e83cf908417c89818e2b715/impacts"
                }
            },
            {
                "id": "productid.043803d055f4b9edf71987956e10d0cf",
                "name": "Creamy Tomato Pasta Bake",
                "gtin": "5010251460268",
                "image": "https://static.euw1.sustained.app/products/images/5010251460268_0.jpg",
                "pack": "500g",
                "grade": "B",
                "points": {
                    "pef": 15.146653786306352
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.043803d055f4b9edf71987956e10d0cf",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.043803d055f4b9edf71987956e10d0cf/impacts"
                }
            },
            {
                "id": "productid.04622b333e014ebc664347799fce8139",
                "name": "Classic Barbecue Sauce",
                "gtin": "0000050457878",
                "image": "https://static.euw1.sustained.app/products/images/0000050457878_0.jpg",
                "pack": "480g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04622b333e014ebc664347799fce8139",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04622b333e014ebc664347799fce8139/impacts"
                }
            },
            {
                "id": "productid.0468b6baa37d71c53a2b856ae7b98e56",
                "name": "The Best Italian Passata with Spanish Peppers",
                "gtin": "5010525015668",
                "image": "https://static.euw1.sustained.app/products/images/5010525015668_0.jpg",
                "pack": "430g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0468b6baa37d71c53a2b856ae7b98e56",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0468b6baa37d71c53a2b856ae7b98e56/impacts"
                }
            },
            {
                "id": "productid.047199944ba2ceaaf10be52a3deae552",
                "name": "Organic Virgin Coconut Oil Infused with Ginger",
                "gtin": "5038695000567",
                "image": "https://static.euw1.sustained.app/products/images/5038695000567_0.jpg",
                "pack": "283ml",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.047199944ba2ceaaf10be52a3deae552",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.047199944ba2ceaaf10be52a3deae552/impacts"
                }
            },
            {
                "id": "productid.04ab5ee2f6f405a08b1950af9876b03c",
                "name": "Rapeseed Oil",
                "gtin": "5000232029546",
                "image": "https://static.euw1.sustained.app/products/images/5000232029546_0.jpg",
                "pack": "1l",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04ab5ee2f6f405a08b1950af9876b03c",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04ab5ee2f6f405a08b1950af9876b03c/impacts"
                }
            },
            {
                "id": "productid.04ea1b815eb7a6c2910ec0c662a44dbd",
                "name": "White Wine & Mushroom Sauce",
                "gtin": "5018624001711",
                "image": "https://static.euw1.sustained.app/products/images/5018624001711_0.jpg",
                "pack": "250g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.04ea1b815eb7a6c2910ec0c662a44dbd",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.04ea1b815eb7a6c2910ec0c662a44dbd/impacts"
                }
            },
            {
                "id": "productid.05139a73d01fcc0ec0c0d3641879fa29",
                "name": "Taste the Difference Summer Edition Sriracha Style Rippled Houmous",
                "gtin": "00460088",
                "image": "https://static.euw1.sustained.app/products/images/00460088_0.jpg",
                "pack": "200g",
                "grade": "D",
                "points": {
                    "pef": 25.80953129746936
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05139a73d01fcc0ec0c0d3641879fa29",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05139a73d01fcc0ec0c0d3641879fa29/impacts"
                }
            },
            {
                "id": "productid.0518c5e9a252dcb838f7428c35018efe",
                "name": "Harissa Paste",
                "gtin": "5052320703883",
                "image": "https://static.euw1.sustained.app/products/images/5052320703883_0.jpg",
                "pack": "90g",
                "grade": "B",
                "points": {
                    "pef": 12.50911629445234
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0518c5e9a252dcb838f7428c35018efe",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0518c5e9a252dcb838f7428c35018efe/impacts"
                }
            },
            {
                "id": "productid.053737343c76f3f4107dbaaecf8ca487",
                "name": "Peppercorn Sauce",
                "gtin": "5060161210330",
                "image": "https://static.euw1.sustained.app/products/images/5060161210330_0.jpg",
                "pack": "250g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.053737343c76f3f4107dbaaecf8ca487",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.053737343c76f3f4107dbaaecf8ca487/impacts"
                }
            },
            {
                "id": "productid.05541fefee52037b3c91a5d6a32150d6",
                "name": "Be Good to Yourself Mini Houmous Snack Pots",
                "gtin": "01299182",
                "image": "https://static.euw1.sustained.app/products/images/01299182_0.jpg",
                "pack": "60g",
                "grade": "C",
                "points": {
                    "pef": 17.90673451862976
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05541fefee52037b3c91a5d6a32150d6",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05541fefee52037b3c91a5d6a32150d6/impacts"
                }
            },
            {
                "id": "productid.056c3ddd5db7d2a2086ca126f678c511",
                "name": "Onion & Garlic",
                "gtin": "5054781579007",
                "image": "https://static.euw1.sustained.app/products/images/5054781579007_0.jpg",
                "pack": "200g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.056c3ddd5db7d2a2086ca126f678c511",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.056c3ddd5db7d2a2086ca126f678c511/impacts"
                }
            },
            {
                "id": "productid.059212c1e40b9c4a18ab726c1bb83d8d",
                "name": "Mustard Dijon",
                "gtin": "0000000738460",
                "image": "https://static.euw1.sustained.app/products/images/0000000738460_0.jpg",
                "pack": "185g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.059212c1e40b9c4a18ab726c1bb83d8d",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.059212c1e40b9c4a18ab726c1bb83d8d/impacts"
                }
            },
            {
                "id": "productid.059477a02a53e1a8e77fe14b758216bc",
                "name": "Essential Waitrose & Partners Light in Colour Olive Oil Composed of Refined Olive",
                "gtin": "5000169627495",
                "image": "https://static.euw1.sustained.app/products/images/5000169627495_0.jpg",
                "pack": "1l",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.059477a02a53e1a8e77fe14b758216bc",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.059477a02a53e1a8e77fe14b758216bc/impacts"
                }
            },
            {
                "id": "productid.05abd77acfcbe33aa230fbf000d60772",
                "name": "Soured Cream & Chive",
                "gtin": "0000010066959",
                "image": "https://static.euw1.sustained.app/products/images/0000010066959_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05abd77acfcbe33aa230fbf000d60772",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05abd77acfcbe33aa230fbf000d60772/impacts"
                }
            },
            {
                "id": "productid.05b409802c97de10b0ea3b08b43ad226",
                "name": "Reduced Fat Onion & Garlic",
                "gtin": "03264799",
                "image": "https://static.euw1.sustained.app/products/images/03264799_0.jpg",
                "pack": "200g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05b409802c97de10b0ea3b08b43ad226",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05b409802c97de10b0ea3b08b43ad226/impacts"
                }
            },
            {
                "id": "productid.05f0d479a0c91e34de4bc0d22a80a4b5",
                "name": "Sriracha Hot Chilli Sauce",
                "gtin": "0087666052789",
                "image": "https://static.euw1.sustained.app/products/images/0087666052789_0.jpg",
                "pack": "200ml",
                "grade": "C",
                "points": {
                    "pef": 20.70802161308786
                },
                "info_icons": [],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05f0d479a0c91e34de4bc0d22a80a4b5",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05f0d479a0c91e34de4bc0d22a80a4b5/impacts"
                }
            },
            {
                "id": "productid.0662e3e9b6e25a257e7b5840dae72d58",
                "name": "Piri Piri",
                "gtin": "5010251876366",
                "image": "https://static.euw1.sustained.app/products/images/5010251876366_0.jpg",
                "pack": "48g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0662e3e9b6e25a257e7b5840dae72d58",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0662e3e9b6e25a257e7b5840dae72d58/impacts"
                }
            },
            {
                "id": "productid.068c280b45389f0431faaefcaf6f7f31",
                "name": "Chef's Favourites Smoky BBQ Sauce",
                "gtin": "5000354400988",
                "image": "https://static.euw1.sustained.app/products/images/5000354400988_0.jpg",
                "pack": "2kg",
                "grade": "D",
                "points": {
                    "pef": 30.436090548699436
                },
                "info_icons": [
                    {
                        "name": "Somewhat Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/somewhat-recyclable.svg",
                        "description": "This product\u2019s packaging is somewhat recyclable. Please check with your local council to find out where!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.068c280b45389f0431faaefcaf6f7f31",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.068c280b45389f0431faaefcaf6f7f31/impacts"
                }
            },
            {
                "id": "productid.06a3bbc1a6025998c8be63e02ec39831",
                "name": "Mild Curry Powder",
                "gtin": "5410531544716",
                "image": "https://static.euw1.sustained.app/products/images/5410531544716_0.jpg",
                "pack": "39g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Table Sauces & Condiments",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06a3bbc1a6025998c8be63e02ec39831",
                    "category": "https://api.sustained.com/choice/v1/categories/table-sauces-and-condiments",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06a3bbc1a6025998c8be63e02ec39831/impacts"
                }
            }
        ],
        [
            {
                "id": "productid.00e46e0152fc38b7e8aa580f6a2765e6",
                "name": "Cider Vinegar",
                "gtin": "0000000793933",
                "image": "https://static.euw1.sustained.app/products/images/0000000793933_0.jpg",
                "pack": "250ml",
                "grade": "B",
                "points": {
                    "pef": 12.247481479088183
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.00e46e0152fc38b7e8aa580f6a2765e6",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.00e46e0152fc38b7e8aa580f6a2765e6/impacts"
                }
            },
            {
                "id": "productid.0179c72f67329c6b5b25275286a4e830",
                "name": "Cannellini Beans",
                "gtin": "5010251958130",
                "image": "https://static.euw1.sustained.app/products/images/5010251958130_0.jpg",
                "pack": "400g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0179c72f67329c6b5b25275286a4e830",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0179c72f67329c6b5b25275286a4e830/impacts"
                }
            },
            {
                "id": "productid.0201e683dc08eb1a41106b1da53c597f",
                "name": "Tesco Custard",
                "gtin": "5052109886455",
                "image": "https://static.euw1.sustained.app/products/images/5052109886455_0.jpg",
                "pack": "",
                "grade": "C",
                "points": {
                    "pef": 18.002449857052667
                },
                "info_icons": [],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0201e683dc08eb1a41106b1da53c597f",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0201e683dc08eb1a41106b1da53c597f/impacts"
                }
            },
            {
                "id": "productid.0242775c9898909fed416a6c5e39003c",
                "name": "Peeled Plum Tomatoes in a Rich Tomato Juice",
                "gtin": "5010061242573",
                "image": "https://static.euw1.sustained.app/products/images/5010061242573_0.jpg",
                "pack": "400g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0242775c9898909fed416a6c5e39003c",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0242775c9898909fed416a6c5e39003c/impacts"
                }
            },
            {
                "id": "productid.0256362290ec081132b9c7bf960e863f",
                "name": "Tuna Chunks in Sunflower Oil",
                "gtin": "5000171058058",
                "image": "https://static.euw1.sustained.app/products/images/5000171058058_0.jpg",
                "pack": "132g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0256362290ec081132b9c7bf960e863f",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0256362290ec081132b9c7bf960e863f/impacts"
                }
            },
            {
                "id": "productid.026ac68421b8cc4be67db0ad356aca3d",
                "name": "Mixed Vegetables in Water",
                "gtin": "5000232022127",
                "image": "https://static.euw1.sustained.app/products/images/5000232022127_0.jpg",
                "pack": "400g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.026ac68421b8cc4be67db0ad356aca3d",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.026ac68421b8cc4be67db0ad356aca3d/impacts"
                }
            },
            {
                "id": "productid.02827a570daafdf201322f4b799def7a",
                "name": "Green Label Mango Chutney",
                "gtin": "5000354923012",
                "image": "https://static.euw1.sustained.app/products/images/5000354923012_0.jpg",
                "pack": "227g",
                "grade": "D",
                "points": {
                    "pef": 26.462409694015847
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.02827a570daafdf201322f4b799def7a",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.02827a570daafdf201322f4b799def7a/impacts"
                }
            },
            {
                "id": "productid.030ac8042e1fc8069f6c8d5617549e97",
                "name": "Tuna Chunks in Spring Water",
                "gtin": "5000171055026",
                "image": "https://static.euw1.sustained.app/products/images/5000171055026_0.jpg",
                "pack": "145g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.030ac8042e1fc8069f6c8d5617549e97",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.030ac8042e1fc8069f6c8d5617549e97/impacts"
                }
            },
            {
                "id": "productid.032282e4f722dda160f57677e2c81b0b",
                "name": "Sauerkraut Fermented Cabbage",
                "gtin": "5012427642102",
                "image": "https://static.euw1.sustained.app/products/images/5012427642102_0.jpg",
                "pack": "480g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.032282e4f722dda160f57677e2c81b0b",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.032282e4f722dda160f57677e2c81b0b/impacts"
                }
            },
            {
                "id": "productid.03aea2f24e36ccb2f2a26901b861e847",
                "name": "Marinated Mushrooms",
                "gtin": "5901871000128",
                "image": "https://static.euw1.sustained.app/products/images/5901871000128_0.jpg",
                "pack": "290g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03aea2f24e36ccb2f2a26901b861e847",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03aea2f24e36ccb2f2a26901b861e847/impacts"
                }
            },
            {
                "id": "productid.03e94ec6b9c57e3f47379fb721e6e5f1",
                "name": "Baked Beans in Tasty Tomato Sauce",
                "gtin": "01226010",
                "image": "https://static.euw1.sustained.app/products/images/01226010_0.jpg",
                "pack": "400g",
                "grade": "A",
                "points": {
                    "pef": 10.629110435060102
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.03e94ec6b9c57e3f47379fb721e6e5f1",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.03e94ec6b9c57e3f47379fb721e6e5f1/impacts"
                }
            },
            {
                "id": "productid.040315601e7becbe49a0c5fa0c98e78c",
                "name": "Essential Waitrose & Partners Mixed Bean Salad in Water",
                "gtin": "5000169043141",
                "image": "https://static.euw1.sustained.app/products/images/5000169043141_0.jpg",
                "pack": "400g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.040315601e7becbe49a0c5fa0c98e78c",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.040315601e7becbe49a0c5fa0c98e78c/impacts"
                }
            },
            {
                "id": "productid.0597e1ccb7ea23144432b6cddf451b68",
                "name": "Dill Chips",
                "gtin": "8710553750803",
                "image": "https://static.euw1.sustained.app/products/images/8710553750803_0.jpg",
                "pack": "9700g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0597e1ccb7ea23144432b6cddf451b68",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0597e1ccb7ea23144432b6cddf451b68/impacts"
                }
            },
            {
                "id": "productid.05a2573b19742cf50ac11d4ad5016bb2",
                "name": "Chinese Style Chicken Curry",
                "gtin": "5054781876076",
                "image": "https://static.euw1.sustained.app/products/images/5054781876076_0.jpg",
                "pack": "392g",
                "grade": "F",
                "points": {
                    "pef": 65.60010952027791
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05a2573b19742cf50ac11d4ad5016bb2",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05a2573b19742cf50ac11d4ad5016bb2/impacts"
                }
            },
            {
                "id": "productid.05b9a7a644ec2617e00a2dde91b433df",
                "name": "Pineapple Slices in Juice",
                "gtin": "5010793381465",
                "image": "https://static.euw1.sustained.app/products/images/5010793381465_0.jpg",
                "pack": "227g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05b9a7a644ec2617e00a2dde91b433df",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05b9a7a644ec2617e00a2dde91b433df/impacts"
                }
            },
            {
                "id": "productid.05bec437b6fb9a55dbd1ea7e928c889f",
                "name": "Wild Pacific Red Salmon",
                "gtin": "8429583017694",
                "image": "https://static.euw1.sustained.app/products/images/8429583017694_0.jpg",
                "pack": "213g",
                "grade": "G",
                "points": {
                    "pef": 128.43915920999876
                },
                "info_icons": [],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05bec437b6fb9a55dbd1ea7e928c889f",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05bec437b6fb9a55dbd1ea7e928c889f/impacts"
                }
            },
            {
                "id": "productid.05c8464a48decdd1bc11ffcebc9f2f64",
                "name": "Pickled Onions",
                "gtin": "5060336500266",
                "image": "https://static.euw1.sustained.app/products/images/5060336500266_0.jpg",
                "pack": "400g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.05c8464a48decdd1bc11ffcebc9f2f64",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.05c8464a48decdd1bc11ffcebc9f2f64/impacts"
                }
            },
            {
                "id": "productid.062a53389c8c343506ddb4710dcc8ba0",
                "name": "Greek Kalamata Olives",
                "gtin": "01831009",
                "image": "https://static.euw1.sustained.app/products/images/01831009_0.jpg",
                "pack": "345g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.062a53389c8c343506ddb4710dcc8ba0",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.062a53389c8c343506ddb4710dcc8ba0/impacts"
                }
            },
            {
                "id": "productid.064cf3724ceb56ec6d1503d454b01471",
                "name": "Beet Kraut Juniper & Black Pepper",
                "gtin": "5060336200012",
                "image": "https://static.euw1.sustained.app/products/images/5060336200012_0.jpg",
                "pack": "410g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.064cf3724ceb56ec6d1503d454b01471",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.064cf3724ceb56ec6d1503d454b01471/impacts"
                }
            },
            {
                "id": "productid.06e780c37c60576fc108f126c4bbeff9",
                "name": "Organic Red Kidney Beans in Water",
                "gtin": "5011673104655",
                "image": "https://static.euw1.sustained.app/products/images/5011673104655_0.jpg",
                "pack": "400g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.06e780c37c60576fc108f126c4bbeff9",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.06e780c37c60576fc108f126c4bbeff9/impacts"
                }
            },
            {
                "id": "productid.07764a23b3e65d19b9f91c746887eccc",
                "name": "Diced Beetroot in Vinegar",
                "gtin": "00440561",
                "image": "https://static.euw1.sustained.app/products/images/00440561_0.jpg",
                "pack": "340g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.07764a23b3e65d19b9f91c746887eccc",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.07764a23b3e65d19b9f91c746887eccc/impacts"
                }
            },
            {
                "id": "productid.082970eb9d736c2e71ca0ccd0f3e523d",
                "name": "Stoneless Manzanilla Olives Filled with Anchovy",
                "gtin": "8410134105403",
                "image": "https://static.euw1.sustained.app/products/images/8410134105403_0.jpg",
                "pack": "200g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.082970eb9d736c2e71ca0ccd0f3e523d",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.082970eb9d736c2e71ca0ccd0f3e523d/impacts"
                }
            },
            {
                "id": "productid.083f9dffe92df00c2dec7f7424332b02",
                "name": "Intensely Rich Sun-Dried Tomatoes",
                "gtin": "5000169324684",
                "image": "https://static.euw1.sustained.app/products/images/5000169324684_0.jpg",
                "pack": "280g",
                "grade": "D",
                "points": {
                    "pef": 22.08682977477559
                },
                "info_icons": [
                    {
                        "name": "Widely Recyclable",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/widely-recyclable.svg",
                        "description": "This product\u2019s packaging should be recyclable in your nearest recycling bins. Choose the right one!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.083f9dffe92df00c2dec7f7424332b02",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.083f9dffe92df00c2dec7f7424332b02/impacts"
                }
            },
            {
                "id": "productid.087c5439491e4e7b24bfadfc571ee9e6",
                "name": "Tuna Chunks in Spring Water",
                "gtin": "5000232027047",
                "image": "https://static.euw1.sustained.app/products/images/5000232027047_0.jpg",
                "pack": "145g",
                "grade": "G",
                "points": {
                    "pef": 123.94354272017374
                },
                "info_icons": [],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.087c5439491e4e7b24bfadfc571ee9e6",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.087c5439491e4e7b24bfadfc571ee9e6/impacts"
                }
            },
            {
                "id": "productid.09344395e2f76741d3e7a17a63ba500e",
                "name": "Hot Beef Curry",
                "gtin": "0000000809894",
                "image": "https://static.euw1.sustained.app/products/images/0000000809894_0.jpg",
                "pack": "400g",
                "grade": "G",
                "points": {
                    "pef": 173.9788591708001
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09344395e2f76741d3e7a17a63ba500e",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09344395e2f76741d3e7a17a63ba500e/impacts"
                }
            },
            {
                "id": "productid.096c50a5dbd0eb9f62806bb4c6f9a2ed",
                "name": "Wild Pacific Pink Salmon",
                "gtin": "5000171001450",
                "image": "https://static.euw1.sustained.app/products/images/5000171001450_0.jpg",
                "pack": "418g",
                "grade": "G",
                "points": {
                    "pef": 128.43915920999876
                },
                "info_icons": [],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.096c50a5dbd0eb9f62806bb4c6f9a2ed",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.096c50a5dbd0eb9f62806bb4c6f9a2ed/impacts"
                }
            },
            {
                "id": "productid.09850dd65b20bab7cdf67f1dbf64c1a5",
                "name": "Food Rhubarb Chunks in Light Syrup",
                "gtin": "0000029061907",
                "image": "https://static.euw1.sustained.app/products/images/0000029061907_0.jpg",
                "pack": "560g",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Local product",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/local.svg",
                        "description": "Made in your country, so less travel, energy use and pollution! Generally a good thing."
                    },
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09850dd65b20bab7cdf67f1dbf64c1a5",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09850dd65b20bab7cdf67f1dbf64c1a5/impacts"
                }
            },
            {
                "id": "productid.099380a323c8e346a6853baf5673b9fb",
                "name": "Spanish Olives Stuffed with Pimiento",
                "gtin": "8410134012879",
                "image": "https://static.euw1.sustained.app/products/images/8410134012879_0.jpg",
                "pack": "907g",
                "grade": "C",
                "points": {
                    "pef": 19.167445027793775
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.099380a323c8e346a6853baf5673b9fb",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.099380a323c8e346a6853baf5673b9fb/impacts"
                }
            },
            {
                "id": "productid.09c0558968d02e129a74dc36e481b690",
                "name": "Antipasti Olives",
                "gtin": "5060312395589",
                "image": "https://static.euw1.sustained.app/products/images/5060312395589_0.jpg",
                "pack": "",
                "grade": "B",
                "points": {
                    "pef": 13.787332805957924
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.09c0558968d02e129a74dc36e481b690",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.09c0558968d02e129a74dc36e481b690/impacts"
                }
            },
            {
                "id": "productid.0a4dfc1e08808f378243a451b0a36d5e",
                "name": "Mini Gherkins Cornichons in Vinegar",
                "gtin": "5025820003180",
                "image": "https://static.euw1.sustained.app/products/images/5025820003180_0.jpg",
                "pack": "480g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a4dfc1e08808f378243a451b0a36d5e",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a4dfc1e08808f378243a451b0a36d5e/impacts"
                }
            },
            {
                "id": "productid.0a647a3e8c88b656b4aa49ceaf714ad1",
                "name": "Light Coconut Milk",
                "gtin": "5010338013363",
                "image": "https://static.euw1.sustained.app/products/images/5010338013363_0.jpg",
                "pack": "400ml",
                "grade": "B",
                "points": {
                    "pef": 13.711312513763914
                },
                "info_icons": [
                    {
                        "name": "Low CO\u2082 Emissions",
                        "svg_icon": "https://static.euw1.sustained.app/client/v1/key-info-badges/low-carbon-footprint.svg",
                        "description": "The CO\u2082 emissions emitted when making this product are less than most, which is good!"
                    }
                ],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a647a3e8c88b656b4aa49ceaf714ad1",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a647a3e8c88b656b4aa49ceaf714ad1/impacts"
                }
            },
            {
                "id": "productid.0a95db711d2f0592482e833b4f085e91",
                "name": "Pickled Cornichons",
                "gtin": "8699073290610",
                "image": "https://static.euw1.sustained.app/products/images/8699073290610_0.jpg",
                "pack": "680g",
                "grade": "E",
                "points": {
                    "pef": 39.33508642431711
                },
                "info_icons": [],
                "category": "Tins, Pickles & Chutney",
                "links": {
                    "product": "https://api.sustained.com/choice/v1/products/productid.0a95db711d2f0592482e833b4f085e91",
                    "category": "https://api.sustained.com/choice/v1/categories/tins-comma-pickles-and-chutney",
                    "impacts": "https://api.sustained.com/choice/v1/products/productid.0a95db711d2f0592482e833b4f085e91/impacts"
                }
            }
        ]
    ]
}

In [17]:
import json

In [28]:
sustained_categories = [k["name"] for k in sustained_categories_json["categories"]]
display(sustained_categories[:4])

['Baby Foods & Drinks', 'Bacon & Sausages', 'Beef', 'Beer & Cider']

In [23]:
with open("/Users/joey/Github_Keep/vegi-esc/vegi-esc-api/backup_localstorage/sustained-cooking-sauces.json") as f:
    sustained_example_products=json.load(f)

In [25]:
sustained_example_product_names = [p["name"] for p in sustained_example_products]
display(sustained_example_product_names[:4])

['Mealmaker Lamb Casserole',
 'Food Made Without Wheat Gravy',
 'Mealmaker Pork Casserole',
 'Tesco Breadcrumbs']

In [26]:
dim_red="TSNE"
dimension="2D"
top_n=10
annotation="On"
user_input="Raw kimchi hot sauce"
vegi_category="Condiment section"
view_to_similar_words: list[str] = []
perplexity = 0
learning_rate = 0
iteration = 0

In [13]:
if dim_red == "TSNE":
    perplexity = max(min(5, top_n), 0)
    # perplexity = st.sidebar.slider(
    #     "Adjust the perplexity. The perplexity is related to the number of nearest neighbors that is used in other manifold learning algorithms. Larger datasets usually require a larger perplexity",
    #     5,
    #     50,
    #     (30),
    # )

    learning_rate = 200
    # learning_rate = st.sidebar.slider("Adjust the learning rate", 10, 1000, (200))

    iteration = 1000
    # iteration = st.sidebar.slider("Adjust the number of iteration", 250, 100000, (1000))
    

In [32]:
user_inputs = [x.strip() for x in user_input.split(",")]
result_word = []

In [ ]:
def most_similar_sentences(sentence:str, topn:int=10):
    '''
    Gets a list of product names (sentences) from sustained that are most similar to input sentence.
    '''
    pass

In [15]:

for sentence in user_input:
    if not view_to_similar_words:
        sim_words = model.most_similar(sentence, topn=top_n) # todo: If words have spaces (i.e. sentences -> use wmdistance)
        # result = model.wmdistance(
        #     sentence, 
        # )
        sim_words = append_list(sim_words, sentence)
    else:
        sim_words = []
        vocab = model.key_to_index.keys()
        if all((phrase in vocab for phrase in view_to_similar_words)):
            for measure_similarity_to_word in view_to_similar_words:
                _similarity = model.similarity(sentence, measure_similarity_to_word)
                sim_words.append((measure_similarity_to_word, _similarity, sentence))
            sim_words.sort(key=lambda t: t[1])
        else:
            for measure_similarity_to_word in view_to_similar_words:
                _similarity = model.wmdistance(sentence, measure_similarity_to_word)
                # sim_words.append((measure_similarity_to_word, _similarity, words))
                sim_words.append((measure_similarity_to_word, 1.0 / _similarity, sentence))
            # sim_words.sort(key=lambda t: t[1], reverse=True)
            sim_words.sort(key=lambda t: t[1])
        sim_words = sim_words[:top_n]
    result_word.extend(sim_words)
display(result_word)

[('pastas', 0.7083908319473267, 'pasta'),
 ('tomato_sauce', 0.7069307565689087, 'pasta'),
 ('polenta', 0.7017028331756592, 'pasta'),
 ('tortellini', 0.701042652130127, 'pasta'),
 ('ravioli', 0.6902726888656616, 'pasta'),
 ('gnocchi', 0.68798828125, 'pasta'),
 ('fettuccine', 0.6732334494590759, 'pasta'),
 ('cavatelli', 0.6724798083305359, 'pasta'),
 ('risotto', 0.6708415150642395, 'pasta'),
 ('fusilli', 0.6701416373252869, 'pasta'),
 ('tomatoes', 0.8442263007164001, 'tomato'),
 ('lettuce', 0.7069937586784363, 'tomato'),
 ('asparagus', 0.7050933837890625, 'tomato'),
 ('peaches', 0.6938520073890686, 'tomato'),
 ('cherry_tomatoes', 0.6897530555725098, 'tomato'),
 ('strawberry', 0.6888598799705505, 'tomato'),
 ('strawberries', 0.6832595467567444, 'tomato'),
 ('bell_peppers', 0.6813562512397766, 'tomato'),
 ('potato', 0.6784172058105469, 'tomato'),
 ('cantaloupe', 0.6780219078063965, 'tomato')]

'Raw kimchi hot sauce'

In [36]:
sustained_comparitors = [
    (vegi_category, "vegi_category"),
    *[(sc, "sustained_category") for sc in sustained_categories],
    *[(sp, "sustained_product") for sp in sustained_example_product_names],
]
result_word = [
    (sent, sent_type, model.wmdistance(sentence, sent), sentence) for sentence in user_inputs for (sent, sent_type) in sustained_comparitors
]
display(result_word)

[('Condiment section',
  'vegi_category',
  0.5145042127476691,
  'Raw kimchi hot sauce'),
 ('Baby Foods & Drinks',
  'sustained_category',
  0.7428487534610955,
  'Raw kimchi hot sauce'),
 ('Bacon & Sausages',
  'sustained_category',
  0.6937898660008991,
  'Raw kimchi hot sauce'),
 ('Beef', 'sustained_category', 1.0287058240955504, 'Raw kimchi hot sauce'),
 ('Beer & Cider',
  'sustained_category',
  0.9148966426226118,
  'Raw kimchi hot sauce'),
 ('Biscuits', 'sustained_category', 0.594511005453057, 'Raw kimchi hot sauce'),
 ('Bread', 'sustained_category', 1.0005694612012068, 'Raw kimchi hot sauce'),
 ('Breakfast Foods',
  'sustained_category',
  0.687539107741492,
  'Raw kimchi hot sauce'),
 ('Butter & Margarine',
  'sustained_category',
  0.7656186664590532,
  'Raw kimchi hot sauce'),
 ('Cakes', 'sustained_category', 0.9269577144555489, 'Raw kimchi hot sauce'),
 ('Carbonated Drinks',
  'sustained_category',
  0.5756362810406008,
  'Raw kimchi hot sauce'),
 ('Cereals', 'sustained_ca

In [45]:

similar_word = [word[0] for word in result_word]
# similarity = [word[1] for word in result_word]
similar_word.extend(user_inputs) # * adds the search terms to the array of word vectors to display on the plot
similar_word_vec_embeds = get_word_vectors(words=similar_word, sample=10)
# labels = [word[3] for word in result_word]
labels = [
    word[1] for word in result_word
]
label_dict = dict([(y, i + 1) for i, y in enumerate(set(labels))])
color_map = [label_dict[x] for x in labels]

EOFError: Ran out of input

In [38]:
# Initialize the app
if dimension == "2D":
    plot = display_scatterplot_2D(
        model,
        user_inputs,
        similar_word,
        labels,
        color_map,
        annotation,
        dim_red,
        perplexity,
        learning_rate,
        iteration,
        top_n,
    )
else:
    plot = display_scatterplot_3D(
        model,
        user_inputs,
        similar_word,
        labels,
        color_map,
        annotation,
        dim_red,
        perplexity,
        learning_rate,
        iteration,
        top_n,
    )

UnsupportedOperation: read